In [ ]:
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium import webdriver
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from bs4 import BeautifulSoup
import time

def checkgeneexists(cookiezi, driver):
    
    #driver = webdriver.Firefox()
                                 #if not working (not in env variables), paste in executable path to geckodriver.exe file
                                 #executable_path=r'C:\Users\Kevin\geckodriver-v0.29.1-win64\geckodriver.exe')
        
    replace = "https://www.genecards.org/cgi-bin/carddisp.pl?gene=" + cookiezi
    driver.get(replace)
    time.sleep(2)
    
    dog = driver.find_elements_by_id("aliases_descriptions")
    
    cat = 0
    vaxei = "Target found"

    if not dog:
        driver.get("https://www.genecards.org/Search/Keyword?queryString=" + cookiezi)
        time.sleep(5)
        check = driver.find_elements_by_class_name("gc-gene-symbol")
        if check:
            replace = "https://www.genecards.org/cgi-bin/carddisp.pl?gene=" + driver.find_element_by_class_name("gc-gene-symbol").text
            vaxei = driver.find_element_by_class_name("gc-gene-symbol").text
            driver.get(replace)
            cat = 1
        else:
            vaxei = "Target does not exist, nor does it have related targets! Try again"
            cat = 2

    
    return replace, driver, cat, vaxei

##########################################################################################################


def genecards(osugame, woof): #osugame input will be a gene name
    #Test links:
    
    #driver = webdriver.Firefox(executable_path=r'C:\Users\Kevin\geckodriver-v0.29.1-win64\geckodriver.exe')
        
    source =  "https://www.genecards.org/cgi-bin/carddisp.pl?gene=" + osugame   

    source, driver, alternatesearch, newtarget = checkgeneexists(osugame, woof)
    
    n = 0
    
    if alternatesearch != 2:
        table = driver.find_element_by_id("aliases_descriptions")
        aliasnames = table.find_elements_by_tag_name("li")
        title = driver.find_element_by_tag_name("strong").text

        ensembl = ""
        mrekk = []
        check = 0
        
        
        for cat in aliasnames:
            cow = 0
            numbers = ""
            sups = cat.find_elements_by_tag_name("sup")
            if not sups:
                sups = cat.find_elements_by_class_name("gc-ga-link")
                cow = 1
            #print(cow)
            
        
            if cow == 0:
                for fish in sups:
                    numbers = numbers + " " + fish.text
            elif cow == 1:
                if ":" not in cat.text:
                    for fish in sups:
                        numbers = numbers + ", " + fish.text
                numbers = numbers[2:]
                numbers = " (" + numbers + ")"
            #print(cat.text)
            #print(numbers)
            if numbers!= "" and numbers!=" ()":
                #print(cat.text.split(numbers)[0])
                mrekk.append(str(cat.text[:-len(numbers)]))


            if "Ensembl" in cat.text: 
                ensembl = cat.text
                check = 1
            elif check == 0:
                ensembl = "ENSEMBL NOT FOUND"

        #print()
        ensembl = ensembl.split()[1]

        #driver.close()

        return ensembl,mrekk,title, driver, source, alternatesearch, newtarget
    
    else:
        
        return "","","", driver, "", alternatesearch, newtarget
    

############################################################################################
 

def getopentargets(genename, drivername): #genename will be a gene name

    ensemblname, aliases,name, nd, genecardslink, searchno, target2 = genecards(genename, drivername)
    link = ""
    woof = 0
    if searchno !=2:
        conditions = []
        if ensemblname == "NOT":
            nd.get("https://platform.opentargets.org/search?q=" + name + "&page=1")
            time.sleep(2)
            link = nd.find_element_by_class_name("jss28").get_attribute("href")
            woof = 1
        #print(link)
        
        driver = nd #webdriver.Firefox() #executable_path=r'C:\Users\Kevin\geckodriver-v0.29.1-win64\geckodriver.exe' #make sure this exists somewhere in a local, varies from user to user, and copy the path here
        
        if woof == 0:
            opentargetslink = "https://platform.opentargets.org/target/" + ensemblname + "/associations"
        else:
            opentargetslink = link
            
        if "ENSG00000105810" in link and genename!="CDK6":
                return [], aliases, name, genecardslink, "OpenTargets gene not found!", target2, nd
        
        driver.get(opentargetslink)
        time.sleep(4)

        results = driver.find_elements_by_class_name("MuiTableRow-root")
        if not results:
            time.sleep(2)
        elif  "Name" in results[0].text:
            results.pop(0)

        for result in results:
            product_name = result.find_element_by_tag_name('span')
            #print("                        " + product_name.text)
            conditions.append(product_name.text)

        search = ""
        if searchno == 1:
            search = "Original target not found, closest target found: " + target2
        elif searchno == 0:
            search = target2
        
        
        if len(conditions)>10:
            conditions = conditions[:10]
        

        return conditions, aliases, name, genecardslink, opentargetslink, search, nd
    
    else:
        
        #nd.close()
        return [], [], str(genename) + "?", "DNE", "DNE", target2, nd

In [ ]:
doggie = ['GINM1', 'GINS1', 'GINS2', 'GINS3', 'GINS4', 'GIP', 'GIPC1', 'GIPC2', 'GIPC3', 'GIPR', 'GIT1', 'GIT2', 'GJA1', 'GJA3', 'GJA4', 'GJA5', 'GJA8', 'GJA9', 'GJA10', 'GJB1', 'GJB2', 'GJB3', 'GJB4', 'GJB5', 'GJB6', 'GJB7', 'GJC1', 'GJC2', 'GJC3', 'GJD2', 'GJD3', 'GJD4', 'GJE1', 'GK', 'GK2', 'GK5', 'GKAP1', 'GKN1', 'GKN2', 'GLA', 'GLB1', 'GLB1L', 'GLB1L2', 'GLB1L3', 'GLCCI1', 'GLCE', 'GLDC', 'GLDN', 'GLE1', 'GLG1', 'GLI1', 'GLI2', 'GLI3', 'GLI4', 'GLIPR1', 'GLIPR1L1', 'GLIPR1L2', 'GLIPR2', 'GLIS1', 'GLIS2', 'GLIS3', 'GLMN', 'GLMP', 'GLO1', 'GLOD4', 'GLOD5', 'GLP1R', 'GLP2R', 'GLRA1', 'GLRA2', 'GLRA3', 'GLRB', 'GLRX', 'GLRX2', 'GLRX3', 'GLRX5', 'GLS', 'GLS2', 'GLT1D1', 'GLT6D1', 'GLT8D1', 'GLT8D2', 'GLTP', 'GLTPD2', 'GLUD1', 'GLUD2', 'GLUL', 'GLYAT', 'GLYATL1', 'GLYATL1B', 'GLYATL2', 'GLYATL3', 'GLYCTK', 'GLYR1', 'GM2A', 'GMCL1', 'GMCL2', 'GMDS', 'GMEB1', 'GMEB2', 'GMFB', 'GMFG', 'GMIP', 'GML', 'GMNC', 'GMNN', 'GMPPA', 'GMPPB', 'GMPR', 'GMPR2', 'GMPS', 'GNA11', 'GNA12', 'GNA13', 'GNA14', 'GNA15', 'GNAI1', 'GNAI2', 'GNAI3', 'GNAL', 'GNAO1', 'GNAQ', 'GNAS', 'GNAT1', 'GNAT2', 'GNAT3', 'GNAZ', 'GNB1', 'GNB1L', 'GNB2', 'GNB3', 'GNB4', 'GNB5', 'GNE', 'GNG2', 'GNG3', 'GNG4', 'GNG5', 'GNG7', 'GNG8', 'GNG10', 'GNG11', 'GNG12', 'GNG13', 'GNG14', 'GNGT1', 'GNGT2', 'GNL1', 'GNL2', 'GNL3', 'GNL3L', 'GNLY', 'GNMT', 'GNPAT', 'GNPDA1', 'GNPDA2', 'GNPNAT1', 'GNPTAB', 'GNPTG', 'GNRH1', 'GNRH2', 'GNRHR', 'GNS', 'GOLGA1', 'GOLGA2', 'GOLGA3', 'GOLGA4', 'GOLGA5', 'GOLGA6A', 'GOLGA6B', 'GOLGA6C', 'GOLGA6D', 'GOLGA6L1', 'GOLGA6L2', 'GOLGA6L4', 'GOLGA6L6', 'GOLGA6L7', 'GOLGA6L9', 'GOLGA6L10', 'GOLGA6L22', 'GOLGA7', 'GOLGA7B', 'GOLGA8A', 'GOLGA8B', 'GOLGA8F', 'GOLGA8G', 'GOLGA8H', 'GOLGA8J', 'GOLGA8K', 'GOLGA8M', 'GOLGA8N', 'GOLGA8O', 'GOLGA8Q', 'GOLGA8R', 'GOLGA8S', 'GOLGA8T', 'GOLGB1', 'GOLIM4', 'GOLM1', 'GOLM2', 'GOLPH3', 'GOLPH3L', 'GOLT1A', 'GOLT1B', 'GON4L', 'GON7', 'GOPC', 'GORAB', 'GORASP1', 'GORASP2', 'GOSR1', 'GOSR2', 'GOT1', 'GOT1L1', 'GOT2', 'GP1BA', 'GP1BB', 'GP2', 'GP5', 'GP6', 'GP9', 'GPA33', 'GPAA1', 'GPALPP1', 'GPAM', 'GPANK1', 'GPAT2', 'GPAT3', 'GPAT4', 'GPATCH1', 'GPATCH2', 'GPATCH2L', 'GPATCH3', 'GPATCH4', 'GPATCH8', 'GPATCH11', 'GPBAR1', 'GPBP1', 'GPBP1L1', 'GPC1', 'GPC2', 'GPC3', 'GPC4', 'GPC5', 'GPC6', 'GPCPD1', 'GPD1', 'GPD1L', 'GPD2', 'GPER1', 'GPHA2', 'GPHB5', 'GPHN', 'GPI', 'GPIHBP1', 'GPKOW', 'GPLD1', 'GPM6A', 'GPM6B', 'GPN1', 'GPN2', 'GPN3', 'GPNMB', 'GPR3', 'GPR4', 'GPR6', 'GPR12', 'GPR15', 'GPR17', 'GPR18', 'GPR19', 'GPR20', 'GPR21', 'GPR22', 'GPR25', 'GPR26', 'GPR27', 'GPR31', 'GPR32', 'GPR33', 'GPR34', 'GPR35', 'GPR37', 'GPR37L1', 'GPR39', 'GPR42', 'GPR45', 'GPR50', 'GPR52', 'GPR55', 'GPR61', 'GPR62', 'GPR63', 'GPR65', 'GPR68', 'GPR75', 'GPR78', 'GPR82', 'GPR83', 'GPR84', 'GPR85', 'GPR87', 'GPR88', 'GPR89A', 'GPR89B', 'GPR101', 'GPR107', 'GPR108', 'GPR119', 'GPR132', 'GPR135', 'GPR137', 'GPR137B', 'GPR137C', 'GPR139', 'GPR141', 'GPR142', 'GPR143', 'GPR146', 'GPR148', 'GPR149', 'GPR150', 'GPR151', 'GPR152', 'GPR153', 'GPR155', 'GPR156', 'GPR157', 'GPR158', 'GPR160', 'GPR161', 'GPR162', 'GPR171', 'GPR173', 'GPR174', 'GPR176', 'GPR179', 'GPR180', 'GPR182', 'GPR183', 'GPRASP1', 'GPRASP2', 'GPRC5A', 'GPRC5B', 'GPRC5C', 'GPRC5D', 'GPRC6A', 'GPRIN1', 'GPRIN2', 'GPRIN3', 'GPS1', 'GPS2', 'GPSM1', 'GPSM2', 'GPSM3', 'GPT', 'GPT2', 'GPX1', 'GPX2', 'GPX3', 'GPX4', 'GPX5', 'GPX6', 'GPX7', 'GPX8', 'GRAMD1A', 'GRAMD1B', 'GRAMD1C', 'GRAMD2A', 'GRAMD2B', 'GRAMD4', 'GRAP', 'GRAP2', 'GRAPL', 'GRB2', 'GRB7', 'GRB10', 'GRB14', 'GREB1', 'GREB1L', 'GREM1', 'GREM2', 'GREP1', 'GRHL1', 'GRHL2', 'GRHL3', 'GRHPR', 'GRIA1', 'GRIA2', 'GRIA3', 'GRIA4', 'GRID1', 'GRID2', 'GRID2IP', 'GRIFIN', 'GRIK1', 'GRIK2', 'GRIK3', 'GRIK4', 'GRIK5', 'GRIN1', 'GRIN2A', 'GRIN2B', 'GRIN2C', 'GRIN2D', 'GRIN3A', 'GRIN3B', 'GRINA', 'GRIP1', 'GRIP2', 'GRIPAP1', 'GRK1', 'GRK2', 'GRK3', 'GRK4', 'GRK5', 'GRK6', 'GRK7', 'GRM1', 'GRM2', 'GRM3', 'GRM4', 'GRM5', 'GRM6', 'GRM7', 'GRM8', 'GRN', 'GRP', 'GRPEL1', 'GRPEL2', 'GRPR', 'GRSF1', 'GRTP1', 'GRWD1', 'GRXCR1', 'GRXCR2', 'G0S2', 'GSAP', 'GSC', 'GSC2', 'GSDMA', 'GSDMB', 'GSDMC', 'GSDMD', 'GSDME', 'GSE1', 'GSG1', 'GSG1L', 'GSG1L2', 'GSK3A', 'GSK3B', 'GSKIP', 'GSN', 'GSPT1', 'GSPT2', 'GSR', 'GSS', 'GSTA1', 'GSTA2', 'GSTA3', 'GSTA4', 'GSTA5', 'GSTCD', 'GSTK1', 'GSTM1', 'GSTM2', 'GSTM3', 'GSTM4', 'GSTM5', 'GSTO1', 'GSTO2', 'GSTP1', 'GSTT2', 'GSTT2B', 'GSTT4', 'GSTZ1', 'GSX1', 'GSX2', 'GTDC1', 'GTF2A1', 'GTF2A1L', 'GTF2A2', 'GTF2B', 'GTF2E1', 'GTF2E2', 'GTF2F1', 'GTF2F2', 'GTF2H1', 'GTF2H2', 'GTF2H2C', 'GTF2H3', 'GTF2H4', 'GTF2H5', 'GTF2I', 'GTF2IRD1', 'GTF2IRD2', 'GTF2IRD2B', 'GTF3A', 'GTF3C1', 'GTF3C2', 'GTF3C3', 'GTF3C4', 'GTF3C5', 'GTF3C6', 'GTPBP1', 'GTPBP2', 'GTPBP3', 'GTPBP4', 'GTPBP6', 'GTPBP8', 'GTPBP10', 'GTSE1', 'GTSF1', 'GTSF1L', 'GUCA1A', 'GUCA1ANB', 'GUCA1B', 'GUCA1C', 'GUCA2A', 'GUCA2B', 'GUCD1', 'GUCY1A1', 'GUCY1A2', 'GUCY1B1', 'GUCY2C', 'GUCY2D', 'GUCY2F', 'GUF1', 'GUK1', 'GULP1', 'GUSB', 'GVQW3', 'GXYLT1', 'GXYLT2', 'GYG1', 'GYG2', 'GYPA', 'GYPB', 'GYPC', 'GYPE', 'GYS1', 'GYS2', 'GZF1', 'GZMA', 'GZMB', 'GZMH', 'GZMK', 'GZMM', 'H1-0', 'H1-1', 'H1-2', 'H1-3', 'H1-4', 'H1-5', 'H1-6', 'H1-7', 'H1-8', 'H1-10', 'H2AB1', 'H2AB2', 'H2AB3', 'H2AC1', 'H2AC4', 'H2AC6', 'H2AC7', 'H2AC8', 'H2AC11', 'H2AC12', 'H2AC13', 'H2AC14', 'H2AC15', 'H2AC16', 'H2AC17', 'H2AC18', 'H2AC19', 'H2AC20', 'H2AC21', 'H2AJ', 'H2AP', 'H2AW', 'H2AX', 'H2AZ1', 'H2AZ2', 'H2BC1', 'H2BC3', 'H2BC4', 'H2BC5', 'H2BC6', 'H2BC7', 'H2BC8', 'H2BC9', 'H2BC10', 'H2BC11', 'H2BC12', 'H2BC13', 'H2BC14', 'H2BC15', 'H2BC17', 'H2BC18', 'H2BC21', 'H2BE1', 'H2BS1', 'H2BU1', 'H2BW1', 'H2BW2', 'H3-2', 'H3-3A', 'H3-3B', 'H3-4', 'H3-5', 'H3C1', 'H3C2', 'H3C3', 'H3C4', 'H3C6', 'H3C7', 'H3C8', 'H3C10', 'H3C11', 'H3C12', 'H3C13', 'H3C14', 'H3C15', 'H3Y1', 'H3Y2', 'H4-16', 'H4C1', 'H4C2', 'H4C3', 'H4C4', 'H4C5', 'H4C6', 'H4C7', 'H4C8', 'H4C9', 'H4C11', 'H4C12', 'H4C13', 'H4C14', 'H4C15', 'H6PD', 'HAAO', 'HABP2', 'HABP4', 'HACD1', 'HACD2', 'HACD3', 'HACD4', 'HACE1', 'HACL1', 'HADH', 'HADHA', 'HADHB', 'HAGH', 'HAGHL', 'HAL', 'HAMP', 'HAND1', 'HAND2', 'HAO1', 'HAO2', 'HAP1', 'HAPLN1', 'HAPLN2', 'HAPLN3', 'HAPLN4', 'HARBI1', 'HARS1', 'HARS2', 'HAS1', 'HAS2', 'HAS3', 'HASPIN', 'HAT1', 'HAUS1', 'HAUS2', 'HAUS3', 'HAUS4', 'HAUS5', 'HAUS6', 'HAUS7', 'HAUS8', 'HAVCR1', 'HAVCR2', 'HAX1', 'HBA1', 'HBA2', 'HBB', 'HBD', 'HBE1', 'HBEGF', 'HBG1', 'HBG2', 'HBM', 'HBP1', 'HBQ1', 'HBS1L', 'HBZ', 'HCAR1', 'HCAR2', 'HCAR3', 'HCCS', 'HCFC1', 'HCFC1R1', 'HCFC2', 'HCG22', 'HCK', 'HCLS1', 'HCN1', 'HCN2', 'HCN3', 'HCN4', 'HCRT', 'HCRTR1', 'HCRTR2', 'HCST', 'HDAC1', 'HDAC2', 'HDAC3', 'HDAC4', 'HDAC5', 'HDAC6', 'HDAC7', 'HDAC8', 'HDAC9', 'HDAC10', 'HDAC11', 'HDC', 'HDDC2', 'HDDC3', 'HDGF', 'HDGFL1', 'HDGFL2', 'HDGFL3', 'HDHD2', 'HDHD3', 'HDHD5', 'HDLBP', 'HDX', 'HEATR1', 'HEATR3', 'HEATR4', 'HEATR5A', 'HEATR5B', 'HEATR6', 'HEATR9', 'HEBP1', 'HEBP2', 'HECA', 'HECTD1', 'HECTD2', 'HECTD3', 'HECTD4', 'HECW1', 'HECW2', 'HEG1', 'HELB', 'HELLS', 'HELQ', 'HELT', 'HELZ', 'HELZ2', 'HEMGN', 'HEMK1', 'HENMT1', 'HEPACAM', 'HEPACAM2', 'HEPH', 'HEPHL1', 'HEPN1', 'HERC1', 'HERC2', 'HERC3', 'HERC4', 'HERC5', 'HERC6', 'HERPUD1', 'HERPUD2', 'HES1', 'HES2', 'HES3', 'HES4', 'HES5', 'HES6', 'HES7', 'HESX1', 'HEXA', 'HEXB', 'HEXD', 'HEXIM1', 'HEXIM2', 'HEY1', 'HEY2', 'HEYL', 'HFE', 'HFM1', 'HGD', 'HGF', 'HGFAC', 'HGH1', 'HGS', 'HGSNAT', 'HHAT', 'HHATL', 'HHEX', 'HHIP', 'HHIPL1', 'HHIPL2', 'HHLA1', 'HHLA2', 'HIBADH', 'HIBCH', 'HIC1', 'HIC2', 'HID1', 'HIF1A', 'HIF1AN', 'HIF3A', 'HIGD1A', 'HIGD1B', 'HIGD1C', 'HIGD2A', 'HIGD2B', 'HIKESHI', 'HILPDA', 'HINFP', 'HINT1', 'HINT2', 'HINT3', 'HIP1', 'HIP1R', 'HIPK1', 'HIPK2', 'HIPK3', 'HIPK4', 'HIRA', 'HIRIP3', 'HIVEP1', 'HIVEP2', 'HIVEP3', 'HJURP', 'HJV', 'HK1', 'HK2', 'HK3', 'HKDC1', 'HLA-A', 'HLA-B', 'HLA-C', 'HLA-DMA', 'HLA-DMB', 'HLA-DOA', 'HLA-DOB', 'HLA-DPA1', 'HLA-DPB1', 'HLA-DQA1', 'HLA-DQA2', 'HLA-DQB1', 'HLA-DQB2', 'HLA-DRA', 'HLA-DRB1', 'HLA-DRB5', 'HLA-E', 'HLA-F', 'HLA-G', 'HLCS', 'HLF', 'HLTF', 'HLX', 'HM13', 'HMBOX1', 'HMBS', 'HMCES', 'HMCN1', 'HMCN2', 'HMG20A', 'HMG20B', 'HMGA1', 'HMGA2', 'HMGB1', 'HMGB2', 'HMGB3', 'HMGB4', 'HMGCL', 'HMGCLL1', 'HMGCR', 'HMGCS1', 'HMGCS2', 'HMGN1', 'HMGN2', 'HMGN3', 'HMGN4', 'HMGN5', 'HMGXB3', 'HMGXB4', 'HMHB1', 'HMMR', 'HMOX1', 'HMOX2', 'HMSD', 'HMX1', 'HMX2', 'HMX3', 'HNF1A', 'HNF1B', 'HNF4A', 'HNF4G', 'HNMT', 'HNRNPA0', 'HNRNPA1', 'HNRNPA1L2', 'HNRNPA2B1', 'HNRNPA3', 'HNRNPAB', 'HNRNPC', 'HNRNPCL1', 'HNRNPCL2', 'HNRNPCL3', 'HNRNPCL4', 'HNRNPD', 'HNRNPDL', 'HNRNPF', 'HNRNPH1', 'HNRNPH2', 'HNRNPH3', 'HNRNPK', 'HNRNPL', 'HNRNPLL', 'HNRNPM', 'HNRNPR', 'HNRNPU', 'HNRNPUL1', 'HNRNPUL2', 'HOATZ', 'HOGA1', 'HOMER1', 'HOMER2', 'HOMER3', 'HOMEZ', 'HOOK1', 'HOOK2', 'HOOK3', 'HOPX', 'HORMAD1', 'HORMAD2', 'HOXA1', 'HOXA2', 'HOXA3', 'HOXA4', 'HOXA5', 'HOXA6', 'HOXA7', 'HOXA9', 'HOXA10', 'HOXA11', 'HOXA13', 'HOXB1', 'HOXB2', 'HOXB3', 'HOXB4', 'HOXB5', 'HOXB6', 'HOXB7', 'HOXB8', 'HOXB9', 'HOXB13', 'HOXC4', 'HOXC5', 'HOXC6', 'HOXC8', 'HOXC9', 'HOXC10', 'HOXC11', 'HOXC12', 'HOXC13', 'HOXD1', 'HOXD3', 'HOXD4', 'HOXD8', 'HOXD9', 'HOXD10', 'HOXD11', 'HOXD12', 'HOXD13', 'HP', 'HP1BP3', 'HPCA', 'HPCAL1', 'HPCAL4', 'HPD', 'HPDL', 'HPF1', 'HPGD', 'HPGDS', 'HPN', 'HPR', 'HPRT1', 'HPS1', 'HPS3', 'HPS4', 'HPS5', 'HPS6', 'HPSE', 'HPSE2', 'HPX', 'HR', 'HRAS', 'HRC', 'HRCT1', 'HRG', 'HRH1', 'HRH2', 'HRH3', 'HRH4', 'HRK', 'HRNR', 'HROB', 'HRURF', 'HS1BP3', 'HS2ST1', 'HS3ST1', 'HS3ST2', 'HS3ST3A1', 'HS3ST3B1', 'HS3ST4', 'HS3ST5', 'HS3ST6', 'HS6ST1', 'HS6ST2', 'HS6ST3', 'HSBP1', 'HSBP1L1', 'HSCB', 'HSD3B1', 'HSD3B2', 'HSD3B7', 'HSD11B1', 'HSD11B1L', 'HSD11B2', 'HSD17B1', 'HSD17B2', 'HSD17B3', 'HSD17B4', 'HSD17B6', 'HSD17B7', 'HSD17B8', 'HSD17B10', 'HSD17B11', 'HSD17B12', 'HSD17B13', 'HSD17B14', 'HSDL1', 'HSDL2', 'HSF1', 'HSF2', 'HSF2BP', 'HSF4', 'HSF5', 'HSFX1', 'HSFX2', 'HSFX3', 'HSFX4', 'HSFY1', 'HSFY2', 'HSH2D', 'HSP90AA1', 'HSP90AB1', 'HSP90B1', 'HSPA1A', 'HSPA1B', 'HSPA1L', 'HSPA2', 'HSPA4', 'HSPA4L', 'HSPA5', 'HSPA6', 'HSPA8', 'HSPA9', 'HSPA12A', 'HSPA12B', 'HSPA13', 'HSPA14', 'HSPB1', 'HSPB2', 'HSPB3', 'HSPB6', 'HSPB7', 'HSPB8', 'HSPB9', 'HSPB11', 'HSPBAP1', 'HSPBP1', 'HSPD1', 'HSPE1', 'HSPG2', 'HSPH1', 'HTATIP2', 'HTATSF1', 'HTD2', 'HTN1', 'HTN3', 'HTR1A', 'HTR1B', 'HTR1D', 'HTR1E', 'HTR1F', 'HTR2A', 'HTR2B', 'HTR2C', 'HTR3A', 'HTR3B', 'HTR3C', 'HTR3D', 'HTR3E', 'HTR4', 'HTR5A', 'HTR6', 'HTR7', 'HTRA1', 'HTRA2', 'HTRA3', 'HTRA4', 'HTT', 'HUNK', 'HUS1', 'HUS1B', 'HUWE1', 'HVCN1', 'HYAL1', 'HYAL2', 'HYAL3', 'HYAL4', 'HYDIN', 'HYI', 'HYKK', 'HYLS1', 'HYOU1', 'HYPK', 'IAH1', 'IAPP', 'IARS1', 'IARS2', 'IBA57', 'IBSP', 'IBTK', 'ICA1', 'ICA1L', 'ICAM1', 'ICAM2', 'ICAM3', 'ICAM4', 'ICAM5', 'ICE1', 'ICE2', 'ICMT', 'ICOS', 'ICOSLG', 'ID1', 'ID2', 'ID3', 'ID4', 'IDE', 'IDH1', 'IDH2', 'IDH3A', 'IDH3B', 'IDH3G', 'IDI1', 'IDI2', 'IDNK', 'IDO1', 'IDO2', 'IDS', 'IDUA', 'IER2', 'IER3', 'IER3IP1', 'IER5', 'IER5L', 'IFFO1', 'IFFO2', 'IFI6', 'IFI16', 'IFI27', 'IFI27L1', 'IFI27L2', 'IFI30', 'IFI35', 'IFI44', 'IFI44L', 'IFIH1', 'IFIT1', 'IFIT1B', 'IFIT2', 'IFIT3', 'IFIT5', 'IFITM1', 'IFITM2', 'IFITM3', 'IFITM5', 'IFITM10', 'IFNA1', 'IFNA2', 'IFNA4', 'IFNA5', 'IFNA6', 'IFNA7', 'IFNA8', 'IFNA10', 'IFNA13', 'IFNA14', 'IFNA16', 'IFNA17', 'IFNA21', 'IFNAR1', 'IFNAR2', 'IFNB1', 'IFNE', 'IFNG', 'IFNGR1', 'IFNGR2', 'IFNK', 'IFNL1', 'IFNL2', 'IFNL3', 'IFNL4', 'IFNLR1', 'IFNW1', 'IFRD1', 'IFRD2', 'IFT20', 'IFT22', 'IFT27', 'IFT43', 'IFT46', 'IFT52', 'IFT57', 'IFT74', 'IFT80', 'IFT81', 'IFT88', 'IFT122', 'IFT140', 'IFT172', 'IFTAP', 'IGBP1', 'IGDCC3', 'IGDCC4', 'IGF1', 'IGF1R', 'IGF2', 'IGF2BP1', 'IGF2BP2', 'IGF2BP3', 'IGF2R', 'IGFALS', 'IGFBP1', 'IGFBP2', 'IGFBP3', 'IGFBP4', 'IGFBP5', 'IGFBP6', 'IGFBP7', 'IGFBPL1', 'IGFL1', 'IGFL2', 'IGFL3', 'IGFL4', 'IGFLR1', 'IGFN1', 'IGH', 'IGHMBP2', 'IGIP', 'IGK', 'IGL', 'IGLL1', 'IGLL5', 'IGLON5', 'IGSF1', 'IGSF3', 'IGSF5', 'IGSF6', 'IGSF8', 'IGSF9', 'IGSF9B', 'IGSF10', 'IGSF11', 'IGSF21', 'IGSF22', 'IGSF23', 'IHH', 'IHO1', 'IK', 'IKBIP', 'IKBKB', 'IKBKE', 'IKBKG', 'IKZF1', 'IKZF2', 'IKZF3', 'IKZF4', 'IKZF5', 'IL1A', 'IL1B', 'IL1F10', 'IL1R1', 'IL1R2', 'IL1RAP', 'IL1RAPL1', 'IL1RAPL2', 'IL1RL1', 'IL1RL2', 'IL1RN', 'IL2', 'IL2RA', 'IL2RB', 'IL2RG', 'IL3', 'IL3RA', 'IL4', 'IL4I1', 'IL4R', 'IL5', 'IL5RA', 'IL6', 'IL6R', 'IL6ST', 'IL7', 'IL7R', 'IL9', 'IL9R', 'IL10', 'IL10RA', 'IL10RB', 'IL11', 'IL11RA', 'IL12A', 'IL12B', 'IL12RB1', 'IL12RB2', 'IL13', 'IL13RA1', 'IL13RA2', 'IL15', 'IL15RA', 'IL16', 'IL17A', 'IL17B', 'IL17C', 'IL17D', 'IL17F', 'IL17RA', 'IL17RB', 'IL17RC', 'IL17RD', 'IL17RE', 'IL17REL', 'IL18', 'IL18BP', 'IL18R1', 'IL18RAP', 'IL19', 'IL20', 'IL20RA', 'IL20RB', 'IL21', 'IL21R', 'IL22', 'IL22RA1', 'IL22RA2', 'IL23A', 'IL23R', 'IL24', 'IL25', 'IL26', 'IL27', 'IL27RA', 'IL31', 'IL31RA', 'IL32', 'IL33', 'IL34', 'IL36A', 'IL36B', 'IL36G', 'IL36RN', 'IL37', 'ILDR1', 'ILDR2', 'ILF2', 'ILF3', 'ILK', 'ILKAP', 'ILRUN', 'ILVBL', 'IMMP1L', 'IMMP2L', 'IMMT', 'IMP3', 'IMP4', 'IMPA1', 'IMPA2', 'IMPACT', 'IMPDH1', 'IMPDH2', 'IMPG1', 'IMPG2', 'INA', 'INAFM1', 'INAFM2', 'INAVA', 'INCA1', 'INCENP', 'INF2', 'ING1', 'ING2', 'ING3', 'ING4', 'ING5', 'INHA', 'INHBA', 'INHBB', 'INHBC', 'INHBE', 'INIP', 'INKA1', 'INKA2', 'INMT', 'INO80', 'INO80B', 'INO80C', 'INO80D', 'INO80E', 'INPP1', 'INPP4A', 'INPP4B', 'INPP5A', 'INPP5B', 'INPP5D', 'INPP5E', 'INPP5F', 'INPP5J', 'INPP5K', 'INPPL1', 'INS', 'INSC', 'INSIG1', 'INSIG2', 'INSL3', 'INSL4', 'INSL5', 'INSL6', 'INSM1', 'INSM2', 'INSR', 'INSRR', 'INSYN1', 'INSYN2A', 'INSYN2B', 'INTS1', 'INTS2', 'INTS3', 'INTS4', 'INTS5', 'INTS6', 'INTS6L', 'INTS7', 'INTS8', 'INTS9', 'INTS10', 'INTS11', 'INTS12', 'INTS13', 'INTS14', 'INTU', 'INVS', 'IP6K1', 'IP6K2', 'IP6K3', 'IPCEF1', 'IPMK', 'IPO4', 'IPO5', 'IPO7', 'IPO8', 'IPO9', 'IPO11', 'IPO13', 'IPP', 'IPPK', 'IQANK1', 'IQCA1', 'IQCA1L', 'IQCB1', 'IQCC', 'IQCD', 'IQCE', 'IQCF1', 'IQCF2', 'IQCF3', 'IQCF5', 'IQCF6', 'IQCG', 'IQCH', 'IQCJ', 'IQCK', 'IQCM', 'IQCN', 'IQGAP1', 'IQGAP2', 'IQGAP3', 'IQSEC1', 'IQSEC2', 'IQSEC3', 'IQUB', 'IRAG1', 'IRAG2', 'IRAK1', 'IRAK1BP1', 'IRAK2', 'IRAK3', 'IRAK4', 'IREB2', 'IRF1', 'IRF2', 'IRF2BP1', 'IRF2BP2', 'IRF2BPL', 'IRF3', 'IRF4', 'IRF5', 'IRF6', 'IRF7', 'IRF8', 'IRF9', 'IRGC', 'IRGM', 'IRGQ', 'IRS1', 'IRS2', 'IRS4', 'IRX1', 'IRX2', 'IRX3', 'IRX4', 'IRX5', 'IRX6', 'ISCA1', 'ISCA2', 'ISCU', 'ISG15', 'ISG20', 'ISG20L2', 'ISL1', 'ISL2', 'ISLR', 'ISLR2', 'ISM1', 'ISM2', 'ISOC1', 'ISOC2', 'IST1', 'ISX', 'ISY1', 'ISYNA1', 'ITCH', 'ITFG1', 'ITFG2', 'ITGA1', 'ITGA2', 'ITGA2B', 'ITGA3', 'ITGA4', 'ITGA5', 'ITGA6', 'ITGA7', 'ITGA8', 'ITGA9', 'ITGA10', 'ITGA11', 'ITGAD', 'ITGAE', 'ITGAL', 'ITGAM', 'ITGAV', 'ITGAX', 'ITGB1', 'ITGB1BP1', 'ITGB1BP2', 'ITGB2', 'ITGB3', 'ITGB3BP', 'ITGB4', 'ITGB5', 'ITGB6', 'ITGB7', 'ITGB8', 'ITGBL1', 'ITIH1', 'ITIH2', 'ITIH3', 'ITIH4', 'ITIH5', 'ITIH6', 'ITK', 'ITLN1', 'ITLN2', 'ITM2A', 'ITM2B', 'ITM2C', 'ITPA', 'ITPK1', 'ITPKA', 'ITPKB', 'ITPKC', 'ITPR1', 'ITPR2', 'ITPR3', 'ITPRID1', 'ITPRID2', 'ITPRIP', 'ITPRIPL1', 'ITPRIPL2', 'ITSN1', 'ITSN2', 'IVD', 'IVL', 'IVNS1ABP', 'IWS1', 'IYD', 'IZUMO1', 'IZUMO1R', 'IZUMO2', 'IZUMO3', 'IZUMO4', 'JADE1', 'JADE2', 'JADE3', 'JAG1', 'JAG2', 'JAGN1', 'JAK1', 'JAK2', 'JAK3', 'JAKMIP1', 'JAKMIP2', 'JAKMIP3', 'JAM2', 'JAM3', 'JAML', 'JARID2', 'JAZF1', 'JCAD', 'JCHAIN', 'JDP2', 'JHY', 'JKAMP', 'JMJD1C', 'JMJD4', 'JMJD6', 'JMJD7', 'JMJD8', 'JMY', 'JOSD1', 'JOSD2', 'JPH1', 'JPH2', 'JPH3', 'JPH4', 'JPT1', 'JPT2', 'JRK', 'JRKL', 'JSRP1', 'JTB', 'JUN', 'JUNB', 'JUND', 'JUP', 'KAAG1', 'KALRN', 'KANK1', 'KANK2', 'KANK3', 'KANK4', 'KANSL1', 'KANSL1L', 'KANSL2', 'KANSL3', 'KARS1', 'KASH5', 'KAT2A', 'KAT2B', 'KAT5', 'KAT6A', 'KAT6B', 'KAT7', 'KAT8', 'KAT14', 'KATNA1', 'KATNAL1', 'KATNAL2', 'KATNB1', 'KATNBL1', 'KATNIP', 'KAZALD1', 'KAZN', 'KBTBD2', 'KBTBD3', 'KBTBD4', 'KBTBD6', 'KBTBD7', 'KBTBD8', 'KBTBD11', 'KBTBD12', 'KBTBD13', 'KCMF1', 'KCNA1', 'KCNA2', 'KCNA3', 'KCNA4', 'KCNA5', 'KCNA6', 'KCNA7', 'KCNA10', 'KCNAB1', 'KCNAB2', 'KCNAB3', 'KCNB1', 'KCNB2', 'KCNC1', 'KCNC2', 'KCNC3', 'KCNC4', 'KCND1', 'KCND2', 'KCND3', 'KCNE1', 'KCNE1B', 'KCNE2', 'KCNE3', 'KCNE4', 'KCNE5', 'KCNF1', 'KCNG1', 'KCNG2', 'KCNG3', 'KCNG4', 'KCNH1', 'KCNH2', 'KCNH3', 'KCNH4', 'KCNH5', 'KCNH6', 'KCNH7', 'KCNH8', 'KCNIP1', 'KCNIP2', 'KCNIP3', 'KCNIP4', 'KCNJ1', 'KCNJ2', 'KCNJ3', 'KCNJ4', 'KCNJ5', 'KCNJ6', 'KCNJ8', 'KCNJ9', 'KCNJ10', 'KCNJ11', 'KCNJ12', 'KCNJ13', 'KCNJ14', 'KCNJ15', 'KCNJ16', 'KCNJ18', 'KCNK1', 'KCNK2', 'KCNK3', 'KCNK4', 'KCNK5', 'KCNK6', 'KCNK7', 'KCNK9', 'KCNK10', 'KCNK12', 'KCNK13', 'KCNK15', 'KCNK16', 'KCNK17', 'KCNK18', 'KCNMA1', 'KCNMB1', 'KCNMB2', 'KCNMB3', 'KCNMB4', 'KCNN1', 'KCNN2', 'KCNN3', 'KCNN4', 'KCNQ1', 'KCNQ2', 'KCNQ3', 'KCNQ4', 'KCNQ5', 'KCNRG', 'KCNS1', 'KCNS2', 'KCNS3', 'KCNT1', 'KCNT2', 'KCNU1', 'KCNV1', 'KCNV2', 'KCP', 'KCTD1', 'KCTD2', 'KCTD3', 'KCTD4', 'KCTD5', 'KCTD6', 'KCTD7', 'KCTD8', 'KCTD9', 'KCTD10', 'KCTD11', 'KCTD12', 'KCTD13', 'KCTD14', 'KCTD15', 'KCTD16', 'KCTD17', 'KCTD18', 'KCTD19', 'KCTD20', 'KCTD21', 'KDELR1', 'KDELR2', 'KDELR3', 'KDF1', 'KDM1A', 'KDM1B', 'KDM2A', 'KDM2B', 'KDM3A', 'KDM3B', 'KDM4A', 'KDM4B', 'KDM4C', 'KDM4D', 'KDM4E', 'KDM5A', 'KDM5B', 'KDM5C', 'KDM5D', 'KDM6A', 'KDM6B', 'KDM7A', 'KDM8', 'KDR', 'KDSR', 'KEAP1', 'KEL', 'KERA', 'KHDC1', 'KHDC1L', 'KHDC3L', 'KHDC4', 'KHDRBS1', 'KHDRBS2', 'KHDRBS3', 'KHK', 'KHNYN', 'KHSRP', 'KIAA0040', 'KIAA0100', 'KIAA0232', 'KIAA0319', 'KIAA0319L', 'KIAA0408', 'KIAA0513', 'KIAA0586', 'KIAA0753', 'KIAA0754', 'KIAA0825', 'KIAA0895', 'KIAA0895L', 'KIAA0930', 'KIAA1107', 'KIAA1109', 'KIAA1143', 'KIAA1191', 'KIAA1210', 'KIAA1217', 'KIAA1328', 'KIAA1522', 'KIAA1549', 'KIAA1549L', 'KIAA1586', 'KIAA1614', 'KIAA1671', 'KIAA1755', 'KIAA1958', 'KIAA2012', 'KIAA2013', 'KIAA2026', 'KICS2', 'KIDINS220', 'KIF1A', 'KIF1B', 'KIF1C', 'KIF2A', 'KIF2B', 'KIF2C', 'KIF3A', 'KIF3B', 'KIF3C', 'KIF4A', 'KIF4B', 'KIF5A', 'KIF5B', 'KIF5C', 'KIF6', 'KIF7', 'KIF9', 'KIF11', 'KIF12', 'KIF13A', 'KIF13B', 'KIF14', 'KIF15', 'KIF16B', 'KIF17', 'KIF18A', 'KIF18B', 'KIF19', 'KIF20A', 'KIF20B', 'KIF21A', 'KIF21B', 'KIF22', 'KIF23', 'KIF24', 'KIF25', 'KIF26A', 'KIF26B', 'KIF27', 'KIFAP3', 'KIFBP', 'KIFC1', 'KIFC2', 'KIFC3', 'KIN', 'KIR2DL1', 'KIR2DL3', 'KIR2DL4', 'KIR2DS4', 'KIR3DL1', 'KIR3DL2', 'KIR3DL3', 'KIRREL1', 'KIRREL2', 'KIRREL3', 'KISS1', 'KISS1R', 'KIT', 'KITLG', 'KIZ', 'KL', 'KLB', 'KLC1', 'KLC2', 'KLC3', 'KLC4', 'KLF1', 'KLF2', 'KLF3', 'KLF4', 'KLF5', 'KLF6', 'KLF7', 'KLF8', 'KLF9', 'KLF10', 'KLF11', 'KLF12', 'KLF13', 'KLF14', 'KLF15', 'KLF16', 'KLF17', 'KLF18', 'KLHDC1', 'KLHDC2', 'KLHDC3', 'KLHDC4', 'KLHDC7A', 'KLHDC7B', 'KLHDC8A', 'KLHDC8B', 'KLHDC9', 'KLHDC10', 'KLHL1', 'KLHL2', 'KLHL3', 'KLHL4', 'KLHL5', 'KLHL6', 'KLHL7', 'KLHL8', 'KLHL9', 'KLHL10', 'KLHL11', 'KLHL12', 'KLHL13', 'KLHL14', 'KLHL15', 'KLHL17', 'KLHL18', 'KLHL20', 'KLHL21', 'KLHL22', 'KLHL23', 'KLHL24', 'KLHL25', 'KLHL26', 'KLHL28', 'KLHL29', 'KLHL30', 'KLHL31', 'KLHL32', 'KLHL33', 'KLHL34', 'KLHL35', 'KLHL36', 'KLHL38', 'KLHL40', 'KLHL41', 'KLHL42', 'KLK1', 'KLK2', 'KLK3', 'KLK4', 'KLK5', 'KLK6', 'KLK7', 'KLK8', 'KLK9', 'KLK10', 'KLK11', 'KLK12', 'KLK13', 'KLK14', 'KLK15', 'KLKB1', 'KLLN', 'KLRB1', 'KLRC1', 'KLRC2', 'KLRC3', 'KLRC4', 'KLRD1', 'KLRF1', 'KLRF2', 'KLRG1', 'KLRG2', 'KLRK1', 'KMO', 'KMT2A', 'KMT2B', 'KMT2C', 'KMT2D', 'KMT2E', 'KMT5A', 'KMT5B', 'KMT5C', 'KNCN', 'KNDC1', 'KNG1', 'KNL1', 'KNOP1', 'KNSTRN', 'KNTC1', 'KPNA1', 'KPNA2', 'KPNA3', 'KPNA4', 'KPNA5', 'KPNA6', 'KPNA7', 'KPNB1', 'KPRP', 'KPTN', 'KRAS', 'KRBA1', 'KRBA2', 'KRBOX1', 'KRBOX4', 'KRCC1', 'KREMEN1', 'KREMEN2', 'KRI1', 'KRIT1', 'KRR1', 'KRT1', 'KRT2', 'KRT3', 'KRT4', 'KRT5', 'KRT6A', 'KRT6B', 'KRT6C', 'KRT7', 'KRT8', 'KRT9', 'KRT10', 'KRT12', 'KRT13', 'KRT14', 'KRT15', 'KRT16', 'KRT17', 'KRT18', 'KRT19', 'KRT20', 'KRT23', 'KRT24', 'KRT25', 'KRT26', 'KRT27', 'KRT28', 'KRT31', 'KRT32', 'KRT33A', 'KRT33B', 'KRT34', 'KRT35', 'KRT36', 'KRT37', 'KRT38', 'KRT39', 'KRT40', 'KRT71', 'KRT72', 'KRT73', 'KRT74', 'KRT75', 'KRT76', 'KRT77', 'KRT78', 'KRT79', 'KRT80', 'KRT81', 'KRT82', 'KRT83', 'KRT84', 'KRT85', 'KRT86', 'KRT222', 'KRTAP1-1', 'KRTAP1-3', 'KRTAP1-4', 'KRTAP1-5', 'KRTAP2-1', 'KRTAP2-2', 'KRTAP2-3', 'KRTAP2-4', 'KRTAP3-1', 'KRTAP3-2', 'KRTAP3-3', 'KRTAP4-1', 'KRTAP4-2', 'KRTAP4-3', 'KRTAP4-4', 'KRTAP4-5', 'KRTAP4-6', 'KRTAP4-7', 'KRTAP4-8', 'KRTAP4-9', 'KRTAP4-11', 'KRTAP4-12', 'KRTAP5-1', 'KRTAP5-2', 'KRTAP5-3', 'KRTAP5-4', 'KRTAP5-5', 'KRTAP5-6', 'KRTAP5-7', 'KRTAP5-8', 'KRTAP5-9', 'KRTAP5-10', 'KRTAP5-11', 'KRTAP6-1', 'KRTAP6-2', 'KRTAP6-3', 'KRTAP7-1', 'KRTAP8-1', 'KRTAP9-1', 'KRTAP9-2', 'KRTAP9-3', 'KRTAP9-4', 'KRTAP9-6', 'KRTAP9-7', 'KRTAP9-8', 'KRTAP9-9', 'KRTAP10-1', 'KRTAP10-2', 'KRTAP10-3', 'KRTAP10-4', 'KRTAP10-5', 'KRTAP10-6', 'KRTAP10-7', 'KRTAP10-8', 'KRTAP10-9', 'KRTAP10-10', 'KRTAP10-11', 'KRTAP10-12', 'KRTAP11-1', 'KRTAP12-1', 'KRTAP12-2', 'KRTAP12-3', 'KRTAP12-4', 'KRTAP13-1', 'KRTAP13-2', 'KRTAP13-3', 'KRTAP13-4', 'KRTAP15-1', 'KRTAP16-1', 'KRTAP17-1', 'KRTAP19-1', 'KRTAP19-2', 'KRTAP19-3', 'KRTAP19-4', 'KRTAP19-5', 'KRTAP19-6', 'KRTAP19-7', 'KRTAP19-8', 'KRTAP20-1', 'KRTAP20-2', 'KRTAP20-3', 'KRTAP20-4', 'KRTAP21-1', 'KRTAP21-2', 'KRTAP21-3', 'KRTAP22-1', 'KRTAP22-2', 'KRTAP23-1', 'KRTAP24-1', 'KRTAP25-1', 'KRTAP26-1', 'KRTAP27-1', 'KRTAP29-1', 'KRTCAP2', 'KRTCAP3', 'KRTDAP', 'KSR1', 'KSR2', 'KTI12', 'KTN1', 'KXD1', 'KY', 'KYAT1', 'KYAT3', 'KYNU', 'L1CAM', 'L1TD1', 'L2HGDH', 'L3HYPDH', 'L3MBTL1', 'L3MBTL2', 'L3MBTL3', 'L3MBTL4', 'LACC1', 'LACRT', 'LACTB', 'LACTB2', 'LACTBL1', 'LAD1', 'LAG3', 'LAGE3', 'LAIR1', 'LAIR2', 'LALBA', 'LAMA1', 'LAMA2', 'LAMA3', 'LAMA4', 'LAMA5', 'LAMB1', 'LAMB2', 'LAMB3', 'LAMB4', 'LAMC1', 'LAMC2', 'LAMC3', 'LAMP1', 'LAMP2', 'LAMP3', 'LAMP5', 'LAMTOR1', 'LAMTOR2', 'LAMTOR3', 'LAMTOR4', 'LAMTOR5', 'LANCL1', 'LANCL2', 'LANCL3', 'LAP3', 'LAPTM4A', 'LAPTM4B', 'LAPTM5', 'LARGE1', 'LARGE2', 'LARP1', 'LARP1B', 'LARP4', 'LARP4B', 'LARP6', 'LARP7', 'LARS1', 'LARS2', 'LAS1L', 'LASP1', 'LAT', 'LAT2', 'LATS1', 'LATS2', 'LAX1', 'LAYN', 'LBH', 'LBHD1', 'LBHD2', 'LBP', 'LBR', 'LBX1', 'LBX2', 'LCA5', 'LCA5L', 'LCAT', 'LCE1A', 'LCE1B', 'LCE1C', 'LCE1D', 'LCE1E', 'LCE1F', 'LCE2A', 'LCE2B', 'LCE2C', 'LCE2D', 'LCE3A', 'LCE3B', 'LCE3C', 'LCE3D', 'LCE3E', 'LCE4A', 'LCE5A', 'LCE6A', 'LCK', 'LCLAT1', 'LCMT1', 'LCMT2', 'LCN1', 'LCN2', 'LCN6', 'LCN8', 'LCN9', 'LCN10', 'LCN12', 'LCN15', 'LCNL1', 'LCOR', 'LCORL', 'LCP1', 'LCP2', 'LCT', 'LCTL', 'LDAF1', 'LDAH', 'LDB1', 'LDB2', 'LDB3', 'LDHA', 'LDHAL6A', 'LDHAL6B', 'LDHB', 'LDHC', 'LDHD', 'LDLR', 'LDLRAD1', 'LDLRAD2', 'LDLRAD3', 'LDLRAD4', 'LDLRAP1', 'LDOC1', 'LEAP2', 'LECT2', 'LEF1', 'LEFTY1', 'LEFTY2', 'LEKR1', 'LELP1', 'LEMD1', 'LEMD2', 'LEMD3', 'LENEP', 'LENG1', 'LENG8', 'LENG9', 'LEO1', 'LEP', 'LEPR', 'LEPROT', 'LEPROTL1', 'LETM1', 'LETM2', 'LETMD1', 'LEUTX', 'LEXM', 'LFNG', 'LGALS1', 'LGALS2', 'LGALS3', 'LGALS3BP', 'LGALS4', 'LGALS7', 'LGALS7B', 'LGALS8', 'LGALS9', 'LGALS9B', 'LGALS9C', 'LGALS12', 'LGALS13', 'LGALS14', 'LGALS16', 'LGALSL', 'LGI1', 'LGI2', 'LGI3', 'LGI4', 'LGMN', 'LGR4', 'LGR5', 'LGR6', 'LGSN', 'LHB', 'LHCGR', 'LHFPL1', 'LHFPL2', 'LHFPL3', 'LHFPL4', 'LHFPL5', 'LHFPL6', 'LHPP', 'LHX1', 'LHX2', 'LHX3', 'LHX4', 'LHX5', 'LHX6', 'LHX8', 'LHX9', 'LIAS', 'LIF', 'LIFR', 'LIG1', 'LIG3', 'LIG4', 'LILRA1', 'LILRA2', 'LILRA4', 'LILRA5', 'LILRA6', 'LILRB1', 'LILRB2', 'LILRB3', 'LILRB4', 'LILRB5', 'LIM2', 'LIMA1', 'LIMCH1', 'LIMD1', 'LIMD2', 'LIME1', 'LIMK1', 'LIMK2', 'LIMS1', 'LIMS2', 'LIMS3', 'LIMS4', 'LIN7A', 'LIN7B', 'LIN7C', 'LIN9', 'LIN28A', 'LIN28B', 'LIN37', 'LIN52', 'LIN54', 'LINGO1', 'LINGO2', 'LINGO3', 'LINGO4', 'LINS1', 'LIPA', 'LIPC', 'LIPE', 'LIPF', 'LIPG', 'LIPH', 'LIPI', 'LIPJ', 'LIPK', 'LIPM', 'LIPN', 'LIPT1', 'LIPT2', 'LITAF', 'LITAFD', 'LIX1', 'LIX1L', 'LKAAEAR1', 'LLCFC1', 'LLGL1', 'LLGL2', 'LLPH', 'LMAN1', 'LMAN1L', 'LMAN2', 'LMAN2L', 'LMBR1', 'LMBR1L', 'LMBRD1', 'LMBRD2', 'LMCD1', 'LMF1', 'LMF2', 'LMLN', 'LMLN2', 'LMNA', 'LMNB1', 'LMNB2', 'LMNTD1', 'LMNTD2', 'LMO1', 'LMO2', 'LMO3', 'LMO4', 'LMO7', 'LMOD1', 'LMOD2', 'LMOD3', 'LMTK2', 'LMTK3', 'LMX1A', 'LMX1B', 'LNP1', 'LNPEP', 'LNPK', 'LNX1', 'LNX2', 'LONP1', 'LONP2', 'LONRF1', 'LONRF2', 'LONRF3', 'LORICRIN', 'LOX', 'LOXHD1', 'LOXL1', 'LOXL2', 'LOXL3', 'LOXL4', 'LPA', 'LPAR1', 'LPAR2', 'LPAR3', 'LPAR4', 'LPAR5', 'LPAR6', 'LPCAT1', 'LPCAT2', 'LPCAT3', 'LPCAT4', 'LPGAT1', 'LPIN1', 'LPIN2', 'LPIN3', 'LPL', 'LPO', 'LPP', 'LPXN', 'LRAT', 'LRATD1', 'LRATD2', 'LRBA', 'LRCH1', 'LRCH2', 'LRCH3', 'LRCH4', 'LRCOL1', 'LRFN1', 'LRFN2', 'LRFN3', 'LRFN4', 'LRFN5', 'LRG1', 'LRGUK', 'LRIF1', 'LRIG1', 'LRIG2', 'LRIG3', 'LRIT1', 'LRIT2', 'LRIT3', 'LRMDA', 'LRP1', 'LRP1B', 'LRP2', 'LRP2BP', 'LRP3', 'LRP4', 'LRP5', 'LRP5L', 'LRP6', 'LRP8', 'LRP10', 'LRP11', 'LRP12', 'LRPAP1', 'LRPPRC', 'LRR1', 'LRRC1', 'LRRC2', 'LRRC3', 'LRRC3B', 'LRRC3C', 'LRRC4', 'LRRC4B', 'LRRC4C', 'LRRC7', 'LRRC8A', 'LRRC8B', 'LRRC8C', 'LRRC8D', 'LRRC8E', 'LRRC9', 'LRRC10', 'LRRC10B', 'LRRC14', 'LRRC14B', 'LRRC15', 'LRRC17', 'LRRC18', 'LRRC19', 'LRRC20', 'LRRC23', 'LRRC24', 'LRRC25', 'LRRC26', 'LRRC27', 'LRRC28', 'LRRC29', 'LRRC30', 'LRRC31', 'LRRC32', 'LRRC34', 'LRRC36', 'LRRC37A', 'LRRC37A2', 'LRRC37A3', 'LRRC37B', 'LRRC38', 'LRRC39', 'LRRC40', 'LRRC41', 'LRRC42', 'LRRC43', 'LRRC45', 'LRRC46', 'LRRC47', 'LRRC49', 'LRRC51', 'LRRC52', 'LRRC53', 'LRRC55', 'LRRC56', 'LRRC57', 'LRRC58', 'LRRC59', 'LRRC61', 'LRRC63', 'LRRC66', 'LRRC69', 'LRRC70', 'LRRC71', 'LRRC72', 'LRRC73', 'LRRC74A', 'LRRC74B', 'LRRC75A', 'LRRC75B', 'LRRCC1', 'LRRD1', 'LRRFIP1', 'LRRFIP2', 'LRRIQ1', 'LRRIQ3', 'LRRIQ4', 'LRRK1', 'LRRK2', 'LRRN1', 'LRRN2', 'LRRN3', 'LRRN4', 'LRRN4CL', 'LRRTM1', 'LRRTM2', 'LRRTM3', 'LRRTM4', 'LRSAM1', 'LRTM1', 'LRTM2', 'LRWD1', 'LSAMP', 'LSG1', 'LSM1', 'LSM2', 'LSM3', 'LSM4', 'LSM5', 'LSM6', 'LSM7', 'LSM8', 'LSM10', 'LSM11', 'LSM12', 'LSM14A', 'LSM14B', 'LSMEM1', 'LSMEM2', 'LSP1', 'LSR', 'LSS', 'LST1', 'LTA', 'LTA4H', 'LTB', 'LTB4R', 'LTB4R2', 'LTBP1', 'LTBP2', 'LTBP3', 'LTBP4', 'LTBR', 'LTC4S', 'LTF', 'LTK', 'LTN1', 'LTO1', 'LTV1', 'LUC7L', 'LUC7L2', 'LUC7L3', 'LUM', 'LURAP1', 'LURAP1L', 'LUZP1', 'LUZP2', 'LUZP4', 'LUZP6', 'LVRN', 'LXN', 'LY6D', 'LY6E', 'LY6G5B', 'LY6G5C', 'LY6G6C', 'LY6G6D', 'LY6G6F', 'LY6H', 'LY6K', 'LY6L', 'LY9', 'LY75', 'LY86', 'LY96', 'LYAR', 'LYG1', 'LYG2', 'LYL1', 'LYN', 'LYNX1', 'LYPD1', 'LYPD2', 'LYPD3', 'LYPD4', 'LYPD5', 'LYPD6', 'LYPD6B', 'LYPD8', 'LYPLA1', 'LYPLA2', 'LYPLAL1', 'LYRM1', 'LYRM2', 'LYRM4', 'LYRM7', 'LYRM9', 'LYSMD1', 'LYSMD2', 'LYSMD3', 'LYSMD4', 'LYST', 'LYVE1', 'LYZ', 'LYZL1', 'LYZL2', 'LYZL4', 'LYZL6', 'LZIC', 'LZTFL1', 'LZTR1', 'LZTS1', 'LZTS2', 'LZTS3', 'M1AP', 'M6PR', 'MAB21L1', 'MAB21L2', 'MAB21L3', 'MAB21L4', 'MACC1', 'MACF1', 'MACIR', 'MACO1', 'MACROD1', 'MACROD2', 'MACROH2A1', 'MACROH2A2', 'MAD1L1', 'MAD2L1', 'MAD2L1BP', 'MAD2L2', 'MADCAM1', 'MADD', 'MAEA', 'MAEL', 'MAF', 'MAF1', 'MAFA', 'MAFB', 'MAFF', 'MAFG', 'MAFK', 'MAG', 'MAGEA1', 'MAGEA2', 'MAGEA2B', 'MAGEA3', 'MAGEA4', 'MAGEA6', 'MAGEA8', 'MAGEA9', 'MAGEA9B', 'MAGEA10', 'MAGEA11', 'MAGEA12', 'MAGEB1', 'MAGEB2', 'MAGEB3', 'MAGEB4', 'MAGEB5', 'MAGEB6', 'MAGEB10', 'MAGEB16', 'MAGEB17', 'MAGEB18', 'MAGEC1', 'MAGEC2', 'MAGEC3', 'MAGED1', 'MAGED2', 'MAGED4', 'MAGED4B', 'MAGEE1', 'MAGEE2', 'MAGEF1', 'MAGEH1', 'MAGEL2', 'MAGI1', 'MAGI2', 'MAGI3', 'MAGIX', 'MAGOH', 'MAGOHB', 'MAGT1', 'MAIP1', 'MAJIN', 'MAK', 'MAK16', 'MAL', 'MAL2', 'MALL', 'MALRD1', 'MALSU1', 'MALT1', 'MAMDC2', 'MAMDC4', 'MAML1', 'MAML2', 'MAML3', 'MAMLD1', 'MAMSTR', 'MAN1A1', 'MAN1A2', 'MAN1B1', 'MAN1C1', 'MAN2A1', 'MAN2A2', 'MAN2B1', 'MAN2B2', 'MAN2C1', 'MANBA', 'MANBAL', 'MANEA', 'MANEAL', 'MANF', 'MANSC1', 'MANSC4', 'MAOA', 'MAOB', 'MAP1A', 'MAP1B', 'MAP1LC3A', 'MAP1LC3B', 'MAP1LC3B2', 'MAP1LC3C', 'MAP1S', 'MAP2', 'MAP2K1', 'MAP2K2', 'MAP2K3', 'MAP2K4', 'MAP2K5', 'MAP2K6', 'MAP2K7', 'MAP3K1', 'MAP3K2', 'MAP3K3', 'MAP3K4', 'MAP3K5', 'MAP3K6', 'MAP3K7', 'MAP3K7CL', 'MAP3K8', 'MAP3K9', 'MAP3K10', 'MAP3K11', 'MAP3K12', 'MAP3K13', 'MAP3K14', 'MAP3K15', 'MAP3K19', 'MAP3K20', 'MAP3K21', 'MAP4', 'MAP4K1', 'MAP4K2', 'MAP4K3', 'MAP4K4', 'MAP4K5', 'MAP6', 'MAP6D1', 'MAP7', 'MAP7D1', 'MAP7D2', 'MAP7D3', 'MAP9', 'MAP10', 'MAPK1', 'MAPK1IP1L', 'MAPK3', 'MAPK4', 'MAPK6', 'MAPK7', 'MAPK8', 'MAPK8IP1', 'MAPK8IP2', 'MAPK8IP3', 'MAPK9', 'MAPK10', 'MAPK11', 'MAPK12', 'MAPK13', 'MAPK14', 'MAPK15', 'MAPKAP1', 'MAPKAPK2', 'MAPKAPK3', 'MAPKAPK5', 'MAPKBP1', 'MAPRE1', 'MAPRE2', 'MAPRE3', 'MAPT', 'MARCHF1', 'MARCHF2', 'MARCHF3', 'MARCHF4', 'MARCHF5', 'MARCHF6', 'MARCHF7', 'MARCHF8', 'MARCHF9', 'MARCHF10', 'MARCHF11', 'MARCKS', 'MARCKSL1', 'MARCO', 'MARCOL', 'MARF1', 'MARK1', 'MARK2', 'MARK3', 'MARK4', 'MARS1', 'MARS2', 'MARVELD1', 'MARVELD2', 'MARVELD3', 'MAS1', 'MAS1L', 'MASP1', 'MASP2', 'MAST1', 'MAST2', 'MAST3', 'MAST4', 'MASTL', 'MAT1A', 'MAT2A', 'MAT2B', 'MATK', 'MATN1', 'MATN2', 'MATN3', 'MATN4', 'MATR3', 'MAU2', 'MAVS', 'MAX', 'MAZ', 'MB', 'MB21D2', 'MBD1', 'MBD2', 'MBD3', 'MBD3L1', 'MBD3L2', 'MBD3L2B', 'MBD3L3', 'MBD3L4', 'MBD3L5', 'MBD4', 'MBD5', 'MBD6', 'MBIP', 'MBL2', 'MBLAC1', 'MBLAC2', 'MBNL1', 'MBNL2', 'MBNL3', 'MBOAT1', 'MBOAT2', 'MBOAT4', 'MBOAT7', 'MBP', 'MBTD1', 'MBTPS1', 'MBTPS2', 'MC1R', 'MC2R', 'MC3R', 'MC4R', 'MC5R', 'MCAM', 'MCAT', 'MCC', 'MCCC1', 'MCCC2', 'MCCD1', 'MCEE', 'MCEMP1', 'MCF2', 'MCF2L', 'MCF2L2', 'MCFD2', 'MCHR1', 'MCHR2', 'MCIDAS', 'MCL1', 'MCM2', 'MCM3', 'MCM3AP', 'MCM4', 'MCM5', 'MCM6', 'MCM7', 'MCM8', 'MCM9', 'MCM10', 'MCMBP', 'MCMDC2', 'MCOLN1', 'MCOLN2', 'MCOLN3', 'MCPH1', 'MCRIP1', 'MCRIP2', 'MCRS1', 'MCTP1', 'MCTP2', 'MCTS1', 'MCU', 'MCUB', 'MCUR1', 'MDC1', 'MDFI', 'MDFIC', 'MDFIC2', 'MDGA1', 'MDGA2', 'MDH1', 'MDH1B', 'MDH2', 'MDK', 'MDM1', 'MDM2', 'MDM4', 'MDN1', 'MDP1', 'ME1', 'ME2', 'ME3', 'MEA1', 'MEAF6', 'MEAK7', 'MECOM', 'MECP2', 'MECR', 'MED1', 'MED4', 'MED6', 'MED7', 'MED8', 'MED9', 'MED10', 'MED11', 'MED12', 'MED12L', 'MED13', 'MED13L', 'MED14', 'MED14OS', 'MED15', 'MED16', 'MED17', 'MED18', 'MED19', 'MED20', 'MED21', 'MED22', 'MED23', 'MED24', 'MED25', 'MED26', 'MED27', 'MED28', 'MED29', 'MED30', 'MED31', 'MEDAG', 'MEF2A', 'MEF2B', 'MEF2C', 'MEF2D', 'MEFV', 'MEGF6', 'MEGF8', 'MEGF9', 'MEGF10', 'MEGF11', 'MEI1', 'MEI4', 'MEIG1', 'MEIKIN', 'MEIOB', 'MEIOC', 'MEIOSIN', 'MEIS1', 'MEIS2', 'MEIS3', 'MELK', 'MELTF', 'MEMO1', 'MEN1', 'MEOX1', 'MEOX2', 'MEP1A', 'MEP1B', 'MEPCE', 'MEPE', 'MERTK', 'MESD', 'MESP1', 'MESP2', 'MEST', 'MET', 'METAP1', 'METAP1D', 'METAP2', 'METRN', 'METRNL', 'METTL1', 'METTL2A', 'METTL2B', 'METTL3', 'METTL4', 'METTL5', 'METTL6', 'METTL7A', 'METTL7B', 'METTL8', 'METTL9', 'METTL13', 'METTL14', 'METTL15', 'METTL16', 'METTL17', 'METTL18', 'METTL21A', 'METTL21C', 'METTL22', 'METTL23', 'METTL24', 'METTL25', 'METTL25B', 'METTL26', 'METTL27', 'MEX3A', 'MEX3B', 'MEX3C', 'MEX3D', 'MFAP1', 'MFAP2', 'MFAP3', 'MFAP3L', 'MFAP4', 'MFAP5', 'MFF', 'MFGE8', 'MFHAS1', 'MFN1', 'MFN2', 'MFNG', 'MFRP', 'MFSD1', 'MFSD2A', 'MFSD2B', 'MFSD3', 'MFSD4A', 'MFSD4B', 'MFSD5', 'MFSD6', 'MFSD6L', 'MFSD8', 'MFSD9', 'MFSD10', 'MFSD11', 'MFSD12', 'MFSD13A', 'MFSD14A', 'MFSD14B', 'MFSD14C', 'MGA', 'MGAM', 'MGAM2', 'MGARP', 'MGAT1', 'MGAT2', 'MGAT3', 'MGAT4A', 'MGAT4B', 'MGAT4C', 'MGAT4D', 'MGAT5', 'MGAT5B', 'MGLL', 'MGME1', 'MGMT', 'MGP', 'MGRN1', 'MGST1', 'MGST2', 'MGST3', 'MIA', 'MIA2', 'MIA3', 'MIB1', 'MIB2', 'MICA', 'MICAL1', 'MICAL2', 'MICAL3', 'MICALCL', 'MICALL1', 'MICALL2', 'MICB', 'MICOS10', 'MICOS13', 'MICU1', 'MICU2', 'MICU3', 'MID1', 'MID1IP1', 'MID2', 'MIDEAS', 'MIDN', 'MIEF1', 'MIEF2', 'MIEN1', 'MIER1', 'MIER2', 'MIER3', 'MIF', 'MIF4GD', 'MIGA1', 'MIGA2', 'MIIP', 'MILR1', 'MINAR1', 'MINAR2', 'MINDY1', 'MINDY2', 'MINDY3', 'MINDY4', 'MINDY4B', 'MINK1', 'MINPP1', 'MIOS', 'MIOX', 'MIP', 'MIPEP', 'MIPOL1', 'MIS12', 'MIS18A', 'MIS18BP1', 'MISP', 'MISP3', 'MITD1', 'MITF', 'MIX23', 'MIXL1', 'MKI67', 'MKKS', 'MKLN1', 'MKNK1', 'MKNK2', 'MKRN1', 'MKRN2', 'MKRN2OS', 'MKRN3', 'MKS1', 'MKX', 'MLANA', 'MLC1', 'MLEC', 'MLF1', 'MLF2', 'MLH1', 'MLH3', 'MLIP', 'MLKL', 'MLLT1', 'MLLT3', 'MLLT6', 'MLLT10', 'MLLT11', 'MLN', 'MLNR', 'MLPH', 'MLST8', 'MLX', 'MLXIP', 'MLXIPL', 'MLYCD', 'MMAA', 'MMAB', 'MMACHC', 'MMADHC', 'MMD', 'MMD2', 'MME', 'MMEL1', 'MMGT1', 'MMP1', 'MMP2', 'MMP3', 'MMP7', 'MMP8', 'MMP9', 'MMP10', 'MMP11', 'MMP12', 'MMP13', 'MMP14', 'MMP15', 'MMP16', 'MMP17', 'MMP19', 'MMP20', 'MMP21', 'MMP23B', 'MMP24', 'MMP24OS', 'MMP25', 'MMP26', 'MMP27', 'MMP28', 'MMRN1', 'MMRN2', 'MMS19', 'MMS22L', 'MMUT', 'MN1', 'MNAT1', 'MND1', 'MNDA', 'MNS1', 'MNT', 'MNX1', 'MOAP1', 'MOB1A', 'MOB1B', 'MOB2', 'MOB3A', 'MOB3B', 'MOB3C', 'MOB4', 'MOBP', 'MOCOS', 'MOCS1', 'MOCS2', 'MOCS3', 'MOG', 'MOGAT1', 'MOGAT2', 'MOGAT3', 'MOGS', 'MOK', 'MON1A', 'MON1B', 'MON2', 'MORC1', 'MORC2', 'MORC3', 'MORC4', 'MORF4L1', 'MORF4L2', 'MORN1', 'MORN2', 'MORN3', 'MORN4', 'MORN5', 'MOS', 'MOSMO', 'MOSPD1', 'MOSPD2', 'MOSPD3', 'MOV10', 'MOV10L1', 'MOXD1', 'MPC1', 'MPC1L', 'MPC2', 'MPDU1', 'MPDZ', 'MPEG1', 'MPG', 'MPHOSPH6', 'MPHOSPH8', 'MPHOSPH9', 'MPHOSPH10', 'MPI', 'MPIG6B', 'MPL', 'MPLKIP', 'MPND', 'MPO', 'MPP1', 'MPP2', 'MPP3', 'MPP4', 'MPP7', 'MPPE1', 'MPPED1', 'MPPED2', 'MPRIP', 'MPST', 'MPV17', 'MPV17L', 'MPV17L2', 'MPZ', 'MPZL1', 'MPZL2', 'MPZL3', 'MR1', 'MRAP', 'MRAP2', 'MRAS', 'MRC1', 'MRC2', 'MRE11', 'MREG', 'MRFAP1', 'MRFAP1L1', 'MRGBP', 'MRGPRD', 'MRGPRE', 'MRGPRF', 'MRGPRG', 'MRGPRX1', 'MRGPRX2', 'MRGPRX3', 'MRGPRX4', 'MRI1', 'MRLN', 'MRM1', 'MRM2', 'MRM3', 'MRNIP', 'MRO', 'MROH1', 'MROH2A', 'MROH2B', 'MROH5', 'MROH6', 'MROH7', 'MROH8', 'MROH9', 'MRPL1', 'MRPL2', 'MRPL3', 'MRPL4', 'MRPL9', 'MRPL10', 'MRPL11', 'MRPL12', 'MRPL13', 'MRPL14', 'MRPL15', 'MRPL16', 'MRPL17', 'MRPL18', 'MRPL19', 'MRPL20', 'MRPL21', 'MRPL22', 'MRPL23', 'MRPL24', 'MRPL27', 'MRPL28', 'MRPL30', 'MRPL32', 'MRPL33', 'MRPL34', 'MRPL35', 'MRPL36', 'MRPL37', 'MRPL38', 'MRPL39', 'MRPL40', 'MRPL41', 'MRPL42', 'MRPL43', 'MRPL44', 'MRPL45', 'MRPL46', 'MRPL47', 'MRPL48', 'MRPL49', 'MRPL50', 'MRPL51', 'MRPL52', 'MRPL53', 'MRPL54', 'MRPL55', 'MRPL57', 'MRPL58', 'MRPS2', 'MRPS5', 'MRPS6', 'MRPS7', 'MRPS9', 'MRPS10', 'MRPS11', 'MRPS12', 'MRPS14', 'MRPS15', 'MRPS16', 'MRPS17', 'MRPS18A', 'MRPS18B', 'MRPS18C', 'MRPS21', 'MRPS22', 'MRPS23', 'MRPS24', 'MRPS25', 'MRPS26', 'MRPS27', 'MRPS28', 'MRPS30', 'MRPS31', 'MRPS33', 'MRPS34', 'MRPS35', 'MRPS36', 'MRRF', 'MRS2', 'MRTFA', 'MRTFB', 'MRTO4', 'MS4A1', 'MS4A2', 'MS4A3', 'MS4A4A', 'MS4A4E', 'MS4A5', 'MS4A6A', 'MS4A6E', 'MS4A7', 'MS4A8', 'MS4A10', 'MS4A12', 'MS4A13', 'MS4A14', 'MS4A15', 'MS4A18', 'MSANTD1', 'MSANTD2', 'MSANTD3', 'MSANTD4', 'MSANTD5', 'MSC', 'MSGN1', 'MSH2', 'MSH3', 'MSH4', 'MSH5', 'MSH6', 'MSI1', 'MSI2', 'MSL1', 'MSL2', 'MSL3', 'MSLN', 'MSMB', 'MSMO1', 'MSMP', 'MSN', 'MSR1', 'MSRA', 'MSRB1', 'MSRB2', 'MSRB3', 'MSS51', 'MST1', 'MST1R', 'MSTN', 'MSTO1', 'MSX1', 'MSX2', 'MT-ATP6', 'MT-ATP8', 'MT-CO1', 'MT-CO2', 'MT-CO3', 'MT-CYB', 'MT-ND1', 'MT-ND2', 'MT-ND3', 'MT-ND4', 'MT-ND4L', 'MT-ND5', 'MT-ND6', 'MT1A', 'MT1B', 'MT1E', 'MT1F', 'MT1G', 'MT1H', 'MT1HL1', 'MT1M', 'MT1X', 'MT2A', 'MT3', 'MT4', 'MTA1', 'MTA2', 'MTA3', 'MTAP', 'MTARC1', 'MTARC2', 'MTBP', 'MTCH1', 'MTCH2', 'MTCL1', 'MTCP1', 'MTDH', 'MTERF1', 'MTERF2', 'MTERF3', 'MTERF4', 'MTF1', 'MTF2', 'MTFMT', 'MTFP1', 'MTFR1', 'MTFR1L', 'MTFR2', 'MTG1', 'MTG2', 'MTHFD1', 'MTHFD1L', 'MTHFD2', 'MTHFD2L', 'MTHFR', 'MTHFS', 'MTHFSD', 'MTIF2', 'MTIF3', 'MTLN', 'MTM1', 'MTMR1', 'MTMR2', 'MTMR3', 'MTMR4']
#ended at GIN1 from EOLA1

'''
MOST COMMON "UNCOMMON" SCENARIOS ENCOUNTERED (FEEL FREE TO ADD MORE HERE)


expect GAGE4 to have a genecards.org page about it, but no ensembl code and is unsearchable on opentargets.org, 
therefore no page about it exists on opentargets.org
    solved by returning alias list but returning empty list for diseases/conditions
    
expect NALF1 to not have a genecards.org page about it, (implied) no ensembl code and therefore no opentargets.org page
    solved by default returning empty list for both aliases and diseases/conditions

expect GREP1 to have a genecards.org page about it, no ensembl code but it *is* searchable on opentargets.org,
therefore there is an opentargets.org page for it
    solved by returning the aliases list and diseases/conditions as usual
'''

print(len(doggie))
driver = webdriver.Firefox(executable_path=r'C:\Users\Kevin\geckodriver-v0.29.1-win64\geckodriver.exe')
rajeevram = [] #list for diseases/conditions
joesalisbury = [] #list for aliases
cat = 0

while len(doggie)!=0:
    conditions, aliases, name, genecardslink, opentargetslink, search, driver = getopentargets(doggie[0], driver)
    cat = cat+1
    rajeevram.append(conditions)
    joesalisbury.append(aliases)
    #print(doggie[0] + " " + name + " " + str(cat))
    #print(conditions)
    print(str(cat) + ": " + name)
    if cat%250 == 0:
        print("Aliases")
        print(joesalisbury)
        print()
        print("Conditions")
        print(rajeevram)
    doggie.pop(0)

driver.close()

print()
print("Aliases")
print(joesalisbury)

print()
print("Conditions")
print(rajeevram)

In [ ]:
import pandas as pd

owo = [['Endothelium And Lymphocyte Associated ASCH Domain 1', 'Endothelial-Overexpressed Lipopolysaccharide-Associated Factor 1', 'Protein EOLA1', 'CXorf40A', 'CXorf40', 'Chromosome X Open Reading Frame 40A', 'Chromosome X Open Reading Frame 40', 'Protein CXorf40A', 'EOLA1'], ['Endothelium And Lymphocyte Associated ASCH Domain 2', 'Protein CXorf40B', 'Protein EOLA2', 'CXorf40B', 'Chromosome X Open Reading Frame 40B', 'EOLA2'], ['Eomesodermin', 'TBR2', 'T-Box Brain Protein 2', 'Eomesodermin Homolog', 'Eomesodermin (Xenopus Laevis) Homolog', 'T-Box Brain2', 'T-Brain-2', 'EOMES', 'TBR-2'], ['E1A Binding Protein P300', 'Histone Acetyltransferase P300', 'P300', 'Protein 2-Hydroxyisobutyryltransferase P300', 'Protein Propionyltransferase P300', 'Histone Crotonyltransferase P300', 'Histone Butyryltransferase P300', 'E1A-Associated Protein P300', 'EC 2.3.1.48', 'P300 HAT', 'KAT3B', 'E1A-Binding Protein, 300kD', 'EC 2.3.1.-', 'EC 2.3.1', 'MKHK2', 'RSTS2', 'EP300'], ['E1A Binding Protein P400', 'CAGH32', 'Trinucleotide Repeat-Containing Gene 12 Protein', 'Trinucleotide Repeat Containing 12', 'P400 KDa SWI2/SNF2-Related Protein', 'E1A-Binding Protein P400', 'CAG Repeat Protein 32', 'Domino Homolog', 'HDomino(P400)', 'KIAA1498', 'KIAA1818', 'HDomino', 'TNRC12', 'P400', 'P400 SWI2/SNF2-Related Protein', 'DKFZP434I225', 'EC 3.6.4.-', 'EC 3.6.1', 'EP400'], ['Endothelial PAS Domain Protein 1', 'BHLHe73', 'HIF2A', 'PASD2', 'MOP2', 'HLF', 'Endothelial PAS Domain-Containing Protein 1', 'Class E Basic Helix-Loop-Helix Protein 73', 'Basic-Helix-Loop-Helix-PAS Protein MOP2', 'PAS Domain-Containing Protein 2', 'HIF-1-Alpha-Like Factor', 'Member Of PAS Protein 2', 'HIF-2-Alpha', 'HIF2-Alpha', 'EPAS-1', 'Hypoxia-Inducible Factor 2 Alpha', 'Hypoxia-Inducible Factor 2-Alpha', 'HIF-1 Alpha-Like Factor', 'HIF-1alpha-Like Factor', 'BHLHE73', 'ECYT4', 'EPAS1'], ['Erythrocyte Membrane Protein Band 4.1', '4.1R', 'Elliptocytosis 1, RH-Linked', 'Protein 4.1', 'Band 4.1', 'EPB4.1', 'P4.1', 'Erythrocyte Surface Protein Band 4.1', 'EPB41', 'E41P', 'EL1', 'HE'], ['Erythrocyte Membrane Protein Band 4.1 Like 1', '4.1N', 'Band 4.1-Like Protein 1', 'Neuronal Protein 4.1', 'KIAA0338', 'Erythrocyte Membrane Protein Band 4.1-Like 1', 'Neuron-Type Nonerythroid Protein 4.1', 'EPB41L1', 'MRD11'], ['Erythrocyte Membrane Protein Band 4.1 Like 2', '4.1G', 'Generally Expressed Protein 4.1', 'Band 4.1-Like Protein 2', '4.1-G', 'Erythrocyte Membrane Protein Band 4.1 Like-Protein 2', 'Erythrocyte Membrane Protein Band 4.1-Like 2', 'EPB41L2'], ['Erythrocyte Membrane Protein Band 4.1 Like 3', '4.1B', 'DAL1', 'Differentially Expressed In Adenocarcinoma Of The Lung Protein 1', 'Band 4.1-Like Protein 3', 'KIAA0987', 'DAL-1', 'Erythrocyte Membrane Protein Band 4.1-Like 3', 'EPB41L3'], ['Erythrocyte Membrane Protein Band 4.1 Like 4A', 'Band 4.1-Like Protein 4A', 'EPB41L4', 'NBL4', 'Erythrocyte Membrane Protein Band 4.1-Like 4A', 'Erythrocyte Protein Band 4.1-Like 4', 'Protein NBL4', 'EPB41L4A'], ['Erythrocyte Membrane Protein Band 4.1 Like 4B', 'LULU2', 'EHM2', 'FERM-Containing Protein CG1', 'Band 4.1-Like Protein 4B', 'Erythrocyte Membrane Protein Band 4.1-Like 4B', 'Protein EHM2', 'EPB41L4B', 'CG1'], ['Erythrocyte Membrane Protein Band 4.1 Like 5', 'Band 4.1-Like Protein 5', 'KIAA1548', 'LULU1', 'BE37', 'LULU', 'YMO1', 'YRT', 'Erythrocyte Membrane Protein Band 4.1-Like 5', 'Yurt Homolog (Drosophila)', 'Yurt Homolog', 'FLJ12957', 'EPB41L5'], ['Erythrocyte Membrane Protein Band 4.2', 'Erythrocyte Surface Protein Band 4.2', 'Erythrocyte Protein 4.2', 'Protein 4.2', 'P4.2', 'PA', 'MGC116735', 'MGC116737', 'EPB42', 'SPH5', 'E42P'], ['Enhancer Of Polycomb Homolog 1', 'Epl1', 'Enhancer Of Polycomb Homolog 1 (Drosophila)', 'EPC1'], ['Enhancer Of Polycomb Homolog 2', 'Enhancer Of Polycomb Homolog 2 (Drosophila)', 'DKFZP566F2124', 'EPC-LIKE', 'EPC-Like', 'EPC2'], ['Epithelial Cell Adhesion Molecule', 'Tumor-Associated Calcium Signal Transducer 1', 'TROP1', 'KSA', 'Major Gastrointestinal Tumor-Associated Protein GA733-2', 'Trophoblast Cell Surface Antigen 1', 'Adenocarcinoma-Associated Antigen', 'Cell Surface Glycoprotein Trop-1', 'Epithelial Glycoprotein 314', 'TACSTD1', 'GA733-2', 'EGP314', 'Ep-CAM', 'EGP-2', 'EGP40', 'KS1/4', 'MIC18', 'M4S1', 'MK-1', 'ESA', 'Membrane Component, Chromosome 4, Surface Marker (35kD Glycoprotein)', 'Antigen Identified By Monoclonal Antibody AUA1', 'Human Epithelial Glycoprotein-2', 'Epithelial Cell Surface Antigen', 'Epithelial Glycoprotein', 'KS 1/4 Antigen', 'CD326 Antigen', 'TACST-1', 'HEGP314', 'HNPCC8', 'HEA125', '323/A3', 'CO-17A', 'DIAR5', 'EPCAM', 'EGP34', 'CD326', 'MOC31', '17-1A', 'Ly74', 'MH99', 'M1S2', 'EGP'], ['Ependymin Related 1', 'MERP-1', 'MERP1', 'UCC1', 'Upregulated In Colorectal Cancer Gene 1 Protein', 'Mammalian Ependymin-Related Protein 1', 'EPDR', 'Ependymin Related Protein 1 (Zebrafish)', 'Mammalian Ependymin Related Protein 1', 'Ependymin Related Protein 1', 'EPDR1'], ['Ectopic P-Granules Autophagy Protein 5 Homolog', 'KIAA1632', 'Ectopic P Granules Protein 5 Homolog', 'Ectopic P-Granules Autophagy Protein 5 Homolog (C. Elegans)', 'HEEW1', 'VICIS', 'HEPG5', 'EPG5'], ['Epithelial Mitogen', 'Epigen', 'EPG', 'ALGV3072', 'PRO9904', 'Epithelial Mitogen Homolog (Mouse)', 'Epithelial Mitogen Homolog', 'EPGN'], ['EPH Receptor A1', 'EPH', 'Erythropoietin-Producing Hepatoma Receptor', 'Tyrosine-Protein Kinase Receptor EPH', 'Ephrin Type-A Receptor 1', 'EC 2.7.10.1', 'HEpha1', 'EPHT1', 'EPHT', 'Erythropoietin-Producing Hepatoma Amplified Sequence', 'Soluble EPHA1 Variant 1', 'Soluble EPHA1 Variant 2', 'Eph Tyrosine Kinase 1', 'EPH Tyrosine Kinase 1', 'EPH Tyrosine Kinase', 'Oncogene EPH', 'EC 2.7.10', 'EphA1', 'EPHA1'], ['EPH Receptor A2', 'Tyrosine-Protein Kinase Receptor ECK', 'Ephrin Type-A Receptor 2', 'EC 2.7.10.1', 'ECK', 'Epithelial Cell Receptor Protein Tyrosine Kinase', 'Soluble EPHA2 Variant 1', 'Epithelial Cell Kinase', 'EC 2.7.10', 'CTRCT6', 'EphA2', 'ARCC2', 'CTPP1', 'EPHA2', 'CTPA'], ['EPH Receptor A3', 'HEK4', 'HEK', 'Tyrosine-Protein Kinase Receptor ETK1', 'Eph-Like Tyrosine Kinase 1', 'Ephrin Type-A Receptor 3', 'EPH-Like Kinase 4', 'EC 2.7.10.1', 'TYRO4', 'ETK1', 'EK4', 'ETK', 'Testicular Tissue Protein Li 64', 'TYRO4 Protein Tyrosine Kinase', 'Tyrosine-Protein Kinase TYRO4', 'Human Embryo Kinase 1', 'Human Embryo Kinase', 'EC 2.7.10', 'EphA3', 'EPHA3'], ['EPH Receptor A4', 'HEK8', 'Tyrosine-Protein Kinase Receptor SEK', 'Tyrosine-Protein Kinase TYRO1', 'Ephrin Type-A Receptor 4', 'EPH-Like Kinase 8', 'EC 2.7.10.1', 'TYRO1', 'EK8', 'SEK', 'Receptor Protein-Tyrosine Kinase HEK8', 'TYRO1 Protein Tyrosine Kinase', 'EC 2.7.10', 'EphA4', 'EPHA4', 'Hek8'], ['EPH Receptor A5', 'TYRO4', 'EHK1', 'HEK7', 'Ephrin Type-A Receptor 5', 'EPH Homology Kinase 1', 'Brain-Specific Kinase', 'EPH-Like Kinase 7', 'EC 2.7.10.1', 'EHK-1', 'CEK7', 'EK7', 'Epididymis Secretory Sperm Binding Protein', 'Tyrosine-Protein Kinase Receptor EHK-1', 'Receptor Protein-Tyrosine Kinase HEK7', 'EC 2.7.10', 'EphA5', 'EPHA5', 'Hek7', 'BSK'], ['EPH Receptor A6', 'Ephrin Type-A Receptor 6', 'EPH Homology Kinase 2', 'EPH-Like Kinase 12', 'EC 2.7.10.1', 'EHK-2', 'HEK12', 'EHK2', 'EK12', 'Ephrin Receptor EphA6', 'EC 2.7.10', 'PRO57066', 'FLJ35246', 'EPHA6', 'EPA6'], ['EPH Receptor A7', 'HEK11', 'Ephrin Type-A Receptor 7', 'EPH Homology Kinase 3', 'EPH-Like Kinase 11', 'EC 2.7.10.1', 'EHK-3', 'EHK3', 'EK11', 'Receptor Protein-Tyrosine Kinase HEK11', 'Tyrosine-Protein Kinase Receptor EHK-3', 'Eph Homology Kinase-3', 'EC 2.7.10', 'EphA7', 'EPHA7', 'Hek11'], ['EPH Receptor A8', 'HEK3', 'Tyrosine-Protein Kinase Receptor EEK', 'EPH- And ELK-Related Kinase', 'Ephrin Type-A Receptor 8', 'EPH-Like Kinase 3', 'EC 2.7.10.1', 'EEK', 'EK3', 'EPH- And ELK-Related Tyrosine Kinase', 'Hydroxyaryl-Protein Kinase', 'Protein-Tyrosine Kinase', 'Tyrosylprotein Kinase', 'EC 2.7.10', 'KIAA1459', 'EphA8', 'EPHA8', 'Hek3'], ['EPH Receptor A10', 'Ephrin Type-A Receptor 10', 'EC 2.7.10.1', 'EphA10s Protein', 'FLJ16103', 'FLJ33655', 'EPHA10'], ['EPH Receptor B1', 'Neuronally-Expressed EPH-Related Tyrosine Kinase', 'Tyrosine-Protein Kinase Receptor EPH-2', 'Ephrin Type-B Receptor 1', 'EPH-Like Kinase 6', 'EC 2.7.10.1', 'EPHT2', 'Hek6', 'HEK6', 'ELK', 'NET', 'EK6', 'Soluble EPHB1 Variant 1', 'Eph Tyrosine Kinase 2', 'EPH Tyrosine Kinase 2', 'EC 2.7.10', 'EphB1', 'EPHB1'], ['EPH Receptor B2', 'Developmentally-Regulated Eph-Related Tyrosine Kinase', 'Tyrosine-Protein Kinase Receptor EPH-3', 'Renal Carcinoma Antigen NY-REN-47', 'Tyrosine-Protein Kinase TYRO5', 'Ephrin Type-B Receptor 2', 'EPH-Like Kinase 5', 'EC 2.7.10.1', 'EPHT3', 'Tyro5', 'Hek5', 'HEK5', 'DRT', 'EK5', 'ERK', 'Protein-Tyrosine Kinase HEK5', 'Elk-Related Tyrosine Kinase', 'ELK-Related Tyrosine Kinase', 'Eph Tyrosine Kinase 3', 'EPH Tyrosine Kinase 3', 'EC 2.7.10', 'BDPLT22', 'EphB2', 'EPHB2', 'EPTH3', 'TYRO5', 'CAPB', 'PCBC'], ['EPH Receptor B3', 'HEK2', 'Tyrosine-Protein Kinase TYRO6', 'EPH-Like Tyrosine Kinase 2', 'Ephrin Type-B Receptor 3', 'Embryonic Kinase 2', 'EC 2.7.10.1', 'TYRO6', 'ETK2', 'EK2', 'Human Embryo Kinase 2', 'EPH-Like Kinase 2', 'EC 2.7.10', 'EphB3', 'EPHB3', 'Tyro6', 'Hek2'], ['EPH Receptor B4', 'Tyrosine-Protein Kinase TYRO11', 'Hepatoma Transmembrane Kinase', 'Ephrin Type-B Receptor 4', 'EC 2.7.10.1', 'TYRO11', 'MYK1', 'HTK', 'Tyrosine-Protein Kinase Receptor HTK', 'Soluble EPHB4 Variant 1', 'Soluble EPHB4 Variant 2', 'Soluble EPHB4 Variant 3', 'Ephrin Receptor EphB4', 'EC 2.7.10', 'CMAVM2', 'LMPHM7', 'Tyro11', 'EphB4', 'HFASD', 'EPHB4'], ['EPH Receptor B6', 'HEP', 'Tyrosine-Protein Kinase-Defective Receptor EPH-6', 'Ephrin Type-B Receptor 6', 'Human Kinase-Defective Eph-Family Receptor Protein', 'EC 2.7.10.1', 'EphB6', 'EPHB6'], ['Epoxide Hydrolase 1', 'Epoxide Hydrolase 1, Microsomal (Xenobiotic)', 'Epoxide Hydratase', 'EPHX', 'EPOX', 'MEH', 'Epoxide Hydrolase 1 Microsomal', 'Microsomal Epoxide Hydrolase', 'EC 3.3.2.9', 'EPHX1', 'HYL1'], ['Epoxide Hydrolase 2', 'Epoxide Hydrolase 2, Cytoplasmic', 'Bifunctional Epoxide Hydrolase 2', 'ABHD20', 'Epoxide Hydrolase 2, Cytosolic', 'Epoxide Hydrolase, Soluble', 'Epoxide Hydratase', 'EPHX2', 'CEH', 'SEH'], ['Epoxide Hydrolase 3', 'Abhydrolase Domain-Containing Protein 9', 'Abhydrolase Domain Containing 9', 'ABHD9', 'EH3', 'EC 3.3.2.10', 'FLJ22408', 'EPHX3'], ['Epoxide Hydrolase 4', 'EPHXRP', 'EH4', 'Abhydrolase Domain-Containing Protein 7', 'Epoxide Hydrolase-Related Protein', 'Abhydrolase Domain Containing 7', 'ABHD7', 'EC 3.3.-.-', 'FLJ90341', 'EPHX4'], ['EPM2A Glucan Phosphatase, Laforin', 'Epilepsy, Progressive Myoclonus Type 2A, Lafora Disease (Laforin)', 'Epilepsy, Progressive Myoclonus Type 2, Lafora Disease (Laforin)', 'EPM2A, Laforin Glucan Phosphatase', 'Glycogen Phosphatase', 'Glucan Phosphatase', 'Lafora PTPase', 'LAFPTPase', 'Laforin', 'Laforin, Isoform 9', 'EC 3.1.3.16', 'EC 3.1.3.48', 'EC 3.1.3.-', 'EPM2A', 'EPM2', 'MELF', 'LDE', 'LD'], ['EPM2A Interacting Protein 1', 'EPM2A (Laforin) Interacting Protein 1', 'EPM2A-Interacting Protein 1', 'Laforin-Interacting Protein', 'KIAA0766', 'Laforin Interacting Protein 1', 'EPM2AIP1', 'FLJ11207'], ['Epsin 1', 'EH Domain-Binding Mitotic Phosphoprotein', 'EPS-15-Interacting Protein 1', 'Epsin-1', 'EPN1'], ['Epsin 2', 'EPS-15-Interacting Protein 2', 'Eps15 Binding Protein', 'KIAA1065', 'Epsin-2', 'EHB21', 'EPN2'], ['Epsin 3', 'EPS-15-Interacting Protein 3', 'Epsin-3', 'MGC129899', 'FLJ20778', 'EPN3'], ['Erythropoietin', 'Epoetin', 'EP', 'ECYT5', 'MVCD2', 'DBAL', 'EPO'], ['Elongin BC And Polycomb Repressive Complex 2 Associated Protein', 'PRR28', 'Elongin BC And Polycomb Repressive Complex 2-Associated Protein', 'Proline-Rich Protein 28', 'Proline Rich 28', 'C17orf96', 'Chromosome 17 Open Reading Frame 96', 'Uncharacterized Protein C17orf96', 'LOC100170841', 'EPOP'], ['Erythropoietin Receptor', 'EPO-R', 'Truncated Erythropoietin Receptor', 'EPOR'], ['Epididymal Peptidase Inhibitor', 'Epididymal Protease Inhibitor', 'WFDC7', 'CT71', 'WAP7', 'Serine Peptidase Inhibitor-Like, With Kunitz And WAP Domains 1 (Eppin)', 'WAP Four-Disulfide Core Domain Protein 7', 'Cancer/Testis Antigen 72', 'Cancer/Testis Antigen 71', 'Protease Inhibitor WAP7', 'DJ461P17.2', 'SPINLW1', 'Eppin', 'Serine Protease Inhibitor-Like, With Kunitz And WAP Domains 1 (Eppin)', 'Serine Protease Inhibitor-Like With Kunitz And WAP Domains 1', 'Epididymis Secretory Sperm Binding Protein', 'EPPIN1', 'EPPIN2', 'EPPIN3', 'EPPIN', 'CT72'], ['Epiplakin 1', 'Epidermal Autoantigen 450K', '450 KDa Epidermal Antigen', 'Epiplakin', 'EPIPL1', 'EPIPL', 'EPPK1'], ['Glutamyl-Prolyl-TRNA Synthetase 1', 'PARS', 'Bifunctional Glutamate/Proline--TRNA Ligase', 'Cell Proliferation-Inducing Gene 32 Protein', 'Bifunctional Aminoacyl-TRNA Synthetase', 'Glutamatyl-Prolyl-TRNA Synthetase', 'Glutamate TRNA Ligase', 'GLUPRORS', 'EARS', 'EPRS', 'QARS', 'QPRS', 'Proliferation-Inducing Gene 32 Protein', 'Proliferation-Inducing Protein 32', 'Glutamyl-Prolyl-TRNA Synthetase', 'Glutaminyl-TRNA Synthetase', 'Prolyl-TRNA Synthetase', 'Proline TRNA Ligase', 'Proline-TRNA Ligase', 'EC 6.1.1.15', 'EC 6.1.1.18', 'EC 6.1.1', 'HLD15', 'PIG32', 'EPRS1', 'GLNS'], ['Epidermal Growth Factor Receptor Pathway Substrate 8', 'Epidermal Growth Factor Receptor Kinase Substrate 8', 'DFNB102', 'EPS8'], ['EPS8 Like 1', 'DRC3', 'Epidermal Growth Factor Receptor Pathway Substrate 8-Related Protein 1', 'Epidermal Growth Factor Receptor Kinase Substrate 8-Like Protein 1', 'EPS8-Related Protein 1', 'EPS8-Like Protein 1', 'EPS8R1', 'FLJ20258', 'MGC23164', 'PP10566', 'MGC4642', 'EPS8L1'], ['EPS8 Like 2', 'Epidermal Growth Factor Receptor Pathway Substrate 8-Related Protein 2', 'Epidermal Growth Factor Receptor Kinase Substrate 8-Like Protein 2', 'EPS8-Related Protein 2', 'EPS8R2', 'EPS8-Like Protein 2', 'EPS8-Like 2', 'FLJ21935', 'FLJ22171', 'DFNB106', 'MGC3088', 'EPS8L2'], ['EPS8 Like 3', 'Epidermal Growth Factor Receptor Pathway Substrate 8-Related Protein 3', 'Epidermal Growth Factor Receptor Kinase Substrate 8-Like Protein 3', 'EPS8-Related Protein 3', 'EPS8-Like Protein 3', 'EPS8R3', 'EPS8-Like 3', 'FLJ21522', 'MGC16817', 'EPS8L3', 'HYPT5'], ['Epidermal Growth Factor Receptor Pathway Substrate 15', 'Epidermal Growth Factor Receptor Substrate 15', 'Protein AF-1p', 'AF-1P', 'MLLT5', 'AF1P', 'ALL1 Fused Gene From Chromosome 1', 'Protein Eps15', 'EPS15'], ['Epidermal Growth Factor Receptor Pathway Substrate 15 Like 1', 'Epidermal Growth Factor Receptor Substrate 15-Like 1', 'Eps15-Related Protein', 'EPS15R', 'Eps15R', 'Epidermal Growth Factor Receptor Pathway Substrate 15-Like 1', 'Epidermal Growth Factor Receptor Substrate EPS15R', 'EPS15L1'], ['Epithelial Stromal Interaction 1', 'Epithelial Stromal Interaction 1 (Breast)', 'Epithelial-Stromal Interaction Protein 1', 'BRESI1', 'Epithelial Stromal Interaction Protein 1', 'EC 3.1.4.16', 'EC 3.6.3.6', 'MGC29634', 'EPSTI1'], ['Eosinophil Peroxidase', 'EPO', 'EPP', 'EC 1.11.1.7', 'EPX-PEN', 'EC 1.11.1', 'EPXD', 'EPER', 'EPX'], ['Epiphycan', 'Dermatan Sulfate Proteoglycan 3', 'SLRR3B', 'Small Chondroitin/Dermatan Sulfate Proteoglycan', 'Epiphycan Proteoglycan', 'Proteoglycan-Lb', 'DSPG3', 'Pg-Lb', 'PGLB', 'Dermatan Sulphate Proteoglycan 3', 'PG-Lb', 'EPYC'], ['Equatorin', 'C9orf11', 'AFAF', 'Acrosome Formation Associated Factor', 'Acrosome Formation-Associated Factor', 'Sperm Acrosome Associated 8', 'SPACA8', 'Equatorin, Sperm Acrosome Associated', 'Acr Formation Associated Factor', 'EQTN'], ['Era Like 12S Mitochondrial RRNA Chaperone 1', 'Era (E. Coli G-Protein Homolog)-Like 1', 'GTPase Era, Mitochondrial', 'Conserved ERA-Like GTPase', 'ERA-Like Protein 1', 'HERA-B', 'ERA-W', 'H-ERA', 'CEGA', 'HERA', 'GTPase, Human Homolog Of E. Coli Essential Cell Cycle Protein Era', 'Era-Like 12S Mitochondrial RRNA Chaperone 1', 'GTP-Binding Protein Era Homolog', 'Era G-Protein-Like 1 (E. Coli)', 'Era G-Protein-Like 1', 'ERAL1A', 'ERAL1B', 'HERA-A', 'PRLTS6', 'ERAL1', 'ERA'], ['Endoplasmic Reticulum Aminopeptidase 1', 'Puromycin-Insensitive Leucyl-Specific Aminopeptidase', 'Adipocyte-Derived Leucine Aminopeptidase', 'PILS-AP', 'ARTS-1', 'A-LAP', 'Type 1 Tumor Necrosis Factor Receptor Shedding Aminopeptidase Regulator', 'Aminopeptidase Regulator Of TNFR1 Shedding', 'Aminopeptidase PILS', 'KIAA0525', 'APPILS', 'ERAAP1', 'ARTS1', 'Endoplasmic Reticulum Aminopeptidase Associated With Antigen Processing', 'CTD-2260A17.2', 'EC 3.4.11.3', 'EC 3.4.11.-', 'EC 3.4.11', 'PILSAP', 'ERAAP', 'ERAP1', 'ALAP'], ['Endoplasmic Reticulum Aminopeptidase 2', 'Leukocyte-Derived Arginine Aminopeptidase', 'L-RAP', 'LRAP', 'EC 3.4.11.3', 'EC 3.4.11.-', 'EC 3.4.11', 'ERAP2'], ['ES Cell Expressed Ras', 'V-Ha-Ras Harvey Rat Sarcoma Viral Oncogene Homolog Pseudogene', 'Embryonic Stem Cell-Expressed Ras', 'GTPase ERas', 'HRAS2', 'HRASP', 'E-Ras', 'V-Ha-Ras Harvey Rat Sarcoma Viral Oncogene Homolog 2', 'Small GTPase Protein E-Ras', 'EC 3.6.5.2', 'ERAS'], ['Erb-B2 Receptor Tyrosine Kinase 2', 'HER2', 'NEU', 'V-Erb-B2 Avian Erythroblastic Leukemia Viral Oncogene Homolog 2', 'Tyrosine Kinase-Type Cell Surface Receptor HER2', 'Neuro/Glioblastoma Derived Oncogene Homolog', 'Human Epidermal Growth Factor Receptor 2', 'Receptor Tyrosine-Protein Kinase ErbB-2', 'Metastatic Lymph Node Gene 19 Protein', 'Proto-Oncogene C-ErbB-2', 'Proto-Oncogene Neu', 'EC 2.7.10.1', 'MLN 19', 'CD340', 'HER-2', 'NGL', 'V-Erb-B2 Avian Erythroblastic Leukemia Viral Oncogene Homolog 2 (Neuro/Glioblastoma Derived Oncogene Homolog)', 'V-Erb-B2 Erythroblastic Leukemia Viral Oncogene Homolog 2, Neuro/Glioblastoma Derived Oncogene Homolog', 'V-Erb-B2 Avian Erythroblastic Leukemia Viral Oncoprotein 2', 'Neuroblastoma/Glioblastoma Derived Oncogene Homolog', 'C-Erb B2/Neu Protein', 'CD340 Antigen', 'HER-2/Neu', 'Herstatin', 'P185erbB2', 'EC 2.7.10', 'ERBB2', 'MLN19', 'TKR1'], ['Erb-B2 Receptor Tyrosine Kinase 3', 'HER3', 'V-Erb-B2 Avian Erythroblastic Leukemia Viral Oncogene Homolog 3', 'Tyrosine Kinase-Type Cell Surface Receptor HER3', 'Human Epidermal Growth Factor Receptor 3', 'Receptor Tyrosine-Protein Kinase ErbB-3', 'Proto-Oncogene-Like Protein C-ErbB-3', 'EC 2.7.10.1', 'Lethal Congenital Contracture Syndrome 2', 'P180-ErbB3', 'P45-SErbB3', 'P85-SErbB3', 'EC 2.7.10', 'MDA-BF-1', 'C-ErbB-3', 'C-ErbB3', 'ErbB3-S', 'ErbB-3', 'FERLK', 'LCCS2', 'ERBB3'], ['Erb-B2 Receptor Tyrosine Kinase 4', 'HER4', 'V-Erb-B2 Avian Erythroblastic Leukemia Viral Oncogene Homolog 4', 'Tyrosine Kinase-Type Cell Surface Receptor HER4', 'Human Epidermal Growth Factor Receptor 4', 'Receptor Tyrosine-Protein Kinase ErbB-4', 'Proto-Oncogene-Like Protein C-ErbB-4', 'EC 2.7.10.1', 'P180erbB4', 'ALS19', 'V-Erb-A Avian Erythroblastic Leukemia Viral Oncogene Homolog-Like 4', 'Avian Erythroblastic Leukemia Viral (V-Erb-B2) Oncogene Homolog 4', 'V-Erb-A Erythroblastic Leukemia Viral Oncogene Homolog 4', 'EC 2.7.10', 'ERBB4'], ['Erbb2 Interacting Protein', 'Densin-180-Like Protein', 'LAP2', 'Erbb2-Interacting Protein', 'Protein LAP2', 'ERBB2IP', 'Erbin', 'Epididymis Secretory Protein Li 78', 'ERBB2-Interacting Protein', 'HEL-S-78', 'KIAA1225', 'ERBIN'], ['ELKS/RAB6-Interacting/CAST Family Member 1', 'ELKS', 'ELKS/Rab6-Interacting/CAST Family Member 1', 'RAB6 Interacting Protein 2', 'KIAA1081', 'RAB6IP2', 'ERC-1', 'Rab6-Interacting Protein 2', 'MGC12974', 'Cast2', 'CAST2', 'ERC1'], ['ELKS/RAB6-Interacting/CAST Family Member 2', 'ERC Protein 2', 'KIAA0378', 'SPBC110', 'Spc110', 'CAST1', 'ELKSL', 'CAST', 'CAZ-Associated Structural Protein', 'Cytomatrix Protein P110', 'ERC2'], ['ERCC Excision Repair 1, Endonuclease Non-Catalytic Subunit', 'Excision Repair Cross-Complementing Rodent Repair Deficiency, Complementation Group 1 (Includes Overlapping Antisense Sequence)', 'Excision Repair Cross-Complementation Group 1', 'DNA Excision Repair Protein ERCC-1', 'RAD10', 'COFS4', 'ERCC1', 'UV20'], ['ERCC Excision Repair 2, TFIIH Core Complex Helicase Subunit', 'Excision Repair Cross-Complementing Rodent Repair Deficiency, Complementation Group 2', 'General Transcription And DNA Repair Factor IIH Helicase Subunit XPD', 'TFIIH Basal Transcription Factor Complex Helicase XPB Subunit', 'TFIIH Basal Transcription Factor Complex 80 KDa Subunit', 'Xeroderma Pigmentosum Group D-Complementing Protein', 'Excision Repair Cross-Complementation Group 2', 'Xeroderma Pigmentosum Complementary Group D', 'DNA Repair Protein Complementing XP-D Cells', 'Basic Transcription Factor 2 80 KDa Subunit', 'DNA Excision Repair Protein ERCC-2', 'TFIIH 80 KDa Subunit', 'TFIIH Subunit XPD', 'TFIIH P80', 'BTF2 P80', 'CXPD', 'EM9', 'XPD', 'Excision Repair Cross-Complementing Rodent Repair Deficiency, Complementation Group 2 Protein', 'TFIIH Basal Transcription Factor Complex Helicase XPD Subunit', 'TFIIH Basal Transcription Factor Complex Helicase Subunit', 'EC 3.6.4.12', 'MGC102762', 'MGC126218', 'MGC126219', 'EC 3.6.1', 'COFS2', 'TFIIH', 'ERCC2', 'TTD1', 'XPDC', 'TTD', 'MAG'], ['ERCC Excision Repair 3, TFIIH Core Complex Helicase Subunit', 'XPB', 'Excision Repair Cross-Complementing Rodent Repair Deficiency, Complementation Group 3', 'General Transcription And DNA Repair Factor IIH Helicase Subunit XPB', 'TFIIH Basal Transcription Factor Complex 89 KDa Subunit', 'Xeroderma Pigmentosum Group B-Complementing Protein', 'Excision Repair Cross-Complementation Group 3', 'DNA Repair Protein Complementing XP-B Cells', 'Basic Transcription Factor 2 89 KDa Subunit', 'DNA Excision Repair Protein ERCC-3', 'TFIIH 89 KDa Subunit', 'TFIIH Subunit XPB', 'TFIIH P89', 'BTF2 P89', 'RAD25', 'BTF2', 'Ssl2', 'TFIIH Basal Transcription Factor Complex Helicase XPB Subunit', 'Xeroderma Pigmentosum, Complementation Group B', 'Xeroderma Pigmentosum Group B Complementing', 'DNA Repair Helicase', 'EC 3.6.4.12', 'EC 3.6.1', 'GTF2H', 'TFIIH', 'ERCC3', 'TTD2', 'XPBC'], ['ERCC Excision Repair 4, Endonuclease Catalytic Subunit', 'Excision Repair Cross-Complementing Rodent Repair Deficiency, Complementation Group 4', 'Xeroderma Pigmentosum Group F-Complementing Protein', 'Xeroderma Pigmentosum, Complementation Group F', 'Excision Repair Cross-Complementation Group 4', 'DNA Repair Protein Complementing XP-F Cells', 'DNA Excision Repair Protein ERCC-4', 'DNA Repair Endonuclease XPF', 'ERCC11', 'FANCQ', 'RAD1', 'XPF', 'Excision-Repair, Complementing Defective, In Chinese Hamster', 'EC 3.1.-.-', 'XFEPS', 'ERCC4'], ['ERCC Excision Repair 5, Endonuclease', 'Excision Repair Cross-Complementing Rodent Repair Deficiency, Complementation Group 5', 'Xeroderma Pigmentosum, Complementation Group G', 'Excision Repair Cross-Complementation Group 5', 'DNA Repair Protein Complementing XP-G Cells', 'DNA Excision Repair Protein ERCC-5', 'ERCM2', 'XPGC', 'XPG', 'Xeroderma Pigmentosum Group G-Complementing Protein', 'XPG-Complementing Protein', 'Cockayne Syndrome', 'EC 3.1.-.-', 'ERCC5-201', 'COFS3', 'ERCC5', 'UVDR'], ['ERCC Excision Repair 6, Chromatin Remodeling Factor', 'CSB', 'Excision Repair Cross-Complementing Rodent Repair Deficiency, Complementation Group 6', 'Excision Repair Cross-Complementation Group 6', 'DNA Excision Repair Protein ERCC-6', 'Cockayne Syndrome Protein CSB', 'ATP-Dependent Helicase ERCC6', 'Chimeric ERCC6-PGBD3 Protein', 'Chimeric CSB-PGBD3 Protein', 'ARMD5', 'RAD26', 'Cockayne Syndrome Group B Protein', 'Cockayne Syndrome B Protein', 'ERCC6-PGBD3 Fusion Protein', 'EC 3.6.4.-', 'CSB-PGBD3', 'EC 3.6.1', 'COFS1', 'POF11', 'UVSS1', 'ERCC6', 'CKN2', 'COFS'], ['ERCC Excision Repair 6 Like, Spindle Assembly Checkpoint Helicase', 'PICH', 'Excision Repair Cross-Complementing Rodent Repair Deficiency, Complementation Group 6-Like', 'Excision Repair Cross-Complementation Group 6 Like', 'DNA Excision Repair Protein ERCC-6-Like', 'PLK1-Interacting Checkpoint Helicase', 'ATP-Dependent Helicase ERCC6-Like', 'Tumor Antigen BJ-HCC-15', 'RAD26L', 'Excision Repair Cross-Complementing Rodent Repair Deficiency Complementation Group 6 - Like', 'Plk1-Interacting Checkpoint Helicase', 'Excision Repair Protein ERCC6-Like', 'SNF2/RAD54 Family Protein', 'EC 3.6.4.12', 'FLJ20105', 'EC 3.6.1', 'ERCC6L'], ['ERCC Excision Repair 6 Like 2', 'RAD26L', 'Excision Repair Cross-Complementing Rodent Repair Deficiency, Complementation Group 6-Like 2', 'Excision Repair Cross-Complementation Group 6 Like 2', 'DNA Repair And Recombination Protein RAD26-Like', 'DNA Excision Repair Protein ERCC-6-Like 2', 'C9orf102', 'HEBO', 'Putative Repair And Recombination Helicase RAD26L', 'Chromosome 9 Open Reading Frame 102', 'Stretch Responsive Protein 278', 'EC 3.6.4.-', 'FLJ37706', 'EC 3.6.1', 'ERCC6L2', 'BMFS2', 'SR278'], ['ERCC Excision Repair 8, CSA Ubiquitin Ligase Complex Subunit', 'CSA', 'Excision Repair Cross-Complementing Rodent Repair Deficiency, Complementation Group 8', 'Excision Repair Cross-Complementation Group 8', 'Cockayne Syndrome WD Repeat Protein CSA', 'DNA Excision Repair Protein ERCC-8', 'CKN1', 'Cockayne Syndrome WD-Repeat Protein CSA', 'Cockayne Syndrome 1 (Classical)', 'UVSS2', 'ERCC8'], ['Epiregulin', 'Proepiregulin', 'ER', 'EREG', 'EPR', 'Ep'], ['ETS2 Repressor Factor', 'PE-2', 'ETS Domain-Containing Transcription Factor ERF', 'Ets2 Repressor Factor', 'PE2', 'CHYTS', 'CRS4', 'ERF'], ['Erythroferrone', 'Myonectin', 'C1QTNF15', 'CTRP15', 'Complement C1q Tumor Necrosis Factor-Related Protein 15', 'Family With Sequence Similarity 132, Member B', 'FAM132B', 'Protein FAM132B', 'FLJ37034', 'ERFE'], ['ETS Repressor Factor Like', 'ETS Domain-Containing Transcription Factor ERF-Like', 'Erfl1', 'ETS Domain-Containing Transcription Factor ERF', 'Ets2 Repressor Factor-Like', 'ETS Repressor Factor-Like', 'ERFL'], ['ETS Transcription Factor ERG', 'Transcriptional Regulator ERG (Transforming Protein ERG)', 'V-Ets Avian Erythroblastosis Virus E26 Oncogene Related', 'V-Ets Avian Erythroblastosis Virus E26 Oncogene Homolog', 'V-Ets Erythroblastosis Virus E26 Oncogene Like', 'Transcriptional Regulator ERG', 'Erg-3', 'P55', 'Erythroblast Transformation-Specific Transcription Factor ERG Variant 10', 'V-Ets Erythroblastosis Virus E26 Oncogene Homolog', 'TMPRSS2-ERG Prostate Cancer Specific', 'ERG, ETS Transcription Factor', 'Transforming Protein ERG', 'FUS/ERG Fusion Protein', 'TMPRSS2/ERG Fusion', 'ETS-Related Gene', 'Ets-Related', 'ERG'], ['Ergosterol Biosynthesis 28 Homolog', 'Ergosterol Biosynthetic Protein 28 Homolog', 'C14orf1', 'NET51', 'Probable Ergosterol Biosynthetic Protein 28', 'Chromosome 14 Open Reading Frame 1', 'ERG28'], ['Endoplasmic Reticulum-Golgi Intermediate Compartment 1', 'ERGIC-32', 'ERGIC32', 'Endoplasmic Reticulum-Golgi Intermediate Compartment (ERGIC) 1', 'Endoplasmic Reticulum-Golgi Intermediate Compartment Protein 1', 'ER-Golgi Intermediate Compartment 32 KDa Protein', 'KIAA1181', 'NET24', 'Endoplasmic Reticulum-Golgi Intermediate Compartment 32 KDa Protein', 'ERGIC1', 'AMC2', 'AMCN'], ['ERGIC And Golgi 2', 'PTX1', 'Endoplasmic Reticulum-Golgi Intermediate Compartment Protein 2', 'Erv41', 'CD14 Protein', 'ERGIC2', 'CDA14', 'Cd002', 'ERV41'], ['ERGIC And Golgi 3', 'Endoplasmic Reticulum-Golgi Intermediate Compartment Protein 3', 'Serologically Defined Breast Cancer Antigen NY-BR-84', 'Serologically Defined Breast Cancer Antigen 84', 'C20orf47', 'NY-BR-84', 'SDBCAG84', 'PRO0989', 'CGI-54', 'Erv46', 'Endoplasmic Reticulum-Localized Protein ERp43', 'Chromosome 20 Open Reading Frame 47', 'DJ477O4.2', 'C2orf47', 'ERGIC3', 'ERV46'], ['ERH MRNA Splicing And Mitosis Factor', 'Enhancer Of Rudimentary Homolog', 'DROER', 'Enhancer Of Rudimentary (Drosophila) Homolog', 'Enhancer Of Rudimentary Homolog (Drosophila)', 'ERH'], ['Exoribonuclease 1', "Histone MRNA 3'-End-Specific Exoribonuclease", 'Three Prime Histone MRNA Exonuclease 1', "Histone MRNA 3'-Exonuclease 1", "3'-5' Exoribonuclease 1", "3'-5' Exonuclease ERI1", "3'HEXO", 'THEX1', 'HEXO', 'Enhanced RNAi Three Prime MRNA Exonuclease Homolog 1 (C.Elegans)', 'Enhanced RNAi Three Prime MRNA Exonuclease Homolog 1', "Histone MRNA 3' End-Specific Exonuclease", 'Three Prime MRNA Exonuclease 1', "3' Exoribonuclease", "Protein 3'HExo", 'Eri-1 Homolog', 'EC 3.1.-.-', "3'EXO", 'ERI1'], ['ERI1 Exoribonuclease Family Member 2', 'Exonuclease Domain-Containing Protein 1', 'Zinc Finger, GRF-Type Containing 5', 'Exonuclease Domain Containing 1', 'ERI1 Exoribonuclease 2', 'Exoribonuclease 2', 'KIAA1504', 'EXOD1', 'ZGRF5', 'Enhanced RNAi Three Prime MRNA Exonuclease Homolog 2 (C.Elegans)', 'Enhanced RNAi Three Prime MRNA Exonuclease Homolog 2', 'EC 3.1.-.-', 'MGC16943', 'ERI2'], ['ERI1 Exoribonuclease Family Member 3', 'PINT1', 'Prion Protein-Interacting Protein', 'ERI1 Exoribonuclease 3', 'Prion Interactor 1', 'PRNPIP', 'Enhanced RNAi Three Prime MRNA Exonuclease Homolog 3 (C.Elegans)', 'Enhanced RNAi Three Prime MRNA Exonuclease Homolog 3', 'Prion Protein Interacting Protein', 'Exoribonuclease 3', 'EC 3.1.-.-', 'FLJ22943', 'PRNPIP1', 'ERI3'], ['Glutamate Rich 1', 'Glutamate-Rich Protein 1', 'Glutamate-Rich 1', 'HSPC319', 'ERICH1'], ['Glutamate Rich 2', 'Glutamate-Rich Protein 2', 'Glutamate-Rich 2', 'ERICH2'], ['Glutamate Rich 3', 'Glutamate-Rich Protein 3', 'RP11-653A5.1', 'C1orf173', 'Chromosome 1 Open Reading Frame 173', 'Glutamate-Rich 3', 'DKFZp547I048', 'ERICH3'], ['Glutamate Rich 4', 'Glutamate-Rich Protein 4', 'C19orf69', 'Chromosome 19 Open Reading Frame 69', 'Glutamate-Rich 4', 'LOC100170765', 'ERICH4'], ['Glutamate Rich 5', 'Glutamate-Rich Protein 5', 'C8orf47', 'Chromosome 8 Open Reading Frame 47', 'Glutamate-Rich 5', 'FLJ39553', 'ERICH5'], ['Glutamate Rich 6', 'Family With Sequence Similarity 194, Member A', 'Glutamate-Rich Protein 6', 'Glutamate-Rich 6A', 'Protein FAM194A', 'C3orf44', 'ERICH6A', 'FAM194A', 'Chromosome 3 Open Reading Frame 44', 'Glutamate-Rich 6', 'MGC39662', 'ERICH6'], ['Glutamate Rich 6B', 'Family With Sequence Similarity 194, Member B', 'Glutamate-Rich Protein 6B', 'Protein FAM194B', 'FAM194B', 'Glutamate-Rich 6B', 'FLJ32682', 'ERICH6B'], ['Endoplasmic Reticulum Lectin 1', 'XTP3TPB', 'XTP3-Transactivated Gene B Protein', 'Erlectin 1', 'ER Lectin', 'C2orf30', 'CL25084', 'XTP3-B', 'Cancer Invasion And Metastasis-Related', 'Chromosome 2 Open Reading Frame 30', 'Epididymis Luminal Protein 117', 'XTP3-Transactivated Protein B', 'ERLECTIN', 'Erlectin', 'CL24936', 'HEL117', 'ERLEC1', 'CIM'], ['ER Lipid Raft Associated 1', 'Erlin-1', 'KE04', 'Band_7 23-211 Keo4 (Interim) Similar To C.Elegans Protein C42C1.9', 'Stomatin-Prohibitin-Flotillin-HflC/K Domain-Containing Protein 1', 'Endoplasmic Reticulum Lipid Raft-Associated Protein 1', 'SPFH Domain-Containing Protein 1', 'SPFH Domain Family, Member 1', 'C10orf69', 'SPFH1', 'SPG62', 'KEO4', 'Chromosome 10 Open Reading Frame 69', 'Protein KE04', 'ERLIN1'], ['ER Lipid Raft Associated 2', 'Erlin-2', 'Stomatin-Prohibitin-Flotillin-HflC/K Domain-Containing Protein 2', 'Endoplasmic Reticulum Lipid Raft-Associated Protein 2', 'Spastic Paraplegia 18 (Autosomal Dominant)', 'SPFH Domain Family, Member 2', 'C8orf2', 'NET32', 'SPFH2', 'Epididymis Secretory Sperm Binding Protein', 'Chromosome 8 Open Reading Frame 2', 'SPFH Domain-Containing Protein 2', 'ERLIN2', 'SPG18'], ['Erythroblast Membrane Associated Protein (Scianna Blood Group)', 'Erythroid Membrane-Associated Protein', 'Scianna Blood Group Antigen', 'Radin Blood Group Antigen', 'BTN5', 'RD', 'SC', 'Erythroblast Membrane-Associated Protein (RD And SC Blood Groups)', 'Erythroblast Membrane-Associated Protein (Scianna Blood Group)', 'Erythroblast Membrane-Associated Protein', 'Scianna Blood Group (Sc)', 'Radin Blood Group (Rd)', 'Scianna Blood Group', 'Radin Blood Group', 'PRO2801', 'HERMAP', 'ERMAP'], ['ER Membrane Associated RNA Degradation', 'Endoplasmic Reticulum Membrane-Associated RNA Degradation Protein', 'DJ266L20.3', 'C6orf70', 'ER Membrane-Associated RNA Degradation Protein', 'Chromosome 6 Open Reading Frame 70', 'Transmembrane Protein C6orf70', 'FLJ11152', 'ERMARD', 'PVNH6'], ['Ermin', 'Juxtanodin', 'KIAA1189', 'JN', 'Ermin, ERM-Like Protein', 'ERMIN', 'ERMN'], ['Endoplasmic Reticulum Metallopeptidase 1', 'Felix-Ina', 'KIAA1815', 'FXNA', 'Aminopeptidase Fxna', 'BA207C16.3', 'EC 3.4.-.-', 'FLJ23309', 'ERMP1'], ['Endoplasmic Reticulum To Nucleus Signaling 1', 'Serine/Threonine-Protein Kinase/Endoribonuclease IRE1', 'IRE1', 'Inositol-Requiring Protein 1', 'Inositol-Requiring Enzyme 1', 'ER To Nucleus Signalling 1', 'Ire1-Alpha', 'HIRE1p', 'IRE1P', 'IRE1a', 'Endoplasmic Reticulum-To-Nucleus Signaling 1', 'Protein Kinase/Endoribonuclease', 'Inositol-Requiring 1', 'EC 2.7.11', 'ERN1'], ['Endoplasmic Reticulum To Nucleus Signaling 2', 'IRE1b', 'Serine/Threonine-Protein Kinase/Endoribonuclease IRE2', 'Inositol-Requiring Protein 2', 'ER To Nucleus Signalling 2', 'HIRE2p', 'Endoplasmic Reticulum To Nucleus Signalling 2', 'Endoplasmic Reticulum-To-Nucleus Signaling 2', 'IRE1, S. Cerevisiae, Homolog Of', 'Inositol-Requiring 1 Beta', 'IRE1-BETA', 'IRE1 Beta', 'Ire1-Beta', 'EC 2.7.11', 'IRE2p', 'ERN2', 'IRE2'], ['Endoplasmic Reticulum Oxidoreductase 1 Alpha', 'Endoplasmic Reticulum Oxidoreductase Alpha', 'Endoplasmic Oxidoreductin-1-Like Protein', 'ERO1-Like Protein Alpha', 'Oxidoreductin-1-L-Alpha', 'ERO1-L-Alpha', 'ERO1-Alpha', 'Ero1alpha', 'ERO1-L', 'ERO1L', 'ERO1 (S. Cerevisiae)-Like', 'ERO1-Like (S. Cerevisiae)', 'EC 1.8.4.-', 'EC 1.8.4', 'ERO1LA', 'ERO1A'], ['Endoplasmic Reticulum Oxidoreductase 1 Beta', 'Endoplasmic Reticulum Oxidoreductin-1-Like Protein B', 'Endoplasmic Reticulum Oxidoreductase Beta', 'ERO1-Like Protein Beta', 'Oxidoreductin-1-L-Beta', 'ERO1-L-Beta', 'Ero1beta', 'ERO1LB', 'Endoplasmic Oxidoreductin-1-Like Protein B', 'ERO1-Like Beta (S. Cerevisiae)', 'ERO1-Like Beta', 'ERO1-L(Beta)', 'EC 1.8.4.-', 'EC 1.8.4', 'ERO1B'], ['Endoplasmic Reticulum Protein 27', 'Protein Disulfide Isomerase Family A, Member 8', 'Endoplasmic Reticulum Resident Protein 27', 'Inactive Protein Disulfide-Isomerase 27', 'Endoplasmic Reticulum Protein 27 KDa', 'ER Protein 27', 'C12orf46', 'PDIA8', 'Chromosome 12 Open Reading Frame 46', 'FLJ32115', 'ERP27', 'ERp27'], ['Endoplasmic Reticulum Protein 29', 'ERp28', 'ERp31', 'Protein Disulfide Isomerase Family A, Member 9', 'Endoplasmic Reticulum Resident Protein 29', 'Endoplasmic Reticulum Resident Protein 28', 'Endoplasmic Reticulum Resident Protein 31', 'C12orf8', 'PDI-DB', 'PDIA9', 'Endoplasmic Reticulum Lumenal Protein ERp28', 'Epididymis Secretory Protein Li 107', 'Chromosome 12 Open Reading Frame 8', 'HEL-S-107', 'ERP29', 'ERP28', 'ERp29'], ['Endoplasmic Reticulum Protein 44', 'Thioredoxin Domain Containing 4 (Endoplasmic Reticulum)', 'Protein Disulfide Isomerase Family A, Member 10', 'Endoplasmic Reticulum Resident Protein 44', 'Thioredoxin Domain-Containing Protein 4', 'ER Protein 44', 'KIAA0573', 'PDIA10', 'TXNDC4', 'Endoplasmic Reticulum Resident Protein 44 KDa', 'Epididymis Secretory Sperm Binding Protein', 'ERP44', 'ERp44'], ['ERBB Receptor Feedback Inhibitor 1', 'MIG-6', 'Mitogen-Inducible Gene 6 Protein', 'GENE-33', 'MIG6', 'RALT', 'Receptor-Associated Late Transducer', 'ERRFI1'], ['Endogenous Retrovirus Group 3 Member 1, Envelope', 'Endogenous Retrovirus Group 3 Member 1 Env Polyprotein', 'HERV-R_7q21.2 Provirus Ancestral Env Polyprotein', 'Endogenous Retrovirus Group 3, Member 1', 'Endogenous Retroviral Sequence 3', 'ERV3-1 Envelope Protein', 'HERV-R Envelope Protein', 'ERV-R Envelope Protein', 'HERV-R', 'ERV-R', 'ERV3', 'EnvR', 'Endogenous Retroviral Sequence 3 (Includes Zinc Finger Protein H-Plk/HPF9)', 'Endogenous Retrovirus Group 3 Member 1', 'ERV-3 Envelope Protein', 'ERV3 Envelope Protein', 'Envelope Polyprotein', 'ERV3-1', 'HERVR', 'H-PLK', 'ERVR'], ['Endogenous Retrovirus Group FRD Member 1, Envelope', 'Syncytin-2', 'ERVFRDE1', 'HERV-FRD', 'HERV-FRD_6p24.1 Provirus Ancestral Env Polyprotein', 'Endogenous Retrovirus Group FRD, Member 1', 'Endogenous Retrovirus Group FRD Member 1', 'Envelope Polyprotein', 'Syncytin 2', 'HERV-W/FRD', 'EnvFRD', 'HERV-FRD Provirus Ancestral Env Polyprotein', 'GLLL6191', 'ERVFRD-1', 'UNQ6191'], ['Endogenous Retrovirus Group MER34 Member 1, Envelope', 'Endogenous Retrovirus Group MER34 Member 1 Env Polyprotein', 'HERV-MER_4q12 Provirus Ancestral Env Polyprotein', 'Endogenous Retrovirus Group MER34, Member 1', 'Endogenous Retroviral Envelope Protein HEMO', 'Human Endogenous MER34 ORF', 'EnvMER34', 'HEMO', 'Human Endogenous MER34 (Medium-Reiteration-Frequency-Family-34) Open Reading Frame', 'Endogenous MER34 (Medium-Reiteration-Frequency-Family-34) ORF', 'Endogenous Retrovirus Group MER34 Member 1', 'LP9056 Protein', 'ERVMER34-1'], ['Endogenous Retrovirus Group V Member 1, Envelope', 'ENVV1', 'Endogenous Retrovirus Group V Member 1 Env Polyprotein', 'HERV-V_19q13.41 Provirus Ancestral Env Polyprotein 1', 'Endogenous Retrovirus Group V, Member 1', 'HERV-V1', 'Endogenous Retrovirus Group V, Member 2', 'Endogenous Retrovirus Group V Member 1', 'Envelope Glycoprotein ENVV1', 'Envelope Protein ENVV1', 'FLJ32214', 'ERVV-1'], ['Endogenous Retrovirus Group V Member 2, Envelope', 'ENVV2', 'Endogenous Retrovirus Group V Member 2 Env Polyprotein', 'HERV-V_19q13.41 Provirus Ancestral Env Polyprotein 2', 'Endogenous Retrovirus Group V, Member 2', 'Endogenous Retrovirus Group V Member 2', 'Envelope Glycoprotein ENVV2', 'Endogenous Retrovirus HSV2', 'Envelope Protein ENVV2', 'HERV-V2', 'ERVV-2'], ['Endogenous Retrovirus Group W Member 1, Envelope', 'HERV-7q Envelope Protein', 'Syncytin-1', 'Enverin', 'Endogenous Retroviral Family W, Env(C7), Member 1', 'HERV-W_7q21.2 Provirus Ancestral Env Polyprotein', 'Endogenous Retrovirus Group W, Member 1', 'Endogenous Retrovirus Group W Member 1', 'HERV-Tryptophan Envelope Protein', 'Envelope Polyprotein GPr73', 'HERV-W Env Glycoprotein', 'HERV-W Envelope Protein', 'Envelope Glycoprotein', 'HERV-W-ENV', 'Syncytin', 'HERV-7q', 'ERVWE1', 'HERVW', 'Human Endogenous Retrovirus W EnvC7-1 Envelope Protein', 'HERV-W{7q21.1} Provirus Ancestral Env Polyprotein', 'Envelope Protein', 'HERVWENV', 'HERV7Q', 'ERVW-1', 'HERV-W', 'Env-W', 'ENVW', 'EnvW', 'ENV'], ['Endothelial Cell Adhesion Molecule', 'Endothelial Cell-Selective Adhesion Molecule', 'W117m', 'HUEL (C4orf1)-Interacting Protein', 'LP4791 Protein', '2310008D05Rik', 'ESAM'], ['Establishment Of Sister Chromatid Cohesion N-Acetyltransferase 1', 'EFO1', 'Establishment Of Cohesion 1 Homolog 1', 'Establishment Factor-Like Protein 1', 'N-Acetyltransferase ESCO1', 'CTF7 Homolog 1', 'ECO1 Homolog 1', 'ESO1 Homolog 1', 'KIAA1911', 'EFO1p', 'HEFO1', 'ESO1', 'Establishment Of Cohesion 1 Homolog 1 (S. Cerevisiae)', 'N-Acetyltransferase ESCO1 Variant 2', 'A930014I12Rik', 'EC 2.3.1.-', 'ESCO1', 'ECO1', 'CTF'], ['Establishment Of Sister Chromatid Cohesion N-Acetyltransferase 2', 'EFO2', 'Establishment Factor-Like Protein 2', 'N-Acetyltransferase ESCO2', 'ECO1 Homolog 2', 'EFO2p', 'HEFO2', 'Establishment Of Cohesion 1 Homolog 2 (S. Cerevisiae)', 'Establishment Of Cohesion 1 Homolog 2', 'Roberts Syndrome', '2410004I17Rik', 'EC 2.3.1.-', 'ESCO2', 'JHS', 'RBS'], ['Esterase D', 'S-Formylglutathione Hydrolase', 'Esterase D/Formylglutathione Hydrolase', 'Methylumbelliferyl-Acetate Deacetylase', 'EC 3.1.2.12', 'FGH', 'Testicular Tissue Protein Li 66', 'Esterase 10', 'EC 3.1.1.56', 'ESD'], ['ESF1 Nucleolar Pre-RRNA Processing Protein Homolog', 'ABT1-Associated Protein', 'ESF1 Homolog', 'BA526K24.1', 'C20orf6', 'ABTAP', 'ESF1, Nucleolar Pre-RRNA Processing Protein, Homolog (S. Cerevisiae)', 'Chromosome 20 Open Reading Frame 6', 'HDCMC28P', 'ESF1'], ['Endothelial Cell Specific Molecule 1', 'Endothelial Cell-Specific Molecule 1', 'ESM-1', 'Endocan', 'ESM1'], ['Extra Spindle Pole Bodies Like 1, Separase', 'Separin', 'ESP1', 'Extra Spindle Poles-Like 1 Protein', 'Separin, Cysteine Protease', 'Caspase-Like Protein ESPL1', 'Separase', 'KIAA0165', 'SEPA', 'Extra Spindle Pole Bodies Homolog 1 (S. Cerevisiae)', 'Extra Spindle Poles Like 1 (S. Cerevisiae)', 'Extra Spindle Pole Bodies 1, Separase', 'Extra Spindle Pole Bodies Homolog 1', 'Extra Spindle Poles Like 1', 'EC 3.4.22.49', 'ESPL1'], ['Espin', 'Autosomal Recessive Deafness Type 36 Protein', 'Ectoplasmic Specialization Protein', 'DFNB36', 'Deafness, Autosomal Recessive 36', 'LP2654', 'USH1M', 'ESPN'], ['Espin Like', 'Espin-Like Protein', 'FLJ42568', 'ESPNL'], ['Estrogen Receptor 1', 'Nuclear Receptor Subfamily 3 Group A Member 1', 'ER-Alpha', 'NR3A1', 'Oestrogen Receptor Alpha', 'Estradiol Receptor', 'E2 Receptor Alpha', 'Estrogen Receptor', 'ESR', 'Era', 'ER', 'Estrogen Receptor Alpha E1-N2-E2-1-2', 'Estrogen Receptor Alpha E1-E2-1-2', 'Estrogen Nuclear Receptor Alpha', 'Estrogen Receptor Alpha', 'ESTRR', 'ESRA', 'ESR1'], ['Estrogen Receptor 2', 'Nuclear Receptor Subfamily 3 Group A Member 2', 'Estrogen Receptor Beta', 'NR3A2', 'Oestrogen Receptor Beta', 'ER-Beta', 'ESTRB', 'Erb', 'Estrogen Receptor Beta Splice Variant, ERbeta2delta7', 'Estrogen Receptor Beta Splice Variant, ERbeta4delta7', 'Estrogen Receptor Beta Splice Variant, ERbeta6delta7', 'Estrogen Receptor Beta Splice Variant, ERbeta7delta7', 'Estrogen Receptor Beta Splice Variant, ERbetaEx. 4L', 'Estrogen Receptor Beta Splice Variant, ERbetaEx. 6L', 'Estrogen Receptor Beta Splice Variant, ERbeta6', 'Estrogen Receptor Beta Splice Variant, ERbeta7', 'Estrogen Receptor 2 (ER Beta)', 'Estrogen Receptor Beta 4', 'ESR-BETA', 'ER Beta', 'ER-BETA', 'ESRB', 'ODG8', 'ESR2'], ['Epithelial Splicing Regulatory Protein 1', 'RNA-Binding Motif Protein 35A', 'RNA-Binding Protein 35A', 'RBM35A', 'RNA Binding Motif Protein 35A', 'FLJ20171', 'DFNB109', 'RMB35A', 'ESRP1'], ['Epithelial Splicing Regulatory Protein 2', 'RNA-Binding Motif Protein 35B', 'RNA-Binding Protein 35B', 'RBM35B', 'RNA Binding Motif Protein 35B', 'RNA Binding Motif Protein 35A', 'FLJ21918', 'ESRP2'], ['Estrogen Related Receptor Alpha', 'NR3B1', 'ERR1', 'Nuclear Receptor Subfamily 3 Group B Member 1', 'Steroid Hormone Receptor ERR1', 'Estrogen Receptor-Like 1', 'ERR-Alpha', 'ERRalpha', 'ESRL1', 'ERRa', 'Estrogen-Related Nuclear Receptor Alpha', 'Estrogen-Related Receptor Alpha', 'ESRRA'], ['Estrogen Related Receptor Beta', 'NR3B2', 'Nuclear Receptor Subfamily 3 Group B Member 2', 'Steroid Hormone Receptor ERR2', 'Estrogen Receptor-Like 2', 'ERR Beta-2', 'ESRL2', 'ERR2', 'ERRb', 'Estrogen-Related Nuclear Receptor Beta', 'Deafness, Autosomal Recessive 35', 'Estrogen-Related Receptor Beta', 'Orphan Nuclear Receptor', 'Nuclear Receptor ERRB2', 'ERRbeta2', 'ERR-Beta', 'ERRbeta', 'DFNB35', 'ESRRB', 'ERRB2'], ['Estrogen Related Receptor Gamma', 'NR3B3', 'Nuclear Receptor Subfamily 3 Group B Member 3', 'Estrogen Receptor-Related Protein 3', 'Estrogen-Related Receptor Gamma', 'ERR Gamma-2', 'ERR-Gamma', 'ERR3', 'ERRg', 'ERRgamma', 'KIAA0832', 'ESRRG', 'ERRG2'], ['Ess-2 Splicing Factor Homolog', 'DGS-H', 'DGSI', 'ES2', 'DiGeorge Syndrome Critical Region Gene 13', 'DiGeorge Syndrome Critical Region Gene 14', 'Splicing Factor ESS-2 Homolog', 'DiGeorge Syndrome Protein H', 'DGCR13', 'DGCR14', 'ESS-2', 'Es2el', 'DGSH', 'Bis1', 'DiGeorge Syndrome Critical Region Gene DGSI', 'DiGeorge Syndrome Critical Region 13', 'DiGeorge Syndrome Critical Region 14', 'DiGeorge Syndrome Gene H', 'DiGeorge Syndrome Gene I', 'Protein DGCR13', 'Protein DGCR14', 'Protein ES2', 'DGS-I', 'ESS2'], ['ESX Homeobox 1', 'Homeobox Protein ESX1', 'ESX1L', 'ESXR1', 'Extraembryonic, Spermatogenesis, Homeobox 1 Homolog (Mouse)', 'Extraembryonic, Spermatogenesis, Homeobox 1 Homolog', 'Extraembryonic, Spermatogenesis, Homeobox 1', 'ESX1-Related Protein', 'ESX1R', 'ESX1'], ['Extended Synaptotagmin 1', 'MBC2', 'Family With Sequence Similarity 62 (C2 Domain Containing), Member A', 'Membrane-Bound C2 Domain-Containing Protein', 'Extended Synaptotagmin-1', 'KIAA0747', 'FAM62A', 'Extended Synaptotagmin-Like Protein 1', 'Extended Synaptotagmin Like Protein 1', 'Extended Synaptotagmin Protein 1', 'E-Syt1', 'ESYT1'], ['Extended Synaptotagmin 2', 'Family With Sequence Similarity 62 (C2 Domain Containing), Member B', 'Extended Synaptotagmin-2', 'KIAA1228', 'CHR2SYT', 'E-Syt2', 'FAM62B', 'Extended Synaptotagmin-Like Protein 2', 'Extended Synaptotagmin Like Protein 2', 'Extended Synaptotagmin Protein 2', 'Chr2 Synaptotagmin', 'Chr2Syt', 'ESYT2'], ['Extended Synaptotagmin 3', 'Family With Sequence Similarity 62 (C2 Domain Containing), Member C', 'Extended Synaptotagmin-3', 'CHR3SYT', 'E-Syt3', 'FAM62C', 'Extended Synaptotagmin-Like Protein 3', 'Extended Synaptotagmin Protein 3', 'Chr3 Synaptotagmin', 'Chr3Syt', 'ESYT3'], ['ETAA1 Activator Of ATR Kinase', "Ewing'S Tumor-Associated Antigen 1", 'ETAA16', "Ewing'S Tumor-Associated Antigen 16", 'ETAA1, ATR Kinase Activator', 'Ewing Tumor Associated Antigen 1', 'ETAA1'], ['Embryonic Testis Differentiation Homolog A', 'Embryonic Testis Differentiation Protein Homolog A', 'ETDA'], ['Embryonic Testis Differentiation Homolog B', 'Long Intergenic Non-Protein Coding RNA 633', 'Embryonic Testis Differentiation Protein Homolog B', 'Embryonic Testis Differentiation Protein', 'LINC00633', 'ETDA', 'ETDB'], ['Embryonic Testis Differentiation Homolog C', 'Embryonic Testis Differentiation Protein Homolog C', 'ETDC'], ['Eukaryotic Translation Termination Factor 1', 'ERF1', 'TB3-1', 'RF1', 'Sup45 (Yeast Omnipotent Suppressor 45) Homolog-Like 1', 'Eukaryotic Peptide Chain Release Factor Subunit 1', 'Polypeptide Chain Release Factor 1', 'Protein Cl1', 'SUP45L1', 'Eukaryotic Release Factor 1', 'D5S1995', 'ETF1', 'ERF'], ['Electron Transfer Flavoprotein Subunit Alpha', 'Electron Transfer Flavoprotein Subunit Alpha, Mitochondrial', 'Glutaric Aciduria II', 'Alpha-ETF', 'MADD', 'EMA', 'GA2', 'Electron-Transfer-Flavoprotein, Alpha Polypeptide', 'Electron Transfer Flavoprotein, Alpha Polypeptide', 'Electron Transfer Flavoprotein Alpha Subunit', 'Epididymis Secretory Sperm Binding Protein', 'ETFA'], ['Electron Transfer Flavoprotein Subunit Beta', 'Beta-ETF', 'Electron-Transferring-Flavoprotein, Beta Polypeptide', 'Electron-Transfer-Flavoprotein, Beta Polypeptide', 'Electron Transfer Flavoprotein, Beta Polypeptide', 'Electron Transfer Flavoprotein Beta Subunit', 'FP585', 'MADD', 'ETFB'], ['Electron Transfer Flavoprotein Subunit Beta Lysine Methyltransferase', 'Electron Transfer Flavoprotein Beta Subunit Lysine Methyltransferase', 'Protein N-Lysine Methyltransferase METTL20', 'ETFB Lysine Methyltransferase', 'Methyltransferase Like 20', 'C12orf72', 'ETFB-KMT', 'METTL20', 'Chromosome 12 Open Reading Frame 72', 'Methyltransferase-Like Protein 20', 'DKFZp451L235', 'EC 2.1.1.-', 'MGC50559', 'ETFBKMT'], ['Electron Transfer Flavoprotein Dehydrogenase', 'Electron-Transferring-Flavoprotein Dehydrogenase', 'Electron Transfer Flavoprotein-Ubiquinone Oxidoreductase, Mitochondrial', 'ETF-Ubiquinone Oxidoreductase', 'ETF Dehydrogenase', 'EC 1.5.5.1', 'ETF-QO', 'ETFQO', 'Mutant Electron Transfer Flavoprotein Dehydrogenase', 'ETFDH', 'MADD'], ['Electron Transfer Flavoprotein Regulatory Factor 1', 'LYR Motif-Containing Protein 5', 'LYR Motif Containing 5', 'LYRM5', 'ETFRF1'], ['ETHE1 Persulfide Dioxygenase', 'HSCO', 'Persulfide Dioxygenase ETHE1, Mitochondrial', 'Hepatoma Subtracted Clone One Protein', 'Ethylmalonic Encephalopathy 1', 'Sulfur Dioxygenase ETHE1', 'YF13H12', 'Ethylmalonic Encephalopathy Protein 1', 'Protein ETHE1, Mitochondrial', 'EC 1.13.11.18', 'EC 3.1.2.6', 'ETHE1'], ['Ethanolamine Kinase 1', 'EKI1', 'EKI 1', 'EKI', 'Putative Protein Product Of Nbla10396', 'EC 2.7.1.82', 'Nbla10396', 'ETNK1'], ['Ethanolamine Kinase 2', 'EKI2', 'Ethanolamine Kinase-Like Protein', 'EC 2.7.1.82', 'HMFT1716', 'FLJ10761', 'ETNK2', 'EKI 2'], ['Ethanolamine-Phosphate Phospho-Lyase', 'Alanine--Glyoxylate Aminotransferase 2-Like 1', 'AGXT2L1', 'Alanine-Glyoxylate Aminotransferase 2-Like 1', 'EC 2.6.1.11', 'EC 2.6.1.44', 'EC 4.2.3.2', 'EC 2.6.1', 'ETNPPL'], ['ETS Proto-Oncogene 1, Transcription Factor', 'Avian Erythroblastosis Virus E26 (V-Ets) Oncogene Homolog-1', 'V-Ets Avian Erythroblastosis Virus E26 Oncogene Homolog 1', 'Protein C-Ets-1', 'ETS-1', 'EWSR2', 'P54', 'V-Ets Avian Erythroblastosis Virus E2 Oncogene Homolog 1', 'Ets Protein', 'FLJ10768', 'C-Ets-1', 'ETS1'], ['ETS Proto-Oncogene 2, Transcription Factor', 'V-Ets Avian Erythroblastosis Virus E26 Oncogene Homolog 2', 'Protein C-Ets-2', 'V-Ets Erythroblastosis Virus E26 Oncogene Homolog 2 (Avian)', 'V-Ets Avian Erythroblastosis Virus E2 Oncogene Homolog 2', 'V-Ets Erythroblastosis Virus E26 Oncogene Homolog 2', 'Oncogene ETS-2', 'ETS2IT1', 'ETS2'], ['ETS Variant Transcription Factor 1', 'ER81', 'ETS Translocation Variant 1', 'Ets-Related Protein 81', 'Ets Variant Gene 1', 'ETS Variant 1', 'ETV1'], ['ETS Variant Transcription Factor 2', 'ER71', 'ETS Translocation Variant 2', 'Ets-Related Protein 71', 'Ets Variant Gene 2', 'ETS Variant 2', 'ETSRP71', 'ETV2'], ['ETS Variant Transcription Factor 3', 'PE-1', 'Ets Variant Gene 3, ETS Family Transcriptional Repressor', 'ETS Domain Transcriptional Repressor PE1', 'Mitogenic Ets Transcriptional Suppressor', 'ETS Translocation Variant 3', 'ETS Variant 3', 'METS', 'PE1', 'ETV3'], ['ETS Variant Transcription Factor 3 Like', 'ETS Translocation Variant 3-Like Protein', 'Ets Variant Gene 3-Like', 'ETS Variant 3 Like', 'Ets Variant 3-Like', 'FLJ16478', 'ETV3L'], ['ETS Variant Transcription Factor 4', 'E1A-F', 'E1AF', 'PEA3', 'Ets Variant Gene 4 (E1A Enhancer-Binding Protein, E1AF)', 'Polyomavirus Enhancer Activator 3 Homolog', 'Adenovirus E1A Enhancer-Binding Protein', 'ETS Translocation Variant 4', 'ETS Variant 4', 'EWS Protein/E1A Enhancer Binding Protein Chimera', 'E1A Enhancer Binding Protein', 'Protein PEA3', 'PEAS3', 'ETV4'], ['ETS Variant Transcription Factor 5', 'ERM', 'ETS Translocation Variant 5', 'Ets-Related Protein ERM', 'Ets-Related Molecule', 'ETS Variant 5', 'Ets Variant Gene 5 (Ets-Related Molecule)', 'ETV5'], ['ETS Variant Transcription Factor 6', 'TEL', 'Ets Variant Gene 6 (TEL Oncogene)', 'ETS Translocation Variant 6', 'Transcription Factor ETV6', 'ETS-Related Protein Tel1', 'ETS Variant 6', 'TEL1 Oncogene', 'TEL Oncogene', 'TEL/ABL', 'THC5', 'ETV6', 'TEL1', 'Tel'], ['ETS Variant Transcription Factor 7', 'TEL2', 'Ets Variant Gene 7 (TEL2 Oncogene)', 'ETS Translocation Variant 7', 'Transcription Factor ETV7', 'ETS-Related Protein Tel2', 'Tel-Related Ets Factor', 'TEL2 Oncogene', 'ETS Variant 7', 'TEL-2', 'TELB', 'Ets Transcription Factor TEL-2b', 'Transcription Factor Tel-2', 'ETV7', 'TREF'], ['Eva-1 Homolog A, Regulator Of Programmed Cell Death', 'Transmembrane Protein 166', 'Family With Sequence Similarity 176, Member A', 'Protein Eva-1 Homolog A', 'Protein FAM176A', 'FAM176A', 'TMEM166', 'Eva-1 Homolog A (C. Elegans)', 'FLJ13391', 'EVA1A'], ['Eva-1 Homolog B', 'Family With Sequence Similarity 176, Member B', 'Protein Eva-1 Homolog B', 'Protein FAM176B', 'C1orf78', 'FAM176B', 'Chromosome 1 Open Reading Frame 78', 'Eva-1 Homolog B (C. Elegans)', 'FLJ10647', 'EVA1B'], ['Eva-1 Homolog C', 'Family With Sequence Similarity 176, Member C', 'Protein Eva-1 Homolog C', 'Protein FAM176C', 'C21orf63', 'C21orf64', 'FAM176C', 'PRED34', 'SUE21', 'B18', 'B19', 'Chromosome 21 Open Reading Frame 64', 'Chromosome 21 Open Reading Frame 63', 'Eva-1 Homolog C (C. Elegans)', 'EVA1C'], ['EvC Ciliary Complex Subunit 1', 'DWF-1', 'Ellis-Van Creveld Syndrome Protein', 'Ellis Van Creveld Syndrome', 'Ellis Van Creveld Protein', 'EVC1', 'EVCL', 'EVC'], ['EvC Ciliary Complex Subunit 2', 'Limbin', 'LBN', 'Ellis-Van Creveld Syndrome Protein 2', 'Ellis Van Creveld Syndrome 2', 'EVC2', 'WAD'], ['Ecotropic Viral Integration Site 2A', 'EVDA', 'Ecotropic Viral Integration Site 2A Protein Homolog', 'Protein EVI2A', 'EVI-2A', 'EVI2', 'EVI2A'], ['Ecotropic Viral Integration Site 2B', 'EVDB', 'Ecotropic Viral Integration Site 2B Protein Homolog', 'Protein EVI2B', 'D17S376', 'EVI-2B', 'CD361', 'CD361 Antigen', 'EVI2B'], ['Ecotropic Viral Integration Site 5', 'NB4S', 'Ecotropic Viral Integration Site 5 Protein Homolog', 'Neuroblastoma Stage 4S Gene Protein', 'EVI-5', 'DJ846D11.1 (Ecotropic Viral Integration Site 5)', 'Neuroblastoma Stage 4S Gene', 'EVI5'], ['Ecotropic Viral Integration Site 5 Like', 'Ecotropic Viral Integration Site 5-Like Protein', 'EVI5-Like Protein', 'Epididymis Secretory Sperm Binding Protein', 'EVI5L'], ['Enah/Vasp-Like', 'RNB6', 'Ena/Vasodilator-Stimulated Phosphoprotein-Like', 'Ena/VASP-Like Protein', 'Epididymis Secretory Sperm Binding Protein', 'EVL'], ['Envoplakin', '210 KDa Cornified Envelope Precursor Protein', '210 KDa Paraneoplastic Pemphigus Antigen', 'EVPK', 'EVPL', 'P210'], ['Envoplakin Like', 'Envoplakin-Like Protein', 'EVPLL'], ['Even-Skipped Homeobox 1', 'Homeobox Even-Skipped Homolog Protein 1', 'EVX-1', 'Eve, Even-Skipped Homeobox Homolog 1 (Drosophila)', 'Even-Skipped Homeo Box 1 (Homolog Of Drosophila)', 'Eve, Even-Skipped Homeo Box Homolog 1', 'Eve, Even-Skipped Homeobox Homolog 1', 'EVX1'], ['Even-Skipped Homeobox 2', 'Homeobox Even-Skipped Homolog Protein 2', 'EVX-2', 'Even-Skipped Homeo Box 2 (Homolog Of Drosophila Eve)', 'Eve, Even-Skipped Homeobox Homolog 2 (Drosophila)', 'Eve, Even-Skipped Homeo Box Homolog 2', 'EVX2'], ['EWS RNA Binding Protein 1', 'EWS', 'Ewing Sarcoma Breakpoint Region 1', 'RNA-Binding Protein EWS', 'Ewings Sarcoma EWS-Fli1 (Type 1) Oncogene', 'Ewing Sarcoma Breakpoint Region 1 Protein', 'EWS RNA-Binding Protein Variant 6', 'EWS Oncogene', 'BK984G1.4', 'EWS-FLI1', 'EWSR1'], ["Exonuclease 3'-5' Domain Containing 1", "Exonuclease 3'-5' Domain-Like-Containing Protein 1", "Exonuclease 3'-5' Domain-Containing Protein 1", 'PiRNA Biogenesis Protein EXD1', 'Inactive Exonuclease EXD1', 'EXDL1', "Exonuclease 3'-5' Domain-Like 1", 'MGC33637', 'EXD1'], ["Exonuclease 3'-5' Domain Containing 2", "Exonuclease 3'-5' Domain-Like-Containing Protein 2", "Exonuclease 3'-5' Domain-Containing Protein 2", "Exonuclease 3'-5' Domain-Like 2", "3'-5' Exoribonuclease EXD2", 'C14orf114', 'EXDL2', 'Chromosome 14 Open Reading Frame 114', 'EC 3.1.11.1', 'EC 3.1.13.-', 'FLJ10738', 'EXD2'], ["Exonuclease 3'-5' Domain Containing 3", "Exonuclease 3'-5' Domain-Containing Protein 3", 'Exonuclease Mut-7 Homolog', 'Nibbler Homolog', 'Mut-7', 'Nbr', 'Testicular Tissue Protein Li 125', 'EC 3.1.-.-', 'LOC54932', 'FLJ20433', 'EXD3'], ['Exonuclease 1', 'HExoI', 'HEX1', 'Rad2 Nuclease Family Member, Homolog Of S. Cerevisiae Exonuclease 1', 'Exonuclease I', 'EC 3.1.-.-', 'HExo1', 'EXO1', 'EXOI'], ['Exonuclease 5', 'Defects In Morphology Protein 1 Homolog', 'Exonuclease V', 'C1orf176', 'Exo V', 'HExo5', 'DEM1', 'Defects In Morphology 1 Homolog (S. Cerevisiae)', 'Chromosome 1 Open Reading Frame 176', 'Defects In Morphology 1 Homolog', 'Probable Exonuclease V', 'EC 3.1.-.-', 'FLJ21144', 'EXO5'], ['Exocyst Complex Component 1', 'SEC3', 'Exocyst Complex Component Sec3', 'BM-102', 'SEC3L1', 'SEC3-Like 1 (S. Cerevisiae)', 'SEC3-Like 1', 'FLJ10893', 'SEC3P', 'EXOC1', 'Sec3p'], ['Exocyst Complex Component 1 Like', 'Exocyst Complex Component 1-Like', 'Exocyst Complex Component 1 Pseudogene', 'Exocyst -Like Pseudogene', 'EXOC1L'], ['Exocyst Complex Component 2', 'Exocyst Complex Component Sec5', 'SEC5L1', 'Sec5p', 'SEC5', 'SEC5-Like 1 (S. Cerevisiae)', 'SEC5-Like 1', 'FLJ11026', 'EXOC2'], ['Exocyst Complex Component 3', 'Exocyst Complex Component Sec6', 'SEC6L1', 'Sec6p', 'SEC6', 'SEC6-Like 1 (S. Cerevisiae)', 'Sec 6 Homolog', 'SEC6-Like 1', 'EXOC3'], ['Exocyst Complex Component 3 Like 1', 'Exocyst Complex Component 3-Like Protein', 'Protein Jiangli', 'EXOC3L', 'Exocyst Complex Component 3-Like 1', 'Exocyst Complex Component 3-Like', 'FLJ35539', 'FLJ35587', 'EXOC3L1'], ['Exocyst Complex Component 3 Like 2', 'XTP7', 'Exocyst Complex Component 3-Like Protein 2', 'HBV X-Transactivated Gene 7 Protein', 'HBV XAg-Transactivated Protein 7', 'Protein 7 Transactivated By Hepatitis B Virus X Antigen (HBxAg)', 'FLJ36147', 'EXOC3L2'], ['Exocyst Complex Component 3 Like 4', 'Exocyst Complex Component 3-Like Protein 4', 'C14orf73', 'Chromosome 14 Open Reading Frame 73', 'SEC6-Like Protein C14orf73', 'EXOC3L4'], ['Exocyst Complex Component 4', 'SEC8', 'Exocyst Complex Component Sec8', 'KIAA1699', 'SEC8L1', 'Sec8p', 'SEC8-Like 1 (S. Cerevisiae)', 'SEC8-Like 1', 'MGC27170', 'EXOC4'], ['Exocyst Complex Component 5', 'SEC10', 'Exocyst Complex Component Sec10', 'SEC10L1', 'SEC10P', 'SEC10 (S. Cerevisiae)-Like 1', 'SEC10-Like 1 (S. Cerevisiae)', 'SEC10-Like 1', 'PRO1912', 'HSEC10', 'HSec10', 'EXOC5'], ['Exocyst Complex Component 6', 'SEC15L', 'Exocyst Complex Component Sec15A', 'SEC15L1', 'EXOC6A', 'Sec15p', 'SEC15-Like 1 (S. Cerevisiae)', 'SEC15-Like Protein 3', 'SEC15-Like Protein 1', 'Rsec15-Like Protein', 'DKFZp761I2124', 'SEC15-Like 1', 'MGC33397', 'SEC15L3', 'FLJ1125', 'SEC15A', 'SEC15', 'EXOC6'], ['Exocyst Complex Component 6B', 'Exocyst Complex Component Sec15B', 'SEC15-Like Protein 2', 'KIAA0919', 'SEC15L2', 'SEC15B', 'SEC15 Homolog B (S. Cerevisiae)', 'SEC15-Like 2 (S. Cerevisiae)', 'SEC15 Homolog B', 'SEMDJL3', 'EXOC6B'], ['Exocyst Complex Component 7', 'EXO70', 'Exocyst Complex Component Exo70', 'KIAA1067', 'YJL085W', 'Exo70p', 'BLOM4', 'NEDSEBA', '2-5-3p', 'EX070', 'EXOC1', 'EXOC7'], ['Exocyst Complex Component 8', 'Exocyst Complex 84 KDa Subunit', 'Exo84p', 'EXO84', 'SEC84', 'NEDMISB', 'EXOC8'], ['Exo/Endonuclease G', 'Endonuclease G-Like 1', 'ENGL', "Endo/Exonuclease (5'-3'), Endonuclease G-Like", 'Nuclease EXOG, Mitochondrial', 'Endonuclease G-Like 2', 'Endo G-Like 1', 'ENDOGL1', 'ENDOGL2', 'ENGL-A', 'ENGL-B', 'EC 3.1.30.-', 'EC 3.1.30', 'ENGLA', 'ENGLB', 'EXOG'], ['Exosome Component 1', 'CSL4', 'Exosome Complex Component CSL4', 'CGI-108', 'Csl4p', 'Ski4p', 'SKI4', 'P13', 'Homolog Of Yeast Exosomal Core Protein CSL4', 'CSL4 Exosomal Core Protein Homolog (Yeast)', "3'-5' Exoribonuclease CSL4 Homolog", 'Exosomal Core Protein CSL4', 'EXOSC1', 'HCsl4p'], ['Exosome Component 2', 'RRP4', "Homolog Of Yeast RRP4 (Ribosomal RNA Processing 4), 3' 5' Exoribonuclease (RRP4)", 'Ribosomal RNA-Processing Protein 4', 'Exosome Complex Component RRP4', 'HRrp4p', 'Rrp4p', 'P7', "Homolog Of Yeast RRP4 (Ribosomal RNA Processing 4), 3'-5'-Exoribonuclease", 'Exosome Complex Exonuclease RRP4', 'EXOSC2', 'SHRF'], ['Exosome Component 3', 'RRP40', 'P10', 'Ribosomal RNA-Processing Protein 40', 'Exosome Complex Component RRP40', 'CGI-102', 'HRrp-40', 'Rrp40p', 'Exosome Complex Exonuclease RRP40', 'Exosome Component Rrp40', 'CGI-102 Protein', 'BA3J10.7', 'HRrp40p', 'EXOSC3', 'PCH1B'], ['Exosome Component 4', 'RRP41', 'SKI6', 'P12A', 'Ribosomal RNA-Processing Protein 41', 'Exosome Complex Component RRP41', 'Exosome Component Rrp41', 'HRrp41p', 'RRP41A', 'Rrp41p', 'Ski6p', 'Exosome Complex Exonuclease RRP41', 'FLJ20591', 'EXOSC4'], ['Exosome Component 5', 'RRP46', 'P12B', 'Chronic Myelogenous Leukemia Tumor Antigen 28', 'Ribosomal RNA-Processing Protein 46', 'Exosome Complex Component RRP46', 'Exosome Component Rrp46', 'HRrp46p', 'RRP41B', 'Rrp46p', 'Exosome Complex Exonuclease RRP46', 'MGC12901', 'EXOSC5', 'CML28'], ['Exosome Component 6', 'MTR3', 'P11', 'MRNA Transport Regulator 3 Homolog', 'Exosome Complex Component MTR3', 'HMtr3p', 'Mtr3p', 'HMtr3', 'EAP4', 'Mtr3 (MRNA Transport Regulator 3)-Homolog (Yeast)', 'Homolog Of Yeast MRNA Transport Regulator 3', 'Mtr3 (MRNA Transport Regulator 3)-Homolog', 'Exosome Complex Exonuclease MTR3', 'EXOSC6'], ['Exosome Component 7', 'RRP42', 'P8', 'Ribosomal RNA-Processing Protein 42', 'Exosome Complex Component RRP42', 'KIAA0116', 'HRrp42p', 'Rrp42p', 'EAP1', 'Exosome Complex Exonuclease RRP42', 'EXOSC7'], ['Exosome Component 8', 'RRP43', 'OIP2', 'P9', 'Ribosomal RNA-Processing Protein 43', 'Exosome Complex Component RRP43', 'CBP-Interacting Protein 3', 'Opa Interacting Protein 2', 'BA421P11.3', 'Rrp43p', 'OIP-2', 'CIP3', 'EAP2', 'Exosome Complex Exonuclease RRP43', 'Opa-Interacting Protein 2', 'EXOSC8', 'PCH1C'], ['Exosome Component 9', 'PM/Scl-75', 'Polymyositis/Scleroderma Autoantigen 1, 75kDa', 'Exosome Complex Component RRP45', 'Autoantigen PM/Scl 1', 'PMSCL1', 'Rrp45p', 'RRP45', 'P5', 'P6', 'P75 Polymyositis-Scleroderma Overlap Syndrome Associated Autoantigen', 'P75 Polymyositis-Scleroderma Overlap Syndrome-Associated Autoantigen', 'Polymyositis/Scleroderma Autoantigen 1 (75kD)', 'Polymyositis/Scleroderma Autoantigen 75 KDa', 'Polymyositis/Scleroderma Autoantigen 1', 'Exosome Complex Exonuclease RRP45', 'PMSCL Autoantigen, 75kD', 'EC 3.1.13', 'EXOSC9', 'PCH1D'], ['Exosome Component 10', 'PM/Scl-100', 'RRP6', 'P100 Polymyositis-Scleroderma Overlap Syndrome-Associated Autoantigen', 'Polymyositis/Scleroderma Autoantigen 100 KDa', 'Polymyositis/Scleroderma Autoantigen 2', 'PM-Scl', 'PMSCL2', 'PMSCL', 'Rrp6p', 'P2', 'P3', 'P4', 'Polymyositis/Scleroderma Autoantigen 2 (100kD)', 'Polymyositis/Scleroderma Autoantigen 2, 100kDa', 'Autoantigen PM/Scl 2', 'Autoantigen PM-SCL', 'EC 3.1.13.-', 'EXOSC10'], ['Exophilin 5', 'Synaptotagmin-Like Protein Homolog Lacking C2 Domains B', 'Synaptotagmin-Like Homologue Lacking C2 Domains B', 'Slp Homolog Lacking C2 Domains B', 'Exophilin-5', 'SLAC2-B', 'SLAC2B', 'KIAA0624', 'SlaC2-B', 'EXPH5'], ['Exostosin Glycosyltransferase 1', 'Glucuronosyl-N-Acetylglucosaminyl-Proteoglycan/N-Acetylglucosaminyl-Proteoglycan 4-Alpha-N-Acetylglucosaminyltransferase', 'Glucuronosyl-N-Acetylglucosaminyl-Proteoglycan 4-Alpha-N- Acetylglucosaminyltransferase', 'N-Acetylglucosaminyl-Proteoglycan 4-Beta-Glucuronosyltransferase', 'Langer-Giedion Syndrome Chromosome Region', 'Putative Tumor Suppressor Protein EXT1', 'Multiple Exostoses Protein 1', 'Exostoses (Multiple) 1', 'Exostosin-1', 'EC 2.4.1.224', 'EC 2.4.1.225', 'Exostosin 1', 'TRPS2', 'LGCR', 'EXT1', 'EXT', 'LGS', 'TTV', 'Ttv'], ['Exostosin Glycosyltransferase 2', 'Glucuronosyl-N-Acetylglucosaminyl-Proteoglycan/N-Acetylglucosaminyl-Proteoglycan 4-Alpha-N-Acetylglucosaminyltransferase', 'N-Acetylglucosaminyl-Proteoglycan 4-Beta-Glucuronosyltransferase', 'Putative Tumor Suppressor Protein EXT2', 'Multiple Exostoses Protein 2', 'Exostosin-2', 'SOTV', 'Glucuronosyl-N-Acetylglucosaminyl-Proteoglycan 4-Alpha-N- Acetylglucosaminyltransferase', 'Exostoses (Multiple) 2', 'Multiple Exostosis 2', 'EC 2.4.1.224', 'EC 2.4.1.225', 'Exostosin 2', 'SSMS', 'EXT2'], ['Exostosin Like Glycosyltransferase 1', 'Exostosin-L', 'EXTL', 'Glucuronosyl-N-Acetylglucosaminyl-Proteoglycan 4-Alpha-N- Acetylglucosaminyltransferase', 'Glucuronyl-N-Acetylglucosaminylproteoglycan Alpha-1,4-N- Acetylglucosaminyltransferase', 'Alpha-N-Acetylglucosaminyltransferase II', 'Multiple Exostosis-Like Protein', 'Exostoses (Multiple)-Like 1', 'Exostosin-Like 1', 'Glucuronosyl-N-Acetylglucosaminyl-Proteoglycan 4-Alpha-N-Acetylglucosaminyltransferase', 'EC 2.4.1.224', 'MGC70794', 'EXTL1'], ['Exostosin Like Glycosyltransferase 2', 'Glucuronyl-Galactosyl-Proteoglycan 4-Alpha-N-Acetylglucosaminyltransferase', 'Alpha-1,4-N-Acetylhexosaminyltransferase EXTL2', 'Exostoses (Multiple)-Like 2', 'EXT-Related Protein 2', 'Alpha-GalNAcT EXTL2', 'Exostosin-Like 2', 'EXTR2', 'Alpha-1,4-N-Acteylhexosaminyltransferase', 'Processed Exostosin-Like 2', 'EC 2.4.1.223', 'EXTL2'], ['Exostosin Like Glycosyltransferase 3', 'Glucuronyl-Galactosyl-Proteoglycan 4-Alpha-N-Acetylglucosaminyltransferase', 'Hereditary Multiple Exostoses Gene Isolog', 'Putative Tumor Suppressor Protein EXTL3', 'Exostoses (Multiple)-Like 3', 'Exostosin-Like 3', 'EXTL1L', 'EXTR1', 'REGR', 'Epididymis Secretory Sperm Binding Protein', 'Multiple Exostosis-Like Protein 3', 'Exostosin Tumor-Like 3', 'EXT-Related Protein 1', 'EXT-Related 1', 'REG Receptor', 'Reg Receptor', 'EC 2.4.1.223', 'KIAA0519', 'ISDNA', 'EXTL3', 'BOTV', 'Botv', 'RPR'], ['EYA Transcriptional Coactivator And Phosphatase 1', 'Eyes Absent Homolog 1', 'EC 3.1.3.48', 'Eyes Absent (Drosophila) Homolog 1', 'Eyes Absent Homolog 1 (Drosophila)', 'EC 3.1.3.16', 'BOS1', 'OFC1', 'EYA1', 'BOP', 'BOR'], ['EYA Transcriptional Coactivator And Phosphatase 2', 'EAB1', 'Eyes Absent Homolog 2', 'EC 3.1.3.48', 'Eyes Absent (Drosophila) Homolog 2', 'Eyes Absent Homolog 2 (Drosophila)', 'EYA2'], ['EYA Transcriptional Coactivator And Phosphatase 3', 'Eyes Absent Homolog 3', 'EC 3.1.3.48', 'Eyes Absent (Drosophila) Homolog 3', 'Eyes Absent Homolog 3 (Drosophila)', 'Eyes Absent 3', 'DKFZp686C132', 'EYA3'], ['EYA Transcriptional Coactivator And Phosphatase 4', 'Eyes Absent Homolog 4', 'EC 3.1.3.48', 'Eyes Absent (Drosophila) Homolog 4', 'Eyes Absent Homolog 4 (Drosophila)', 'Eyes Absent-Like Protein 4', 'DJ78N10.1 (Eyes Absent)', 'DFNA10', 'CMD1J', 'EYA4'], ['Eyes Shut Homolog', 'SPAM', 'Epidermal Growth Factor-Like Protein 10', 'Epidermal Growth Factor-Like Protein 11', 'EGF-Like-Domain, Multiple 11', 'EGF-Like-Domain, Multiple 10', 'Protein Spacemaker Homolog', 'Protein Eyes Shut Homolog', 'BA166P24.2', 'BA307F22.3', 'DJ1018A4.2', 'DJ303F19.1', 'BA74E24.1', 'C6orf178', 'C6orf179', 'C6orf180', 'EGFL10', 'EGFL11', 'Retinitis Pigmentosa 25 (Autosomal Recessive)', 'Chromosome 6 Open Reading Frame 180', 'Chromosome 6 Open Reading Frame 178', 'Chromosome 6 Open Reading Frame 179', 'Eyes Shut Homolog (Drosophila)', 'EGF-Like Protein 10', 'EGF-Like Protein 11', 'DJ22I17.2', 'RP25', 'EYS'], ['Enhancer Of Zeste 1 Polycomb Repressive Complex 2 Subunit', 'Histone-Lysine N-Methyltransferase EZH1', 'Enhancer Of Zeste Homolog 1', 'KIAA0388', 'KMT6B', 'ENX-2', 'Enhancer Of Zeste (Drosophila) Homolog 1', 'Enhancer Of Zeste Homolog 1 (Drosophila)', 'EC 2.1.1.356', 'EC 2.1.1.43', 'EZH1'], ['Enhancer Of Zeste 2 Polycomb Repressive Complex 2 Subunit', 'ENX-1', 'KMT6', 'Histone-Lysine N-Methyltransferase EZH2', 'Lysine N-Methyltransferase 6', 'Enhancer Of Zeste Homolog 2', 'KMT6A', 'Enhancer Of Zeste (Drosophila) Homolog 2', 'Enhancer Of Zeste Homolog 2 (Drosophila)', "EC 2.1.1.43');", 'EC 2.1.1.356', 'EC 2.1.1.43', 'EC 2.1.1', 'EZH2b', 'ENX1', 'WVS2', 'EZH2', 'EZH1', 'WVS'], ['EZH Inhibitory Protein', 'Catalytic Antagonist Of Polycomb', 'CATACOMB', 'CXorf67', 'Chromosome X Open Reading Frame 67', 'Uncharacterized Protein CXorf67', 'K27M-Like Inhibitor Of PRC2', 'KIP75', 'EZHIP'], ['Ezrin', 'Villin 2 (Ezrin)', 'Cytovillin 2', 'VIL2', 'P81', 'Epididymis Secretory Protein Li 105', 'Cytovillin', 'HEL-S-105', 'Villin-2', 'CVIL', 'CVL', 'EZR'], ['Coagulation Factor II, Thrombin', 'Prepro-Coagulation Factor II', 'Prothrombin', 'EC 3.4.21.5', 'Coagulation Factor II (Thrombin)', 'Coagulation Factor II', 'Prothrombin B-Chain', 'Thrombin Factor II', 'EC 3.4.21', 'RPRGL2', 'THPH1', 'PT', 'F2'], ['Coagulation Factor II Thrombin Receptor', 'PAR-1', 'CF2R', 'PAR1', 'TR', 'Proteinase-Activated Receptor 1', 'Coagulation Factor II (Thrombin) Receptor', 'Coagulation Factor II Receptor', 'Protease Activated Receptor 1', 'Protease-Activated Receptor 1', 'Thrombin Receptor', 'HTR', 'F2R'], ['F2R Like Trypsin Receptor 1', 'PAR2', 'Coagulation Factor II (Thrombin) Receptor-Like 1', 'Proteinase-Activated Receptor 2', 'G-Protein Coupled Receptor 11', 'Thrombin Receptor-Like 1', 'GPR11', 'Coagulation Factor II Receptor-Like 1', 'Proteinase-Activated Receptor-2', 'Protease-Activated Receptor 2', 'F2RL1', 'PAR-2'], ['Coagulation Factor II Thrombin Receptor Like 2', 'PAR3', 'Proteinase-Activated Receptor 3', 'Thrombin Receptor-Like 2', 'PAR-3', 'Coagulation Factor II Receptor-Like 2 (Protease-Actovated Receptor 3)', 'Coagulation Factor II (Thrombin) Receptor-Like 2', 'Coagulation Factor II Receptor-Like 2', 'Proteinase-Activated Receptor-3', 'Protease-Activated Receptor 3', 'F2RL2'], ['F2R Like Thrombin Or Trypsin Receptor 3', 'PAR4', 'Coagulation Factor II (Thrombin) Receptor-Like 3', 'F2R Like Thrombin/Trypsin Receptor 3', 'Proteinase-Activated Receptor 4', 'Thrombin Receptor-Like 3', 'PAR-4', 'Coagulation Factor II Receptor-Like 3', 'Proteinase-Activated Receptor-4', 'Protease-Activated Receptor-4', 'F2RL3'], ['Coagulation Factor III, Tissue Factor', 'Tissue Factor', 'TF', 'Coagulation Factor III (Thromboplastin, Tissue Factor)', 'CD142', 'Coagulation Factor III', 'Thromboplastin', 'CD142 Antigen', 'TFA', 'F3'], ['Coagulation Factor V', 'Coagulation Factor V (Proaccelerin, Labile Factor)', 'Activated Protein C Cofactor', 'Coagulation Factor V Jinjiang A2 Domain', 'Proaccelerin, Labile Factor', 'Factor V Leiden', 'RPRGL1', 'THPH2', 'PCCF', 'FVL', 'F5'], ['Coagulation Factor VII', 'Eptacog Alfa', 'Coagulation Factor VII (Serum Prothrombin Conversion Accelerator)', 'FVII Coagulation Protein', 'Proconvertin', 'EC 3.4.21.21', 'SPCA', 'Serum Prothrombin Conversion Accelerator', 'Factor VII', 'EC 3.4.21', 'F7'], ['Coagulation Factor VIII', 'Coagulation Factor VIII, Procoagulant Component', 'Antihemophilic Factor', 'DXS1253E', 'FVIII', 'HEMA', 'AHF', 'F8C', 'Coagulation Factor VIII A1 Domain', 'Coagulation Factor VIII C2 Domain', 'Coagulation Factor VIIIc', 'Procoagulant Component', 'Factor VIII F8B', 'Factor VIIIF8B', 'Hemophilia A', 'F8B', 'F8'], ['Coagulation Factor VIII Associated 1', 'Coagulation Factor VIII-Associated (Intronic Transcript) 1', '40-KDa Huntingtin-Associated Protein', 'Factor VIII Intron 22 Protein', 'CpG Island Protein', 'DXS522E', 'HAP40', 'Huntingtin-Associated Protein 40', 'Factor VIII Associated Protein', 'F8A1', 'F8A'], ['Coagulation Factor VIII Associated 2', 'Coagulation Factor VIII-Associated (Intronic Transcript) 2', '40-KDa Huntingtin-Associated Protein', 'Factor VIII Intron 22 Protein', 'CpG Island Protein', 'HAP40', 'F8A2'], ['Coagulation Factor VIII Associated 3', 'Coagulation Factor VIII-Associated (Intronic Transcript) 3', '40-KDa Huntingtin-Associated Protein', 'Factor VIII Intron 22 Protein', 'CpG Island Protein', 'HAP40', 'F8A3'], ['Coagulation Factor IX', 'Plasma Thromboplastic Component', 'Plasma Thromboplastin Component', 'Christmas Factor', 'EC 3.4.21.22', 'FIX', 'PTC', 'Christmas Disease', 'Hemophilia B', 'Factor IX F9', 'Factor IX', 'EC 3.4.21', 'Factor 9', 'F9 P22', 'THPH8', 'HEMB', 'P19', 'F9'], ['Coagulation Factor X', 'Stuart-Prower Factor', 'EC 3.4.21.6', 'Prothrombinase', 'Stuart Factor', 'Factor Xa', 'EC 3.4.21', 'Factor X', 'FXA', 'F10', 'FX'], ['Coagulation Factor XI', 'Plasma Thromboplastin Antecedent', 'FXI', 'PTA', 'Coagualtion Factor XI', 'EC 3.4.21.27', 'EC 3.4.21', 'F11'], ['F11 Receptor', 'Junctional Adhesion Molecule 1', 'PAM-1', 'JCAM', 'Junctional Adhesion Molecule A', 'Platelet Adhesion Molecule 1', 'Platelet F11 Receptor', 'CD321', 'JAM-1', 'JAM-A', 'JAM1', 'JAMA', 'CD321 Antigen', 'F11R', 'JAM', 'KAT'], ['Coagulation Factor XII', 'Hageman Factor', 'EC 3.4.21.38', 'HAF', 'Coagulation Factor XII (Hageman Factor)', 'Coagulation Factor XIIa Heavy Chain', 'Coagulation Factor XIIa Light Chain', 'Beta-Factor XIIa Part 1', 'Beta-Factor XIIa Part 2', 'EC 3.4.21', 'HAE3', 'HAEX', 'F12'], ['Coagulation Factor XIII A Chain', 'Protein-Glutamine Gamma-Glutamyltransferase A Chain', 'Coagulation Factor XIII, A1 Polypeptide', 'Coagulation Factor XIIIa', 'Transglutaminase A Chain', 'EC 2.3.2.13', 'F13A', 'BA525O21.1 (Coagulation Factor XIII, A1 Polypeptide)', 'Coagulation Factor XIII, A Polypeptide', 'Fibrin Stabilizing Factor, A Subunit', 'Transglutaminase. Plasma', 'FSF, A Subunit', 'Fibrinoligase', 'Factor XIIIa', 'TGase', 'F13A1'], ['Coagulation Factor XIII B Chain', 'Protein-Glutamine Gamma-Glutamyltransferase B Chain', 'Coagulation Factor XIII, B Polypeptide', 'Fibrin-Stabilizing Factor B Subunit', 'Transglutaminase B Chain', 'FXIIIB', 'TGase', 'F13B'], ['Fatty Acid 2-Hydroxylase', 'FAAH', 'Fatty Acid Hydroxylase Domain-Containing Protein 1', 'Spastic Paraplegia 35 (Autosomal Recessive)', 'Fatty Acid Hydroxylase Domain Containing 1', 'Fatty Acid Alpha-Hydroxylase', 'FAXDC1', 'Fatty Acid Hydroxylase', 'EC 1.14.18.-', 'FLJ25287', 'SPG35', 'FAH1', 'SCS7', 'FA2H'], ['Fatty Acid Amide Hydrolase', 'Fatty-Acid Amide Hydrolase 1', 'Anandamide Amidohydrolase 1', 'Fatty Acid Ester Hydrolase', 'Oleamide Hydrolase 1', 'FAAH-1', 'EC 3.5.1.n2', 'EC 3.5.1.99', 'EC 3.1.1.-', 'EC 3.5.1', 'FAAH1', 'PSAB', 'FAAH'], ['Fatty Acid Amide Hydrolase 2', 'Amidase Domain-Containing Protein', 'Fatty-Acid Amide Hydrolase 2', 'Anandamide Amidohydrolase 2', 'Amidase Domain Containing', 'Oleamide Hydrolase 2', 'AMDD', 'RP11-479E16.1', 'EC 3.5.1.99', 'FLJ31204', 'FAAH-2', 'FAAH2'], ['FA Core Complex Associated Protein 20', 'Fanconi Anemia Core Complex Associated Protein 20', 'Fanconi Anemia Core Complex-Associated Protein 20', 'Fanconi Anemia-Associated Protein Of 20 KDa', 'FANCA-Associated Protein Of 20 KDa', 'C1orf86', 'Chromosome 1 Open Reading Frame 86', 'FLJ31031', 'FP7162', 'FAAP20'], ['FA Core Complex Associated Protein 24', 'Fanconi Anemia Core Complex Associated Protein 24', 'Fanconi Anemia Core Complex-Associated Protein 24', 'Fanconi Anemia-Associated Protein Of 24 KDa', 'C19orf40', 'Fanconi Anemia-Associated Protein, 24kDa', 'Chromosome 19 Open Reading Frame 40', 'FLJ46828', 'MGC32020', 'FAAP24'], ['FA Core Complex Associated Protein 100', 'Fanconi Anemia Core Complex Associated Protein 100', 'Fanconi Anemia Core Complex-Associated Protein 100', 'Fanconi Anemia-Associated Protein Of 100 KDa', 'Fanconi Anemia-Associated Protein, 100kDa', 'C17orf70', 'Fanconi Anemia Associated Protein 100 KDa Subunit', 'Fanconi Anemia Core Complex 100 KDa Subunit', 'Chromosome 17 Open Reading Frame 70', 'FLJ22175', 'FAAP100'], ['Fatty Acid Binding Protein 1', 'L-FABP', 'Liver-Type Fatty Acid-Binding Protein', 'Fatty Acid Binding Protein 1, Liver', 'Fatty Acid-Binding Protein, Liver', 'FABPL', 'Fatty Acid-Binding Protein 1', 'FABP1'], ['Fatty Acid Binding Protein 2', 'I-FABP', 'Intestinal-Type Fatty Acid-Binding Protein', 'Fatty Acid Binding Protein 2, Intestinal', 'Fatty Acid-Binding Protein, Intestinal', 'FABPI', 'Fatty Acid-Binding Protein 2', 'FABP2'], ['Fatty Acid Binding Protein 3', 'Mammary-Derived Growth Inhibitor', 'H-FABP', 'Fatty Acid Binding Protein 3, Muscle And Heart', 'Heart-Type Fatty Acid-Binding Protein', 'Fatty Acid-Binding Protein, Heart', 'Muscle Fatty Acid-Binding Protein', 'Fatty Acid Binding Protein 11', 'FABP11', 'M-FABP', 'O-FABP', 'MDGI', 'Fatty Acid Binding Protein 3, Muscle And Heart (Mammary-Derived Growth Inhibitor)', 'Epididymis Secretory Sperm Binding Protein', 'Fatty Acid-Binding Protein 3', 'FABP3'], ['Fatty Acid Binding Protein 4', 'A-FABP', 'Adipocyte-Type Fatty Acid-Binding Protein', 'Fatty Acid-Binding Protein, Adipocyte', 'Adipocyte Fatty Acid Binding Protein', 'Adipocyte Lipid-Binding Protein', 'AFABP', 'ALBP', 'AP2', 'Fatty Acid Binding Protein 4, Adipocyte', 'Epididymis Secretory Protein Li 104', 'Fatty Acid-Binding Protein 4', 'HEL-S-104', 'FABP4'], ['Fatty Acid Binding Protein 5', 'PA-FABP', 'E-FABP', 'Psoriasis-Associated Fatty Acid-Binding Protein Homolog', 'Fatty Acid Binding Protein 5 (Psoriasis-Associated)', 'Epidermal-Type Fatty Acid-Binding Protein', 'Fatty Acid-Binding Protein 5', 'KFABP', 'Fatty Acid-Binding Protein, Epidermal', 'PAFABP', 'EFABP', 'FABP5'], ['Fatty Acid Binding Protein 6', 'Gastrotropin', 'I-BABP', 'I-15P', 'ILLBP', 'ILBP', 'Fatty Acid Binding Protein 6, Ileal', 'Ileal Bile Acid Binding Protein', 'Illeal Lipid-Binding Protein', 'Ileal Lipid-Binding Protein', 'Intestinal 15 KDa Protein', 'I-BALB', 'I-BAP', 'ILBP3', 'GT', 'Intestinal Bile Acid-Binding Protein', 'Fatty Acid-Binding Protein 6', 'FABP6'], ['Fatty Acid Binding Protein 7', 'B-FABP', 'BLBP', 'Brain-Type Fatty Acid-Binding Protein', 'Fatty Acid-Binding Protein, Brain', 'Brain Lipid-Binding Protein', 'FABPB', 'MRG', 'Mammary-Derived Growth Inhibitor-Related', 'Mammary-Derived Growth Inhibitor Related', 'Fatty Acid Binding Protein 7, Brain', 'Hypothetical Protein DKFZp547J2313', 'Fatty Acid-Binding Protein 7', 'Brain Lipid Binding Protein', 'FABP7'], ['Fatty Acid Binding Protein 9', 'T-FABP', 'Testis-Type Fatty Acid-Binding Protein', 'Fatty Acid-Binding Protein 9', 'PERF15', 'PERF', 'TLBP', 'Fatty Acid Binding Protein 9, Testis', 'Testis Fatty Acid Binding Protein', 'Testis Lipid Binding Protein', 'Testis Lipid-Binding Protein', 'FABP9'], ['Fatty Acid Binding Protein 12', 'Fatty Acid-Binding Protein 12', 'Probable Fatty Acid-Binding Protein ENSP00000353650', 'Intracellular Fatty Acid-Binding Protein FABP12', 'Fatty Acid Binding Protein ORF', 'FABP12'], ['Fas Associated Via Death Domain', 'Growth-Inhibiting Gene 3 Protein', 'MORT1', 'Fas-Associating Death Domain-Containing Protein', 'Fas-Associating Protein With Death Domain', 'Fas (TNFRSF6)-Associated Via Death Domain', 'Mediator Of Receptor-Induced Toxicity', 'FAS-Associated Death Domain Protein', 'GIG3', 'FAS-Associating Death Domain-Containing Protein', 'Mediator Of Receptor Induced Toxicity', 'Protein FADD', 'FADD'], ['Fatty Acid Desaturase 1', 'Acyl-CoA (8-3)-Desaturase', 'FADSD5', 'D5D', 'Delta(5) Desaturase', 'Delta-5 Desaturase', 'FADS6', 'TU12', 'Linoleoyl-CoA Desaturase (Delta-6-Desaturase)-Like 1', 'Delta(5) Fatty Acid Desaturase', 'Delta-5 Fatty Acid Desaturase', 'EC 1.14.19.44', 'EC 1.14.19', 'LLCDL1', 'FADS1'], ['Fatty Acid Desaturase 2', 'D6D', 'Acyl-CoA 6-Desaturase', 'Delta-6-Desaturase', 'SLL0262', 'FADSD6', 'DES6', 'TU13', 'Linoleoyl-CoA Desaturase (Delta-6-Desaturase)-Like 2', 'Delta(6) Fatty Acid Desaturase', 'Delta-6 Fatty Acid Desaturase', 'Delta(6) Desaturase', 'Delta-6 Desaturase', 'EC 1.14.19.3', 'EC 1.14.19', 'LLCDL2', 'FADS2'], ['Fatty Acid Desaturase 3', 'CYB5RP', 'Delta(13) Fatty Acid Desaturase', 'Delta(13) Desaturase', 'Delta-9-Desaturase', 'Linoleoyl-CoA Desaturase (Delta-9-Desaturase)-Like 3', 'Cytochrome B5-Related Protein', 'Delta-9 Fatty Acid Desaturase', 'EC 1.14.19.-', 'EC 1.14.19', 'LLCDL3', 'FADS3'], ['Fatty Acid Desaturase 6', 'Fatty Acid Desaturase Domain Family, Member 6', 'EC 1.14.19.-', 'FP18279', 'FADS6'], ['Fas Associated Factor 1', 'UBXD12', 'UBXN3A', 'HFAF1', 'Fas (TNFRSF6) Associated Factor 1', 'UBX Domain-Containing Protein 12', 'UBX Domain-Containing Protein 3A', 'TNFRSF6-Associated Factor 1', 'FAS-Associated Factor 1', 'UBX Domain Protein 3A', 'CGI-03', 'HFAF1s', 'FAF1'], ['Fas Associated Factor Family Member 2', 'UBXN3B', 'ETEA', 'UBX Domain-Containing Protein 3B', 'UBX Domain-Containing Protein 8', 'UBX Domain Containing 8', 'FAS-Associated Factor 2', 'UBX Domain Protein 3B', 'KIAA0887', 'UBXD8', 'Expressed In T Cells And Eosinophils In Atopic Dermatitis', 'Expressed In T-Cells And Eosinophils In Atopic Dermatitis', 'Protein ETEA', 'FAF2'], ['Fumarylacetoacetate Hydrolase', 'Fumarylacetoacetase', 'Beta-Diketonase', 'EC 3.7.1.2', 'FAA', 'Fumarylacetoacetate Hydrolase (Fumarylacetoacetase)', 'Epididymis Secretory Sperm Binding Protein', 'FAH'], ['Fumarylacetoacetate Hydrolase Domain Containing 1', 'Fumarylacetoacetate Hydrolase Domain-Containing Protein 1', 'Acylpyruvase FAHD1, Mitochondrial', 'FAH Domain-Containing Protein 1', 'Oxaloacetate Decarboxylase', 'Acylpyruvate Hydrolase', 'OAA Decarboxylase', 'YisK-Like Protein', 'C16orf36', 'YISKL', 'Chromosome 16 Open Reading Frame 36', 'YISK Like/RJD15', 'DKFZP566J2046', 'EC 4.1.1.112', 'EC 3.7.1.5', 'FAHD1'], ['Fumarylacetoacetate Hydrolase Domain Containing 2A', 'Fumarylacetoacetate Hydrolase Domain-Containing Protein 2A', 'CGI-105', 'Fumarylacetoacetate Hydrolase Domain Containing 1', 'EC 3.-.-.-', 'FAHD2A'], ['Fumarylacetoacetate Hydrolase Domain Containing 2B', 'Fumarylacetoacetate Hydrolase Domain-Containing Protein 2B', 'DKFZp434N062', 'EC 3.-.-.-', 'FAHD2B'], ['Fas Apoptotic Inhibitory Molecule', 'FAIM1', 'Fas Apoptotic Inhibitory Molecule 1', 'FLJ10582', 'FAIM'], ['Fas Apoptotic Inhibitory Molecule 2', 'TMBIM2', 'NMP35', 'LFG2', 'LFG', 'Transmembrane BAX Inhibitor Motif-Containing Protein 2', 'Neural Membrane Protein 35', 'Protein Lifeguard 2', 'KIAA0950', 'Transmembrane BAX Inhibitor Motif Containing 2', 'LIFEGUARD', 'NGP35', 'FAIM2'], ['FAM3 Metabolism Regulating Signaling Molecule A', 'Family With Sequence Similarity 3 Member A', 'Cytokine-Like Protein 2-19', 'Protein FAM3A', 'DXS560S', 'XAP-7', '2.19', '2-19', 'FAM3A', 'DLD'], ['FAM3 Metabolism Regulating Signaling Molecule B', 'C21orf76', 'PANDER', 'Family With Sequence Similarity 3 Member B', 'Cytokine-Like Protein 2-21', 'Pancreatic-Derived Factor', 'Protein FAM3B', 'D21M16SJHU19e', 'C21orf11', 'PRED44', '2-21', 'ORF9', 'Chromosome 21 Open Reading Frame 11', 'Pancreatic Derived Factor', 'FAM3B'], ['FAM3 Metabolism Regulating Signaling Molecule C', 'Interleukin-Like EMT Inducer', 'ILEI', 'Interleukin-Like Epithelial-Mesenchymal Transition Inducer', 'Family With Sequence Similarity 3 Member C', 'Predicted Osteoblast Protein', 'Protein FAM3C', 'GS3786', 'GS3876', 'FAM3C'], ['FAM3 Metabolism Regulating Signaling Molecule D', 'Family With Sequence Similarity 3 Member D', 'Oncoprotein-Induced Transcript 1', 'Protein FAM3D', 'OIT1', 'EF7', 'Cytokine-Like Protein EF-7', 'FAM3D'], ['Family With Sequence Similarity 8 Member A1', 'AHCP', 'Autosomal Highly Conserved Protein', 'Protein FAM8A1', 'Family With Sequence Similarity 8, Member A1', 'FAM8A1'], ['Family With Sequence Similarity 9 Member A', 'Testis Expressed 39A', 'Protein FAM9A', 'TEX39A', 'Family With Sequence Similarity 9, Member A', 'FAM9A'], ['Family With Sequence Similarity 9 Member B', 'Testis Expressed 39B', 'Protein FAM9B', 'TEX39B', 'Family With Sequence Similarity 9, Member B', 'FAM9B'], ['Family With Sequence Similarity 9 Member C', 'Testis Expressed 39C', 'Protein FAM9C', 'TEX39C', 'Family With Sequence Similarity 9, Member C', 'FAM9C'], ['Family With Sequence Similarity 13 Member A', 'Family With Sequence Similarity 13, Member A1', 'Protein FAM13A', 'ARHGAP48', 'KIAA0914', 'FAM13A1', 'Family With Sequence Similarity 13, Member A', 'FAM13A1_v2 Protein', 'FAM13A'], ['Family With Sequence Similarity 13 Member B', 'Family With Sequence Similarity 13, Member B1', 'GAP-Like Protein N61', 'Protein FAM13B', 'ARHGAP49', 'FAM13B1', 'C5orf5', 'KHCHP', 'N61', 'Family With Sequence Similarity 13, Member B', 'Chromosome 5 Open Reading Frame 5', 'FAM13B'], ['Family With Sequence Similarity 13 Member C', 'Family With Sequence Similarity 13, Member C1', 'Protein FAM13C', 'FAM13C1', 'Family With Sequence Similarity 13, Member C', 'FAM13C'], ['FAM20A Golgi Associated Secretory Pathway Pseudokinase', 'Family With Sequence Similarity 20, Member A', 'Pseudokinase FAM20A', 'FAM20A, Golgi Associated Secretory Pathway Pseudokinase', 'Protein FAM20A', 'DKFZp434F2322', 'FP2747', 'FAM20A', 'AIGFS', 'AI1G'], ['FAM20B Glycosaminoglycan Xylosylkinase', 'Family With Sequence Similarity 20, Member B', 'Glycosaminoglycan Xylosylkinase', 'Xylose Kinase', 'KIAA0475', 'FAM20B, Glycosaminoglycan Xylosylkinase', 'EC 2.7.1.-', 'FAM20B', 'Gxk1', 'GXK1'], ['FAM20C Golgi Associated Secretory Pathway Kinase', 'Dentin Matrix Protein 4', 'Golgi Casein Kinase', 'DMP4', 'Extracellular Serine/Threonine Protein Kinase FAM20C', 'Family With Sequence Similarity 20, Member C', 'Golgi-Enriched Fraction Casein Kinase', 'GEF-CK', 'DMP-4', 'G-CK', 'FAM20C, Golgi Associated Secretory Pathway Kinase', 'IMAGE:4942737', 'DKFZp547D065', 'EC 2.7.11.1', 'FAM20C', 'RNS'], ['Family With Sequence Similarity 24 Member A', 'Protein FAM24A', 'Family With Sequence Similarity 24, Member A', 'AC073585.4', 'FAM24A'], ['Family With Sequence Similarity 24 Member B', 'Protein FAM24B', 'Family With Sequence Similarity 24, Member B', 'AC073585.2', 'MGC45962', 'FAM24B'], ['Family With Sequence Similarity 25 Member A', 'Protein FAM25A', 'BA96C23.5', 'Family With Sequence Similarity 25, Member A', 'Protein FAM25C', 'Protein FAM25G', 'Protein FAM25', 'FAM25A'], ['Family With Sequence Similarity 25 Member C', 'Protein FAM25C', 'BA164N7.4', 'Family With Sequence Similarity 25, Member C', 'Protein FAM25G', 'Protein FAM25', 'FAM25C'], ['Family With Sequence Similarity 25 Member G', 'Protein FAM25G', 'BA301J7.4', 'Family With Sequence Similarity 25, Member G', 'Protein FAM25C', 'Protein FAM25', 'FAM25A', 'FAM25B', 'FAM25C', 'FAM25G'], ['Family With Sequence Similarity 32 Member A', 'OTAG-12', 'Ovarian Tumor Associated Gene-12', 'Protein FAM32A', 'OTAG12', 'Family With Sequence Similarity 32, Member A', 'Ovarian Tumor-Associated Gene 12', 'DKFZP586O0120', 'FAM32A'], ['Family With Sequence Similarity 43 Member A', 'Protein FAM43A', 'Family With Sequence Similarity 43, Member A', 'FLJ90022', 'FAM43A'], ['Family With Sequence Similarity 43 Member B', 'Protein FAM43B', 'Family With Sequence Similarity 43, Member B', 'FLJ44952', 'FAM43B'], ['Family With Sequence Similarity 47 Member A', 'Protein FAM47A', 'Family With Sequence Similarity 47, Member A', 'Similar To Hypothetical Protein FLJ35782', 'MGC27003', 'FAM47A'], ['Family With Sequence Similarity 47 Member B', 'Protein FAM47B', 'Family With Sequence Similarity 47, Member B', 'FLJ35782', 'FAM47B'], ['Family With Sequence Similarity 47 Member C', 'Putative Protein FAM47C', 'Family With Sequence Similarity 47, Member C', 'FAM47C'], ['Family With Sequence Similarity 47 Member E', 'Protein FAM47E', 'Family With Sequence Similarity 47, Member E', 'Similar To Genethonin 1', 'LOC100129583', 'FLJ42946', 'FAM47E'], ['Family With Sequence Similarity 50 Member A', 'DXS9928E', 'XAP5', 'Protein FAM50A', 'Protein XAP-5', 'HXC-26', 'HXC26', '9F', 'DNA Segment On Chromosome X (Unique) 9928 Expressed Sequence', 'Family With Sequence Similarity 50, Member A', 'Protein HXC-26', 'FAM50A', 'MRXSA'], ['Family With Sequence Similarity 50 Member B', 'X5L', 'Protein FAM50B', 'XAP5 Like', 'D6S2654E', 'Family With Sequence Similarity 50, Member B', 'Protein XAP-5-Like', 'XAP5-Like Protein', 'FAM50B'], ['Family With Sequence Similarity 53 Member A', 'Dorsal Neural-Tube Nuclear Protein', 'Protein FAM53A', 'DNTNP', 'Family With Sequence Similarity 53, Member A', 'Dorsal Neural Tube Nuclear Protein', 'FAM53A'], ['Family With Sequence Similarity 53 Member B', 'KIAA0140', 'Protein Simplet', 'Protein FAM53B', 'BA12J10.2', 'Smp', 'Family With Sequence Similarity 53, Member B', 'Simplet Homolog (Zebrafish)', 'Simplet Homolog', 'FAM53B', 'SMP'], ['Family With Sequence Similarity 53 Member C', 'Protein FAM53C', 'C5orf6', 'Family With Sequence Similarity 53, Member C', 'Chromosome 5 Open Reading Frame 6', 'Putative Nuclear Protein', 'FAM53C'], ['Family With Sequence Similarity 71 Member A', 'Protein FAM71A', 'Testis Secretory Sperm-Binding Protein Li 209a', 'Family With Sequence Similarity 71, Member A', 'Golgi-Associated Rab2B Interactor-Like 4', 'FLJ32796', 'GARI-L4', 'FAM71A'], ['Family With Sequence Similarity 71 Member B', 'Protein FAM71B', 'Family With Sequence Similarity 71, Member B', 'Testicular Tissue Protein Li 68', 'MGC26988', 'FAM71B'], ['Family With Sequence Similarity 71 Member C', 'Protein FAM71C', 'Family With Sequence Similarity 71, Member C', 'MGC39520', 'FAM71C'], ['Family With Sequence Similarity 71 Member D', 'Protein FAM71D', 'C14orf54', 'Family With Sequence Similarity 71, Member D', 'Chromosome 14 Open Reading Frame 54', 'FAM71D'], ['Family With Sequence Similarity 71 Member E1', 'Protein FAM71E1', 'Family With Sequence Similarity 71, Member E1', 'FAM71E1'], ['Family With Sequence Similarity 71 Member E2', 'Protein FAM71E2', 'C19orf16', 'Family With Sequence Similarity 71, Member E2', 'Chromosome 19 Open Reading Frame 16', 'Putative Protein FAM71E2', 'DKFZp434G1729', 'FAM71E2'], ['Family With Sequence Similarity 71 Member F1', 'Family With Sequence Similarity 137, Member A', 'Testis Development Protein NYD-SP18', 'Protein FAM71F1', 'NYD-SP18', 'FAM137A', 'Family With Sequence Similarity 71, Member F1', 'Testes Development-Related NYD-SP18', 'Testicular Tissue Protein Li 67', 'Protein FAM137A', 'FAM71F1'], ['Family With Sequence Similarity 71 Member F2', 'Family With Sequence Similarity 137, Member B', 'Protein FAM71F2', 'FAM137B', 'Family With Sequence Similarity 71, Member F2', 'Protein FAM137B', 'FAM71F2'], ['Family With Sequence Similarity 72 Member A', 'LMP1-Induced Protein', 'LMPIP', 'Latent Membrane Protein 1-Induced Protein', 'Protein FAM72A', 'UGENE', 'P17', 'Family With Sequence Similarity 72, Member A', 'RP11-312O7.1', 'MGC57827', 'FAM72A', 'Ugene'], ['Family With Sequence Similarity 72 Member B', 'Protein FAM72B', 'RP11-439A17.6', 'Family With Sequence Similarity 72, Member B', 'Amyloid-Beta Peptide-Induced Protein P17', 'FAM72B', 'P17'], ['Family With Sequence Similarity 72 Member C', 'Protein FAM72C', 'RP5-998N21.9', 'Family With Sequence Similarity 72, Member C', 'FAM72C'], ['Family With Sequence Similarity 72 Member D', 'Gastric Cancer Up-Regulated Protein 2', 'Protein FAM72D', 'GCUD2', 'Family With Sequence Similarity 72, Member D', 'Gastric Cancer Up-Regulated-2', 'FAM72D'], ['Family With Sequence Similarity 76 Member A', 'Protein FAM76A', 'Family With Sequence Similarity 76, Member A', 'MGC34648', 'FAM76A'], ['Family With Sequence Similarity 76 Member B', 'Protein FAM76B', 'Family With Sequence Similarity 76, Member B', 'MGC33371', 'FAM76B'], ['Family With Sequence Similarity 78 Member A', 'Protein FAM78A', 'C9orf59', 'Family With Sequence Similarity 78, Member A', 'Chromosome 9 Open Reading Frame 59', 'FLJ00024', 'FAM78A'], ['Family With Sequence Similarity 78 Member B', 'Protein FAM78B', 'Family With Sequence Similarity 78, Member B', 'FAM78B'], ['Family With Sequence Similarity 81 Member A', 'Protein FAM81A', 'Family With Sequence Similarity 81, Member A', 'MGC26690', 'FAM81A'], ['Family With Sequence Similarity 81 Member B', 'Protein FAM81B', 'Family With Sequence Similarity 81, Member B', 'FLJ25333', 'FAM81B'], ['Family With Sequence Similarity 83 Member A', 'Tumor-Specific Gene Expressed In Prostate Protein', 'Tumor Antigen BJ-TSA-9', 'Protein FAM83A', 'BJ-TSA-9', 'Family With Sequence Similarity 83, Member A', 'MGC14128', 'FAM83A', 'TSGP'], ['Family With Sequence Similarity 83 Member B', 'Protein FAM83B', 'C6orf143', 'Family With Sequence Similarity 83, Member B', 'Chromosome 6 Open Reading Frame 143', 'FLJ30642', 'FAM83B'], ['Family With Sequence Similarity 83 Member C', 'Protein FAM83C', 'C20orf128', 'DJ614O4.7', 'Family With Sequence Similarity 83, Member C', 'Chromosome 20 Open Reading Frame 128', 'EC 6.3.5.3', 'EC 1.6.5', 'FAM83C'], ['Family With Sequence Similarity 83 Member D', 'Spindle Protein CHICA', 'Protein FAM83D', 'C20orf129', 'DJ616B8.3', 'CHICA', 'Family With Sequence Similarity 83, Member D', 'Chromosome 20 Open Reading Frame 129', 'FAM83D'], ['Family With Sequence Similarity 83 Member E', 'Protein FAM83E', 'Family With Sequence Similarity 83, Member E', 'FLJ20200', 'FAM83E'], ['Family With Sequence Similarity 83 Member F', 'Protein FAM83F', 'Family With Sequence Similarity 83, Member F', 'FAM83F'], ['Family With Sequence Similarity 83 Member G', 'Protein Associated With SMAD1', 'PAWS1', 'Protein FAM83G', 'Family With Sequence Similarity 83, Member G', 'FLJ41564', 'FAM83G'], ['Family With Sequence Similarity 83 Member H', 'Protein FAM83H', 'Family With Sequence Similarity 83, Member H', 'FAM83H Variant 1', 'FAM83H Variant 2', 'Truncated FAM83H', 'FLJ46072', 'FAM83H', 'AI3A', 'AI3'], ['Family With Sequence Similarity 86 Member B1', 'Putative Protein N-Methyltransferase FAM86B1', 'Family With Sequence Similarity 86, Member B1', 'Protein FAM86B1', 'EC 2.1.1.-', 'MGC16279', 'FAM86B1'], ['Family With Sequence Similarity 86 Member B2', 'Putative Protein N-Methyltransferase FAM86B2', 'Family With Sequence Similarity 86, Member B2', 'Protein FAM86B2', 'EC 2.1.1.-', 'FAM86B2'], ['Family With Sequence Similarity 89 Member A', 'Protein FAM89A', 'C1orf153', 'Family With Sequence Similarity 89, Member A', 'Chromosome 1 Open Reading Frame 153', 'MGC15887', 'FAM89A'], ['Family With Sequence Similarity 89 Member B', 'Mammary Tumor Virus Receptor Homolog 1', 'Leucine Repeat Adaptor Protein 25', 'Leucine Repeat Adapter Protein 25', 'LRAP25', 'MTVR1', 'Family With Sequence Similarity 89, Member B', 'Protein FAM89B', 'FAM89B', 'Lrap25', 'MTVR'], ['Family With Sequence Similarity 90 Member A1', 'Protein FAM90A1', 'Family With Sequence Similarity 90, Member A1', 'FLJ10408', 'FAM90A1'], ['Family With Sequence Similarity 90 Member A26', 'Family With Sequence Similarity 90, Member A26, Pseudogene', 'Putative Protein FAM90A26', 'FAM90A26P', 'Family With Sequence Similarity 90, Member A26', 'FAM90A26'], ['Family With Sequence Similarity 91 Member A1', 'Protein FAM91A1', 'Family With Sequence Similarity 91, Member A1', 'Skeletal Muscle Cells Re-Entry Induced', 'FLJ23790', 'FAM91A1'], ['Family With Sequence Similarity 98 Member A', 'Protein FAM98A', 'Family With Sequence Similarity 98, Member A', 'DKFZP564F0522', 'FAM98A'], ['Family With Sequence Similarity 98 Member B', 'Protein FAM98B', 'Family With Sequence Similarity 98, Member B', 'FLJ38426', 'FAM98B'], ['Family With Sequence Similarity 98 Member C', 'Protein FAM98C', 'Family With Sequence Similarity 98, Member C', 'FLJ44669', 'FAM98C'], ['Family With Sequence Similarity 102 Member A', 'Early Estrogen-Induced Gene 1 Protein', 'Protein FAM102A', 'BA203J24.7', 'C9orf132', 'SYM-3A', 'EEIG1', 'Family With Sequence Similarity 102, Member A', 'Chromosome 9 Open Reading Frame 132', 'Early Estrogen-Induced Gene 1', 'Sym-3 Homolog A (C. Elegans)', 'Sym-3 Homolog A', 'C230093N12Rik', 'FAM102A', 'Eeig1'], ['Family With Sequence Similarity 102 Member B', 'Protein FAM102B', 'SYM-3B', 'Family With Sequence Similarity 102, Member B', 'Sym-3 Homolog B (C. Elegans)', 'Sym-3 Homolog B', 'DKFZp779B126', 'FAM102B'], ['Family With Sequence Similarity 104 Member A', 'Protein FAM104A', 'Family With Sequence Similarity 104, Member A', 'FLJ14775', 'FAM104A'], ['Family With Sequence Similarity 104 Member B', 'Protein FAM104B', 'CXorf44', 'Family With Sequence Similarity 104, Member B', 'Chromosome X Open Reading Frame 44', 'FLJ20434', 'FAM104B'], ['Family With Sequence Similarity 107 Member A', 'DRR1', 'TU3A', 'Down-Regulated In Renal Cell Carcinoma 1', 'Actin-Associated Protein FAM107A', 'Family With Sequence Similarity 107, Member A', 'Down-Regulated In Renal Cell Carcinoma', 'Protein FAM107A', 'Protein TU3A', 'FAM107A'], ['Family With Sequence Similarity 107 Member B', 'Heat Shock-Inducible Tumor Small Protein', 'Protein FAM107B', 'C10orf45', 'HITS', 'Family With Sequence Similarity 107, Member B', 'Chromosome 10 Open Reading Frame 45', 'FAM107B/CDNF Fusion', 'FLJ45505', 'MGC11034', 'FAM107B'], ['Family With Sequence Similarity 110 Member A', 'Protein FAM110A', 'BA371L19.3', 'C20orf55', 'F10', 'Family With Sequence Similarity 110, Member A', 'Chromosome 20 Open Reading Frame 55', 'FAM110A'], ['Family With Sequence Similarity 110 Member B', 'Protein FAM110B', 'C8orf72', 'Family With Sequence Similarity 110, Member B', 'Chromosome 8 Open Reading Frame 72', 'MGC39325', 'FAM110B'], ['Family With Sequence Similarity 110 Member C', 'Protein FAM110C', 'Family With Sequence Similarity 110, Member C', 'FAM110C'], ['Family With Sequence Similarity 110 Member D', 'Protein FAM110D', 'GRRP1', 'Family With Sequence Similarity 110, Member D', 'Glycine/Arginine Rich Protein 1', 'Glycine/Arginine-Rich Protein 1', 'FLJ14050', 'FAM110D'], ['FAM111 Trypsin Like Peptidase A', 'Family With Sequence Similarity 111 Member A', 'Serine Protease FAM111A', 'KIAA1895', 'Protein FAM111A', 'EC 3.4.21.-', 'FLJ22794', 'FAM111A', 'GCLEB', 'KCS2'], ['FAM111 Trypsin Like Peptidase B', 'Family With Sequence Similarity 111 Member B', 'CANP', 'Cancer-Associated Nucleoprotein', 'Serine Protease FAM111B', 'EC 3.4.21.-', 'POIKTMP', 'FAM111B'], ['Family With Sequence Similarity 114 Member A1', 'Protein NOXP20', 'Noxp20', 'Family With Sequence Similarity 114, Member A1', 'Nervous System Over-Expressed Protein 20', 'Nervous System Overexpressed Protein 20', 'Protein FAM114A1', 'FAM114A1', 'NOXP20'], ['Family With Sequence Similarity 114 Member A2', 'Protein FAM114A2', '133K02', 'C5orf3', 'Family With Sequence Similarity 114, Member A2', 'Testis Tissue Sperm-Binding Protein Li 81P', 'Chromosome 5 Open Reading Frame 3', 'FAM114A2'], ['Family With Sequence Similarity 117 Member A', 'C/EBP-Induced Protein', 'Protein FAM117A', 'Family With Sequence Similarity 117, Member A', 'C/EBP Induced Protein', 'FAM117A'], ['Family With Sequence Similarity 117 Member B', 'Amyotrophic Lateral Sclerosis 2 Chromosomal Region Candidate Gene 13 Protein', 'Amyotrophic Lateral Sclerosis 2 (Juvenile) Chromosome Region, Candidate 13', 'Protein FAM117B', 'ALS2CR13', 'Family With Sequence Similarity 117, Member B', 'FLJ38771', 'FAM117B'], ['Family With Sequence Similarity 118 Member A', 'Protein FAM118A', 'BK268H5.C22.4', 'C22orf8', 'Family With Sequence Similarity 118, Member A', 'Chromosome 22 Open Reading Frame 8', 'FLJ20635', 'FAM118A'], ['Family With Sequence Similarity 118 Member B', 'Protein FAM118B', 'Family With Sequence Similarity 118, Member B', 'FLJ21103', 'FAM118B'], ['Family With Sequence Similarity 120A', 'OSSA', 'Constitutive Coactivator Of PPAR-Gamma-Like Protein 1', 'Oxidative Stress-Associated Src Activator', 'Oxidative Stess-Associated Src Activator', 'KIAA0183', 'C9orf10', 'Chromosome 9 Open Reading Frame 10', 'Protein FAM120A', 'HBVPTPAP', 'FAM120A'], ['Family With Sequence Similarity 120A Opposite Strand', 'Uncharacterized Protein FAM120AOS', 'FAM120A Opposite Strand Protein', 'C9orf10OS', 'Chromosome 9 Open Reading Frame 10 Opposite Strand', 'FAM120AOS'], ['Family With Sequence Similarity 120B', 'Constitutive Coactivator Of PPAR-Gamma', 'KIAA1838', 'PGCC1', 'CCPG', 'Constitutive Coactivator Of Peroxisome Proliferator-Activated Receptor Gamma', 'PPARgamma Constitutive Coactivator 1', 'PPARG Constitutive Coactivator 1', 'Senataxin-Associated Nuclease 1', 'SAN1', 'Constitutive Coactivator Of PPARG', 'Protein FAM120B', 'DJ894D12.1', 'FAM120B'], ['Family With Sequence Similarity 120C', 'Constitutive Coactivator Of PPAR-Gamma-Like Protein 2', 'Tumor Antigen BJ-HCC-21', 'CXorf17', 'ORF34', 'Chromosome X Open Reading Frame 17', 'Protein FAM120C', 'FLJ20506', 'FAM120C'], ['Family With Sequence Similarity 124 Member A', 'Family With Sequence Similarity 124A', 'Protein FAM124A', 'FLJ30707', 'FAM124A'], ['Family With Sequence Similarity 124 Member B', 'Family With Sequence Similarity 124B', 'Protein FAM124B', 'FLJ22746', 'FAM124B'], ['Family With Sequence Similarity 126 Member A', 'DRCTNNB1A', 'Hyccin', 'Down-Regulated By CTNNB1 Protein A', 'Down Regulated By Ctnnb1, A', 'HYCC1', 'HCC', 'Family With Sequence Similarity 126, Member A', 'Protein FAM126A', 'FAM126A', 'HLD5'], ['Family With Sequence Similarity 126 Member B', 'Protein FAM126B', 'HYCC2', 'Family With Sequence Similarity 126, Member B', 'MGC39518', 'FAM126B'], ['Family With Sequence Similarity 131 Member A', 'Protein FAM131A', 'C3orf40', 'Family With Sequence Similarity 131, Member A', 'Chromosome 3 Open Reading Frame 40', 'MGC21688', 'FLAT715', 'PRO1378', 'FAM131A'], ['Family With Sequence Similarity 131 Member B', 'Protein FAM131B', 'KIAA0773', 'Family With Sequence Similarity 131, Member B', 'FAM131B'], ['Family With Sequence Similarity 131 Member C', 'Protein FAM131C', 'C1orf117', 'Family With Sequence Similarity 131, Member C', 'Chromosome 1 Open Reading Frame 117', 'FLJ36766', 'FAM131C'], ['Family With Sequence Similarity 133 Member A', 'Cancer/Testis Antigen 115', 'Protein FAM133A', 'CT115', 'Family With Sequence Similarity 133, Member A', 'RP1-32F7.2', 'FLJ37659', 'FAM133A'], ['Family With Sequence Similarity 133 Member B', 'Protein FAM133B', 'Family With Sequence Similarity 133, Member B', 'MGC40405', 'FAM133B'], ['Family With Sequence Similarity 135 Member A', 'KIAA1411', 'Protein FAM135A', 'Family With Sequence Similarity 135, Member A', 'FLJ20176', 'FAM135A'], ['Family With Sequence Similarity 135 Member B', 'Protein FAM135B', 'C8ORFK32', 'Family With Sequence Similarity 135, Member B', 'C8orfK32', 'FAM135B'], ['Family With Sequence Similarity 136 Member A', 'Protein FAM136A', 'Family With Sequence Similarity 136, Member A', 'Hypothetical Protein FLJ14668', 'FLJ14668', 'FAM136A'], ['Family With Sequence Similarity 149 Member A', 'Protein FAM149A', 'MSTP119', 'MST119', 'Family With Sequence Similarity 149, Member A', 'DKFZP564J102', 'FAM149A'], ['Family With Sequence Similarity 149 Member B1', 'KIAA0974', 'Primary Cilium Assembly Protein FAM149B1', 'Family With Sequence Similarity 149, Member B1', 'Protein FAM149B1', 'FAM149B1', 'JBTS36'], ['Family With Sequence Similarity 151 Member A', 'Protein FAM151A', 'C1orf179', 'Family With Sequence Similarity 151, Member A', 'Chromosome 1 Open Reading Frame 179', 'Hyporthetical Protein MGC27169', 'MGC27169', 'FAM151A'], ['Family With Sequence Similarity 151 Member B', 'Protein FAM151B', 'UNQ9217', 'Family With Sequence Similarity 151, Member B', 'AASA9217', 'FAM151B'], ['Family With Sequence Similarity 153 Member A', 'Renal Carcinoma Antigen NY-REN-7', 'NY REN 7 Antigen', 'Protein FAM153A', 'NY-REN-7', 'Family With Sequence Similarity 153, Member A', 'KIAA0752', 'FAM153A'], ['Family With Sequence Similarity 153 Member B', 'Protein FAM153B', 'Family With Sequence Similarity 153, Member B', 'FAM153B'], ['Family With Sequence Similarity 156 Member A', 'Transmembrane Protein 29/29B', 'Transmembrane Protein 29', 'Protein FAM156A/FAM156B', 'PRO0659', 'TMEM29', 'Family With Sequence Similarity 156, Member A', 'Protein FAM156A', 'FAM156A', 'TMEM29B'], ['Family With Sequence Similarity 156 Member B', 'Transmembrane Protein 29/29B', 'Transmembrane Protein 29B', 'Protein FAM156A/FAM156B', 'TMEM29B', 'Family With Sequence Similarity 156, Member B', 'Transmembrane Protein 29', 'Protein FAM156A', 'FAM156B', 'TMEM29'], ['FAM161 Centrosomal Protein A', 'Retinitis Pigmentosa 28 (Autosomal Recessive)', 'Family With Sequence Similarity 161 Member A', 'Protein FAM161A', 'Family With Sequence Similarity 161, Member A', 'FAM161A, Centrosomal Protein', 'FLJ13305', 'FAM161A', 'RP28'], ['FAM161 Centrosomal Protein B', 'Family With Sequence Similarity 161 Member B', 'FAM161B, Centrosomal Protein', 'Protein FAM161B', 'C14orf44', 'Family With Sequence Similarity 161, Member B', 'Chromosome 14 Open Reading Frame 44', 'C14_5547', 'FLJ31697', 'FAM161B'], ['Family With Sequence Similarity 162 Member A', 'E2IG5', 'Growth And Transformation-Dependent Protein', 'E2-Induced Gene 5 Protein', 'Protein FAM162A', 'C3orf28', 'HGTD-P', 'Family With Sequence Similarity 162, Member A', 'HIF-1 Alpha-Responsive Proapoptotic Molecule', 'Chromosome 3 Open Reading Frame 28', 'FAM162A'], ['Family With Sequence Similarity 162 Member B', 'Protein FAM162B', 'C6orf189', 'BA86F4.2', 'Family With Sequence Similarity 162, Member B', 'Chromosome 6 Open Reading Frame 189', 'FAM162B'], ['Family With Sequence Similarity 163 Member A', 'Cebelin', 'NDSP', 'Neuroblastoma Derived Secretory Protein', 'Protein FAM163A', 'C1orf76', 'Family With Sequence Similarity 163, Member A', 'Neuroblastoma-Derived Secretory Protein', 'Chromosome 1 Open Reading Frame 76', 'MGC16664', 'FAM163A'], ['Family With Sequence Similarity 163 Member B', 'Protein FAM163B', 'Cebelin-Like', 'C9orf166', 'Family With Sequence Similarity 163, Member B', 'Chromosome 9 Open Reading Frame 166', 'FAM163B'], ['Family With Sequence Similarity 166 Member A', 'Protein FAM166A', 'Family With Sequence Similarity 166, Member A', 'FAM166A', 'HSD46'], ['Family With Sequence Similarity 166 Member B', 'Protein FAM166B', 'Family With Sequence Similarity 166, Member B', 'FAM166B'], ['Family With Sequence Similarity 166 Member C', 'Protein FAM166C', 'C2orf70', 'Chromosome 2 Open Reading Frame 70', 'Hypothetical Protein LOC339778', 'UPF0573 Protein C2orf70', 'LOC339778', 'FAM166C'], ['Family With Sequence Similarity 167 Member A', 'Disordered Autoimmunity 1', 'Protein FAM167A', 'C8orf13', 'DIORA-1', 'Family With Sequence Similarity 167, Member A', 'Chromosome 8 Open Reading Frame 13', 'FAM167A', 'D8S265'], ['Family With Sequence Similarity 167 Member B', 'Disordered Autoimmunity 2', 'Protein FAM167B', 'C1orf90', 'DIORA-2', 'Family With Sequence Similarity 167, Member B', 'Chromosome 1 Open Reading Frame 90', 'MGC10820', 'FAM167B'], ['Family With Sequence Similarity 168 Member A', 'Tongue Cancer Chemotherapy Resistance-Associated Protein 1', 'KIAA0280', 'TCRP1', 'Protein FAM168A', 'Family With Sequence Similarity 168, Member A', 'FAM168A'], ['Family With Sequence Similarity 168 Member B', 'MANI', 'Myelin-Associated Neurite-Outgrowth Inhibitor', 'KIAA0280L', 'Myelin Associated Neurite Outgrowth Inhibitor', 'Family With Sequence Similarity 168, Member B', 'Protein FAM168B', 'FAM168B', 'Mani', 'P20'], ['Family With Sequence Similarity 169 Member A', 'SLAP75', 'Soluble Lamina-Associated Protein Of 75 KD', 'Soluble Lamin-Associated Protein Of 75 KDa', 'Protein FAM169A', 'KIAA0888', 'Family With Sequence Similarity 169, Member A', 'FAM169A'], ['Family With Sequence Similarity 169 Member B', 'Protein FAM169B', 'KIAA0888L', 'Family With Sequence Similarity 169, Member B', 'FLJ39743', 'FAM169B'], ['Family With Sequence Similarity 170 Member A', 'ZNFD', 'Zinc Finger Domain-Containing Protein', 'Zinc Finger Protein ZNFD', 'Protein FAM170A', 'Family With Sequence Similarity 170, Member A', 'Znf Domain Containing Protein', 'FAM170A'], ['Family With Sequence Similarity 170 Member B', 'Acrosome-Related Protein', 'Protein FAM170B', 'C10orf73', 'Family With Sequence Similarity 170, Member B', 'Chromosome 10 Open Reading Frame 73', 'Putative Protein FAM170B', 'AC084727.4', 'FAM170B'], ['Family With Sequence Similarity 171 Member A1', 'Astroprincin', 'APCN', 'Protein FAM171A1', 'C10orf38', 'Family With Sequence Similarity 171, Member A1', 'Chromosome 10 Open Reading Frame 38', 'FAM171A1', 'FLJ12884'], ['Family With Sequence Similarity 171 Member A2', 'Protein FAM171A2', 'Family With Sequence Similarity 171, Member A2', 'FAM171A2', 'MGC34829'], ['Family With Sequence Similarity 171 Member B', 'KIAA1946', 'Protein FAM171B', 'Family With Sequence Similarity 171, Member B', 'FLJ34104', 'FAM171B'], ['Family With Sequence Similarity 172 Member A', 'Cotranscriptional Regulator FAM172A', 'Protein FAM172A', 'C5orf21', 'Toupee', 'Family With Sequence Similarity 172, Member A', 'Chromosome 5 Open Reading Frame 21', 'DKFZP564D172', 'FAM172A'], ['Family With Sequence Similarity 174 Member A', 'Transmembrane Protein 157', 'NS5ATP6', 'Hepatitis C Virus Nonstructural Protein 5A Trans-Activated Protein 6', 'Hepatitis C Virus NS5A-Transactivated Protein 6', 'HCV NS5A-Transactivated Protein 6', 'Membrane Protein FAM174A', 'TMEM157', 'UNQ1912', 'Family With Sequence Similarity 174, Member A', 'HGS_RE408', 'FAM174A'], ['Family With Sequence Similarity 174 Member B', 'Membrane Protein FAM174B', 'Family With Sequence Similarity 174, Member B', 'LOC400451', 'MGC102891', 'FAM174B'], ['Family With Sequence Similarity 174 Member C', 'Protein FAM174C', 'C19orf24', 'Chromosome 19 Open Reading Frame 24', 'FLJ20640', 'FAM174C'], ['Family With Sequence Similarity 177 Member A1', 'Protein FAM177A1', 'C14orf24', 'Family With Sequence Similarity 177, Member A1', 'Chromosome 14 Open Reading Frame 24', 'FAM177A1'], ['Family With Sequence Similarity 177 Member B', 'Protein FAM177B', 'Family With Sequence Similarity 177, Member B', 'RP11-452F19.2', 'FLJ43505', 'FAM177B'], ['Family With Sequence Similarity 178 Member B', 'Protein FAM178B', 'Family With Sequence Similarity 178, Member B', 'LOC51252', 'FAM178B'], ['Family With Sequence Similarity 180 Member A', 'Protein FAM180A', 'HWKM1940', 'UNQ1940', 'Family With Sequence Similarity 180, Member A', 'FAM180A'], ['Family With Sequence Similarity 180 Member B', 'Hypothetical Gene Supported By BC065704', 'Protein FAM180B', 'Family With Sequence Similarity 180, Member B', 'LOC399888', 'FAM180B'], ['Family With Sequence Similarity 181 Member A', 'Protein FAM181A', 'C14orf152', 'Family With Sequence Similarity 181, Member A', 'Chromosome 14 Open Reading Frame 152', 'FAM181A'], ['Family With Sequence Similarity 181 Member B', 'Protein FAM181B', 'Family With Sequence Similarity 181, Member B', 'LOC220382', 'MGC33846', 'FAM181B'], ['Family With Sequence Similarity 183 Member A', 'Protein FAM183A', 'HCG23177', 'Family With Sequence Similarity 183, Member A', 'LOC440585', 'FAM183A'], ['Family With Sequence Similarity 184 Member A', 'Protein FAM184A', 'C6orf60', 'Family With Sequence Similarity 184, Member A', 'Chromosome 6 Open Reading Frame 60', 'FLJ13942', 'FAM184A'], ['Family With Sequence Similarity 184 Member B', 'Protein FAM184B', 'KIAA1276', 'Family With Sequence Similarity 184, Member B', 'FAM184B'], ['Family With Sequence Similarity 185 Member A', 'Protein FAM185A', 'Family With Sequence Similarity 185, Member A', 'MGC35361', 'FAM185A'], ['Family With Sequence Similarity 186 Member A', 'Protein FAM186A', 'Family With Sequence Similarity 186, Member A', 'LOC121006', 'FAM186A'], ['Family With Sequence Similarity 186 Member B', 'Protein FAM186B', 'C12orf25', 'Family With Sequence Similarity 186, Member B', 'Chromosome 12 Open Reading Frame 25', 'DKFZP434J0113', 'FAM186B'], ['Family With Sequence Similarity 187 Member A', 'Ig-Like V-Type Domain-Containing Protein FAM187A', 'Family With Sequence Similarity 187, Member A', 'FAM187A'], ['Family With Sequence Similarity 187 Member B', 'Transmembrane Protein 162', 'Protein FAM187B', 'TMEM162', 'Family With Sequence Similarity 187, Member B', 'FLJ25660', 'FAM187B'], ['Family With Sequence Similarity 189 Member A1', 'Transmembrane Protein 228', 'TMEM228', 'Protein FAM189A1', 'KIAA0574', 'Family With Sequence Similarity 189, Member A1', 'FAM189A1'], ['Family With Sequence Similarity 189 Member A2', 'X123', 'Protein FAM189A2', 'C9orf61', 'Family With Sequence Similarity 189, Member A2', 'Chromosome 9 Open Reading Frame 61', 'Friedreich Ataxia Region Gene X123', 'Protein X123', 'FAM189A2', 'ENTREP'], ['Family With Sequence Similarity 189 Member B', 'Protein FAM189B', 'C1orf2', 'COTE1', 'Family With Sequence Similarity 189, Member B', 'Chromosome 1 Open Reading Frame 2', 'Protein COTE1', 'FAM189B', 'Cote1'], ['Family With Sequence Similarity 193 Member A', 'Protein FAM193A', 'RES4-22', 'C4orf8', 'Family With Sequence Similarity 193, Member A', 'Chromosome 4 Open Reading Frame 8', 'Protein IT14', 'FAM193A'], ['Family With Sequence Similarity 193 Member B', 'IRIZIO', 'Protein FAM193B', 'KIAA1931', 'Family With Sequence Similarity 193, Member B', 'FLJ10404', 'FAM193B'], ['Family With Sequence Similarity 199, X-Linked', 'Protein FAM199X', 'CXorf39', 'Chromosome X Open Reading Frame 39', 'FAM199X'], ['Family With Sequence Similarity 200 Member A', 'Protein FAM200A', 'C7orf38', 'Family With Sequence Similarity 200, Member A', 'Chromosome 7 Open Reading Frame 38', 'DKFZp727G131', 'FLJ36794', 'FAM200A'], ['Family With Sequence Similarity 200 Member B', 'C4orf53', 'Protein FAM200B', 'Family With Sequence Similarity 200, Member B', 'Chromosome 4 Open Reading Frame 53', 'Putative Protein FAM200B', 'FAM200B'], ['Family With Sequence Similarity 204 Member A', 'Protein FAM204A', 'BA319I23.1', 'C10orf84', 'Family With Sequence Similarity 204, Member A', 'Chromosome 10 Open Reading Frame 84', 'FLJ13188', 'FAM204A'], ['Family With Sequence Similarity 205 Member A', 'C9orf144B', 'Protein FAM205A', 'Family With Sequence Similarity 205, Member A', 'Transmembrane Protein C9orf144B', 'FAM205A'], ['Family With Sequence Similarity 205 Member C', 'Family With Sequence Similarity 205, Member C, Pseudogene', 'FAM205C Pseudogene', 'Protein FAM205C', 'FAM205CP', 'Putative Family With Sequence Similarity 205, Member C Protein', 'Family With Sequence Similarity 205, Member C', 'Transmembrane Protein C9orf144B Pseudogene', 'Protein FAM205CP', 'FAM205C'], ['Family With Sequence Similarity 209 Member A', 'Protein FAM209A', 'DJ1153D9.3', 'C20orf106', 'Family With Sequence Similarity 209, Member A', 'Chromosome 20 Open Reading Frame 106', 'Uncharacterized Protein C20orf106', 'FAM209A'], ['Family With Sequence Similarity 209 Member B', 'Protein FAM209B', 'DJ1153D9.4', 'C20orf107', 'Family With Sequence Similarity 209, Member B', 'Chromosome 20 Open Reading Frame 107', 'Uncharacterized Protein C20orf107', 'FAM209B'], ['Family With Sequence Similarity 210 Member A', 'Protein FAM210A', 'C18orf19', 'HsT2329', 'Family With Sequence Similarity 210, Member A', 'Chromosome 18 Open Reading Frame 19', 'Uncharacterized Protein C18orf19', 'MGC24180', 'FAM210A'], ['Family With Sequence Similarity 210 Member B', 'Protein FAM210B, Mitochondrial', 'DJ1167H4.1', 'C20orf108', 'Family With Sequence Similarity 210, Member B', 'Chromosome 20 Open Reading Frame 108', 'Transmembrane Protein C20orf108', 'Hypothetical Protein LOC116151', 'DKFZP434A1114', 'FAM210B', '5A3'], ['Family With Sequence Similarity 214 Member A', 'KIAA1370', 'Protein FAM214A', 'Family With Sequence Similarity 214, Member A', 'FLJ10980', 'FAM214A'], ['Family With Sequence Similarity 214 Member B', 'KIAA1539', 'Protein FAM214B', 'Family With Sequence Similarity 214, Member B', 'BA182N22.6', 'FLJ11560', 'FAM214B'], ['Family With Sequence Similarity 216 Member A', 'Protein FAM216A', 'C12orf24', 'HSU79274', 'Family With Sequence Similarity 216, Member A', 'Chromosome 12 Open Reading Frame 24', 'Protein Predicted By Clone 23733', 'Uncharacterized Protein C12orf24', 'FAM216A'], ['Family With Sequence Similarity 216 Member B', 'Protein FAM216B', 'C13orf30', 'Family With Sequence Similarity 216, Member B', 'Chromosome 13 Open Reading Frame 30', 'FLJ40919', 'FAM216B'], ['Family With Sequence Similarity 217 Member A', 'Protein FAM217A', 'C6orf146', 'Family With Sequence Similarity 217, Member A', 'Chromosome 6 Open Reading Frame 146', 'Uncharacterized Protein C6orf146', 'MGC43581', 'FAM217A'], ['Family With Sequence Similarity 217 Member B', 'Protein FAM217B', 'C20orf177', 'DJ551D2.5', 'Family With Sequence Similarity 217, Member B', 'Chromosome 20 Open Reading Frame 177', 'Uncharacterized Protein C20orf177', 'FAM217B'], ['Family With Sequence Similarity 219 Member A', 'Protein FAM219A', 'C9orf25', 'Family With Sequence Similarity 219, Member A', 'Chromosome 9 Open Reading Frame 25', 'Uncharacterized Protein C9orf25', 'BA573M23.5', 'FLJ39031', 'FAM219A'], ['Family With Sequence Similarity 219 Member B', 'Protein FAM219B', 'C15orf17', 'Family With Sequence Similarity 219, Member B', 'Chromosome 15 Open Reading Frame 17', 'Uncharacterized Protein C15orf17', 'FLJ00005', 'FAM219B'], ['Family With Sequence Similarity 220 Member A', 'STAT3-Interacting Protein As A Repressor', 'SIPAR', 'Acrosomal Protein 1', 'Protein FAM220A', 'C7orf70', 'ACPIN1', 'Family With Sequence Similarity 220, Member A', 'Chromosome 7 Open Reading Frame 70', 'MGC12966', 'FAM220A'], ['Family With Sequence Similarity 221 Member A', 'Protein FAM221A', 'C7orf46', 'Family With Sequence Similarity 221, Member A', 'Chromosome 7 Open Reading Frame 46', 'Uncharacterized Protein C7orf46', 'DKFZp686F0810', 'FLJ45875', 'MGC72075', 'FAM221A'], ['Family With Sequence Similarity 221 Member B', 'Protein FAM221B', 'C9orf128', 'Family With Sequence Similarity 221, Member B', 'Chromosome 9 Open Reading Frame 128', 'Uncharacterized Protein C9orf128', 'FAM221B'], ['Family With Sequence Similarity 222 Member A', 'Protein FAM222A', 'Aggregatin', 'C12orf34', 'Family With Sequence Similarity 222, Member A', 'Chromosome 12 Open Reading Frame 34', 'FLJ14721', 'FAM222A'], ['Family With Sequence Similarity 222 Member B', 'Protein FAM222B', 'C17orf63', 'Family With Sequence Similarity 222, Member B', 'Chromosome 17 Open Reading Frame 63', 'Uncharacterized Protein C17orf63', 'FLJ10700', 'FAM222B'], ['Family With Sequence Similarity 227 Member A', 'Protein FAM227A', 'Family With Sequence Similarity 227, Member A', 'FAM227A'], ['Family With Sequence Similarity 227 Member B', 'Protein FAM227B', 'C15orf33', 'Family With Sequence Similarity 227, Member B', 'Chromosome 15 Open Reading Frame 33', 'FLJ32800', 'FAM227B'], ['Family With Sequence Similarity 228 Member A', 'Protein FAM228A', 'C2orf84', 'Family With Sequence Similarity 228, Member A', 'Chromosome 2 Open Reading Frame 84', 'UPF0638 Protein C2orf84', 'FLJ30851', 'FAM228A'], ['Family With Sequence Similarity 228 Member B', 'Protein FAM228B', 'Family With Sequence Similarity 228, Member B', 'UPF0638 Protein B', 'FAM228B'], ['Family With Sequence Similarity 229 Member A', 'Protein FAM229A', 'Family With Sequence Similarity 229, Member A', 'FAM229A'], ['Family With Sequence Similarity 229 Member B', 'Protein FAM229B', 'C6orf225', 'Family With Sequence Similarity 229, Member B', 'Chromosome 6 Open Reading Frame 225', 'UPF0731 Protein C6orf225', 'DKFZp586F0922', 'LOC619208', 'FAM229B'], ['Family With Sequence Similarity 234 Member A', 'Integrin Alpha FG-GAP Repeat Containing 3', 'Protein FAM234A', 'Protein ITFG3', 'C16orf9', 'ITFG3', 'Family With Sequence Similarity 234, Member A', 'Chromosome 16 Open Reading Frame 9', 'DKFZP761D0211', 'FLJ32603', 'FAM234A', 'Gs19'], ['Family With Sequence Similarity 234 Member B', 'KIAA1467', 'Protein FAM234B', 'Family With Sequence Similarity 234, Member B', 'FAM234B'], ['Family With Sequence Similarity 235 Member A', 'DMRTC1 Antisense RNA 1 (Non-Protein Coding)', 'Long Intergenic Non-Protein Coding RNA 684', 'Protein FAM236A', 'DMRTC1 Antisense RNA 1', 'DMRTC1-AS1', 'LINC00684', 'FAM236A'], ['Family With Sequence Similarity 236 Member B', 'Protein FAM236B', 'Family With Sequence Similarity 235 Member B', 'FAM236B'], ['Family With Sequence Similarity 236 Member C', 'Protein FAM236C', 'FAM236D', 'FAM236C'], ['Family With Sequence Similarity 236 Member D', 'Protein FAM236D', 'Family With Sequence Similarity 235 Member D', 'Protein FAM236C', 'FAM236C', 'FAM236D'], ['Family With Sequence Similarity 237 Member A', 'Protein FAM237A', 'FAM237A'], ['Family With Sequence Similarity 237 Member B', 'Protein FAM237B', 'FAM237B'], ['Family With Sequence Similarity 240 Member A', 'Protein FAM240A', 'FAM240A'], ['Family With Sequence Similarity 240 Member B', 'Protein FAM240B', 'FAM240B'], ['Family With Sequence Similarity 240 Member C', 'Protein FAM240C', 'FAM240C'], ['Family With Sequence Similarity 241 Member A', 'Uncharacterized Protein FAM241A', 'C4orf32', 'Chromosome 4 Open Reading Frame 32', 'Uncharacterized Protein C4orf32', 'FLJ39370', 'FAM241A'], ['Family With Sequence Similarity 241 Member B', 'Protein FAM241B', 'C10orf35', 'Chromosome 10 Open Reading Frame 35', 'Uncharacterized Protein C10orf35', 'FAM241B'], ['Family With Sequence Similarity 243 Member A', 'Protein FAM243A', 'C21orf140', 'Chromosome 21 Open Reading Frame 140', 'Uncharacterized Protein C21orf140', 'FAM243A'], ['Family With Sequence Similarity 243 Member B', 'Protein FAM243B', 'Uncharacterized Protein C21orf140', 'FAM243B'], ['Family With Sequence Similarity 246 Member A', 'Protein FAM246A', 'FAM246A'], ['Family With Sequence Similarity 246 Member B', 'Protein FAM246B', 'FAM246B'], ['Family With Sequence Similarity 246 Member C (Gene/Pseudogene)', 'Protein FAM246C', 'FAM246C'], ['FANCD2 And FANCI Associated Nuclease 1', 'FANCD2/FANCI-Associated Nuclease 1', 'KIAA1018', 'Myotubularin-Related Protein 15', 'Fanconi-Associated Nuclease 1', 'MTMR15', 'HFAN1', 'Coiled-Coil Domain-Containing Protein MTMR15', 'Fanconi Anemia Associated Nuclease 1', 'Myotubularin Related Protein 15', 'EC 3.1.21.-', 'EC 3.1.4.1', 'KMIN', 'FAN1'], ['FA Complementation Group A', 'FAA', 'Fanconi Anemia Complementation Group A', 'Fanconi Anemia Group A Protein', 'FANCH', 'FA-H', 'FACA', 'FAH', 'Fanconi Anemia, Complementation Group H', 'Fanconi Anemia, Type 1', 'Protein FACA', 'FANCA', 'FA1', 'FA'], ['FA Complementation Group B', 'FAAP95', 'Fanconi Anemia-Associated Polypeptide Of 95 KDa', 'Fanconi Anemia Complementation Group B', 'Fanconi Anemia Group B Protein', 'FAB', 'Protein FACB', 'EC 3.6.3.14', 'FLJ34064', 'EC 2.8.1', 'FAAP90', 'FANCB', 'FACB', 'FA2'], ['FA Complementation Group C', 'FAC', 'Fanconi Anemia Complementation Group C', 'Fanconi Anemia Group C Protein', 'FACC', 'FA3', 'Protein FACC', 'FANCC'], ['FA Complementation Group D2', 'Fanconi Anemia Complementation Group D2', 'Fanconi Anemia Group D2 Protein', 'FA-D2', 'FACD', 'FAD', 'Protein FACD2', 'FANCD2', 'FANCD', 'FAD2', 'FA4'], ['FANCD2 Opposite Strand', 'Fanconi Anemia Group D2 Protein Opposite Strand Transcript Protein', 'FANCD2 Opposite Strand Protein', 'C3orf24', 'Chromosome 3 Open Reading Frame 24', 'FANCD2OS', 'MGC40179'], ['FA Complementation Group E', 'Fanconi Anemia Complementation Group E', 'Fanconi Anemia Group E Protein', 'FACE', 'FAE', 'Protein FACE', 'FANCE'], ['FA Complementation Group F', 'Fanconi Anemia Complementation Group F', 'Fanconi Anemia Group F Protein', 'FAF', 'Protein FACF', 'FANCF'], ['FA Complementation Group G', 'DNA Repair Protein XRCC9', 'X-Ray Repair Complementing Defective Repair In Chinese Hamster Cells 9', 'X-Ray Repair, Complementing Defective, In Chinese Hamster, 9', 'Fanconi Anemia Complementation Group G', 'Fanconi Anemia Group G Protein', 'XRCC9', 'FAG', 'Truncated Fanconi Anemia Group G Protein', 'Protein FACG', 'FANCG'], ['FA Complementation Group I', 'KIAA1794', 'Fanconi Anemia Complementation Group I', 'Fanconi Anemia Group I Protein', 'Protein FACI', 'FLJ10719', 'FANCI'], ['FA Complementation Group L', 'FAAP43', 'Fanconi Anemia-Associated Polypeptide Of 43 KDa', 'RING-Type E3 Ubiquitin Transferase FANCL', 'Fanconi Anemia Complementation Group L', 'E3 Ubiquitin-Protein Ligase FANCL', 'Fanconi Anemia Group L Protein', 'PHD Finger Protein 9', 'PHF9', 'EC 2.3.2.27', 'FLJ10335', 'FANCL', 'POG', 'Pog'], ['FA Complementation Group M', 'KIAA1596', 'FAAP250', 'Fanconi Anemia-Associated Polypeptide Of 250 KDa', 'Fanconi Anemia Complementation Group M', 'ATP-Dependent RNA Helicase FANCM', 'Fanconi Anemia Group M Protein', 'Protein Hef Ortholog', 'Protein FACM', 'EC 3.6.4.13', 'EC 3.6.1', 'SPGF28', 'POF15', 'FANCM'], ['Fibronectin Type III And Ankyrin Repeat Domains 1', 'Fibronectin Type 3 And Ankyrin Repeat Domains Protein 1', 'Fibronectin Type 3 And Ankyrin Repeat Domains 1', 'HSD13', 'FANK1'], ['Fibroblast Activation Protein Alpha', 'Seprase', '170 KDa Melanoma Membrane-Bound Gelatinase', 'Gelatine Degradation Protease FAP', 'Integral Membrane Serine Protease', 'Post-Proline Cleaving Enzyme', 'Surface-Expressed Protease', 'Prolyl Endopeptidase FAP', 'Dipeptidyl Peptidase FAP', 'FAPalpha', 'DPPIV', 'SIMP', 'Fibroblast Activation Protein, Alpha', 'Serine Integral Membrane Protease', 'EC 3.4.21.26', 'EC 3.4.14.5', 'EC 3.4.21.-', 'FAPA', 'FAP'], ['Fatty Acyl-CoA Reductase 1', 'Short Chain Dehydrogenase/Reductase Family 10E, Member 1', 'Male Sterility Domain-Containing Protein 2', 'SDR10E1', 'MLSTD2', 'Male Sterility Domain Containing 2', 'EC 1.2.1.n2', 'EC 1.2.1.84', 'FLJ22728', 'PFCRD', 'FAR1'], ['Fatty Acyl-CoA Reductase 2', 'Short Chain Dehydrogenase/Reductase Family 10E, Member 2', 'Male Sterility Domain-Containing Protein 1', 'SDR10E2', 'MLSTD1', 'Male Sterility Domain Containing 1', 'Epididymis Secretory Protein Li 81', 'EC 1.2.1.n2', 'EC 1.2.1.84', 'HEL-S-81', 'FLJ10462', 'FAR2'], ['FERM, ARH/RhoGEF And Pleckstrin Domain Protein 1', 'PLEKHC2', 'CDEP', 'FERM, RhoGEF (ARHGEF) And Pleckstrin Domain Protein 1 (Chondrocyte-Derived)', 'FERM, ARHGEF And Pleckstrin Domain-Containing Protein 1', 'FERM, RhoGEF And Pleckstrin Domain-Containing Protein 1', 'Pleckstrin Homology Domain-Containing Family C Member 2', 'Protein Phosphatase 1, Regulatory Subunit 75', 'PH Domain-Containing Family C Member 2', 'Chondrocyte-Derived Ezrin-Like Protein', 'PPP1R75', 'FARP1-IT1', 'MGC87400', 'FARP1'], ['FERM, ARH/RhoGEF And Pleckstrin Domain Protein 2', 'PLEKHC3', 'FIR', 'FERM, ARHGEF And Pleckstrin Domain-Containing Protein 2', 'FERM, RhoGEF And Pleckstrin Domain-Containing Protein 2', 'Pleckstrin Homology Domain-Containing Family C Member 3', 'FERM, RhoGEF And Pleckstrin Domain Protein 2', 'PH Domain-Containing Family C Member 3', 'KIAA0793', 'FRG', 'FERM Domain Including RhoGEF', 'FERM Domain-Including RhoGEF', 'FGD1-Related Cdc42-GEF', 'FARP2'], ['Phenylalanyl-TRNA Synthetase 2, Mitochondrial', 'Phenylalanine-TRNA Synthetase 1 (Mitochondrial)', 'Phenylalanine TRNA Ligase 2, Mitochondrial', 'Phenylalanine--TRNA Ligase, Mitochondrial', 'EC 6.1.1.20', 'MtPheRS', 'FARS1', 'PheRS', 'DJ520B18.2 (FARS1 (Phenylalanine-TRNA Synthetase))', 'DJ236A3.1 (Phenylalanine-TRNA Synthetase)', 'Phenylalanyl-TRNA Synthetase', 'Phenylalanine Translase', 'Mitochondrial PHERS', 'DJ236A3.1', 'EC 6.1.1', 'COXPD14', 'HSPC320', 'SPG77', 'FARS2'], ['Phenylalanyl-TRNA Synthetase Subunit Alpha', 'CML33', 'Phenylalanyl-TRNA Synthetase-Like, Alpha Subunit', 'Phenylalanine TRNA Ligase 1, Alpha, Cytoplasmic', 'Phenylalanyl-TRNA Synthetase Alpha Subunit', 'Phenylalanine--TRNA Ligase Alpha Subunit', 'EC 6.1.1.20', 'FARSLA', 'FARSL', 'PheRS', 'Phenylalanine-TRNA Synthetase-Like, Alpha Subunit', 'Phenylalanine-TRNA Synthetase Alpha-Subunit', 'Phenylalanyl-TRNA Synthetase Alpha Chain', 'Phenylalanine--TRNA Ligase Alpha Chain', 'Phenylalanine-TRNA Synthetase-Like', 'RILDBC2', 'PheHA', 'FARSA', 'FRSA', 'FARS'], ['Phenylalanyl-TRNA Synthetase Subunit Beta', 'FRSB', 'Phenylalanyl-TRNA Synthetase-Like, Beta Subunit', 'Phenylalanine TRNA Ligase 1, Beta, Cytoplasmic', 'Phenylalanyl-TRNA Synthetase Beta Subunit', 'Phenylalanine--TRNA Ligase Beta Subunit', 'EC 6.1.1.20', 'FARSLB', 'PheHB', 'PheRS', 'Phenylalanine-TRNA Synthetase-Like, Beta Subunit', 'Phenylalanyl-TRNA Synthetase Beta Chain', 'Phenylalanine-TRNA Ligase Beta Chain', 'EC 6.1.1', 'HSPC173', 'NEDBLLA', 'RILDBC1', 'RILDBC', 'FARSB'], ['Fas Cell Surface Death Receptor', 'Tumor Necrosis Factor Receptor Superfamily, Member 6', 'Tumor Necrosis Factor Receptor Superfamily Member 6', 'Fas (TNF Receptor Superfamily, Member 6)', 'Apoptosis-Mediating Surface Antigen FAS', 'TNF Receptor Superfamily Member 6', 'FASLG Receptor', 'CD95 Antigen', 'TNFRSF6', 'APO-1', 'APT1', 'CD95', 'FAS1', 'Mutant Tumor Necrosis Receptor Superfamily Member 6', 'Apoptosis Signaling Receptor FAS', 'APO-1 Cell Surface Antigen', 'Apoptosis Antigen 1', 'Apo-1 Antigen', 'Fas AMA', 'ALPS1A', 'FASTM', 'FAS'], ['Fas Ligand', 'Tumor Necrosis Factor Ligand Superfamily Member 6', 'Fas Ligand (TNF Superfamily, Member 6)', 'Apoptosis Antigen Ligand', 'Fas Antigen Ligand', 'CD95 Ligand', 'APT1LG1', 'CD95-L', 'TNFSF6', 'CD178', 'CD95L', 'APTL', 'FASL', 'FasL', 'Tumor Necrosis Factor (Ligand) Superfamily, Member 6', 'Mutant Tumor Necrosis Factor Family Member 6', 'Apoptosis (APO-1) Antigen Ligand 1', 'Tumor Necrosis Factor Ligand 1A', 'CD178 Antigen', 'ALPS1B', 'TNLG1A', 'FASLG'], ['Fatty Acid Synthase', 'FAS', 'Short Chain Dehydrogenase/Reductase Family 27X, Member 1', 'Type I Fatty Acid Synthase', 'SDR27X1', 'EC 2.3.1.85', 'EC 6.3.3.1', 'EC 2.3.1', 'OA-519', 'FASN'], ['Fas Activated Serine/Threonine Kinase', 'Fas-Activated Serine/Threonine Kinase', 'FAST Kinase', 'FAST', 'EC 2.7.11.8', 'FASTK'], ['FAST Kinase Domains 1', 'FAST Kinase Domain-Containing Protein 1, Mitochondrial', 'FLJ21901', 'KIAA1800', 'FASTKD1'], ['FAST Kinase Domains 2', 'KIAA0971', 'FAST Kinase Domain-Containing Protein 2, Mitochondrial', 'COXPD44', 'FASTKD2'], ['FAST Kinase Domains 3', 'FAST Kinase Domain-Containing Protein 3, Mitochondrial', 'FLJ23274', 'FASTKD3', 'MGC5297'], ['FAST Kinase Domains 5', 'FAST Kinase Domain-Containing Protein 5, Mitochondrial', 'DJ1187M17.5', 'FLJ13149', 'KIAA1792', 'FASTKD5'], ['FAT Atypical Cadherin 1', 'CDHF7', 'Cadherin-Related Tumor Suppressor Homolog', 'Cadherin-Related Family Member 8', 'Cadherin Family Member 7', 'Protocadherin Fat 1', 'Protein Fat Homolog', 'CDHR8', 'FAT', 'FAT Tumor Suppressor Homolog 1 (Drosophila)', 'FAT Tumor Suppressor (Drosophila) Homolog', 'FAT Tumor Suppressor 1', 'Cadherin ME5', 'HFat1', 'FAT1', 'ME5'], ['FAT Atypical Cadherin 2', 'CDHF8', 'MEGF1', 'Multiple Epidermal Growth Factor-Like Domains Protein 1', 'Multiple EGF-Like Domains Protein 1', 'Cadherin-Related Family Member 9', 'Cadherin Family Member 8', 'Protocadherin Fat 2', 'CDHR9', 'HFAT2', 'Multiple Epidermal Growth Factor-Like Domains 1', 'FAT Tumor Suppressor (Drosophila) Homolog 2', 'FAT Tumor Suppressor Homolog 2 (Drosophila)', 'FAT Tumor Suppressor Homolog 2', 'Protocadherin FAT2', 'KIAA0811', 'SCA45', 'HFat2', 'FAT2'], ['FAT Atypical Cadherin 3', 'CDHF15', 'Cadherin-Related Family Member 10', 'FAT Tumor Suppressor Homolog 3', 'Cadherin Family Member 15', 'Protocadherin Fat 3', 'KIAA1989', 'CDHR10', 'HFat3', 'FAT Tumor Suppressor Homolog 3 (Drosophila)', 'FAT3'], ['FAT Atypical Cadherin 4', 'CDHF14', 'Cadherin-Related Family Member 11', 'Fat-Like Cadherin Protein FAT-J', 'FAT Tumor Suppressor Homolog 4', 'Cadherin Family Member 14', 'Protocadherin Fat 4', 'CDHR11', 'FAT-J', 'FATJ', 'FAT Tumor Suppressor Homolog 4 (Drosophila)', 'Putative Protein Product Of Nbla00548', 'NBLA00548', 'HKLLS2', 'VMLDS2', 'HFat4', 'FAT4'], ['Fetal And Adult Testis Expressed 1', 'Cancer/Testis Antigen 43', 'CT43', 'FATE', 'Fetal And Adult Testis-Expressed Transcript Protein', 'Tumor Antigen BJ-HCC-2', 'BJ-HCC-2 Antigen', 'FATE1'], ['FAU Ubiquitin Like And Ribosomal Protein S30 Fusion', 'Finkel-Biskis-Reilly Murine Sarcoma Virus (FBR-MuSV) Ubiquitously Expressed (Fox Derived)', 'Monoclonal Nonspecific Suppressor Factor Beta', 'Small Ribosomal Subunit Protein ES30', '40S Ribosomal Protein S30', 'Ribosomal Protein S30', 'MNSFbeta', 'RPS30', 'Fub1', 'Fubi', 'Asr1', 'S30', 'Finkel-Biskis-Reilly Murine Sarcoma Virus (FBR-MuSV) Ubiquitously Expressed', 'Ubiquitin-Like Protein Fubi And Ribosomal Protein S30', 'FAU, Ubiquitin Like And Ribosomal Protein S30 Fusion', 'FBR-MuSV-Associated Ubiquitously Expressed', 'FAU-Encoded Ubiquitin-Like Protein', 'Ubiquitin-Like-S30 Fusion Protein', 'Ubiquitin-Like Protein FUBI', 'FLJ22986', 'FAU1', 'FAU'], ['Failed Axon Connections Homolog, Metaxin Like GST Domain Containing', 'Failed Axon Connections Homolog', 'C6orf168', 'DJ273F20', 'Failed Axon Connections Homolog (Drosophila)', 'Chromosome 6 Open Reading Frame 168', 'MGC2817', 'FAXC'], ['Fatty Acid Hydroxylase Domain Containing 2', 'Fatty Acid Hydroxylase Domain-Containing Protein 2', 'C5orf4', 'Chromosome 5 Open Reading Frame 4', 'FLJ13758', 'FAXDC2'], ['Fas Binding Factor 1', 'FBF-1', 'Fas (TNFRSF6) Binding Factor 1', 'Fas-Binding Factor 1', 'Protein Albatross', 'Albatross', 'KIAA1863', 'ALB', 'Fas Binding Protein 1', 'FLJ00103', 'FBF1', 'Alb'], ['F-Box DNA Helicase 1', 'F-Box Protein, Helicase, 18', 'F-Box Only Protein 18', 'FBXO18', 'Fbx18', 'HFBH1', 'F-Box Only Protein, Helicase, 18', 'EC 3.6.4.12', 'FLJ14590', 'FBX18', 'FBH1'], ['Fibrillarin', 'FLRN', "U6 SnRNA 2'-O-Methyltransferase Fibrillarin", "RRNA 2'-O-Methyltransferase Fibrillarin", '34 KDa Nucleolar Scleroderma Antigen', 'Histone-Glutamine Methyltransferase', 'RNU3IP1', 'Nop1', 'FIB', 'RNA, U3 Small Nucleolar Interacting Protein 1', '34-KD Nucleolar Scleroderma Antigen', 'EC 2.1.1.37', 'EC 2.1.1.-', 'EC 2.1.1', 'FIB1', 'FBL'], ['Filamin Binding LIM Protein 1', 'Migfilin', 'FBLP-1', 'Filamin-Binding LIM Protein 1', 'MIG2-Interacting Protein', 'FBLP1', 'CAL', 'Mitogen-Inducible 2 Interacting Protein', 'Mitogen-Inducible 2-Interacting Protein', 'CSX-Associated LIM', 'FBLIM1'], ['Fibrillarin Like 1', "RRNA/TRNA 2'-O-Methyltransferase Fibrillarin-Like Protein 1", 'Protein-Glutamine Methyltransferase', 'EC 2.1.1.-', 'LOC345630', 'EC 2.1.1', 'FBLL1'], ['Fibulin 1', 'Fibulin-1', 'FBLN', 'FIBL-1', 'FIBL1', 'FBLN1'], ['Fibulin 2', 'Fibulin-2', 'FIBL-2', 'FBLN2'], ['Fibulin 5', 'DANCE', 'UP50', 'Developmental Arteries And Neural Crest EGF-Like Protein', 'Embryonic Vascular EGF-Like Repeat-Containing Protein', 'Urine P50 Protein', 'Fibulin-5', 'FIBL-5', 'ARMD3', 'EVEC', 'Developmental Arteries And Neural Crest EGF-Like', 'Testis Tissue Sperm-Binding Protein Li 75n', 'ARCL1A', 'HNARMD', 'ADCL2', 'FBLN5', 'Dance'], ['Fibulin 7', 'TM14', 'Fibulin-7', 'FIBL-7', 'FLJ37440', 'FBLN7'], ['Fibrillin 1', 'Fibrillin-1', 'Asprosin', 'MASS', 'OCTD', 'FBN', 'SGS', 'Epididymis Secretory Sperm Binding Protein', 'Fibrillin 1 (Marfan Syndrome)', 'Fibrillin-1 Preproprotein', 'Marfan Syndrome', 'Fibrillin 15', 'GPHYSD2', 'ACMICD', 'ECTOL1', 'MFLS', 'MFS1', 'SSKS', 'WMS2', 'FBN1', 'WMS'], ['Fibrillin 2', 'Fibrillin 5', 'Fibrillin-2', 'DA9', 'Congenital Contractural Arachnodactyly', 'EOMD', 'FBN2', 'CCA'], ['Fibrillin 3', 'Fibrillin-3', 'KIAA1776', 'FBN3'], ['Fructose-Bisphosphatase 1', 'Fructose-1,6-Bisphosphatase 1', 'D-Fructose-1,6-Bisphosphate 1-Phosphohydrolase 1', 'Liver FBPase', 'EC 3.1.3.11', 'FBPase 1', 'FBP', 'Growth-Inhibiting Protein 17', 'EC 3.1.3', 'FBP1'], ['Fructose-Bisphosphatase 2', 'D-Fructose-1,6-Bisphosphate 1-Phosphohydrolase 2', 'Fructose-1,6-Bisphosphatase Isozyme 2', 'Fructose-1,6-Bisphosphatase 2', 'Muscle FBPase', 'EC 3.1.3.11', 'FBPase 2', 'Muscle Fructose-Bisphosphatase', 'Hexosediphosphatase', 'EC 3.1.3', 'FBP2'], ['Fibrosin', 'FBS', 'Probable Fibrosin-1', 'Fibrosin 1', 'FBS1', 'Probable Fibrosin-1 Long Transcript Protein', 'Fibrogenic Lymphokine', 'FLJ11618', 'FBRS'], ['Fibrosin Like 1', 'HBV X-Transactivated Gene 9 Protein', 'HBV XAg-Transactivated Protein 9', 'Fibrosin-1-Like Protein', 'AUTS2-Like Protein', 'KIAA1545', 'Fibrosin-Like 1', 'FBRSL1', 'AUTS2L', 'XTP9'], ['F-Box And Leucine Rich Repeat Protein 2', 'FBL2', 'FBL3', 'F-Box/LRR-Repeat Protein 2', 'F-Box Protein FBL2/FBL3', 'F-Box Protein Containing Leucine-Rich Repeats', 'F-Box And Leucine-Rich Repeat Protein 2', 'FBXL2'], ['F-Box And Leucine Rich Repeat Protein 3', 'F-Box And Leucine-Rich Repeat Protein 3A', 'FBL3A', 'F-Box/LRR-Repeat Protein 3A', 'F-Box/LRR-Repeat Protein 3', 'FBXL3A', 'FBL3', 'F-Box Protein Fbl3a', 'IDDSFAS', 'FBXL3'], ['F-Box And Leucine Rich Repeat Protein 4', 'FBL4', 'FBL5', 'F-Box/LRR-Repeat Protein 4', 'F-Box And Leucine-Rich Repeat Protein 4', 'F-Box Protein FBL4/FBL5', 'MTDPS13', 'FBXL4'], ['F-Box And Leucine Rich Repeat Protein 5', 'FBL4', 'FBL5', 'FLR1', 'F-Box/LRR-Repeat Protein 5', 'F-Box Protein FBL4/FBL5', 'P45SKP2-Like Protein', 'F-Box And Leucine-Rich Repeat Protein 5', 'FBXL5'], ['F-Box And Leucine Rich Repeat Protein 6', 'FBL6', 'F-Box/LRR-Repeat Protein 6', 'FBL6A', 'F-Box And Leucine-Rich Repeat Protein 6', 'F-Box Protein Fbl6', 'F-Box Protein FBL6', 'PP14630', 'FBXL6'], ['F-Box And Leucine Rich Repeat Protein 7', 'FBL6', 'FBL7', 'F-Box/LRR-Repeat Protein 7', 'KIAA0840', 'F-Box And Leucine-Rich Repeat Protein 7', 'F-Box Protein FBL6/FBL7', 'F-Box Protein Fbl7', 'FBXL7'], ['F-Box And Leucine Rich Repeat Protein 8', 'F-Box/LRR-Repeat Protein 8', 'F-Box Protein FBL8', 'FBL8', 'F-Box And Leucine-Rich Repeat Protein 8', 'FBXL8', 'Fbl8'], ['F-Box And Leucine Rich Repeat Protein 12', 'F-Box/LRR-Repeat Protein 12', 'F-Box Protein FBL12', 'Fbl12', 'F-Box And Leucine-Rich Repeat Protein 12', 'FLJ20188', 'FBXL12', 'FBL12'], ['F-Box And Leucine Rich Repeat Protein 13', 'DRC6', 'Cilia And Flagella Associated Protein 169', 'Dynein Regulatory Complex Subunit 6', 'F-Box/LRR-Repeat Protein 13', 'CFAP169', 'Fbl13', 'F-Box And Leucine-Rich Repeat Protein 13', 'MGC21636', 'FBXL13', 'FBL13'], ['F-Box And Leucine Rich Repeat Protein 14', 'F-Box/LRR-Repeat Protein 14', 'Fbl14', 'F-Box And Leucine-Rich Repeat Protein 14', 'MGC40195', 'FBXL14', 'FBL14'], ['F-Box And Leucine Rich Repeat Protein 15', 'F-Box Only Protein 37', 'F-Box/LRR-Repeat Protein 15', 'FBXO37', 'Fbl15', 'Pleckstrin And Sec7 Domain Protein', 'MGC11279', 'FBXL15', 'JET', 'PSD'], ['F-Box And Leucine Rich Repeat Protein 16', 'F-Box/LRR-Repeat Protein 16', 'C16orf22', 'Fbl16', 'F-Box And Leucine-Rich Repeat Protein 16', 'Chromosome 16 Open Reading Frame 22', 'C380A1.1', 'MGC33974', 'FBXL16', 'FBL16'], ['F-Box And Leucine Rich Repeat Protein 17', 'F-Box Only Protein 13', 'F-Box/LRR-Repeat Protein 17', 'FBXO13', 'Fbl17', 'Fbx13', 'F-Box And Leucine-Rich Repeat Protein 17', 'DKFZP434C1715', 'FBXL17', 'FBL17', 'FBX13'], ['F-Box And Leucine Rich Repeat Protein 18', 'F-Box/LRR-Repeat Protein 18', 'Fbl18', 'F-Box And Leucine-Rich Repeat Protein 18', 'FLJ11467', 'FBXL18', 'FBL18'], ['F-Box And Leucine Rich Repeat Protein 19', 'Jumonji C Domain-Containing Histone Demethylase 1C', 'F-Box/LRR-Repeat Protein 19', 'CXXC11', 'JHDM1C', 'Fbl19', 'F-Box And Leucine-Rich Repeat Protein 19', 'DKFZp434K0410', 'FBXL19', 'FBL19'], ['F-Box And Leucine Rich Repeat Protein 20', 'F-Box/LRR-Repeat Protein 20', 'Fbl20', 'Fbl2', 'F-Box And Leucine-Rich Repeat Protein 20', 'F-Box/LRR-Repeat Protein 2-Like', 'F-Box Protein FBL2', 'MGC15482', 'FBXL20', 'FBL2'], ['F-Box And Leucine Rich Repeat Protein 22', 'F-Box And Leucine-Rich Protein 22', 'Fbl22', 'F-Box/LRR-Repeat Protein 22', 'FLJ39626', 'FBXL22'], ['F-Box Protein 2', 'F-Box Only Protein 2', 'FBX2', 'Organ Of Corti Protein 1', 'Fbs1', 'F-Box Gene 1', 'NFB42', 'FBXO2', 'Nfb42', 'FBG1', 'OCP1', 'Fbg1'], ['F-Box Protein 3', 'F-Box Only Protein 3', 'FBX3', 'FBA', 'F-Box Protein FBX3', 'FBXO3'], ['F-Box Protein 4', 'F-Box Only Protein 4', 'FBX4', 'F-Box Protein Fbx4', 'FBXO4'], ['F-Box Protein 5', 'F-Box Only Protein 5', 'EMI1', 'FBX5', 'Early Mitotic Inhibitor 1', 'Fbxo31', 'F-Box Protein Fbx5', 'FBXO5'], ['F-Box Protein 6', 'F-Box Only Protein 6', 'FBG2', 'FBS2', 'FBX6', 'F-Box Protein That Recognizes Sugar Chains 2', 'F-Box/G-Domain Protein 2', 'Fbx6b', 'F-Box Protein FBG2', 'F-Box Protein Fbx6', 'FBXO6'], ['F-Box Protein 7', 'F-Box Only Protein 7', 'FBX7', 'PARK15', 'FBX07', 'FBXO7', 'PKPS', 'FBX', 'Fbx'], ['F-Box Protein 8', 'F-Box Only Protein 8', 'FBX8', 'FBS', 'F-Box/SEC7 Protein FBS', 'F-Box Protein Fbx8', 'FBXO8', 'DC10'], ['F-Box Protein 9', 'F-Box Only Protein 9', 'FBX9', 'Renal Carcinoma Antigen NY-REN-57', 'Cross-Immune Reaction Antigen 1', 'NY-REN-57', 'VCIA1', 'F-Box Protein Fbx9', 'DJ341E18.2', 'FBXO9'], ['F-Box Protein 10', 'F-Box Only Protein 10', 'FBX10', 'PRMT11', 'F-Box Protein Fbx10', 'FBXO10'], ['F-Box Protein 11', 'F-Box Only Protein 11', 'FBX11', 'Ubiquitin Protein Ligase E3 Component N-Recognin 6', 'Protein Arginine N-Methyltransferase 9', 'Vitiligo-Associated Protein 1', 'PRMT9', 'UBR6', 'VIT1', 'Vitiligo-Associated Protein VIT-1', 'UG063H01', 'IDDFBA', 'FBXO11', 'VIT-1'], ['F-Box Protein 15', 'F-Box Only Protein 15', 'FBX15', 'MGC39671', 'FBXO15'], ['F-Box Protein 16', 'F-Box Only Protein 16', 'FBX16', 'FBXO16'], ['F-Box Protein 17', 'F-Box Only Protein 26', 'F-Box Only Protein 17', 'FBG4', 'FBXO26', 'FBX26', 'Fbx17', 'F-Box Protein FBG4', 'FLJ25205', 'FLJ11798', 'MGC9379', 'FBXO17', 'FBX17'], ['F-Box Protein 21', 'F-Box Only Protein 21', 'FBX21', 'KIAA0875', 'FBXO21'], ['F-Box Protein 22', 'F-Box Only Protein 22', 'FBX22', 'FIST Domain Containing 1', 'F-Box Protein FBX22p44', 'FISTC1', 'FBXO22'], ['F-Box Protein 24', 'F-Box Only Protein 24', 'FBX24', 'F-Box Protein Fbx24', 'FBXO24'], ['F-Box Protein 25', 'F-Box Only Protein 25', 'FBX25', 'F-Box Protein Fbx25', 'FBXO25'], ['F-Box Protein 27', 'F-Box Only Protein 27', 'F-Box/G-Domain Protein 5', 'Fbx27', 'FBG5', 'Epididymis Secretory Sperm Binding Protein', 'F-Box Protein FBG5', 'FBXO27', 'FBX27', 'Fbg5'], ['F-Box Protein 28', 'CENP-30', 'Centromere Protein 30', 'F-Box Only Protein 28', 'KIAA0483', 'Fbx28', 'FLJ10766', 'FBXO28'], ['F-Box Protein 30', 'F-Box Only Protein, Helicase, 18', 'F-Box Only Protein 30', 'Fbx30', 'F-Box Domain Protein', 'MGC21674', 'FBXO30', 'FBX30'], ['F-Box Protein 31', 'F-Box Only Protein 31', 'FBX14', 'FBXO14', 'Fbx31', 'SCF Ubiquitin Ligase Specificity Factor', 'Putative Breast Cancer Tumor-Suppressor', 'MGC15419', 'Pp2386', 'FBXO31', 'MRT45', 'FBX31'], ['F-Box Protein 32', 'F-Box Only Protein 32', 'MAFbx', 'Muscle Atrophy F-Box Protein', 'Fbx32', 'Atrogin 1', 'Atrogin-1', 'ATROGIN1', 'FBXO32'], ['F-Box Protein 33', 'F-Box Only Protein 33', 'Fbx33', 'C14_5247', 'BMND12', 'FBXO33', 'FBX33'], ['F-Box Protein 34', 'F-Box Only Protein 34', 'Fbx34', 'Protein CGI-301', 'FLJ20725', 'CGI-301', 'FBXO34', 'FBX34'], ['F-Box Protein 36', 'F-Box Only Protein 36', 'Fbx36', 'FLJ37592', 'FBXO36', 'FBX36'], ['F-Box Protein 38', 'F-Box Only Protein 38', 'Fbx38', 'SP329', 'MOKA', 'Modulator Of KLF7 Activity Homolog', 'FLJ13962', 'FBXO38', 'HMN2D'], ['F-Box Protein 39', 'F-Box Only Protein 39', 'CT144', 'Fbx39', 'MGC35179', 'FBOX39', 'FBXO39', 'FBX39'], ['F-Box Protein 40', 'Muscle Disease-Related Protein', 'F-Box Only Protein 40', 'KIAA1195', 'FBX40', 'FBXO40', 'Fbx40'], ['F-Box Protein 41', 'F-Box Only Protein 41', 'KIAA1940', 'FBX41', 'FBXO41', 'Fbx41'], ['F-Box Protein 42', 'Just One F-Box And Kelch Domain-Containing Protein', 'F-Box Only Protein 42', 'KIAA1332', 'Fbx42', 'JFK', 'FBXO42', 'FBX42'], ['F-Box Protein 43', 'Endogenous Meiotic Inhibitor 2', 'F-Box Only Protein 43', 'EMI2', 'Early Mitotic Inhibitor 2', 'FBXO43', 'FBX43', 'Fbx43', 'ERP1'], ['F-Box Protein 44', 'FBX30', 'FBG3', 'F-Box/G-Domain Protein 3', 'F-Box Only Protein 44', 'F-Box Protein FBX30', 'Fbxo6a', 'FBX6A', 'Fbx44', 'F-Box Gene 3', 'MGC14140', 'FBXO44', 'FBXO6A', 'FBX44'], ['F-Box Protein 45', 'F-Box/SPRY Domain-Containing Protein 1', 'F-Box Only Protein 45', 'HFbxo45', 'Fbx45', 'FBXO45', 'FBX45'], ['F-Box Protein 46', 'F-Box Only Protein 34-Like', 'F-Box Only Protein 46', '20D7-FC4', 'FBXO34L', 'Fbx46', 'FBXO46', 'FBX46'], ['F-Box Protein 47', 'F-Box Only Protein 47', 'FBXO47'], ['F-Box Protein 48', 'F-Box Only Protein 48', 'FBXO48', 'FBX48'], ['F-Box And WD Repeat Domain Containing 2', 'FBW2', 'F-Box And WD-40 Domain-Containing Protein 2', 'F-Box/WD Repeat-Containing Protein 2', 'F-Box And WD-40 Domain Protein 2', 'Fwd2', 'Md6', 'Protein MD6', 'FBXW2', 'FWD2'], ['F-Box And WD Repeat Domain Containing 4', 'Dactylin', 'F-Box And WD-40 Domain-Containing Protein 4', 'F-Box/WD Repeat-Containing Protein 4', 'F-Box And WD-40 Domain Protein 4', 'SHFM3', 'FBW4', 'Split Hand/Foot Malformation (Ectrodactyly) Type 3', 'Epididymis Secretory Sperm Binding Protein', 'F-Box/WD Repeat Protein 4', 'FBWD4', 'SHSF3', 'FBXW4', 'Fbw4', 'DAC'], ['F-Box And WD Repeat Domain Containing 5', 'F-Box And WD-40 Domain-Containing Protein 5', 'F-Box/WD Repeat-Containing Protein 5', 'Fbw5', 'WD Repeat-Containing F-Box Protein FBW5', 'F-Box And WD-40 Domain Protein 5', 'DKFZP434B205', 'MGC20962', 'FBXW5', 'FBW5'], ['F-Box And WD Repeat Domain Containing 7', 'SEL-10', 'FBX30', 'SEL10', 'FBW7', 'F-Box And WD Repeat Domain Containing 7, E3 Ubiquitin Protein Ligase', 'F-Box And WD-40 Domain Protein 7 (Archipelago Homolog, Drosophila)', 'F-Box/WD Repeat-Containing Protein 7', 'F-Box Protein FBX30', 'FBXW6', 'HCdc4', 'CDC4', 'HAgo', 'AGO', 'F-Box And WD-40 Domain-Containing Protein 7', 'Archipelago Homolog (Drosophila)', 'Homolog Of C Elegans Sel-10', 'F-Box Protein SEL-10', 'Archipelago Homolog', 'F-Box Protein FBW7', 'Archipelago', 'FLJ11071', 'FBXO30', 'FBXW7', 'FBW6'], ['F-Box And WD Repeat Domain Containing 8', 'F-Box Only Protein 29', 'FBX29', 'FBW6', 'FBW8', 'F-Box And WD-40 Domain-Containing Protein 8', 'F-Box/WD Repeat-Containing Protein 8', 'F-Box And WD-40 Domain Protein 8', 'FBXO29', 'FBXW6', 'FBXW8'], ['F-Box And WD Repeat Domain Containing 9', 'F-Box And WD-40 Domain-Containing Protein 9', 'F-Box/WD Repeat-Containing Protein 9', 'F-Box And WD-40 Domain Protein 9', 'MEC-15', 'Fbw9', 'F-Box And WD-40 Repeat Containing Protein 9', 'Specificity Factor For SCF Ubiquitin Ligase', 'MGC10870', 'FBXW9', 'FBW9'], ['F-Box And WD Repeat Domain Containing 10', 'F-Box/WD Repeat-Containing Protein 10', 'F-Box And WD-40 Domain Protein 10', 'SM2SH2', 'Fbw10', 'HREP', 'F-Box And WD-40 Domain-Containing Protein 10', 'Chromosome 17 Open Reading Frame 1A', 'Ubiquitin Ligase Specificity Factor', 'Ubiquitin Ligase-Specificity Factor', 'Testicular Tissue Protein Li 69', 'SM25H2', 'FBXW10'], ['F-Box And WD Repeat Domain Containing 11', 'BTRCP2', 'F-Box And WD Repeats Protein Beta-TrCP2', 'F-Box/WD Repeat-Containing Protein 11', 'F-Box/WD Repeat-Containing Protein 1B', 'F-Box And WD-40 Domain Protein 1B', 'F-Box And WD-40 Domain Protein 11', 'Homologous To Slimb Protein', 'KIAA0696', 'FBXW1B', 'BTRC2', 'FBW1B', 'Fbw11', 'Hos', 'Beta-Transducin Repeat-Containing Protein 2', 'F-Box Protein Fbw1b', 'NEDJED', 'FBXW11', 'Fbw1b', 'HOS'], ['F-Box And WD Repeat Domain Containing 12', 'F-Box Only Protein 35', 'F-Box And WD-40 Domain-Containing Protein 12', 'F-Box/WD Repeat-Containing Protein 12', 'F-Box And WD-40 Domain Protein 12', 'FBXO12', 'FBXO35', 'FBW12', 'F-Box- And WD40-Repeat-Containing Protein', 'FBXW12', 'Fbw12'], ['Fc Fragment Of IgA And IgM Receptor', 'High Affinity Immunoglobulin Alpha And Immunoglobulin Mu Fc Receptor', 'Fc Receptor, IgA, IgM, High Affinity', 'Fc Alpha/Mu Receptor', 'FCA/MR', 'FKSG87', 'CD351', 'Receptor For Fc Fragment Of IgA And IgM', 'Immunity Related Factor', 'CD351 Antigen', 'FCAMR'], ['Fc Fragment Of IgA Receptor', 'CD89', 'Fc Fragment Of IgA, Receptor For', 'Immunoglobulin Alpha Fc Receptor', 'Fc Alpha Receptor', 'FCAR Variant 14', 'IgA Fc Receptor', 'CD89 Antigen', 'CTB-61M7.2', 'FcalphaRI', 'FCAR'], ['Fc Fragment Of IgE Receptor Ia', 'Fc Fragment Of IgE, High Affinity I, Receptor For; Alpha Polypeptide', 'High Affinity Immunoglobulin Epsilon Receptor Subunit Alpha', 'IgE Fc Receptor Subunit Alpha', 'Fc Epsilon Receptor Ia', 'Fc-Epsilon RI-Alpha', 'FCE1A', 'FcERI', 'Immunoglobulin E Receptor, High-Affinity, Of Mast Cells, Alpha Polypeptide', 'High Affinity Immunoglobulin Epsilon Receptor Alpha-Subunit', 'Fc IgE Receptor, Alpha Polypeptide', 'Fc Epsilon RI Alpha-Chain', 'FCER1A'], ['Fc Fragment Of IgE Receptor Ig', 'Fc Fragment Of IgE, High Affinity I, Receptor For; Gamma Polypeptide', 'High Affinity Immunoglobulin Epsilon Receptor Subunit Gamma', 'Fc Receptor Gamma-Chain', 'Fc Epsilon Receptor Ig', 'Fc-Epsilon RI-Gamma', 'FceRI Gamma', 'FcRgamma', 'Immunoglobulin E Receptor, High Affinity, Gamma Chain', 'IgE Fc Receptor Subunit Gamma', 'FCER1G', 'FCRG'], ['Fc Fragment Of IgE Receptor II', 'CLEC4J', 'Fc Fragment Of IgE, Low Affinity II, Receptor For (CD23)', 'Low Affinity Immunoglobulin Epsilon Fc Receptor', 'Immunoglobulin E-Binding Factor', 'Lymphocyte IgE Receptor', 'Fc Epsilon Receptor II', 'Fc-Epsilon-RII', 'CD23 Antigen', 'BLAST-2', 'CD23A', 'IGEBF', 'CD23', 'FCE2', 'Fc Fragment Of IgE, Low Affinity II, Receptor For (CD23A)', 'C-Type Lectin Domain Family 4, Member J', 'C-Type Lectin Domain Family 4 Member J', 'Immunoglobulin Epsilon-Chain', 'FCER2'], ['FCF1 RRNA-Processing Protein', 'RRNA-Processing Protein FCF1 Homolog', 'C14orf111', 'CGI-35', 'UTP24', 'Bka', 'FCF1 Small Subunit (SSU) Processome Component Homolog (S. Cerevisiae)', 'Chromosome 14 Open Reading Frame 111', 'FCF1 Small Subunit', 'FCF1'], ['Fc Fragment Of IgG Binding Protein', 'Fcgamma-Binding Protein Antigen', 'IgG Fc Binding Protein', 'IgGFc-Binding Protein', 'Human Fc Gamma BP', 'FC(GAMMA)BP', 'FcgammaBP', 'FCGBP'], ['Fc Fragment Of IgG Receptor Ia', 'Fc Fragment Of IgG, High Affinity Ia, Receptor For (CD64)', 'Fc Fragment Of IgG, High Affinity Ia, Receptor (CD64)', 'High Affinity Immunoglobulin Gamma Fc Receptor I', 'Fc Gamma Receptor Ia', 'IgG Fc Receptor I', 'Fc-Gamma RIA', 'Fc-Gamma RI', 'FcgammaRIa', 'CD64A', 'IGFR1', 'CD64', 'Fc-Gamma Receptor I A1', 'CD64 Antigen', 'FCGR1A', 'FCGR1', 'FCRI', 'FCG1', 'FcRI'], ['Fc Fragment Of IgG Receptor Ib', 'Fc Fragment Of IgG, High Affinity Ib, Receptor (CD64)', 'High Affinity Immunoglobulin Gamma Fc Receptor IB', 'Fc Gamma Receptor Ib', 'IgG Fc Receptor IB', 'Fc-Gamma RIB', 'HFcgammaRIB', 'CD64b', 'IGFRB', 'FcRIB', 'Fc Fragment Of IgG, High Affinity Ib, Receptor For (CD64)', 'High Affinity Immunoglobulin Gamma Fc Receptor I', 'Fc-Gamma Receptor I B2', 'IgG Fc Receptor I', 'Fc-Gamma RIA', 'Fc-Gamma RI', 'FcgammaRIa', 'FCGR1B', 'FCGR1', 'IGFR1', 'CD34', 'FCG1', 'FcRI'], ['Fc Fragment Of IgG Receptor IIa', 'CDw32', 'IGFR2', 'CD32', 'Low Affinity Immunoglobulin Gamma Fc Region Receptor II-A', 'Fc Fragment Of IgG, Low Affinity IIa, Receptor (CD32)', 'Immunoglobulin G Fc Receptor II', 'IgG Fc Receptor II-A', 'Fc-Gamma-RIIa', 'FCGR2A1', 'FcRII-A', 'CD32A', 'FCG2', 'Fc Fragment Of IgG, Low Affinity IIa, Receptor For (CD32)', 'Fc Gamma Receptor RIIa3', 'Fc Gamma Receptor IIa', 'Fc-Gamma RII-A', 'CD32 Antigen', 'FCGR2A', 'FCGR2', 'FcGR'], ['Fc Fragment Of IgG Receptor IIb', 'CD32', 'Fc Fragment Of IgG, Low Affinity IIb, Receptor For (CD32)', 'Low Affinity Immunoglobulin Gamma Fc Region Receptor II-B', 'Fc Fragment Of IgG, Low Affinity IIb, Receptor (CD32)', 'Fc Gamma Receptor IIb', 'IgG Fc Receptor II-B', 'Fc-Gamma RII-B', 'Fc-Gamma-RIIb', 'FcRII-B', 'CD32B', 'IGFR2', 'CDw32', 'FCG2', 'Low Affinity Immunoglobulin Gamma Fc Region Receptor II-C', 'Fc Fragment Of IgG, Low Affinity II, Receptor For (CD32)', 'IgG Fc Receptor II-C', 'Fc-Gamma RII-C', 'Fc Gamma RIIb', 'Fc-Gamma-RIIc', 'CD32 Antigen', 'FcRII-C', 'FCGR2C', 'FCGR2B', 'FCGR2'], ['Fc Fragment Of IgG Receptor IIc (Gene/Pseudogene)', 'Fc Fragment Of IgG, Low Affinity IIc, Receptor For (CD32)', 'Low Affinity Immunoglobulin Gamma Fc Region Receptor II-C', 'IgG Fc Receptor II-C', 'Fc-Gamma-RIIc', 'CD32C', 'IGFR2', 'CD32', 'FCG2', 'Fc Fragment Of IgG, Low Affinity IIc, Receptor For (CD32) (Gene/Pseudogene)', 'Immunoglobulin G Fc Receptor II-C', 'Fc Gamma Receptor IIc', 'Fc Gamma Receptor IIC', 'Fc-Gamma RII-C', 'CD32 Antigen', 'HFcRII-C', 'FcRII-C', 'FCRIIC', 'FCGR2C', 'CDW32', 'CDw32'], ['Fc Fragment Of IgG Receptor IIIa', 'Low Affinity Immunoglobulin Gamma Fc Region Receptor III-A', 'Fc Fragment Of IgG, Low Affinity IIIa, Receptor (CD16a)', 'IgG Fc Receptor III-2', 'Fc-Gamma RIII-Alpha', 'CD16a Antigen', 'CD16A', 'FCGR3', 'IGFR3', 'CD16', 'FCG3', 'Low Affinity Immunoglobulin Gamma Receptor III-A Fc Fragment', 'Fc Fragment Of IgG, Low Affinity IIIa, Receptor For (CD16)', 'Fc Fragment Of IgG, Low Affinity III, Receptor For (CD16)', 'Immunoglobulin G Fc Receptor III', 'Fc-Gamma Receptor III-2 (CD 16)', 'Neutrophil-Specific Antigen NA', 'Fc-Gamma Receptor IIIb (CD16)', 'Fc Gamma Receptor III-A', 'Fc Gamma Receptor IIIa', 'Fc-Gamma RIIIa', 'Fc-Gamma RIII', 'FcgammaRIIIA', 'FCGRIII', 'FCRIIIA', 'FcRIIIa', 'FCR-10', 'FCRIII', 'FCGR3A', 'FcRIII', 'FcR-10', 'IMD20', 'CD16a'], ['Fc Fragment Of IgG Receptor IIIb', 'Low Affinity Immunoglobulin Gamma Fc Region Receptor III-B', 'Fc Fragment Of IgG, Low Affinity IIIb, Receptor (CD16b)', 'Fc Gamma Receptor IIIb', 'IgG Fc Receptor III-1', 'Fc-Gamma RIII-Beta', 'Fc-Gamma RIIIb', 'CD16b', 'FCGR3', 'CD16', 'FCG3', 'Fc Fragment Of IgG, Low Affinity IIIb, Receptor For (CD16)', 'Fc-Gamma Receptor IIIb (CD 16)', 'Fc Gamma Receptor III-B', 'Fc-Gamma RIII', 'CD16b Antigen', 'FCRIIIb', 'FcRIIIb', 'FCGR3A', 'FCR-10', 'FCRIII', 'FCGR3B', 'FcRIII', 'FcR-10', 'CD16A', 'CD16B', 'IGFR3'], ['Fc Fragment Of IgG Receptor And Transporter', 'FCRN', 'Heavy Chain Of The Major Histocompatibility Complex Class I-Like Fc Receptor', 'Transmembrane Alpha Chain Of The Neonatal Receptor', 'Fc Fragment Of IgG, Receptor, Transporter, Alpha', 'IgG Fc Fragment Receptor Transporter Alpha Chain', 'IgG Receptor FcRn Large Subunit P51', 'Neonatal Fc Receptor', 'Alpha-Chain', 'Major Histocompatibility Complex Class I-Like Fc Receptor', 'Immunoglobulin Receptor, Intestinal, Heavy Chain', 'Neonatal Fc-Receptor For Ig', 'FcRn Alpha Chain', 'FCGRT', 'FcRn'], ['FCH And Mu Domain Containing Endocytic Adaptor 1', 'F-BAR Domain Only Protein 1', 'FCH Domain Only 1', 'KIAA0290', 'FCH Domain Only Protein 1', 'IMD76', 'FCHO1'], ['FCH And Mu Domain Containing Endocytic Adaptor 2', 'F-BAR Domain Only Protein 2', 'FCH Domain Only 2', 'FCH Domain Only Protein 2', 'FCHO2'], ['FCH And Double SH3 Domains 1', 'F-BAR And Double SH3 Domains Protein 1', 'Protein Nervous Wreck 2', 'NWK2', 'FCH And Double SH3 Domains Protein 1', 'Nervous Wreck Homolog 2', 'FLJ00007', 'FCHSD1'], ['FCH And Double SH3 Domains 2', 'F-BAR And Double SH3 Domains Protein 2', 'SH3 Multiple Domains Protein 3', 'Protein Nervous Wreck 1', 'SH3 Multiple Domains 3', 'KIAA0769', 'SH3MD3', 'Carom', 'NWK1', 'FCH And Double SH3 Domains Protein 2', 'Nervous Wreck Homolog', 'FCHSD2', 'NWK'], ['Fc Fragment Of IgM Receptor', 'Fas Apoptotic Inhibitory Molecule 3', 'TOSO', 'Regulator Of Fas-Induced Apoptosis Toso', 'IgM Fc Fragment Receptor', 'FAIM3', 'Immunoglobulin Mu Fc Receptor', 'IgM Fc Receptor', 'Fc Mu Receptor', 'FCMR'], ['Ficolin 1', 'FCNM', 'Ficolin (Collagen/Fibrinogen Domain-Containing) 1', 'Ficolin-Alpha', 'Ficolin-1', 'M-Ficolin', 'Ficolin-A', 'Ficolin (Collagen/Fibrinogen Domain Containing) 1', 'Collagen/Fibrinogen Domain-Containing Protein 1', 'FCN1'], ['Ficolin 2', 'Collagen/Fibrinogen Domain-Containing Protein 2', 'Serum Lectin P35', 'L-Ficolin', 'Ficolin-2', 'EBP-37', 'FCNL', '37 KDa Elastin-Binding Protein', 'Ficolin-Beta', 'Ficolin B', 'Hucolin', 'P35', 'Ficolin (Collagen/Fibrinogen Domain-Containing Lectin) 2 (Hucolin)', 'Ficolin (Collagen/Fibrinogen Domain Containing Lectin) 2 (Hucolin)', 'Ficolin (Collagen/Fibrinogen Domain Containing Lectin) 2', 'Ficolin-B', 'FCN2'], ['Ficolin 3', 'HAKA1', 'FCNH', 'Collagen/Fibrinogen Domain-Containing Lectin 3 P35', 'Collagen/Fibrinogen Domain-Containing Protein 3', 'Hakata Antigen', 'Ficolin-3', 'Ficolin (Collagen/Fibrinogen Domain-Containing) 3 (Hakata Antigen)', 'Ficolin (Collagen/Fibrinogen Domain Containing) 3 (Hakata Antigen)', 'Ficolin (Collagen/Fibrinogen Domain Containing) 3', 'H-Ficolin', 'FCN3'], ['Fc Receptor Like 1', 'FCRH1', 'IFGP1', 'IRTA5', 'Immune Receptor Translocation-Associated Protein 5', 'Fc Receptor-Like Protein 1', 'IFGP Family Protein 1', 'Fc Receptor Homolog 1', 'FcR-Like Protein 1', 'CD307a', 'HIFGP1', 'Immunoglobulin Superfamily Fc Receptor, Gp42', 'Fc Receptor-Like 1', 'CD307a Antigen', 'FCRL1', 'FcRL1', 'FcRH1'], ['Fc Receptor Like 2', 'FCRH2', 'IRTA4', 'Immunoglobulin Receptor Translocation-Associated Protein 4', 'SH2 Domain-Containing Phosphatase Anchor Protein 1', 'Fc Receptor-Like Protein 2', 'IFGP Family Protein 4', 'Fc Receptor Homolog 2', 'CD307b', 'IFGP4', 'SPAP1', 'SH2 Domain Containing Phosphatase Anchor Protein 1', 'Immune Receptor Translocation-Associated Protein 4', 'Immunoglobulin Superfamily Fc Receptor, Gp42', 'Fc Receptor-Like 2', 'FcR-Like Protein 2', 'CD307b Antigen', 'SPAP1A', 'SPAP1B', 'SPAP1C', 'FCRL2', 'FcRL2', 'FcRH2'], ['Fc Receptor Like 3', 'FCRH3', 'IFGP3', 'IRTA3', 'SPAP2', 'SH2 Domain-Containing Phosphatase Anchor Protein 2', 'Immune Receptor Translocation-Associated Protein 3', 'Fc Receptor-Like Protein 3', 'Fc Receptor Homolog 3', 'IFGP Family Protein 3', 'FcR-Like Protein 3', 'CD307c', 'HIFGP3', 'Immunoglobulin Superfamily Receptor Translocation Associated Protein 3', 'Fc Receptor-Like 3', 'CD307c Antigen', 'SPAP2a', 'SPAP2b', 'SPAP2c', 'SPAP2d', 'SPAP2e', 'FCRL3', 'FcRL3', 'FcRH3'], ['Fc Receptor Like 4', 'FCRH4', 'IRTA1', 'Immune Receptor Translocation-Associated Protein 1', 'Fc Receptor-Like Protein 4', 'IFGP Family Protein 2', 'Fc Receptor Homolog 4', 'FcR-Like Protein 4', 'CD307d', 'HIFGP2', 'IGFP2', 'Immunoglobulin Superfamily Receptor Translocation Associated 1', 'Immunoglobulin Superfamily Fc Receptor, Gp42', 'Fc Receptor-Like 4', 'CD307d Antigen', 'FCRL4', 'IFGP2', 'FcRL4', 'FcRH4'], ['Fc Receptor Like 5', 'BXMAS1', 'FCRH5', 'IRTA2', 'Immune Receptor Translocation-Associated Protein 2', 'Fc Receptor-Like Protein 5', 'Fc Receptor Homolog 5', 'FcR-Like Protein 5', 'CD307e', 'Immunoglobulin Superfamily Receptor Translocation Associated 2 (IRTA2)', 'Fc Receptor-Like 5', 'CD307e Antigen', 'PRO820', 'CD307', 'FCRL5', 'FcRL5', 'FcRH5'], ['Fc Receptor Like 6', 'FcRH6', 'IFGP6', 'Fc Receptor-Like Protein 6', 'Fc Receptor Homolog 6', 'FcR-Like Protein 6', 'IgSF Type I Transmembrane Receptor', 'Fc Receptor-Like Protein 7', 'Fc Receptor-Like 6', 'Leukocyte Receptor', 'FLJ16056', 'FCRL6', 'FCRH6', 'FcRL6'], ['Fc Receptor Like A', 'Fc Receptor-Like A', 'FCRL', 'FREB', 'Fc Receptor-Like And Mucin-Like Protein 1', 'Fc Receptor Homolog Expressed In B-Cells', 'Fc Receptor-Like And Mucin-Like 1', 'Fc Receptor-Related Protein X', 'Fc Receptor-Like Protein', 'FCRLM1', 'FCRLc1', 'FCRLc2', 'FCRL1', 'FCRLX', 'FCRLb', 'FCRLd', 'FCRLe', 'FCRX', 'Fc Receptor Homolog Expressed In B Cells (FREB)', 'Fc Receptor Related Protein X', 'MGC4595', 'FCRLA', 'FcRX'], ['Fc Receptor Like B', 'Fc Receptor-Like B', 'FREB-2', 'FCRL2', 'Fc Receptor-Like And Mucin-Like Protein 2', 'Fc Receptor-Related Protein Y', 'Fc Receptor-Like Protein 2', 'FCRLM2', 'FCRLY', 'FREB2', 'FcRY', 'Fc Receptor Homolog Expressed In B Cells Protein 2', 'Fc Receptor Homolog Expressed In B-Cells Protein 2', 'Fc Receptor-Like And Mucin-Like 2', 'FLJ31052', 'FCRLB', 'FCRY'], ['Fucose Kinase', 'L-Fucose Kinase', 'Fucokinase', 'FUK', '1110046B12Rik', 'EC 2.7.1.52', 'FLJ39408', 'CDGF2', 'FCSK'], ['Follicular Dendritic Cell Secreted Protein', 'FDC-SP', 'Follicular Dendritic Cell Secreted Peptide', 'FDC Secreted Protein', 'C4orf7', 'Chromosome 4 Open Reading Frame 7', 'FDCSP'], ['Farnesyl-Diphosphate Farnesyltransferase 1', 'Squalene Synthase', 'SQS', 'FPP:FPP Farnesyltransferase', 'EC 2.5.1.21', 'SS', 'Farnesyl-Diphosphate Farnesyltransferase', 'Presqualene-Di-Diphosphate Synthase', 'Squalene Synthetase', 'FDFT1', 'DGPT', 'ERG9', 'SQSD'], ['Farnesyl Diphosphate Synthase', 'Farnesyl Pyrophosphate Synthetase, Dimethylallyltranstransferase, Geranyltranstransferase', '(2E,6E)-Farnesyl Diphosphate Synthase', 'Farnesyl Pyrophosphate Synthase', 'FPP Synthase', 'FPS', 'Dimethylallyltranstransferase', 'Geranyltranstransferase', 'FPP Synthetase', 'EC 2.5.1.10', 'EC 2.5.1.1', 'KIAA1293', 'POROK9', 'FPPS', 'FDPS'], ['Ferredoxin 1', 'ADX', 'Adrenodoxin, Mitochondrial', 'Adrenal Ferredoxin', 'Hepatoredoxin', 'Mitochondrial Adrenodoxin', 'Ferredoxin-1', 'Adrenodoxin', 'LOH11CR1D', 'FDX1', 'FDX'], ['Ferredoxin 2', 'Ferredoxin-2, Mitochondrial', 'FDX1L', 'Adrenodoxin-Like Protein, Mitochondrial', 'Ferredoxin-1-Like Protein', 'Adrenodoxin-Like Protein', 'Ferredoxin 1-Like', 'Ferredoxin 1 Like', 'MGC19604', 'MEOAL', 'FDX2'], ['Ferredoxin-Fold Anticodon Binding Domain Containing 1', 'Ferredoxin-Fold Anticodon-Binding Domain-Containing Protein 1', 'FDX-ACDB Domain-Containing Protein 1', 'Hypothetical Protein BC006136', 'HCG_2033039', 'LOC91893', 'FDXACB1'], ['Ferredoxin Reductase', 'Adrenodoxin Reductase', 'NADPH:Adrenodoxin Oxidoreductase, Mitochondrial', 'Adrenodoxin-NADP(+) Reductase', 'Ferredoxin--NADP(+) Reductase', 'ADXR', 'ADR', 'AR', 'Epididymis Secretory Sperm Binding Protein', 'EC 1.18.1.2', 'EC 1.18.1.6', 'ANOA', 'FDXR'], ['Ferrochelatase', 'Ferrochelatase, Mitochondrial', 'Protoheme Ferro-Lyase', 'Protoporphyria', 'Heme Synthase', 'EC 4.99.1.1', 'Ferrochelatase (Protoporphyria)', 'Heme Synthetase', 'EPP1', 'FECH', 'EPP', 'FCE'], ['Fem-1 Homolog A', 'Prostaglandin E Receptor 4-Associated Protein', 'Protein Fem-1 Homolog A', 'FEM1-Alpha', 'EPRAP', 'Fem-1 Homolog A (C. Elegans)', 'FEM1A', 'FEM1a'], ['Fem-1 Homolog B', 'Fem-1-Like Death Receptor-Binding Protein Alpha', 'Fem-1-Like In Apoptotic Pathway Protein Alpha', 'Protein Fem-1 Homolog B', 'FEM1-Beta', 'F1AA', 'FEM-1-Like Death Receptor Binding Protein', 'FEM-1 (C. Elegans) Homolog B', 'Fem-1 Homolog B (C. Elegans)', 'F1A-ALPHA', 'F1A-Alpha', 'KIAA0396', 'FEM1B', 'FEM1b'], ['Fem-1 Homolog C', 'Protein Fem-1 Homolog C', 'EUROIMAGE686608', 'EUROIMAGE783647', 'FEM1-Gamma', 'KIAA1785', 'FEM1A', 'Fem-1 Homolog C (C. Elegans)', 'FEM1C', 'FEM1c'], ['Flap Structure-Specific Endonuclease 1', 'DNase IV', 'FEN-1', 'MF1', 'Maturation Factor-1', 'Flap Endonuclease 1', 'RAD2', 'Maturation Factor 1', 'EC 3.1.-.-', 'HFEN-1', 'FEN1'], ['FER Tyrosine Kinase', 'TYK3', 'Fujinami Poultry Sarcoma/Feline Sarcoma-Related Protein Fer', 'Protein Phosphatase 1, Regulatory Subunit 74', 'Feline Encephalitis Virus-Related Kinase FER', 'Fer (Fps/Fes Related) Tyrosine Kinase', 'Tyrosine-Protein Kinase Fer', 'Phosphoprotein NCP94', 'Proto-Oncogene C-Fer', 'Tyrosine Kinase 3', 'EC 2.7.10.2', 'PPP1R74', 'P94-Fer', 'EC 2.7.10', 'FER'], ['Fer-1 Like Family Member 5', 'Fer-1-Like Protein 5', 'Fer-1-Like 5 (C. Elegans)', 'DKFZp434I0121', 'Fer-1-Like 5', 'FER1L5'], ['Fer-1 Like Family Member 6', 'Fer-1-Like Protein 6', 'C8ORFK23', 'Fer-1-Like 6 (C. Elegans)', 'Fer-1-Like 6', 'C8orfK23', 'FER1L6'], ['Fer3 Like BHLH Transcription Factor', 'BHLHa31', 'NATO3', 'Class A Basic Helix-Loop-Helix Protein 31', 'Basic Helix-Loop-Helix Protein N-Twist', 'Nephew Of Atonal 3', 'Fer3-Like Protein', 'Neuronal Twist', 'N-TWIST', 'NTWIST', 'Pancreas-Specific Transcription Factor B', 'Fer3-Like BHLH Transcription Factor', 'Fer3-Like (Drosophila)', 'BHLHA31', 'FERD3L', 'PTFB'], ['FERM Domain Containing Kindlin 1', 'Fermitin Family Member 1', 'KIND1', 'URP1', 'Fermitin Family Homolog 1', 'Unc-112-Related Protein 1', 'Kindlin Syndrome Protein', 'Kindlin-1', 'Kindlerin', 'C20orf42', 'UNC112A', 'Fermitin Family Homolog 1 (Drosophila)', 'Chromosome 20 Open Reading Frame 42', 'UNC112 Related Protein 1', 'Kinderlin', 'Kindlin 1', 'FLJ20116', 'DTGCU2', 'FERMT1'], ['FERM Domain Containing Kindlin 2', 'Fermitin Family Member 2', 'KIND2', 'Pleckstrin Homology Domain Containing, Family C (With FERM Domain) Member 1', 'PH Domain-Containing Family C Member 1', 'Fermitin Family Homolog 2', 'Mitogen Inducible Gene-2', 'Kindlin-2', 'PLEKHC1', 'UNC112B', 'Mig-2', 'MIG2', 'Pleckstrin Homology Domain Containing, Family C Member 1', 'Pleckstrin Homology Domain-Containing Family C Member 1', 'Fermitin Family Homolog 2 (Drosophila)', 'Mitogen Inducible Gene 2 Protein', 'Mitogen-Inducible Gene 2 Protein', 'Kindlin 2', 'UNC112', 'FERMT2', 'MIG-2'], ['FERM Domain Containing Kindlin 3', 'Fermitin Family Member 3', 'KIND3', 'MIG2B', 'URP2', 'UNC-112 Related Protein 2', 'Fermitin Family Homolog 3', 'Unc-112-Related Protein 2', 'MIG2-Like Protein', 'Kindlin-3', 'UNC112C', 'Fermitin Family Homolog 3 (Drosophila)', 'Kindlin 3', 'MGC10966', 'URP2SF', 'FERMT3', 'MIG-2'], ['FES Proto-Oncogene, Tyrosine Kinase', 'FPS', 'Feline Sarcoma (Snyder-Theilen) Viral (V-Fes)/Fujinami Avian Sarcoma (PRCII) Viral (V-Fps) Oncogene Homolog', 'Feline Sarcoma/Fujinami Avian Sarcoma Oncogene Homolog', 'Oncogene FES, Feline Sarcoma Virus', 'Tyrosine-Protein Kinase Fes/Fps', 'Feline Sarcoma Oncogene', 'Proto-Oncogene C-Fes', 'Proto-Oncogene C-Fps', 'EC 2.7.10.2', 'P93c-Fes', 'Proto-Oncogene Tyrosine-Protein Kinase Fes/Fps', 'C-Fes/Fps Protein', 'EC 2.7.10', 'FES'], ['Fetuin B', 'Fetuin-Like Protein IRL685', 'Fetuin-B', '16G2', 'Gugu', 'IRL685', 'FETUB'], ['FEV Transcription Factor, ETS Family Member', 'PC12 ETS Domain-Containing Transcription Factor 1', 'FEV, ETS Transcription Factor', 'Fifth Ewing Variant Protein', 'FEV (ETS Oncogene Family)', 'Protein FEV', 'Pet-1', 'FEV (Fifth Ewing Variant)', 'PC12 ETS Factor 1', 'HSRNAFEV', 'PET-1', 'PET1', 'FEV'], ['Fasciculation And Elongation Protein Zeta 1', 'Zygin I', 'Fasciculation And Elongation Protein Zeta-1', 'Zygin-1', 'UNC-76', 'FEZ1'], ['Fasciculation And Elongation Protein Zeta 2', 'Zygin II', 'Fasciculation And Elongation Protein Zeta-2', 'Pre-T/NK Cell Associated Protein (3Cl)', 'Zygin 2', 'Zygin-2', 'HUM3CL', 'FEZ2'], ['FEZ Family Zinc Finger 1', 'Zinc Finger Protein 312B', 'Fez Family Zinc Finger Protein 1', 'ZNF312B', 'FEZ', 'FEZF1', 'HH22'], ['FEZ Family Zinc Finger 2', 'Zinc Finger Protein 312', 'FEZL', 'Forebrain Embryonic Zinc Finger-Like Protein 2', 'Fez Family Zinc Finger Protein 2', 'Too Few', 'FKSG36', 'ZNF312', 'TOF', 'Forebrain Embryonic Zinc Finger-Like Protein', 'Testis Tissue Sperm-Binding Protein Li 80P', 'Zinc Finger Protein Fez-Like', 'FLJ10142', 'ZFP312', 'Zfp312', 'FEZF2', 'FEZ'], ['Free Fatty Acid Receptor 1', 'G-Protein Coupled Receptor 40', 'FFA1R', 'GPR40', 'G Protein-Coupled Receptor 40', 'GPCR40', 'FFAR1'], ['Free Fatty Acid Receptor 2', 'G-Protein Coupled Receptor 43', 'FFA2R', 'GPR43', 'Free Fatty Acid Activated Receptor 2', 'G Protein-Coupled Receptor 43', 'Fatty Acid Receptor 2', 'GPCR43', 'FFAR2', 'FFA2'], ['Free Fatty Acid Receptor 3', 'G-Protein Coupled Receptor 41', 'FFA3R', 'GPR41', 'G Protein-Coupled Receptor 41', 'GPR42', 'FFAR3'], ['Free Fatty Acid Receptor 4', 'Omega-3 Fatty Acid Receptor 1', 'PGR4', 'G-Protein Coupled Receptor GT01', 'G-Protein Coupled Receptor PGR4', 'G-Protein Coupled Receptor 120', 'G-Protein Coupled Receptor 129', 'GPR120', 'GPR129', 'O3FAR1', 'G Protein-Coupled Receptor 129', 'G Protein-Coupled Receptor 120', 'BMIQ10', 'FFAR4', 'GT01'], ['Fibrinogen Alpha Chain', 'Fibrinogen, A Alpha Polypeptide', 'Fib2', 'FGA'], ['Fibrinogen Beta Chain', 'Fibrinogen, B Beta Polypeptide', 'Epididymis Secretory Sperm Binding Protein Li 78p', 'Beta-Fibrinogen', 'HEL-S-78p', 'FGB'], ['FYVE, RhoGEF And PH Domain Containing 1', 'ZFYVE3', 'FYVE, RhoGEF And PH Domain-Containing Protein 1', 'Rho/Rac Guanine Nucleotide Exchange Factor FGD1', 'Zinc Finger FYVE Domain-Containing Protein 3', 'Faciogenital Dysplasia 1 Protein', 'Rho/Rac GEF', 'FGDY', 'Faciogenital Dysplasia (Aarskog-Scott Syndrome)', 'MRXS16', 'FGD1', 'AAS'], ['FYVE, RhoGEF And PH Domain Containing 2', 'ZFYVE4', 'FYVE, RhoGEF And PH Domain-Containing Protein 2', 'Zinc Finger FYVE Domain-Containing Protein 4', 'FGD1 Family, Member 2', 'Epididymis Secretory Sperm Binding Protein', 'FLJ00276 Protein', 'FGD2'], ['FYVE, RhoGEF And PH Domain Containing 3', 'ZFYVE5', 'FYVE, RhoGEF And PH Domain-Containing Protein 3', 'Zinc Finger FYVE Domain-Containing Protein 5', 'FGD1 Family, Member 3', 'Faciogenital Dysplasia 3', 'FLJ00004', 'FGD3'], ['FYVE, RhoGEF And PH Domain Containing 4', 'ZFYVE6', 'FRABP', 'FYVE, RhoGEF And PH Domain-Containing Protein 4', 'Zinc Finger FYVE Domain-Containing Protein 6', 'FGD1-Related F-Actin-Binding Protein', 'FGD1 Family, Member 4', 'CMT4H', 'Actin-Filament Binding Protein Frabin', 'Actin Filament-Binding Protein Frabin', 'Frabin', 'FGD4'], ['FYVE, RhoGEF And PH Domain Containing 5', 'ZFYVE23', 'FYVE, RhoGEF And PH Domain-Containing Protein 5', 'Zinc Finger FYVE Domain-Containing Protein 23', 'FLJ39957', 'FLJ00274', 'FGD5'], ['FYVE, RhoGEF And PH Domain Containing 6', 'ZFYVE24', 'FYVE, RhoGEF And PH Domain-Containing Protein 6', 'Zinc Finger FYVE Domain-Containing Protein 24', 'FLJ11183', 'KIAA1362', 'FGD6'], ['Fibroblast Growth Factor 1', 'Heparin-Binding Growth Factor 1', 'AFGF', 'ECGF', 'Endothelial Cell Growth Factor, Alpha', 'Endothelial Cell Growth Factor, Beta', 'Fibroblast Growth Factor 1 (Acidic)', 'ECGF-Beta', 'FGF-Alpha', 'GLIO703', 'HBGF-1', 'ECGFA', 'ECGFB', 'FGF-1', 'HBGF1', 'FGFA', 'Beta-Endothelial Cell Growth Factor', 'Acidic Fibroblast Growth Factor', 'Endothelial Cell Growth Factor', 'FGF1'], ['Fibroblast Growth Factor 2', 'Fibroblast Growth Factor 2 (Basic)', 'Heparin-Binding Growth Factor 2', 'HBGF-2', 'FGF-2', 'BFGF', 'FGFB', 'Basic Fibroblast Growth Factor BFGF', 'Basic Fibroblast Growth Factor', 'Prostatropin', 'FGF2'], ['Fibroblast Growth Factor 3', 'HBGF-3', 'Fibroblast Growth Factor 3 (Murine Mammary Tumor Virus Integration Site (V-Int-2) Oncogene Homolog)', 'V-INT2 Murine Mammary Tumor Virus Integration Site Oncogene Homolog', 'Murine Mammary Tumor Virus Integration Site 2, Mouse', 'Heparin-Binding Growth Factor 3', 'INT-2 Proto-Oncogene Protein', 'Proto-Oncogene Int-2', 'Oncogene INT2', 'FGF-3', 'INT2', 'FGF3'], ['Fibroblast Growth Factor 4', 'Transforming Protein KS3', 'HBGF-4', 'HST-1', 'HST', 'Human Stomach Cancer, Transforming Factor From FGF-Related Oncogene', 'Heparin Secretory Transforming Protein 1', 'Heparin-Binding Growth Factor 4', 'Kaposi Sarcoma Oncogene', 'HSTF-1', 'FGF-4', 'HSTF1', 'K-FGF', 'KFGF', 'Heparin Secretory-Transforming Protein 1', 'Oncogene HST', 'FGF4', 'KS3'], ['Fibroblast Growth Factor 5', 'Heparin-Binding Growth Factor 5', 'Smag-82', 'HBGF-5', 'TCMGLY', 'FGF-5', 'FGF5'], ['Fibroblast Growth Factor 6', 'Heparin Secretory-Transforming Protein 2', 'Heparin-Binding Growth Factor 6', 'HBGF-6', 'HSTF-2', 'FGF-6', 'HST-2', 'HST2', 'HSTF2', 'FGF6'], ['Fibroblast Growth Factor 7', 'Keratinocyte Growth Factor', 'KGF', 'Heparin-Binding Growth Factor 7', 'HBGF-7', 'FGF-7', 'Fibroblast Growth Factor 7 (Keratinocyte Growth Factor)', 'FGF7'], ['Fibroblast Growth Factor 8', 'Androgen-Induced Growth Factor', 'AIGF', 'Fibroblast Growth Factor 8 (Androgen-Induced)', 'Heparin-Binding Growth Factor 8', 'HBGF-8', 'FGF-8', 'KAL6', 'FGF8', 'HH6'], ['Fibroblast Growth Factor 9', 'Fibroblast Growth Factor 9 (Glia-Activating Factor)', 'Heparin-Binding Growth Factor 9', 'Glia-Activating Factor', 'HBGF-9', 'FGF-9', 'GAF', 'HBFG-9', 'SYNS3', 'FGF9'], ['Fibroblast Growth Factor 10', 'Keratinocyte Growth Factor 2', 'FGF-10', 'Produced By Fibroblasts Of Urinary Bladder Lamina Propria', 'FGF10'], ['Fibroblast Growth Factor 11', 'Fibroblast Growth Factor Homologous Factor 3', 'FHF3', 'FGF-11', 'FHF-3', 'MGC102953', 'FLJ16061', 'MGC45269', 'FGF11'], ['Fibroblast Growth Factor 12', 'Fibroblast Growth Factor Homologous Factor 1', 'Myocyte-Activating Factor', 'FHF1', 'Fibroblast Growth Factor FGF-12b', 'Fibroblast Growth Factor 12B', 'FGF12B', 'EIEE47', 'FGF-12', 'DEE47', 'FGF12', 'FHF-1'], ['Fibroblast Growth Factor 13', 'Fibroblast Growth Factor Homologous Factor 2', 'FHF2', 'FGF-13', 'FHF-2', 'FGF2', 'Long Intergenic Non-Protein Coding RNA 889', 'LINC00889', 'FLJ30672', 'DEE90', 'FGF13'], ['Fibroblast Growth Factor 14', 'FHF4', 'Fibroblast Growth Factor Homologous Factor 4', 'FGF-14', 'FHF-4', 'SCA27', 'FGF14'], ['Fibroblast Growth Factor 16', 'Metacarpal 4-5 Fusion', 'FGF-16', 'FGF16', 'MF4'], ['Fibroblast Growth Factor 17', 'FGF-13', 'FGF-17', 'FGF17', 'HH20'], ['Fibroblast Growth Factor 18', 'FGF-18', 'ZFGF5', 'FGF18'], ['Fibroblast Growth Factor 19', 'FGF-19', 'FGF19'], ['Fibroblast Growth Factor 20', 'FGF-20', 'RHDA2', 'FGF20'], ['Fibroblast Growth Factor 21', 'FGF-21', 'FGF21'], ['Fibroblast Growth Factor 22', 'FGF-22', 'FGF22'], ['Fibroblast Growth Factor 23', 'Phosphatonin', 'HYPF', 'Tumor-Derived Hypophosphatemia Inducing Factor', 'Tumor-Derived Hypophosphatemia-Inducing Factor', 'FGF-23', 'HFTC2', 'HPDR2', 'PHPTC', 'FGF23', 'ADHR', 'FGFN'], ['Fibroblast Growth Factor Binding Protein 1', 'FGFBP', 'HBP17', '17 KDa Heparin-Binding Growth Factor-Binding Protein', 'Fibroblast Growth Factor-Binding Protein 1', '17 KDa HBGF-Binding Protein', 'FGF-Binding Protein 1', 'FGF-BP1', 'FGFBP-1', 'FGF-BP', 'Heparin-Binding Growth Factor Binding Protein', 'FGFBP1', 'HBp17'], ['Fibroblast Growth Factor Binding Protein 2', 'KSP37', 'Killer-Specific Secretory Protein Of 37 KDa', 'Fibroblast Growth Factor-Binding Protein 2', '37 KDa Killer-Specific Secretory Protein', 'FGF-Binding Protein 2', 'HBp17-Related Protein', 'HBp17-RP', 'FGF-BP2', 'FGFBP-2', 'HBP17RP', 'FGFBP2', 'Ksp37'], ['Fibroblast Growth Factor Binding Protein 3', 'Fibroblast Growth Factor-Binding Protein 3', 'FGF-Binding Protein 3', 'C10orf13', 'FGF-BP3', 'FGFBP-3', 'Chromosome 10 Open Reading Frame 13', 'MGC39320', 'FGFBP3'], ['Fibroblast Growth Factor Receptor 1', 'BFGFR', 'CEK', 'FLG', 'Basic Fibroblast Growth Factor Receptor 1', 'Fms-Related Tyrosine Kinase 2', 'Proto-Oncogene C-Fgr', 'EC 2.7.10.1', 'BFGF-R-1', 'FGFR-1', 'CD331', 'FGFBR', 'FLT-2', 'HBGFR', 'N-SAM', 'FLT2', 'Heparin-Binding Growth Factor Receptor', 'FMS-Like Tyrosine Kinase 2', 'Hydroxyaryl-Protein Kinase', 'Fms-Like Tyrosine Kinase 2', 'FGFR1/PLAG1 Fusion', 'Pfeiffer Syndrome', 'CD331 Antigen', 'EC 2.7.10', 'HRTFDS', 'FGFR1', 'N-Sam', 'ECCL', 'KAL2', 'HH2', 'OGD', 'H2', 'H3', 'H4', 'H5'], ['FGFR1 Oncogene Partner 2', 'Fibroblast Growth Factor Receptor 1 Oncogene Partner 2', 'Wound Inducible Transcript 3.0', 'DKFZp564O1863', 'HSPC123-Like', 'FGFR1OP2', 'WIT3.0'], ['Fibroblast Growth Factor Receptor 2', 'Keratinocyte Growth Factor Receptor', 'Bacteria-Expressed Kinase', 'EC 2.7.10.1', 'CD332', 'K-SAM', 'CEK3', 'ECT1', 'KGFR', 'TK14', 'TK25', 'BEK', 'Protein Tyrosine Kinase, Receptor Like 14', 'BEK Fibroblast Growth Factor Receptor', 'Craniofacial Dysostosis 1', 'Jackson-Weiss Syndrome', 'Pfeiffer Syndrome', 'Crouzon Syndrome', 'CD332 Antigen', 'EC 2.7.10', 'FGFR-2', 'BFR-1', 'FGFR2', 'K-Sam', 'BBDS', 'CFD1', 'KSAM', 'JWS'], ['Fibroblast Growth Factor Receptor 3', 'JTK4', 'EC 2.7.10.1', 'FGFR-3', 'CD333', 'CEK2', 'Fibroblast Growth Factor Receptor 3 Variant 4', 'Achondroplasia, Thanatophoric Dwarfism', 'Fibroblast Growth Factor Receptor 3-S', 'Hydroxyaryl-Protein Kinase', 'Tyrosine Kinase JTK4', 'CD333 Antigen', 'HSFGFR3EX', 'EC 2.7.10', 'FGFR3', 'ACH'], ['Fibroblast Growth Factor Receptor 4', 'JTK2', 'EC 2.7.10.1', 'CD334', 'TKF', 'Tyrosine Kinase Related To Fibroblast Growth Factor Receptor', 'Hydroxyaryl-Protein Kinase', 'Protein-Tyrosine Kinase', 'Tyrosylprotein Kinase', 'CD334 Antigen', 'EC 2.7.10', 'FGFR-4', 'FGFR4'], ['Fibroblast Growth Factor Receptor Like 1', 'Fibroblast Growth Factor Receptor-Like 1', 'Fibroblast Growth Factor Receptor 5', 'FGF Homologous Factor Receptor', 'FGF Receptor-Like Protein 1', 'FGFR-Like Protein', 'FGFR-5', 'FGFR5', 'FHFR', 'FGFRL1'], ['Fibrinogen Gamma Chain', 'Fibrinogen, Gamma Polypeptide', 'Testicular Tissue Protein Li 70', 'FGG'], ['FGGY Carbohydrate Kinase Domain Containing', 'FGGY Carbohydrate Kinase Domain-Containing Protein', 'D-Ribulokinase', 'EC 2.7.1.-', 'FLJ10986', 'EC 2.7.1', 'FGGY'], ['Fibrinogen Like 1', 'Hepassocin', 'HPS', 'Hepatocyte-Derived Fibrinogen-Related Protein 1', 'Liver Fibrinogen-Related Protein 1', 'Fibrinogen-Like Protein 1', 'LFIRE-1', 'HFREP-1', 'HFREP1', 'HP-041', 'Hepatocellular Carcinoma-Related Sequence', 'LFIRE1', 'FGL1'], ['Fibrinogen Like 2', 'Fibroleukin', 'PT49', 'Fibrinogen-Like Protein 2', 'T49', 'Fibrinogen-Like 2', 'FGL2'], ['FGR Proto-Oncogene, Src Family Tyrosine Kinase', 'Gardner-Rasheed Feline Sarcoma Viral (V-Fgr) Oncogene Homolog', 'V-Fgr Feline Gardner-Rasheed Sarcoma Viral Oncogene Homolog', 'Feline Gardner-Rasheed Sarcoma Viral Oncogene Homolog', 'Tyrosine-Protein Kinase Fgr', 'Proto-Oncogene C-Fgr', 'EC 2.7.10.2', 'P55c-Fgr', 'P58c-Fgr', 'P55-Fgr', 'P58-Fgr', 'C-Fgr', 'SRC2', 'Proto-Oncogene Tyrosine-Protein Kinase FGR', 'C-Src-2 Proto-Oncogene', 'C-Fgr Protooncogene', 'P55-C-Fgr Protein', 'EC 2.7.10', 'C-Src2', 'FGR'], ['Fumarate Hydratase', 'Fumarase', 'Fumarate Hydratase, Mitochondrial', 'EC 4.2.1.2', 'HsFH', 'Epididymis Secretory Sperm Binding Protein', 'HLRCC', 'MCUL1', 'FMRD', 'LRCC', 'MCL', 'FH'], ['Forkhead Associated Phosphopeptide Binding Domain 1', 'Forkhead-Associated (FHA) Phosphopeptide Binding Domain 1', 'Forkhead-Associated Domain-Containing Protein 1', 'FHA Domain-Containing Protein 1', 'KIAA1937', 'FHAD1'], ['FH2 Domain Containing 1', 'Inverted Formin-1', 'INF1', 'FH2 Domain-Containing Protein 1', 'KIAA1727', 'FHDC1'], ['FHF Complex Subunit HOOK Interacting Protein 1A', 'Family With Sequence Similarity 160 Member A1', 'Protein FAM160A1', 'FAM160A1', 'FHIP-L', 'FLJ43373'], ['FHF Complex Subunit HOOK Interacting Protein 1B', 'Family With Sequence Similarity 160 Member A2', 'FHIP', 'FTS And Hook-Interacting Protein', 'C11orf56', 'FAM160A2', 'KIAA1759', 'Chromosome 11 Open Reading Frame 56', 'FTS And Hook Interacting Protein', 'Protein FAM160A2', 'DKFZP566M1046', 'FLJ22665'], ['FHF Complex Subunit HOOK Interacting Protein 2A', 'Family With Sequence Similarity 160 Member B1', 'KIAA1600', 'Protein FAM160B1', 'BA106M7.3', 'FAM160B1'], ['FHF Complex Subunit HOOK Interacting Protein 2B', 'Family With Sequence Similarity 160 Member B2', 'Retinoic Acid-Induced Protein 16', 'Retinoic Acid Induced 16', 'Protein FAM160B2', 'FAM160B2', 'RAI16', 'FLJ21801'], ['Fragile Histidine Triad Diadenosine Triphosphatase', "Bis(5'-Adenosyl)-Triphosphatase", 'AP3Aase', "Diadenosine 5',5'''-P1,P3-Triphosphate Hydrolase", 'Dinucleosidetriphosphatase', 'AP3A Hydrolase', 'FRA3B', 'Fragile Histidine Triad Protein', 'Fragile Histidine Triad Gene', 'Fragile Histidine Triad', 'EC 3.6.1.29', 'FHIT'], ['Four And A Half LIM Domains 1', 'SLIM1', 'Four And A Half LIM Domains Protein 1', 'Skeletal Muscle LIM-Protein 1', 'LIM Protein SLIMMER', 'SLIM-1', 'FHL-1', 'FHL1B', 'FLH1A', 'XMPMA', 'SLIM', 'Four-And-A-Half Lin11, Isl-1 And Mec-3 Domains 1', 'Four-And-A-Half LIM Domains 1', 'BA535K18.1', 'MGC111107', 'SLIMMER', 'RBMX1A', 'RBMX1B', 'FCMSU', 'FHL1A', 'KYO-T', 'KYOT', 'FHL1'], ['Four And A Half LIM Domains 2', 'SLIM3', 'DRAL', 'Four And A Half LIM Domains Protein 2', 'Skeletal Muscle LIM-Protein 3', 'LIM Domain Protein DRAL', 'SLIM-3', 'FHL-2', 'Down-Regulated In Rhabdomyosarcoma LIM Protein', 'Aging-Associated Gene 11', 'AAG11', 'FHL2'], ['Four And A Half LIM Domains 3', 'SLIM2', 'Four And A Half LIM Domains Protein 3', 'Skeletal Muscle LIM-Protein 2', 'SLIM-2', 'FHL-3', 'LIM-Only Protein FHL3', 'FHL3'], ['Four And A Half LIM Domains 5', 'ACT', 'Activator Of CAMP-Responsive Element Modulator In Testis', 'Four And A Half LIM Domains Protein 5', 'DJ393D12.2', 'FHL-5', 'Activator Of CAMP-Responsive Element Modulator (CREM) In Testis', 'Activator Of CREM In Testis', 'LIM Protein ACT', '1700027G07Rik', 'FLJ33049', 'FHL5'], ['Formin Homology 2 Domain Containing 1', 'FHOS', 'Formin Homolog Overexpressed In Spleen 1', 'FH1/FH2 Domain-Containing Protein 1', 'Formin Homology 2 Domain-Containing Protein 1', 'FHOD1', 'FHOS1'], ['Formin Homology 2 Domain Containing 3', 'FHOS2', 'Formin Homolog Overexpressed In Spleen 2', 'FH1/FH2 Domain-Containing Protein 3', 'Formactin-2', 'KIAA1695', 'Formactin2', 'FLJ22297', 'FLJ22717', 'HFHOS2', 'FHOD3'], ['Fibrinogen C Domain Containing 1', 'Fibrinogen C Domain-Containing Protein 1', 'FLJ14810', 'FIBCD1'], ['Fin Bud Initiation Factor Homolog', 'Fin Bud Initiation Factor Homolog (Zebrafish)', 'MGC24932', 'FIBIN'], ['FGF1 Intracellular Binding Protein', 'Acidic Fibroblast Growth Factor Intracellular-Binding Protein', 'AFGF Intracellular-Binding Protein', 'FGFIBP', 'Fibroblast Growth Factor (Acidic) Intracellular Binding Protein', 'FGF-1 Intracellular-Binding Protein', 'FIBP-1', 'TROFAS', 'FIBP'], ['FIC Domain Protein Adenylyltransferase', 'HIP13', 'HYPE', 'Huntingtin-Interacting Protein 13', 'Protein Adenylyltransferase FICD', 'Huntingtin-Interacting Protein E', 'FIC Domain-Containing Protein', 'Huntingtin Yeast Partner E', 'FIC Domain Containing', 'De-AMPylase FICD', 'AMPylator FICD', 'HIP-13', 'Fic S-Phase Protein Cell Division Homolog (E. Coli)', 'Adenosine Monophosphate-Protein Transferase FICD', 'Fic S-Phase Protein Cell Division Homolog', 'Huntingtin Interacting Protein 13', 'Huntingtin Interacting Protein E', 'Huntingtin Interactor Protein E', 'EC 2.7.7.n1', 'EC 3.1.4.-', 'UNQ3041', 'FICD'], ['FIG4 Phosphoinositide 5-Phosphatase', 'KIAA0274', 'SAC3', 'Phosphatidylinositol 3,5-Bisphosphate 5-Phosphatase', 'Polyphosphoinositide Phosphatase', 'SAC Domain-Containing Protein 3', 'DJ249I4.1', 'ALS11', 'CMT4J', 'FIG4 Homolog, SAC1 Lipid Phosphatase Domain Containing (S. Cerevisiae)', 'FIG4 Homolog, SAC1 Domain Containing Lipid Phosphatase', 'FIG4 Homolog, SAC1 Lipid Phosphatase Domain Containing', 'FIG4 Homolog, SAC Domain Containing Lipid Phosphatase', 'Sac Domain-Containing Inositol Phosphatase 3', 'FIG4 Homolog (S. Cerevisiae)', 'EC 3.1.3.36', 'EC 3.1.3.-', 'EC 3.1.3', 'HSac3', 'BTOP', 'FIG4', 'YVS'], ['Folliculogenesis Specific BHLH Transcription Factor', 'Factor In The Germline Alpha', 'Folliculogenesis-Specific Basic Helix-Loop-Helix Protein', 'Class C Basic Helix-Loop-Helix Protein 8', 'Transcription Factor FIGa', 'BHLHC8', 'BHLHc8', 'Folliculogenesis Specific Basic Helix-Loop-Helix', 'FIGALPHA', 'Figalpha', 'FIGalpha', 'FIGLA', 'POF6'], ['Fidgetin, Microtubule Severing Factor', 'Fidgetin', 'FIGN'], ['Fidgetin Like 1', 'Fidgetin-Like Protein 1', 'Fidgetin-Like 1', 'EC 3.6.4.-', 'EC 3.6.4', 'FIGNL1'], ['Fidgetin Like 2', 'Fidgetin-Like Protein 2', 'Putative Fidgetin-Like Protein 2', 'Fidgetin-Like 2', 'FIGNL2'], ['Filamin A Interacting Protein 1', 'FILIP', 'Filamin-A-Interacting Protein 1', 'KIAA1275', 'FILIP1'], ['Filamin A Interacting Protein 1 Like', 'DOC-1', 'Protein Down-Regulated In Ovarian Cancer 1', 'Filamin A-Interacting Protein 1-Like', '130 KDa GPBP-Interacting Protein', '90 KDa GPBP-Interacting Protein', 'GIP130', 'GIP90', 'DOC1', 'GPBP-Interacting Protein Of 130 KDa', 'Downregulated In Ovarian Cancer 1', 'COL4A3BPIP', 'FILIP1L'], ['Factor Interacting With PAPOLA And CPSF1', 'FIP1', 'FIP1L1 Cleavage And Polyadenylation Specific Factor Subunit', "Pre-MRNA 3'-End-Processing Factor FIP1", 'Rearranged In Hypereosinophilia', 'Factor Interacting With PAP', 'FIP1-Like 1 Protein', 'HFip1', 'FIP1 Like 1 (S. Cerevisiae)', 'DKFZp586K0717', 'FIP1 Like 1', 'EC 4.2.1.33', 'EC 1.1.1.86', 'FIP1L1', 'Rhe', 'RHE'], ['Fission, Mitochondrial 1', 'Tetratricopeptide Repeat Protein 11', 'Tetratricopeptide Repeat Domain 11', 'Mitochondrial Fission 1 Protein', 'Mitochondrial Fission Molecule', 'TPR Repeat Protein 11', 'H_NH0132A01.6', 'FIS1 Homolog', 'CGI-135', 'TTC11', 'HFis1', 'Fission 1 (Mitochondrial Outer Membrane) Homolog (S. Cerevisiae)', 'Fission 1 (Mitochondrial Outer Membrane) Homolog (Yeast)', 'Fission 1 (Mitochondrial Outer Membrane) Homolog', 'CGI-135 Protein', 'FIS1'], ['Fat Storage Inducing Transmembrane Protein 1', 'FIT1', 'Fat Storage-Inducing Transmembrane Protein 1', 'Fat-Inducing Transcript 1', 'Fat-Inducing Protein 1', 'FITM1'], ['Fat Storage Inducing Transmembrane Protein 2', 'Acyl-Coenzyme A Diphosphatase FITM2', 'Fat-Inducing Protein 2', 'DJ881L22.2', 'C20orf142', 'FIT2', 'Fat Storage-Inducing Transmembrane Protein 2', 'Chromosome 20 Open Reading Frame 142', 'Fat Inducing Transcript 2', 'Fat-Inducing Transcript 2', 'EC 3.6.1.-', 'SIDDIS', 'FITM2', 'Fit2'], ['FLT3 Interacting Zinc Finger 1', 'ZNF798', 'Flt3-Interacting Zinc Finger Protein 1', 'Zinc Finger Protein 798', 'FLJ14768', 'FIZ1'], ['Four-Jointed Box Kinase 1', 'Putative Secreted Ligand Homologous To Fjx1', 'Four-Jointed Protein Homolog', 'Four-Jointed Box Protein 1', 'Four Jointed Box 1', 'Four Jointed Box 1 (Drosophila)', 'FLJ22416', 'FLJ25593', 'FJX1'], ['FKBP Prolyl Isomerase 1A', 'FKBP-12', 'FKBP12', 'Peptidyl-Prolyl Cis-Trans Isomerase FKBP1A', 'FK506 Binding Protein 1A, 12kDa', '12 KDa FK506-Binding Protein', 'FK506-Binding Protein 1A', 'Immunophilin FKBP12', 'PPIase FKBP1A', '12 KDa FKBP', 'Calstabin-1', 'EC 5.2.1.8', 'Rotamase', 'FKBP-1A', 'PPIASE', 'FKBP1', 'PKC12', 'FK506-Binding Protein, T-Cell, 12-KD', 'FK506-Binding Protein 1A (12kD)', 'Protein Kinase C Inhibitor 2', 'FK506 Binding Protein 1A', 'FK506-Binding Protein 12', 'FK506 Binding Protein12', 'FK506-Binding Protein 1', 'FKBP12-Exip3', 'Calstabin 1', 'FKBP12C', 'FKBP1A', 'PKCI2'], ['FKBP Prolyl Isomerase 1B', 'FKBP12.6', 'OTK4', 'Peptidyl-Prolyl Cis-Trans Isomerase FKBP1B', 'FK506 Binding Protein 1B, 12.6 KDa', '12.6 KDa FK506-Binding Protein', 'FK506-Binding Protein 1B', 'Immunophilin FKBP12.6', '12.6 KDa FKBP', 'PPIase FKBP1B', 'Calstabin 2', 'EC 5.2.1.8', 'FKBP-12.6', 'H-FKBP-12', 'Rotamase', 'FKBP-1B', 'FKBP1L', 'PPIase', 'FKBP9', 'FK506-Binding Protein 1B (12.6 KD)', 'FK506-Binding Protein 12.6', 'FK506 Binding Protein 1B', 'PKBP1L', 'FKBP1B'], ['FKBP Prolyl Isomerase 2', 'FKBP-13', 'FKBP13', 'Peptidyl-Prolyl Cis-Trans Isomerase FKBP2', 'FK506 Binding Protein 2, 13kDa', '13 KDa FK506-Binding Protein', 'Rapamycin-Binding Protein', 'FK506-Binding Protein 2', 'Immunophilin FKBP13', 'Proline Isomerase', 'PPIase FKBP2', '13 KDa FKBP', 'Rotamase', 'PPIase', 'Epididymis Secretory Sperm Binding Protein', 'Peptidyl-Prolyl Cis-Trans Isomerase', 'FK506 Binding Protein 2 (13kD)', 'FK506-Binding Protein 2 (13kD)', 'FK506 Binding Protein 2', 'EC 5.2.1.8', 'FKBP-2', 'FKBP2'], ['FKBP Prolyl Isomerase 3', 'FKBP-25', 'FKBP25', 'Peptidyl-Prolyl Cis-Trans Isomerase FKBP3', 'Rapamycin-Selective 25 KDa Immunophilin', 'FK506 Binding Protein 3, 25kDa', '25 KDa FK506-Binding Protein', 'FK506-Binding Protein 3', 'Immunophilin FKBP25', 'PPIase FKBP3', '25 KDa FKBP', 'Rotamase', 'FKBP-3', 'FK506-Binding Protein 25, T-Cell', 'FK506-Binding Protein 3 (25kD)', 'Rapamycin Binding Protein', 'FK506 Binding Protein 3', 'EC 5.2.1.8', 'PPIase', 'FKBP3'], ['FKBP Prolyl Isomerase 4', 'FKBP52', 'FKBP59', 'Peptidyl-Prolyl Cis-Trans Isomerase FKBP4', 'FK506 Binding Protein 4, 59kDa', 'EC 5.2.1.8', 'Rotamase', 'FKBP51', 'HBI', 'T-Cell FK506-Binding Protein, 59kD', 'Peptidylprolyl Cis-Trans Isomerase', 'FK506-Binding Protein 4 (59kD)', '51 KDa FK506-Binding Protein', '52 KDa FK506-Binding Protein', 'HSP Binding Immunophilin', 'HSP-Binding Immunophilin', 'FK506 Binding Protein 4', 'FK506-Binding Protein 4', '59 KDa Immunophilin', 'Immunophilin FKBP52', 'PPIase FKBP4', '52 KDa FKBP', 'FKBP-52', 'PPIase', 'FKBP-4', 'Hsp56', 'FKBP4', 'P52', 'P59'], ['FKBP Prolyl Isomerase 5', 'FKBP51', 'FKBP54', 'P54', '54 KDa Progesterone Receptor-Associated Immunophilin', 'Peptidyl-Prolyl Cis-Trans Isomerase FKBP5', '51 KDa FK506-Binding Protein', 'Androgen-Regulated Protein 6', 'HSP90-Binding Immunophilin', 'FK506-Binding Protein 5', 'FK506 Binding Protein 5', 'PPIase FKBP5', '51 KDa FKBP', 'FF1 Antigen', 'EC 5.2.1.8', 'Rotamase', 'FKBP-51', 'PPIase', 'Ptg-10', 'AIG6', 'Peptidylprolyl Cis-Trans Isomerase', 'T-Cell FK506-Binding Protein', 'FKBP-5', 'FKBP5'], ['FKBP Prolyl Isomerase Family Member 6 (Inactive)', 'Immunophilin FKBP36', 'FKBP36', 'Inactive Peptidyl-Prolyl Cis-Trans Isomerase FKBP6', 'FK506 Binding Protein 6, 36kDa', 'FK506-Binding Protein 6', 'FKBP Prolyl Isomerase 6', 'Inactive PPIase FKBP6', '36 KDa FKBP', 'Rotamase', 'Peptidyl-Prolyl Cis-Trans Isomerase FKBP6', 'Peptidylprolyl Cis-Trans Isomerase', 'FK506 Binding Protein 6 (36kD)', 'FK506-Binding Protein 6 (36kD)', '36 KDa FK506-Binding Protein', 'PPIase FKBP6', 'FKBP-36', 'PPIase', 'FKBP-6', 'FKBP6'], ['FKBP Prolyl Isomerase 7', 'FKBP23', 'Peptidyl-Prolyl Cis-Trans Isomerase FKBP7', '23 KDa FK506-Binding Protein', 'FK506-Binding Protein 7', 'FK506 Binding Protein 7', 'PPIase FKBP7', '23 KDa FKBP', 'Rotamase', 'FKBP-23', 'FKBP-7', 'FK506-Binding Protein 23', 'EC 5.2.1.8', 'PPIase', 'FKBP7'], ['FKBP Prolyl Isomerase 8', 'FKBP38', 'Peptidyl-Prolyl Cis-Trans Isomerase FKBP8', 'FK506 Binding Protein 8, 38kDa', '38 KDa FK506-Binding Protein', 'FK506-Binding Protein 8', 'PPIase FKBP8', '38 KDa FKBP', 'Rotamase', 'FKBPr38', 'FKBP-38', 'HFKBP38', 'FKBP-8', 'FK506-Binding Protein 8 (38kD)', 'FK506 Binding Protein 8', 'EC 5.2.1.8', 'FKBPR38', 'FKBP8'], ['FKBP Prolyl Isomerase 9', 'FKBP60', 'FKBP63', 'Peptidyl-Prolyl Cis-Trans Isomerase FKBP9', 'FK506 Binding Protein 9, 63 KDa', '63 KDa FK506-Binding Protein', 'FK506-Binding Protein 9', 'PPIase FKBP9', '63 KDa FKBP', 'EC 5.2.1.8', 'Rotamase', 'FKBP-63', 'FKBP-9', 'FK506-Binding Protein 9 (63 KD)', 'FK506 Binding Protein 9', 'PPIase', 'FKBP9'], ['FKBP Prolyl Isomerase 10', 'FKBP65', 'Peptidyl-Prolyl Cis-Trans Isomerase FKBP10', 'FK506 Binding Protein 10, 65 KDa', '65 KDa FK506-Binding Protein', 'FK506-Binding Protein 10', 'Immunophilin FKBP65', 'PPIase FKBP10', '65 KDa FKBP', 'EC 5.2.1.8', 'Rotamase', 'HFKBP65', 'FKBP-10', 'FKBP-65', 'FK506 Binding Protein 10 (65 KDa)', 'FK506 Binding Protein 10', 'FLJ22041', 'FLJ20683', 'FLJ23833', 'PPIASE', 'FKBP10', 'BRKS1', 'FKBP6', 'OI11', 'OI6'], ['FKBP Prolyl Isomerase 11', 'FKBP19', 'Peptidyl-Prolyl Cis-Trans Isomerase FKBP11', 'FK506 Binding Protein 11, 19 KDa', '19 KDa FK506-Binding Protein', 'FK506-Binding Protein 11', 'PPIase FKBP11', '19 KDa FKBP', 'Rotamase', 'FKBP-11', 'FKBP-19', 'FK506 Binding Protein 11 (19 KDa)', 'FK506 Binding Protein 11', 'EC 5.2.1.8', 'FKBP11'], ['FKBP Prolyl Isomerase 14', 'FKBP22', 'Peptidyl-Prolyl Cis-Trans Isomerase FKBP14', 'FK506 Binding Protein 14, 22 KDa', '22 KDa FK506-Binding Protein', 'FK506-Binding Protein 14', 'PPIase FKBP14', '22 KDa FKBP', 'Rotamase', 'FKBP-22', 'FK506 Binding Protein 14 (22 KDa)', 'FK506 Binding Protein 14', 'EC 5.2.1.8', 'EDSKSCL2', 'FLJ20731', 'FKBP-14', 'EDSKMH', 'IPBP12', 'FKBP14'], ['FKBP Prolyl Isomerase Family Member 15', 'KIAA0674', 'WAFL', 'Protein Phosphatase 1, Regulatory Subunit 76', 'FK506 Binding Protein 15, 133kDa', '133 KDa FK506-Binding Protein', 'WASP And FKBP-Like Protein', 'FK506-Binding Protein 15', 'FKBP Prolyl Isomerase 15', '133 KDa FKBP', 'FKBP-133', 'FKBP133', 'PPP1R76', 'FKBP-15', 'FK506-Binding Protein 133kDa', 'WASP- And FKBP-Like Protein', 'FK506 Binding Protein 15', 'FKBP15'], ['FKBP Prolyl Isomerase Like', 'WAF-1/CIP1 Stabilizing Protein 39', 'DIR1', 'NG7', 'FK506 Binding Protein Like', 'FK506-Binding Protein-Like', 'WISp39', 'Peptidyl-Prolyl Cis-Trans Isomerase', 'FK506-Binding Protein Like', 'WISP39', 'FKBP4', 'FKBPL'], ['Fukutin Related Protein', 'Ribitol-5-Phosphate Transferase', 'Fukutin-Related Protein', 'LGMD2I', 'MDC1C', 'FKTR', 'EC 2.4.2.-', 'LGMDR9', 'MDDGA5', 'MDDGB5', 'MDDGC5', 'FKRP'], ['Fukutin', 'Ribitol-5-Phosphate Transferase', 'LGMD2M', 'FCMD', 'Fukuyama Type Congenital Muscular Dystrophy (Fukutin)', 'Fukuyama Type Congenital Muscular Dystrophy Protein', 'Fukuyama-Type Congenital Muscular Dystrophy Protein', 'Patient Fukutin', 'EC 2.4.2.-', 'LGMDR13', 'MDDGA4', 'MDDGB4', 'MDDGC4', 'CMD1X', 'FKTN'], ['Flagellum Associated Containing Coiled-Coil Domains 1', 'Amyotrophic Lateral Sclerosis 2 Chromosomal Region Candidate Gene 12 Protein', 'Amyotrophic Lateral Sclerosis 2 (Juvenile) Chromosome Region, Candidate 12', 'Flagellum-Associated Coiled-Coil Domain-Containing Protein 1', 'ALS2 Chromosome Region 12', 'ALS2CR12', 'Amyotrophic Lateral Sclerosis 2 Chromosome Region Candidate 12', 'Amyotrophic Lateral Sclerosis 2 Chromosome Region 12', 'FLACC1'], ['Flavin Adenine Dinucleotide Synthetase 1', 'FMN Adenylyltransferase', 'FAD Pyrophosphorylase', 'FAD Synthase', 'EC 2.7.7.2', 'PP591', 'FAD1', 'FAD1 Flavin Adenine Dinucleotide Synthetase Homolog (S. Cerevisiae)', 'Fad1, Flavin Adenine Dinucleotide Synthetase, Homolog (Yeast)', 'Fad1, Flavin Adenine Dinucleotide Synthetase, Homolog', 'Flavin Adenine Dinucleotide Synthetase', 'Flavin Adenine Dinucleotide Synthase', 'FAD-Synthetase', 'EC 2.7.7', 'LSMFLAD', 'FLAD1', 'FADS'], ['Folliculin', 'BHD', 'BHD Skin Lesion Fibrofolliculoma Protein', 'Birt-Hogg-Dube Syndrome Protein', 'DENND8B', 'MGC17998', 'MGC23445', 'FLCL', 'FLCN'], ['Filaggrin', 'Epidermal Filaggrin', 'ATOD2', 'FLG'], ['Filaggrin Family Member 2', 'IFPS', 'Ifapsoriasin', 'Filaggrin-2', 'Intermediate Filament-Associated And Psoriasis Susceptibility Protein', 'Intermediate Filament-Associated And Psoriasis-Susceptibility Protein', 'FLG-2', 'PSS6', 'FLG2'], ['Fli-1 Proto-Oncogene, ETS Transcription Factor', 'Friend Leukemia Integration 1 Transcription Factor', 'Friend Leukemia Virus Integration 1', 'Transcription Factor ERGB', 'EWSR2', 'SIC-1', 'Ewing Sarcoma Breakpoint Region 2', 'Proto-Oncogene Fli-1', 'BDPLT21', 'FLI1'], ['FLII Actin Remodeling Protein', 'FLIL', 'Flightless I Actin Binding Protein', 'Protein Flightless-1 Homolog', 'Fli1', 'FLI', 'Flightless I (Drosophila) Homolog', 'Flightless I Homolog (Drosophila)', 'Flightless I Homolog', 'MGC39265', 'FLII'], ['Filamin A', 'ABP-280', 'Endothelial Actin-Binding Protein', 'Actin Binding Protein 280', 'Non-Muscle Filamin', 'Filamin A, Alpha', 'Alpha-Filamin', 'Filamin-A', 'Filamin-1', 'FLN-A', 'FLN1', 'FLN', 'Filamin A, Alpha (Actin Binding Protein 280)', 'Epididymis Secretory Sperm Binding Protein', 'Actin-Binding Protein 280', 'Alpha Filamin', 'EC 2.1.1.43', 'EC 6.3.4.4', 'ABPX', 'CSBS', 'CVD1', 'FGS2', 'NHBP', 'OPD1', 'OPD2', 'XLVD', 'XMVD', 'FLNA', 'FMD', 'MNS', 'OPD'], ['Filamin B', 'ABP-278', 'TABP', 'TAP', 'Larsen Syndrome 1 (Autosomal Dominant)', 'Actin-Binding-Like Protein', 'Actin Binding Protein 278', 'Thyroid Autoantigen', 'Filamin Homolog 1', 'Filamin B, Beta', 'ABP-280 Homolog', 'Beta-Filamin', 'Filamin-B', 'Filamin-3', 'FLN-B', 'FLN1L', 'FH1', 'Filamin B, Beta (Actin Binding Protein 278)', 'Truncated Actin-Binding Protein', 'Truncated ABP', 'Beta Filamin', 'ABP-280', 'LRS1', 'FLNB', 'FLN3', 'AOI', 'SCT', 'Fh1'], ['Filamin C', 'ABPL', 'Actin Binding Protein 280', 'ABP-280-Like Protein', 'Filamin C, Gamma', 'Gamma Filamin', 'Filamin-C', 'Filamin-2', 'ABP-280', 'FLN2', 'Filamin C, Gamma (Actin Binding Protein 280)', 'Actin-Binding-Like Protein', 'ABP-L, Gamma Filamin', 'Gamma-Filamin', 'ABP280A', 'CMH26', 'FLN-C', 'ABP-L', 'ABPA', 'MFM5', 'MPD4', 'RCM5', 'FLNC', 'FLNc'], ['Flotillin 1', 'Flotillin-1', 'Integral Membrane Component Of Caveolae', 'FLOT1'], ['Flotillin 2', 'ESA1', 'ESA', 'Membrane Component, Chromosome 17, Surface Marker 1 (35kD Protein Identified By Monoclonal Antibody ECS-1)', 'Membrane Component Chromosome 17 Surface Marker 1', 'Flotillin 2 (Epidermal Surface Antigen 1)', 'Epidermal Surface Antigen', 'Flotillin-2', 'ECS-1', 'M17S1', 'ECS1', 'FLOT2'], ['Fibronectin Leucine Rich Transmembrane Protein 1', 'Fibronectin-Like Domain-Containing Leucine-Rich Transmembrane Protein 1', 'Leucine-Rich Repeat Transmembrane Protein FLRT1', 'SPG68', 'MGC21624', 'FLRT1'], ['Fibronectin Leucine Rich Transmembrane Protein 2', 'Fibronectin-Like Domain-Containing Leucine-Rich Transmembrane Protein 2', 'Leucine-Rich Repeat Transmembrane Protein FLRT2', 'KIAA0405', 'FLRT2'], ['Fibronectin Leucine Rich Transmembrane Protein 3', 'Fibronectin-Like Domain-Containing Leucine-Rich Transmembrane Protein 3', 'Leucine-Rich Repeat Transmembrane Protein FLRT3', 'KIAA1469', 'FLRT3', 'HH21'], ['Fms Related Receptor Tyrosine Kinase 1', 'Vascular Endothelial Growth Factor Receptor 1', 'Vascular Permeability Factor Receptor', 'VEGFR1', 'Fms-Related Tyrosine Kinase 1 (Vascular Endothelial Growth Factor/Vascular Permeability Factor Receptor)', 'Tyrosine-Protein Kinase Receptor FLT', 'Fms Related Tyrosine Kinase 1', 'Tyrosine-Protein Kinase FRT', 'Fms-Like Tyrosine Kinase 1', 'EC 2.7.10.1', 'VEGFR-1', 'FLT-1', 'FLT', 'Fms-Related Tyrosine Kinase 1', 'EC 2.7.10', 'FLT1', 'FRT'], ['Fms Related Receptor Tyrosine Kinase 3', 'CD135', 'FLK2', 'STK1', 'Receptor-Type Tyrosine-Protein Kinase FLT3', 'Fms Related Tyrosine Kinase 3', 'Stem Cell Tyrosine Kinase 1', 'Fms-Like Tyrosine Kinase 3', 'FL Cytokine Receptor', 'CD135 Antigen', 'EC 2.7.10.1', 'FLK-2', 'Growth Factor Receptor Tyrosine Kinase Type III', 'Fms-Related Tyrosine Kinase 3', 'Fetal Liver Kinase 2', 'Fetal Liver Kinase-2', 'EC 2.7.10', 'FLT-3', 'STK-1', 'FLT3'], ['Fms Related Receptor Tyrosine Kinase 3 Ligand', 'Fms-Related Tyrosine Kinase 3 Ligand', 'Fms Related Tyrosine Kinase 3 Ligand', 'Flt3 Ligand', 'SL Cytokine', 'FLT3LG', 'FLG3L', 'FLT3L', 'Flt3L', 'FL'], ['Fms Related Receptor Tyrosine Kinase 4', 'Vascular Endothelial Growth Factor Receptor 3', 'VEGFR-3', 'VEGFR3', 'Feline McDonough Sarcoma (FMS)-Like Tyrosine Kinase 4', 'Tyrosine-Protein Kinase Receptor FLT4', 'Primary Congenital Lymphedema', 'Fms Related Tyrosine Kinase 4', 'Fms-Like Tyrosine Kinase 4', 'VEGF Receptor-3', 'EC 2.7.10.1', 'FLT-4', 'PCL', 'Fms-Related Tyrosine Kinase 4', 'EC 2.7.10', 'LMPH1A', 'LMPHM1', 'CHTD7', 'FLT41', 'FLT4'], ['FLVCR Heme Transporter 1', 'FLVCR', 'Feline Leukemia Virus Subgroup C Receptor-Related Protein 1', 'Feline Leukemia Virus Subgroup C Cellular Receptor 1', 'SLC49A1', 'MFSD7B', 'PCA', 'Ataxia, Posterior Column 1, With Retinitis Pigmentosa', 'Feline Leukemia Virus Subgroup C Receptor', 'FLVCR1', 'HFLVCR', 'AXPC1', 'PCARP'], ['FLVCR Heme Transporter 2', 'CCT', 'Feline Leukemia Virus Subgroup C Cellular Receptor Family Member 2', 'Feline Leukemia Virus Subgroup C Receptor-Related Protein 2', 'Calcium-Chelate Transporter', 'C14orf58', 'SLC49A2', 'MFSD7C', 'Feline Leukemia Virus Subgroup C Cellular Receptor 2', 'Chromosome 14 Open Reading Frame 58', 'FLVCRL14q', 'FLJ20371', 'FLVCR2', 'PVHH', 'EPV'], ['FLYWCH-Type Zinc Finger 1', 'FLYWCH-Type Zinc Finger-Containing Protein 1', 'DKFZp761A132', 'KIAA1552', 'FLYWCH1'], ['FLYWCH Family Member 2', 'FLYWCH2'], ['Formation Of Mitochondrial Complex V Assembly Factor 1 Homolog', 'ATP Synthase Assembly Factor FMC1, Mitochondrial', 'Protein FMC1 Homolog', 'C7orf55', 'HSPC268', 'Formation Of Mitochondrial Complexes 1 Homolog (S. Cerevisiae)', 'Formation Of Mitochondrial Complex V Assembly Factor 1', 'Formation Of Mitochondrial Complexes 1 Homolog', 'Chromosome 7 Open Reading Frame 55', 'UPF0562 Protein C7orf55', 'FMC1'], ['Formin 1', 'Limb Deformity Protein Homolog', 'Formin (Limb Deformity)', 'Formin-1', 'FMN', 'LD', 'Limb Deformity Protein', 'DKFZP686C2281', 'MGC125288', 'MGC125289', 'FLJ45135', 'FMN1'], ['Formin 2', 'Formin-2', 'FMN2'], ['Formin Like 1', 'C17orf1', 'CLL-Associated Antigen KW-13', 'Formin-Like Protein 1', 'Leukocyte Formin', 'C17orf1B', 'FMNL', 'Formin-Like 1', 'Formin-Like', 'FHOD4', 'KW-13', 'FMNL1', 'FRL1'], ['Formin Like 2', 'Formin Homology 2 Domain-Containing Protein 2', 'Formin Homology 2 Domain Containing 2', 'Formin-Like Protein 2', 'KIAA1902', 'FHOD2', 'Formin-Like 2', 'FMNL2'], ['Formin Like 3', 'WBP3', 'Formin Homology 2 Domain-Containing Protein 3', 'Formin-Like Protein 3', 'FHOD3', 'WBP-3', 'FRL2', 'WW Domain Binding Protein 3', 'WW Domain-Binding Protein 3', 'Formin-Like 3', 'DKFZp762B245', 'MGC45819', 'KIAA2014', 'FMNL3'], ['Flavin Containing Dimethylaniline Monoxygenase 1', 'Dimethylaniline Monooxygenase [N-Oxide-Forming] 1', 'Fetal Hepatic Flavin-Containing Monooxygenase 1', 'Flavin Containing Monooxygenase 1', 'Dimethylaniline Oxidase 1', 'EC 1.14.13.8', 'FMO 1', 'Flavin-Containing Monooxygenase 1 (Fetal Liver)', 'FMO1'], ['Flavin Containing Dimethylaniline Monoxygenase 2', 'Dimethylaniline Monooxygenase [N-Oxide-Forming] 2', 'Flavin Containing Monooxygenase 2 (Non-Functional)', 'Pulmonary Flavin-Containing Monooxygenase 2', 'Dimethylaniline Oxidase 2', 'EC 1.14.13.8', 'Flavin Containing Monooxygenase 2', 'FMO, Pulmonary', 'FMO 1B1', 'FMO1B1', 'FMO 2', 'FMO2'], ['Flavin Containing Dimethylaniline Monoxygenase 3', 'Dimethylaniline Monooxygenase [N-Oxide-Forming] 3', 'Flavin Containing Monooxygenase 3', 'Trimethylamine Monooxygenase', 'Dimethylaniline Oxidase 3', 'EC 1.14.13.8', 'FMO Form 2', 'FMOII', 'Hepatic Flavin-Containing Monooxygenase-3', 'Hepatic Flavin-Containing Monooxygenase 3', 'EC 1.14.13.148', 'EC 1.14.14.73', 'DJ127D3.1', 'FMO II', 'FMO 3', 'TMAU', 'FMO3'], ['Flavin Containing Dimethylaniline Monoxygenase 4', 'Dimethylaniline Monooxygenase [N-Oxide-Forming] 4', 'Hepatic Flavin-Containing Monooxygenase 4', 'Flavin Containing Monooxygenase 4', 'Dimethylaniline Oxidase 4', 'EC 1.14.13.8', 'FMO 4', 'FMO2', 'FMO4'], ['Flavin Containing Dimethylaniline Monoxygenase 5', 'Dimethylaniline Monooxygenase [N-Oxide-Forming] 5', 'Flavin Containing Monooxygenase 5', 'Flavin-Containing Monooxygenase 5', 'Baeyer-Villiger Monooxygenase 1', 'Dimethylaniline Oxidase 5', 'NAPDH Oxidase', 'EC 1.14.13.8', 'HBVMO1', 'FMO 5', 'Hepatic Flavin-Containing Monooxygenase 5', 'EC 1.14.13.-', 'EC 1.6.3.1', 'FMO5'], ['Fibromodulin', 'SLRR2E', 'Keratan Sulfate Proteoglycan Fibromodulin', 'Collagen-Binding 59 KDa Protein', 'KSPG Fibromodulin', 'FM', 'Epididymis Secretory Sperm Binding Protein', 'Fibromodulin Proteoglycan', 'FMOD'], ['FMRP Translational Regulator 1', 'FMRP', 'Fragile X Mental Retardation Protein 1', 'Synaptic Functional Regulator FMR1', 'Fragile X Mental Retardation 1', 'FRAXA', 'Premature Ovarian Failure 1', 'Truncated FMRP', 'Protein FMR-1', 'MGC87458', 'POF1', 'FMR1', 'POF'], ['FMR1 Neighbor', 'Cancer/Testis Antigen 37', 'CT37', 'Fragile X Mental Retardation 1 Neighbor Protein', 'Sarcoma Antigen NY-SAR-35', 'NY-SAR-35', 'Fragile X Mental Retardation 1 Neighbor', 'FLJ25736', 'NYSAR35', 'FMR1NB'], ['Fibronectin 1', 'Cold-Insoluble Globulin', 'CIG', 'Migration-Stimulating Factor', 'Fibronectin', 'GFND2', 'FINC', 'LETS', 'MSF', 'FN', 'Epididymis Secretory Sperm Binding Protein', 'SMDCF', 'ED-B', 'GFND', 'FNZ', 'FN1'], ['Fructosamine 3 Kinase', 'Protein-Ribulosamine 3-Kinase FN3K', 'Protein-Psicosamine 3-Kinase FN3K', 'Fructosamine-3-Kinase', 'EC 2.7.1.171', 'EC 2.7.1.172', 'FN3K'], ['Fructosamine 3 Kinase Related Protein', 'Protein-Psicosamine 3-Kinase FN3KRP', 'FN3K-Related Protein', 'Ketosamine-3-Kinase', 'FN3KL', 'Testis Secretory Sperm-Binding Protein Li 211a', 'Fructosamine-3-Kinase-Related Protein', 'EC 2.7.1.172', 'EC 2.7.1.-', 'FLJ12171', 'FN3K-RP', 'FN3KRP'], ['Formin Binding Protein 1', 'FBP17', 'Formin-Binding Protein 17', 'Formin-Binding Protein 1', 'KIAA0554', 'HFBP17', 'FNBP1'], ['Formin Binding Protein 1 Like', 'TOCA1', 'Transducer Of Cdc42-Dependent Actin Assembly Protein 1', 'Formin-Binding Protein 1-Like', 'C1orf39', 'Toca-1', 'Transducer Of Cdc42-Dependent Actin Assembly 1', 'Chromosome 1 Open Reading Frame 39', 'Formin Binding Protein 1-Like', 'FLJ20275', 'FNBP1L'], ['Formin Binding Protein 4', 'Formin-Binding Protein 30', 'Formin-Binding Protein 4', 'KIAA1014', 'FBP30', 'FNBP4'], ['Fibronectin Type III Domain Containing 1', 'Fibronectin Type III Domain-Containing Protein 1', 'Fibronectin Type III Domain Containing 2', 'Expressed In Synovial Lining Protein', 'Activation-Associated CDNA Protein', 'BA243O10.1', 'DJ322A24.1', 'KIAA1866', 'MEL4B3', 'FNDC2', 'EC 6.1.1.1', 'EC 2.7.2.3', 'FNDC1', 'AGS8'], ['Fibronectin Type III Domain Containing 3A', 'Fibronectin Type-III Domain-Containing Protein 3A', 'Human Gene Expressed In Odontoblasts', 'BA203I16.5', 'KIAA0970', 'FNDC3', 'HUGO', 'Fibronectin Type III Domain Containing 3', 'BA203I16.1', 'FNDC3A'], ['Fibronectin Type III Domain Containing 3B', 'FAD104', 'Fibronectin Type III Domain-Containing Protein 3B', 'Factor For Adipocyte Differentiation 104', 'HCV NS5A-Binding Protein 37', 'YVTM2421', 'PRO4979', 'DKFZp762K137', 'FLJ23399', 'NS5ABP37', 'FNDC3B'], ['Fibronectin Type III Domain Containing 4', 'FRCP1', 'Fibronectin Type III Domain-Containing Protein 4', 'Fibronectin Type III Repeat-Containing Protein 1', 'FLJ22362', 'FNDC4'], ['Fibronectin Type III Domain Containing 5', 'FRCP2', 'Fibronectin Type III Domain-Containing Protein 5', 'Fibronectin Type III Repeat-Containing Protein 2', 'Irisin', 'FNDC5'], ['Fibronectin Type III Domain Containing 7', 'Fibronectin Type III Domain-Containing Protein 7', 'FLJ35838', 'FNDC7'], ['Fibronectin Type III Domain Containing 8', 'Fibronectin Type III Domain-Containing Protein 8', 'DKFZp434H2215', 'FNDC8'], ['Fibronectin Type III Domain Containing 9', 'Fibronectin Type III Domain-Containing Protein 9', 'C5orf40', 'Fibronectin Type-III Domain-Containing Protein C5orf40', 'Chromosome 5 Open Reading Frame 40', 'MGC27121', 'FNDC9'], ['Fibronectin Type III Domain Containing 10', 'Fibronectin Type III Domain-Containing Protein 10', 'C1orf233', 'Fibronectin Type-III Domain-Containing Transmembrane Protein C1orf233', 'Chromosome 1 Open Reading Frame 233', 'FNDC10'], ['Fibronectin Type III Domain Containing 11', 'Fibronectin Type III Domain-Containing Protein 11', 'C20orf195', 'Chromosome 20 Open Reading Frame 195', 'MGC5356', 'FNDC11'], ['Folliculin Interacting Protein 1', 'Folliculin-Interacting Protein 1', 'KIAA1961', 'FNIP1'], ['Folliculin Interacting Protein 2', 'FNIPL', 'MAPO1', 'O6-Methylguanine-Induced Apoptosis 1 Protein', 'Folliculin-Interacting Protein 2', 'FNIP1-Like Protein', 'KIAA1450', 'O6-Methylguanine-Induced Apoptosis 1', 'FNIP2'], ['Farnesyltransferase, CAAX Box, Alpha', 'Protein Farnesyltransferase/Geranylgeranyltransferase Type-1 Subunit Alpha', 'Protein Prenyltransferase Alpha Subunit Repeat Containing 2', 'Ras Proteins Prenyltransferase Subunit Alpha', 'GGTase-I-Alpha', 'FTase-Alpha', 'PGGT1A', 'PTAR2', 'FPTA', 'Type I Protein Geranyl-Geranyltransferase Alpha Subunit', 'Type I Protein Geranyl-Geranyltransferase Subunit Alpha', 'Farnesyl-Protein Transferase Alpha-Subunit', 'CAAX Farnesyltransferase Subunit Alpha', 'EC 2.5.1.58', 'EC 2.5.1.59', 'FNTA'], ['Farnesyltransferase, CAAX Box, Beta', 'Ras Proteins Prenyltransferase Subunit Beta', 'Protein Farnesyltransferase Subunit Beta', 'CAAX Farnesyltransferase Subunit Beta', 'EC 2.5.1.58', 'FTase-Beta', 'FPTB', 'Epididymis Secretory Sperm Binding Protein', 'FNTB'], ['Focadhesin', 'KIAA1797', 'FLJ20375', 'FOCAD'], ['Folate Hydrolase 1', 'Glutamate Carboxypeptidase 2', 'NAALAD1', 'GCPII', 'PSMA', 'PSM', 'N-Acetylated-Alpha-Linked Acidic Dipeptidase I', 'Pteroylpoly-Gamma-Glutamate Carboxypeptidase', 'Folylpoly-Gamma-Glutamate Carboxypeptidase', 'Cell Growth-Inhibiting Gene 27 Protein', 'Membrane Glutamate Carboxypeptidase', 'Glutamate Carboxypeptidase II', 'Glutamate Carboxylase II', 'EC 3.4.17.21', 'NAALADase I', 'NAALAdase', 'FGCP', 'FOLH', 'GCP2', 'MGCP', 'Folate Hydrolase (Prostate-Specific Membrane Antigen) 1', 'N-Acetylated Alpha-Linked Acidic Dipeptidase 1', 'Prostate Specific Membrane Antigen Variant F', 'Prostate-Specific Membrane Antigen', 'FOLH1'], ['Folate Receptor Alpha', 'Ovarian Tumor-Associated Antigen MOv18', 'Adult Folate-Binding Protein', 'Folate Receptor 1 (Adult)', 'Folate Receptor, Adult', 'Folate Receptor 1', 'KB Cells FBP', 'FR-Alpha', 'FOLR', 'FBP', 'Folate Binding Protein', 'FRalpha', 'NCFTD', 'FOLR1', 'FRα'], ['Folate Receptor Beta', 'Folate Receptor, Fetal/Placental', 'Placental Folate-Binding Protein', 'Folate Receptor 2 (Fetal)', 'FBP', 'Folate-Binding Protein, Fetal/Placental', 'Folate Receptor Alpha', 'Folate Receptor 2', 'BETA-HFR', 'FBP/PL-1', 'FR-BETA', 'FR-Beta', 'FRbeta', 'FOLR1', 'FR-P3', 'FOLR2', 'FRβ'], ['Folate Receptor Gamma', 'Folate Receptor 3 (Gamma)', 'Folate Receptor 3', 'FR-Gamma', 'FR-G', 'Gamma-HFR', 'FRgamma', 'FOLR3', 'FRγ'], ['Fos Proto-Oncogene, AP-1 Transcription Factor Subunit', 'FBJ Murine Osteosarcoma Viral Oncogene Homolog', 'G0/G1 Switch Regulatory Protein 7', 'Proto-Oncogene C-Fos', 'AP-1', 'FBJ Murine Osteosarcoma Viral (V-Fos) Oncogene Homolog (Oncogene FOS)', 'V-Fos FBJ Murine Osteosarcoma Viral Oncogene Homolog', 'Fos Proto-Oncogene, AP-1 Trancription Factor Subunit', 'Cellular Oncogene C-Fos', 'Cellular Oncogene Fos', 'Activator Protein 1', 'C-FOS', 'C-Fos', 'G0S7', 'P55', 'FOS'], ['FosB Proto-Oncogene, AP-1 Transcription Factor Subunit', 'G0S3', 'FBJ Murine Osteosarcoma Viral Oncogene Homolog B', 'G0/G1 Switch Regulatory Protein 3', 'Activator Protein 1', 'Protein FosB', 'AP-1', 'GOS3', 'GOSB', 'FosB Proto-Oncogene, AP-1 Trancription Factor Subunit', 'Oncogene FOS-B', 'DKFZp686C0818', 'MGC42291', 'FOSB'], ['FOS Like 1, AP-1 Transcription Factor Subunit', 'Fos-Related Antigen 1', 'Fra-1', 'FRA1', 'FOS Like 1, AP-1 Trancription Factor Subunit', 'FOS Like Antigen 1', 'FOS-Like Antigen-1', 'FOSL1', 'FRA-1', 'FRA'], ['FOS Like 2, AP-1 Transcription Factor Subunit', 'FRA2', 'Fos-Related Antigen 2', 'FOS Like Antigen 2', 'FRA-2', 'FOS Like 2, AP-1 Trancription Factor Subunit', 'FLJ23306', 'FOSL2'], ['Forkhead Box A1', 'Hepatocyte Nuclear Factor 3-Alpha', 'Forkhead Box Protein A1', 'Transcription Factor 3A', 'HNF-3-Alpha', 'HNF-3A', 'TCF-3A', 'HNF3A', 'TCF3A', 'Hepatocyte Nuclear Factor 3, Alpha', 'FOXA1'], ['Forkhead Box A2', 'Hepatocyte Nuclear Factor 3-Beta', 'Forkhead Box Protein A2', 'Transcription Factor 3B', 'HNF-3-Beta', 'HNF3B', 'TCF3B', 'Hepatocyte Nuclear Factor 3, Beta', 'Hepatic Nuclear Factor-3-Beta', 'HNF-3B', 'TCF-3B', 'FOXA2'], ['Forkhead Box A3', 'Hepatocyte Nuclear Factor 3-Gamma', 'Fork Head-Related Protein FKH H3', 'Forkhead Box Protein A3', 'Transcription Factor 3G', 'HNF-3-Gamma', 'HNF-3G', 'TCF-3G', 'HNF3G', 'TCF3G', 'Hepatocyte Nuclear Factor 3, Gamma', 'FKHH3', 'FOXA3'], ['Forkhead Box B1', 'FKH5', 'Transcription Factor FKH-5', 'Forkhead Box Protein B1', 'HFKH-5', 'FOXB1'], ['Forkhead Box B2', 'Forkhead Box Protein B2', 'BA159H20.4', 'FOXB2'], ['Forkhead Box C1', 'FREAC3', 'Forkhead-Related Transcription Factor 3', 'Forkhead-Related Protein FKHL7', 'Forkhead Box Protein C1', 'FREAC-3', 'FKHL7', 'IGDA', 'IHG1', 'ARA', 'Forkhead/Winged Helix-Like Transcription Factor 7', 'Forkhead, Drosophila, Homolog-Like 7', 'Forkhead-Related Activator 3', 'Forkhead Box C1 Protein', 'Myeloid Factor-Delta', 'ASGD3', 'IRID1', 'RIEG3', 'FOXC1'], ['Forkhead Box C2', 'Forkhead Box C2 (MFH-1, Mesenchyme Forkhead 1)', 'Forkhead-Related Protein FKHL14', 'Mesenchyme Fork Head Protein 1', 'Transcription Factor FKH-14', 'Forkhead Box Protein C2', 'Mesenchyme Forkhead 1', 'FKHL14', 'MFH-1', 'MFH1', 'Forkhead, Drosophila, Homolog-Like 14', 'MFH-1,Mesenchyme Forkhead 1', 'MFH-1 Protein', 'FOXC2', 'LD'], ['Forkhead Box D1', 'FREAC4', 'Forkhead-Related Transcription Factor 4', 'Forkhead-Related Protein FKHL8', 'Forkhead Box Protein D1', 'FREAC-4', 'FKHL8', 'Forkhead, Drosophila, Homolog-Like 8', 'Forkhead-Related Activator 4', 'Forkhead-Like 8', 'FOXD1'], ['Forkhead Box D2', 'FREAC9', 'Forkhead-Related Transcription Factor 9', 'Forkhead-Related Protein FKHL17', 'Forkhead Box Protein D2', 'FREAC-9', 'FKHL17', 'Forkhead, Drosophila, Homolog-Like 17', 'Forkhead-Related Activator 9', 'Forkhead-Like 17', 'FOXD2'], ['Forkhead Box D3', 'HFH2', 'HNF3/FH Transcription Factor Genesis', 'Forkhead Box Protein D3', 'Genesis', 'VAMAS2', 'FOXD3', 'AIS1'], ['Forkhead Box D4', 'FREAC5', 'Forkhead-Related Transcription Factor 5', 'Forkhead-Related Protein FKHL9', 'Forkhead Box Protein D4', 'Myeloid Factor-Alpha', 'FREAC-5', 'FOXD4A', 'FKHL9', 'Forkhead, Drosophila, Homolog-Like 9', 'Forkhead-Related Activator 5', 'FOXD4a', 'FOXD4'], ['Forkhead Box D4 Like 1', 'Forkhead Box Protein D4-Like 1', 'FOXD4-Like 1', 'FOXD5', 'BA395L14.1', 'FOXD4L1'], ['Forkhead Box D4 Like 3', 'Forkhead Box Protein D4-Like 3', 'FOXD4-Like 3', 'FOXD6', 'Forkhead Box Protein D4-Like 4', 'Forkhead Box Protein D4-Like 2', 'Forkhead Box Protein D4B', 'Myeloid Factor-Gamma', 'OTTHUMG00000019959', 'FOXD4-Like 2', 'FOXD4-Like 4', 'FOXD4L2', 'FOXD4L4', 'FOXD4L3', 'FOXD4B'], ['Forkhead Box D4 Like 4', 'Forkhead Box Protein D4-Like 4', 'Forkhead Box Protein D4-Like 2', 'Forkhead Box D4-Like 2', 'Myeloid Factor-Gamma', 'FOXD4-Like 4', 'BA460E7.2', 'FOXD4L2', 'Winged Helix Transcription Factor Beta', 'Forkhead Box Protein D4b', 'Forkhead Box Protein D4B', 'Winged Helix Factor-1', 'OTTHUMG00000013337', 'OTTHUMG00000066752', 'FOXD4L4', 'FOXD4b', 'FOXD4B'], ['Forkhead Box D4 Like 5', 'Forkhead Box Protein D4-Like 5', 'FOXD4-Like 5', 'BA15J10.2', 'OTTHUMG00000013332', 'FOXD4L5'], ['Forkhead Box D4 Like 6', 'Forkhead Box Protein D4-Like 6', 'FOXD4-Like 6', 'OTTHUMG00000066822', 'FOXD4L6'], ['Forkhead Box E1', 'Thyroid Transcription Factor 2', 'HFKH4', 'TTF-2', 'Forkhead-Related Protein FKHL15', 'HNF-3/Fork Head-Like Protein 5', 'Forkhead Box Protein E1', 'Forkhead Box Protein E2', 'FKHL15', 'FOXE2', 'HFKL5', 'TITF2', 'TTF2', 'Forkhead Box E1 (Thyroid Transcription Factor 2)', 'Forkhead, Drosophila, Homolog-Like 15', 'Forkhead Box E2', 'NMTC4', 'FOXE1'], ['Forkhead Box E3', 'FREAC8', 'Forkhead-Related Transcription Factor 8', 'Forkhead-Related Protein FKHL12', 'Forkhead Box Protein E3', 'FREAC-8', 'FKHL12', 'Forkhead, Drosophila, Homolog-Like 12', 'CTRCT34', 'AAT11', 'ASGD2', 'FOXE3'], ['Forkhead Box F1', 'FREAC1', 'Forkhead-Related Transcription Factor 1', 'Forkhead-Related Protein FKHL5', 'Forkhead-Related Activator 1', 'Forkhead Box Protein F1', 'FREAC-1', 'FKHL5', 'Forkhead, Drosophila, Homolog-Like 5', 'ACDMPV', 'FOXF1'], ['Forkhead Box F2', 'FREAC2', 'Forkhead-Related Transcription Factor 2', 'Forkhead-Related Protein FKHL6', 'Forkhead-Related Activator 2', 'Forkhead Box Protein F2', 'FREAC-2', 'FKHL6', 'Forkhead-Like 6', 'FOXF2'], ['Forkhead Box G1', 'Brain Factor 1', 'HFK1', 'HFK2', 'HFK3', 'BF1', 'Forkhead Box Protein G1', 'Brain Factor 2', 'FOXG1A', 'FOXG1B', 'FOXG1C', 'FKHL1', 'FKHL2', 'FKHL3', 'FKHL4', 'HBF-2', 'HBF-3', 'FKH2', 'BF2', 'QIN', 'Forkhead-Related Protein FKHL1', 'Forkhead-Related Protein FKHL2', 'Forkhead-Related Protein FKHL3', 'Forkhead Box Protein G1A', 'Forkhead Box Protein G1B', 'Forkhead Box Protein G1C', 'Forkhead Box G1B', 'Forkhead Box G1C', 'Forkhead Box G1A', 'Forkhead-Like 1', 'Forkhead-Like 2', 'Forkhead-Like 3', 'Forkhead-Like 4', 'Oncogene QIN', 'HBF-G2', 'FHKL3', 'HBF-1', 'FOXG1', 'HBF2', 'KHL2', 'BF-1', 'BF-2'], ['Forkhead Box H1', 'FAST1', 'Forkhead Activin Signal Transducer 2', 'Forkhead Box Protein H1', 'HFAST-1', 'Fast-2', 'Forkhead Activin Signal Transducer-1', 'Forkhead Activin Signal Transducer 1', 'TGF-Beta/Activin Signal Transducer', 'FAST-1', 'Fast-1', 'FOXH1', 'FAST2'], ['Forkhead Box I1', 'FREAC6', 'Hepatocyte Nuclear Factor 3 Forkhead Homolog 3', 'Forkhead-Related Transcription Factor 6', 'Forkhead-Related Protein FKHL10', 'Forkhead Box Protein I1', 'FREAC-6', 'FKHL10', 'HFH-3', 'Forkhead-Related Activator 6', 'HNF-3/Fork-Head Homolog-3', 'HNF-3/Fork-Head Homolog 3', 'Forkhead-Like 10', 'FKH10', 'FOXI1', 'HFH3'], ['Forkhead Box I2', 'Forkhead Box Protein I2', 'Homolog Of Mouse Foxi2', 'FLJ46831', 'FOXI2'], ['Forkhead Box I3', 'Forkhead Box Protein I3', 'FOXI3'], ['Forkhead Box J1', 'HFH-4', 'HFH4', 'Hepatocyte Nuclear Factor 3 Forkhead Homolog 4', 'Forkhead-Related Protein FKHL13', 'Forkhead Box Protein J1', 'FKHL13', 'Forkhead Transcription Factor HFH-4', 'Fork Head Homologue 4', 'Forkhead-Like 13', 'CILD43', 'FOXJ1'], ['Forkhead Box J2', 'FHX', 'Forkhead Box Protein J2', 'Fork Head Homologous X', 'FOXJ2 Forkhead Factor', 'FOXJ2'], ['Forkhead Box J3', 'Forkhead Box Protein J3', 'KIAA1041', 'FOXJ3'], ['Forkhead Box K1', 'Forkhead Box Protein K1', 'Myocyte Nuclear Factor', 'MNF', 'IMAGE:5164497', 'FOXK1L', 'FOXK1'], ['Forkhead Box K2', 'G/T-Mismatch Specific Binding Protein', 'Interleukin Enhancer-Binding Factor 1', 'Forkhead Box Protein K2', 'NGTBP', 'ILF1', 'ILF', 'Interleukin Enhancer Binding Factor 1', 'Cellular Transcription Factor ILF-1', 'ILF-1', 'FOXK1', 'FOXK2'], ['Forkhead Box L1', 'FREAC7', 'Forkhead-Related Transcription Factor 7', 'Forkhead-Related Protein FKHL11', 'Forkhead Box Protein L1', 'FKHL11', 'FKH6', 'Forkhead-Like 11', 'FREAC-7', 'FOXL1'], ['Forkhead Box L2', 'Forkhead Box Protein L2', 'BPES1', 'Forkhead Transcription Factor FOXL2', 'PINTO', 'FOXL2', 'BPES', 'PFRK', 'POF3'], ['FOXL2 Neighbor', 'FOXL2 Neighbor Protein', 'C3orf72', 'Chromosome 3 Open Reading Frame 72', 'FLJ43329 Protein', 'FLJ43329', 'FOXL2NB'], ['Forkhead Box L3', 'Forkhead Box Protein L3', 'Forkhead Box L1-Like', 'FOXL3'], ['Forkhead Box M1', 'M-Phase Phosphoprotein 2', 'HFH-11', 'MPP2', 'Hepatocyte Nuclear Factor 3 Forkhead Homolog 11', 'Winged-Helix Factor From INS-1 Cells', 'MPM-2 Reactive Phosphoprotein 2', 'Forkhead-Related Protein FKHL16', 'Transcription Factor Trident', 'HNF-3/Fork-Head Homolog 11', 'Forkhead Box Protein M1', 'MPHOSPH2', 'FKHL16', 'HFH11', 'HNF-3', 'INS-1', 'Forkhead, Drosophila, Homolog-Like 16', 'Forkhead Box M1-D', 'TRIDENT', 'Trident', 'FOXM1A', 'FOXM1B', 'FOXM1C', 'MPP-2', 'PIG29', 'FOXM1', 'TGT3', 'WIN'], ['Forkhead Box N1', 'Winged-Helix Transcription Factor Nude', 'Forkhead Box Protein N1', 'Winged-Helix Nude', 'Rowett Nude', 'FKHL20', 'RONU', 'WHN', 'TIDAND', 'TLIND', 'FOXN1'], ['Forkhead Box N2', 'Human T-Cell Leukemia Virus Enhancer Factor', 'Forkhead Box Protein N2', 'HTLF', 'FOXN2'], ['Forkhead Box N3', 'Checkpoint Suppressor 1', 'Forkhead Box Protein N3', 'C14orf116', 'CHES1', 'Chromosome 14 Open Reading Frame 116', 'PRO1635', 'FOXN3'], ['Forkhead Box N4', 'Forkhead Box Protein N4', 'Forkhead/Winged Helix Transcription Factor FOXN4', 'FOXN4'], ['Forkhead Box O1', 'Forkhead Box Protein O1A', 'Forkhead Box Protein O1', 'FOXO1A', 'FKH1', 'FKHR', 'Forkhead, Drosophila, Homolog Of, In Rhabdomyosarcoma', 'Forkhead Homolog In Rhabdomyosarcoma', 'Forkhead In Rhabdomyosarcoma', 'FOXO1'], ['Forkhead Box O3', 'Forkhead In Rhabdomyosarcoma-Like 1', 'Forkhead Box Protein O3', 'AF6q21', 'FKHRL1', 'FOXO3A', 'FOXO2', 'Forkhead, Drosophila, Homolog Of, In Rhabdomyosarcoma-Like 1', 'Forkhead Homolog (Rhabdomyosarcoma) Like 1', 'Forkhead Box O3A', 'AF6q21 Protein', 'FKHRL1P2', 'FOXO3A-', 'FOXO3'], ['Forkhead Box O3B', 'Forkhead Box Protein O3B', 'Forkhead (Drosophila) Homolog (Rhabdomyosarcoma) Like 1 Pseudogene 1', 'Forkhead(Drosophila) Homolog (Rhabdomyosarcoma) 1 Like Pseudogene 1', 'Forkhead Box O3B (Pseudogene)', 'Forkhead Box O3B Pseudogene', 'FKHRL1P1', 'ZNF286C', 'FOXO3B'], ['Forkhead Box O4', 'AFX1', 'Myeloid/Lymphoid Or Mixed-Lineage Leukemia (Trithorax Homolog, Drosophila); Translocated To, 7', 'Fork Head Domain Transcription Factor AFX1', 'Forkhead Box Protein O4', 'MLLT7', 'AFX', 'Myeloid/Lymphoid Or Mixed-Lineage Leukemia (Trithorax (Drosophila) Homolog); Translocated To, 7', 'FOXO4'], ['Forkhead Box O6', 'Forkhead Box Protein O6', 'FOXO6'], ['Forkhead Box P1', 'Fork Head-Related Protein Like B', 'Mac-1-Regulated Forkhead', 'Glutamine-Rich Factor 1', 'Forkhead Box Protein P1', 'HSPC215', 'HFKH1B', '12CC4', 'QRF1', 'MFH', 'PAX5/FOXP1 Fusion Protein', 'FOXP1'], ['Forkhead Box P2', 'CAG Repeat Protein 44', 'CAGH44', 'Trinucleotide Repeat-Containing Gene 10 Protein', 'Forkhead/Winged-Helix Transcription Factor', 'Trinucleotide Repeat Containing 10', 'Forkhead Box Protein P2', 'TNRC10', 'Speech And Language Disorder 1', 'SPCH1', 'FOXP2'], ['Forkhead Box P3', 'Immune Dysregulation, Polyendocrinopathy, Enteropathy, X-Linked', 'Forkhead Box Protein P3', 'DIETER', 'AIID', 'IPEX', 'PIDX', 'XPID', 'JM2', 'Immunodeficiency, Polyendocrinopathy, Enteropathy, X-Linked', 'FOXP3delta7', 'SCURFIN', 'Scurfin', 'FOXP3'], ['Forkhead Box P4', 'Forkhead Box Protein P4', 'Fork Head-Related Protein Like A', 'Fork Head-Related Protein-Like A', 'Winged-Helix Repressor FOXP4', 'FLJ40908', 'HFKHLA', 'FOXP4', 'FKHLA'], ['Forkhead Box Q1', 'HFH1', 'Hepatocyte Nuclear Factor 3 Forkhead Homolog 1', 'HNF-3/Forkhead-Like Protein 1', 'Forkhead Box Protein Q1', 'HFH-1', 'Winged Helix/Forkhead Transcription Factor', 'FOXQ1'], ['Forkhead Box R1', 'FOXN5', 'Forkhead Box Protein R1', 'Forkhead Box Protein N5', 'DLNB13', 'Forkhead Box R1 Variant 1', 'Forkhead Box R1 Variant 2', 'FOXR1'], ['Forkhead Box R2', 'FOXN6', 'Forkhead Box Protein R2', 'Forkhead Box Protein N6', 'MGC21658', 'FOXR2'], ['FAD Dependent Oxidoreductase Domain Containing 1', 'FAD-Dependent Oxidoreductase Domain-Containing Protein 1', 'H17', 'EC 1.-.-.-', 'MC1DN19', 'FOXRED1', 'FP634'], ['FAD Dependent Oxidoreductase Domain Containing 2', 'ERFAD', 'Endoplasmic Reticulum Flavoprotein Associated With Degradation', 'FAD-Dependent Oxidoreductase Domain-Containing Protein 2', 'ER Flavoprotein Associated With Degradation', 'FAD-Dependent Oxidoreductase Domain Containing 2', 'FLJ23322', 'FOXRED2'], ['Forkhead Box S1', 'FREAC10', 'Forkhead-Related Transcription Factor 10', 'Forkhead-Like 18 Protein', 'Forkhead Box Protein S1', 'FREAC-10', 'FKHL18', 'Forkhead-Related Transcription Factor FREAC-10', 'Forkhead (Drosophila)-Like 18', 'Forkhead-Like 18 (Drosophila)', 'Forkhead-Related Activator 10', 'FOXS1'], ['Folylpolyglutamate Synthase', 'Folylpolyglutamate Synthase, Mitochondrial', 'Tetrahydrofolylpolyglutamate Synthase', 'Folylpoly-Gamma-Glutamate Synthetase', 'Tetrahydrofolate Synthase', 'EC 6.3.2.17', 'FPGS'], ['Fucose-1-Phosphate Guanylyltransferase', 'GFPP', 'GDP-L-Fucose Diphosphorylase', 'Fucose-1-Phosphate Guanyltransferase', 'GDP-Beta-L-Fucose Pyrophosphorylase', 'GDP-L-Fucose Pyrophosphorylase', 'EC 2.7.7.30', 'FPGT'], ['Formyl Peptide Receptor 1', 'FPR', 'N-Formylpeptide Chemoattractant Receptor', 'FMet-Leu-Phe Receptor', 'FMLP Receptor', 'FMLP', 'N-Formyl Peptide Receptor', 'FPR1'], ['Formyl Peptide Receptor 2', 'Formyl Peptide Receptor-Like 1', 'LXA4R', 'HM63', 'N-Formyl Peptide Receptor 2', 'FMLP-Related Receptor I', 'LXA4 Receptor', 'FMLP-R-II', 'FMLP-R-I', 'FMLPX', 'FPR2A', 'FPRH1', 'FPRH2', 'FPRL1', 'ALXR', 'ALX', 'RFP', 'Lipoxin A4 Receptor (Formyl Peptide Receptor Related)', 'Lipoxin A4 Receptor', 'FPR2'], ['Formyl Peptide Receptor 3', 'Formyl Peptide Receptor-Like 2', 'FPRH1', 'N-Formyl Peptide Receptor 3', 'FMLP-Related Receptor II', 'FMLP-R-II', 'RMLP-R-I', 'FMLPY', 'FPRL2', 'FML2_HUMAN', 'FPRH2', 'FPR3'], ['FRA10A Associated CGG Repeat 1', 'Fragile Site, Folic Acid Type, Rare, Fra(10)(Q23.3) Or Fra(10)(Q24.2) Candidate 1', 'Protein FRA10AC1', 'C10orf4', 'Rare Folic Acid-Type Fragile Site, FRA(10)(Q23.3), Candidate Gene 1', 'Chromosome 10 Open Reading Frame 4', 'Fragile Site 10q23.3', 'F26C11.1-Like', 'FRA10AC1', 'FRA10A'], ['Fraser Extracellular Matrix Complex Subunit 1', 'Extracellular Matrix Organizing Protein FRAS1', 'Fraser Syndrome 1 Protein', 'Fraser Syndrome 1', 'KIAA1500', 'Extracellular Matrix Protein FRAS1', 'FLJ22031', 'FLJ14927', 'FRASRS1', 'FRAS1'], ['FRAT Regulator Of WNT Signaling Pathway 1', 'Frequently Rearranged In Advanced T-Cell Lymphomas 1', 'FRAT1, WNT Signaling Pathway Regulator', 'Proto-Oncogene FRAT1', 'FRAT-1', 'Frequently Rearranged In Advanced T-Cell Lymphomas', 'FRAT1'], ['FRAT Regulator Of WNT Signaling Pathway 2', 'Frequently Rearranged In Advanced T-Cell Lymphomas 2', 'FRAT2, WNT Signaling Pathway Regulator', 'GSK-3-Binding Protein FRAT2', 'FRAT-2', 'GSK-3 Binding Protein FRAT2', 'FRAT2'], ['FRAS1 Related Extracellular Matrix 1', 'C9orf143', 'C9orf145', 'FRAS1-Related Extracellular Matrix Protein 1', 'C9orf154', 'TILRR', 'Chromosome 9 Open Reading Frame 154', 'Extracellular Matrix Protein QBRICK', 'DKFZp686M16108', 'Protein QBRICK', 'FLJ25461', 'TRIGNO2', 'FREM1', 'BNAR', 'MOTA'], ['FRAS1 Related Extracellular Matrix 2', 'FRAS1-Related Extracellular Matrix Protein 2', 'ECM3 Homolog', 'FRAS1 Related Extracellular Matrix Protein 2', 'DKFZp686J0811', 'CRYPTOP', 'FRASRS2', 'FREM2'], ['FRAS1 Related Extracellular Matrix 3', 'FRAS1-Related Extracellular Matrix Protein 3', 'FREM3'], ['FSHD Region Gene 1', 'FSHD Region Gene 1 Protein', 'Protein FRG1', 'FRG1A', 'FSG1', 'Facioscapulohumeral Muscular Dystrophy Region Gene-1', 'FRG1'], ['FSHD Region Gene 2', 'FRG2A', 'FSHD Region Gene 2 Protein', 'Protein FRG2', 'FRG2'], ['FSHD Region Gene 2 Family Member B', 'FSHD Region Gene 2 Protein Family Member B', 'Protein FRG2-Like-1', 'HSA10-FRG2', 'FSHD Region Gene 2 Family, Member B', 'FRG2B'], ['FSHD Region Gene 2 Family Member C', 'FSHD Region Gene 2 Protein Family Member C', 'Protein FRG2-Like-2', 'HSA3-FRG2', 'FSHD Region Gene 2 Family, Member C-Like', 'FSHD Region Gene 2 Family, Member C', 'FRG2C'], ['Fyn Related Src Family Tyrosine Kinase', 'RAK', 'Nuclear Tyrosine Protein Kinase RAK', 'PTK5 Protein Tyrosine Kinase 5', 'Tyrosine-Protein Kinase FRK', 'Protein-Tyrosine Kinase 5', 'Fyn-Related Kinase', 'EC 2.7.10.2', 'PTK5', 'GTK', 'Fyn-Related Src Family Tyrosine Kinase', 'FYN-Related Kinase', 'EC 2.7.10', 'FRK'], ['FERM Domain Containing 1', 'FERM Domain-Containing Protein 1', 'BA164L23.1', 'DKFZp434O0117', 'FLJ00181', 'FLJ40260', 'FLJ22615', 'FRMD1'], ['FERM Domain Containing 3', 'EPB41L4O', 'FERM Domain-Containing Protein 3', 'Band 4.1-Like Protein 4O', 'Ovary Type Protein 4.1', '4.1O', 'Band 4.1-Like Protein 4', 'Protein 4.1O', 'MGC20553', 'EPB41LO', 'FRMD3', 'P410'], ['FERM Domain Containing 4A', 'FERM Domain-Containing Protein 4A', 'FERM Domain Containing 4', 'BA295P9.4', 'KIAA1294', 'FRMD4', 'FLJ10210', 'CCAFCA', 'FRMD4A'], ['FERM Domain Containing 4B', 'GRSP1', 'FERM Domain-Containing Protein 4B', 'GRP1-Binding Protein GRSP1', 'KIAA1013', '6030440G05Rik', 'FRMD4B'], ['FERM Domain Containing 5', 'FERM Domain-Containing Protein 5', 'MGC14161', 'FRMD5'], ['FERM Domain Containing 6', 'Willin', 'FERM Domain-Containing Protein 6', 'Expanded Homolog', 'C14orf31', 'EX1', '4.1 Ezrin Radixin Moesin (FERM)-Containing Protein', 'Chromosome 14 Open Reading Frame 31', 'C14_5320', 'MGC17921', 'FRMD6'], ['FERM Domain Containing 7', 'FERM Domain-Containing Protein 7', 'Nystagmus 1, Congenital', 'FLJ43346', 'XIPAN', 'FRMD7', 'NYS1', 'NYS'], ['FERM Domain Containing 8', 'IRhom Tail-Associated Protein', 'ITAP', 'FERM Domain-Containing Protein 8', 'Band4.1 Inhibitor LRP Interactor', 'FKSG44', 'Bili', 'FLJ90369', 'FRMD8'], ['FERM And PDZ Domain Containing 1', 'FRMD2', 'FERM And PDZ Domain-Containing Protein 1', 'FERM Domain-Containing Protein 2', 'KIAA0967', 'FRMPD1'], ['FERM And PDZ Domain Containing 2', 'FERM And PDZ Domain-Containing Protein 2', 'PDZ Domain-Containing Protein 4', 'PDZ Domain Containing 5C', 'PDZD5C', 'PDZK5C', 'PDZK4', 'PDZ Domain-Containing Protein 5C', 'MGC35285', 'FRMPD2'], ['FERM And PDZ Domain Containing 3', 'FERM And PDZ Domain-Containing Protein 3', 'KIAA1817', 'RP5-1070B1.1', 'EC 6.1.1', 'EC 3.6.1', 'FRMPD3'], ['FERM And PDZ Domain Containing 4', 'PSD-95-Interacting Regulator Of Spine Morphogenesis', 'FERM And PDZ Domain-Containing Protein 4', 'PDZ Domain-Containing Protein 10', 'KIAA0316', 'PDZD10', 'PDZK10', 'Preso', 'PSD-95-Interacting FERM And PDZ Domain Protein', 'PDZ Domain Containing 10', 'MRX104', 'FRMPD4'], ['Ferric Chelate Reductase 1', 'Ferric-Chelate Reductase 1', 'SDR2', 'Stromal Cell-Derived Receptor 2', 'SDFR2', 'SDR-2', 'Stromal Cell Derived Factor Receptor 2 Homolog (Mouse)', 'Stromal Cell Derived Factor Receptor 2 Homolog', 'EC 1.-.-.-', 'FRRS1'], ['Ferric Chelate Reductase 1 Like', 'DOMON Domain-Containing Protein FRRS1L', 'Brain Protein CG-6', 'C9orf4', 'CG-6', 'Ferric-Chelate Reductase 1-Like Protein', 'Chromosome 9 Open Reading Frame 4', 'Ferric-Chelate Reductase 1-Like', 'EIEE37', 'FRRS1L', 'DEE37', 'CG6'], ['Fibroblast Growth Factor Receptor Substrate 2', 'SNT-1', 'Suc1-Associated Neurotrophic Factor Target 1', 'FGFR-Signaling Adaptor SNT', 'FGFR Substrate 2', 'FRS2alpha', 'FRS2A', 'SNT1', 'Epididymis Secretory Sperm Binding Protein', 'FGFR Signalling Adaptor', 'FRS1A', 'FRS2', 'SNT'], ['Fibroblast Growth Factor Receptor Substrate 3', 'SNT-2', 'FGFR-Signaling Adaptor SNT2', 'FGFR Substrate 3', 'FRS2beta', 'FRS2B', 'Suc1-Associated Neurotrophic Factor Target 2 (FGFR Signalling Adaptor)', 'Suc1-Associated Neurotrophic Factor Target 2', 'Testicular Tissue Protein Li 71', 'FRS2-Beta', 'SNT2', 'FRS3'], ['FRY Microtubule Binding Protein', 'Protein Furry Homolog', 'BA37E23.1', '13CDNA73', 'C13orf14', 'CG003', 'Chromosome 13 Open Reading Frame 14', 'Furry Homolog (Drosophila)', 'WUGSC:H_2G3A.1', 'Furry Homolog', 'BA207N4.2', '214K23.2', 'FRY'], ['FRY Like Transcription Coactivator', 'KIAA0826', 'ALL1-Fused Gene From Chromosome 4p12 Protein', 'Protein Furry Homolog-Like', 'AF4p12', 'MOR2', 'Mor2 Cell Polarity Protein Homolog (S. Pombe)', 'Mor2 Cell Polarity Protein Homolog', 'Furry Homolog-Like (Drosophila)', 'Furry Homolog-Like', 'DKFZp686E205', 'Furry-Like', 'FRY Like', 'FRY-Like', 'AF4P12', 'FRYL'], ['Frizzled Related Protein', 'Secreted Frizzled-Related Protein 3', 'FRZB1', 'SFRP3', 'FRE', 'Frizzled-Related Protein 1', 'FRZB-PEN', 'Frezzled', 'FRZB-1', 'SFRP-3', 'FRITZ', 'FRP-3', 'SRFP3', 'FZRB', 'HFIZ', 'Frizzled Homolog-Related', 'FrzB-1', 'Fritz', 'FRZB', 'OS1', 'FIZ', 'FRP'], ['Fibrinogen Silencer Binding Protein', 'Fibrinogen Silencer-Binding Protein', 'FSBP'], ['Fibrous Sheath CABYR Binding Protein', 'Fibrous Sheath CABYR-Binding Protein', 'C14orf155', 'Chromosome 14 Open Reading Frame 155', 'DKFZP434F1017', 'FSCB'], ['Fascin Actin-Bundling Protein 1', 'P55', '55 KDa Actin-Bundling Protein', 'Fascin', 'FAN1', 'HSN', 'SNL', 'Fascin Homolog 1, Actin-Bundling Protein (Strongylocentrotus Purpuratus)', 'Singed (Drosophila)-Like (Sea Urchin Fascin Homolog Like)', 'Epididymis Secretory Sperm Binding Protein', 'Fascin Homolog 1, Actin-Bundling Protein', 'Singed-Like (Fascin Homolog, Sea Urchin)', 'Singed, Drosophila, Homolog-Like', 'Actin Bundling Protein', 'Singed-Like Protein', 'FLJ38511', 'FSCN1'], ['Fascin Actin-Bundling Protein 2, Retinal', 'Retinal Fascin', 'Fascin-2', 'RFSN', 'RP30', 'Fascin (Strongylocentrotus Purpuratus) Homolog 2 (Actin-Bundling Protein, Retinal)', 'Fascin Homolog 2, Actin-Bundling Protein, Retinal (Strongylocentrotus Purpuratus)', 'Fascin Homolog 2, Actin-Bundling Protein, Retinal', 'FSCN2'], ['Fascin Actin-Bundling Protein 3', 'Fascin Actin-Bundling Protein 3, Testicular', 'Testis Fascin', 'Fascin-3', 'Fascin (Strongylocentrotus Purpuratus) Homolog 3 (Actin-Bundling Protein, Testicular)', 'Fascin Homolog 3, Actin-Bundling Protein, Testicular (Strongylocentrotus Purpuratus)', 'Fascin Homolog 3, Actin-Bundling Protein, Testicular', 'Testicular Secretory Protein Li 18', 'FSCN3'], ['Fibronectin Type III And SPRY Domain Containing 1', 'MIR1', 'Fibronectin Type III And SPRY Domain-Containing Protein 1', 'Fibronectin Type 3 And SPRY Domain Containing 1', 'Microtubule-Associated Protein GLFND', 'MID1-Related Protein 1', 'GLFND', 'Fibronectin Type 3 And SPRY (Spla, Ryanodine) Domain Containing (With Coiled-Coil Motif) 1', 'Midline 1-Related Protein 1', 'MGC3213', 'FSD1'], ['Fibronectin Type III And SPRY Domain Containing 1 Like', 'Cystatin And DUF19 Domain Containing 1', 'Coiled-Coil Domain Containing 10', 'FSD1-Like Protein', 'CSDUFD1', 'CCDC10', 'FSD1CL', 'FSD1NL', 'Fibronectin Type III And SPRY Domain Containing 1-Like', 'Coiled-Coil Domain-Containing Protein 10', 'FSD1 N-Terminal-Like Protein', 'FSD1 C-Terminal Like', 'FSD1 N-Terminal Like', 'FSD1L', 'MIR1'], ['Fibronectin Type III And SPRY Domain Containing 2', 'Fibronectin Type III And SPRY Domain-Containing Protein 2', 'SPRY Domain-Containing Protein 1', 'SPRY Domain Containing 1', 'Minispryn', 'SPRYD1', 'RP11-127F21', 'FSD2'], ['Follicle Stimulating Hormone Subunit Beta', 'Follicle Stimulating Hormone, Beta Polypeptide', 'Follitropin Subunit Beta', 'Follitropin, Beta Chain', 'FSH-Beta', 'FSH-B', 'Follicle Stimulating Hormone Beta Subunit', 'Follicle-Stimulating Hormone Beta Subunit', 'Follitropin Beta Chain', 'HH24', 'FSHB'], ['Follicle Stimulating Hormone Receptor', 'LGR1', 'Follicle-Stimulating Hormone Receptor', 'Follitropin Receptor', 'FSHRO', 'FSH Receptor', 'FSHR1', 'FSH-R', 'ODG1', 'FSHR'], ['Fibrous Sheath Interacting Protein 1', 'Fibrous Sheath-Interacting Protein 1', 'FLJ35989', 'HSD10', 'FSIP1'], ['Fibrous Sheath Interacting Protein 2', 'Fibrous Sheath-Interacting Protein 2', 'FLJ34780', 'SPGF34', 'FSIP2'], ['Follistatin', 'FS', 'Activin-Binding Protein', 'FST'], ['Follistatin Like 1', 'FRP', 'Follistatin-Related Protein 1', 'Follistatin-Like Protein 1', 'OCC-1', 'Tsc36', 'FSL1', 'OCC1', 'Follistatin-Like 1', 'MIR198', 'FSTL1'], ['Follistatin Like 3', 'FLRG', 'Follistatin-Like 3 (Secreted Glycoprotein)', 'Follistatin-Related Gene Protein', 'Follistatin-Related Protein 3', 'Follistatin-Like Protein 3', 'FSRP', 'Follistatin-Related Protein', 'FSTL3'], ['Follistatin Like 4', 'Follistatin-Related Protein 4', 'Follistatin-Like Protein 4', 'KIAA1061', 'Follistatin-Like 4', 'FSTL4'], ['Follistatin Like 5', 'Follistatin-Related Protein 5', 'Follistatin-Like Protein 5', 'KIAA1263', 'Follistatin-Like 5', 'DKFZp566D234', 'FSTL5'], ['Formimidoyltransferase Cyclodeaminase', 'Formimidoyltransferase-Cyclodeaminase', 'Formiminotransferase-Cyclodeaminase', 'LCHC1', 'Formiminotransferase Cyclodeaminase', 'FTCD'], ['Formiminotransferase Cyclodeaminase N-Terminal Like', 'FONG', 'Formiminotransferase N-Terminal Subdomain-Containing Protein', 'Formiminotransferase-Cyclodeaminase N-Terminal-Like Protein', 'Formiminotransferase N-Terminal Sub-Domain Containing Gene', 'Formiminotransferase N-Terminal Sub-Domain Containing', 'FTCDNL1'], ['Ferritin Heavy Chain 1', 'FTH', 'Cell Proliferation-Inducing Gene 15 Protein', 'Proliferation-Inducing Protein 15', 'Placenta Immunoregulatory Factor', 'Ferritin, Heavy Polypeptide 1', 'Ferritin Heavy Chain', 'Ferritin H Subunit', 'Apoferritin', 'EC 1.16.3.1', 'FTHL6', 'PIG15', 'PLIF', 'FHC', 'HFE5', 'FTH1'], ['Ferritin Heavy Chain Like 17', 'Cancer/Testis Antigen 38', 'CT38', 'Ferritin, Heavy Polypeptide-Like 17', 'Ferritin Heavy Polypeptide-Like 17', 'Epididymis Secretory Sperm Binding Protein', 'FTHL17'], ['Ferritin Light Chain', 'Ferritin L Subunit', 'Ferritin Light Polypeptide-Like 3', 'Ferritin, Light Polypeptide', 'Ferritin L-Chain', 'NBIA3', 'Neurodegeneration With Brain Iron Accumulation 3', 'Epididymis Secretory Sperm Binding Protein', 'L Apoferritin', 'MGC71996', 'LFTD', 'FTL'], ['Ferritin Mitochondrial', 'Ferritin, Mitochondrial', 'EC 1.16.3.1', 'Mitochondrial Ferritin', 'Ferritin H Subunit', 'FTMT', 'MTF', 'MtF'], ['FTO Alpha-Ketoglutarate Dependent Dioxygenase', "U6 Small Nuclear RNA (2'-O-Methyladenosine-N(6)-)-Demethylase FTO", 'U6 Small Nuclear RNA N(6)-Methyladenosine-Demethylase FTO', "MRNA (2'-O-Methyladenosine-N(6)-)-Demethylase FTO", 'Alpha-Ketoglutarate-Dependent Dioxygenase FTO', 'MRNA N(6)-Methyladenosine Demethylase FTO', 'Fat Mass And Obesity-Associated Protein', 'TRNA N1-Methyl Adenine Demethylase FTO', 'Fat Mass And Obesity Associated', 'M6A(M)-Demethylase FTO', 'AlkB Homolog 9', 'KIAA1752', 'ALKBH9', 'FTO, Alpha-Ketoglutarate Dependent Dioxygenase', 'Alpha-Ketoglutarate-Dependent Dioxygenase', 'EC 1.14.11.53', 'EC 1.14.11.-', 'MGC5149', 'BMIQ14', 'GDFD', 'FTO'], ["FtsJ RNA 2'-O-Methyltransferase 1", "Putative TRNA (Cytidine(32)/Guanosine(34)-2'-O)-Methyltransferase", "2'-O-Ribose RNA Methyltransferase TRM7 Homolog", 'FtsJ RNA Methyltransferase Homolog 1', 'CDLIV', 'TRMT7', 'JM23', 'SPB1', 'TRNA Methyltransferase 7 Homolog (S. Cerevisiae)', 'FtsJ RNA Methyltransferase Homolog 1 (E. Coli)', 'Putative Ribosomal RNA Methyltransferase 1', "RRNA (Uridine-2'-O-)-Methyltransferase", 'TRNA Methyltransferase 7 Homolog', 'Mental Retardation, X-Linked 44', 'Mental Retardation, X-Linked 9', 'Protein FtsJ Homolog 1', 'Cell Division Protein', 'FtsJ-Like Protein 1', 'EC 2.1.1.205', 'MRX44', 'FTSJ1', 'MRX9', 'TRM7'], ["FtsJ RNA 2'-O-Methyltransferase 3", "Pre-RRNA 2'-O-Ribose RNA Methyltransferase FTSJ3", 'FtsJ RNA Methyltransferase Homolog 3', 'Putative RRNA Methyltransferase 3', 'Protein FtsJ Homolog 3', 'SPB1', 'SPB1 RNA Methyltransferase Homolog (S. Cerevisiae)', "2'-O-Ribose RNA Methyltransferase SPB1 Homolog", "RRNA (Uridine-2'-O-)-Methyltransferase 3", 'SPB1 RNA Methyltransferase Homolog', 'Pre-RRNA Processing Protein FTSJ3', 'FtsJ Homolog 3 (E. Coli)', 'FtsJ Homolog 3', 'EC 2.1.1.-', 'EPCS3', 'FTSJ3'], ['Far Upstream Element Binding Protein 1', 'FBP', 'Far Upstream Element (FUSE) Binding Protein 1', 'Far Upstream Element-Binding Protein 1', 'FUSE Binding Protein 1', 'DNA Helicase V', 'HDH V', 'Epididymis Secretory Sperm Binding Protein', 'FUSE-Binding Protein 1', 'FUBP1', 'FUBP'], ['Far Upstream Element Binding Protein 3', 'Far Upstream Element (FUSE) Binding Protein 3', 'Far Upstream Element-Binding Protein 3', 'FUSE-Binding Protein 3', 'FBP3', 'FUBP3'], ['Alpha-L-Fucosidase 1', 'Alpha-L-Fucoside Fucohydrolase 1', 'Fucosidase, Alpha-L- 1, Tissue', 'Tissue Alpha-L-Fucosidase', 'Alpha-L-Fucosidase I', 'Tissue Fucosidase', 'A-L-Fucosidase 1', 'EC 3.2.1.51', 'I+/--L-Fucosidase 1', 'Α-L-Fucosidase 1', 'EC 3.2.1', 'FUCA1', 'FUCA'], ['Alpha-L-Fucosidase 2', 'Alpha-L-Fucoside Fucohydrolase 2', 'Fucosidase, Alpha-L- 2, Plasma', 'Plasma Alpha-L-Fucosidase', 'Plasma Fucosidase', 'A-L-Fucosidase 2', 'EC 3.2.1.51', 'DJ20N2.5', 'I+/--L-Fucosidase 2', 'Α-L-Fucosidase 2', 'MGC1314', 'FUCA2'], ['FUN14 Domain Containing 1', 'FUN14 Domain-Containing Protein 1', 'MGC51029', 'FUNDC1'], ['FUN14 Domain Containing 2', 'HCBP6', 'Cervical Cancer Proto-Oncogene 3 Protein', 'Hepatitis C Virus Core-Binding Protein 6', 'FUN14 Domain-Containing Protein 2', 'HCC-3', 'DC44', 'Cervical Cancer Oncogene 3', 'PD03104', 'FUNDC2', 'HCC3'], ['Fucose Mutarotase', 'C10orf125', 'FucM', 'Chromosome 10 Open Reading Frame 125', 'Protein FucU Homolog', 'EC 5.1.3.29', 'FLJ26016', 'FUCU', 'FUOM', 'FucU'], ['Furin, Paired Basic Amino Acid Cleaving Enzyme', 'Paired Basic Amino Acid Residue-Cleaving Enzyme', 'Proprotein Convertase Subtilisin/Kexin 3', 'Subtilisin-Like Proprotein Convertase 1', 'EC 3.4.21.75', 'PCSK3', 'Furin', 'PACE', 'SPC1', 'FUR', 'Paired Basic Amino Acid Cleaving Enzyme (Furin, Membrane Associated Receptor Protein)', 'Furin (Paired Basic Amino Acid Cleaving Enzyme)', 'Proprotein Convertase Subtilisin/Kexin Type 3', 'Furin, Membrane Associated Receptor Protein', 'Dibasic Processing Enzyme', 'Dibasic-Processing Enzyme', 'FES Upstream Region', 'EC 3.4.21', 'FURIN'], ['FUS RNA Binding Protein', 'TLS', 'Heterogeneous Nuclear Ribonucleoprotein P2', 'Translocated In Liposarcoma Protein', '75 KDa DNA-Pairing Protein', 'RNA-Binding Protein FUS', 'Fused In Sarcoma', 'Oncogene FUS', 'Oncogene TLS', 'HNRNPP2', 'FUS1', 'Fusion (Involved In T(12;16) In Malignant Liposarcoma)', 'Fusion, Derived From T(12;16) Malignant Liposarcoma', 'Fusion Gene In Myxoid Liposarcoma', 'Amyotrophic Lateral Sclerosis 6', 'Translocated In Liposarcoma', 'Fus-Like Protein', 'HnRNP-P2', 'POMP75', 'AltFUS', 'POMp75', 'ALS6', 'ETM4', 'FUS'], ['Fucosyltransferase 1 (H Blood Group)', 'GDP-L-Fucose:Beta-D-Galactoside 2-Alpha-L-Fucosyltransferase 1', 'Type 1 Galactoside Alpha-(1,2)-Fucosyltransferase FUT1', 'Type 2 Galactoside Alpha-(1,2)-Fucosyltransferase FUT1', 'Galactoside Alpha-(1,2)-Fucosyltransferase 1', 'Galactoside 2-Alpha-L-Fucosyltransferase', 'Blood Group H Alpha 2-Fucosyltransferase', 'Alpha(1,2)FT 1', 'EC 2.4.1.69', 'HSC', 'H', 'Fucosyltransferase 1 (Galactoside 2-Alpha-L-Fucosyltransferase, Bombay Phenotype Included)', 'Fucosyltransferase 1 (Galactoside 2-Alpha-L-Fucosyltransferase)', 'Galactoside 2-L-Fucosyltransferase Enzyme', 'Truncated Alpha(1,2)Fucosyltransferase', 'Nonfunctional Fucosyltransferase 1', 'Alpha(1,2) Fucosyltransferase 1', 'Truncated Fucosyltransferase 1', 'H Glycosyltransferase', 'H Glycosytransferase', 'Fucosyltransferase 1', 'Truncated FUT1', 'EC 2.4.1.344', 'FUT1', 'HH'], ['Fucosyltransferase 2', 'GDP-L-Fucose:Beta-D-Galactoside 2-Alpha-L-Fucosyltransferase 2', 'Secretor Blood Group Alpha-2-Fucosyltransferase', 'Secretor Factor', 'SEC2', 'Type 1 Galactoside Alpha-(1,2)-Fucosyltransferase FUT2', 'Type 2 Galactoside Alpha-(1,2)-Fucosyltransferase FUT2', 'Fucosyltransferase 2 (Secretor Status Included)', 'Galactoside Alpha-(1,2)-Fucosyltransferase 2', 'Galactoside 2-Alpha-L-Fucosyltransferase 2', 'Alpha (1,2) Fucosyltransferase', 'Alpha(1,2)FT 2', 'Alpha(1,2)FT2', 'EC 2.4.1.69', 'Se2', 'Sej', 'Galactoside 2-L-Fucosyltransferase', 'Truncated Fucosyltransferase 2', 'EC 2.4.1.344', 'B12QTL1', 'FUT2', 'SE2', 'SE', 'Se'], ['Fucosyltransferase 3 (Lewis Blood Group)', '3-Galactosyl-N-Acetylglucosaminide 4-Alpha-L-Fucosyltransferase FUT3', 'Blood Group Lewis Alpha-4-Fucosyltransferase', 'Galactoside 3(4)-L-Fucosyltransferase', 'Alpha-3-Fucosyltransferase FUT3', 'Fucosyltransferase III', 'EC 2.4.1.65', 'FucT-III', 'Lewis FT', 'CD174', 'FT3B', 'LE', 'Fucosyltransferase 3 (Galactoside 3(4)-L-Fucosyltransferase, Lewis Blood Group Included)', 'Fucosyltransferase 3 (Galactoside 3(4)-L-Fucosyltransferase, Lewis Blood Group)', 'Alpha-(1,3/1,4)-Fucosyltransferase', 'Truncated Fucosyltransferase 3', 'Fucosyltransferase 3', 'EC 2.4.1.-', 'EC 2.4.1', 'FUT3', 'Les'], ['Fucosyltransferase 4', 'Galactoside 3-L-Fucosyltransferase', 'Alpha (1,3) Fucosyltransferase, Myeloid-Specific', 'Alpha-(1,3)-Fucosyltransferase 4', 'ELAM-1 Ligand Fucosyltransferase', 'ELAM Ligand Fucosyltransferase', 'Fucosyltransferase IV', 'FUC-TIV', 'FucT-IV', 'FCT3A', 'ELFT', 'Stage-Specific Embryonic Antigen 1', 'EC 2.4.1.65', 'EC 2.4.1.-', 'EC 2.4.1', 'Lewis X', 'Fuc-TIV', 'SSEA-1', 'FUTIV', 'CD15', 'FUT4', 'LeX'], ['Fucosyltransferase 5', '4-Galactosyl-N-Acetylglucosaminide 3-Alpha-L-Fucosyltransferase FUT5', '3-Galactosyl-N-Acetylglucosaminide 4-Alpha-L-Fucosyltransferase FUT5', 'Galactoside 3-L-Fucosyltransferase', 'Alpha (1,3) Fucosyltransferase', 'Fucosyltransferase V', 'EC 2.4.1.65', 'FUC-TV', 'FucT-V', 'Alpha-(1,3)-Fucosyltransferase 5', 'EC 2.4.1.152', 'EC 2.4.1', 'Fuc-TV', 'FUT5'], ['Fucosyltransferase 6', 'Galactoside 3-L-Fucosyltransferase', 'FucT-VI', 'FCT3A', '4-Galactosyl-N-Acetylglucosaminide 3-Alpha-L-Fucosyltransferase FUT6', 'Fucosyltransferase VI', 'Fuc-TVI', 'FT1A', 'Fucosyltransferase 6 (Alpha (1,3) Fucosyltransferase)', 'Alpha-(1,3)-Fucosyltransferase 6', 'Alpha-(1,3)-Fucosyltransferase', 'EC 2.4.1.152', 'EC 2.4.1.65', 'FLJ40754', 'EC 2.4.1', 'FUT6'], ['Fucosyltransferase 7', 'Fucosyltransferase 7 (Alpha (1,3) Fucosyltransferase)', 'Galactoside 3-L-Fucosyltransferase', 'Alpha-(1,3)-Fucosyltransferase 7', 'Fucosyltransferase VII', 'FucT-VII', 'Fuc-TVII', 'Alpha (1,3) Fucosyltransferase', 'Selectin-Ligand Synthase', 'Selectin Ligand Synthase', 'EC 2.4.1.65', 'EC 2.4.1.-', 'EC 2.4.1', 'FUT7'], ['Fucosyltransferase 8', 'GDP-L-Fuc:N-Acetyl-Beta-D-Glucosaminide Alpha1,6-Fucosyltransferase', 'Fucosyltransferase 8 (Alpha (1,6) Fucosyltransferase)', 'GDP-Fucose--Glycoprotein Fucosyltransferase', 'Glycoprotein 6-Alpha-L-Fucosyltransferase', 'Alpha-(1,6)-Fucosyltransferase', 'Alpha1-6FucT', 'EC 2.4.1.68', 'Alpha (1,6) Fucosyltransferase', 'CDGF1', 'CDGF', 'FUT8'], ['Fucosyltransferase 9', 'Fuc-TIX', '4-Galactosyl-N-Acetylglucosaminide 3-Alpha-L-Fucosyltransferase 9', 'Fucosyltransferase 9 (Alpha (1,3) Fucosyltransferase)', 'Galactoside 3-L-Fucosyltransferase', 'Fucosyltransferase IX', 'FucT-IX', 'Alpha-(1,3)-Fucosyltransferase 9', 'Alpha (1,3) Fucosyltransferase', 'EC 2.4.1.152', 'EC 2.4.1.65', 'EC 2.4.1', 'FUT9'], ['Fucosyltransferase 10', 'Galactoside 3-L-Fucosyltransferase 10', 'Alpha-(1,3)-Fucosyltransferase 10', 'Alpha (1,3) Fucosyltransferase', 'Fucosyltransferase X', 'Fuc-TX', 'FucT-X', 'Fucosyltransferase 10 (Alpha (1,3) Fucosyltransferase)', 'Alpha 1,3-Fucosyl Transferase', 'EC 2.4.1.-', 'EC 2.4.1', 'FUCTX', 'FUT10'], ['Fucosyltransferase 11', 'Galactoside 3-L-Fucosyltransferase 11', 'Alpha-(1,3)-Fucosyltransferase 11', 'Alpha (1,3) Fucosyltransferase', 'Fucosyltransferase XI', 'Fuc-TXI', 'FucT-XI', 'Fucosyltransferase 11 (Alpha (1,3) Fucosyltransferase)', 'EC 2.4.1.-', 'MGC33202', 'EC 2.4.1', 'FUCTXI', 'FUT11'], ['Fuzzy Planar Cell Polarity Protein', 'Ciliogenesis And Planar Polarity Effector 3', 'Protein Fuzzy Homolog', 'CPLANE3', 'FY', 'Fuzzy Homolog (Drosophila)', 'Fuzzy Homolog', 'FLJ22688', 'NTD', 'FUZ', 'Fy'], ['Frataxin', 'X25', 'Friedreich Ataxia Protein', 'Frataxin, Mitochondrial', 'CyaY', 'FARR', 'FRDA', 'FA', 'Friedreich Ataxia', 'EC 1.16.3.1', 'FXN', 'Fxn'], ['FMR1 Autosomal Homolog 1', 'Fragile X Mental Retardation Syndrome-Related Protein 1', 'Fragile X Mental Retardation, Autosomal Homolog 1', 'MYOPMIL', 'MYORIBF', 'HFXR1p', 'FXR1P', 'FXR1'], ['FMR1 Autosomal Homolog 2', 'Fragile X Mental Retardation Syndrome-Related Protein 2', 'Fragile X Mental Retardation, Autosomal Homolog 2', 'FMR1L2', 'Fragile X-Mental Retardation 1-Like 2', 'FXR2P', 'FXR2'], ['FXYD Domain Containing Ion Transport Regulator 1', 'Phospholemman', 'Sodium/Potassium-Transporting ATPase Subunit FXYD1', 'PLM', 'FXYD Domain-Containing Ion Transport Regulator 1', 'FXYD1'], ['FXYD Domain Containing Ion Transport Regulator 2', 'Sodium/Potassium-Transporting ATPase Subunit Gamma', 'Na(+)/K(+) ATPase Subunit Gamma', 'Sodium Pump Gamma Chain', 'ATP1G1', 'ATPase, Na+/K+ Transporting, Gamma 1 Polypeptide', 'FXYD Domain-Containing Ion Transport Regulator 2', 'Hypomagnesemia 2, Renal', 'MGC12372', 'HOMG2', 'FXYD2', 'ATP1C'], ['FXYD Domain Containing Ion Transport Regulator 3', 'FXYD Domain-Containing Ion Transport Regulator 3', 'Sodium/Potassium-Transporting ATPase Subunit FXYD3', 'Chloride Conductance Inducer Protein Mat-8', 'Mammary Tumor 8 KDa Protein', 'MAT8', 'PLML', 'Phospholemman-Like Protein', 'Phospholemman-Like', 'FXYD3', 'MAT-8'], ['FXYD Domain Containing Ion Transport Regulator 4', 'FXYD Domain-Containing Ion Transport Regulator 4', 'CHIF', 'Channel-Inducing Factor', 'FXYD4'], ['FXYD Domain Containing Ion Transport Regulator 5', 'FXYD Domain-Containing Ion Transport Regulator 5', 'Dysadherin', 'DYSAD', 'IWU1', 'OIT2', 'Keratinocytes Associated Transmembrane Protein 1', 'HSPC113', 'PRO6241', 'FXYD5', 'KCT1', 'RIC'], ['FXYD Domain Containing Ion Transport Regulator 6', 'FXYD Domain-Containing Ion Transport Regulator 6', 'Phosphohippolin', 'FXYD6'], ['FXYD Domain Containing Ion Transport Regulator 7', 'FXYD Domain-Containing Ion Transport Regulator 7', 'FXYD7'], ['FYN Binding Protein 1', 'FYB-120/130', 'SLAP-130', 'ADAP', 'Adhesion And Degranulation Promoting Adaptor Protein', 'SLP-76-Associated Phosphoprotein', 'FYN-Binding Protein 1', 'FYN-T-Binding Protein', 'P120/P130', 'SLAP130', 'FYB', 'Adhesion And Degranulation-Promoting Adaptor Protein', 'FYN-Binding Protein (FYB-120/130)', 'FYN Binding Protein', 'PRO0823', 'THC3', 'FYB1'], ['FYN Binding Protein 2', 'Activation-Dependent, Raft-Recruited ADAP-Like Phosphoprotein', 'ARAP', 'FYN-Binding Protein 2', 'RP4-758N20.2', 'C1orf168', 'Chromosome 1 Open Reading Frame 168', 'Uncharacterized Protein C1orf168', 'FLJ43208', 'FYB2'], ['FYVE And Coiled-Coil Domain Autophagy Adaptor 1', 'ZFYVE7', 'FYVE And Coiled-Coil Domain-Containing Protein 1', 'Zinc Finger FYVE Domain-Containing Protein 7', 'FYVE And Coiled-Coil Domain Containing 1', 'RUN And FYVE Domain Containing 3', 'FLJ13335', 'CTRCT18', 'CATC2', 'RUFY3', 'FYCO1'], ['FYN Proto-Oncogene, Src Family Tyrosine Kinase', 'SLK', 'FYN Oncogene Related To SRC, FGR, YES', 'Tyrosine-Protein Kinase Fyn', 'Proto-Oncogene C-Fyn', 'Proto-Oncogene Syn', 'Src-Like Kinase', 'EC 2.7.10.2', 'SYN', 'OKT3-Induced Calcium Influx Regulator', 'Tyrosine Kinase P59fyn(T)', 'Src/Yes-Related Novel', 'C-Syn Protooncogene', "EC 2.7.10.2');", 'EC 2.7.10', 'MGC45350', 'P59-FYN', 'P59-Fyn', 'FYN'], ['Forty-Two-Three Domain Containing 1', 'UAP56-Interacting Factor', 'UIF', 'Forty-Two-Three Domain-Containing Protein 1', 'Protein 40-2-3', 'DKFZp761B1514', 'FYTTD1'], ['Frizzled Class Receptor 1', 'Frizzled 1, Seven Transmembrane Spanning Receptor', 'Frizzled, Drosophila, Homolog Of, 1', 'Frizzled Family Receptor 1', 'Wnt Receptor', 'Frizzled-1', 'Fz-1', 'FzE1', 'HFz1', 'Frizzled (Drosophila) Homolog 1', 'Frizzled Homolog 1 (Drosophila)', 'Frizzled Homolog 1', 'DKFZp564G072', 'FZD1'], ['Frizzled Class Receptor 2', 'Frizzled 2, Seven Transmembrane Spanning Receptor', 'Frizzled Family Receptor 2', 'Frizzled-2', 'Fz-2', 'FzE2', 'HFz2', 'Frizzled (Drosophila) Homolog 2', 'Frizzled Homolog 2 (Drosophila)', 'Frizzled Homolog 2', 'OMOD2', 'FZD2', 'Fz2'], ['Frizzled Class Receptor 3', 'Frizzled 3, Seven Transmembrane Spanning Receptor', 'Frizzled Family Receptor 3', 'Frizzled-3', 'Fz-3', 'Frizzled (Drosophila) Homolog 3', 'Frizzled Homolog 3 (Drosophila)', 'Frizzled Homolog 3', 'FZD3', 'HFz3'], ['Frizzled Class Receptor 4', 'Frizzled 4, Seven Transmembrane Spanning Receptor', 'Frizzled Family Receptor 4', 'Frizzled-4', 'CD344', 'Fz-4', 'FzE4', 'HFz4', 'Frizzled (Drosophila) Homolog 4', 'Frizzled Homolog 4 (Drosophila)', 'Exudative Vitreoretinopathy 1', 'WNT Receptor Frizzled-4', 'Frizzled Homolog 4', 'CD344 Antigen', 'FZD4S', 'EVR1', 'FEVR', 'GPCR', 'FZD4', 'Fz4'], ['Frizzled Class Receptor 5', 'Frizzled 5, Seven Transmembrane Spanning Receptor', 'Frizzled Family Receptor 5', 'Frizzled-5', 'C2orf31', 'HFZ5', 'Fz-5', 'FzE5', 'Seven-Transmembrane Receptor Frizzled-5', 'Chromosome 2 Open Reading Frame 31', 'Frizzled (Drosophila) Homolog 5', 'Frizzled Homolog 5 (Drosophila)', 'DKFZP434E2135', 'Wnt Receptor', 'FZD5', 'HFz5'], ['Frizzled Class Receptor 6', 'Frizzled 6, Seven Transmembrane Spanning Receptor', 'Frizzled Family Receptor 6', 'Frizzled-6', 'Seven Transmembrane Helix Receptor', 'Frizzled (Drosophila) Homolog 6', 'Frizzled Homolog 6 (Drosophila)', 'Frizzled Homolog 6', 'NDNC10', 'NDNC1', 'FZ-6', 'HFZ6', 'FZD6', 'Hfz6', 'Fz-6', 'HFz6', 'FZ6'], ['Frizzled Class Receptor 7', 'FzE3', 'Frizzled 7, Seven Transmembrane Spanning Receptor', 'Frizzled Family Receptor 7', 'Frizzled-7', 'Fz-7', 'HFz7', 'Frizzled, Drosophila, Homolog Of, 7', 'Frizzled (Drosophila) Homolog 7', 'Frizzled Homolog 7 (Drosophila)', 'Frizzled Homolog 7', 'FZD7'], ['Frizzled Class Receptor 8', 'Frizzled 8, Seven Transmembrane Spanning Receptor', 'Frizzled Family Receptor 8', 'Frizzled-8', 'Frizzled (Drosophila) Homolog 8', 'Frizzled Homolog 8 (Drosophila)', 'Frizzled Homolog 8', 'FZ-8', 'HFZ8', 'FZD8', 'Fz-8', 'HFz8'], ['Frizzled Class Receptor 9', 'FZD3', 'Frizzled 9, Seven Transmembrane Spanning Receptor', 'Frizzled Family Receptor 9', 'Frizzled-9', 'CD349', 'Fz-9', 'FzE6', 'HFz9', 'Frizzled (Drosophila) Homolog 9', 'Frizzled Homolog 9 (Drosophila)', 'Frizzled Homolog 9', 'CD349 Antigen', 'FZD9'], ['Frizzled Class Receptor 10', 'Frizzled 10, Seven Transmembrane Spanning Receptor', 'Frizzled Family Receptor 10', 'Frizzled-10', 'CD350', 'HFz10', 'FzE7', 'Frizzled (Drosophila) Homolog 10', 'Frizzled Homolog 10 (Drosophila)', 'Frizzled Homolog 10', 'CD350 Antigen', 'FZ-10', 'FZD10', 'Fz-10', 'Fz10'], ['Fizzy And Cell Division Cycle 20 Related 1', 'HCDH1', 'CDH1', 'FZR', 'Fizzy/Cell Division Cycle 20 Related 1', 'Fizzy-Related Protein Homolog', 'CDC20-Like Protein 1', 'Cdh1/Hct1 Homolog', 'CDC20 Homolog 1', 'KIAA1242', 'CDC20C', 'FZR2', 'HCDH', 'Fizzy/Cell Division Cycle 20 Related 1 (Drosophila)', 'CDC20-Like 1b', 'FZR1', 'FYR', 'Fzr'], ['G2/M-Phase Specific E3 Ubiquitin Protein Ligase', 'KIAA1333', 'G2/M Phase-Specific HECT-Type E3 Ubiquitin Transferase', 'G2/M Phase-Specific E3 Ubiquitin-Protein Ligase', 'PHD Finger Protein 7B', 'PHF7B', 'G2/M-Phase Specific E3 Ubiquitin Ligase', 'EC 2.3.2.26', 'FLJ20333', 'EC 6.3.2', 'G2E3'], ['G3BP Stress Granule Assembly Factor 1', 'G3BP', 'Ras-GTPase-Activating Protein SH3-Domain-Binding Protein', 'GTPase Activating Protein (SH3 Domain) Binding Protein 1', 'Ras GTPase-Activating Protein-Binding Protein 1', 'GAP SH3 Domain-Binding Protein 1', 'ATP-Dependent DNA Helicase VIII', 'DNA Helicase VIII', 'HDH-VIII', 'G3BP-1', 'RasGAP-Associated Endoribonuclease G3BP', 'GAP Binding Protein', 'EC 3.6.4.12', 'EC 3.6.4.13', 'HDH VIII', 'EC 3.6.1', 'G3BP1'], ['G3BP Stress Granule Assembly Factor 2', 'Ras-GTPase Activating Protein SH3 Domain-Binding Protein 2', 'GTPase Activating Protein (SH3 Domain) Binding Protein 2', 'Ras GTPase-Activating Protein-Binding Protein 2', 'GAP SH3 Domain-Binding Protein 2', 'KIAA0660', 'G3BP-2', 'G3BP2'], ['Glucose-6-Phosphatase Catalytic Subunit 1', 'Glucose-6-Phosphatase Catalytic Subunit', 'Glucose-6-Phosphatase Alpha', 'Glucose-6-Phosphatase', 'G6Pase-Alpha', 'EC 3.1.3.9', 'G-6-Pase', 'G6Pase', 'GSD1a', 'G6PC', 'G6PT', 'Glucose-6-Phosphatase, Catalytic (Glycogen Storage Disease Type I, Von Gierke Disease)', 'Glycogen Storage Disease Type I, Von Gierke Disease', 'GSD1'], ['Glucose-6-Phosphatase Catalytic Subunit 2', 'IGRP', 'Islet-Specific Glucose-6-Phosphatase Catalytic Subunit-Related Protein', 'Glucose-6-Phosphatase, Catalytic, 2', 'Glucose-6-Phosphatase 2', 'G-6-Pase 2', 'EC 3.1.3.9', 'G6Pase 2', 'Islet Specific Glucose 6 Phosphatase Catalytic Subunit Related Protein', 'Islet-Specific Glucose-6-Phosphatase-Related Protein', 'Islet-Specific G6CP-Related Protein', 'G6PC2'], ['Glucose-6-Phosphatase Catalytic Subunit 3', 'UGRP', 'Ubiquitous Glucose-6-Phosphatase Catalytic Subunit-Related Protein', 'Glucose 6 Phosphatase, Catalytic, 3', 'Glucose-6-Phosphatase 3', 'G6Pase-Beta', 'G-6-Pase 3', 'EC 3.1.3.9', 'G6Pase 3', 'Ubiquitously Expressed G6Pase Catalytic Subunit-Related Protein', 'Glucose-6-Phosphatase Beta', 'G6PC3', 'SCN4'], ['Glucose-6-Phosphate Dehydrogenase', 'Glucose-6-Phosphate 1-Dehydrogenase', 'EC 1.1.1.49', 'G6PD1', 'Epididymis Secretory Sperm Binding Protein', 'G6PD'], ['Alpha Glucosidase', 'Glucosidase Alpha, Acid', 'Lysosomal Alpha-Glucosidase', 'Aglucosidase Alfa', 'Acid Maltase', 'EC 3.2.1.20', 'Glycogen Storage Disease Type II', 'Glucosidase, Alpha; Acid', 'Pompe Disease', 'LYAG', 'GAA'], ['GRB2 Associated Binding Protein 1', 'Growth Factor Receptor Bound Protein 2-Associated Protein 1', 'GRB2-Associated-Binding Protein 1', 'Deafness, Autosomal Recessive 26', 'GRB2-Associated Binder 1', 'Symbol Withdrawn, See GAB1', 'DFNB26', 'GAB1'], ['GRB2 Associated Binding Protein 2', 'Growth Factor Receptor Bound Protein 2-Associated Protein 2', 'GRB2-Associated-Binding Protein 2', 'Grb2-Associated Binder 2', 'KIAA0571', 'Pp100', 'GRB2-Associated Binder 2', 'GAB2'], ['GRB2 Associated Binding Protein 3', 'Growth Factor Receptor Bound Protein 2-Associated Protein 3', 'GRB2-Associated-Binding Protein 3', 'Gab3 Scaffolding Protein', 'GRB2-Associated Binder 3', 'DOS/Gab Family Member 3', 'GAB3'], ['GRB2 Associated Binding Protein Family Member 4', 'Growth Factor Receptor Bound Protein 2-Associated Protein 4', 'GRB2-Associated-Binding Protein 4', 'GRB2-Associated Binder 4', 'GRB2-Associated Binding Protein Family, Member 4', 'GRB2-Associated-Binding Protein 2-Like', 'GRB2-Associated Binder 2-Like Protein', 'GRB2-Associated Binder 2-Like', 'GAB2-Like', 'GAB4'], ['GABA Type A Receptor-Associated Protein', 'GABA(A) Receptor-Associated Protein', 'MM46', 'Gamma-Aminobutyric Acid Receptor-Associated Protein', 'ATG8A', 'Epididymis Secretory Sperm Binding Protein', 'GABARAP-A', 'GABARAP', 'FLC3B'], ['GABA Type A Receptor Associated Protein Like 1', 'Gamma-Aminobutyric Acid Receptor-Associated Protein-Like 1', 'GABA(A) Receptor-Associated Protein-Like 1', 'Glandular Epithelial Cell Protein 1', 'Early Estrogen-Regulated Protein', 'APG8L', 'ATG8B', 'ATG8L', 'GEC1', 'GABA(A) Receptor-Associated Protein Like 1', 'APG8-LIKE', 'GABARAPL1', 'GEC-1', 'ATG8', 'Gec1'], ['GABA Type A Receptor Associated Protein Like 2', 'GATE-16', 'GEF2', 'Gamma-Aminobutyric Acid Receptor-Associated Protein-Like 2', 'GABA(A) Receptor-Associated Protein-Like 2', 'Golgi-Associated ATPase Enhancer Of 16 KDa', 'General Protein Transport Factor P16', 'Ganglioside Expression Factor 2', 'GATE16', 'ATG8C', 'GEF-2', 'ATG8', 'GABA(A) Receptor-Associated Protein Like 2', 'MAP1 Light Chain 3 Related Protein', 'MAP1 Light Chain 3-Related Protein', 'GABARAPL2', 'FLC3A'], ['Gamma-Aminobutyric Acid Type B Receptor Subunit 1', 'GPRC3A', 'Gamma-Aminobutyric Acid (GABA) B Receptor, 1', 'GABABR1', 'Seven Transmembrane Helix Receptor', 'GABA-B Receptor, R1 Subunit', 'GABA-B Receptor 1', 'GABA-B Receptor', 'GABA-B-R1', 'GABBR1-3', 'GABA-BR1', 'GABBR1', 'HGB1a', 'GB1', 'Gb1'], ['Gamma-Aminobutyric Acid Type B Receptor Subunit 2', 'GABABR2', 'GPRC3B', 'HG20', 'Gamma-Aminobutyric Acid (GABA) B Receptor, 2', 'G-Protein Coupled Receptor 51', 'GABA-B Receptor 2', 'GABA-B-R2', 'GABA-BR2', 'GPR51', 'Gb2', 'Gamma-Aminobutyric Acid B Receptor 2', 'G Protein-Coupled Receptor 51', 'GABA-B Receptor, R2 Subunit', 'HRIHFB2099', 'EIEE59', 'NDPLHS', 'GABBR2', 'DEE59'], ['GA Binding Protein Transcription Factor Subunit Alpha', 'E4TF1A', 'Human Nuclear Respiratory Factor-2 Subunit Alpha', 'Nuclear Respiratory Factor 2 Alpha Subunit', 'Nuclear Respiratory Factor 2 Subunit Alpha', 'GA-Binding Protein Alpha Chain', 'Transcription Factor E4TF1-60', 'GABP Subunit Alpha', 'E4TF1-60', 'NRF2A', 'NFT2', 'NRF2', 'GA-Binding Protein Transcription Factor, Alpha Subunit (60kD)', 'GA Binding Protein Transcription Factor, Alpha Subunit 60kDa', 'GA Binding Protein Transcription Factor Alpha Subunit 60kDa', 'GA Binding Protein Transcription Factor Alpha Subunit', 'RCH04A07', 'GABPA'], ['GA Binding Protein Transcription Factor Subunit Beta 1', 'GABPB', 'GA-Binding Protein Subunit Beta-1', 'Transcription Factor E4TF1-47', 'Transcription Factor E4TF1-53', 'Nuclear Respiratory Factor 2', 'GABP Subunit Beta-2', 'E4TF1-47', 'GABPB-1', 'E4TF1B', 'GABPB2', 'GA Binding Protein Transcription Factor Beta Subunit 1 Transcript Variant Gamma-2', 'GA Binding Protein Transcription Factor, Beta Subunit 2', 'GA Binding Protein Transcription Factor Beta Subunit 1', 'GABP Subunit Beta-1', 'E4TF1-53', 'GABPB-2', 'BABPB2', 'NRF2B1', 'NRF2B2', 'GABPB1', 'E4TF1'], ['GA Binding Protein Transcription Factor Subunit Beta 2', 'GA Binding Protein Transcription Factor Beta Subunit 2', 'GA-Binding Protein Subunit Beta-2', 'GABP Subunit Beta-2', 'GABPB-2', 'MGC29891', 'GABPB2'], ['Gamma-Aminobutyric Acid Type A Receptor Subunit Alpha1', 'Gamma-Aminobutyric Acid (GABA) A Receptor, Alpha 1', 'Gamma-Aminobutyric Acid Receptor Subunit Alpha-1', 'GABA(A) Receptor Subunit Alpha-1', 'GABA(A) Receptor, Alpha 1', 'EJM5', 'Gamma-Aminobutyric Acid Type A Receptor Alpha1 Subunit', 'EIEE19', 'GABRA1', 'DEE19', 'ECA4', 'EJM'], ['Gamma-Aminobutyric Acid Type A Receptor Subunit Alpha2', 'Gamma-Aminobutyric Acid Receptor Subunit Alpha-2', 'GABA(A) Receptor Subunit Alpha-2', 'Gamma-Aminobutyric Acid Type A Receptor Alpha2 Subunit', 'Gamma-Aminobutyric Acid (GABA) A Receptor, Alpha 2', 'GABA(A) Receptor, Alpha 2', 'EIEE78', 'GABRA2', 'DEE78'], ['Gamma-Aminobutyric Acid Type A Receptor Subunit Alpha3', 'Gamma-Aminobutyric Acid (GABA) A Receptor, Alpha 3', 'Gamma-Aminobutyric Acid Receptor Subunit Alpha-3', 'GABA(A) Receptor Subunit Alpha-3', 'GABA(A) Receptor, Alpha 3', 'Gamma-Aminobutyric Acid Type A Receptor Alpha3 Subunit', 'Gamma-Animobutyric Acid (GABA) A Receptor, Alpha 3', 'GABRA3'], ['Gamma-Aminobutyric Acid Type A Receptor Subunit Alpha4', 'Gamma-Aminobutyric Acid (GABA) A Receptor, Alpha 4', 'Gamma-Aminobutyric Acid Receptor Subunit Alpha-4', 'GABA(A) Receptor, Alpha 4', 'Gamma-Aminobutyric Acid Type A Receptor Alpha4 Subunit', 'Gamma-Aminobutyric Acid A Receptor Alpha 4', 'GABA(A) Receptor Subunit Alpha-4', 'GABRA4'], ['Gamma-Aminobutyric Acid Type A Receptor Subunit Alpha5', 'Gamma-Aminobutyric Acid (GABA) A Receptor, Alpha 5', 'Gamma-Aminobutyric Acid Receptor Subunit Alpha-5', 'GABA(A) Receptor Subunit Alpha-5', 'Gamma-Aminobutyric Acid Type A Receptor Alpha5 Subunit', 'GABA(A) Receptor, Alpha 5', 'EIEE79', 'GABRA5', 'DEE79'], ['Gamma-Aminobutyric Acid Type A Receptor Subunit Alpha6', 'Gamma-Aminobutyric Acid (GABA) A Receptor, Alpha 6', 'Gamma-Aminobutyric Acid Receptor Subunit Alpha-6', 'GABA(A) Receptor Subunit Alpha-6', 'GABA(A) Receptor, Alpha 6', 'Gamma-Aminobutyric Acid Type A Receptor Alpha6 Subunit', 'GABA Subunit A Receptor Alpha 6', 'GABRA6'], ['Gamma-Aminobutyric Acid Type A Receptor Subunit Beta1', 'Gamma-Aminobutyric Acid (GABA) A Receptor, Beta 1', 'Gamma-Aminobutyric Acid Receptor Subunit Beta-1', 'Gamma-Aminobutyric Acid Type A Receptor Beta1 Subunit', 'GABA(A) Receptor Subunit Beta-1', 'GABA(A) Receptor, Beta 1', 'EIEE45', 'GABRB1', 'DEE45'], ['Gamma-Aminobutyric Acid Type A Receptor Subunit Beta2', 'Gamma-Aminobutyric Acid Receptor Subunit Beta-2', 'GABA(A) Receptor Subunit Beta-2', 'Gamma-Aminobutyric Acid Type A Receptor Beta2 Subunit', 'Gamma-Aminobutyric Acid (GABA) A Receptor, Beta 2', 'Gamma-Aminobutyric Acid A Receptor Beta 2', 'GABA(A) Receptor, Beta 2', 'GABRB2', 'DEE92', 'ICEE2'], ['Gamma-Aminobutyric Acid Type A Receptor Subunit Beta3', 'Gamma-Aminobutyric Acid (GABA) A Receptor, Beta 3', 'Gamma-Aminobutyric Acid Receptor Subunit Beta-3', 'Gamma-Aminobutyric Acid Type A Receptor Beta3 Subunit', 'Gamma-Aminobutyric Acid A Receptor Beta 3', 'GABA-Alpha Receptor Beta-2 Subunit', 'GABA(A) Receptor Subunit Beta-3', 'GABAA Receptor Beta-3 Subunit', 'GABA(A) Receptor, Beta 3', 'EIEE43', 'GABRB3', 'DEE43', 'ECA5'], ['Gamma-Aminobutyric Acid Type A Receptor Subunit Delta', 'Gamma-Aminobutyric Acid (GABA) A Receptor, Delta', 'Gamma-Aminobutyric Acid Receptor Subunit Delta', 'GABA(A) Receptor Subunit Delta', 'GABA(A) Receptor, Delta', 'Gamma-Aminobutyric Acid Type A Receptor Delta Subunit', 'GABA-A Receptor, Delta Polypeptide', 'GEFSP5', 'EIG10', 'GABRD', 'EJM7'], ['Gamma-Aminobutyric Acid Type A Receptor Subunit Epsilon', 'Gamma-Aminobutyric Acid (GABA) A Receptor, Epsilon', 'Gamma-Aminobutyric Acid Receptor Subunit Epsilon', 'GABA(A) Receptor Subunit Epsilon', 'GABA(A) Receptor, Epsilon', 'Gamma-Aminobutyric Acid Type A Receptor Epsilon Subunit', 'GABRE'], ['Gamma-Aminobutyric Acid Type A Receptor Subunit Gamma1', 'Gamma-Aminobutyric Acid (GABA) A Receptor, Gamma 1', 'Gamma-Aminobutyric Acid Receptor Subunit Gamma-1', 'GABA(A) Receptor Subunit Gamma-1', 'GABA(A) Receptor, Gamma', 'Gamma-Aminobutyric Acid Type A Receptor Gamma1 Subunit', 'Gamma-1 Polypeptide', 'GABRG1'], ['Gamma-Aminobutyric Acid Type A Receptor Subunit Gamma2', 'Gamma-Aminobutyric Acid (GABA) A Receptor, Gamma 2', 'Gamma-Aminobutyric Acid Receptor Subunit Gamma-2', 'GABA(A) Receptor Subunit Gamma-2', 'GABA(A) Receptor, Gamma 2', 'Gamma-Aminobutyric Acid Type A Receptor Gamma2 Subunit', 'EIEE74', 'GEFSP3', 'GABRG2', 'DEE74', 'CAE2', 'ECA2', 'FEB8'], ['Gamma-Aminobutyric Acid Type A Receptor Subunit Gamma3', 'Gamma-Aminobutyric Acid (GABA) A Receptor, Gamma 3', 'Gamma-Aminobutyric Acid Receptor Subunit Gamma-3', 'GABA(A) Receptor Subunit Gamma-3', 'GABA(G) Receptor, Gamma 3', 'Gamma-Aminobutyric Acid Type A Receptor Gamma3 Subunit', 'GABRG3'], ['Gamma-Aminobutyric Acid Type A Receptor Subunit Pi', 'Gamma-Aminobutyric Acid (GABA) A Receptor, Pi', 'Gamma-Aminobutyric Acid Receptor Subunit Pi', 'GABA(A) Receptor Subunit Pi', 'GABA(A) Receptor, Pi', 'Gamma-Aminobutyric Acid Type A Receptor Pi Subunit', 'GABRP'], ['Gamma-Aminobutyric Acid Type A Receptor Subunit Theta', 'Gamma-Aminobutyric Acid Receptor Subunit Theta', 'GABA(A) Receptor Subunit Theta', 'Gamma-Aminobutyric Acid Type A Receptor Theta Subunit', 'Gamma-Aminobutyric Acid (GABA) A Receptor, Theta', 'Gamma-Aminobutyric Acid (GABA) Receptor, Theta', 'GABA(A) Receptor, Theta', 'THETA', 'GABRQ'], ['Gamma-Aminobutyric Acid Type A Receptor Subunit Rho1', 'Gamma-Aminobutyric Acid (GABA) Receptor, Rho 1', 'Gamma-Aminobutyric Acid Receptor Subunit Rho-1', 'GABA(A) Receptor Subunit Rho-1', 'GABA(A) Receptor, Rho 1', 'GABA(C) Receptor', 'BA135P14.1 (Gamma-Aminobutyric Acid (GABA) Receptor, Rho 1)', 'Gamma-Aminobutyric Acid Type A Receptor Rho1 Subunit', 'Gamma-Aminobutyric Acid (GABA) A Receptor, Rho-1', 'GABRR1'], ['Gamma-Aminobutyric Acid Type A Receptor Subunit Rho2', 'Gamma-Aminobutyric Acid Receptor Subunit Rho-2', 'Gamma-Aminobutyric Acid Type A Receptor Rho2 Subunit', 'Gamma-Aminobutyric Acid (GABA) A Receptor, Rho 2', 'Gamma-Aminobutyric Acid (GABA) Receptor, Rho 2', 'GABA-C Receptor, Rho-2 Subunit', 'GABA(A) Receptor Subunit Rho-2', 'GABA(A) Receptor, Rho 2', 'GABA(C) Receptor', 'GABRR2'], ['Gamma-Aminobutyric Acid Type A Receptor Subunit Rho3', 'Gamma-Aminobutyric Acid Receptor Subunit Rho-3', 'GABA(A) Receptor Subunit Rho-3', 'GABA(C) Receptor', 'Gamma-Aminobutyric Acid Type A Receptor Subunit Rho3 (Gene/Pseudogene)', 'Gamma-Aminobutyric Acid Type A Receptor Rho3 Subunit', 'Gamma-Aminobutyric Acid (GABA) A Receptor, Rho 3', 'Gamma-Aminobutyric Acid (GABA) Receptor, Rho 3', 'GABA(A) Receptor, Rho 3', 'GABRR3'], ['Glutamate Decarboxylase 1', 'Glutamate Decarboxylase 1 (Brain, 67kDa)', '67 KDa Glutamic Acid Decarboxylase', 'EC 4.1.1.15', 'GAD-67', 'GAD', 'Glutamate Decarboxylase 1 (Brain, 67kD)', 'Glutamate Decarboxylase 67 KDa Isoform', 'EC 4.1.1', 'CPSQ1', 'DEE89', 'GAD67', 'GAD1', 'SCP'], ['Glutamate Decarboxylase 2', 'GAD65', 'Glutamate Decarboxylase 2 (Pancreatic Islets And Brain, 65kDa)', '65 KDa Glutamic Acid Decarboxylase', 'EC 4.1.1.15', 'GAD-65', 'Glutamate Decarboxylase 2 (Pancreatic Islets And Brain, 65kD)', 'Glutamate Decarboxylase 65 KDa Isoform', 'Glutamate Decarboxylase-2 (Pancreas)', 'EC 4.1.1', 'GAD2'], ['Growth Arrest And DNA Damage Inducible Alpha', 'GADD45', 'Growth Arrest And DNA Damage-Inducible Protein GADD45 Alpha', 'DNA Damage-Inducible Transcript 1 Protein', 'DDIT-1', 'DDIT1', 'Growth Arrest And DNA-Damage-Inducible 45 Alpha', 'Growth Arrest And DNA-Damage-Inducible, Alpha', 'DNA Damage-Inducible Transcript-1', 'GADD45A'], ['Growth Arrest And DNA Damage Inducible Beta', 'Growth Arrest And DNA Damage-Inducible Protein GADD45 Beta', 'Myeloid Differentiation Primary Response Protein MyD118', 'Negative Growth Regulatory Protein MyD118', 'GADD45BETA', 'MYD118', 'Growth Arrest And DNA-Damage-Inducible, Beta', 'Growth Arrest And DNA-Damage-Inducible Beta', 'Myeloid Differentiation Primary Response', 'DKFZP566B133', 'GADD45B'], ['Growth Arrest And DNA Damage Inducible Gamma', 'DDIT2', 'CR6', 'Growth Arrest And DNA Damage-Inducible Protein GADD45 Gamma', 'DNA Damage-Inducible Transcript 2 Protein', 'Cytokine-Responsive Protein CR6', 'Gadd-Related Protein, 17 KD', 'GADD45gamma', 'DDIT-2', 'GRP17', 'Growth Arrest And DNA-Damage-Inducible, Gamma', 'Growth Arrest And DNA-Damage-Inducible Gamma', 'GADD45-Gamma', 'GADD45G'], ['GADD45G Interacting Protein 1', 'PLINP-1', 'CRIF1', 'PRG6', 'Growth Arrest And DNA Damage-Inducible Proteins-Interacting Protein 1', 'Growth Arrest And DNA-Damage-Inducible, Gamma Interacting Protein 1', 'Mitochondrial Large Ribosomal Subunit Protein ML64', 'Papillomavirus L2 Interacting Nuclear Protein 1', '39S Ribosomal Protein L59, Mitochondrial', 'CKII Beta-Associating Protein', 'P53-Responsive Gene 6 Protein', 'CKII Beta Binding Protein 2', 'CR6 Interacting Factor 1', 'CKbetaBP2', 'MRP-L59', 'CKBBP2', 'Plinp1', 'PLINP', 'Growth Arrest- And DNA Damage-Inducible GADD45G-Interacting Protein', 'Mitochondrial Large Ribosomal Subunit Protein CRIF1', 'Papillomavirus L2-Interacting Nuclear Protein 1', 'CR6-Interacting Factor 1', 'P53-Responsive Gene 6', 'GADD45GIP1', 'MGC4667', 'MGC4758', 'MRPL59', 'PLINP1'], ['Glutamate Decarboxylase Like 1', 'Glutamate Decarboxylase-Like Protein 1', 'Acidic Amino Acid Decarboxylase GADL1', 'Cysteine Sulfinic Acid Decarboxylase', 'Aspartate 1-Decarboxylase', 'EC 4.1.1.29', 'HuCSADC', 'CSADC', 'HuADC', 'ADC', 'Glutamate Decarboxylase-Like 1', 'EC 4.1.1.15', 'EC 4.1.1.11', 'EC 4.1.1', 'GADL1'], ['G Antigen 1', 'CT4.1', 'Cancer/Testis Antigen Family 4, Member 1', 'Cancer/Testis Antigen 4.1', 'GAGE-1', 'Cancer/Testis Antigen 4.4', 'MZ2-F Antigen', 'Antigen MZ2-F', 'G Antigen 4', 'GAGE-4', 'CT4.4', 'GAGE4', 'GAGE1'], ['G Antigen 2A', 'Cancer/Testis Antigen Family 4, Member 2', 'G Antigen 2', 'GAGE-2A', 'CT4.2', 'GAGE2', 'Cancer/Testis Antigen 4.2', 'G Antigen 2A/2B', 'GAGE-2', 'GAGE2A'], ['G Antigen 2B', 'Cancer/Testis Antigen 4.2', 'G Antigen 2B/2C', 'GAGE-2B', 'CT4.2', 'G Antigen 2A/2B', 'G Antigen 2C', 'GAGE-2C', 'GAGE-2', 'GAGE2'], ['G Antigen 2C', 'Cancer/Testis Antigen 4.2', 'G Antigen 2B/2C', 'GAGE-2C', 'CT4.2', 'GAGE2', 'G Antigen 2', 'GAGE-2B', 'GAGE-2'], ['G Antigen 2D', 'Cancer/Testis Antigen 4.8', 'GAGE-2D', 'GAGE-8', 'CT4.8', 'GAGE8', 'G Antigen 8'], ['G Antigen 2E', 'GAGE-2E', 'GAGE8', 'GAGE2E'], ['G Antigen 4', 'CT4.4', 'Cancer/Testis Antigen Family 4, Member 4', 'Cancer/Testis Antigen 4.4', 'GAGE-4'], ['G Antigen 5', 'CT4.5', 'Cancer/Testis Antigen Family 4, Member 5', 'Cancer/Testis Antigen 4.5', 'GAGE-5'], ['G Antigen 6', 'CT4.6', 'Cancer/Testis Antigen Family 4, Member 6', 'Cancer/Testis Antigen 4.6', 'GAGE-6'], ['G Antigen 7', 'GAGE-7', 'CT4.7', 'Cancer/Testis Antigen Family 4, Member 7', 'Cancer/Testis Antigen 4.7', 'AL4', 'G Antigen 12G', 'GAGE-12I', 'GAGE12I', 'GAGE-7B', 'GAGE7B', 'GAGE-8'], ['G Antigen 8', 'CT4.8', 'Cancer/Testis Antigen Family 4, Member 8', 'Cancer/Testis Antigen 4.8', 'G Antigen 2D', 'GAGE-8', 'GAGE-2D'], ['G Antigen 10', 'GAGE-10', 'OTTHUMG00000024136', 'GAGE10'], ['G Antigen 12B', 'G Antigen 12B/C/D/E', 'GAGE-12B', 'OTTHUMG00000024142', 'GAGE-12C', 'GAGE-12D', 'GAGE-12E', 'GAGE12B'], ['G Antigen 12C', 'G Antigen 12B/C/D/E', 'GAGE-12B', 'GAGE-12C', 'OTTHUMG00000024144', 'G Antigen 12C/D/E', 'GAGE-12D', 'GAGE-12E', 'GAGE12C'], ['G Antigen 12D', 'G Antigen 12B/C/D/E', 'GAGE-12B', 'GAGE-12D', 'OTTHUMG00000024145', 'G Antigen 12C/D/E', 'GAGE-12C', 'GAGE-12E', 'GAGE12D'], ['G Antigen 12E', 'G Antigen 12B/C/D/E', 'GAGE-12B', 'OTTHUMG00000024146', 'GAGE-12C', 'GAGE-12D', 'GAGE-12E', 'GAGE12E'], ['G Antigen 12F', 'GAGE-12F', 'Cancer/Testis Antigen 4.7', 'OTTHUMG00000024147', 'G Antigen 12F/G/I', 'G Antigen 12G', 'G Antigen 12I', 'G Antigen 7', 'GAGE-7B', 'GAGE12F', 'GAGE-7', 'GAGE-8', 'GAGE7B', 'CT4.7', 'GAGE7', 'AL4'], ['G Antigen 12G', 'GAGE-12G', 'Cancer/Testis Antigen 4.7', 'OTTHUMG00000024148', 'G Antigen 12F/G/I', 'G Antigen 12I', 'GAGE12G', 'CT4.7', 'AL4'], ['G Antigen 12H', 'GAGE-12H', 'OTTHUMG00000024149', 'GAGE12H'], ['G Antigen 12I', 'G Antigen 7B', 'Cancer/Testis Antigen 4.7', 'OTTHUMG00000067390', 'G Antigen 12F/G/I', 'G Antigen 12G', 'GAGE-12I', 'GAGE-7B', 'GAGE7B', 'CT4.7', 'AL4'], ['G Antigen 12J', 'G Antigen 11', 'OTTHUMG00000024137', 'GAGE-12J', 'GAGE12J', 'GAGE11'], ['G Antigen 13', 'G Antigen 12A', 'GAGE-12A', 'GAGE-13', 'GAGE12A', 'OTTHUMG00000024138', 'GAGE13'], ['Cyclin G Associated Kinase', 'Cyclin-G-Associated Kinase', 'EC 2.7.11.1', 'Auxilin-2', 'DNAJC26', 'DNAJ26', 'GAK'], ['Galanin And GMAP Prepropeptide', 'GLNN', 'Galanin-Message-Associated Peptide', 'Galanin Peptides', 'GAL-GMAP', 'GALN', 'GMAP', 'Galanin/GMAP Prepropeptide', 'Galanin-Related Peptide', 'Galanin Prepropeptide', 'Galanin', 'ETL8', 'GAL1', 'GAL'], ['Galactose-3-O-Sulfotransferase 1', 'CST', "3'-Phosphoadenosine-5'-Phosphosulfate:GalCer Sulfotransferase", 'Galactosylceramide Sulfotransferase', "Cerebroside (3' Phosphoadenylylsulfate:Galactosylceramide 3') Sulfotransferase", "Cerebroside (3'-Phosphoadenylylsulfate:Galactosylceramide 3') Sulfotransferase", "3'-Phosphoadenylylsulfate:Galactosylceramide 3'-Sulfotransferase", 'Cerebroside Sulfotransferase', 'GalCer Sulfotransferase', 'EC 2.8.2.11', 'GAL3ST1'], ['Galactose-3-O-Sulfotransferase 2', 'GP3ST', "Glycoprotein Beta-Gal 3'-Sulfotransferase 2", "Gal-Beta-1, 3-GalNAc 3'-Sulfotransferase 2", 'Beta-Galactose-3-O-Sulfotransferase 2', "Galbeta1-3GalNAc 3'-Sulfotransferase 2", 'Gal 3-O-Sulphotransferase', 'EC 2.8.2.-', 'GAL3ST-2', 'Gal3ST-2', 'GAL3ST2'], ['Galactose-3-O-Sulfotransferase 3', "Gal-Beta-1, 3-GalNAc 3'-Sulfotransferase 3", 'Beta-Galactose-3-O-Sulfotransferase 3', 'GAL3ST2', "Galbeta1-3GalNAc 3'-Sulfotransferase 3", "Galactose 3'-Sulfotransferase", 'EC 2.8.2.-', 'GAL3ST-3', 'Gal3ST-3', 'GAL3ST3', 'Gal3ST3'], ['Galactose-3-O-Sulfotransferase 4', "Gal-Beta-1,3-GalNAc 3'-Sulfotransferase", 'Beta-Galactose-3-O-Sulfotransferase 4', 'EC 2.8.2.-', 'GAL3ST-4', 'FLJ12116', 'Gal3ST-4', 'GAL3ST4'], ['Galactosylceramidase', 'Galactocerebroside Beta-Galactosidase', 'Galactosylceramide Beta-Galactosidase', 'Galactocerebrosidase', 'EC 3.2.1.46', 'GALCERase', 'Testis Tissue Sperm-Binding Protein Li 88E', 'Testis Tissue Sperm-Binding Protein Li 89A', 'Galactosylceramidase (Krabbe Disease)', 'Galactosylceraminidase', 'Krabbe Disease', 'GALC'], ['UDP-Galactose-4-Epimerase', 'UDP-Glucose 4-Epimerase', 'Short Chain Dehydrogenase/Reductase Family 1E, Member 1', 'UDP-N-Acetylgalactosamine 4-Epimerase', 'UDP-N-Acetylglucosamine 4-Epimerase', 'Galactose-4-Epimerase, UDP-', 'UDP-GalNAc 4-Epimerase', 'UDP-GlcNAc 4-Epimerase', 'Galactowaldenase', 'EC 5.1.3.2', 'SDR1E1', 'Epididymis Secretory Sperm Binding Protein', "UDP Galactose-4'-Epimerase", 'UDP-Galactose 4-Epimerase', 'EC 5.1.3.7', 'EC 5.1.3', 'GALE'], ['Galactokinase 1', 'Galactose Kinase', 'Galactokinase', 'EC 2.7.1.6', 'GALK', 'Epididymis Secretory Protein Li 19', 'HEL-S-19', 'GALK1', 'GK1'], ['Galactokinase 2', 'GK2', 'N-Acetylgalactosamine Kinase', 'GalNAc Kinase', 'EC 2.7.1.157', 'EC 2.7.1', 'GALK2'], ['Galactose Mutarotase', 'Aldose 1-Epimerase', 'Aldose Mutarotase', 'Galactomutarotase', 'EC 5.1.3.3', 'Epididymis Secretory Sperm Binding Protein Li 63p', 'Galactose Enzyme Activator', 'HEL-S-63p', 'BLOCK25', 'GALAC4', 'GLAT', 'IBD1', 'GALM'], ['Galactosamine (N-Acetyl)-6-Sulfatase', 'Galactosamine (N-Acetyl)-6-Sulfate Sulfatase', 'N-Acetylgalactosamine-6-Sulfate Sulfatase', 'N-Acetylgalactosamine-6-Sulfatase', 'Galactose-6-Sulfate Sulfatase', 'Chondroitinsulfatase', 'GalNAc6S Sulfatase', 'Chondroitinase', 'EC 3.1.6.4', 'GALNAC6S', 'GalN6S', 'GAS', 'Mucopolysaccharidosis Type IVA', 'Morquio Syndrome', 'EC 3.1.6', 'MPS4A', 'GALNS'], ['Polypeptide N-Acetylgalactosaminyltransferase 1', 'Protein-UDP Acetylgalactosaminyltransferase 1', 'Polypeptide GalNAc Transferase 1', 'UDP-N-Acetyl-Alpha-D-Galactosamine:Polypeptide N-Acetylgalactosaminyltransferase 1 (GalNAc-T1)', 'UDP-GalNAc:Polypeptide N-Acetylgalactosaminyltransferase 1', 'Pp-GaNTase 1', 'EC 2.4.1.41', 'GalNAc-T1', 'GalNAc Transferase 1', 'GALNAC-T1', 'GALNT1'], ['Polypeptide N-Acetylgalactosaminyltransferase 2', 'Polypeptide GalNAc Transferase 2', 'GalNAc-T2', 'UDP-N-Acetyl-Alpha-D-Galactosamine:Polypeptide N-Acetylgalactosaminyltransferase 2 (GalNAc-T2)', 'UDP-GalNAc:Polypeptide N-Acetylgalactosaminyltransferase 2', 'Pp-GaNTase 2', 'EC 2.4.1.41', 'Protein-UDP Acetylgalactosaminyltransferase 2', 'GALNT2', 'CDG2T'], ['Polypeptide N-Acetylgalactosaminyltransferase 3', 'Polypeptide GalNAc Transferase 3', 'GalNAc-T3', 'UDP-N-Acetyl-Alpha-D-Galactosamine:Polypeptide N-Acetylgalactosaminyltransferase 3 (GalNAc-T3)', 'UDP-GalNAc:Polypeptide N-Acetylgalactosaminyltransferase 3', 'Protein-UDP Acetylgalactosaminyltransferase 3', 'Pp-GaNTase 3', 'EC 2.4.1.41', 'HFTC', 'HHS', 'Polypeptide GalNAc-Transferase T3', 'GalNAc Transferase 3', 'GALNT3', 'HFTC1'], ['Polypeptide N-Acetylgalactosaminyltransferase 4', 'Polypeptide GalNAc Transferase 4', 'UDP-N-Acetyl-Alpha-D-Galactosamine:Polypeptide N-Acetylgalactosaminyltransferase 4 (GalNAc-T4)', 'UDP-GalNAc:Polypeptide N-Acetylgalactosaminyltransferase 4', 'Protein-UDP Acetylgalactosaminyltransferase 4', 'Pp-GaNTase 4', 'EC 2.4.1.41', 'GalNAc-T4', 'GALNAC-T4', 'GALNACT4', 'GALNT4'], ['Polypeptide N-Acetylgalactosaminyltransferase 5', 'Polypeptide GalNAc Transferase 5', 'UDP-N-Acetyl-Alpha-D-Galactosamine:Polypeptide N-Acetylgalactosaminyltransferase 5 (GalNAc-T5)', 'UDP-GalNAc:Polypeptide N-Acetylgalactosaminyltransferase 5', 'Protein-UDP Acetylgalactosaminyltransferase 5', 'Pp-GaNTase 5', 'EC 2.4.1.41', 'GalNAc-T5', 'GALNAC-T5', 'GALNACT5', 'GALNT5'], ['Polypeptide N-Acetylgalactosaminyltransferase 6', 'Polypeptide GalNAc Transferase 6', 'UDP-N-Acetyl-Alpha-D-Galactosamine:Polypeptide N-Acetylgalactosaminyltransferase 6 (GalNAc-T6)', 'UDP-GalNAc:Polypeptide N-Acetylgalactosaminyltransferase 6', 'Protein-UDP Acetylgalactosaminyltransferase 6', 'Pp-GaNTase 6', 'EC 2.4.1.41', 'GalNAc-T6', 'UDP-N-Acetyl-Alpha-D-Galactosamine: Polypeptide N-Acetylgalactosaminyltransferase 6', 'GalNAc Transferase 6', 'GALNAC-T6', 'GalNAcT6', 'GALNT6'], ['Polypeptide N-Acetylgalactosaminyltransferase 7', 'Polypeptide GalNAc Transferase 7', 'UDP-N-Acetyl-Alpha-D-Galactosamine:Polypeptide N-Acetylgalactosaminyltransferase 7 (GalNAc-T7)', 'UDP-GalNAc:Polypeptide N-Acetylgalactosaminyltransferase 7', 'Protein-UDP Acetylgalactosaminyltransferase 7', 'N-Acetylgalactosaminyltransferase 7', 'Pp-GaNTase 7', 'EC 2.4.1.41', 'GALNAC-T7', 'EC 2.4.1.37', 'GalNAc-T7', 'GalNAcT7', 'EC 2.4.1', 'GALNT7'], ['Polypeptide N-Acetylgalactosaminyltransferase 8', 'Polypeptide GalNAc Transferase 8', 'UDP-N-Acetyl-Alpha-D-Galactosamine:Polypeptide N-Acetylgalactosaminyltransferase 8 (GalNAc-T8)', 'Probable Polypeptide N-Acetylgalactosaminyltransferase 8', 'Protein-UDP Acetylgalactosaminyltransferase 8', 'Pp-GaNTase 8', 'EC 2.4.1.41', 'GALNAC-T8', 'UDP-GalNAc: Polypeptide N-Acetylgalactosaminyltransferase 8', 'UDP-GalNAc:Polypeptide N-Acetylgalactosaminyltransferase 8', 'GalNAc Transferase 8', 'GalNAc-T8', 'GALNT8'], ['Polypeptide N-Acetylgalactosaminyltransferase 9', 'Polypeptide GalNAc Transferase 9', 'UDP-N-Acetyl-Alpha-D-Galactosamine:Polypeptide N-Acetylgalactosaminyltransferase 9 (GalNAc-T9)', 'Protein-UDP Acetylgalactosaminyltransferase 9', 'Pp-GaNTase 9', 'EC 2.4.1.41', 'GALNAC-T9', 'UDP-GalNAc: Polypeptide N-Acetylgalactosaminyltransferase 9', 'UDP-GalNAc:Polypeptide N-Acetylgalactosaminyltransferase 9', 'GalNAc Transferase 9', 'GalNAc-T9', 'GALNACT9', 'GALNT9'], ['Polypeptide N-Acetylgalactosaminyltransferase 10', 'UDP-N-Acetyl-Alpha-D-Galactosamine:Polypeptide N-Acetylgalactosaminyltransferase 10 (GalNAc-T10)', 'UDP-GalNAc:Polypeptide N-Acetylgalactosaminyltransferase 10', 'Polypeptide GalNAc Transferase 10', 'EC 2.4.1.41', 'GalNAc-T10', 'Protein-UDP Acetylgalactosaminyltransferase 10', 'GalNAc Transferase 10', 'Pp-GaNTase 10', 'PPGALNACT10', 'PPGANTASE10', 'GALNACT10', 'GALNT10'], ['Polypeptide N-Acetylgalactosaminyltransferase 11', 'Polypeptide GalNAc Transferase 11', 'UDP-N-Acetyl-Alpha-D-Galactosamine:Polypeptide N-Acetylgalactosaminyltransferase 11 (GalNAc-T11)', 'UDP-GalNAc:Polypeptide N-Acetylgalactosaminyltransferase 11', 'Protein-UDP Acetylgalactosaminyltransferase 11', 'Pp-GaNTase 11', 'EC 2.4.1.41', 'GalNAc-T11', 'GALNAC-T11', 'GALNACT11', 'GALNT11'], ['Polypeptide N-Acetylgalactosaminyltransferase 12', 'Polypeptide GalNAc Transferase 12', 'GalNAc-T12', 'UDP-N-Acetyl-Alpha-D-Galactosamine:Polypeptide N-Acetylgalactosaminyltransferase 12 (GalNAc-T12)', 'UDP-GalNAc:Polypeptide N-Acetylgalactosaminyltransferase 12', 'Protein-UDP Acetylgalactosaminyltransferase 12', 'Pp-GaNTase 12', 'EC 2.4.1.41', 'Colorectal Cancer, Susceptibility To 1', 'GALNT12', 'CRCS1'], ['Polypeptide N-Acetylgalactosaminyltransferase 13', 'Polypeptide GalNAc Transferase 13', 'GalNAc-T13', 'UDP-N-Acetyl-Alpha-D-Galactosamine:Polypeptide N-Acetylgalactosaminyltransferase 13 (GalNAc-T13)', 'UDP-GalNAc:Polypeptide N-Acetylgalactosaminyltransferase 13', 'Protein-UDP Acetylgalactosaminyltransferase 13', 'Pp-GaNTase 13', 'EC 2.4.1.41', 'KIAA1918', 'Polypeptide N-Acetylgalactosaminyltransferase 13 Variant DeltaEx6 DeltaEx8 Delta39bpEx9', 'UDP-N-Acetyl-Alpha-D-Galactosamine:Polypeptide N-Acetylgalactosaminyltransferase 13', 'Polypeptide N-Acetylgalactosaminyltransferase 13 Variant DeltaEx6 Delta39bpEx9', 'Polypeptide N-Acetylgalactosaminyltransferase 13 Variant Delta39bpEx9', 'Polypeptide N-Acetylgalactosaminyltransferase 13 Variant DeltaEx2-7', 'Polypeptide N-Acetylgalactosaminyltransferase 13 Variant DeltaEx8', 'Polypeptide N-Acetylgalactosaminyltransferase 13 Variant DeltaEx9', 'GalNAc Transferase 13', 'GALNT13'], ['Polypeptide N-Acetylgalactosaminyltransferase 14', 'Polypeptide GalNAc Transferase 14', 'UDP-N-Acetyl-Alpha-D-Galactosamine:Polypeptide N-Acetylgalactosaminyltransferase 14 (GalNAc-T14)', 'UDP-GalNAc:Polypeptide N-Acetylgalactosaminyltransferase 14', 'Protein-UDP Acetylgalactosaminyltransferase 14', 'Pp-GaNTase 14', 'EC 2.4.1.41', 'GalNac-T10', 'GalNac-T14', 'UDP-GalNAc:Polypeptide N-Acetylgalactosaminyltransferase T10', 'Polypeptide N-Acetylgalactosaminyltransferase 15', 'GalNAc-T14', 'FLJ12691', 'GALNT15', 'GALNT14'], ['Polypeptide N-Acetylgalactosaminyltransferase 15', 'UDP-N-Acetyl-Alpha-D-Galactosamine:Polypeptide N-Acetylgalactosaminyltransferase-Like 2', 'UDP-N-Acetyl-Alpha-D-Galactosamine:Polypeptide N-Acetylgalactosaminyltransferase 15', 'UDP-GalNAc:Polypeptide N-Acetylgalactosaminyltransferase-Like Protein 2', 'Polypeptide N-Acetylgalactosaminyltransferase-Like Protein 2', 'Protein-UDP Acetylgalactosaminyltransferase-Like Protein 2', 'Polypeptide GalNAc Transferase-Like Protein 2', 'Polypeptide GalNAc Transferase 15', 'Pp-GaNTase-Like Protein 2', 'GalNAc-T-Like Protein 2', 'Pp-GalNAc-T15', 'GALNTL2', 'GALNT7', 'UDP-N-Acetyl-Alpha-D-Galactosamine:Polypeptide N-Acetylgalactosaminyltransferase 7', 'Pregnancy-Induced Hypertension Syndrome-Related Protein 5', 'Polypeptide N-Acetylgalactosaminyltransferase 13', 'UDP-GalNAc Transferase T15', 'EC 2.4.1.41', 'GALNACT15', 'GALNT13', 'GALNT15', 'PIH5'], ['Polypeptide N-Acetylgalactosaminyltransferase 16', 'Polypeptide GalNAc Transferase 16', 'GalNAc-T16', 'UDP-N-Acetyl-Alpha-D-Galactosamine:Polypeptide N-Acetylgalactosaminyltransferase-Like 1', 'UDP-N-Acetyl-Alpha-D-Galactosamine:Polypeptide N-Acetylgalactosaminyltransferase 16', 'UDP-GalNAc:Polypeptide N-Acetylgalactosaminyltransferase-Like Protein 1', 'Polypeptide N-Acetylgalactosaminyltransferase-Like Protein 1', 'Protein-UDP Acetylgalactosaminyltransferase-Like Protein 1', 'Polypeptide GalNAc Transferase-Like Protein 1', 'Pp-GaNTase-Like Protein 1', 'GalNAc-T-Like Protein 1', 'KIAA1130', 'GALNTL1', 'Putative Polypeptide N-Acetylgalactosaminyltransferase-Like Protein 1', 'EC 2.4.1.41', 'GALNACT16', 'GALNT16'], ['Polypeptide N-Acetylgalactosaminyltransferase 17', 'UDP-GalNAc:Polypeptide N-Acetylgalactosaminyltransferase-Like Protein 3', 'Protein-UDP Acetylgalactosaminyltransferase-Like Protein 3', 'Williams-Beuren Syndrome Chromosomal Region 17 Protein', 'Polypeptide N-Acetylgalactosaminyltransferase-Like 3', 'Williams-Beuren Syndrome Chromosome Region 17', 'Polypeptide GalNAc Transferase-Like Protein 3', 'Polypeptide GalNAc Transferase 3', 'Pp-GaNTase-Like Protein 3', 'GalNAc-T-Like Protein 3', 'PpGalNAc-T17', 'EC 2.4.1.41', 'GalNAc-T17', 'GalNAc-T19', 'GalNAc-T5L', 'GALNTL3', 'WBSCR17', 'UDP-N-Acetyl-Alpha-D-Galactosamine:Polypeptide N-Acetylgalactosaminyltransferase 16', 'Putative Polypeptide N-Acetylgalactosaminyltransferase-Like Protein 3', 'UDP-GalNAc:Polypeptide N-Acetylgalactosaminyltransferase-Like 3', 'Polypeptide N-Acetylgalactosaminyltransferase WBSCR17', 'Pp-GalNAc-Transferase 20', 'GALNACT17', 'GALNT16', 'GALNT20', 'GALNT17'], ['Polypeptide N-Acetylgalactosaminyltransferase 18', 'Polypeptide GalNAc Transferase 18', 'GalNAc-T18', 'UDP-N-Acetyl-Alpha-D-Galactosamine:Polypeptide N-Acetylgalactosaminyltransferase-Like 4', 'UDP-N-Acetyl-Alpha-D-Galactosamine:Polypeptide N-Acetylgalactosaminyltransferase 18', 'UDP-GalNAc:Polypeptide N-Acetylgalactosaminyltransferase-Like Protein 4', 'Polypeptide N-Acetylgalactosaminyltransferase-Like Protein 4', 'Protein-UDP Acetylgalactosaminyltransferase-Like Protein 4', 'Polypeptide GalNAc Transferase-Like Protein 4', 'Pp-GaNTase-Like Protein 4', 'GalNAc-T-Like Protein 4', 'GALNT15', 'GALNTL4', 'UDP-N-Acetyl-Alpha-D-Galactosamine:Polypeptide N-Acetylgalactosaminyltransferase 15', 'Putative Polypeptide N-Acetylgalactosaminyltransferase-Like Protein 4', 'GalNAc-Transferase 18', 'EC 2.4.1.41', 'GalNAc-T15', 'GALNACT18', 'MGC71806', 'GALNT18'], ['Polypeptide N-Acetylgalactosaminyltransferase Like 5', 'UDP-N-Acetyl-Alpha-D-Galactosamine:Polypeptide N-Acetylgalactosaminyltransferase-Like 5', 'Inactive Polypeptide N-Acetylgalactosaminyltransferase-Like Protein 5', 'UDP-GalNAc:Polypeptide N-Acetylgalactosaminyltransferase 15', 'Protein-UDP Acetylgalactosaminyltransferase 15', 'Polypeptide GalNAc Transferase-Like 5', 'Polypeptide GalNAc Transferase 15', 'Pp-GaNTase 15', 'GalNAc-T5L', 'GalNAc-T15', 'GALNT15', 'UDP-N-Acetyl-Alpha-D-Galactosamine:Polypeptide N-Acetylgalactosaminyltransferase 15', 'Putative Polypeptide N-Acetylgalactosaminyltransferase-Like Protein 5', 'Testis Tissue Sperm-Binding Protein Li 61n', 'EC 2.4.1.41', 'GALNACT19', 'GALNTL5'], ['Polypeptide N-Acetylgalactosaminyltransferase Like 6', 'GALNT17', 'UDP-N-Acetyl-Alpha-D-Galactosamine:Polypeptide N-Acetylgalactosaminyltransferase-Like 6', 'UDP-GalNAc:Polypeptide N-Acetylgalactosaminyltransferase 17', 'Putative Polypeptide N-Acetylgalactosaminyltransferase 17', 'Polypeptide N-Acetylgalactosaminyltransferase-Like 6', 'Protein-UDP Acetylgalactosaminyltransferase 17', 'Polypeptide GalNAc Transferase-Like 6', 'Polypeptide GalNAc Transferase 17', 'Pp-GaNTase 17', 'EC 2.4.1.41', 'GalNAc-T6L', 'GalNAc-T17', 'UDP-N-Acetyl-Alpha-D-Galactosamine:Polypeptide N-Acetylgalactosaminyltransferase 20', 'GalNAc Transferase 17', 'GaNTase 17', 'GALNACT20', 'GALNTL6'], ['Galanin Like Peptide', 'Galanin-Like Peptide', 'Gal-Like Peptide', 'Alarin', 'GALP'], ['Galanin Receptor 1', 'Galanin Receptor Type 1', 'GALNR1', 'GAL1-R', 'GALR-1', 'GALNR', 'GALR1'], ['Galanin Receptor 2', 'GALNR2', 'Galanin Receptor Type 2', 'GAL2-R', 'GALR-2', 'GALR2'], ['Galanin Receptor 3', 'Galanin Receptor Type 3', 'GAL3-R', 'GALR-3', 'Galanin Receptor, Family Member 3', 'GALNR3', 'GALR3'], ['Galactose-1-Phosphate Uridylyltransferase', 'UDP-Glucose--Hexose-1-Phosphate Uridylyltransferase', 'Gal-1-P Uridylyltransferase', 'Galactose-1-Phosphate Uridyl Transferase', 'EC 2.7.7.12', 'GALT'], ['Guanidinoacetate N-Methyltransferase', 'EC 2.1.1.2', 'TP53I2', 'PIG2', 'Epididymis Secretory Protein Li 20', 'HEL-S-20', 'CCDS2', 'GAMT'], ['Gigaxonin', 'KLHL16', 'GAN1', 'Kelch-Like Family Member 16', 'Kelch-Like Protein 16', 'GIG', 'Epididymis Secretory Sperm Binding Protein', 'Giant Axonal Neuropathy (Gigaxonin)', 'GAN'], ['Glucosidase II Alpha Subunit', 'Neutral Alpha-Glucosidase AB', 'G2AN', 'Alpha-Glucosidase 2', 'KIAA0088', 'GIIA', 'Glucosidase, Alpha; Neutral AB', 'Glucosidase II Subunit Alpha', 'EC 3.2.1.207', 'EC 3.2.1.84', 'EC 3.2.1', 'GLUII', 'GANAB', 'GluII', 'PKD3'], ['Glucosidase Alpha, Neutral C', 'Neutral Alpha-Glucosidase C', 'Glucosidase, Alpha; Neutral C', 'EC 3.2.1.20', 'EC 3.2.1', 'GANC'], ['Growth Associated Protein 43', 'Axonal Membrane Protein GAP-43', 'Neural Phosphoprotein B-50', 'Neuromodulin', 'Neuron Growth-Associated Protein 43', 'Nerve Growth-Related Peptide GAP43', 'Calmodulin-Binding Protein P-57', 'Protein F1', 'GAP-43', 'B-50', 'PP46', 'Growth-Associated Protein 43', 'GAP43', 'Pp46'], ['Glyceraldehyde-3-Phosphate Dehydrogenase', 'Peptidyl-Cysteine S-Nitrosylase GAPDH', 'EC 1.2.1.12', 'GAPD', 'Epididymis Secretory Sperm Binding Protein Li 162eP', 'Oct1 Coactivator In S Phase, 38 Kd Component', 'Aging-Associated Gene 9 Protein', 'OCAS, P38 Component', 'HEL-S-162eP', 'EC 2.6.99.-', 'EC 1.2.1', 'GAPDH', 'G3PD'], ['Glyceraldehyde-3-Phosphate Dehydrogenase, Spermatogenic', 'GAPDH-2', 'GAPD2', 'Spermatogenic Cell-Specific Glyceraldehyde 3-Phosphate Dehydrogenase 2', 'Glyceraldehyde-3-Phosphate Dehydrogenase, Testis-Specific', 'Spermatogenic Glyceraldehyde-3-Phosphate Dehydrogenase', 'EC 1.2.1.12', 'GAPDS', 'Epididymis Secretory Protein Li 278', 'HEL-S-278', 'EC 1.2.1', 'HSD-35', 'GAPDHS', 'GAPDH2'], ['GRB2 Binding Adaptor Protein, Transmembrane', 'Growth Factor Receptor-Bound Protein 2-Binding Adapter Protein, Transmembrane', 'GRB2-Binding Transmembrane Adaptor', 'Protein GAPT', 'C5orf29', 'Chromosome 5 Open Reading Frame 29', 'FLJ33641', 'GAPT'], ['GTPase Activating Protein And VPS9 Domains 1', 'GTPase-Activating Protein And VPS9 Domain-Containing Protein 1', 'Rab5-Activating Protein 6', 'KIAA1521', 'GAPex-5', 'GAPEX5', 'RAP6', 'Rab5 Exchange Factor', 'DKFZP434C212', 'GAPVD1'], ['GAR1 Ribonucleoprotein', 'Nucleolar Protein Family A, Member 1 (H/ACA Small Nucleolar RNPs)', 'H/ACA Ribonucleoprotein Complex Subunit 1', 'Nucleolar Protein Family A Member 1', 'SnoRNP Protein GAR1', 'NOLA1', 'GAR1 Ribonucleoprotein Homolog (Yeast)', 'GAR1 Homolog, Ribonucleoprotein', 'GAR1 Ribonucleoprotein Homolog', 'GAR1'], ['GRB2 Associated Regulator Of MAPK1 Subtype 1', 'GRB2-Associated And Regulator Of MAPK Protein 1', 'Family With Sequence Similarity 59, Member A', 'Grb2-Associated And Regulator Of Erk/MAPK', 'GRB2-Associated And Regulator Of MAPK1', 'GRB2 Associated, Regulator Of MAPK1', 'C18orf11', 'FAM59A', 'GAREM', 'GRB2 Associated Regulator Of MAPK1 1', 'Chromosome 18 Open Reading Frame 11', 'Protein FAM59A', 'FLJ21610', 'GAREM1', 'Gm944'], ['GRB2 Associated Regulator Of MAPK1 Subtype 2', 'GRB2-Associated And Regulator Of MAPK Protein 2', 'Family With Sequence Similarity 59, Member B', 'GRB2-Associated And Regulator Of MAPK1-Like', 'GRB2 Associated, Regulator Of MAPK1-Like', 'KIAA2038', 'FAM59B', 'GAREML', 'GRB2-Associated And Regulator Of MAPK Protein-Like', 'GRB2 Associated Regulator Of MAPK1 2', 'Protein FAM59B', 'FLJ00375', 'GAREM2'], ['GTPase Activating Rap/RanGAP Domain Like 3', 'GTPase-Activating Rap/Ran-GAP Domain-Like Protein 3', 'GTPase Activating RANGAP Domain-Like 3', 'BA356B19.1', 'GTPase Activating Rap/RanGAP Domain-Like 3', 'DKFZp761J1523', 'GARNL3'], ['Granule Associated Rac And RHOG Effector 1', 'KIAA0355', 'Granule Associated Rac And RHOG Effector Protein 1', 'GARRE1'], ['Glycyl-TRNA Synthetase 1', 'GlyRS', 'Diadenosine Tetraphosphate Synthetase', 'Charcot-Marie-Tooth Neuropathy 2D', 'Glycyl-TRNA Synthetase', 'Glycine--TRNA Ligase', 'Ap4A Synthetase', 'EC 6.1.1.14', 'DSMAV', 'SMAD1', 'GARS', 'Charcot-Marie-Tooth Neuropathy, Neuronal Type, D', 'Glycine TRNA Ligase', 'AP-4-A Synthetase', 'EC 2.7.7.-', 'CMT2D', 'HMN5A', 'SMAJI', 'GARS1', 'HMN5'], ['Phosphoribosylglycinamide Formyltransferase, Phosphoribosylglycinamide Synthetase, Phosphoribosylaminoimidazole Synthetase', 'Trifunctional Purine Biosynthetic Protein Adenosine-3', 'Glycinamide Ribonucleotide Synthetase-Aminoimidazole Ribonucleotide Synthetase-Glycinamide Ribonucleotide Transformylase', 'Glycinamide Ribonucleotide Formyltransferase', 'GAR Transformylase', 'GARS-AIRS-GART', 'PGFT', 'PRGS', 'EC 6.3.4.13', 'EC 6.3.3.1', 'GARTF', 'AIRS', 'GARS', 'PAIS', 'GART'], ['Growth Arrest Specific 1', 'Growth Arrest-Specific Protein 1', 'Growth Arrest-Specific Gene-1', 'GAS-1', 'GAS1'], ['Growth Arrest Specific 2', 'Growth Arrest-Specific Protein 2', 'GAS-2', 'GAS2'], ['Growth Arrest Specific 2 Like 1', 'GAR22', 'Growth Arrest-Specific Protein 2-Like 1', 'GAS2-Related Protein On Chromosome 22', 'GAS2-Like Protein 1', 'Epididymis Secretory Sperm Binding Protein', 'GAS2L1'], ['Growth Arrest Specific 2 Like 2', 'GAR17', 'GAS2-Related Protein On Chromosome 17', 'GAS2-Like Protein 2', 'Growth Arrest-Specific Protein 2-Like 2', 'CILD41', 'GAS2L2'], ['Growth Arrest Specific 2 Like 3', 'Growth Arrest-Specific Protein 2-Like 3', 'GAS2-Like Protein 3', 'GAS2L3', 'G2L3'], ['Growth Arrest Specific 6', 'AXL Receptor Tyrosine Kinase Ligand', 'Growth Arrest-Specific Protein 6', 'AXL Stimulatory Factor', 'AXLLG', 'AXSF', 'DKFZp666G247', 'FLJ34709', 'GAS-6', 'GAS6'], ['Growth Arrest Specific 7', 'Growth Arrest-Specific Protein 7', 'KIAA0394', 'MGC1348', 'GAS-7', 'GAS7'], ['Growth Arrest Specific 8', 'DRC4', 'Dynein Regulatory Complex Subunit 4', 'Growth Arrest-Specific Protein 11', 'Growth Arrest-Specific Protein 8', 'GAS-11', 'GAS11', 'Epididymis Secretory Sperm Binding Protein', 'Growth Arrest-Specific 11', 'CILD33', 'GAS-8', 'GAS8'], ['Golgi Associated Kinase 1A', 'Family With Sequence Similarity 198 Member A', 'Golgi-Associated Kinase 1A', 'Protein FAM198A', 'C3orf41', 'FAM198A', 'Family With Sequence Similarity 198, Member A', 'Chromosome 3 Open Reading Frame 41', 'DKFZP434B172', 'GASK1A'], ['Golgi Associated Kinase 1B', 'Expressed In Nerve And Epithelium During Development', 'ENED', 'Family With Sequence Similarity 198 Member B', 'Golgi-Associated Kinase 1B', 'Protein FAM198B', 'C4orf18', 'FAM198B', 'Family With Sequence Similarity 198, Member B', 'Chromosome 4 Open Reading Frame 18', 'DKFZp434L142', 'FLJ38155', 'GASK1B', 'AD021', 'AD036'], ['Gastrin', 'Preprogastrin', 'GAS', 'GAST'], ['GATA Binding Protein 1', 'GATA-1', 'ERYF1', 'Erythroid Transcription Factor', 'Globin Transcription Factor 1', 'Nuclear Factor, Erythroid 1', 'NF-E1 DNA-Binding Protein', 'GATA-Binding Factor 1', 'NF-E1', 'GF-1', 'NFE1', 'GF1', 'GATA-Binding Protein 1 (Globin Transcription Factor 1)', 'Erythroid Transcription Factor 1', 'Transcription Factor GATA1', 'XLANP', 'XLTDA', 'GATA1', 'Eryf1', 'XLTT'], ['GATA Binding Protein 2', 'Endothelial Transcription Factor GATA-2', 'GATA-Binding Protein 2', 'NFE1B', 'MONOMAC', 'IMD21', 'GATA2', 'DCML'], ['GATA Binding Protein 3', 'Trans-Acting T-Cell-Specific Transcription Factor GATA-3', 'GATA-Binding Factor 3', 'HDR', 'GATA-Binding Protein 3', 'GATA3', 'HDRS'], ['GATA Binding Protein 4', 'Transcription Factor GATA-4', 'GATA-Binding Factor 4', 'GATA-Binding Protein 4', 'TACHD', 'GATA4', 'ASD2', 'VSD1', 'TOF'], ['GATA Binding Protein 5', 'Transcription Factor GATA-5', 'BB379O24.1', 'GATAS', 'GATA-Binding Protein 5', 'GATA Binding Factor-5', 'GATA-Binding Factor 5', 'CHTD5', 'GATA5'], ['GATA Binding Protein 6', 'Transcription Factor GATA-6', 'GATA-Binding Factor 6', 'GATA-Binding Protein 6', 'GATA6'], ['GATA Zinc Finger Domain Containing 1', 'ODAG', 'GATA Zinc Finger Domain-Containing Protein 1', 'Ocular Development-Associated Gene Protein', 'RG083M05.2', 'Ocular Development Associated Gene', 'FLJ22489', 'GATAD1', 'CMD2B'], ['GATA Zinc Finger Domain Containing 2A', 'GATA Zinc Finger Domain-Containing Protein 2A', 'Transcriptional Repressor P66-Alpha', 'P66alpha', 'P66 Alpha', 'Hp66alpha', 'GATAD2A'], ['GATA Zinc Finger Domain Containing 2B', 'Transcription Repressor P66 Beta Component Of The MeCP1 Complex', 'GATA Zinc Finger Domain-Containing Protein 2B', 'Transcriptional Repressor P66-Beta', 'P66beta', 'KIAA1150', 'GATAD2B', 'P66/P68', 'GANDS', 'MRD18', 'P68'], ['Glutamyl-TRNA Amidotransferase Subunit B', 'Glutamyl-TRNA(Gln) Amidotransferase Subunit B, Mitochondrial', 'Cytochrome C Oxidase Assembly Factor PET112 Homolog', 'Glutamyl-TRNA(Gln) Amidotransferase, Subunit B', 'PET112L', 'PET112', 'Cytochrome Oxidase Assembly Factor PET112 Homolog', 'PET112 (Yeast Homolog)-Like', 'Glu-AdT Subunit B', 'PET112 Homolog', 'EC 6.3.5.-', 'EC 6.3.5', 'COXPD41', 'HSPC199', 'GATB'], ['Glutamyl-TRNA Amidotransferase Subunit C', '15E1.2', 'Glutamyl-TRNA(Gln) Amidotransferase Subunit C, Mitochondrial', 'Glu-AdT Subunit C', 'Glutamyl-TRNA(Gln) Amidotransferase, Subunit C Homolog (Bacterial)', 'Glutamyl-TRNA(Gln) Amidotransferase, Subunit C Homolog', 'Protein 15E1.2', 'EC 6.3.5.-', 'FLJ37000', 'COXPD42', 'GATC'], ['Glutamine Amidotransferase Like Class 1 Domain Containing 1', 'Glutamine Amidotransferase-Like Class 1 Domain-Containing Protein 1', 'Parkinson Disease 7 Domain-Containing Protein 1', 'Parkinson Disease 7 Domain Containing 1', 'PDDC1', 'FLJ34283', 'GATD1'], ['Glutamine Amidotransferase Like Class 1 Domain Containing 3A', 'Glutamine Amidotransferase-Like Class 1 Domain-Containing Protein 3A, Mitochondrial', 'C21orf33', 'GT335', 'HES1', 'KNPH', 'KNPI', 'ES1', 'Human HES1 Protein, Homolog To E.Coli And Zebrafish ES1 Protein', 'Testis Secretory Sperm-Binding Protein Li 237E', 'Chromosome 21 Open Reading Frame 33', 'ES1 Protein Homolog, Mitochondrial', 'Keio Novel Protein I', 'ES1 Protein Homolog', 'D21S2048E', 'GATD3A', 'KNP-Ia', 'GATD3', 'KNP-I'], ['Glutamine Amidotransferase Like Class 1 Domain Containing 3B', 'Glutamine Amidotransferase-Like Class 1 Domain-Containing Protein 3B, Mitochondrial', 'Keio Novel Protein-I', 'KNP-I', 'ES1 Protein Homolog, Mitochondrial', 'Protein GT335', 'Protein HES1', 'GATD3B', 'HES1', 'KNPI'], ['Glycine Amidinotransferase', 'AGAT', 'Glycine Amidinotransferase, Mitochondrial', 'L-Arginine:Glycine Amidinotransferase', 'Transamidinase', 'EC 2.1.4.1', 'Glycine Amidinotransferase (L-Arginine:Glycine Amidinotransferase)', 'Testicular Secretory Protein Li 19', 'EC 2.1.4', 'CCDS3', 'FRTS1', 'GATM', 'AT'], ['Glucosylceramidase Beta', 'D-Glucosyl-N-Acylsphingosine Glucohydrolase', 'Lysosomal Acid Glucosylceramidase', 'Cholesterol Glucosyltransferase', 'Cholesteryl-Beta-Glucosidase', 'Glucosidase, Beta, Acid', 'Beta-Glucocerebrosidase', 'Acid Beta-Glucosidase', 'Lysosomal Acid GCase', 'Imiglucerase', 'Alglucerase', 'EC 3.2.1.45', 'Beta-GC', 'SGTase', 'GBA1', 'GLUC', 'Glucosidase, Beta; Acid (Includes Glucosylceramidase)', 'Glucosylceramidase-Like Protein', 'Lysosomal Glucocerebrosidase', 'Glucosylceramidase', 'EC 2.4.1.-', 'EC 3.2.1.-', 'GCB', 'GBA', 'GC'], ['Glucosylceramidase Beta 2', 'Non-Lysosomal Glucosylceramidase', 'Cholesterol Glucosyltransferase GBA2', 'Bile Acid Glucosyl Transferase GBA2', 'Cholesteryl-Beta-Glucosidase GBA2', 'Glucosidase, Beta (Bile Acid) 2', 'Bile Acid Beta-Glucosidase GBA2', 'Beta-Glucocerebrosidase 2', 'Glucosylceramidase 2', 'EC 3.2.1.45', 'KIAA1605', 'NLGase', 'AD035', 'SPG46', 'Spastic Paraplegia 46 (Autosomal Recessive)', 'Bile Acid Beta-Glucosidase', 'Beta-Glucosidase 2', 'DKFZp762K054', 'EC 2.4.1.-', 'EC 3.2.1.-', 'GBA2'], ['Glucosylceramidase Beta 3 (Gene/Pseudogene)', 'Klotho-Related Protein', 'Cytosolic Beta-Glucosidase-Like Protein 1', 'Glucosidase, Beta, Acid 3 (Cytosolic)', 'Cytosolic Glycosylceramidase', 'Cytosolic Beta-Glucosidase', 'Glucosidase Beta Acid 3', 'Cytosolic GCase', 'EC 3.2.1.21', 'CBGL1', 'GLUC', 'KLrP', 'CBG', 'Glucosidase, Beta, Acid 3 (Gene/Pseudogene)', 'Glucosylceramidase Beta 3', 'KLRP', 'GBA3'], ['1,4-Alpha-Glucan Branching Enzyme 1', 'Glucan (1,4-Alpha-), Branching Enzyme 1', '1,4-Alpha-Glucan-Branching Enzyme', 'Glycogen Branching Enzyme', 'Brancher Enzyme', 'EC 2.4.1.18', 'Amylo-(1,4 To 1,6) Transglucosidase', 'Amylo-(1,4 To 1,6) Transglycosylase', 'Glycogen Storage Disease Type IV', 'Glycogen-Branching Enzyme', 'Andersen Disease', 'APBD', 'GSD4', 'GBE1', 'GBE'], ['Golgi Brefeldin A Resistant Guanine Nucleotide Exchange Factor 1', 'Golgi-Specific Brefeldin A-Resistance Guanine Nucleotide Exchange Factor 1', 'BFA-Resistant GEF 1', 'KIAA0248', 'ARF1GEF', 'Golgi-Specific Brefeldin A Resistance Factor 1', 'GBF1'], ['Globoside Alpha-1,3-N-Acetylgalactosaminyltransferase 1 (FORS Blood Group)', 'Globoside Alpha-1,3-N-Acetylgalactosaminyltransferase 1', 'Forssman Glycolipid Synthase-Like Protein', 'Forssman Glycolipid Synthetase (FS)', 'Forssman Blood Group', 'A3GALNT', 'FS', 'Forssman Synthetase', 'UDP-GalNAc', 'EC 2.4.1.-', 'MGC44848', 'UNQ2513', 'GBGT1'], ['Guanylate Binding Protein 1', 'Guanylate Binding Protein 1, Interferon-Inducible, 67kDa', 'Interferon-Induced Guanylate-Binding Protein 1', 'Guanine Nucleotide-Binding Protein 1', 'Guanylate-Binding Protein 1', 'GTP-Binding Protein 1', 'HuGBP-1', 'GBP-1', 'Guanylate Binding Protein 1, Interferon-Inducible', 'EC 3.6.5.-', 'GBP1'], ['Guanylate Binding Protein 2', 'Guanylate Binding Protein 2, Interferon-Inducible', 'Interferon-Induced Guanylate-Binding Protein 2', 'Guanine Nucleotide-Binding Protein 2', 'Guanylate-Binding Protein 2', 'GTP-Binding Protein 2', 'EC 3.6.5.-', 'HuGBP-2', 'GBP-2', 'GBP2'], ['Guanylate Binding Protein 3', 'Guanine Nucleotide-Binding Protein 3', 'Guanylate-Binding Protein 3', 'GTP-Binding Protein 3', 'Guanylate-Binding Protein 3 Delta C', 'EC 3.6.5.-', 'FLJ10961', 'GBP-3', 'GBP3'], ['Guanylate Binding Protein 4', 'Guanine Nucleotide-Binding Protein 4', 'Guanylate-Binding Protein 4', 'GTP-Binding Protein 4', 'GBP-4', 'Mpa2', 'EC 3.6.5.-', 'GBP4'], ['Guanylate Binding Protein 5', 'Guanine Nucleotide-Binding Protein 5', 'Guanylate-Binding Protein 5', 'GTP-Binding Protein 5', 'GBP-TA Antigen', 'GBP-5', 'EC 3.6.5.-', 'GBP5'], ['Guanylate Binding Protein Family Member 6', 'Guanine Nucleotide-Binding Protein 6', 'Guanylate-Binding Protein 6', 'GTP-Binding Protein 6', 'GBP-6', 'Guanylate Binding Protein Family, Member 6', 'DKFZp686G0786', 'GBP6'], ['Guanylate Binding Protein 7', 'GBP4L', 'Guanine Nucleotide-Binding Protein 7', 'Guanylate-Binding Protein 7', 'GTP-Binding Protein 7', 'GBP-7', 'Guanylate-Binding Protein 4-Like', 'FLJ38822', 'GBP7'], ['Gastrulation Brain Homeobox 1', 'Gastrulation And Brain-Specific Homeobox Protein 1', 'Homeobox Protein GBX-1', 'Gastrulation Brain Homeo Box 1', 'Huh-17', 'GBX1'], ['Gastrulation Brain Homeobox 2', 'Gastrulation And Brain-Specific Homeobox Protein 2', 'Homeobox Protein GBX-2', 'Gastrulation Brain Homeo Box 2', 'GBX2'], ['GC Vitamin D Binding Protein', 'DBP', 'Vitamin D-Binding Protein-Macrophage Activating Factor', 'Group-Specific Component (Vitamin D Binding Protein)', 'Gc Protein-Derived Macrophage Activating Factor', 'Vitamin D-Binding Protein', 'Gc-Globulin', 'DBP-Maf', 'Gc-MAF', 'GcMAF', 'VDBP', 'VDB', 'Epididymis Secretory Protein Li 51', 'Vitamin D-Binding Alpha-Globulin', 'GC, Vitamin D Binding Protein', 'Group-Specific Component', 'EC 6.3.1.5', 'HEL-S-51', 'DBP/GC', 'GRD3', 'VDBG', 'HDBP', 'GC', 'Gc'], ['Grancalcin', 'GCL', 'Grancalcin, EF-Hand Calcium-Binding Protein', 'Grancalcin, EF-Hand Calcium Binding Protein', 'Grancalcin, Penta-EF-Hand Protein', 'GCA'], ['Glycine C-Acetyltransferase', 'KBL', '2-Amino-3-Ketobutyrate Coenzyme A Ligase, Mitochondrial', 'Glycine Acetyltransferase', 'Aminoacetone Synthase', 'EC 2.3.1.29', 'AKB Ligase', 'Glycine C-Acetyltransferase (2-Amino-3-Ketobutyrate Coenzyme A Ligase)', '2-Amino-3-Ketobutyrate Coenzyme A Ligase', '2-Amino-3-Ketobutyrate-CoA Ligase', 'EC 2.3.1', 'GCAT'], ['GRIP And Coiled-Coil Domain Containing 1', 'GRIP And Coiled-Coil Domain-Containing Protein 1', 'Golgi Coiled-Coil Protein 1', 'Golgi Coiled-Coil 1', 'GCC1P', 'GCC88', 'Peripheral Membrane Golgi Protein', 'FLJ22035', 'MGC20706', 'GCC1'], ['GRIP And Coiled-Coil Domain Containing 2', 'GCC185', 'GRIP And Coiled-Coil Domain-Containing Protein 2', '185 KDa Golgi Coiled-Coil Protein', 'Renal Carcinoma Antigen NY-REN-53', 'CLL-Associated Antigen KW-11', 'Ran-Binding Protein 2-Like 4', 'CTCL Tumor Antigen Se1-1', 'RANBP2L4', 'KIAA0336', 'GRIP And Coiled-Coil Domain-Containing 2', 'Golgi Coiled-Coil Protein GCC185', 'GCC Protein, 185-KD', 'RanBP2L4', 'REN53', 'GCC2'], ['Glutaryl-CoA Dehydrogenase', 'GCD', 'Glutaryl-CoA Dehydrogenase, Mitochondrial', 'Glutaryl-Coenzyme A Dehydrogenase', 'ACAD5', 'Truncated Glutaryl-CoA Dehydrogenase', 'Truncated GCDH', 'EC 1.3.99.7', 'EC 1.3.8.6', 'EC 1.3.99', 'GCDH'], ['GC-Rich Sequence DNA-Binding Factor 2', 'GCF', 'Transcription Factor 9 (Binds GC-Rich Sequences)', 'Intron Large Complex Component GCFC2', 'GC Binding Factor', 'C2orf3', 'DNABF', 'TCF-9', 'TCF9', 'GC-Rich Sequence DNA-Binding Factor', 'Chromosome 2 Open Reading Frame 3', 'Transcription Factor 9', 'GC Bindng Factor', 'GCFC2'], ['Glucagon', 'Glicentin-Related Polypeptide', 'Glucagon-Like Peptide 1', 'Glucagon-Like Peptide 2', 'Preproglucagon', 'Pro-Glucagon', 'GLP-1', 'GLP1', 'GLP2', 'GRPP', 'GCG'], ['Glucagon Receptor', 'GL-R', 'GGR', 'GCGR'], ['GTP Cyclohydrolase 1', 'GTP Cyclohydrolase I', 'Dystonia 14', 'EC 3.5.4.16', 'GTP-CH-I', 'GTPCH1', 'DYT5a', 'DYT5', 'GCH', "Guanosine 5'-Triphosphate Cyclohydrolase I", 'Dopa-Responsive Dystonia', 'GTP-CH-1', 'HPABH4B', 'DYT14', 'GCH1'], ['GTP Cyclohydrolase I Feedback Regulator', 'GTP Cyclohydrolase I Feedback Regulatory Protein', 'GFRP', 'GTP Cyclohydrolase 1 Feedback Regulatory Protein', 'HsT16933', 'P35', 'GCHFR'], ['Glucokinase', 'HK4', 'Glucokinase (Hexokinase 4)', 'Hexokinase Type IV', 'Hexokinase-4', 'HK IV', 'Maturity Onset Diabetes Of The Young 2', 'ATP:D-Hexose 6-Phosphotransferase', 'Hexokinase D, Pancreatic Isozyme', 'Hexokinase 4', 'Hexokinase-D', 'EC 2.7.1.1', 'EC 2.7.1', 'FGQTL3', 'MODY2', 'PNDM1', 'HHF3', 'HKIV', 'HXKP', 'LGLK', 'GLK', 'GCK', 'GK'], ['Glucokinase Regulator', 'Glucokinase Regulatory Protein', 'Hexokinase 4 Regulator', 'GKRP', 'Glucokinase (Hexokinase 4) Regulatory Protein', 'Glucokinase (Hexokinase 4) Regulator', 'FGQTL5', 'GCKR'], ['Glutamate-Cysteine Ligase Catalytic Subunit', 'Glutamate--Cysteine Ligase Catalytic Subunit', 'Gamma-Glutamylcysteine Synthetase', 'GCS Heavy Chain', 'EC 6.3.2.2', 'Gamma-ECS', 'GLCLC', 'GLCL', 'GCS', 'Glutamate-Cysteine Ligase, Catalytic Subunit', 'GCLC', 'GCL'], ['Glutamate-Cysteine Ligase Modifier Subunit', 'Gamma-Glutamylcysteine Synthetase Regulatory Subunit', 'Glutamate--Cysteine Ligase Regulatory Subunit', 'Gamma-ECS Regulatory Subunit', 'GCS Light Chain', 'GLCLR', 'Glutamate-Cysteine Ligase (Gamma-Glutamylcysteine Synthetase), Regulatory (30.8kD)', 'Glutamate-Cysteine Ligase Modifier Subunit Delta2 Alternative Splicing', 'Glutamate-Cysteine Ligase Regulatory Protein', 'Glutamate-Cysteine Ligase, Modifier Subunit', 'Glutamate--Cysteine Ligase Modifier Subunit', 'Gamma-Glutamylcysteine Synthetase', 'GSC Light Chain', 'GCLM'], ['Glial Cells Missing Transcription Factor 1', 'HGCMa', 'Chorion-Specific Transcription Factor GCMa', 'Glial Cells Missing Homolog 1', 'GCM Motif Protein 1', 'GCMA', 'Glial Cells Missing (Drosophila) Homolog A', 'Glial Cells Missing Homolog 1 (Drosophila)', 'GCM1'], ['Glial Cells Missing Transcription Factor 2', 'HGCMb', 'Chorion-Specific Transcription Factor GCMb', 'Glial Cells Missing Homolog 2', 'GCM Motif Protein 2', 'GCMB', 'Glial Cells Missing (Drosophila) Homolog B', 'Glial Cells Missing Homolog 2 (Drosophila)', 'Glial Cells Missing Homolog B', 'Glide/Gcm Protein Homolog', 'Glial Cells Missing 2', 'HRPT4', 'FIH2', 'GCM2'], ['GCN1 Activator Of EIF2AK4', 'GCN1 (General Control Of Amino-Acid Synthesis 1, Yeast)-Like 1', 'General Control Of Amino-Acid Synthesis 1-Like Protein 1', 'GCN1, EIF2 Alpha Kinase Activator Homolog', 'GCN1 EIF-2-Alpha Kinase Activator Homolog', 'EIF-2-Alpha Kinase Activator GCN1', 'Translational Activator GCN1', 'GCN1-Like Protein 1', 'KIAA0219', 'GCN1L1', 'HsGCN1', 'GCN1L', 'Peroxisome Proliferator Activated Receptor Interacting Complex Protein', 'GCN1 General Control Of Amino-Acid Synthesis 1-Like 1 (Yeast)', 'GCN1 General Control Of Amino-Acid Synthesis 1-Like 1', 'PRIC295', 'GCN1'], ['Germ Cell Nuclear Acidic Peptidase', 'Acidic Repeat-Containing Protein', 'Germ Cell Nuclear Antigen', 'Acidic Repeat Containing', 'NAAR1', 'ACRC', 'Putative Nuclear Protein', 'GCNA'], ['Glucosaminyl (N-Acetyl) Transferase 1', 'C2GNT', 'Glucosaminyl (N-Acetyl) Transferase 1, Core 2 (Beta-1,6-N-Acetylglucosaminyltransferase)', 'Beta-1,3-Galactosyl-O-Glycosyl-Glycoprotein Beta-1,6-N-Acetylglucosaminyltransferase', 'Core 2 Beta1,6 N-Acetylglucosaminyltransferase-I', 'Core2-GlcNAc-Transferase', 'NACGT2', 'NAGCT2', 'Beta-1,3-Galactosyl-O-Glycosyl-Glycoprotein Beta-1,6-N--Acetylglucosaminyltransferase', 'Core 2 Beta-1,6-N-Acetylglucosaminyltransferase I', 'Glucosaminyl (N-Acetyl) Transferase 1, Core 2', 'Beta-1,6-N-Acetylglucosaminyltransferase', 'Core 2 Branching Enzyme', 'Core 2-Branching Enzyme', 'EC 2.4.1.102', 'Core 2 GnT', 'Core 2 GNT', 'C2GNT-L', 'C2GNT1', 'GCNT1', 'G6NT'], ['Glucosaminyl (N-Acetyl) Transferase 2 (I Blood Group)', 'IGNT', 'Glucosaminyl (N-Acetyl) Transferase 2, I-Branching Enzyme (I Blood Group)', 'N-Acetyllactosaminide Beta-1,6-N-Acetylglucosaminyl-Transferase', 'Ii Blood Group', 'BA360O19.2', 'BA421M1.1', 'NACGT1', 'NAGCT1', 'GCNT5', 'ULG3', 'II', 'I Beta-1,6-N-Acetylglucosaminyltransferase', 'Beta-1,6-N-Acetylglucosaminyltransferase 2', 'Glucosaminyl (N-Acetyl) Transferase 5', 'N-Acetylglucosaminyltransferase', 'Unassigned Linkage Group 3', 'Cataract, Congenital', 'I-Branching Enzyme', 'EC 2.4.1.150', 'CTRCT13', 'GCNT2C', 'GCNT2', 'CCAT'], ['Glucosaminyl (N-Acetyl) Transferase 3, Mucin Type', 'C2/4GnT', 'Beta-1,3-Galactosyl-O-Glycosyl-Glycoprotein Beta-1,6-N-Acetylglucosaminyltransferase 3', 'Core 2/Core 4 Beta-1,6-N-Acetylglucosaminyltransferase', 'C2GnT-Mucin Type', 'HC2GnT-M', 'C2GnT-M', 'Mucus-Type Core 2 Beta-1,6-N-Acetylglucosaminyltransferase', 'Beta1,6-N-Acetylglucosaminyltransferase-M', 'Beta1,6GlcNAc-Transferase', 'EC 2.4.1.102', 'EC 2.4.1.148', 'EC 2.4.1.150', 'C24GNT', 'C2GNT2', 'C2GNTM', 'C2GnT2', 'GCNT3', 'GNTM'], ['Glucosaminyl (N-Acetyl) Transferase 4', 'Beta-1,3-Galactosyl-O-Glycosyl-Glycoprotein Beta-1,6-N-Acetylglucosaminyltransferase 4', 'Glucosaminyl (N-Acetyl) Transferase 4, Core 2 (Beta-1,6-N-Acetylglucosaminyltransferase)', 'Core 2 Beta-1,6-N-Acetylglucosaminyltransferase 3', 'Long Intergenic Non-Protein Coding RNA 1336', 'Core2-GlcNAc-Transferase 3', 'Core 2-Branching Enzyme 3', 'C2GNT3', 'Glucosaminyl (N-Acetyl) Transferase 4, Core 2', 'EC 2.4.1.102', 'LINC01336', 'C2GnT3', 'GCNT4'], ['Glucosaminyl (N-Acetyl) Transferase Family Member 7', 'Beta-1,3-Galactosyl-O-Glycosyl-Glycoprotein Beta-1,6-N-Acetylglucosaminyltransferase 7', 'DJ1153D9.2', 'C20orf105', 'Beta 1,6-N-Acetylglucosaminyltransferase', 'Chromosome 20 Open Reading Frame 105', 'EC 2.4.1.150', 'EC 2.4.1.-', 'EC 2.4.1', 'GCNT7', 'Gcnt'], ['Germinal Center Associated Signaling And Motility', 'HGAL', 'Germinal Center-Associated Signaling And Motility Protein', 'Germinal Center B-Cell-Expressed Transcript 2 Protein', 'Germinal Center-Associated Lymphoma Protein', 'Human Germinal Center-Associated Lymphoma', 'Germinal Center Expressed Transcript 2', 'GCET2', 'MGC40441', 'GCAT2', 'GCSAM', 'GAL'], ['Germinal Center Associated Signaling And Motility Like', 'Germinal Center-Associated Signaling And Motility-Like Protein', 'C1orf150', 'Chromosome 1 Open Reading Frame 150', 'FLJ44728', 'GCSAML'], ['Glycine Cleavage System Protein H', 'Lipoic Acid-Containing Protein', 'Glycine Cleavage System Protein H (Aminomethyl Carrier)', 'Glycine Cleavage System H Protein, Mitochondrial', 'Mitochondrial Glycine Cleavage System H-Protein', 'GCSH', 'GCE', 'NKH'], ['Guanine Deaminase', 'Guanine Aminohydrolase', 'Guanine Aminase', 'P51-Nedasin', 'EC 3.5.4.3', 'GAH', 'Cytoplasmic PSD95 Interactor', 'KIAA1258', 'GUANASE', 'NEDASIN', 'Guanase', 'CYPIN', 'GDA'], ['Ganglioside Induced Differentiation Associated Protein 1', 'Ganglioside-Induced Differentiation-Associated Protein 1', 'Charcot-Marie-Tooth Neuropathy 4A', 'CMT4', 'CMTRIA', 'CMT4A', 'GDAP1', 'CMT2K'], ['Ganglioside Induced Differentiation Associated Protein 1 Like 1', 'Ganglioside-Induced Differentiation-Associated Protein 1-Like 1', 'GDAP1-L1', 'Ganglioside Induced Differentiation Associated Protein 1-Like 1', 'DJ995J12.1.1', 'DJ881L22.1', 'GDAP1L1'], ['Ganglioside Induced Differentiation Associated Protein 2', 'Ganglioside-Induced Differentiation-Associated Protein 2', 'MACROD3', 'DJ776P7.1', 'FLJ20142', 'SCAR27', 'GDAP2'], ['Glycerophosphodiester Phosphodiesterase 1', 'MIR16', 'Glycerophosphoinositol Glycerophosphodiesterase GDE1', 'Membrane Interacting Protein Of RGS16', 'RGS16-Interacting Membrane Protein', 'Lysophospholipase D GDE1', 'EC 3.1.4.44', 'Membrane-Interacting Protein Of RGS16', 'EC 3.1.4.-', '363E6.2', 'GDE1'], ['Growth Differentiation Factor 1', 'Embryonic Growth/Differentiation Factor 1', 'Longevity Assurance Gene 1 Protein Homolog 1', 'LAG1 Longevity Assurance Homolog 1', 'Ceramide Synthase 1', 'Protein UOG-1', 'CERS1', 'CHTD6', 'DTGA3', 'LASS1', 'GDF-1', 'DORV', 'LAG1', 'UOG1', 'GDF1', 'RAI'], ['Growth Differentiation Factor 2', 'BMP-9', 'BMP9', 'Growth/Differentiation Factor 2', 'Bone Morphogenetic Protein 9', 'GDF-2', 'HHT5', 'GDF2'], ['Growth Differentiation Factor 3', 'Growth/Differentiation Factor 3', 'MCOPCB6', 'MCOP7', 'GDF-3', 'KFS3', 'GDF3'], ['Growth Differentiation Factor 5', 'BMP14', 'CDMP1', 'Cartilage-Derived Morphogenetic Protein-1', 'Lipopolysaccharide-Associated Protein 4', 'Growth/Differentiation Factor 5', 'Bone Morphogenetic Protein 14', 'LPS-Associated Protein 4', 'Radotermin', 'BMP-14', 'LAP-4', 'Cartilage-Derived Morphogenetic Protein 1', 'DUPANS', 'CDMP-1', 'BDA1C', 'SYM1B', 'SYNS2', 'GDF-5', 'LAP4', 'GDF5', 'OS5'], ['Growth Differentiation Factor 6', 'BMP13', 'Growth/Differentiation Factor 16', 'Growth/Differentiation Factor 6', 'Bone Morphogenetic Protein 13', 'BMP-13', 'KFS1', 'KFS', 'Segmentation Syndrome 1', 'CDMP2', 'SYNS4', 'GDF16', 'GDF-6', 'KFSL', 'SGM1', 'GDF6', 'KFM'], ['Growth Differentiation Factor 7', 'Growth/Differentiation Factor 7', 'BMP12', 'GDF-7', 'GDF7'], ['Growth Differentiation Factor 9', 'Growth/Differentiation Factor 9', 'POF14', 'GDF-9', 'GDF9'], ['Growth Differentiation Factor 10', 'Growth/Differentiation Factor 10', 'Bone Morphogenetic Protein 3B', 'Bone-Inducing Protein', 'BMP-3b', 'BMP3B', 'BIP', 'GDF-10', 'BMP-3B', 'GDF10'], ['Growth Differentiation Factor 11', 'BMP-11', 'BMP11', 'Growth/Differentiation Factor 11', 'Bone Morphogenetic Protein 11', 'GDF-11', 'GDF11', 'VHO'], ['Growth Differentiation Factor 15', 'Prostate Differentiation Factor', 'MIC-1', 'NAG-1', 'PTGFB', 'MIC1', 'PLAB', 'PDF', 'Non-Steroidal Anti-Inflammatory Drug-Activated Gene-1', 'Placental Bone Morphogenetic Protein', 'Growth/Differentiation Factor 15', 'Macrophage Inhibitory Cytokine 1', 'NSAID-Activated Gene 1 Protein', 'NSAID-Regulated Gene 1 Protein', 'Placental TGF-Beta', 'GDF-15', 'NRG-1', 'NSAID (Nonsteroidal Anti-Inflammatory Drug)-Activated Protein 1', 'Macrophage Inhibitory Cytokine-1', 'PTGF-Beta', 'GDF15'], ['GDP Dissociation Inhibitor 1', 'RABGDIA', 'OPHN2', 'Guanosine Diphosphate Dissociation Inhibitor 1', 'Rab GDP-Dissociation Inhibitor, Alpha', 'Rab GDP Dissociation Inhibitor Alpha', 'Mental Retardation, X-Linked 48', 'Oligophrenin-2', 'Protein XAP-4', 'Rab GDI Alpha', 'XAP-4', 'GDI-1', 'GDIL', 'Testis Secretory Sperm-Binding Protein Li 208a', 'Mental Retardation, X-Linked 41', 'FLJ41411', 'RABGD1A', 'MRX41', 'MRX48', 'GDI1', 'XAP4', '1A'], ['GDP Dissociation Inhibitor 2', 'RABGDIB', 'Guanosine Diphosphate Dissociation Inhibitor 2', 'Rab GDP Dissociation Inhibitor Beta', 'Rab GDI Beta', 'GDI-2', 'Epididymis Secretory Sperm Binding Protein Li 46e', 'Rab GDP-Dissociation Inhibitor, Beta', 'Rab GDP-Dissociation', 'HEL-S-46e', 'GDI2'], ['Glial Cell Derived Neurotrophic Factor', 'Astrocyte-Derived Trophic Factor', 'Glial Cell Line-Derived Neurotrophic Factor', 'HFB1-GDNF', 'ATF1', 'ATF2', 'ATF', 'Glial Cell Line Derived Neurotrophic Factor', 'Glial Derived Neurotrophic Factor', 'HSCR3', 'HGDNF', 'GDNF'], ['Glycerophosphodiester Phosphodiesterase Domain Containing 1', 'GDE4', 'Glycerophosphodiester Phosphodiesterase Domain-Containing Protein 1', 'Glycerophosphodiester Phosphodiesterase 4', 'Lysophospholipase D GDPD1', 'EC 3.1.4.-', 'FLJ37451', 'GDPD1'], ['Glycerophosphodiester Phosphodiesterase Domain Containing 2', 'Osteoblast Differentiation Promoting Factor', 'OBDPF', 'GDE3', 'Glycerophosphodiester Phosphodiesterase Domain-Containing Protein 2', 'Glycerophosphoinositol Inositolphosphodiesterase GDPD2', 'Glycerophosphodiester Phosphodiesterase 3', 'EC 3.1.4.43', 'FLJ20207', 'GDPD2'], ['Glycerophosphodiester Phosphodiesterase Domain Containing 3', 'Glycerophosphodiester Phosphodiesterase Domain-Containing Protein 3', 'Glycerophosphodiester Phosphodiesterase 7', 'Lysophospholipase D GDPD3', 'GDE7', 'EC 3.1.4.-', 'MGC4171', 'GDPD3'], ['Glycerophosphodiester Phosphodiesterase Domain Containing 4', 'GDE6', 'Glycerophosphodiester Phosphodiesterase Domain-Containing Protein 4', 'Glycerophosphodiester Phosphodiesterase 6', 'UgpQ', 'GDPD Domain Containing Protein', 'EC 3.1.-.-', 'GDPD4', 'UGPQ'], ['Glycerophosphodiester Phosphodiesterase Domain Containing 5', 'GDE2', 'Glycerophosphodiester Phosphodiesterase Domain-Containing Protein 5', 'Glycerophosphocholine Phosphodiesterase GDPD5', 'Glycerophosphodiester Phosphodiesterase 2', 'Phosphoinositide Phospholipase C GDPD5', 'PP1665', 'EC 3.1.4.11', 'EC 3.1.4.2', 'GDPD5'], ['GDP-D-Glucose Phosphorylase 1', 'C15orf58', 'GDP-D-Glucose Phosphorylase C15orf58', 'Chromosome 15 Open Reading Frame 58', 'EC 2.7.7.78', 'GDPGP1', 'VTC2'], ['GTP Binding Protein Overexpressed In Skeletal Muscle', 'KIR', 'GTP-Binding Mitogen-Induced T-Cell Protein', 'Kinase-Inducible Ras-Like Protein', 'GTP-Binding Protein GEM', 'RAS-Like Protein KIR', 'GTP-Binding Protein Overexpressed In Skeletal Muscle', 'GEM'], ['Gem Nuclear Organelle Associated Protein 2', 'Survival Of Motor Neuron Protein Interacting Protein 1', 'Gem-Associated Protein 2', 'Component Of Gems 2', 'Gemin-2', 'SIP1', 'Survival Of Motor Neuron Protein-Interacting Protein 1', 'Gem (Nuclear Organelle) Associated Protein 2', 'SMN Interacting Protein 1-Delta', 'SMN-Interacting Protein 1', 'SIP1-Delta', 'GEMIN2'], ['Gem Nuclear Organelle Associated Protein 4', 'Component Of Gems 4', 'P97', 'HCC-Associated Protein 1', 'Gem-Associated Protein 4', 'Gemin-4', 'HHRF-1', 'HCAP1', 'HC56', 'Gem (Nuclear Organelle) Associated Protein 4', 'DKFZP434B131', 'DKFZP434D174', 'NEDMCR', 'GEMIN4'], ['Gem Nuclear Organelle Associated Protein 5', 'Gem-Associated Protein 5', 'Gem (Nuclear Organelle) Associated Protein 5', 'EC 5.4.2.1', 'EC 2.7.7.8', 'GEMIN-5', 'GEMIN5', 'Gemin5'], ['Gem Nuclear Organelle Associated Protein 6', 'Gem-Associated Protein 6', 'Gemin-6', 'SIP2', 'Gem (Nuclear Organelle) Associated Protein 6', 'FLJ23459', 'GEMIN6'], ['Gem Nuclear Organelle Associated Protein 7', 'Gem-Associated Protein 7', 'Gemin-7', 'SIP3', 'Gem (Nuclear Organelle) Associated Protein 7', 'FLJ13956', 'GEMIN7'], ['Gem Nuclear Organelle Associated Protein 8', 'Family With Sequence Similarity 51, Member A1', 'Gem-Associated Protein 8', 'FAM51A1', 'Gemin-8', 'Gem (Nuclear Organelle) Associated Protein 8', 'Protein FAM51A1', 'FLJ20514', 'GEMIN8'], ["GEN1 Holliday Junction 5' Flap Endonuclease", 'Flap Endonuclease GEN Homolog 1', 'Holliday Junction Resolvase', 'Gen', 'Gen Endonuclease Homolog 1 (Drosophila)', 'Gen Endonuclease Homolog 1', 'EC 3.1.-.-', 'FLJ40869', 'GEN1'], ['Guided Entry Of Tail-Anchored Proteins Factor 1', 'CHD5', 'Tail-Anchored Protein Insertion Receptor WRB', 'Congenital Heart Disease 5 Protein', 'Tryptophan Rich Basic Protein', 'WRB', 'Tryptophan-Rich Basic Protein', 'GET1'], ['Guided Entry Of Tail-Anchored Proteins Factor 3, ATPase', 'TRC40', 'Transmembrane Domain Recognition Complex 40 KDa ATPase Subunit', 'ATPase GET3', 'ARSA-I', 'ASNA1', 'ArsA (Bacterial) Arsenite Transporter, ATP-Binding, Homolog 1', 'ArsA Arsenite Transporter, ATP-Binding, Homolog 1', 'Transmembrane Domain Recognition Complex, 40kDa', 'Golgi To ER Traffic 3 Homolog (S. Cerevisiae)', 'Golgi To ER Traffic 3 Homolog', 'Arsenical Pump-Driving ATPase', 'Arsenite-Stimulated ATPase', 'EC 3.6.3.16', 'EC 3.6.-.-', 'HARSA-I', 'HASNA-I', 'ASNA-I', 'ARSA1', 'GET3', 'ARSA'], ['Guided Entry Of Tail-Anchored Proteins Factor 4', 'TRC35', 'CEE', 'Transmembrane Domain Recognition Complex 35 KDa Subunit', 'Transmembrane Domain Recognition Complex, 35kDa', 'Golgi To ER Traffic Protein 4 Homolog', 'Golgi To ER Traffic Protein 4', 'Conserved Edge Protein', 'H_NH1244M04.5', 'C7orf20', 'CGI-20', 'Golgi To ER Traffic Protein 4 Homolog (S. Cerevisiae)', 'Chromosome 7 Open Reading Frame 20', 'Conserved Edge Expressed Protein', 'Conserved Edge-Expressed Protein', 'CGI-20 Protein', 'GET4'], ['Glial Fibrillary Acidic Protein', 'Intermediate Filament Protein', 'FLJ45472', 'ALXDRD', 'GFAP'], ['Growth Factor, Augmenter Of Liver Regeneration', 'HERV1', 'FAD-Linked Sulfhydryl Oxidase ALR', 'ALR', 'Hepatic Regenerative Stimulation Substance', 'ERV1', 'HPO1', 'HPO2', 'HPO', 'HSS', 'Growth Factor, Erv1 (S. Cerevisiae)-Like (Augmenter Of Liver Regeneration)', 'Augmenter Of Liver Regeneration', 'ERV1 Homolog (S. Cerevisiae)', 'Erv1-Like Growth Factor', 'Hepatopoietin Protein', 'Hepatopoietin', 'ERV1 Homolog', 'EC 1.8.3.2', 'MMCHD', 'MPMCD', 'GFER'], ['Growth Factor Independent 1 Transcriptional Repressor', 'Growth Factor Independent Protein 1', 'Zinc Finger Protein Gfi-1', 'Zinc Finger Protein 163', 'ZNF163', 'GFI-1', 'GFI1A', 'Growth Factor Independent 1 Transcription Repressor', 'Growth Factor Independence-1', 'Growth Factor Independent 1', 'SCN2', 'GFI1'], ['Growth Factor Independent 1B Transcriptional Repressor', 'Growth Factor Independent 1B (Potential Regulator Of CDKN1A, Translocated In CML)', 'Zinc Finger Protein Gfi-1b', 'ZNF163B', 'Growth Factor Independent 1B Transcription Repressor', 'Potential Regulator Of CDKN1A Translocated In CML', 'Growth Factor Independent Protein 1B', 'BDPLT17', 'GFI1B'], ['G Elongation Factor Mitochondrial 1', 'GFM', 'G Translation Elongation Factor, Mitochondrial', 'Elongation Factor G, Mitochondrial', 'MtEF-G1', 'HEFG1', 'EFG1', 'EFGM', 'EGF1', 'EFG', 'Elongation Factor G 1, Mitochondrial', 'Mitochondrial Elongation Factor G1', 'Mitochondrial Elongation Factor G', 'Elongation Factor G1', 'MEF-G 1', 'COXPD1', 'EF-Gmt', 'GFM1'], ['GTP Dependent Ribosome Recycling Factor Mitochondrial 2', 'EF-G2mt', 'EFG2', 'Ribosome-Releasing Factor 2, Mitochondrial', 'G Elongation Factor Mitochondrial 2', 'Ribosome Releasing Factor 2', 'MEF-G 2', 'RRF2mt', 'HEFG2', 'Mitochondrial Ribosome Recycling Factor 2', 'Mitochondrial Ribosome-Releasing Factor 2', 'Elongation Factor G 2, Mitochondrial', 'Mitochondrial Elongation Factor G2', 'Elongation Factor G2', 'FLJ21661', 'MSTP027', 'MST027', 'MRRF2', 'RRF2', 'GFM2', 'RRF'], ['Glucose-Fructose Oxidoreductase Domain Containing 1', 'Glucose-Fructose Oxidoreductase Domain-Containing Protein 1', 'C6orf114', 'ADG-90', 'Chromosome 6 Open Reading Frame 114', 'EC 1.-.-.-', 'FLJ20330', 'GFOD1'], ['Glucose-Fructose Oxidoreductase Domain Containing 2', 'Glucose-Fructose Oxidoreductase Domain-Containing Protein 2', 'EC 1.-.-.-', 'FLJ23802', 'MGC11335', 'GFOD2'], ['Glutamine--Fructose-6-Phosphate Transaminase 1', 'GFAT1', 'GFAT', 'Glutamine--Fructose-6-Phosphate Aminotransferase [Isomerizing] 1', 'Glutamine:Fructose-6-Phosphate Amidotransferase 1', 'D-Fructose-6-Phosphate Amidotransferase 1', 'Hexosephosphate Aminotransferase 1', 'EC 2.6.1.16', 'GFAT 1', 'GFPT', 'GFA', 'Glucosamine--Fructose-6-Phosphate Aminotransferase [Isomerizing] 1', 'Glutamine-Fructose-6-Phosphate Transaminase 1', 'CMSTA1', 'GFAT1m', 'GFPT1L', 'CMS12', 'GFPT1', 'MSLG'], ['Glutamine-Fructose-6-Phosphate Transaminase 2', 'GFAT2', 'Glutamine--Fructose-6-Phosphate Aminotransferase [Isomerizing] 2', 'Glutamine: Fructose-6-Phosphate Aminotransferase 2', 'Glutamine:Fructose-6-Phosphate Amidotransferase 2', 'D-Fructose-6-Phosphate Amidotransferase 2', 'Hexosephosphate Aminotransferase 2', 'EC 2.6.1.16', 'GFAT 2', 'Glucosamine--Fructose-6-Phosphate Aminotransferase [Isomerizing] 2', 'GFPT2', 'GFAT'], ['GDNF Family Receptor Alpha 1', 'RETL1', 'TRNR1', 'TGF-Beta-Related Neurotrophic Factor Receptor 1', 'GDNF Family Receptor Alpha-1', 'GDNFR-Alpha-1', 'RET Ligand 1', 'GFR-ALPHA-1', 'GDNFRA', 'GDNFR', 'RET1L', 'Glial Cell Line-Derived Neurotrophic Factor Receptor Alpha', 'PI-Linked Cell-Surface Accessory Protein', 'GPI-Linked Anchor Protein', 'GDNF Receptor Alpha 1d', 'GDNF Receptor Alpha 1e', 'GDNF Receptor Alpha-1', 'GFR-Alpha-1', 'GFRalpha-1', 'GFRA1'], ['GDNF Family Receptor Alpha 2', 'GDNFRB', 'RETL2', 'TRNR2', 'TGF-Beta-Related Neurotrophic Factor Receptor 2', 'GDNF Family Receptor Alpha-2', 'Neurturin Receptor Alpha', 'GDNF Receptor Beta', 'GDNFR-Alpha-2', 'RET Ligand 2', 'GDNFR-Beta', 'NTNR-Alpha', 'NTNRA', 'Glial Cell Line Derived Neurotrophic Factor Receptor, Beta', 'PI-Linked Cell-Surface Accessory Protein', 'TRN Receptor, GPI-Anchored', 'GDNF Receptor Alpha-2', 'NRTNR-ALPHA', 'GFR-Alpha 2', 'GFR-Alpha-2', 'NRTNR-Alpha', 'GFRA2'], ['GDNF Family Receptor Alpha 3', 'GDNF Family Receptor Alpha-3', 'GDNF Receptor Alpha-3', 'GDNFR-Alpha-3', 'GFR-Alpha-3', 'Glial Cell Line-Derived Neurotrophic Factor Receptor Alpha-3', 'GPI-Linked Receptor', 'GDNFR3', 'GFRa-3', 'GFRA3'], ['GDNF Family Receptor Alpha 4', 'Persephin Receptor', 'GDNF Family Receptor Alpha-4', 'GDNF Receptor Alpha-4', 'GDNFR-Alpha-4', 'GFR-Alpha-4', 'GFR Receptor Alpha 4', 'GFRA4'], ['GDNF Family Receptor Alpha Like', 'GDNF Family Receptor Alpha-Like', 'BA360D14.1', 'C6orf144', 'UNQ9356', 'GRAL', 'Chromosome 6 Open Reading Frame 144', 'IVFI9356', 'GFRAL'], ['GDP-L-Fucose Synthase', 'SDR4E1', 'Short Chain Dehydrogenase/Reductase Family 4E, Member 1', 'GDP-4-Keto-6-Deoxy-D-Mannose-3,5-Epimerase-4-Reductase', 'Tissue Specific Transplantation Antigen P35B', 'Red Cell NADP(H)-Binding Protein', 'EC 1.1.1.271', 'TSTA3', 'P35B', 'FX', 'Short-Chain Dehydrogenase/Reductase Family 4E Member 1', 'GDP-4-Keto-6-Deoxy-D-Mannose Epimerase-Reductase', 'Testis Tissue Sperm-Binding Protein Li 45a', 'Tissue Specific Transplantation Antigen 3', '3-5 Epimerase/4-Reductase', 'Protein FX', 'GFUS'], ['Golgi Associated Olfactory Signaling Regulator', 'Golgi Protein In Olfactory Neurons', 'Goofy', 'Golgi-Associated Olfactory Signaling Regulator', 'Protein Goofy', 'GFY'], ['Golgi Associated, Gamma Adaptin Ear Containing, ARF Binding Protein 1', 'Golgi-Localized, Gamma Ear-Containing, ARF-Binding Protein 1', 'ADP-Ribosylation Factor-Binding Protein GGA1', 'ADP-Ribosylation Factor Binding Protein 1', 'Gamma-Adaptin Related Protein 1', 'Gamma-Adaptin-Related Protein 1', 'GGA1'], ['Golgi Associated, Gamma Adaptin Ear Containing, ARF Binding Protein 2', 'Golgi-Localized, Gamma Ear-Containing, ARF-Binding Protein 2', 'ADP-Ribosylation Factor-Binding Protein GGA2', 'VHS Domain And Ear Domain Of Gamma-Adaptin', 'Gamma-Adaptin-Related Protein 2', 'KIAA1080', 'VEAR', 'Golgi-Associated, Gamma Adaptin Ear Containing, ARF Binding Protein 2', 'VHS Domain And Ear Domain-Containing Protein', 'GGA2', 'Vear'], ['Golgi Associated, Gamma Adaptin Ear Containing, ARF Binding Protein 3', 'Golgi-Localized, Gamma Ear-Containing, ARF-Binding Protein 3', 'ADP-Ribosylation Factor-Binding Protein GGA3', 'KIAA0154', 'Golgi-Associated, Gamma Adaptin Ear Containing, ARF Binding Protein 3', 'GGA3'], ['Gamma-Glutamylamine Cyclotransferase', 'AIG2-Like Domain-Containing Protein 1', 'Gamma-Glutamylaminecyclotransferase', 'AIG2-Like Domain 1', 'A2LD1', 'EC 4.3.2.8', 'GGACT'], ['Gamma-Glutamylcyclotransferase', 'CRF21', 'Cytochrome C-Releasing Factor 21', 'C7orf24', 'Chromosome 7 Open Reading Frame 24', 'Gamma -Glutamyl Cyclotransferase', 'EC 4.3.2.9', 'MGC3077', 'GCTG', 'GGCT', 'GGC', 'Ggc'], ['Gamma-Glutamyl Carboxylase', 'Vitamin K-Dependent Gamma-Carboxylase', 'Peptidyl-Glutamate 4-Carboxylase', 'VKCFD1', 'Vitamin K Gamma Glutamyl Carboxylase', 'EC 4.1.1.90', 'GGCX', 'GC'], ['Gamma-Glutamyl Hydrolase', 'Gamma-Glu-X Carboxypeptidase', 'Conjugase', 'GH', 'Gamma-Glutamyl Hydrolase (Conjugase, Folylpolygammaglutamyl Hydrolase)', 'Folylpolygammaglutamyl Hydrolase', 'EC 3.4.19.9', 'GGH'], ['Gametogenetin', 'FLJ35713', 'MGC33369', 'GGN'], ['Gametogenetin Binding Protein 2', 'LZK1', 'Laryngeal Carcinoma-Related Protein 1', 'Gametogenetin-Binding Protein 2', 'Zinc Finger Protein 403', 'ZFP403', 'ZNF403', 'DIF-3', 'LCRG1', 'DIF3', 'Laryngeal Carcinoma Related Gene 1', 'C3HC4-Type Zinc Finger Protein', 'Protein ZNF403', 'FLJ22561', 'FLJ21230', 'GGNBP2'], ['Geranylgeranyl Diphosphate Synthase 1', 'Geranylgeranyl Pyrophosphate Synthase', '(2E,6E)-Farnesyl Diphosphate Synthase', 'Dimethylallyltranstransferase', 'Farnesyl Diphosphate Synthase', 'Farnesyltranstransferase', 'Geranyltranstransferase', 'GGPP Synthase', 'GGPPSase', 'GGPPS1', 'Geranylgeranyl Diphosphate Synthase', 'EC 2.5.1.29', 'EC 2.5.1.10', 'EC 2.5.1.-', 'EC 2.5.1.1', 'GGPPS', 'GGPS1'], ['Gamma-Glutamyltransferase 1', 'Glutathione Hydrolase 1 Proenzyme', 'Gamma-Glutamyltranspeptidase 1', 'Leukotriene-C4 Hydrolase', 'EC 2.3.2.2', 'D22S672', 'D22S732', 'CD224', 'GGT 1', 'GGT', 'Testicular Tissue Protein Li 73', 'CD224 Antigen', 'EC 3.4.19.13', 'EC 3.4.19.14', 'GGTD', 'GGT1', 'GTG'], ['Gamma-Glutamyltransferase 2', 'Inactive Gamma-Glutamyltranspeptidase 2', 'Inactive Glutathione Hydrolase 2', 'GGT 2', 'Gamma-Glutamyltranspeptidase 2', 'Glutathione Hydrolase 2', 'EC 2.3.2.2', 'GGT2', 'GGT'], ['Gamma-Glutamyltransferase 5', 'Gamma-Glutamyltransferase-Like Activity 1', 'Gamma-Glutamyl Transpeptidase-Related Enzyme', 'Glutathione Hydrolase 5 Proenzyme', 'Gamma-Glutamyltranspeptidase 5', 'Leukotriene-C4 Hydrolase', 'GGT-REL', 'GGTLA1', 'GGT 5', 'Gamma-Glutamyl Transpeptidase-Related Protein', 'Gamma-Glutamyl Cleaving Enzyme', 'EC 3.4.19.13', 'EC 3.4.19.14', 'EC 2.3.2.2', 'GGT-Rel', 'GGT5', 'GGL'], ['Gamma-Glutamyltransferase 6', 'Gamma-Glutamyltranspeptidase 6', 'Glutathione Hydrolase 6', 'Gamma-Glutamyltransferase 6 Homolog (Rat)', 'Gamma-Glutamyltransferase 6 Homolog', 'EC 3.4.19.13', 'EC 2.3.2.2', 'FLJ90165', 'GGT 6', 'GGT6'], ['Gamma-Glutamyltransferase 7', 'Gamma-Glutamyltransferase-Like 3', 'Gamma-Glutamyltransferase-Like 5', 'Gamma-Glutamyltranspeptidase 7', 'Glutathione Hydrolase 7', 'EC 2.3.2.2', 'D20S101', 'GGTL3', 'GGTL5', 'GGT 7', 'Gamma-Glutamyltranspeptidase-Like 3', 'Gamma-Glutamyltransferase 4', 'EC 3.4.19.13', 'DJ18C9.2', 'GGT4', 'GGT7'], ['Glycoprotein Alpha-Galactosyltransferase 1 (Inactive)', 'Inactive N-Acetyllactosaminide Alpha-1,3-Galactosyltransferase', 'Glycoprotein Alpha-Galactosyltransferase 1, Pseudogene', 'Glycoprotein, Alpha-Galactosyltransferase 1', 'GGTA1P', 'Glycoprotein Alpha-Galactosyltransferase 1 Pseudogene', 'Truncated Alpha-1,3-Galactosyltransferase', 'A1/3GTP', 'GLYT2', 'GGTA1', 'GGTA'], ['Gamma-Glutamyltransferase Light Chain 1', 'Gamma-Glutamyltransferase-Like Activity 4', 'Gamma-Glutamyltransferase-Like Activity 3', 'Gamma-Glutamyltransferase-Like Protein 6', 'Glutathione Hydrolase Light Chain 1', 'DJ831C21.1', 'DJ831C21.2', 'GGTLA4', 'Gamma-Glutamyl Transpeptidase', 'EC 2.3.2.2', 'GGTLA3', 'GGTLC1', 'GGTL6'], ['Gamma-Glutamyltransferase Light Chain 2', 'Gamma-Glutamyltransferase-Like Protein 4', 'Glutathione Hydrolase Light Chain 2', 'Gamma-Glutamyltransferase-Like 4', 'GGTL4', 'EC 2.3.2.2', 'GGTLC2'], ['Gamma-Glutamyltransferase Light Chain Family Member 3', 'Putative Gamma-Glutamyltransferase Light Chain 3', 'Putative Glutathione Hydrolase Light Chain 3', 'Gamma-Glutamyltransferase Light Chain 3', 'EC 2.3.2.2', 'GGTLC3', 'GGT'], ['Growth Hormone 1', 'Pituitary Growth Hormone', 'Somatotropin', 'GH-N', 'GH', 'HGH-N', 'GHN', 'Growth Hormone B5', 'Growth Hormone', 'IGHD1A', 'IGHD1B', 'IGHD2', 'GHB5', 'GH1'], ['Growth Hormone 2', 'Placenta-Specific Growth Hormone', 'GH-V', 'Placental-Specific Growth Hormone', 'Growth Hormone Variant', 'HGH-V', 'GHL', 'GHV', 'Growth Hormone B2', 'GHB2', 'GH2'], ['GH3 Domain Containing', 'LGP1', 'GH3 Domain-Containing Protein', 'Homolog Of Mouse LGP1', 'D11LGP1E', 'D11LGP1', 'GHDC'], ['Growth Hormone Inducible Transmembrane Protein', 'TMBIM5', 'DERP2', 'Transmembrane BAX Inhibitor Motif-Containing Protein 5', 'Mitochondrial Morphology And Cristae Structure 1', 'Transmembrane BAX Inhibitor Motif Containing 5', 'Growth Hormone-Inducible Transmembrane Protein', 'Dermal Papilla-Derived Protein 2', 'HSPC282', 'PTD010', 'MICS1', 'My021', 'GHITM'], ['Growth Hormone Receptor', 'Growth Hormone Binding Protein', 'Somatotropin Receptor', 'GH Receptor', 'GHBP', 'Serum Binding Protein', 'GHIP', 'GHR'], ['Growth Hormone Releasing Hormone', 'Somatoliberin', 'Somatocrinin', 'Sermorelin', 'Somatorelin', 'GHRF', 'GRF', 'Growth Hormone-Releasing Hormone', 'Growth Hormone Releasing Factor', 'Growth Hormone-Releasing Factor', 'GHRH', 'INN'], ['Growth Hormone Releasing Hormone Receptor', 'Growth Hormone-Releasing Hormone Receptor', 'GHRH Receptor', 'GRF Receptor', 'GRFR', 'Growth Hormone-Releasing Factor Receptor', 'IGHD1B', 'GHRFR', 'IGHD4', 'GHRHR'], ['Ghrelin And Obestatin Prepropeptide', 'MTLRP', 'Ghrelin, Growth Hormone Secretagogue Receptor Ligand', 'Prepro-Appetite Regulatory Hormone', 'Growth Hormone-Releasing Peptide', 'Ghrelin/Obestatin Prepropeptide', 'Appetite-Regulating Hormone', 'Motilin-Related Peptide', 'Ghrelin/Obestatin Preprohormone', 'Growth Hormone Secretagogue', 'In2c-Preproghrelin', 'Preproghrelin', 'Protein M46', 'Obestatin', 'Ghrelin', 'GHRL'], ['Growth Hormone Secretagogue Receptor', 'Growth Hormone Secretagogue Receptor Type 1', 'GH-Releasing Peptide Receptor', 'Ghrelin Receptor', 'GHS-R', 'GHRP', 'GHDP', 'GHSR'], ['GID Complex Subunit 4 Homolog', 'VID24', 'Vacuolar Import And Degradation Protein 24 Homolog', 'Glucose-Induced Degradation Protein 4 Homolog', 'Vacuolar Import And Degradation 24', 'C17orf39', 'GID Complex Subunit 4, VID24 Homolog (S. Cerevisiae)', 'GID Complex Subunit 4, VID24 Homolog', 'Chromosome 17 Open Reading Frame 39', 'GID Complex Subunit 4', 'VID2', 'GID4'], ['GID Complex Subunit 8 Homolog', 'TWA1', 'Two Hybrid Associated Protein No. 1 With RanBPM', 'Glucose-Induced Degradation Protein 8 Homolog', 'Two Hybrid-Associated Protein 1 With RanBPM', 'C20orf11', 'GID Complex Subunit 8 Homolog (S. Cerevisiae)', 'Chromosome 20 Open Reading Frame 11', 'GID Complex Subunit 8', 'Protein C20orf11', 'BA305P22.1', 'FLJ20602', 'GID8', 'Twa1'], ['GRB10 Interacting GYF Protein 1', 'PERQ Amino Acid-Rich With GYF Domain-Containing Protein 1', 'GRB10-Interacting GYF Protein 1', 'PERQ1', 'GYF1', 'Postmeiotic Segregation Increased 2-Like 12', 'PERQ Amino Acid Rich, With GYF Domain 1', 'GYF Domain Containing 1', 'GIGYF1', 'CDS2'], ['GRB10 Interacting GYF Protein 2', 'PERQ Amino Acid-Rich With GYF Domain-Containing Protein 2', 'Parkinson Disease (Autosomal Recessive, Early Onset) 11', 'Trinucleotide Repeat-Containing Gene 15 Protein', 'PERQ Amino Acid Rich, With GYF Domain 3', 'GRB10-Interacting GYF Protein 2', 'KIAA0642', 'TNRC15', 'PERQ2', 'GYF2', 'PERQ Amino Acid Rich, With GYF Domain 2', 'Trinucleotide Repeat Containing 15', 'GYF Domain Containing 2', 'PARK11', 'GIGYF2', 'PERQ3'], ['GTPase, IMAP Family Member 1', 'HIMAP1', 'IMAP1', 'Immune-Associated Nucleotide-Binding Protein 2', 'GTPase IMAP Family Member 1', 'IMAP38', 'IAN2', 'Immunity Associated Protein 1', 'Immunity-Associated Protein 1', 'GIMAP1'], ['GTPase, IMAP Family Member 2', 'HIMAP2', 'IMAP2', 'Immune-Associated Nucleotide-Binding Protein 12', 'GTPase IMAP Family Member 2', 'IAN12', 'Immunity Associated Protein 2', 'Immunity-Associated Protein 2', 'DKFZp586D0824', 'GIMAP2'], ['GTPase, IMAP Family Member 4', 'IMAP4', 'IAN1', 'Immune-Associated Nucleotide-Binding Protein 1', 'Immunity-Associated Nucleotide 1 Protein', 'GTPase IMAP Family Member 4', 'IAN-1', 'Human Immune Associated Nucleotide 1', 'Immunity Associated Protein 4', 'Immunity-Associated Protein 4', 'FLJ11110', 'MSTP062', 'GIMAP4', 'HIMAP4', 'HIAN1'], ['GTPase, IMAP Family Member 5', 'Immune-Associated Nucleotide-Binding Protein 5', 'IAN5', 'Immunity-Associated Nucleotide 5 Protein', 'GTPase IMAP Family Member 5', 'HIMAP3', 'IAN4L1', 'IAN-5', 'IMAP3', 'Inhibitor Of Radiation- And OA-Induced Apoptosis, Irod/Ian5', 'Immunity-Associated Nucleotide 4-Like 1 Protein', 'Immune Associated Nucleotide 4 Like 1 (Mouse)', 'Immune Associated Nucleotide 4 Like 1', 'Immunity Associated Protein 3', 'Immunity-Associated Protein 3', 'GIMAP5', 'HIAN5', 'IAN4', 'IROD'], ['GTPase, IMAP Family Member 6', 'IAN6', 'Immune-Associated Nucleotide-Binding Protein 6', 'GTPase IMAP Family Member 6', 'IAN-2', 'IAN-6', 'IAN2', 'Immunity-Associated Nucleotide 2 Protein', 'Immunity-Associated Nucleotide 6 Protein', 'Immune Associated Nucleotide 2', 'Immune Associated Nucleotide 6', 'FLJ22690', 'GIMAP6', 'HIAN2', 'HIAN6'], ['GTPase, IMAP Family Member 7', 'IAN7', 'Immune-Associated Nucleotide-Binding Protein 7', 'Immunity-Associated Nucleotide 7 Protein', 'GTPase IMAP Family Member 7', 'IAN-7', 'Immune Associated Nucleotide', 'MGC27027', 'GIMAP7', 'HIAN7'], ['GTPase, IMAP Family Member 8', 'Immune-Associated Nucleotide-Binding Protein 9', 'IAN9', 'GTPase IMAP Family Member 8', 'IAN-9', 'IANT', 'Human Immune Associated Nucleotide 6', 'DKFZp667I133', 'Protein IanT', 'GIMAP8', 'HIAN6', 'IAN6'], ['GIMAP Family P-Loop NTPase Domain Containing 1', 'GIMAP Family P-Loop NTPase Domain-Containing Protein 1', 'GTPase IMAP Family Member GIMD1', 'GIMD1'], ['Gypsy Retrotransposon Integrase 1', 'Ty3/Gypsy Integrase 1', 'GIN-1', 'TGIN1', 'Gypsy Retrotransposon Integrase-Like Protein 1', 'Zinc Finger H2C2 Domain-Containing Protein', 'ZH2C2', 'Zinc Finger, H2C2 Domain Containing', 'Gypsy Integrase 1', 'FLJ20125', 'GIN1']]#list of list for aliases
umu = [['hemoglobin measurement', 'erythrocyte count', 'hematocrit', 'infect', 'infection', 'diabetic foot', 'cancer', 'ulcer disease'], ['hemoglobin measurement', 'erythrocyte count', 'hematocrit'], ['Microcephaly - polymicrogyria - corpus callosum agenesis', 'rheumatoid arthritis', 'eosinophil count', 'multiple sclerosis', 'myeloid white cell count', 'lymphocyte count', 'chronic lymphocytic leukemia', 'Hodgkins lymphoma', 'basophil count', 'leukocyte count'], ['Rubinstein-Taybi syndrome', 'Rubinstein-Taybi syndrome due to EP300 haploinsufficiency', 'head and neck squamous cell carcinoma', 'colorectal adenocarcinoma', 'Menke-Hennekam syndrome 2', 'cervical squamous cell carcinoma', 'cancer', 'urinary bladder cancer', 'colonic neoplasm', 'T-cell leukemia'], ['type II diabetes mellitus', 'breast adenocarcinoma', 'red blood cell distribution width', 'ovarian cancer', 'glioblastoma multiforme', 'chronic lymphocytic leukemia', 'mean corpuscular volume', 'head and neck squamous cell carcinoma', 'esophageal cancer', 'endometrial cancer'], ['Autosomal dominant secondary polycythemia', 'Paraganglioma', 'Primary familial polycythemia', 'renal cell carcinoma', 'electrocardiography', 'hemoglobin measurement', 'birth weight', 'hematocrit', 'polycythemia vera', 'colorectal adenocarcinoma'], ['Hereditary elliptocytosis', 'mean corpuscular hemoglobin concentration', 'hemoglobin measurement', 'platelet component distribution width', 'sex hormone-binding globulin measurement', 'hematocrit', 'platelet count', 'mean platelet volume', 'reticulocyte count', 'mean corpuscular volume'], ['Autosomal dominant non-syndromic intellectual disability', 'mental retardation', 'Abnormality of brain morphology', 'thyroid gland adenocarcinoma', 'keratinocyte carcinoma', 'basal cell carcinoma', 'body height', 'appendicular lean mass', 'hip circumference', 'anthropometric measurement'], ['body height', 'heel bone mineral density', 'mean corpuscular volume', 'bone quantitative ultrasound measurement', 'mean corpuscular hemoglobin', 'hair shape measurement', 'mean corpuscular hemoglobin concentration', 'platelet count', 'vital capacity', 'balding measurement'], ['glomerular filtration rate', 'birth weight', 'creatinine measurement', 'urate measurement', 'thyroid gland adenocarcinoma', 'stroke', 'gonorrhea', 'osteosarcoma', 'neoplasm', 'protein measurement'], ['Spastic paraplegia', 'Failure to thrive', 'DNA methylation', 'Abnormality of refraction', 'thyroid carcinoma', 'drug use measurement', 'inflammatory bowel disease', 'hemoglobin measurement', 'breast cancer', 'neoplasm'], ['serum alanine aminotransferase measurement', 'waist-hip ratio', 'low density lipoprotein cholesterol measurement', 'BMI-adjusted waist-hip ratio', 'lean body mass', 'body mass index', 'type II diabetes mellitus', 'self reported educational attainment', 'body weight', 'fat body mass'], ['hemoglobin measurement', 'erythrocyte count', 'hematocrit', 'red blood cell density measurement', 'mean corpuscular hemoglobin concentration', 'glomerular filtration rate', 'visual perception measurement', 'platelet crit', 'FEV/FEC ratio', 'coronary artery disease'], ['Hereditary spherocytosis', 'Hereditary elliptocytosis', 'neoplasm', 'cancer', 'breast cancer', 'non-small cell lung carcinoma', 'Blackfan-Diamond anemia', 'Hemolytic anemia due to red cell pyruvate kinase deficiency', 'Dehydrated hereditary stomatocytosis', 'Beta-thalassemia - X-linked thrombocytopenia'], ['BMI-adjusted waist-hip ratio', 'hair shape measurement', 'balding measurement', 'platelet component distribution width', 'waist-hip ratio', 'peak expiratory flow', 'mean platelet volume', 'FEV/FEC ratio', 'BMI-adjusted waist circumference', 'diastolic blood pressure'], ['schizophrenia', 'type II diabetes mellitus', 'chronotype measurement', 'heel bone mineral density', 'body height', 'acute myeloid leukemia', 'emotional symptom measurement', 'Autosomal recessive spondylocostal dysostosis', 'feeling nervous measurement', 'Familial exudative vitreoretinopathy'], ['Intestinal epithelial dysplasia', 'Lynch syndrome', 'colorectal cancer, hereditary nonpolyposis, type 8', 'neoplasm', 'hereditary nonpolyposis colorectal carcinoma', 'cancer', 'urinary bladder cancer', 'stomach neoplasm', 'colon mucinous adenocarcinoma', 'sleep duration'], ['bone quantitative ultrasound measurement', 'Dupuytren Contracture', 'Palmar Fibromatosis', 'heel bone mineral density', 'Fasciitis', 'blood protein measurement', 'bone density', 'Primary ciliary dyskinesia', 'body height', 'sitting height measurement'], ['Vici syndrome', 'X-linked myopathy with excessive autophagy', 'Syndromic retinitis pigmentosa', 'hair colour measurement', 'mean corpuscular hemoglobin', 'hair color', 'genetic disorder', 'Microcephaly', 'platelet count', 'esophageal carcinoma'], ['cancer', 'cytokine measurement', 'response to vaccine', 'hair color', 'balding measurement', 'Autosomal dominant secondary polycythemia', 'portal hypertension', 'psoriasis', 'Hemoglobin H disease', 'esophageal adenocarcinoma'], ['thyroid cancer', 'neoplasm', 'thyroid carcinoma', 'sex hormone-binding globulin measurement', 'non-small cell lung carcinoma', "Alzheimer's disease", 'medullary thyroid gland carcinoma', 'ephrin type-A receptor 1 measurement', 'blood protein measurement', 'family history of Alzheimer’s disease'], ['Non-syndromic congenital cataract', 'Total congenital cataract', 'Posterior polar cataract', 'chronic myelogenous leukemia', 'acute lymphoblastic leukemia', 'neoplasm', 'thyroid cancer', 'Nuclear cataract', 'cholangiocarcinoma', 'thyroid carcinoma'], ['neoplasm', 'thyroid cancer', 'thyroid carcinoma', 'non-small cell lung carcinoma', 'neuroimaging measurement', 'body mass index', 'brain volume measurement', 'brain measurement', 'atrial fibrillation', 'body fat percentage'], ['thyroid cancer', 'neoplasm', 'heel bone mineral density', 'thyroid carcinoma', 'hair colour measurement', 'acute myeloid leukemia', 'hair color', 'protein measurement', 'non-small cell lung carcinoma', 'neuroimaging measurement'], ['thyroid cancer', 'neoplasm', 'self reported educational attainment', 'mathematical ability', 'thyroid carcinoma', 'cognitive function measurement', 'intelligence', 'non-small cell lung carcinoma', 'schizophrenia', 'chronotype measurement'], ['thyroid cancer', 'neoplasm', 'thyroid carcinoma', 'non-small cell lung carcinoma', 'medullary thyroid gland carcinoma', 'systolic blood pressure', 'head and neck squamous cell carcinoma', 'thyroid gland adenocarcinoma', 'lung adenocarcinoma', 'cutaneous melanoma'], ['neoplasm', 'thyroid cancer', 'thyroid carcinoma', 'mathematical ability', 'non-small cell lung carcinoma', 'lung adenocarcinoma', 'smoking status measurement', 'cognitive function measurement', 'self reported educational attainment', 'response to anticoagulant'], ['thyroid cancer', 'neoplasm', 'thyroid carcinoma', 'non-small cell lung carcinoma', 'medullary thyroid gland carcinoma', 'alkaline phosphatase measurement', 'head and neck squamous cell carcinoma', 'lung adenocarcinoma', 'cutaneous melanoma', 'pathological myopia'], ['thyroid cancer', 'neoplasm', 'thyroid carcinoma', 'non-small cell lung carcinoma', 'medullary thyroid gland carcinoma', 'eosinophil count', 'head and neck squamous cell carcinoma', 'eosinophil percentage of leukocytes', 'lung adenocarcinoma', 'mathematical ability'], ['thyroid cancer', 'neoplasm', 'thyroid carcinoma', 'non-small cell lung carcinoma', 'protein measurement', 'medullary thyroid gland carcinoma', 'metastatic colorectal cancer', 'disease progression measurement', 'metastasis measurement', 'forced expiratory volume'], ['Rare hemorrhagic disorder due to a constitutional platelet anomaly', 'Familial prostate cancer', 'thyroid cancer', 'neoplasm', 'thyroid carcinoma', 'blood protein measurement', 'non-small cell lung carcinoma', 'ephrin type-B receptor 2 measurement', 'medullary thyroid gland carcinoma', 'colorectal neoplasm'], ['thyroid cancer', 'neoplasm', 'thyroid carcinoma', 'non-small cell lung carcinoma', 'medullary thyroid gland carcinoma', 'nose morphology measurement', 'facial morphology measurement', 'adolescent idiopathic scoliosis', 'lung adenocarcinoma', 'head and neck squamous cell carcinoma'], ['Capillary malformation - arteriovenous malformation', 'hydrops fetalis, nonimmune, and/or atrial septal defect, susceptibility to', 'thyroid cancer', 'neoplasm', 'thyroid carcinoma', 'non-small cell lung carcinoma', 'medullary thyroid gland carcinoma', 'Tetralogy of Fallot', 'inflammatory bowel disease', 'head and neck squamous cell carcinoma'], ['thyroid cancer', 'neoplasm', 'thyroid carcinoma', 'non-small cell lung carcinoma', 'medullary thyroid gland carcinoma', 'blood protein measurement', 'trypsin-2 measurement', 'lung adenocarcinoma', 'head and neck squamous cell carcinoma', 'anxiety measurement'], ['Familial hypercholanemia', 'sex hormone-binding globulin measurement', 'Cystic fibrosis', 'glioblastoma multiforme', 'appendicular lean mass', 'platelet count', 'red blood cell distribution width', 'platelet crit', 'chronic obstructive pulmonary disease', 'colorectal carcinoma'], ['smoking status measurement', 'protein measurement', 'smoking initiation', 'lung carcinoma', 'cannabis dependence', 'chronic obstructive pulmonary disease', 'schizophrenia', 'familial hypercholesterolemia', 'cutaneous melanoma', 'cigarettes per day measurement'], ['mean corpuscular hemoglobin', 'platelet crit', 'mean corpuscular volume', 'platelet count', 'Down syndrome', 'esophageal adenocarcinoma', 'vital capacity', 'psoriasis', 'oral squamous cell carcinoma', 'non-small cell lung carcinoma'], ['mean corpuscular hemoglobin concentration', 'colorectal adenocarcinoma', 'smoking status measurement', 'hepatocellular carcinoma', 'vital capacity', 'neutrophil percentage of leukocytes', 'colorectal cancer', 'esophageal adenocarcinoma', 'red blood cell distribution width', 'Colon Sessile Serrated Adenoma/Polyp'], ['Lafora disease', 'Progressive myoclonic epilepsy', 'Glycogen storage disease', 'lobe attachment', 'Rolandic epilepsy', 'mean platelet volume', 'heel bone mineral density', 'Unverricht-Lundborg disease', 'self reported educational attainment', 'Seizure'], ['body height', 'platelet component distribution width', 'hypothyroidism', 'C-reactive protein measurement', 'Inherited cancer-predisposing syndrome', 'triglyceride measurement', 'serum alanine aminotransferase measurement', 'high density lipoprotein cholesterol measurement', 'bipolar disorder', 'Partial pancreatic agenesis'], ['lung adenocarcinoma', 'high density lipoprotein cholesterol measurement', 'prostate cancer', 'colorectal adenocarcinoma', 'colonic neoplasm', 'atherosclerosis', 'colon carcinoma', 'malignant colon neoplasm', 'prostate neoplasm', 'juvenile dermatomyositis'], ['diastolic blood pressure', 'lung adenocarcinoma', 'self reported educational attainment', 'white matter hyperintensity measurement', 'schizophrenia', 'lifestyle measurement', 'mathematical ability', 'intelligence', 'lymphocyte count', 'colorectal adenocarcinoma'], ['reticulocyte count', 'triglyceride measurement', 'reticulocyte measurement', 'Autosomal recessive non-syndromic sensorineural deafness type DFNB', 'autosomal recessive nonsyndromic deafness 9', 'hyperproinsulinemia', 'Autosomal dominant non-syndromic sensorineural deafness type DFNA', 'blood protein measurement', 'Glycogen storage disease due to hepatic glycogen synthase deficiency', 'Transient neonatal diabetes mellitus'], ['Primary familial polycythemia', 'Blackfan-Diamond anemia', 'Autosomal dominant secondary polycythemia', 'hematocrit', 'hemoglobin measurement', 'red blood cell density measurement', 'erythrocyte count', 'mean corpuscular hemoglobin concentration', 'diabetic retinopathy', 'head and neck squamous cell carcinoma'], ['chronic obstructive pulmonary disease', 'appendicular lean mass', 'peak expiratory flow', 'forced expiratory volume', 'leukocyte count', 'neutrophil count', 'vital capacity', 'breast carcinoma', 'wellbeing measurement', 'neuroticism measurement'], ['Primary familial polycythemia', 'primary familial polycythemia due to EPO receptor mutation', 'chronic kidney disease', 'anemia (phenotype)', 'myelodysplastic syndrome', 'kidney disease', 'acute myocardial infarction', 'chronic hepatitis C virus infection', 'premature birth', 'subarachnoid hemorrhage'], ['Beta-thalassemia - X-linked thrombocytopenia', 'Familial hypofibrinogenemia', 'Hereditary spherocytosis', 'Hemoglobin H disease', 'Hemoglobin C - beta-thalassemia', 'Dehydrated hereditary stomatocytosis', 'Hereditary persistence of fetal hemoglobin - beta-thalassemia', 'Alpha-thalassemia - myelodysplastic syndrome', 'Overhydrated hereditary stomatocytosis', 'Sea-blue histiocytosis'], ['resting heart rate', 'mean platelet volume', 'platelet count', 'platelet component distribution width', 'dermatitis herpetiformis, familial', 'platelet crit', 'pancreatitis', 'Primary localized amyloidosis', 'Familial primary localized cutaneous amyloidosis', 'diastolic blood pressure'], ['leukodystrophy, hypomyelinating, 15', 'heel bone mineral density', 'glomerular filtration rate', 'bone quantitative ultrasound measurement', 'reticulocyte count', 'bone density', 'Leukodystrophy', 'red blood cell distribution width', 'mean corpuscular hemoglobin', 'triglyceride measurement'], ['Autosomal recessive non-syndromic sensorineural deafness type DFNB', 'autosomal recessive nonsyndromic deafness 102', 'deafness', 'alcohol drinking', 'ovarian cancer', 'chronic myelogenous leukemia', 'mental process', 'arterial stiffness measurement', 'glaucoma', 'Hereditary central diabetes insipidus'], ['platelet count', 'COVID-19', 'reticulocyte count', 'mean platelet volume', 'esophageal squamous cell carcinoma', 'Obesity Hypoventilation Syndrome', 'platelet glycoprotein VI measurement', 'neutrophil count', 'platelet component distribution width', 'platelet aggregation'], ['Autosomal recessive non-syndromic sensorineural deafness type DFNB', 'deafness', 'Non-syndromic genetic deafness', 'body height', 'Autosomal dominant non-syndromic sensorineural deafness type DFNA', 'autosomal recessive nonsyndromic deafness 9', 'X-linked non-syndromic sensorineural deafness type DFN', 'deafness, autosomal recessive 57', 'Waardenburg-Shah syndrome', 'ABCD syndrome'], ['Marie Unna hereditary hypotrichosis', 'hypotrichosis 5', 'osteitis deformans', 'heel bone mineral density', 'body height', 'monocyte count', "Barrett's esophagus", 'esophageal adenocarcinoma', 'Thyroid preparation use measurement', 'Alpha-thalassemia - myelodysplastic syndrome'], ['listeriosis', 'breast ductal adenocarcinoma', 'lymphoid neoplasm', 'esophageal adenocarcinoma', 'B-cell acute lymphoblastic leukemia', 'bile duct carcinoma', 'Ovarian Endometrioid Adenocarcinoma with Squamous Differentiation', 'gastric carcinoma', 'HER2 Positive Breast Carcinoma', 'skin carcinoma'], ['Split hand-split foot malformation', 'lymphocyte count', 'leukocyte count', 'neutrophil percentage of leukocytes', 'multiple sclerosis', 'lymphocyte percentage of leukocytes', 'glioblastoma multiforme', 'monocyte percentage of leukocytes', 'basophil count', 'lung adenocarcinoma'], ['heel bone mineral density', 'chronic venous insufficiency', 'prostate carcinoma', 'breast cancer', 'neoplasm', 'Autosomal dominant severe congenital neutropenia', 'eosinophil peroxidase deficiency', 'Neutropenia - monocytopenia - deafness', 'childhood onset asthma', 'Linear and whorled nevoid hypermelanosis'], ['eosinophil peroxidase deficiency', 'eosinophil count', 'eosinophil percentage of leukocytes', 'blood protein measurement', 'Neutropenia - monocytopenia - deafness', 'eosinophilic esophagitis', 'Autosomal dominant severe congenital neutropenia', 'Acromesomelic dysplasia, Grebe type', 'asthma', 'Recurrent infection due to specific granule deficiency'], ['body fat percentage', 'puberty onset measurement', 'body mass index', 'hemoglobin measurement', 'erythrocyte count', 'hematocrit', 'age at first facial hair', 'red blood cell density measurement', 'appendicular lean mass', 'synophrys measurement'], ['heel bone mineral density', 'blood protein measurement', 'angiopoietin-2 receptor measurement', 'endothelial growth factor measurement', 'angiopoietin-1 receptor, soluble measurement', 'Non-syndromic male infertility due to sperm motility disorder', 'angiopoietin-1 receptor measurement', 'Female infertility due to zona pellucida defect', 'Partial chromosome Y deletion', 'spermatogenic failure 1'], ['Perrault syndrome', 'red blood cell distribution width', 'reticulocyte count', 'mean corpuscular hemoglobin concentration', 'reticulocyte measurement', 'mean corpuscular hemoglobin', 'mean corpuscular volume', 'red blood cell density measurement', 'neoplasm', 'sarcoma'], ['psoriasis', 'ankylosing spondylitis', 'systolic blood pressure', 'endoplasmic reticulum aminopeptidase 1 measurement', 'platelet count', "Behcet's syndrome", 'mean platelet volume', 'protein measurement', "Crohn's disease", 'anterior uveitis'], ['inflammatory bowel disease', "Crohn's disease", 'neutrophil count', 'systolic blood pressure', 'aspartate aminotransferase measurement', 'juvenile idiopathic arthritis', 'diastolic blood pressure', 'Iridocyclitis', 'endoplasmic reticulum aminopeptidase 1 measurement', 'ankylosing spondylitis'], ['neoplasm', 'breast cancer', 'breast neoplasm', 'cancer', 'prostate cancer', 'ovarian cancer', 'gastric cancer', 'hepatocellular carcinoma', 'Posterior cortical atrophy', 'primary cutaneous amyloidosis'], ['non-small cell lung carcinoma', 'breast carcinoma', 'cancer', 'neoplasm', 'breast neoplasm', 'lung adenocarcinoma', 'breast adenocarcinoma', 'gastric cancer', 'gastric adenocarcinoma', 'colorectal adenocarcinoma'], ['Lethal congenital contracture syndrome type 2', 'gastric adenocarcinoma', 'cancer', 'neoplasm', 'bladder transitional cell carcinoma', 'urinary bladder cancer', 'thyroid carcinoma', 'thyroid cancer', 'acute erythroleukemia', 'endometrial cancer'], ['non-small cell lung carcinoma', 'breast carcinoma', 'neoplasm', 'amyotrophic lateral sclerosis', 'cancer', 'cutaneous melanoma', 'familial amyotrophic lateral sclerosis', 'thyroid carcinoma', 'lung adenocarcinoma', 'squamous cell lung carcinoma'], ['cancer', 'prostate carcinoma', 'neoplasm', 'systolic blood pressure', 'hepatocellular carcinoma', 'colorectal carcinoma', 'acute myeloid leukemia', 'cervical cancer', 'Charcot-Marie-Tooth disease type 1F', 'Autosomal dominant slowed nerve conduction velocity'], ['papillary thyroid carcinoma', 'melanocytic neoplasm', 'breast ductal adenocarcinoma', 'Thyroid Gland Oncocytic Follicular Carcinoma', 'vital capacity', 'Thyroid Gland Undifferentiated (Anaplastic) Carcinoma', 'macrocephaly, dysmorphic facies, and psychomotor retardation', 'volumetric brain MRI', 'temporal lobe volume measurement', 'body height'], ['heel bone mineral density', 'bone quantitative ultrasound measurement', 'FEV/FEC ratio', 'adolescent idiopathic scoliosis', 'body height', 'bone density', 'body mass index', 'electrocardiography', 'hip bone size', 'circadian rhythm'], ['COFS syndrome', 'Cockayne syndrome', 'Xeroderma pigmentosum', 'Fanconi anemia', 'Cockayne syndrome type 2', 'Premature ovarian insufficiency', 'Failure to thrive', 'Short stature-delayed bone age due to thyroid hormone metabolism deficiency', 'breast adenocarcinoma', 'heel bone mineral density'], ['Trichothiodystrophy', 'Xeroderma pigmentosum complementation group D', 'Xeroderma pigmentosum', 'COFS syndrome', 'Xeroderma pigmentosum-Cockayne syndrome complex', 'urinary bladder cancer', 'HIV infection', 'Cockayne syndrome', 'gastric carcinoma', 'cutaneous melanoma'], ['Xeroderma pigmentosum-Cockayne syndrome complex', 'Trichothiodystrophy', 'Xeroderma pigmentosum', 'Xeroderma pigmentosum complementation group B', 'HIV infection', 'cutaneous melanoma', 'colorectal adenocarcinoma', 'carcinoma of liver and intrahepatic biliary tract', 'Endometrial Endometrioid Adenocarcinoma', 'prostate carcinoma'], ['Xeroderma pigmentosum complementation group F', 'Xeroderma pigmentosum', 'Xeroderma pigmentosum-Cockayne syndrome complex', 'Fanconi anemia', 'XFE progeroid syndrome', 'acute myeloid leukemia', 'Cockayne syndrome', 'Isolated growth hormone deficiency type IA', 'colorectal adenocarcinoma', 'melanoma'], ['Xeroderma pigmentosum complementation group G', 'COFS syndrome', 'Xeroderma pigmentosum-Cockayne syndrome complex', 'Xeroderma pigmentosum', 'Cockayne syndrome', 'cutaneous melanoma', 'pancreatic neoplasm', 'colorectal adenocarcinoma', 'carcinoma of liver and intrahepatic biliary tract', 'Ileal Neuroendocrine Tumor G1'], ['Cockayne syndrome type 2', 'Cockayne syndrome', 'UV-sensitive syndrome', 'Cockayne syndrome type 1', 'COFS syndrome', 'Cockayne syndrome type 3', 'primary ovarian insufficiency', 'age-related macular degeneration', 'de Sanctis-Cacchione syndrome', 'genetic disorder'], ['cancer', 'colorectal carcinoma', 'kidney cancer', 'neoplasm', 'breast cancer', 'hepatocellular carcinoma', 'Nijmegen breakage syndrome', 'renal cell carcinoma', 'renal cell adenocarcinoma', 'colorectal cancer'], ['Pancytopenia-developmental delay syndrome', 'Inherited acute myeloid leukemia', 'body height', 'bone marrow failure syndrome', 'lymphocyte count', 'alkaline phosphatase measurement', 'adolescent idiopathic scoliosis', 'YKL40 measurement', 'inflammatory biomarker measurement', 'FEV/FEC ratio'], ['Cockayne syndrome', 'Cockayne syndrome type 1', 'Cockayne syndrome type 2', 'Cockayne syndrome type 3', 'UV-sensitive syndrome', 'genetic disorder', "Parkinson's disease", 'breast adenocarcinoma', 'self reported educational attainment', 'Isolated NADH-CoQ reductase deficiency'], ['cancer', 'endometrial cancer', 'lung adenocarcinoma', 'multiple myeloma', 'cutaneous melanoma', 'coronary artery calcification', 'gastric adenocarcinoma', 'mammographic density measurement', 'dense area measurement', 'neoplasm'], ['Isolated scaphocephaly', 'Isolated cloverleaf skull syndrome', 'Familial lambdoid synostosis', 'Chitayat syndrome', 'Isolated craniosynostosis', 'Craniosynostosis', 'Isolated oxycephaly', 'Lambdoidal craniosynostosis', 'genetic disorder', 'Crouzon disease'], ['mean corpuscular hemoglobin concentration', 'mean corpuscular volume', 'mean corpuscular hemoglobin', 'hemoglobin measurement', 'red blood cell distribution width', 'hematocrit', 'transferrin saturation measurement', 'serum iron measurement', 'Fuchs endothelial corneal dystrophy', 'Posterior polymorphous corneal dystrophy'], [], ['mean corpuscular volume', 'refractive error measurement', 'basophil count', 'prostate adenocarcinoma', 'B-cell acute lymphoblastic leukemia', 'neutrophil count', 'eosinophil count', 'monocyte count', 'mean corpuscular hemoglobin concentration', 'prostate intraepithelial neoplasia'], ['sitting height measurement', 'lean body mass', 'fat body mass', 'hyperplasia', 'neoplasm', 'cancer'], ['Neurogenic arthrogryposis multiplex congenita', 'Flexion contracture', 'mean corpuscular hemoglobin concentration', 'mean corpuscular volume', 'prostate cancer', 'whole body water mass', 'lean body mass', 'fat body mass', 'waist-hip ratio', 'BMI-adjusted waist-hip ratio'], ['body height', 'appendicular lean mass', 'interleukin-27 measurement', 'heel bone mineral density', 'protein measurement', 'low density lipoprotein cholesterol measurement', 'triglyceride measurement', 'sitting height measurement', 'vital capacity', 'mean platelet volume'], ['low density lipoprotein cholesterol measurement', 'total cholesterol measurement', 'lung cancer', 'platelet count', 'refractive error measurement', 'interleukin-27 measurement', 'triglyceride measurement', 'diet measurement', 'refractive error', 'appendicular lean mass'], ['neutrophil count', 'neutrophil percentage of leukocytes', 'leukocyte count', 'myeloid white cell count', 'lymphocyte percentage of leukocytes', 'granulocyte percentage of myeloid white cells', 'sum of basophil and neutrophil counts', 'sum of neutrophil and eosinophil counts', 'breast cancer', 'hepatocellular carcinoma'], ['red blood cell distribution width', 'mean corpuscular volume', 'mean corpuscular hemoglobin', 'body fat distribution', 'mean corpuscular hemoglobin concentration', 'platelet component distribution width', 'neuroticism measurement', 'lymphocyte count', 'Diuretic use measurement', 'platelet count'], ['eosinophil count', 'body height', 'mean corpuscular volume', 'eosinophil percentage of leukocytes', 'urate measurement', 'breast cancer', 'renal cell carcinoma', 'renal cell adenocarcinoma', 'sitting height measurement', 'glioblastoma multiforme'], ['infect', 'infection', 'diet measurement', 'Pseudohypoparathyroidism type 2', 'self reported educational attainment', 'Familial primary hypomagnesemia with normocalciuria and normocalcemia', 'Primary hypomagnesemia with secondary hypocalcemia', 'Familial isolated hyperparathyroidism', 'Familial parathyroid adenoma', 'African iron overload'], ['hemoglobin measurement', 'hematocrit', 'self reported educational attainment', 'adrenocortical insufficiency', 'functional gastric disease', 'daytime rest measurement', 'depressive symptom measurement', 'wellbeing measurement', 'drug use measurement', 'neoplasm'], ['cognitive function measurement', 'mathematical ability', 'waist circumference', 'self reported educational attainment', 'triglyceride measurement', 'metabolic syndrome', 'body fat percentage', 'visceral adipose tissue measurement', 'eosinophil peroxidase deficiency', 'Linear and whorled nevoid hypermelanosis'], ['lean body mass', 'protein measurement', 'obsessive-compulsive disorder', 'atypical teratoid rhabdoid tumor', 'Childhood disintegrative disorder', 'blood protein measurement', 'fat body mass', 'glioblastoma multiforme', 'oligodendroglioma', 'astrocytoma'], ['prostate carcinoma', 'testosterone measurement', 'reticulocyte count', "Crohn's disease", 'blood protein measurement', 'diabetic retinopathy', 'sitting height measurement', 'body height', 'clear cell renal carcinoma', 'red blood cell distribution width'], ['colorectal adenocarcinoma', 'Down syndrome', 'blood protein measurement', 'chronic interstitial cystitis', 'urinary tract infection', 'urinary system disease', 'nasopharyngeal neoplasm', 'eye disease', 'osteoporosis', 'Polyuria'], ['protein measurement', 'red blood cell distribution width'], ['body height', 'Interauricular communication', 'atrial heart septal defect', 'ulcerative colitis', 'intestinal cancer'], ['Holoprosencephaly', 'Cystic fibrosis', 'Mandibular prognathia', 'reticulocyte count', 'thyroid gland adenocarcinoma', 'hemoglobin measurement', 'hematocrit', 'mean corpuscular volume', 'breast adenocarcinoma', 'lymphocyte count'], ['Autosomal recessive spastic paraplegia type 62', 'Hereditary spastic paraplegia', 'Cystic fibrosis', 'Juvenile amyotrophic lateral sclerosis', 'cognitive function measurement', 'sex hormone-binding globulin measurement', 'intelligence', 'apolipoprotein A 1 measurement', 'self reported educational attainment', 'high density lipoprotein cholesterol measurement'], ['Autosomal recessive spastic paraplegia type 18', 'Hereditary spastic paraplegia', 'cancer', 'bone development disease', 'Cystic fibrosis', 'Neurodegeneration', 'Spastic paraplegia', 'Autosomal recessive non-syndromic intellectual disability', 'Intellectual disability', 'Recessive intellectual disability - motor dysfunction - multiple joint contractures'], ['red blood cell distribution width', 'reticulocyte count', 'Blackfan-Diamond anemia', 'Saethre-Chotzen syndrome', 'Delta-beta-thalassemia', 'Heinz body anemia', 'Hereditary persistence of fetal hemoglobin - beta-thalassemia', 'Pyruvate kinase hyperactivity', 'Alpha-thalassemia - myelodysplastic syndrome', 'Microcytic anemia with liver iron overload'], ['Periventricular nodular heterotopia', 'Nodular neuronal heterotopia', 'Intellectual disability', 'Seizure', 'self reported educational attainment', 'smoking status measurement', 'monocyte count', 'vital capacity', 'neutrophil count', 'DNA methylation'], ['Hereditary renal hypouricemia', 'Interauricular communication', 'atrial heart septal defect', 'medulloblastoma', 'ependymoma', 'adolescent idiopathic scoliosis', 'urate measurement', 'astrocytoma', 'multiple sclerosis', 'glioblastoma multiforme'], ['childhood onset asthma', 'adult onset asthma', 'acute myeloid leukemia', 'eosinophil count', 'hair color', 'protein measurement', 'monocyte count', 'sex hormone-binding globulin measurement', 'appendicular lean mass', 'adolescent idiopathic scoliosis'], ['type II diabetes mellitus', 'sex hormone-binding globulin measurement', "Crohn's disease", 'ulcerative colitis', 'ankylosing spondylitis', 'psoriasis', 'sclerosing cholangitis', 'neoplasm', 'breast cancer', 'glioblastoma multiforme'], ['mean corpuscular volume', 'Her2-receptor negative breast cancer', 'HER2 Positive Breast Carcinoma', 'HER2 negative breast carcinoma', 'breast neoplasm', "Barrett's esophagus", 'breast cancer', 'Autosomal recessive early-onset inflammatory bowel disease', 'cancer', 'Immunodeficiency due to MASP-2 deficiency'], ['heel bone mineral density', 'chronic kidney disease', 'urinary metabolite measurement', 'protein measurement', 'sitting height measurement', 'endometrial cancer', 'breast adenocarcinoma', 'low grade glioma', 'non-small cell lung carcinoma', 'gastric cancer'], ['blood protein measurement', 'type II diabetes mellitus', 'complication', 'endometrial cancer', 'Partial pancreatic agenesis', 'Transient neonatal diabetes mellitus', 'Exercise-induced hyperinsulinism', 'Hyperinsulinism due to glucokinase deficiency', 'hyperproinsulinemia', 'diabetic ketoacidosis'], ['bone fracture', 'peripheral arterial disease', 'vascular disease', 'grip strength measurement', 'lean body mass', 'urate measurement', 'appendicular lean mass', 'platelet component distribution width', 'FEV/FEC ratio', 'diastolic blood pressure'], ['reticulocyte count', 'psoriasis', 'Niemann-Pick disease type C', 'red blood cell distribution width', 'low density lipoprotein cholesterol measurement', 'reticulocyte measurement', 'type II diabetes mellitus', 'hair color', 'Red cell distribution width', 'eosinophil count'], ['macula measurement', 'hematocrit', 'eye measurement', 'Trichodysplasia - xeroderma', 'Griscelli disease type 3', 'Peeling skin syndrome type B', 'Uncombable hair syndrome', 'Hypertrichosis cubiti - short stature', 'Oculotrichodysplasia', 'Hypotrichosis simplex'], ['hair color', 'ulcerative colitis', 'hair colour measurement', 'balding measurement', 'retinal artery occlusion', 'body height', 'hair shape measurement', 'inflammatory bowel disease', 'skin pigmentation', 'lean body mass'], ['Hodgkins lymphoma', 'Hydrolethalus', 'neoplasm', 'acute myeloid leukemia', 'breast cancer', 'cancer', 'rheumatoid arthritis', 'choriocarcinoma', 'refractory anemia', 'Ataxia-telangiectasia'], ['seminoma', 'prostate carcinoma', 'dental caries', 'actinic keratosis', 'Silver-Russell syndrome due to 11p15 microduplication', 'blood metabolite measurement', 'docosahexaenoic acid measurement', 'lymphocyte count', 'glycerophospholipid measurement', 'eicosapentaenoic acid measurement'], ['body mass index', 'lung carcinoma', 'pancreatic adenocarcinoma', 'cortical surface area measurement', 'adult onset asthma', 'FEV/FEC ratio', 'neuroimaging measurement', 'brain measurement', 'colorectal adenocarcinoma', 'platelet count'], ['colon carcinoma', 'infect', 'infection'], ['nonpapillary renal cell carcinoma', 'infect', 'infection', 'cancer'], ['heel bone mineral density', 'bone density', 'ulcer disease', 'bone quantitative ultrasound measurement', 'mean corpuscular hemoglobin', 'seminoma', 'non-small cell lung carcinoma', 'mycosis fungoides', 'Myotonia fluctuans', 'Marfan syndrome'], ['breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'low grade glioma', 'blood protein measurement', 'endothelial cell-selective adhesion molecule measurement', 'testosterone measurement', 'schizophrenia', 'myxosarcoma', 'prostate carcinoma', 'smoking status measurement'], ['phospholipid measurement', 'neuroimaging measurement', 'endometrial cancer', 'brain volume measurement', 'urinary bladder cancer', 'glioblastoma multiforme', 'appendicular lean mass', 'neoplasm', 'glioma', 'melanoma'], ['Roberts syndrome', 'Roberts-SC phocomelia syndrome', 'acute myeloid leukemia', 'endometrial cancer', 'body height', 'glioblastoma multiforme', 'urinary bladder cancer', 'erythrocyte count', 'DNA methylation', 'renal cell carcinoma'], ['S-formylglutathione hydrolase measurement', 'protein measurement', 'cerebrospinal fluid biomarker measurement', 'systolic blood pressure', 'platelet component distribution width', 'adolescent idiopathic scoliosis', 'pulse pressure measurement', 'Brugada syndrome', 'Familial progressive cardiac conduction defect', 'Familial short QT syndrome'], ['drug use measurement', "Alzheimer's disease", 'skin aging', 'chronic obstructive pulmonary disease', 'astrocytoma', 'hepatocellular carcinoma'], ['endothelial cell-specific molecule 1 measurement', 'FEV/FEC ratio', 'neoplasm', 'prostate cancer', 'urinary bladder cancer', 'renal cell carcinoma', 'renal cell adenocarcinoma', 'chronic obstructive pulmonary disease', 'renal fibrosis', 'hepatocellular carcinoma'], ['spine bone mineral density', 'bone density', 'red blood cell density measurement', 'mean corpuscular volume', 'neoplasm', 'mean platelet volume', 'heel bone mineral density', 'breast cancer', 'lymphoma', 'myelodysplastic syndrome'], ['Autosomal recessive non-syndromic sensorineural deafness type DFNB', 'Usher syndrome type 1', 'Non-syndromic genetic deafness', 'Usher syndrome', 'deafness', 'multiple sclerosis', 'sensorineural hearing loss', 'genetic disorder', 'melanoma', 'autosomal recessive nonsyndromic deafness 9'], ['mathematical ability', 'protein measurement', 'Romano-Ward syndrome', 'Brugada syndrome', 'eosinophil count', 'Familial short QT syndrome', 'Jervell and Lange-Nielsen syndrome', 'Familial progressive cardiac conduction defect', 'Familial atrial fibrillation', 'myeloid white cell count'], ['breast carcinoma', 'breast cancer', 'breast neoplasm', 'neoplasm', 'breast adenocarcinoma', 'polycystic ovary syndrome', 'Estrogen resistance syndrome', 'infertility', 'Premature ovarian insufficiency', 'postmenopausal osteoporosis'], ['schizophrenia', 'breast cancer', 'neoplasm', 'postmenopausal osteoporosis', 'osteoporosis', 'breast carcinoma', 'menopause', 'schizoaffective disorder', 'hepatocellular carcinoma', 'breast neoplasm'], ['Autosomal recessive non-syndromic sensorineural deafness type DFNB', 'cancer', 'urate measurement', 'self reported educational attainment', 'lung adenocarcinoma', 'thyroid gland adenocarcinoma', 'body mass index', 'glioblastoma multiforme', 'cutaneous melanoma', 'diet measurement'], ['cleft lip', 'lung adenocarcinoma', 'breast adenocarcinoma', 'cutaneous melanoma', 'head and neck squamous cell carcinoma', 'breast cancer', 'nonpapillary renal cell carcinoma', 'neoplasm', 'high density lipoprotein cholesterol measurement', 'intelligence'], ['adenoid cystic carcinoma', 'neoplasm', 'Obesity due to melanocortin 4 receptor deficiency', 'breast cancer', 'urinary bladder cancer', 'Hereditary persistence of fetal hemoglobin - beta-thalassemia', 'Ghosal hematodiaphyseal dysplasia', 'Leukocyte adhesion deficiency type III', 'Intermediate osteopetrosis', 'Melorheostosis'], ['Autosomal recessive non-syndromic sensorineural deafness type DFNB', 'deafness', 'hearing loss', 'Non-syndromic genetic deafness', 'Rare genetic deafness', 'protein measurement', 'Tuberculosis', 'Premature ovarian insufficiency', 'dental caries', 'cancer'], ['hemoglobin measurement', 'hematocrit', 'red blood cell density measurement', 'adolescent idiopathic scoliosis', 'female reproductive system disease', 'reticulocyte count', 'erythrocyte count', 'autosomal dominant compelling helio-ophthalmic outburst syndrome', 'breast carcinoma', 'systolic blood pressure'], ['neoplasm', 'atrial fibrillation', 'leukocyte count', 'renal cell adenocarcinoma', 'renal cell carcinoma', 'drug use measurement', 'schizophrenia', '22q11.2 deletion syndrome', 'glioblastoma multiforme', 'parathyroid disease'], ['infect', 'infection', 'Mycobacterium infection', 'mycobacterial infectious disease', 'fetal growth restriction', 'granuloma', 'cancer', 'Tuberculosis', 'neoplasm', 'ependymoma'], ['lean body mass', 'body weight', 'fat body mass', 'body mass index', 'self reported educational attainment', 'body height', 'whole body water mass', 'base metabolic rate measurement', 'waist circumference', 'childhood onset asthma'], ['mean platelet volume', 'platelet count', 'body height', 'adolescent idiopathic scoliosis', 'reaction time measurement', 'portal hypertension', 'Hemoglobin H disease', 'Hemoglobin C - beta-thalassemia', 'mosaic loss of chromosome Y measurement', "Felty's syndrome"], ['chronotype measurement', 'mathematical ability', 'body fat percentage', 'fat body mass', 'coronary artery disease', 'hip circumference', 'red blood cell distribution width', 'Blackfan-Diamond anemia', 'vital capacity', 'Microcytic anemia with liver iron overload'], ['pancreatic carcinoma', 'age at menopause', 'hair shape measurement', 'BMI-adjusted waist-hip ratio', 'waist-hip ratio', 'breast carcinoma', 'balding measurement', 'type II diabetes mellitus', 'FEV/FEC ratio', 'age at first facial hair'], [], [], ['synovial sarcoma', 'Sickle cell anemia'], ['schizophrenia', 'red blood cell density measurement', 'risk-taking behaviour', 'autism spectrum disorder', 'erythrocyte count', 'infect', 'infection', 'mean corpuscular hemoglobin concentration', 'mean corpuscular hemoglobin', 'testosterone measurement'], ['Multiple acyl-CoA dehydrogenase deficiency', 'Elevated circulating glutaric acid concentration', 'Glutaric aciduria', 'Multiple acyl-CoA dehydrogenation deficiency, severe neonatal type', 'Multiple acyl-CoA dehydrogenation deficiency, mild type', 'central nervous system cancer', 'traffic air pollution measurement', 'glioma', 'hepatocellular carcinoma', 'heel bone mineral density'], ['Multiple acyl-CoA dehydrogenase deficiency', 'Elevated circulating glutaric acid concentration', 'Glutaric aciduria', 'Multiple acyl-CoA dehydrogenation deficiency, severe neonatal type', 'Multiple acyl-CoA dehydrogenation deficiency, mild type', 'chronic kidney disease', 'genetic disorder', "Behcet's syndrome", 'Blackfan-Diamond anemia', 'glaucoma'], ['mathematical ability', 'body height', 'Abnormality of refraction', 'osteosarcoma', 'Other metabolic disease', 'metabolic disease'], ['Multiple acyl-CoA dehydrogenase deficiency', 'Elevated circulating glutaric acid concentration', 'Glutaric aciduria', 'genetic disorder', 'Multiple acyl-CoA dehydrogenation deficiency, mild type', 'Multiple acyl-CoA dehydrogenation deficiency, severe neonatal type', 'hypertrophic cardiomyopathy', 'blood metabolite measurement', 'blood protein measurement', 'acylcarnitine measurement'], ['Noonan syndrome', 'Microcephaly - facio-cardio-skeletal syndrome, Hadziselimovic type', 'heel bone mineral density', 'Abnormal sperm morphology', 'ovarian carcinoma', 'pancreatic ductal adenocarcinoma', 'neoplasm', 'insulin resistance', 'Hereditary spastic paraplegia', 'type I diabetes mellitus'], ['Ethylmalonic encephalopathy', 'Isolated cytochrome C oxidase deficiency', 'Leigh syndrome', 'colorectal carcinoma', 'Blackfan-Diamond anemia', 'Autosomal dominant severe congenital neutropenia', 'X-linked severe congenital neutropenia', 'platelet storage pool deficiency', 'body height', 'Cyclic neutropenia'], ['lymphoid neoplasm', 'Ovarian Endometrioid Adenocarcinoma with Squamous Differentiation', 'breast ductal adenocarcinoma', 'carcinoma of liver and intrahepatic biliary tract', 'Hepatobiliary Neoplasm', 'esophageal adenocarcinoma', 'Endometrial Endometrioid Adenocarcinoma', 'bile duct carcinoma', 'refractive error measurement', 'wellbeing measurement'], ['optic cup area measurement', 'breast adenocarcinoma', 'hair shape measurement', 'Alopecia', "Barrett's esophagus", 'systolic blood pressure', 'monocyte count', 'glioma', 'platelet count', 'clear cell renal carcinoma'], ['glomerular filtration rate', 'glioblastoma multiforme', 'breast adenocarcinoma', 'glioma', 'astrocytoma', 'atypical teratoid rhabdoid tumor', 'neoplasm', 'gastric carcinoma', 'medulloblastoma', 'hair color'], ['lymphocyte count', 'systemic lupus erythematosus', 'monocyte count', 'platelet count', 'type II diabetes mellitus', 'neutrophil count', 'lymphocyte percentage of leukocytes', 'psoriasis', 'leukocyte count', 'celiac disease'], ['body mass index', 'lean body mass', 'heel bone mineral density', 'bone quantitative ultrasound measurement', 'bone density', 'body weight', 'bone fracture', 'fat body mass', 'central corneal thickness', 'waist circumference'], ['type II diabetes mellitus', 'lymphocyte count', 'platelet count', 'hair color', 'esophageal adenocarcinoma', 'lung carcinoma', 'myeloid white cell count', 'prostate carcinoma', 'prostate adenocarcinoma', 'Ewing sarcoma'], ['myocardial infarction', 'cardiovascular disease', 'cancer', 'prostate cancer', 'neoplasm', 'Hereditary elliptocytosis', 'Hypereosinophilic syndrome', 'body height', 'infect', 'infection'], ['prostate carcinoma', 'neutrophil count', 'Familial primary hypomagnesemia with normocalciuria and normocalcemia', 'Autosomal dominant secondary polycythemia', 'Cholesterol-ester transfer protein deficiency', 'Homozygous familial hypercholesterolemia', 'Dehydrated hereditary stomatocytosis', 'Hemolytic anemia due to red cell pyruvate kinase deficiency', 'Hereditary spherocytosis', 'Primary hypomagnesemia with secondary hypocalcemia'], ['neutrophil count', 'chin morphology measurement', 'obesity', 'brain measurement', 'neuroimaging measurement', 'monocyte percentage of leukocytes', 'mean corpuscular volume', 'platelet crit', 'platelet count', 'diet measurement'], ['prostate carcinoma', 'prostate adenocarcinoma', 'benign prostatic hyperplasia', 'malignant soft tissue neoplasm', 'Acinar Prostate Mucinous Adenocarcinoma', 'hemangioblastoma', 'Prostate Small Cell Carcinoma', 'Extraskeletal Ewing Sarcoma/Peripheral Primitive Neuroectodermal Tumor', 'Acinar Prostate Adenocarcinoma, Foamy Gland Variant', 'acute myeloid leukemia'], ['body mass index', 'high density lipoprotein cholesterol measurement', 'physical activity measurement', 'glomerular filtration rate', 'lean body mass', 'smoking behavior', 'gastric carcinoma', 'kidney neoplasm', 'lymphoid neoplasm', 'carcinoma of liver and intrahepatic biliary tract'], ['thrombocytopenia 5', 'cancer', 'Thrombocytopenia', 'acute myeloid leukemia', 'acute lymphoblastic leukemia', 'lymphoid neoplasm', 'Autosomal thrombocytopenia with normal platelets', 'body height', 'lean body mass', 'Familial platelet syndrome with predisposition to acute myelogenous leukemia'], ['rheumatoid arthritis', 'Eczema', 'platelet count', 'allergic rhinitis', 'multiple sclerosis', 'Niemann-Pick disease type C', 'leukocyte count', 'breast cancer', 'neoplasm', 'mean platelet volume'], ['heel bone mineral density', 'hepatocellular carcinoma', 'central nervous system cancer', 'neoplasm', 'colorectal adenocarcinoma', 'esophageal squamous cell carcinoma', 'cardiac hypertrophy', 'glioblastoma multiforme', 'hippocampal atrophy', 'heart failure'], ['appendicular lean mass', 'body height', 'FEV/FEC ratio', 'type II diabetes mellitus', 'vital capacity', 'platelet crit', 'platelet count', 'reticulocyte measurement', 'reticulocyte count', 'lean body mass'], ['body height', 'blood protein measurement', 'rheumatoid arthritis', 'refractory anemia', 'dilated cardiomyopathy', 'Hepatitis', 'systemic lupus erythematosus', 'congenital heart disease', 'coronary artery disease', 'Biliary atresia'], ['Ellis Van Creveld syndrome', 'Acrofacial dysostosis, Weyers type', 'Short rib-polydactyly syndrome, Majewski type', 'alkaline phosphatase measurement', 'Abnormality of refraction', 'appendicular lean mass', 'Craniometadiaphyseal dysplasia, wormian bone type', 'Metaphyseal chondrodysplasia, Schmid type', 'Spondyloepimetaphyseal dysplasia with multiple dislocations', 'Osteogenesis imperfecta type 3'], ['Ellis Van Creveld syndrome', 'Acrofacial dysostosis, Weyers type', 'Jeune syndrome', 'Short rib-polydactyly syndrome, Majewski type', 'Short rib-polydactyly syndrome, Beemer-Langer type', 'genetic disorder', 'Meckel syndrome', 'Abnormality of refraction', 'appendicular lean mass', 'Acromesomelic dysplasia, Grebe type'], ['alcohol consumption measurement', 'sex hormone-binding globulin measurement', 'type II diabetes mellitus', 'testosterone measurement', 'lean body mass', 'platelet count', 'hemorrhoid', 'alcohol use disorder measurement', 'endometrial carcinoma', 'aspartate aminotransferase measurement'], ['sex hormone-binding globulin measurement', 'alcohol consumption measurement', 'type II diabetes mellitus', 'platelet count', 'lean body mass', 'apolipoprotein A 1 measurement', 'alcohol use disorder measurement', 'monocyte percentage of leukocytes', 'hemorrhoid', 'aspartate aminotransferase measurement'], ['low density lipoprotein cholesterol measurement', 'monocyte count', 'multiple sclerosis', 'asthma', 'allergy', 'total cholesterol measurement', 'Epstein Barr virus nuclear antigen 1 IgG measurement', 'body mass index', 'testosterone measurement', 'pulse wave reflection index measurement'], ['age at menarche', 'serum alanine aminotransferase measurement', 'anhedonia measurement', 'lean body mass', 'schizophrenia', 'gonorrhea', 'cancer'], ['breast adenocarcinoma', 'white matter hyperintensity measurement', 'systolic blood pressure', 'mean corpuscular hemoglobin', 'heel bone mineral density', 'Fallot complex - intellectual disability - growth delay', 'Congenital unilateral hypoplasia of depressor anguli oris', 'caudate nucleus volume', 'Left ventricular noncompaction', 'neuroimaging measurement'], ['white matter hyperintensity measurement', 'glioblastoma multiforme', 'breast adenocarcinoma', 'clear cell renal carcinoma', 'ovarian cancer', 'head and neck squamous cell carcinoma', 'thyroid gland adenocarcinoma', 'lymphocyte count', 'gastric adenocarcinoma', 'esophageal cancer'], ['protein measurement', 'platelet crit', 'body height', 'alcohol consumption measurement', 'multiple sclerosis', 'systolic blood pressure', 'diastolic blood pressure', 'ulcerative colitis', "Crohn's disease"], ['systolic blood pressure', 'diastolic blood pressure', 'alcohol drinking', 'alcohol consumption measurement', 'mean arterial pressure', 'hypertension', 'pulse pressure measurement', 'vital capacity', 'drug use measurement', 'Diuretic use measurement'], ['urate measurement', 'Brachydactyly type A1', 'glomerular filtration rate', 'Familial digital arthropathy-brachydactyly', 'Brachydactyly type A2', 'Thumb stiffness - brachydactyly - intellectual disability', 'Upper limb mesomelic dysplasia', 'Brachydactyly type C', 'Thiemann disease, familial form', 'Brachydactyly-syndactyly, Zhao type'], ['Ewing sarcoma', 'clear cell sarcoma', 'osteosarcoma', 'myoepithelial tumor', 'Ewing sarcoma/peripheral primitive neuroectodermal tumor', 'desmoplastic small round cell tumor', 'undifferentiated pleomorphic sarcoma', 'rhabdomyosarcoma', 'alveolar rhabdomyosarcoma', 'chondrosarcoma'], ['hematocrit', 'systolic blood pressure', 'hypertension', 'hemoglobin measurement', 'blood pressure', 'childhood onset asthma', 'age at menopause', 'BMI-adjusted waist circumference', 'diastolic blood pressure', 'age at menarche'], ['mean platelet volume', 'neuroticism measurement', 'tiredness measurement', 'feeling "fed-up" measurement', 'emotional symptom measurement', 'insulin like growth factor measurement', 'feeling emotionally hurt measurement', 'Abnormality of refraction', 'platelet count', 'mood disorder'], ['chronotype measurement', 'circadian rhythm', 'mathematical ability', 'brain volume measurement', 'insomnia measurement', 'tiredness measurement', 'self reported educational attainment', 'lean body mass', 'cognitive function measurement', 'platelet component distribution width'], ['mean corpuscular volume', 'eosinophil count', 'mean corpuscular hemoglobin', 'eosinophil percentage of leukocytes', 'benign neoplasm', 'breast carcinoma', 'mean corpuscular hemoglobin concentration', 'age at menopause', 'keratinocyte carcinoma', 'basal cell carcinoma'], ['heart disease', 'mean corpuscular hemoglobin concentration', 'Cutaneous T-cell lymphoma', 'heel bone mineral density', 'bone density', 'cancer', 'Corpuscular Hemoglobin Content', 'glioblastoma multiforme', 'primary cutaneous amyloidosis', 'Posterior cortical atrophy'], ['cervical carcinoma', 'endometrial cancer', 'squamous cell lung carcinoma', 'head and neck squamous cell carcinoma', 'albuminuria', 'infection', 'infect', 'juvenile dermatomyositis', 'cervical adenocarcinoma', 'chronic kidney disease'], ['albuminuria', 'cervical carcinoma'], ['eosinophil count', 'hair colour measurement', 'hair color', 'sunburn', 'eosinophil percentage of leukocytes', 'autism', 'appendicular lean mass', 'intraocular pressure measurement', 'sum of eosinophil and basophil counts', 'body height'], ['hematocrit', 'endometrial cancer', 'hemoglobin measurement', 'erythrocyte count', 'hypospadias', 'squamous cell lung carcinoma', 'mean corpuscular hemoglobin concentration', 'head and neck squamous cell carcinoma', 'red blood cell density measurement', 'Exercise-induced hyperinsulinism'], ['monocyte count', 'high density lipoprotein cholesterol measurement', 'myeloid white cell count', 'eosinophil count', 'hemoglobin measurement', 'mean corpuscular volume', 'eosinophil percentage of leukocytes', 'mean corpuscular hemoglobin', 'systemic lupus erythematosus', 'Sepsis'], ['Meckel syndrome', 'monocyte count', 'platelet component distribution width', 'neutrophil count', 'eosinophil count', 'myeloid white cell count', 'diastolic blood pressure', 'mean platelet volume', 'platelet count', 'body mass index'], ['mean platelet volume', 'platelet count', 'serum gamma-glutamyl transferase measurement', 'platelet component distribution width', 'alkaline phosphatase measurement', 'mean corpuscular hemoglobin', 'primary biliary cirrhosis', 'alcohol consumption measurement', 'biliary liver cirrhosis', 'platelet crit'], ['cognitive function measurement', 'self reported educational attainment', 'mathematical ability', 'Meckel syndrome', 'intelligence', 'diet measurement', 'health study participation', 'vitamin D measurement', 'physical activity', 'physical activity measurement'], ['endometrial cancer', 'adolescent idiopathic scoliosis', 'squamous cell lung carcinoma', 'monocyte count', 'head and neck squamous cell carcinoma', 'cutaneous mastocytosis', 'hydronephrosis', 'Mastocytosis', 'Autosomal dominant polycystic kidney disease', 'cyst'], ['eosinophil count', 'type II diabetes mellitus', 'high density lipoprotein cholesterol measurement', 'lymphocyte count', 'neutrophil percentage of leukocytes', 'intraocular pressure measurement', 'lymphocyte percentage of leukocytes', 'endometrial cancer', 'platelet component distribution width', 'platelet count'], ['Spondyloepimetaphyseal dysplasia with joint laxity', 'genetic disorder', 'diet measurement', 'platelet crit', 'diastolic blood pressure', 'C-reactive protein measurement', 'body height', 'appendicular lean mass', 'reticulocyte count', 'drug use measurement'], ['Brain atrophy', 'developmental disability', 'microcephaly (disease)', 'Seizure', 'endometrial cancer', 'squamous cell lung carcinoma', 'hepatocellular carcinoma', 'lymphocyte count', 'neoplasm', 'cholangiocarcinoma'], ['hemoglobin measurement', 'high altitude adaptation', 'squamous cell lung carcinoma', 'hematocrit', 'Joubert syndrome', 'whole body water mass', 'head and neck squamous cell carcinoma', 'erythrocyte count', 'lymphocyte count', 'skin neoplasm'], ['electrocardiography', 'Brugada syndrome', 'QRS duration', 'disease of pilosebaceous unit', 'Epidermal Inclusion Cyst', 'glomerular filtration rate', 'QT interval', 'lean body mass', 'base metabolic rate measurement', 'fat body mass'], ['breast adenocarcinoma', 'Abnormal sperm morphology', 'acute megakaryoblastic leukaemia', 'myelofibrosis', 'Myelofibrosis', 'systemic juvenile idiopathic arthritis', 'infect', 'infection', 'cancer', 'type II diabetes mellitus'], ['gastric carcinoma', 'breast carcinoma', 'breast adenocarcinoma', 'Syndromic retinitis pigmentosa', 'Familial multiple lipomatosis', 'red blood cell distribution width', 'lymphocyte count', 'platelet count', 'leukocyte count', 'myeloid white cell count'], ['Pontocerebellar hypoplasia type 1', 'Non-syndromic pontocerebellar hypoplasia', 'Microcephaly', 'genetic disorder', 'Severe intrauterine growth retardation', 'Hypotonia', 'Classic lissencephaly', 'Fetal akinesia deformation sequence', 'infection', 'infect'], ['breast adenocarcinoma', 'mean corpuscular volume', 'mean corpuscular hemoglobin concentration', 'mean corpuscular hemoglobin', 'eosinophil count', 'blood metabolite measurement', 'Sepsis', 'serum metabolite measurement', 'metabolite measurement', 'bacteriemia'], ['isolated cerebellar hypoplasia/agenesis', 'Short stature due to GHSR deficiency', 'neuroblastoma', 'cerebellar ataxia', 'red blood cell distribution width', 'colorectal carcinoma', 'reticulocyte count', 'reticulocyte measurement', 'hair color', 'body height'], ['acute myeloid leukemia', 'multiple myeloma', 'B-cell non-Hodgkins lymphoma', 'unipolar depression', 'depressive disorder', 'neoplasm', 'acute lymphoblastic leukemia', 'breast cancer', 'skin aging measurement', 'cancer'], ['breast adenocarcinoma', 'pancreatic carcinoma', 'Malignant Pancreatic Neoplasm', 'pancreatic neoplasm', 'neoplasm', 'insulin resistance', 'cancer', 'blood protein measurement', 'infection', 'infect'], ['Pontocerebellar hypoplasia type 1', 'colorectal carcinoma', 'breast adenocarcinoma', 'Hemoglobin E - beta-thalassemia', 'infection', 'infect', 'Adult-onset autosomal recessive sideroblastic anemia', 'Delta-beta-thalassemia', 'Alpha-thalassemia - myelodysplastic syndrome', 'IRIDA syndrome'], ['Pontocerebellar hypoplasia type 1', 'lean body mass', 'gastric carcinoma', 'body height', 'appendicular lean mass', 'mean corpuscular volume', 'mean corpuscular hemoglobin', 'whole body water mass', 'fat body mass', 'body weight'], ['heel bone mineral density', 'refractive error measurement', 'lean body mass', 'appendicular lean mass', 'fat body mass', 'base metabolic rate measurement', 'whole body water mass', "Alzheimer's disease", 'body height', 'mean corpuscular volume'], ['Epidermolysis bullosa simplex', 'serum alanine aminotransferase measurement', 'lymphocyte count', 'platelet count', 'platelet crit', 'Familial hyperaldosteronism type II', 'protein measurement', 'Pseudohypoparathyroidism type 2', 'Hypokalemic periodic paralysis', 'Nephrogenic syndrome of inappropriate antidiuresis'], ['exostoses, multiple, type 1', 'Multiple osteochondromas', 'hereditary multiple exostoses', 'chondrosarcoma', 'Langer-Giedion syndrome', 'skin carcinoma', 'esophageal adenocarcinoma', 'Ovarian Endometrioid Adenocarcinoma with Squamous Differentiation', 'pancreatic neuroendocrine tumor', 'breast ductal adenocarcinoma'], ['exostoses, multiple, type 2', 'seizures-scoliosis-macrocephaly syndrome', 'Multiple osteochondromas', 'hereditary multiple exostoses', 'esophageal adenocarcinoma', 'hemangioblastoma', 'Hepatobiliary Neoplasm', 'lymphoid neoplasm', 'pancreatic ductal adenocarcinoma', 'basal cell carcinoma'], ['forced expiratory volume', 'mean corpuscular volume', 'myxosarcoma', 'peak expiratory flow', 'neuroblastoma', 'glioblastoma multiforme', 'platelet count', 'neoplasm', 'diet measurement', 'Tetralogy of Fallot'], ['multiple sclerosis', 'lymphocyte count', 'sum of eosinophil and basophil counts', 'neutrophil percentage of granulocytes', 'eosinophil count', 'lymphocyte percentage of leukocytes', 'chronic kidney disease', 'myeloid white cell count', 'leukocyte count', 'monocyte count'], ['immunoskeletal dysplasia with neurodevelopmental abnormalities', 'Skeletal dysplasia - intellectual disability', 'body height', 'white matter microstructure measurement', 'acute myeloid leukemia', 'Transient neonatal diabetes mellitus', 'adolescent idiopathic scoliosis', 'Partial pancreatic agenesis', 'Hyperinsulinism due to glucokinase deficiency', 'colorectal carcinoma'], ['BOR syndrome', 'Otofaciocervical syndrome', 'Branchio-otic syndrome', 'Rare genetic deafness', 'BMI-adjusted waist-hip ratio', 'physical activity measurement', 'waist-hip ratio', 'BMI-adjusted waist circumference', 'reticulocyte count', 'triglyceride measurement'], ['BMI-adjusted waist-hip ratio', 'heel bone mineral density', 'waist-hip ratio', 'bone quantitative ultrasound measurement', 'triglyceride measurement', 'sex hormone-binding globulin measurement', 'type II diabetes mellitus', 'physical activity measurement', 'vital capacity', 'bone density'], ['low grade glioma', 'breast adenocarcinoma', 'ovarian cancer', 'esophageal cancer', 'head and neck squamous cell carcinoma', 'lung adenocarcinoma', 'squamous cell lung carcinoma', 'high density lipoprotein cholesterol measurement', 'smoking status measurement', 'gastric adenocarcinoma'], ['Sensorineural deafness with dilated cardiomyopathy', 'Autosomal dominant non-syndromic sensorineural deafness type DFNA', 'hearing loss', 'deafness', 'heel bone mineral density', 'bone quantitative ultrasound measurement', 'bone density', 'BMI-adjusted waist-hip ratio', 'glomerular filtration rate', 'neutrophil count'], ['Retinitis pigmentosa', 'Retinal dystrophy', 'Macular dystrophy', 'self reported educational attainment', 'Central areolar choroidal dystrophy', 'Posterior column ataxia - retinitis pigmentosa', 'Retinitis punctata albescens', 'alcohol drinking', 'systolic blood pressure', 'physical activity measurement'], ['body height', 'waist-hip ratio', 'Mantle cell lymphoma', 'type II diabetes mellitus', 'cirrhosis of liver', 'Hepatic fibrosis', 'BMI-adjusted waist-hip ratio', 'lean body mass', 'hyperproinsulinemia', 'Bronchopulmonary dysplasia'], ['Weaver syndrome', 'lymphoma', 'non-Hodgkins lymphoma', 'melanoma', 'breast carcinoma', 'viral disease', 'sarcoma', 'diffuse large B-cell lymphoma', 'cutaneous melanoma', 'cancer'], ['ependymoma', 'neoplasm', 'male reproductive organ cancer', 'prostate cancer', 'prostate carcinoma', 'glioma', 'cancer', 'childhood supratentorial ependymoma', 'colorectal cancer', 'diffuse intrinsic pontine glioma'], ['hair color', 'non-small cell lung carcinoma', 'lung adenocarcinoma', 'Autosomal recessive non-syndromic intellectual disability', 'hair colour measurement', 'skin pigmentation', 'FEV/FEC ratio', 'gastric adenocarcinoma', 'testosterone measurement', 'Abnormality of refraction'], ['Congenital factor II deficiency', 'stroke', 'venous thromboembolism', 'deep vein thrombosis', 'Hereditary thrombophilia due to congenital protein C deficiency', 'atrial fibrillation', 'coronary artery disease', 'Thrombocytopenia', 'cardiovascular disease', 'Recurrent thrombophlebitis'], ['myocardial infarction', 'peripheral arterial disease', 'stroke', 'coronary artery disease', 'reticulocyte count', 'Recurrent thrombophlebitis', 'pancreatic adenocarcinoma', 'refractory anemia with excess blasts', 'chronic myelomonocytic leukemia', 'reticulocyte measurement'], ['mean platelet volume', 'glioblastoma multiforme', 'proteinase-activated receptor 1 measurement', 'monocyte count', 'head and neck squamous cell carcinoma', 'medulloblastoma', 'neoplasm', 'hepatocellular carcinoma', 'irritable bowel syndrome', 'inflammatory bowel disease'], ['ovarian cancer', 'breast adenocarcinoma', 'thyroid gland adenocarcinoma', 'platelet crit', 'mean corpuscular volume', 'head and neck squamous cell carcinoma', 'mean platelet volume', 'proteinase-activated receptor 1 measurement', 'mean corpuscular hemoglobin concentration', 'platelet count'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'cancer', 'neoplasm', 'colorectal cancer', 'acute lymphoblastic leukemia', 'esophageal squamous cell carcinoma', 'ovarian cancer', 'type II diabetes mellitus'], ['tissue factor measurement', 'D dimer measurement', 'coagulation factor measurement', 'cervical cancer', 'neutrophil count', 'squamous cell lung carcinoma', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'ovarian cancer', 'fallopian tube cancer'], ['Congenital factor V deficiency', 'thrombophilia due to activated protein C resistance', 'Venous thrombosis', 'Hereditary thrombophilia due to congenital protein C deficiency', 'Sepsis', 'stroke', 'habitual abortion', 'septic shock', 'deep vein thrombosis', 'Recurrent thrombophlebitis'], ['Congenital factor VII deficiency', 'Abnormality of coagulation', 'factor VII deficiency', 'factor VII measurement', 'prothrombin time measurement', 'blood protein measurement', 'Abnormal bleeding', 'coagulation factor measurement', 'protein measurement', 'tissue factor measurement'], ['Hemophilia A', 'Hemophilia B', 'Sepsis', 'Mild hemophilia A', 'Abnormality of coagulation', 'septic shock', 'thrombophilia', 'Symptomatic form of hemophilia A in female carriers', 'acute pancreatitis', 'Severe hemophilia A'], [], [], ['Huntington disease', 'neuroblastoma', 'Glycogen storage disease due to liver glycogen phosphorylase deficiency'], ['Hemophilia B', 'Hemophilia A', 'Venous thrombosis', 'thrombophilia, X-linked, due to factor 9 defect', 'warfarin sensitivity, X-linked', 'Abnormality of coagulation', 'thrombophilia', 'Severe hemophilia B', 'Symptomatic form of hemophilia B in female carriers', 'hemorrhage'], ['Congenital factor X deficiency', 'atrial fibrillation', 'venous thromboembolism', 'deep vein thrombosis', 'pulmonary embolism', 'Hemophilia A', 'stroke', 'COVID-19', 'Recurrent thrombophlebitis', 'Venous thrombosis'], ['Congenital factor XI deficiency', 'Hereditary angioedema', 'factor XI measurement', 'Abnormal bleeding', 'venous thromboembolism', 'deep vein thrombosis', 'pulmonary embolism', 'blood protein measurement', 'partial thromboplastin time', 'Thrombophlebitis'], ['breast adenocarcinoma', 'glioblastoma multiforme', 'optic cup area measurement', 'head and neck squamous cell carcinoma', 'gastric adenocarcinoma', 'breast cancer', 'neoplasm', 'hypertension', 'cancer', 'Miyoshi myopathy'], ['Congenital factor XII deficiency', 'Hereditary angioedema', 'Hereditary angioedema type 3', 'Hereditary angioedema type 1', 'angioedema', 'hypertension', 'Hyperbilirubinemia', 'vibratory urticaria', 'lung adenocarcinoma', 'cutaneous melanoma'], ['Congenital factor XIII deficiency', 'factor XIII, A subunit, deficiency of', 'Factor XIII subunit A deficiency', 'coagulation factor measurement', 'factor XIII deficiency', 'glucose-dependent insulinotropic peptide measurement', 'glucose tolerance test', 'DNA methylation', 'adolescent idiopathic scoliosis', 'facial morphology measurement'], ['Congenital factor XIII deficiency', 'factor XIII deficiency', 'coagulation factor measurement', 'blood protein measurement', 'lung adenocarcinoma', 'glioblastoma multiforme', 'breast adenocarcinoma', 'cutaneous melanoma', 'squamous cell lung carcinoma', 'cerebrospinal fluid biomarker measurement'], ['Autosomal recessive spastic paraplegia type 35', 'Hereditary spastic paraplegia', 'Spastic paraplegia', 'genetic disorder', 'Fatty acid hydroxylase-associated neurodegeneration', 'cognitive disorder', 'dystonic disorder', 'Seizure', 'Neurodegeneration with brain iron accumulation', 'drug use measurement'], ['migraine disorder', 'pain', 'Fever', 'osteoarthritis', 'Low back pain', 'toothache', 'influenza', 'Dysmenorrhea', 'common cold', 'Chronic pain'], ['Intellectual disability', 'autism spectrum disorder', 'mental retardation', 'head and neck squamous cell carcinoma', 'type II diabetes mellitus', 'androgenetic alopecia', 'psychiatric disorder', 'Turner syndrome', 'acromesomelic dysplasia, Demirhan type', 'coronary artery disease'], ['platelet count', 'electrocardiography', 'urate measurement', 'PR interval', 'breast adenocarcinoma', 'TPE interval measurement', 'body height', 'lean body mass', 'sitting height measurement', 'forced expiratory volume'], ['erythrocyte count', 'breast adenocarcinoma', 'creatinine measurement', 'red blood cell density measurement', 'bone quantitative ultrasound measurement', 'Fanconi anemia', 'Sepsis', 'breast cancer', 'calf circumference measurement', 'cholangiocarcinoma'], ['refractive error measurement', 'breast adenocarcinoma', 'red blood cell distribution width', 'mean corpuscular volume', 'erythrocyte count', 'red blood cell density measurement', 'mean corpuscular hemoglobin concentration', 'mean corpuscular hemoglobin', 'diet measurement', 'mean reticulocyte volume'], ['C-reactive protein measurement', 'serum alanine aminotransferase measurement', 'blood protein measurement', 'alkaline phosphatase measurement', 'hair color', 'sleep duration', 'esophageal cancer', 'glioblastoma multiforme', 'acute kidney failure', 'hepatocellular carcinoma'], ["cGMP-specific 3',5'-cyclic phosphodiesterase measurement", 'myeloid white cell count', 'platelet count', 'mean corpuscular volume', 'leukocyte count', 'platelet crit', 'appendicular lean mass', 'adolescent idiopathic scoliosis', 'BMI-adjusted waist circumference', 'necrotizing enterocolitis'], ['protein measurement', 'Nuclear cataract', 'Total congenital cataract', 'Zonular cataract', 'Cerulean cataract', 'acute kidney failure', 'Coppock-like cataract', 'Anterior polar cataract', 'Coralliform cataract', 'Hereditary hyperferritinemia with congenital cataracts'], ['breast adenocarcinoma', 'gestational diabetes', 'neoplasm', 'hydrops fetalis (disease)', 'Hand-foot syndrome', 'breast cancer', 'non-alcoholic steatohepatitis', 'coronary artery disease', 'acute kidney failure', 'diabetes mellitus'], ['pathological myopia', 'hepatocellular carcinoma', 'neoplasm', 'Posterior cortical atrophy', 'primary cutaneous amyloidosis', 'rheumatoid arthritis', 'refractory anemia', 'cholangiocarcinoma', 'chronic obstructive pulmonary disease', 'nonpapillary renal cell carcinoma'], ['mean platelet volume', 'platelet count', 'platelet component distribution width', 'platelet crit', 'body height', 'BMI-adjusted waist circumference', 'heel bone mineral density', 'cutaneous melanoma', 'lean body mass', 'glioblastoma multiforme'], ['protein measurement', 'acute myeloid leukemia', 'drug use measurement', 'neoplasm', 'glioblastoma multiforme', 'breast cancer', 'renal cell carcinoma', 'glioma', 'schizophrenia', 'nonpapillary renal cell carcinoma'], ['Non-syndromic male infertility due to sperm motility disorder', 'Male infertility due to globozoospermia', 'spermatogenic failure 42', 'spermatogenic failure 43', 'Partial chromosome Y deletion', 'spermatogenic failure 31', 'spermatogenic failure 29', 'congenital bilateral aplasia of vas deferens from CFTR mutation', 'spermatogenic failure 1', 'Autosomal recessive non-syndromic sensorineural deafness type DFNB'], ['high density lipoprotein cholesterol measurement', 'esophageal adenocarcinoma', 'Posterior cortical atrophy', 'primary cutaneous amyloidosis', 'prostate cancer', 'obesity', 'metabolic syndrome', 'Retinoblastoma'], ['FADD-related immunodeficiency', 'Abnormality of cardiovascular system morphology', 'genetic disorder', 'head and neck squamous cell carcinoma', 'lung adenocarcinoma', 'ovarian cancer', 'breast adenocarcinoma', 'FEV/FEC ratio', 'cutaneous melanoma', 'basophil count'], ['red blood cell distribution width', 'mean platelet volume', 'platelet count', 'chronic kidney disease', 'serum metabolite measurement', 'blood metabolite measurement', 'low density lipoprotein cholesterol measurement', 'mean corpuscular volume', 'neutrophil count', 'mean corpuscular hemoglobin concentration'], ['red blood cell distribution width', 'blood metabolite measurement', 'mean platelet volume', 'chronic kidney disease', 'serum metabolite measurement', 'fatty acid measurement', 'mean corpuscular volume', 'urate measurement', 'platelet count', 'omega-6 polyunsaturated fatty acid measurement'], ['sphingolipid measurement', 'phosphatidylcholine measurement', 'phosphatidylcholine 36:4 measurement', 'urate measurement', 'phosphatidylcholine 38:5 measurement', 'esterified cholesterol measurement', 'sphingomyelin measurement', 'alpha-linolenic acid measurement', 'serum metabolite measurement', 'adult onset asthma'], ['systemic scleroderma', 'scleroderma', 'X-linked retinal dysplasia', 'Familial exudative vitreoretinopathy', 'Severe early-childhood-onset retinal dystrophy', 'Reticular dystrophy of the retinal pigment epithelium', 'Familial drusen', 'Morning glory syndrome', 'X-linked retinoschisis', 'Familial retinal arterial macroaneurysm'], ['cataract', 'peak expiratory flow', 'lean body mass', 'mean corpuscular volume', 'body height', 'large artery stroke', 'Ischemic stroke', 'forced expiratory volume', 'type II diabetes mellitus', 'FEV/FEC ratio'], ['Gorham-Stout disease', 'age at menopause', 'Melorheostosis', 'body height', 'lung adenocarcinoma', 'atopic eczema', 'Ataxia-telangiectasia', 'viral disease', "Alzheimer's disease", 'Ulnar-mammary syndrome'], ['Tyrosinemia type 1', 'hypertrophic cardiomyopathy', 'mean platelet volume', 'Hypertyrosinemia', 'monocyte count', 'Pentosuria', 'atrial fibrillation', 'atrial flutter', 'Congenital bile acid synthesis defect type 4', 'Neonatal intrahepatic cholestasis due to citrin deficiency'], ['mean corpuscular hemoglobin', 'insulin like growth factor measurement', 'platelet crit', 'monocyte count', 'body height', 'sex hormone-binding globulin measurement', 'IGFBP-3 measurement', 'ulcerative colitis', 'non-Hodgkins lymphoma', 'neoplasm'], ['Delta-beta-thalassemia', 'Alpha-thalassemia - myelodysplastic syndrome', 'Hereditary persistence of fetal hemoglobin - beta-thalassemia', 'Hemoglobin C - beta-thalassemia', 'Hemoglobin E - beta-thalassemia', 'Beta-thalassemia - X-linked thrombocytopenia', 'Beta-thalassemia major', 'Beta-thalassemia intermedia', 'Hemoglobin H disease', 'X-linked dyserythropoetic anemia with abnormal platelets and neutropenia'], ['alcohol consumption measurement', 'bipolar disorder', 'schizophrenia', 'response to lithium ion', 'Delta-beta-thalassemia', 'Alpha-thalassemia - myelodysplastic syndrome', 'Hereditary persistence of fetal hemoglobin - beta-thalassemia', 'Hemoglobin C - beta-thalassemia', 'Hemoglobin E - beta-thalassemia', 'Beta-thalassemia - X-linked thrombocytopenia'], ['red blood cell distribution width', 'chronotype measurement', 'sunburn', 'testosterone measurement', 'obesity', 'vital capacity', 'aging', 'BMI-adjusted waist circumference', 'Congenital bile acid synthesis defect type 1', 'Congenital bile acid synthesis defect type 3'], ['body mass index', 'urate measurement', 'type II diabetes mellitus', 'body weight', 'lean body mass', 'blood urea nitrogen measurement', 'hip circumference', 'fat body mass', 'base metabolic rate measurement', 'whole body water mass'], ['hemoglobin measurement', 'mean corpuscular volume', 'hematocrit', 'Pentosuria', 'HbA1c measurement', 'Hyperglycemia', 'acute myeloid leukemia', 'type II diabetes mellitus', 'diabetes mellitus', 'insulin resistance'], ['cancer biomarker measurement', 'serum carcinoembryonic antigen measurement', 'blood protein measurement', 'body mass index', 'self reported educational attainment', 'mathematical ability', 'cognitive function measurement', 'intelligence', 'schizophrenia', 'body height'], ['bone density', 'bone quantitative ultrasound measurement', 'heel bone mineral density', 'radius bone mineral density', 'lean body mass', 'calcaneal bone quantitative ultrasound measurement', 'protein measurement', 'hip bone mineral density', 'osteoporosis', 'breast adenocarcinoma'], ['blood protein measurement', 'acute myeloid leukemia', 'obsessive-compulsive disorder', 'Hereditary geniospasm', 'Monoamine oxidase A deficiency', 'protein measurement', 'esophageal squamous cell carcinoma', 'low density lipoprotein cholesterol measurement', 'Benign familial chorea', 'colorectal carcinoma'], ['sex hormone-binding globulin measurement', 'cholelithiasis', 'mental or behavioural disorder', 'Glycogen storage disease due to liver glycogen phosphorylase deficiency', "Alzheimer's disease", 'meningococcal infection', 'ulcerative colitis'], ["Kaposi's sarcoma", 'Kindler syndrome', "Klinefelter's syndrome"], ['aortic valve disease', 'hepatocellular carcinoma', 'Microcystic corneal dystrophy', 'Epidermolysis bullosa simplex with muscular dystrophy', 'refractory anemia with excess blasts', 'Sjogren syndrome'], ['hepatocellular carcinoma', 'breast carcinoma', 'colorectal adenocarcinoma', 'myelodysplastic syndrome', 'cancer', 'Down syndrome', 'autism', "Crohn's disease"], ['chronic obstructive pulmonary disease', 'FEV/FEC ratio', 'vital capacity', 'smoking status measurement', 'response to bronchodilator', 'chronic bronchitis', 'myeloid white cell count', 'body height', 'refractive error measurement', 'hypertension'], ['atrial fibrillation', 'balding measurement', 'diet measurement', 'cardiac edema', 'atrial flutter', 'heart rate', 'chronic obstructive pulmonary disease', 'cardiac arrhythmia', 'type II diabetes mellitus', 'hematocrit'], ['prostate carcinoma', 'hemoglobin measurement', 'DNA methylation', 'urate measurement', 'reticulocyte count', 'vital capacity', 'ovarian carcinoma', 'systolic blood pressure', 'alcohol drinking', 'non-small cell lung carcinoma'], ['amelogenesis imperfecta type 1G', 'Amelogenesis imperfecta - nephrocalcinosis', 'heel bone mineral density', 'otosclerosis', 'sitting height measurement', 'phosphorus measurement', 'platelet count', 'Atypical dentin dysplasia due to SMOC2 deficiency', 'Oligodontia', 'amelogenesis imperfecta, type ij'], ['body height', 'arterial stiffness measurement', 'chondrosarcoma', 'Desbuquois syndrome', 'Ataxia-telangiectasia', 'Kashin-Beck disease', 'osteoarthritis', 'psoriasis', 'Lethal osteosclerotic bone dysplasia', "Alzheimer's disease"], ['Lethal osteosclerotic bone dysplasia', 'Hypoplastic amelogenesis imperfecta', 'Isolated focal cortical dysplasia type II', 'Midline cerebral malformation', 'Neonatal death', 'Abnormal facial shape', 'glioma', 'hematocrit', 'visual perception measurement', 'hemoglobin measurement'], ['red blood cell distribution width', 'low density lipoprotein cholesterol measurement', 'alcohol consumption measurement', 'mean reticulocyte volume', 'Red cell distribution width', 'mean corpuscular volume', 'total cholesterol measurement', 'alcohol drinking', 'mean corpuscular hemoglobin', 'reticulocyte count'], ['low density lipoprotein cholesterol measurement', 'red blood cell distribution width', 'alcohol consumption measurement', 'total cholesterol measurement', 'mean reticulocyte volume', 'Red cell distribution width', 'reticulocyte count', 'reticulocyte measurement', 'mean corpuscular hemoglobin', 'mean corpuscular volume'], ["Barrett's esophagus", 'esophageal squamous cell carcinoma', 'esophageal adenocarcinoma', 'low density lipoprotein cholesterol measurement', 'hemoglobin measurement', 'Colon Sessile Serrated Adenoma/Polyp'], [], [], ['monocyte count', 'lymphocyte count', 'Congenital hydronephrosis', 'systemic juvenile idiopathic arthritis', 'ovarian neoplasm', 'diabetes mellitus', 'breast cancer', 'Malignant epithelial tumor of ovary', 'ovarian carcinoma'], ['eosinophil count', 'eosinophil percentage of leukocytes', 'neutrophil count', 'reticulocyte measurement', 'hematocrit', 'hemoglobin measurement', 'erythrocyte count', 'eosinophil percentage of granulocytes', 'red blood cell density measurement', 'mean corpuscular hemoglobin concentration'], ['intelligence', 'self reported educational attainment', 'mathematical ability', 'protein measurement', 'hepatocellular carcinoma', 'type II diabetes mellitus', 'cognitive function measurement', 'hemoglobin measurement', 'basophil count', 'alkaline phosphatase measurement'], ['Abnormality of neuronal migration', 'self reported educational attainment', 'breast carcinoma', 'Interauricular communication', 'atrial heart septal defect', 'Intellectual disability', 'mental retardation'], [], ['breast ductal adenocarcinoma', 'embryonal rhabdomyosarcoma', 'gastric adenocarcinoma', 'pancreatic neuroendocrine tumor', 'skin carcinoma', 'Merkel cell skin cancer', 'oral squamous cell carcinoma', 'bladder transitional cell carcinoma', 'bile duct adenocarcinoma', 'clear cell renal carcinoma'], ['sex hormone-binding globulin measurement', "Parkinson's disease", 'platelet count', 'testosterone measurement', 'alkaline phosphatase measurement', 'mean corpuscular hemoglobin concentration', 'high density lipoprotein cholesterol measurement', 'sleep duration', 'mean corpuscular volume', 'body mass index'], ['low grade glioma', 'glioblastoma multiforme', 'gastric carcinoma', 'cancer', 'ovarian carcinoma', 'systemic juvenile idiopathic arthritis', 'breast cancer', 'neoplasm', 'breast neoplasm', 'breast carcinoma'], ['protein measurement', 'adolescent idiopathic scoliosis', 'clavicle fracture', 'seminoma', 'embryonal carcinoma', 'Esophagitis', "Barrett's esophagus", 'Beckwith-Wiedemann syndrome', 'germ cell tumor', 'neoplasm'], ['mean corpuscular volume', 'osteoarthritis, hip', 'osteoarthritis, knee', 'breast cancer', 'osteoarthritis', 'neuroimaging measurement', 'cortical surface area measurement', 'appendicular lean mass', 'lean body mass', 'fat body mass'], ['lymphocyte count', 'body height', 'basophil count', 'neutrophil count', 'balding measurement', 'myeloid white cell count', 'hair shape measurement', 'monocyte count', 'androgenetic alopecia', 'smoking behavior'], ['schizophrenia', 'bipolar disorder', 'obsessive-compulsive disorder', 'unipolar depression', 'autism spectrum disorder', 'attention deficit hyperactivity disorder', 'anorexia nervosa', 'Tourette syndrome', 'eosinophil count', 'lean body mass'], ['neutrophil count', 'leukocyte count', 'platelet count', 'mean platelet volume', 'mean corpuscular volume', 'erythrocyte count', 'red blood cell distribution width', 'basophil count', 'mean corpuscular hemoglobin concentration', 'monocyte count'], ['Thyroid preparation use measurement', 'heel bone mineral density', 'T-cell immunoglobulin and mucin domain 1 measurement', 'respiratory system disease', 'low density lipoprotein cholesterol measurement', 'total cholesterol measurement', 'mean corpuscular volume', 'leukocyte count', 'triglyceride measurement', 'eosinophil count'], ['smoking status measurement', 'alcohol dependence', 'methamphetamine dependence', 'heroin dependence', 'body mass index', 'self reported educational attainment', 'body weight', 'cognitive function measurement', 'cirrhosis of liver', 'fat body mass'], ['unipolar depression', 'reticulocyte count', 'appendicular lean mass', 'high density lipoprotein cholesterol measurement', 'body height', 'Eyelid malformation', 'lacrimal apparatus disease'], ['red blood cell density measurement', 'hemoglobin measurement', 'hematocrit', 'atrial flutter', 'atrial fibrillation', 'cardiac arrhythmia', 'chronic obstructive pulmonary disease', 'pancreatic adenocarcinoma', 'erythrocyte count', 'drug use measurement'], ['smoking status measurement', 'sitting height measurement', 'vital capacity', 'osteoarthritis, hip', 'small cell lung carcinoma'], ['46,XY partial gonadal dysgenesis', 'cryptorchidism', 'Hypoparathyroidism - deafness - renal disease', 'renal hypodysplasia/aplasia 3', 'self reported educational attainment', 'Familial glucocorticoid deficiency', 'atrial flutter', 'atrial fibrillation', 'intelligence', 'obesity'], ['physical activity', 'Abnormal sperm morphology', 'prostate carcinoma', 'seminoma', 'gastric carcinoma'], ['Autosomal recessive polycystic kidney disease', 'experimental autoimmune encephalomyelitis', 'Late-onset nephronophthisis', 'Infundibulopelvic stenosis - multicystic kidney', 'Congenital hydronephrosis', 'renal hypodysplasia/aplasia 3', 'Autosomal dominant polycystic kidney disease type 1 with tuberous sclerosis', 'Autosomal dominant familial hematuria - retinal arteriolar tortuosity - contractures', 'polycystic liver disease 4 with or without kidney cysts', 'polycystic liver disease 3 with or without kidney cysts'], ['neutrophil count', 'breast carcinoma', 'Autosomal recessive polycystic kidney disease', 'Late-onset nephronophthisis', 'Infundibulopelvic stenosis - multicystic kidney', 'Congenital hydronephrosis', 'renal hypodysplasia/aplasia 3', 'Autosomal dominant polycystic kidney disease type 1 with tuberous sclerosis', 'Autosomal dominant familial hematuria - retinal arteriolar tortuosity - contractures', 'polycystic liver disease 3 with or without kidney cysts'], ['optic cup area measurement', 'Autosomal recessive polycystic kidney disease', 'Late-onset nephronophthisis', 'Infundibulopelvic stenosis - multicystic kidney', 'Congenital hydronephrosis', 'renal hypodysplasia/aplasia 3', 'Autosomal dominant polycystic kidney disease type 1 with tuberous sclerosis', 'Autosomal dominant familial hematuria - retinal arteriolar tortuosity - contractures', 'polycystic liver disease 3 with or without kidney cysts', 'polycystic liver disease 4 with or without kidney cysts'], ['Autosomal recessive polycystic kidney disease', 'Late-onset nephronophthisis', 'Infundibulopelvic stenosis - multicystic kidney', 'Congenital hydronephrosis', 'renal hypodysplasia/aplasia 3', 'Autosomal dominant polycystic kidney disease type 1 with tuberous sclerosis', 'Autosomal dominant familial hematuria - retinal arteriolar tortuosity - contractures', 'polycystic liver disease 3 with or without kidney cysts', 'polycystic liver disease 4 with or without kidney cysts', 'Medullary Cystic Kidney Disease Type II'], ['myeloid white cell count'], ['body height', 'Eczema', 'allergic rhinitis', 'anxiety', 'asthma', 'seasonal allergic rhinitis', 'appendicular lean mass', 'reticulocyte count', 'hypothyroidism', 'cup-to-disc ratio measurement'], ['electrocardiography', 'PR interval', 'mean platelet volume', 'neoplasm', 'chronic interstitial cystitis', 'renal cell adenocarcinoma', 'renal cell carcinoma', 'meningococcal infection', 'Malignant Pancreatic Neoplasm', 'pancreatic neoplasm'], ['mathematical ability', 'self reported educational attainment', 'risk-taking behaviour', 'acute myeloid leukemia', 'lymphocyte count', 'pancreatic adenocarcinoma', 'atypical teratoid rhabdoid tumor', 'ependymoma', "Alzheimer's disease", 'cardiac arrhythmia'], ['drug use measurement', 'glioblastoma multiforme', 'astrocytoma', 'Leishmaniasis', 'ependymoma', 'oligodendroglioma', 'neoplasm'], ['ependymoma', 'serum non-albumin protein measurement', 'central nervous system cancer', 'nasopharyngeal neoplasm', 'Tetralogy of Fallot', 'Abnormal sperm morphology', 'IGA glomerulonephritis', 'endometrial carcinoma', 'non-small cell lung carcinoma', 'Pick disease'], ['lean body mass', 'non-small cell lung carcinoma', 'lung cancer', 'hepatocellular carcinoma', 'pancreatic carcinoma', 'Malignant Pancreatic Neoplasm', 'pancreatic neoplasm', 'cancer', 'attention deficit hyperactivity disorder', 'lung adenocarcinoma'], ['chronotype measurement', 'age at menarche', 'potassium measurement', 'adolescent idiopathic scoliosis', 'sleep duration', 'lean body mass', 'blood urea nitrogen measurement', 'creatinine measurement', 'glomerular filtration rate', 'fat body mass'], ['prostate carcinoma', 'reticulocyte measurement', 'sunburn', 'reticulocyte count', 'Immature Reticulocyte Fraction Measurement', 'appendicular lean mass', 'body height', 'diet measurement', 'body weight', 'skin cancer'], ['sex hormone-binding globulin measurement', 'neoplasm', 'hepatocellular carcinoma', 'cancer', 'ovarian cancer', 'breast cancer', 'gonorrhea', 'nose morphology measurement', 'esophageal squamous cell carcinoma', 'gastric cancer'], ['neutrophil count', 'myeloid white cell count', 'lymphocyte count', 'lymphocyte percentage of leukocytes', 'monocyte count', 'neutrophil percentage of leukocytes', 'granulocyte count', 'sum of neutrophil and eosinophil counts', 'sum of basophil and neutrophil counts', 'leukocyte count'], ['prostate carcinoma', 'type II diabetes mellitus', 'hepatocellular carcinoma', 'reticulocyte measurement', 'non-small cell lung carcinoma', 'pancreatic adenocarcinoma', 'Uterine leiomyoma', 'Renal insufficiency', 'chronotype measurement', 'uterine fibroid'], ['1,5 anhydroglucitol measurement', 'Uncombable hair syndrome', 'Ringed hair disease', 'Pili bifurcati', 'Hypotrichosis simplex', 'Alopecia-intellectual disability syndrome', 'Trichodysplasia - xeroderma', 'Alopecia totalis', 'Autosomal dominant secondary polycythemia', 'Pili torti'], ['Hypocalcified amelogenesis imperfecta', 'Amelogenesis imperfecta', 'lean body mass', 'fat body mass', 'body weight', 'whole body water mass', 'base metabolic rate measurement', 'osteosarcoma', 'hip circumference', 'colorectal cancer'], ['Delta-beta-thalassemia', 'Alpha-thalassemia - myelodysplastic syndrome', 'Hereditary persistence of fetal hemoglobin - beta-thalassemia', 'Hemoglobin C - beta-thalassemia', 'Hemoglobin E - beta-thalassemia', 'peak expiratory flow', 'Beta-thalassemia - X-linked thrombocytopenia', 'Beta-thalassemia major', 'Beta-thalassemia intermedia', 'Hemoglobin H disease'], ['Delta-beta-thalassemia', 'Alpha-thalassemia - myelodysplastic syndrome', 'Hereditary persistence of fetal hemoglobin - beta-thalassemia', 'Hemoglobin C - beta-thalassemia', 'Hemoglobin E - beta-thalassemia', 'Beta-thalassemia - X-linked thrombocytopenia', 'Beta-thalassemia major', 'Beta-thalassemia intermedia', 'Hemoglobin H disease', 'X-linked dyserythropoetic anemia with abnormal platelets and neutropenia'], ['erythrocyte count', 'Sepsis', 'mean platelet volume', 'body height', 'myxosarcoma', 'bacterial disease', 'endometrial carcinoma', 'myelodysplastic syndrome', 'viral disease', 'infect'], ['FEV/FEC ratio', 'osteoarthritis, hip', 'type II diabetes mellitus', 'alkaline phosphatase measurement', 'red blood cell distribution width', 'coronary artery disease', 'Red cell distribution width', 'refractive error measurement', 'body height', 'cup-to-disc ratio measurement'], ['HIV-1 infection', 'viral load', 'gut microbiome measurement', 'neoplasm', 'COVID-19', 'Sepsis', "Alzheimer's disease"], [], ['Non-syndromic male infertility due to sperm motility disorder', 'Female infertility due to zona pellucida defect', 'Male infertility due to large-headed multiflagellar polyploid spermatozoa', 'female infertility due to oocyte meiotic arrest', 'spermatogenic failure 31', 'spermatogenic failure 1', 'Male infertility due to globozoospermia', 'spermatogenic failure 43', 'Deafness-infertility syndrome', 'spermatogenic failure 42'], ['body height', 'protein measurement', 'early onset hypertension', 'mathematical ability', 'gut microbiome measurement', 'optic cup area measurement', 'self reported educational attainment', 'reticulocyte count', 'white matter microstructure measurement', 'brain volume measurement'], ['systemic lupus erythematosus', 'birth weight', 'mathematical ability', 'Abnormal sperm morphology', 'body height', 'inflammatory bowel disease', 'platelet crit', 'type I diabetes mellitus', 'appendicular lean mass', 'lymphocyte count'], ['Short rib-polydactyly syndrome, Majewski type', 'eosinophil count', 'neutrophil percentage of granulocytes', 'eosinophil percentage of leukocytes'], ['urate measurement', 'monocyte count', 'myeloid white cell count', 'neutrophil count', 'primary angle closure glaucoma', 'leukocyte count', 'albumin:globulin ratio measurement', 'bacteriemia', 'Sepsis', 'septic shock'], ['mean platelet volume', 'neutrophil count', 'sum of basophil and neutrophil counts', 'leukocyte count', 'ependymoma', 'oligodendroglioma', 'ovarian carcinoma', 'astrocytoma', 'large cell medulloblastoma', 'chronic kidney disease'], ['mean corpuscular volume', 'mean corpuscular hemoglobin concentration', 'mean corpuscular hemoglobin', 'red blood cell distribution width', 'erythrocyte count', 'reticulocyte count', 'COG1-CDG', 'mean reticulocyte volume', 'Red cell distribution width', 'jaw fracture'], ['visceral Leishmaniasis'], ['acute myeloid leukemia', 'neoplasm', 'laryngeal neoplasm', 'non-small cell lung carcinoma', 'breast carcinoma', 'lung adenocarcinoma', 'laryngeal carcinoma', 'low density lipoprotein cholesterol measurement', 'medulloblastoma', 'lung carcinoma'], ['neuroimaging measurement', 'brain volume measurement', 'brain measurement', 'cortical thickness', 'platelet count', 'platelet component distribution width', 'acute myeloid leukemia', 'platelet crit', 'mean platelet volume', 'Autosomal recessive non-syndromic sensorineural deafness type DFNB'], ['fat body mass', 'BMI-adjusted waist-hip ratio', 'COVID-19', 'visual perception measurement', 'waist-hip ratio', 'triglyceride measurement', 'sex hormone-binding globulin measurement', 'high density lipoprotein cholesterol measurement', 'apolipoprotein A 1 measurement', 'breast ductal carcinoma in situ'], ['response to polyunsaturated fatty acid supplementation', 'delta-5 desaturase measurement', 'spinal fracture', 'refractive error measurement', 'emotional symptom measurement', 'neuroticism measurement', 'non-small cell lung carcinoma', 'Autosomal dominant secondary polycythemia', 'polycythemia vera', 'X-linked dyserythropoetic anemia with abnormal platelets and neutropenia'], ['attempted suicide', 'bipolar disorder', 'psoriasis', 'hepatocellular carcinoma', 'chronic interstitial cystitis', 'Kidney Oncocytoma', 'Osteopenia', 'bone fracture', 'osteoporosis', 'medulloblastoma'], ['heel bone mineral density', 'leukocyte count', 'mean corpuscular volume', 'myeloid white cell count', 'body height at birth', 'lung adenocarcinoma', 'uterine leiomyosarcoma', 'breast carcinoma'], ['Osteocraniostenosis', 'Kenny-Caffey syndrome', 'Autosomal dominant Kenny-Caffey syndrome', 'heel bone mineral density', 'prostate carcinoma', 'genetic disorder', 'Microcephaly', 'mathematical ability', 'sex hormone-binding globulin measurement', 'brain volume measurement'], ['Hereditary sclerosing poikiloderma with tendon and pulmonary involvement', 'Kyphosis', 'myositis', 'Short stature due to GHSR deficiency', 'prostate carcinoma', 'sex hormone-binding globulin measurement', 'neoplasm', 'mathematical ability', 'coagulation protein disease', 'central nervous system cancer'], ['childhood onset asthma', 'red blood cell density measurement', 'mean corpuscular hemoglobin concentration', 'mean corpuscular volume', 'lymphocyte count', 'asthma', 'allergy', 'mean corpuscular hemoglobin', 'allergic sensitization measurement', 'C-C motif chemokine 3 measurement'], ['cancer', 'red blood cell distribution width', 'self reported educational attainment', 'body mass index', 'Red cell distribution width', 'apolipoprotein A 1 measurement', 'lung adenocarcinoma', 'breast adenocarcinoma', 'lean body mass', 'attempted suicide'], ['red blood cell distribution width', 'mean corpuscular volume', 'mean corpuscular hemoglobin', 'erythrocyte count', 'mean corpuscular hemoglobin concentration', 'mosaic loss of chromosome Y measurement', 'FEV/FEC ratio', 'COVID-19', 'response to vaccine', 'cytokine measurement'], ['low density lipoprotein cholesterol measurement', 'reticulocyte count', 'total cholesterol measurement', 'apolipoprotein B measurement', 'high density lipoprotein cholesterol measurement', 'Sarcoidosis', 'monocyte percentage of leukocytes', 'immune system disease', 'apolipoprotein A 1 measurement', 'type II diabetes mellitus'], ['heel bone mineral density', 'puberty onset measurement', 'balding measurement', 'hair shape measurement', 'age at first facial hair', 'chronotype measurement', 'dental caries', 'prostate carcinoma', 'ankylosing spondylitis', "Crohn's disease"], ['neutrophil count', 'eosinophil count', 'leukocyte count', 'forced expiratory volume', 'eosinophil percentage of leukocytes', 'body height', 'FEV/FEC ratio', 'mean corpuscular hemoglobin', 'peak expiratory flow', 'vital capacity'], ['neuroticism measurement', 'self reported educational attainment', 'tiredness measurement', 'depressive symptom measurement', 'cognitive function measurement', 'platelet count', 'insomnia', 'neurotic disorder', 'insomnia measurement', 'mathematical ability'], ['Hypotonia', 'Coarse facial features', 'chronic lung disease', 'scoliosis', 'Cutis laxa', 'Pectus excavatum', 'gastroesophageal reflux disease', 'self reported educational attainment', 'platelet count', 'neuroticism measurement'], ['risk-taking behaviour', 'daytime rest measurement', 'monocyte count', 'intraocular pressure measurement', 'hematocrit', 'glaucoma', 'lymphocyte count', 'hair shape measurement', 'balding measurement', 'age at menarche'], ['Intellectual disability', 'autism', 'infect', 'infection', "Kaposi's sarcoma", 'Microcephaly', 'infectious disease', 'autism spectrum disorder'], ['Abnormality of refraction', 'Nasal Cavity Polyp', 'body height', 'Ghosal hematodiaphyseal dysplasia', 'portal hypertension', 'Hemoglobin H disease', 'facial morphology measurement', 'optic cup area measurement', 'optic disc area measurement', 'cup-to-disc ratio measurement'], ['lean body mass', 'whole body water mass', 'BMI-adjusted waist circumference', 'body height', 'vital capacity', 'appendicular lean mass', 'inflammatory biomarker measurement', 'platelet-derived growth factor BB measurement', 'obsessive-compulsive disorder', 'BMI-adjusted waist-hip ratio'], ['Hypomyelination - congenital cataract', 'Leukodystrophy', 'adolescent idiopathic scoliosis', 'Thrombocytopenia', 'attention deficit hyperactivity disorder', 'Hemolytic anemia due to red cell pyruvate kinase deficiency', 'Hemoglobin E - beta-thalassemia', 'diabetes mellitus, permanent neonatal 4', 'Adult-onset autosomal recessive sideroblastic anemia', 'Congenital dyserythropoietic anemia type III'], ['chronic lymphocytic leukemia', 'monocyte count', 'mean platelet volume', 'rheumatoid arthritis', 'prostate carcinoma', 'mean corpuscular volume', 'serum alanine aminotransferase measurement', 'red blood cell density measurement', 'appendicular lean mass', 'body height'], ['appendicular lean mass', 'body mass index', 'body weight', 'lean body mass', 'cognitive function measurement', 'hip circumference', 'intelligence', 'self reported educational attainment', 'erythrocyte count', 'age at menarche'], ['cancer', 'lung adenocarcinoma', 'colorectal adenocarcinoma', 'astrocytoma', 'alveolar rhabdomyosarcoma', 'pancreatic neuroendocrine tumor', 'gastric carcinoma', 'gastric adenocarcinoma', 'hepatocellular carcinoma', 'melanoma'], ['Hypercholesterolemia', 'testosterone measurement', 'erythrocyte count', 'hematocrit', 'mean corpuscular hemoglobin concentration', 'Inguinal hernia', 'non-small cell lung carcinoma', 'hemoglobin measurement', 'red blood cell density measurement', 'Abnormality of metabolism/homeostasis'], ['3-hydroxy-1-methylpropylmercapturic acid measurement', 'smoking behavior', 'non-small cell lung carcinoma', 'hepatocellular carcinoma', 'small cell lung carcinoma', 'Tetralogy of Fallot', 'colorectal cancer', 'glioma'], ['cardiovascular disease', 'heel bone mineral density', 'lean body mass', 'monocyte percentage of leukocytes', 'bone quantitative ultrasound measurement', 'forced expiratory volume', 'body height', 'eosinophil count', 'appendicular lean mass', 'erythrocyte count'], ['type II diabetes mellitus', 'mean platelet volume', 'breast carcinoma', 'Retinitis pigmentosa', 'Sepsis', 'septic shock', 'infect', 'infection', 'cutaneous mastocytosis', 'Congenital myasthenic syndromes'], ['squamous cell lung carcinoma', 'head and neck squamous cell carcinoma', 'Uterine Carcinosarcoma', 'acute myeloid leukemia', 'breast carcinoma', 'esophageal squamous cell carcinoma', 'colorectal adenocarcinoma', 'gastric carcinoma', 'breast milk measurement', 'parental genotype effect measurement'], ['Meniere disease', 'low density lipoprotein cholesterol measurement', 'total cholesterol measurement', 'hair colour measurement', 'Alopecia', 'balding measurement', 'vital capacity', 'sex hormone-binding globulin measurement', 'insulin like growth factor measurement', 'testosterone measurement'], ['acute myeloid leukemia', "Hashimoto's thyroiditis", 'hypothyroidism', 'Thyroid preparation use measurement', 'autoimmune thyroid disease', 'drug use measurement', 'myxedema', 'blood protein measurement', 'protein measurement', 'plasma kallikrein measurement'], ['Joubert syndrome 36', 'Joubert syndrome with orofaciodigital defect', 'Joubert syndrome', 'serum alanine aminotransferase measurement', 'glomerular filtration rate', 'systolic blood pressure', "Alzheimer's disease", 'aldosterone-producing adenoma'], ['free cholesterol measurement', 'intermediate density lipoprotein measurement', 'clear cell renal carcinoma', "Crohn's disease", 'protein measurement', 'hepatocellular carcinoma', 'blood protein measurement', 'papillary renal cell carcinoma', 'Kidney Oncocytoma', 'acute myeloid leukemia'], ['hair color', 'hair colour measurement', 'X-linked retinal dysplasia', 'psoriasis', 'Central areolar choroidal dystrophy', 'Reticular dystrophy of the retinal pigment epithelium', 'Severe early-childhood-onset retinal dystrophy', 'Familial exudative vitreoretinopathy', 'Familial drusen', 'Morning glory syndrome'], ['body height', 'blood protein measurement', 'mathematical ability', 'eosinophil count', 'Gastrointestinal stromal tumor'], ['Red cell distribution width', 'mean corpuscular volume', 'red blood cell distribution width', 'central nervous system cancer', 'chronotype measurement', 'atypical teratoid rhabdoid tumor', 'glioblastoma multiforme', 'medulloblastoma', 'ependymoma', 'astrocytoma'], ['Posterior cortical atrophy', 'primary cutaneous amyloidosis'], ['multiple sclerosis'], ['Retinitis pigmentosa', 'Retinal dystrophy', 'genetic disorder', 'Cone rod dystrophy', 'Posterior column ataxia - retinitis pigmentosa', 'protein measurement', 'reticulocyte count', 'amino acid measurement', 'threonine measurement', 'methionine measurement'], ['ectonucleoside triphosphate diphosphohydrolase 5 measurement', 'body height', 'platelet component distribution width', 'protein measurement', 'red blood cell distribution width', 'birth weight', 'mean corpuscular volume', 'blood protein measurement', 'mean corpuscular hemoglobin concentration', 'peak expiratory flow'], ['alcohol dependence', 'risky sexual behaviour measurement', 'diastolic blood pressure', 'systolic blood pressure', 'resting heart rate', 'diet measurement', 'mean corpuscular volume', 'self reported educational attainment', 'phosphorus measurement', 'low density lipoprotein cholesterol measurement'], ['parental genotype effect measurement', 'birth weight', 'eosinophil count', 'C-reactive protein measurement', 'hyperlipidemia', 'Lipid storage disease', 'creatine kinase measurement', 'non-small cell lung carcinoma', 'apolipoprotein A 1 measurement', 'forced expiratory volume'], ['monocyte percentage of leukocytes', 'smoking behavior', 'squamous cell lung carcinoma', 'diabetes mellitus, permanent neonatal 4', 'Permanent neonatal diabetes mellitus', 'Sickle cell anemia', 'Hemochromatosis type 4', 'Dehydrated hereditary stomatocytosis', 'Hemolytic anemia due to red cell pyruvate kinase deficiency', 'Adult-onset autosomal recessive sideroblastic anemia'], ['smoking cessation', 'cigarettes per day measurement', 'smoking behavior', 'smoking status measurement', 'smoking behaviour measurement', 'nicotine dependence', 'tobacco smoke exposure measurement', 'leukocyte count', 'neutrophil count', 'time to first cigarette measurement'], ['osteosarcoma', 'IGA glomerulonephritis'], ['Meckel syndrome', 'protein measurement', 'autosomal recessive nonsyndromic deafness 9', 'Autosomal recessive non-syndromic sensorineural deafness type DFNB', 'Autosomal dominant non-syndromic sensorineural deafness type DFNA', 'Hereditary renal hypouricemia', 'endometrial carcinoma', 'multiple symmetric lipomatosis', 'Autosomal dominant Emery-Dreifuss muscular dystrophy'], ['Hereditary persistence of fetal hemoglobin - beta-thalassemia', 'Ghosal hematodiaphyseal dysplasia', 'Melorheostosis', 'Isolated osteopoikilosis', 'Alpha-thalassemia - myelodysplastic syndrome', 'Dacryocystitis - osteopoikilosis', 'Pyknoachondrogenesis', 'Osteomesopyknosis', 'Delta-beta-thalassemia', 'Hemoglobin C - beta-thalassemia'], ['systolic blood pressure', 'smoking status measurement', 'MODY', 'alcohol consumption measurement', 'mathematical ability', 'systemic lupus erythematosus', 'alcohol drinking', 'rheumatoid arthritis', 'systemic scleroderma', 'mucocutaneous lymph node syndrome'], ['lymphocyte count', 'Tetralogy of Fallot', 'autism', 'chronic interstitial cystitis', 'ulcerative colitis', "Crohn's disease", 'Huntington disease'], ['platelet count', 'smoking status measurement', 'smoking behavior', 'tumor necrosis factor receptor superfamily member 19L measurement', 'apolipoprotein A 1 measurement', 'blood protein measurement', 'Abnormality of refraction', 'lung cancer', 'chronic myelogenous leukemia', 'breast cancer'], ["Alzheimer's disease", 'protein measurement', 'coinfection', 'Ehlers-Danlos syndrome, progeroid type', 'neoplasm', 'inflammatory bowel disease', 'irritable bowel syndrome', 'infection', 'infect', 'colitis'], ['protein measurement', 'C-reactive protein measurement', 'low density lipoprotein cholesterol measurement', 'total cholesterol measurement', 'lipoprotein measurement', 'blood metabolite measurement', 'heel bone mineral density', 'DNA methylation', 'colorectal adenocarcinoma', 'clear cell renal carcinoma'], ['body height', 'sitting height measurement', 'monocyte count', 'decubitus ulcer', 'monocyte percentage of leukocytes', 'balding measurement', 'hair shape measurement', 'heel bone mineral density', 'optic cup area measurement', 'appendicular lean mass'], ['acute myeloid leukemia', 'body mass index', 'FEV/FEC ratio', 'drug use measurement', 'body height', 'Non-syndromic male infertility due to sperm motility disorder', 'Partial chromosome Y deletion', 'spermatogenic failure 43', 'Isolated follicle stimulating hormone deficiency', 'spermatogenic failure 42'], ['monocyte count', 'basophil count', 'diet measurement', 'reticulocyte count', 'neuroimaging measurement', 'platelet component distribution width', 'brain measurement', 'insulin like growth factor measurement', 'leukocyte count', 'mean corpuscular hemoglobin concentration'], ['visual perception measurement', 'peak expiratory flow', "Parkinson's disease", 'pituitary cancer', 'transitional cell carcinoma of kidney', 'clear cell renal carcinoma', 'bacteriemia', 'breast cancer', 'neoplasm', 'Sepsis'], ['acute myeloid leukemia', 'mean platelet volume', 'blood protein measurement', 'platelet component distribution width', 'platelet count', 'progranulin measurement', 'drug use measurement', "Parkinson's disease", "Alzheimer's disease", 'platelet crit'], ['blood protein measurement', 'body height', 'brain volume measurement', 'neuroimaging measurement', 'FEV/FEC ratio', 'asthma', 'inflammatory bowel disease', 'vital capacity', 'handedness', 'BMI-adjusted waist-hip ratio'], ['cortical surface area measurement', 'diet measurement', 'neuroimaging measurement', 'brain measurement', 'hemorrhoid', 'mathematical ability', 'age at menarche', 'unipolar depression', 'neuroticism measurement', 'lifestyle measurement'], ['heel bone mineral density', 'neutrophil count', 'bone density', 'acute myeloid leukemia', 'myeloid white cell count', 'colorectal cancer', 'Abnormal sperm morphology', 'Sepsis', 'cancer', 'ulcerative colitis'], ['astrocytoma', 'systolic blood pressure', 'medulloblastoma', 'pulse pressure measurement', 'prostate adenocarcinoma', 'gut microbiome measurement', 'drug use measurement', 'chronic interstitial cystitis', 'psoriasis', "Alzheimer's disease"], ['mean corpuscular volume', 'mean corpuscular hemoglobin', 'mean corpuscular hemoglobin concentration', 'erythrocyte count', 'myxosarcoma', 'Sjogren syndrome'], ['Intellectual disability', 'mental retardation', 'Dolichocephaly', 'obesity', 'Macrocephaly', 'monocyte percentage of leukocytes', 'lymphocyte count', 'monocyte count', 'neutrophil percentage of leukocytes', 'appendicular lean mass'], ['cardiovascular disease', 'appendicular lean mass', 'coronary artery disease', "Barrett's esophagus", 'esophageal adenocarcinoma', 'myocardial infarction', 'non-small cell lung carcinoma', 'Myocardial Ischemia', 'coronary atherosclerosis', 'skin atrophy'], ['emotional symptom measurement', 'allergic rhinitis', 'body height', 'schizophrenia', 'response to lithium ion', 'bipolar disorder', 'self reported educational attainment', 'alcohol consumption measurement', 'vital capacity', 'non-small cell lung carcinoma'], ['self reported educational attainment', 'mathematical ability', 'sitting height measurement', 'central nervous system cancer', 'hepatocellular carcinoma', 'red blood cell density measurement', 'erythrocyte count', 'mean platelet volume', 'eosinophil percentage of leukocytes', 'mean corpuscular volume'], ['body mass index', 'body fat percentage', 'lean body mass', 'fat body mass', 'platelet count', 'hip circumference', 'forced expiratory volume', 'glaucoma', 'neutrophil count', 'neuroticism measurement'], ['FEV/FEC ratio', 'monocyte count', 'glioma', 'vital capacity', 'Down syndrome', 'self reported educational attainment', 'eosinophil count', 'eosinophil percentage of leukocytes', 'lifestyle measurement', 'neutrophil percentage of granulocytes'], ['central nervous system cancer', 'glioblastoma multiforme', 'adolescent idiopathic scoliosis', 'protein measurement', 'disease progression measurement', 'blood protein measurement', 'Ataxia-telangiectasia', 'oligodendroglioma', 'pituitary cancer', 'astrocytoma'], ['adolescent idiopathic scoliosis', 'central nervous system cancer', 'Down syndrome', 'COVID-19', 'Intellectual disability', 'ulcerative colitis', 'lung adenocarcinoma', 'lymph node metastatic carcinoma', 'endometrial carcinoma', "Crohn's disease"], ['body fat percentage', 'hip circumference', 'body weight', 'self reported educational attainment', 'FEV/FEC ratio', 'intelligence', 'schizophrenia', 'adolescent idiopathic scoliosis', 'electrocardiography', 'appendicular lean mass'], ['DNA methylation', 'platelet count', 'platelet crit', 'heel bone mineral density', 'bone quantitative ultrasound measurement', 'mathematical ability', 'self reported educational attainment', 'acute myeloid leukemia', 'cognitive function measurement', 'body height'], ['chronotype measurement', 'type II diabetes mellitus', 'circadian rhythm', 'diverticulitis', 'diverticular disease', 'hemorrhoid', 'diabetes mellitus', 'response to cold pressor test', 'mean arterial pressure'], ['thyroid gland adenocarcinoma', 'neutrophil count', 'leukocyte count', 'Immature Reticulocyte Fraction Measurement', 'granulocyte count', 'sum of neutrophil and eosinophil counts', 'sum of basophil and neutrophil counts', 'myeloid white cell count', 'drug use measurement', 'hypertension'], ['Nephronophthisis', 'pulse pressure measurement', 'vital capacity', 'smoking status measurement', 'hypertension', 'systolic blood pressure', 'forced expiratory volume', 'diet measurement', 'intelligence', 'smoking behaviour measurement'], ['Primary ciliary dyskinesia', 'Kartagener Syndrome', 'Situs inversus totalis', 'Absent inner and outer dynein arms', 'pulse pressure measurement', 'chronotype measurement', 'white matter microstructure measurement', 'childhood onset asthma'], ['reticulocyte count', 'reticulocyte measurement', 'Immature Reticulocyte Fraction Measurement', 'red blood cell distribution width', 'monocyte count', 'mean platelet volume'], ['skin pigmentation measurement', 'heel bone mineral density', 'hair colour measurement', 'glioblastoma multiforme', 'gastric carcinoma', 'medulloblastoma', 'astrocytoma', 'ulcerative colitis', 'oligodendroglioma', "Crohn's disease"], ['intelligence', 'self reported educational attainment', 'prostate carcinoma', 'schizophrenia', 'hemoglobin measurement', 'red blood cell density measurement', 'cognitive function measurement', 'mathematical ability', 'insomnia', 'lung adenocarcinoma'], ['risk-taking behaviour', 'Red cell distribution width', 'hepatocellular carcinoma', "Crohn's disease", 'type II diabetes mellitus', 'red blood cell distribution width', 'vitamin D measurement', 'inflammatory bowel disease', 'magnesium measurement', 'renal system measurement'], ['systolic blood pressure', 'cognitive function measurement', 'eosinophil count', 'self reported educational attainment', 'chronotype measurement', 'intelligence', 'blood pressure', 'hip bone size', 'eosinophil percentage of leukocytes', 'pulse pressure measurement'], ['mean corpuscular hemoglobin', 'appendicular lean mass', 'hemoglobin measurement', 'hematocrit', 'heel bone mineral density', 'mathematical ability', 'self reported educational attainment', 'whole body water mass', 'uterine fibroid', 'Menorrhagia'], ['obsessive-compulsive disorder', 'Double uterus - hemivagina - renal agenesis', '46,XX ovotesticular disorder of sex development', 'meningococcal infection', 'type I diabetes mellitus'], ['blood metabolite measurement', 'myeloid white cell count', 'platelet count', 'dehydroepiandrosterone sulphate measurement', 'serum metabolite measurement', 'hormone measurement', 'body mass index', 'X12063 measurement', 'sitting height measurement', 'reticulocyte count'], ['reticulocyte count', 'urinary metabolite measurement', 'chronic kidney disease', 'mean corpuscular volume', 'cerebrospinal fluid biomarker measurement', 'red blood cell density measurement', 'mean corpuscular hemoglobin', 'mean corpuscular hemoglobin concentration', 'risk-taking behaviour', 'mean reticulocyte volume'], ['triglyceride measurement', 'sitting height measurement', 'hemoglobin measurement', 'body height', 'body fat percentage', 'pancreatic carcinoma', 'adolescent idiopathic scoliosis', 'freckles', 'urate measurement', 'skin pigmentation measurement'], ['vital capacity', 'blood protein measurement', 'eosinophil count', 'eosinophil percentage of leukocytes', 'rheumatoid arthritis', 'ankylosing spondylitis', "Crohn's disease", 'sclerosing cholangitis', 'psoriasis', 'ulcerative colitis'], ['eosinophil count', 'psoriasis', 'eosinophil percentage of leukocytes', 'allergy', 'asthma', 'self reported educational attainment'], ['hair color', 'hair shape measurement', 'smoking status measurement', 'systolic blood pressure', 'protein measurement', 'balding measurement', 'hair colour measurement', 'Abnormality of refraction', 'Alopecia', 'snoring measurement'], ['hair color', 'hair shape measurement', 'smoking status measurement', 'systolic blood pressure', 'balding measurement', 'hair colour measurement', 'Alopecia', 'snoring measurement', 'protein measurement', 'Abnormality of refraction'], ['heel bone mineral density', 'bone quantitative ultrasound measurement', 'bone fracture', 'bone density', 'wrist fracture', 'femoral neck bone mineral density', 'alkaline phosphatase measurement', 'C-reactive protein measurement', 'reticulocyte count', 'hip bone mineral density'], ['forced expiratory volume', 'appendicular lean mass', 'body height', 'lean body mass', 'fat body mass', 'mean corpuscular volume', 'mean corpuscular hemoglobin concentration', 'whole body water mass', 'mean corpuscular hemoglobin', 'sitting height measurement'], ['high density lipoprotein cholesterol measurement', 'body mass index', 'drug use measurement', 'body height', 'waist-hip ratio', 'protein measurement', 'mental process', 'eye neoplasm', 'oral squamous cell carcinoma', 'chronic kidney disease'], ['platelet count', 'monocyte count', 'neutrophil count', 'mean corpuscular hemoglobin concentration', 'myeloid white cell count', 'mean corpuscular hemoglobin', 'mean corpuscular volume', 'colorectal cancer', 'Colon Sessile Serrated Adenoma/Polyp', 'infectious disease'], ['body fat percentage', 'hemoglobin measurement', 'body mass index', 'Total congenital cataract', 'Nuclear cataract', 'Zonular cataract', 'Cerulean cataract', 'Anterior polar cataract', 'Coppock-like cataract', 'Coralliform cataract'], ['childhood onset asthma', 'chronic venous insufficiency', 'Monoamine oxidase A deficiency', 'hypothyroidism', 'endometrial carcinoma', 'lung adenocarcinoma', 'nasopharyngeal neoplasm', 'lung carcinoma', 'non-small cell lung carcinoma', 'squamous cell carcinoma'], ['balding measurement', 'depressive symptom measurement', 'adolescent idiopathic scoliosis', 'protein measurement', 'blood metabolite measurement', 'Abnormal sperm morphology'], ['self reported educational attainment', 'lymphocyte count', 'mathematical ability', 'adolescent idiopathic scoliosis', 'Nuclear cataract', 'Total congenital cataract', 'Zonular cataract', 'Cerulean cataract', 'rheumatoid arthritis', 'Coralliform cataract'], ['diet measurement', 'Atypical dentin dysplasia due to SMOC2 deficiency', 'Temtamy preaxial brachydactyly syndrome', 'Spondyloepimetaphyseal dysplasia - abnormal dentition', 'amelogenesis imperfecta, type ij', 'Dental ankylosis', 'Non-eruption of teeth - maxillary hypoplasia - genu valgum', 'Anonychia - microcephaly', 'Red cell distribution width', 'body mass index'], ['mean corpuscular hemoglobin concentration', 'mean corpuscular hemoglobin', 'smoking status measurement', 'diastolic blood pressure', 'systolic blood pressure', 'red blood cell distribution width', 'alcohol drinking', 'coffee consumption', 'neutrophil percentage of leukocytes', 'mean arterial pressure'], ['lymphocyte percentage of leukocytes', 'neutrophil percentage of leukocytes', 'C-reactive protein measurement', 'hip circumference', 'fat body mass', 'lean body mass', 'body weight', 'whole body water mass', 'neutrophil count', 'base metabolic rate measurement'], ['type II diabetes mellitus', 'vital capacity', 'chronotype measurement', 'body height', 'mathematical ability', 'alkaline phosphatase measurement', 'oral squamous cell carcinoma', 'ovarian carcinoma', 'dysplasia', 'prostate carcinoma'], ['hip circumference', 'hemojuvelin measurement', 'heel bone mineral density', 'body height', 'red blood cell distribution width'], ['self reported educational attainment', 'Abnormality of brain morphology', 'mathematical ability', 'cognitive function measurement', "Alzheimer's disease", 'Familial multiple lipomatosis', 'peak expiratory flow', 'hip circumference', 'Tetralogy of Fallot', 'platelet count'], ['mean corpuscular hemoglobin concentration', 'mean corpuscular hemoglobin', 'mean corpuscular volume', 'red blood cell density measurement', 'red blood cell distribution width', 'hemoglobin measurement', 'reticulocyte count', 'erythrocyte count', 'reticulocyte measurement', 'insomnia measurement'], ['heel bone mineral density', 'mean corpuscular hemoglobin concentration', 'mathematical ability', 'high density lipoprotein cholesterol measurement', 'mean corpuscular volume', 'C-reactive protein measurement', 'bone quantitative ultrasound measurement', 'platelet count', 'age at menopause', 'prostate carcinoma'], ['type II diabetes mellitus', 'lung adenocarcinoma', 'aspartate aminotransferase measurement', 'FEV/FEC ratio', 'low density lipoprotein cholesterol measurement', 'apolipoprotein B measurement', 'breast carcinoma', 'forced expiratory volume', 'Thyrotoxicosis', 'nontoxic goiter'], ['neutrophil count', 'myeloid white cell count', 'leukocyte count', 'low density lipoprotein cholesterol measurement', 'fat body mass', 'appendicular lean mass', 'granulocyte count', 'body fat percentage', 'lean body mass', 'generalised epilepsy'], ['lifestyle measurement', 'myeloid white cell count', 'neutrophil count', 'appendicular lean mass', 'granulocyte count', 'platelet component distribution width', 'low density lipoprotein cholesterol measurement', 'leukocyte count', 'sum of neutrophil and eosinophil counts', 'body fat percentage'], ['HIV-1 infection', 'viral load', 'gastric carcinoma'], ['body height', 'protein measurement', 'adolescent idiopathic scoliosis', 'pancreatic adenocarcinoma', 'nasopharyngeal neoplasm', 'non-small cell lung carcinoma'], ['HbA1c measurement', 'mean corpuscular hemoglobin concentration', 'mean corpuscular volume', 'hemoglobin measurement', 'mean corpuscular hemoglobin', 'type II diabetes mellitus', 'erythrocyte count', 'red blood cell distribution width', 'bilirubin measurement', 'hematocrit'], ['appendicular lean mass', 'body height', 'serum albumin measurement', 'breast cancer', 'heel bone mineral density', 'bone quantitative ultrasound measurement', 'bone density', 'Cholesterol-ester transfer protein deficiency', 'Homozygous familial hypercholesterolemia', 'neoplasm'], [], [], [], [], ['acute myeloid leukemia', 'blood protein measurement', 'diet measurement', 'body fat percentage'], ['homocysteine measurement', 'type II diabetes mellitus', 'FEV/FEC ratio', 'papillary thyroid carcinoma'], ['blood protein measurement', 'teratocarcinoma-derived growth factor 1 measurement', 'neutrophil percentage of granulocytes', 'body height', 'eosinophil percentage of granulocytes', 'C-X-C motif chemokine 16 measurement', 'colorectal adenocarcinoma', 'papillary thyroid carcinoma'], ['atrial flutter', 'atrial fibrillation'], ['Myopia', 'age at onset', 'Eczema', 'refractive error measurement', 'adult onset asthma', 'childhood onset asthma', 'brain aneurysm', 'eosinophil count', 'eosinophil percentage of leukocytes', 'prostate carcinoma'], ['lymphocyte count', 'neutrophil percentage of leukocytes', 'lymphocyte percentage of leukocytes', 'monocyte count', 'insomnia measurement', 'body mass index', 'alcohol dependence', 'mathematical ability', 'adolescent idiopathic scoliosis', 'mean platelet volume'], ['type II diabetes mellitus', 'body height', 'glioblastoma multiforme', 'astrocytoma', 'Ataxia-telangiectasia', 'Abnormal sperm morphology', 'neoplasm', 'oligodendroglioma', 'ependymoma', 'Uterine leiomyoma'], ['coronary artery disease', 'myocardial infarction', 'FEV/FEC ratio', 'body height', 'pulmonary function measurement', 'angina pectoris', 'waist-hip ratio', 'pulse pressure measurement', 'BMI-adjusted waist-hip ratio', 'Myocardial Ischemia'], [], [], [], [], ['Karyomegalic interstitial nephritis', 'chronic kidney disease', 'Lynch syndrome', 'interstitial nephritis', 'low grade glioma', 'breast adenocarcinoma', 'Crigler-Najjar syndrome type 2', 'Sarcosinemia', 'Huntington disease', 'Transient familial neonatal hyperbilirubinemia'], ['Fanconi anemia complementation group A', 'Fanconi anemia', 'acute myeloid leukemia', 'leukemia', 'prostate carcinoma', 'basal cell carcinoma', 'head and neck squamous cell carcinoma', 'Abnormality of radial ray', 'kidney neoplasm', 'hemangioblastoma'], ['Fanconi anemia', 'VACTERL with hydrocephalus', 'Fanconi anemia complementation group A', 'VACTERL association, X-linked, with or without hydrocephalus', 'acute myeloid leukemia', 'Fanconi anemia complementation group C', 'Abnormality of radial ray', 'Anorectal malformation', 'low grade glioma', 'breast adenocarcinoma'], ['Fanconi anemia', 'Fanconi anemia complementation group C', 'acute myeloid leukemia', 'Fanconi anemia complementation group A', 'Inherited cancer-predisposing syndrome', 'leukemia', 'lymphoid neoplasm', 'kidney neoplasm', 'bile duct carcinoma', 'hearing loss'], ['Fanconi anemia', 'acute myeloid leukemia', 'Fanconi anemia complementation group A', 'leukemia', 'colorectal adenocarcinoma', 'genetic disorder', 'prostate carcinoma', 'lung carcinoma', 'bile duct carcinoma', 'carcinoma of liver and intrahepatic biliary tract'], ['Fanconi anemia', 'body mass index', 'lean body mass', 'fat body mass', 'body fat percentage', 'Fanconi anemia complementation group A'], ['Fanconi anemia complementation group E', 'Fanconi anemia', 'acute myeloid leukemia', 'colorectal adenocarcinoma', 'leukemia', 'cutaneous melanoma', 'Abnormality of radial ray', 'prostate carcinoma', 'head and neck squamous cell carcinoma', 'hepatocellular carcinoma'], ['Fanconi anemia complementation group F', 'Fanconi anemia', 'leukemia', 'acute myeloid leukemia', 'prostate carcinoma', 'Abnormality of radial ray', 'Gastric Adenoma', 'hepatocellular carcinoma', 'head and neck squamous cell carcinoma', 'squamous cell lung carcinoma'], ['Fanconi anemia complementation group G', 'Fanconi anemia', 'acute myeloid leukemia', 'leukemia', 'colorectal adenocarcinoma', 'hearing loss', 'Abnormality of radial ray', 'prostate carcinoma', 'lung carcinoma', 'hemangioblastoma'], ['Fanconi anemia', 'Fanconi anemia complementation group A', 'acute myeloid leukemia', 'Abnormality of radial ray', 'bone marrow failure syndrome', 'stomach neoplasm', 'breast adenocarcinoma', 'glioblastoma multiforme', 'red blood cell distribution width', 'age at menopause'], ['Fanconi anemia', 'Fanconi anemia complementation group L', 'Fanconi anemia complementation group A', 'acute myeloid leukemia', 'Abnormality of radial ray', 'wellbeing measurement', 'mathematical ability', 'VACTERL with hydrocephalus', 'VACTERL/VATER association', 'VACTERL association, X-linked, with or without hydrocephalus'], ['Fanconi anemia', 'spermatogenic failure 28', 'Hereditary breast cancer', 'azoospermia', 'Non-acquired premature ovarian failure', 'Male infertility with azoospermia or oligozoospermia due to single gene mutation', 'primary ovarian insufficiency', 'malignant germ cell tumor of ovary', 'Hereditary breast and ovarian cancer syndrome', 'Fanconi anemia complementation group A'], ['cortical surface area measurement', 'neuroimaging measurement', 'mean corpuscular hemoglobin', 'mean corpuscular volume', 'sitting height measurement', 'red blood cell distribution width', 'blood protein measurement', 'late-onset Alzheimers disease', 'appendicular lean mass', 'erythrocyte count'], ['blind loop syndrome', 'leukocyte count', 'eosinophil count', 'autoimmune disease', 'glomerular filtration rate', 'Vitiligo', 'myeloid white cell count', 'hypothyroidism', 'psoriasis', 'lymphocyte count'], ['protein measurement', 'vitamin D measurement', 'reticulocyte count', 'Ghosal hematodiaphyseal dysplasia', 'Cholesterol-ester transfer protein deficiency', 'Autosomal recessive hypophosphatemic rickets', 'sex hormone-binding globulin measurement', 'Isolated osteopoikilosis', 'Mediterranean macrothrombocytopenia', 'Sitosterolemia'], ['platelet count', 'mean platelet volume', 'platelet component distribution width', 'hair shape measurement', 'breast carcinoma', 'balding measurement', 'Alopecia', 'androgenetic alopecia', 'protein measurement', 'heel bone mineral density'], ['coronary artery disease', 'self reported educational attainment', 'mathematical ability', 'pneumococcal meningitis', 'bacterial pathogen genotype measurement', 'risk-taking behaviour', 'diet measurement', 'visual perception measurement', 'hearing loss', 'cognitive function measurement'], ['eosinophil count', 'eosinophil percentage of leukocytes', 'sum of eosinophil and basophil counts', 'fibrinogen measurement', 'chronic lymphocytic leukemia', 'eosinophil percentage of granulocytes', 'triglyceride measurement', 'urate measurement', 'neutrophil percentage of granulocytes', 'sex hormone-binding globulin measurement'], ['Combined oxidative phosphorylation defect type 14', 'Hereditary spastic paraplegia', 'Autosomal recessive pure spastic paraplegia', 'genetic disorder', 'Global developmental delay', 'mitochondrial encephalomyopathy', 'body weight', 'acute myeloid leukemia', 'red blood cell distribution width', 'Leigh syndrome'], ['Brain calcification, Rajab type', 'diabetes mellitus', 'mean corpuscular hemoglobin', 'type II diabetes mellitus', 'mean corpuscular volume', 'erythrocyte count', 'glomerular filtration rate', 'mean corpuscular hemoglobin concentration', 'BMI-adjusted waist circumference', 'hematocrit'], ['Brain calcification, Rajab type', 'cirrhosis of liver', 'Cerebral calcification', 'Vascular dilatation', 'Interstitial pneumonitis', 'interstitial lung disease', 'Intellectual disability', 'Rajab interstitial lung disease with brain calcifications', 'microcephaly (disease)', 'hair shape measurement'], ['Autoimmune lymphoproliferative syndrome', 'cutaneous melanoma', 'adult lymphoma', "Hashimoto's thyroiditis", 'chronic lymphocytic leukemia', 'neoplasm of mature B-cells', 'diffuse large B-cell lymphoma', 'lymphoid neoplasm', 'marginal zone B-cell lymphoma', 'carcinoma of liver and intrahepatic biliary tract'], ['Autoimmune lymphoproliferative syndrome', 'celiac disease', 'allergy', 'Eczema', 'neutrophil count', 'myeloid white cell count', 'Immune dysregulation-polyendocrinopathy-enteropathy-X-linked syndrome', 'eosinophil count', 'psoriasis', 'autoimmune disease'], ['autism spectrum disorder', 'esophageal squamous cell carcinoma', 'colorectal carcinoma', 'esophageal adenocarcinoma', 'Epileptic encephalopathy', 'Seizure', 'neoplasm', 'breast cancer', 'Autosomal recessive non-syndromic intellectual disability', 'ovarian cancer'], ['infect', 'infection', 'astrocytoma', 'hematocrit', 'Autosomal agammaglobulinemia', 'Autosomal dominant severe congenital neutropenia', 'neoplasm', 'Recurrent infection due to specific granule deficiency', 'Recurrent infections-myelofibrosis-nephromegaly syndrome', 'Hyper-IgM syndrome type 3'], ['systolic blood pressure', 'body height', 'platelet crit', 'monocyte count', 'Kidney Oncocytoma', 'nasopharyngeal neoplasm', 'juvenile dermatomyositis', 'squamous cell carcinoma', 'metastatic melanoma', 'macroglobulinemia'], ['leigh syndrome due to mitochondrial complex iv deficiency', 'Mitochondrial disease', 'combined oxidative phosphorylation deficiency', 'hepatic failure, early-onset, and neurologic disorder due to cytochrome c oxidase deficiency', 'Leigh syndrome', 'FASTKD2-related infantile mitochondrial encephalomyopathy', 'combined oxidative phosphorylation deficiency 44', 'episodic memory', 'self reported educational attainment', 'X-linked retinal dysplasia'], ['acute myeloid leukemia', 'self reported educational attainment', 'physical activity measurement', 'fasting blood insulin measurement', 'mean corpuscular volume', 'HbA1c measurement', 'mean corpuscular hemoglobin concentration', 'bipolar disorder', 'attention deficit hyperactivity disorder', 'adolescent idiopathic scoliosis'], ['leukocyte count', 'wellbeing measurement', 'self reported educational attainment', 'Romano-Ward syndrome', 'Brugada syndrome', 'Familial short QT syndrome', 'Familial progressive cardiac conduction defect', 'Wolff-Parkinson-White Syndrome', 'atrial conduction disease', 'long QT syndrome 10'], ['head and neck squamous cell carcinoma', 'squamous cell lung carcinoma', 'cervical squamous cell carcinoma', 'basal cell carcinoma', 'esophageal squamous cell carcinoma', 'urinary bladder cancer', 'colorectal adenocarcinoma', 'heel bone mineral density', 'oral squamous cell carcinoma', 'melanoma'], ['Autosomal dominant cerebellar ataxia type 1', 'vulva cancer', 'protein measurement', 'basophil percentage of leukocytes', 'basophil percentage of granulocytes', 'red blood cell distribution width', 'non-small cell lung carcinoma', 'esophageal adenocarcinoma', 'body mass index', 'basophil count'], ['brain volume measurement', 'smoking status measurement', 'neuroimaging measurement', 'protein measurement', 'acute lymphoblastic leukemia', 'autosomal dominant compelling helio-ophthalmic outburst syndrome', 'cortical surface area measurement', 'brain measurement', 'self reported educational attainment', 'caudate nucleus volume'], ['Cerebro-facio-articular syndrome', 'Hennekam syndrome', 'colorectal adenocarcinoma', 'gastric adenocarcinoma', 'cutaneous melanoma', 'basal cell carcinoma', 'lung adenocarcinoma', 'melanoma', 'gastric carcinoma', 'lymphoid neoplasm'], ['adrenal cortex carcinoma', 'aplasia cutis congenita (disease)', 'neoplasm', 'hepatocellular carcinoma', 'adrenal cortex neoplasm', 'cancer', 'non-small cell lung carcinoma', 'Ewing sarcoma', 'sarcoma', 'colorectal adenocarcinoma'], ['influenza', 'clear cell renal carcinoma', 'osteosarcoma', 'Ewing sarcoma', 'gastric carcinoma', 'B-cell non-Hodgkins lymphoma', 'acute myeloid leukemia', 'multiple myeloma', 'glioblastoma multiforme', 'breast cancer'], ['testosterone measurement', 'body height', 'depressive symptom measurement', 'insomnia measurement', 'mathematical ability', 'chronotype measurement', 'prostate carcinoma', 'unipolar depression', 'self reported educational attainment', 'cognitive function measurement'], ['adolescent idiopathic scoliosis', 'mean corpuscular volume', 'mean corpuscular hemoglobin concentration', 'age at menopause', 'red blood cell distribution width', 'electrocardiography', 'QRS duration', 'DNA methylation', 'heel bone mineral density', 'mean corpuscular hemoglobin'], ['mean platelet volume', 'systolic blood pressure', 'blood pressure', 'hypertension', 'pulse pressure measurement', 'body mass index', 'platelet count', 'cardiovascular disease', 'monocyte count', 'brain measurement'], ['acute myeloid leukemia', 'adolescent idiopathic scoliosis', 'primary biliary cirrhosis', 'Eczema', 'myeloid white cell count', 'prostate carcinoma', 'monocyte count', 'schizophrenia', 'neoplasm', 'melanoma'], ['endometrial cancer', 'neoplasm', 'hepatocellular carcinoma', 'diabetes mellitus', 'diabetic maculopathy', 'dermatomyositis', 'necrotizing enterocolitis', 'neuroendocrine carcinoma', 'Posterior cortical atrophy', 'primary cutaneous amyloidosis'], ['heel bone mineral density', 'taxonomic microbiome measurement', 'gut microbiome measurement', 'bone quantitative ultrasound measurement', 'bone density', 'glioblastoma multiforme', 'head and neck squamous cell carcinoma', 'glioma', 'neoplasm', 'Gollop-Wolfgang complex'], ['body height', 'self reported educational attainment', 'diastolic blood pressure', 'body mass index', 'health study participation', 'lifestyle measurement', 'acute myeloid leukemia', 'lean body mass', 'appendicular lean mass', 'fat body mass'], ['Synpolydactyly type 2', 'FBLN1-related developmental delay-central nervous system anomaly-syndactyly syndrome', 'erythrocyte count', 'hematocrit', 'hemoglobin measurement', 'low density lipoprotein cholesterol measurement', 'behavior or behavioral disorder measurement', 'bipolar disorder', 'FEV/FEC ratio', 'red blood cell density measurement'], ['appendicular lean mass', 'hematocrit', 'hemoglobin measurement', 'refractive error measurement', 'body height', 'corneal topography', 'lung adenocarcinoma', 'BMI-adjusted waist circumference', 'BMI-adjusted hip circumference', 'head and neck squamous cell carcinoma'], ['Autosomal recessive cutis laxa type 1', 'Autosomal dominant cutis laxa', 'Hereditary sensorimotor neuropathy with hyperelastic skin', 'age-related macular degeneration', 'Charcot-Marie-Tooth disease', 'Cutis laxa', 'thoracic aortic aneurysm', 'pulse pressure measurement', 'optic disc size measurement', 'lung adenocarcinoma'], ['varicose disease', 'Varicose veins', 'myeloid white cell count', 'neutrophil count', 'high density lipoprotein cholesterol measurement', 'leukocyte count', 'systolic blood pressure', 'age at first facial hair', 'body fat percentage', 'forced expiratory volume'], ['Marfan syndrome', 'Isolated ectopia lentis', 'Geleophysic dysplasia', 'Acromicric dysplasia', 'Marfan syndrome type 1', 'Stiff skin syndrome', 'Progeroid and marfanoid aspect-lipodystrophy syndrome', 'Weill-Marchesani syndrome', 'MASS syndrome', 'Neonatal Marfan syndrome'], ['Congenital contractural arachnodactyly', 'age-related macular degeneration', 'Abnormality of the cardiovascular system', 'hypertension', 'refractive error measurement', 'body height', 'heel bone mineral density', 'testosterone measurement', 'head and neck squamous cell carcinoma', 'genetic disorder'], ['vital capacity', 'C-C motif chemokine 25 measurement', 'lung adenocarcinoma', 'peak expiratory flow', 'breast adenocarcinoma', 'HIV-1 infection', 'viral load', 'endometrial cancer', 'Abnormality of refraction', 'Flexion contracture'], ['Fructose-1,6-bisphosphatase deficiency', 'Glycogen storage disease', 'type II diabetes mellitus', 'protein measurement', 'lean body mass', 'colon carcinoma', 'body height', 'hepatocellular carcinoma', 'endometrial cancer', 'neoplasm'], ['t-tau:beta-amyloid 1-42 ratio measurement', 'lean body mass', 'protein measurement', 'body height', 'red blood cell density measurement', 'colon carcinoma', 'gonorrhea', 'bone quantitative ultrasound measurement', 'Total congenital cataract', 'neoplasm'], ['Short stature due to GHSR deficiency', 'cancer', 'pulse pressure measurement', 'breast cancer', 'schistosomiasis', 'cholangiocarcinoma', 'hepatocellular carcinoma', 'lung carcinoma', 'calf circumference measurement', 'Hepatic fibrosis'], ['hemoglobin measurement', 'erythrocyte count', 'red blood cell density measurement', 'type II diabetes mellitus', 'mean corpuscular hemoglobin concentration', 'hematocrit', 'prostate carcinoma', 'urate measurement', 'mean corpuscular volume', 'low density lipoprotein cholesterol measurement'], ['childhood onset asthma', 'reticulocyte count', 'heel bone mineral density', 'Blackfan-Diamond anemia', 'Delta-beta-thalassemia', 'Hereditary persistence of fetal hemoglobin - beta-thalassemia', 'Adult-onset autosomal recessive sideroblastic anemia', 'Alpha-thalassemia - myelodysplastic syndrome', 'Microcytic anemia with liver iron overload', 'forced expiratory volume'], ['Intellectual disability', 'Short stature due to GHSR deficiency', 'lung adenocarcinoma', 'endometrial cancer', 'esophageal cancer', 'gastric adenocarcinoma', 'breast adenocarcinoma', 'clear cell renal carcinoma', 'small cell lung carcinoma', 'chronotype measurement'], ['Mitochondrial DNA depletion syndrome, encephalomyopathic form with variable craniofacial anomalies', 'genetic disorder', 'Mitochondrial DNA depletion syndrome', 'Leigh syndrome', 'Seizure', 'mitochondrial encephalomyopathy', 'Global developmental delay', 'cognitive function measurement', 'self reported educational attainment', 'clear cell renal carcinoma'], ['self reported educational attainment', 'erythrocyte count', 'red blood cell density measurement', 'mean corpuscular volume', 'ADP-ribosyl cyclase/cyclic ADP-ribose hydrolase 2 measurement', 'mathematical ability', 'chronic kidney disease', 'urinary metabolite measurement', 'pancreatic adenocarcinoma', 'glioblastoma multiforme'], ['hepatocellular carcinoma', 'neoplasm', 'COVID-19', 'liver cancer', 'leukoplakia'], ['neuroimaging measurement', 'refractive error measurement', 'brain measurement', 'brain volume measurement', "Alzheimer's disease", 'asthma', 'response to diisocyanate', 'reticulocyte count', 'hemorrhoid', 'ovarian cancer'], ['Zonular cataract', 'Non-syndromic congenital cataract', 'cataract 5 multiple types', 'mean corpuscular hemoglobin', 'Autosomal dominant severe congenital neutropenia', 'Neutropenia - monocytopenia - deafness', 'monocyte count', 'autosomal recessive nonsyndromic deafness 9', 'Autosomal recessive non-syndromic sensorineural deafness type DFNB', 'Autosomal dominant non-syndromic sensorineural deafness type DFNA'], ['sleep duration', 'cancer', 'neoplasm', 'colorectal carcinoma'], ['chronotype measurement', 'clear cell renal carcinoma', 'response to cold pressor test', 'mean arterial pressure', 'diverticulitis', 'type II diabetes mellitus', 'diverticular disease', 'circadian rhythm', 'hemorrhoid', 'diabetes mellitus'], ['sitting height measurement', 'protein measurement', 'vital capacity', 'clear cell renal carcinoma', 'drug use measurement', 'Vertigo', 'glioma', 'neoplasm', 'glioblastoma multiforme', 'FEV/FEC ratio'], ['autosomal recessive nonsyndromic deafness 9', 'Autosomal recessive non-syndromic sensorineural deafness type DFNB', 'Autosomal dominant non-syndromic sensorineural deafness type DFNA', 'ABCD syndrome', 'Waardenburg-Shah syndrome', 'self reported educational attainment', 'drug use measurement', 'smoking status measurement', 'psoriasis', "Alzheimer's disease"], ['clear cell renal carcinoma', 'appendicular lean mass', 'smoking behavior', 'mathematical ability', 'smoking status measurement', 'smoking behaviour measurement', 'self reported educational attainment', 'Wolfram-like syndrome', 'Wolfram syndrome', 'atypical teratoid rhabdoid tumor'], ['body mass index', 'mean corpuscular volume', 'body fat percentage', 'self reported educational attainment', 'mean corpuscular hemoglobin concentration', 'fat body mass', 'neuroticism measurement', 'smoking status measurement', 'body weight', 'mathematical ability'], ['body mass index', 'mathematical ability', 'mean platelet volume', 'monocyte percentage of leukocytes', 'psoriatic arthritis', 'ankylosing spondylitis', 'lymphocyte count', 'reticulocyte count', 'low grade glioma', 'ovarian cancer'], ['clear cell renal carcinoma', 'diastolic blood pressure', 'mean corpuscular volume', 'lung cancer', 'glioma', 'mean corpuscular hemoglobin', 'hematocrit', 'neuroticism measurement', 'Graves disease', 'diabetes mellitus, permanent neonatal 4'], ['red blood cell distribution width', 'albuminuria', 'glomerular filtration rate', 'creatinine measurement', 'red blood cell density measurement', 'urinary albumin to creatinine ratio', 'self reported educational attainment', 'mean corpuscular hemoglobin concentration', 'vital capacity', 'alcohol consumption measurement'], ['reticulocyte count', 'reticulocyte measurement', 'electrocardiography', 'FEV/FEC ratio', 'aortic root size', 'smoking status measurement', 'type II diabetes mellitus', 'refractive error measurement', 'erythrocyte count', 'waist-hip ratio'], ['hip bone size', 'clear cell renal carcinoma', 'hip bone mineral density', 'periodontal disease', 'Mental deterioration', 'blood protein measurement', 'Autosomal dominant non-syndromic sensorineural deafness type DFNA', 'Autosomal recessive non-syndromic sensorineural deafness type DFNB', 'X-linked non-syndromic sensorineural deafness type DFN', 'autosomal recessive nonsyndromic deafness 9'], ['chronotype measurement', 'diet measurement', 'blood protein measurement', 'refractory anemia', 'rheumatoid arthritis', 'eosinophil count', 'infect', 'infection', 'amyotrophic lateral sclerosis', 'axial length measurement'], ['head and neck squamous cell carcinoma', 'lung adenocarcinoma', 'endometrial cancer', 'gastric adenocarcinoma', 'breast adenocarcinoma', 'urinary bladder cancer', 'squamous cell lung carcinoma', 'clear cell renal carcinoma', 'hepatocellular carcinoma', 'Ketonuria'], ['endometrial cancer', 'chronotype measurement', 'diet measurement', 'esophageal squamous cell carcinoma', 'non-small cell lung carcinoma', 'glioma', 'neoplasm', 'skin squamous cell carcinoma', 'triple-negative breast cancer', 'body mass index'], ['gastric adenocarcinoma', 'ovarian cancer', 'endometrial cancer', 'breast adenocarcinoma', 'blood protein measurement', 'periodontal disease', 'gastric cancer', 'asthma', 'hip bone mineral density', 'hip bone size'], ['Parkinsonian-pyramidal syndrome', "Parkinson's disease", 'dystonic disorder', 'mean corpuscular volume', 'Kufor-Rakeb syndrome', 'mean corpuscular hemoglobin', 'mean corpuscular hemoglobin concentration', 'red blood cell density measurement', 'red blood cell distribution width', 'Abnormality of refraction'], ['chin morphology measurement', 'electrocardiography', 'hepatocellular carcinoma', 'bone mineral content measurement', 'cancer', 'heel bone mineral density', 'birth weight', 'parental genotype effect measurement', 'neoplasm', 'colorectal carcinoma'], ['pancreatic adenocarcinoma', 'clear cell renal carcinoma', 'smoking status measurement', 'smoking initiation', 'Hemoglobin E - beta-thalassemia', 'Alpha-thalassemia - myelodysplastic syndrome', 'Delta-beta-thalassemia', 'Autosomal dominant secondary polycythemia', 'Hemoglobin C - beta-thalassemia', 'Beta-thalassemia - X-linked thrombocytopenia'], ['coronary artery disease', 'lipoprotein-associated phospholipase A(2) measurement', 'acute coronary syndrome', 'Familial atrial fibrillation', 'Familial short QT syndrome', 'Brugada syndrome', 'Romano-Ward syndrome', 'Blackfan-Diamond anemia', 'Idiopathic ventricular fibrillation, not Brugada type', 'Familial progressive cardiac conduction defect'], ['intellectual developmental disorder with dysmorphic facies and behavioral abnormalities', 'gastric adenocarcinoma', 'non-Hodgkins lymphoma', 'basal cell carcinoma', 'endometrial carcinoma', 'breast ductal adenocarcinoma', 'prostate carcinoma', 'colon adenocarcinoma', 'cecum adenocarcinoma', 'carcinoma of liver and intrahepatic biliary tract'], ['testosterone measurement', 'clear cell renal carcinoma', 'Autosomal dominant severe congenital neutropenia', 'Sea-blue histiocytosis', 'Hereditary neutrophilia', 'Blackfan-Diamond anemia', "Felty's syndrome", 'Familial exudative vitreoretinopathy', 'Recurrent infections-myelofibrosis-nephromegaly syndrome', 'X-linked retinoschisis'], ['birth weight', 'body mass index', 'adolescent idiopathic scoliosis', 'daytime rest measurement', 'lung carcinoma', 'neoplasm', 'autism', "Alzheimer's disease", 'ulcerative colitis', "Crohn's disease"], ['diastolic blood pressure', 'lean body mass', 'fat body mass', 'whole body water mass', 'body height', 'base metabolic rate measurement', 'lung cancer', 'hepatocellular carcinoma', 'adenocarcinoma', 'glioma'], ['body mass index', 'hemoglobin measurement', 'hematocrit', 'red blood cell density measurement', 'acute myeloid leukemia', 'erythrocyte count', 'neuroticism measurement', 'pancreatic adenocarcinoma', 'clear cell renal carcinoma', 'depressive symptom measurement'], ['glomerular filtration rate', 'urate measurement', 'mean corpuscular hemoglobin concentration', 'hemoglobin measurement', 'hematocrit', 'erythrocyte count', 'creatinine measurement', 'diabetes mellitus', 'neoplasm', 'hepatocellular carcinoma'], ['mean corpuscular volume', 'neuroimaging measurement', 'brain measurement', 'brain volume measurement', 'autosomal recessive nonsyndromic deafness 9', 'Autosomal recessive non-syndromic sensorineural deafness type DFNB', 'Immature Reticulocyte Fraction Measurement', 'arterial stiffness measurement', 'Autosomal dominant non-syndromic sensorineural deafness type DFNA', 'mean platelet volume'], ['protein measurement', 'lymphocyte count', 'fetal genotype effect measurement', 'interleukin 4 measurement', 'Beta-thalassemia - X-linked thrombocytopenia', 'Ichthyosis - hepatosplenomegaly - cerebellar degeneration', 'Gray platelet syndrome', 'Hurler-Scheie syndrome', 'Hepatic veno-occlusive disease - immunodeficiency', 'autoinflammation with episodic fever and lymphadenopathy'], ['lean body mass', 'base metabolic rate measurement', 'whole body water mass', 'body height', 'clear cell renal carcinoma', 'fat body mass', 'body weight', "Barrett's esophagus"], ['mean corpuscular volume', 'mean platelet volume', 'erythrocyte count', 'mean reticulocyte volume', 'red blood cell density measurement', 'genetic disorder', 'neutrophil count', 'eosinophil peroxidase deficiency', 'monocyte count', 'Linear and whorled nevoid hypermelanosis'], ['mean platelet volume', 'clear cell renal carcinoma', 'acute myeloid leukemia', 'self reported educational attainment', 'Isolated osteopoikilosis', 'Melorheostosis with osteopoikilosis', 'refractory anemia', 'rheumatoid arthritis', 'lobe attachment', 'primary cutaneous amyloidosis'], ['Autosomal recessive non-syndromic intellectual disability', 'neuroticism measurement', 'mood instability measurement', 'brain volume measurement', 'self reported educational attainment', 'Mood swings', 'smoking status measurement', 'smoking behaviour measurement', 'wellbeing measurement', 'mathematical ability'], ['electrocardiography', 'atrial fibrillation', 'intraocular pressure measurement', 'open-angle glaucoma', 'heel bone mineral density', 'carotid artery intima media thickness', 'glaucoma', 'breast carcinoma', 'atrial flutter', 'neoplasm'], ['appendicular lean mass', 'body height', 'systolic blood pressure', 'body mass index', 'leukocyte count', 'monocyte count', 'platelet component distribution width', 'body fat percentage', 'Benign adult familial myoclonic epilepsy', 'platelet count'], ['mean platelet volume', 'eosinophil count', 'galectin-3 measurement', 'reticulocyte count', 'red blood cell distribution width', 'reticulocyte measurement', 'eosinophil percentage of leukocytes', 'urate measurement', 'Immature Reticulocyte Fraction Measurement', 'platelet count'], ['systolic blood pressure', 'body mass index', 'vital capacity', 'body fat percentage', 'body height', 'waist-hip ratio', 'lean body mass', 'waist circumference', 'glomerular filtration rate', 'fat body mass'], ['Distal hereditary motor neuropathy type 2', 'leukocyte count', 'myeloid white cell count', 'Charcot-Marie-Tooth disease', 'chronic obstructive pulmonary disease', 'FEV/FEC ratio', 'forced expiratory volume', 'smoking status measurement', 'pulmonary function measurement', 'mood instability measurement'], ['blood metabolite measurement', 'influenza A (H1N1)', 'neutrophil count', 'systemic lupus erythematosus', 'breast cancer', 'osteosarcoma', 'cancer', 'neoplasm', 'colorectal carcinoma', 'infect'], ['aging', 'clear cell renal carcinoma', 'mean platelet volume', 'mean corpuscular hemoglobin concentration', 'arterial stiffness measurement', 'central nervous system cancer', 'mean reticulocyte volume', 'mean corpuscular volume', 'asthma', 'acute quadriplegic myopathy'], ['clear cell renal carcinoma', 'self reported educational attainment', 'age at menarche', 'Infantile-onset autosomal recessive nonprogressive cerebellar ataxia', 'Adult-onset autosomal recessive cerebellar ataxia', 'Dysequilibrium syndrome', 'Spinocerebellar ataxia type 37', 'Spinocerebellar ataxia type 15/16', 'Spinocerebellar ataxia type 29', 'Spinocerebellar ataxia type 35'], ['glomerular filtration rate', 'melanoma', 'Ehlers-Danlos syndrome, progeroid type', 'balding measurement', 'mean corpuscular hemoglobin concentration', 'body fat percentage', 'hair shape measurement', 'neutrophil percentage of leukocytes', 'breast cancer', 'fat body mass'], ['Partial chromosome Y deletion', 'Non-syndromic male infertility due to sperm motility disorder', 'Autosomal dominant secondary polycythemia', 'Isolated follicle stimulating hormone deficiency', 'Isochromosomy Yp', 'Blackfan-Diamond anemia', 'Beta-thalassemia - X-linked thrombocytopenia', 'Isochromosomy Yq', 'Spinocerebellar ataxia type 32', 'Primary familial polycythemia'], ['blood protein measurement', 'periodontal disease', 'hip bone size', 'hip bone mineral density', 'cancer', 'breast neoplasm', 'breast carcinoma', 'ulcerative colitis'], ['velopharyngeal dysfunction', 'mouth mucosa disease', 'lip disease', 'self reported educational attainment', 'schizophrenia', 'Lethal arthrogryposis - anterior horn cell disease', 'Arnold-Chiari malformation type II', 'colorectal carcinoma', 'seasonal allergic rhinitis', 'Vitiligo'], ['mean corpuscular hemoglobin', 'mean corpuscular volume', 'ulcerative colitis', 'neoplasm', 'cancer', 'colorectal carcinoma', 'coronary artery disease', 'chronic obstructive pulmonary disease'], ['platelet count', 'platelet crit', 'colorectal adenocarcinoma', 'infertility', 'gastric cancer', 'papillary renal cell carcinoma', 'ovarian cancer', 'gonorrhea', 'cancer', 'multiple system atrophy'], ['multiple sclerosis', 'red blood cell distribution width', 'monocyte count', 'mean platelet volume', 'Limb pain', 'mean corpuscular volume', 'platelet component distribution width', 'insulin resistance'], ['ovarian cancer', 'endometrial cancer', 'lung adenocarcinoma', 'esophageal cancer', 'gastric adenocarcinoma', 'breast adenocarcinoma', 'protein measurement', 'chronic lymphocytic leukemia', 'Blackfan-Diamond anemia', 'lung cancer'], ['Split hand-split foot malformation', 'Distal limb deficiencies - micrognathia syndrome', 'lung adenocarcinoma', 'body height', 'head and neck squamous cell carcinoma', 'esophageal cancer', 'gastric adenocarcinoma', 'breast adenocarcinoma', 'self reported educational attainment', 'hepatocellular carcinoma'], ['lung adenocarcinoma', 'head and neck squamous cell carcinoma', 'endometrial cancer', 'gastric adenocarcinoma', 'breast adenocarcinoma', 'urinary bladder cancer', 'squamous cell lung carcinoma', 'clear cell renal carcinoma', 'platelet crit', 'gastric cancer'], ['colorectal adenocarcinoma', 'head and neck squamous cell carcinoma', 'Uterine Carcinosarcoma', 'endometrial cancer', 'chronic lymphocytic leukemia', 'gastric adenocarcinoma', 'urinary bladder cancer', 'cervical squamous cell carcinoma', 'medulloblastoma', 'squamous cell lung carcinoma'], ['protein measurement', 'body height', 'appendicular lean mass', 'lean body mass', 'fat body mass', 'sitting height measurement', 'whole body water mass', 'vital capacity', 'marginal zone B-cell lymphoma', 'rheumatoid arthritis'], ['lung adenocarcinoma', 'endometrial cancer', 'breast adenocarcinoma', 'urinary bladder cancer', 'clear cell renal carcinoma', 'squamous cell lung carcinoma', 'acute lymphoblastic leukemia', 'platelet component distribution width', 'Familial atrial fibrillation', 'Brugada syndrome'], ['head and neck squamous cell carcinoma', 'endometrial cancer', 'ovarian cancer', 'lung adenocarcinoma', 'glioblastoma multiforme', 'esophageal cancer', 'gastric adenocarcinoma', 'breast adenocarcinoma', 'clear cell renal carcinoma', 'squamous cell lung carcinoma'], ['neurodevelopmental, jaw, eye, and digital syndrome', 'Abnormality of the eye', 'Abnormal digit morphology', 'Global developmental delay', 'Abnormality of the head', 'Intellectual disability', 'syndromic intellectual disability', 'ovarian cancer', 'glioblastoma multiforme', 'clear cell renal carcinoma'], ['mathematical ability', 'cognitive function measurement', 'glomerular filtration rate', 'blood protein measurement', 'Blackfan-Diamond anemia', 'Alpha-thalassemia - myelodysplastic syndrome', 'Adult-onset autosomal recessive sideroblastic anemia', 'Delta-beta-thalassemia', 'Hereditary persistence of fetal hemoglobin - beta-thalassemia', 'Beta-thalassemia - X-linked thrombocytopenia'], ['glioblastoma multiforme', 'head and neck squamous cell carcinoma', 'Hyper-IgM syndrome type 3', 'Hyper-IgM syndrome type 5', 'Hyper-IgM syndrome type 2', 'Activated PIK3-delta syndrome', 'Autoimmune lymphoproliferative syndrome with recurrent viral infections', 'Combined immunodeficiency with skin granulomas', 'Reticular dysgenesis', 'Recurrent infections-myelofibrosis-nephromegaly syndrome'], ['killer cell immunoglobulin-like receptor 2DL4 measurement', 'inflammatory bowel disease', 'ulcerative colitis', 'colorectal adenocarcinoma', 'psoriasis', "Crohn's disease", 'non-small cell lung carcinoma', 'septic shock', 'colorectal cancer', 'IGA glomerulonephritis'], ['serum IgE measurement', 'CCL2 measurement', 'seasonal allergic rhinitis', 'growth-regulated alpha protein measurement', 'self reported educational attainment', 'C-reactive protein measurement', 'Neutrophil immunodeficiency syndrome', 'Recurrent infection due to specific granule deficiency', 'Autosomal dominant severe congenital neutropenia', 'mathematical ability'], ['mean platelet volume', 'allergic rhinitis', 'platelet component distribution width', 'platelet count', 'allergy', 'asthma', 'Eczema', 'mean corpuscular hemoglobin', 'seasonal allergic rhinitis', 'blood metabolite measurement'], ['blood protein measurement', 'low affinity immunoglobulin epsilon Fc receptor measurement', 'cerebrospinal fluid biomarker measurement', 'neutrophil count', 'thyroid gland adenocarcinoma', 'breast adenocarcinoma', 'glioblastoma multiforme', 'monocyte percentage of leukocytes', 'leukocyte count', 'head and neck squamous cell carcinoma'], ['endometrial cancer', 'unipolar depression', 'bipolar disorder', 'worry measurement', 'grip strength measurement', 'refractive error measurement', 'FEV/FEC ratio', 'mood instability measurement', 'guilt measurement', "Parkinson's disease"], ['Gallbladder Adenocarcinoma', 'central nervous system cancer', 'gallbladder cancer', 'colorectal cancer', 'head and neck squamous cell carcinoma', 'Tetralogy of Fallot', 'glioblastoma multiforme', 'neoplasm', 'esophageal squamous cell carcinoma', 'visceral Leishmaniasis'], ['viral disease', 'influenza', 'cutaneous Leishmaniasis', 'endometrial cancer', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'inflammatory bowel disease', 'low grade glioma', 'monocyte count', 'Sepsis'], ['breast carcinoma', 'Autosomal agammaglobulinemia', 'Hyper-IgM syndrome type 4', 'Hyper-IgM syndrome type 2', 'Hyper-IgM syndrome type 5', 'Wiskott-Aldrich syndrome', 'Immunodeficiency by defective expression of HLA class 2', 'Hyper-IgM syndrome type 3', 'Severe combined immunodeficiency due to DCLRE1C deficiency', 'Severe combined immunodeficiency due to CARD11 deficiency'], ['inflammatory bowel disease', 'cutaneous Leishmaniasis', 'ulcerative colitis', 'systemic lupus erythematosus', 'mucocutaneous lymph node syndrome', 'ankylosing spondylitis', 'basophil count', 'basophil percentage of leukocytes', 'blood protein measurement', 'cerebrospinal fluid biomarker measurement'], ['systemic lupus erythematosus', 'breast ductal adenocarcinoma', 'head and neck squamous cell carcinoma', 'thyroid carcinoma', 'low affinity immunoglobulin gamma Fc region receptor II-a/b measurement', 'blood protein measurement', 'hepatocellular carcinoma', 'birth weight', 'lung carcinoma', 'acute myeloid leukemia'], ['inflammatory bowel disease', 'Hyper-IgM syndrome type 3', 'Autosomal agammaglobulinemia', 'Primary systemic amyloidosis', 'Neutrophil immunodeficiency syndrome', 'Severe combined immunodeficiency due to CARD11 deficiency', 'Combined immunodeficiency with skin granulomas', 'Hyper-IgM syndrome type 5', 'Myoclonus-dystonia syndrome', 'IgE responsiveness, atopic'], ['autosomal recessive primary immunodeficiency with defective spontaneous natural killer cell cytotoxicity', 'cutaneous Leishmaniasis', 'Primary immunodeficiency', 'basophil count', 'hematocrit', 'low density lipoprotein cholesterol measurement', 'hemoglobin measurement', 'low affinity immunoglobulin gamma Fc region receptor II-a/b measurement', 'lipid measurement', 'head and neck squamous cell carcinoma'], ['monocyte count', 'neutrophil count', 'basophil percentage of leukocytes', 'serum IgG measurement', 'hematocrit', 'red blood cell density measurement', 'basophil count', 'basophil percentage of granulocytes', 'birth weight', 'erythrocyte count'], ['sex hormone-binding globulin measurement', 'hemoglobin measurement', 'hematocrit', 'erythrocyte count', 'red blood cell density measurement', 'low density lipoprotein cholesterol measurement', 'triglyceride measurement', 'mean corpuscular hemoglobin concentration', 'serum albumin measurement', 'apolipoprotein B measurement'], ['Severe congenital neutropenia', 'basophil count', 'mathematical ability', 'myocardial infarction', 'myeloid white cell count', 'lung adenocarcinoma', 'neutrophil count', 'platelet count', 'coronary artery disease', 'acute myeloid leukemia'], ['mean platelet volume', 'red blood cell distribution width', 'red blood cell density measurement', 'mean corpuscular hemoglobin concentration', 'serum non-albumin protein measurement', 'albumin:globulin ratio measurement', 'mean corpuscular volume', 'aspartate aminotransferase measurement', 'factor VIII measurement', 'von Willebrand factor measurement'], ['body height', 'childhood onset asthma', 'adult onset asthma', 'asthma', 'mean corpuscular hemoglobin', 'leukocyte count', 'neutrophil count', 'mathematical ability', 'initial pursuit acceleration', 'cognitive function measurement'], ['blood protein measurement', 'FEV/FEC ratio', "Crohn's disease", 'systemic lupus erythematosus', 'serum non-albumin protein measurement', 'non-small cell lung carcinoma', 'coronary artery disease', 'cancer', 'diabetes mellitus', 'acute myeloid leukemia'], ['lymphocyte count', 'blood protein measurement', 'polymeric immunoglobulin receptor measurement', 'Autosomal agammaglobulinemia', 'Activated PIK3-delta syndrome', 'Combined immunodeficiency with skin granulomas', 'Autosomal recessive hyper-IgE syndrome due to TYK2 deficiency', 'Hyper-IgM syndrome type 2', 'Severe combined immunodeficiency due to adenosine deaminase deficiency', 'Hyper-IgM syndrome type 3'], ['neutrophil count', 'myeloid white cell count', 'leukocyte count', 'ficolin-1 measurement', 'neutrophil percentage of leukocytes', 'ficolin-2 measurement', 'cutaneous melanoma', 'monocyte percentage of leukocytes', 'skin barrier function measurement', 'monoclonal B-cell lymphocytosis'], ['protein measurement', 'blood protein measurement', 'obesity', 'neutrophil count', 'myeloid white cell count', 'leukocyte count', 'ficolin-1 measurement', 'neutrophil percentage of leukocytes', 'cutaneous melanoma', 'neoplasm'], ['Immunodeficiency due to ficolin3 deficiency', 'rheumatic heart disease', 'cutaneous melanoma', 'Microcephaly', 'systemic lupus erythematosus', 'hepatocellular carcinoma', 'leprosy', 'stroke', 'non-small cell lung carcinoma', "Crohn's disease"], ['blood protein measurement', 'coronary artery disease', 'autoantibody measurement', 'type I diabetes mellitus', 'hairy cell leukemia', 'rheumatoid arthritis', 'Graves disease', 'diffuse large B-cell lymphoma', 'hepatocellular carcinoma', 'Mantle cell lymphoma'], ['coronary artery disease', 'blood protein measurement', 'Graves disease', 'rheumatoid arthritis', 'multiple sclerosis', 'pancreatic adenocarcinoma', 'type I diabetes mellitus', 'autoantibody measurement', "Barrett's esophagus", 'osteosarcoma'], ['rheumatoid arthritis', 'Graves disease', 'blood protein measurement', 'Fc receptor-like protein 3 measurement', 'protein measurement', 'type I diabetes mellitus', 'autoantibody measurement', 'multiple sclerosis', 'acute myeloid leukemia', 'endometriosis'], ['blood protein measurement', 'hemangioblastoma', 'squamous cell lung carcinoma', 'Ovarian Endometrioid Adenocarcinoma with Squamous Differentiation', 'cutaneous melanoma', 'breast ductal adenocarcinoma', 'basal cell carcinoma', 'lung carcinoma', 'carcinoma of liver and intrahepatic biliary tract', 'protein measurement'], ['systemic lupus erythematosus', 'myeloid white cell count', 'adolescent idiopathic scoliosis', 'multiple myeloma', 'myeloid neoplasm', 'infect', 'infection', 'ankylosing spondylitis', 'neoplasm', 'Miyoshi myopathy'], ['Cholesterol-ester transfer protein deficiency', 'Homozygous familial hypercholesterolemia', 'neoplasm', 'C-reactive protein measurement', 'obesity', 'Sitosterolemia', 'protein measurement', 'sum of neutrophil and eosinophil counts', 'HIV-1 infection', 'breast carcinoma'], ['lymphocyte count', 'erythrocyte count', 'blood protein measurement', 'monocyte count', 'lymphocyte percentage of leukocytes', 'albumin:globulin ratio measurement', 'coronary artery disease', 'serum non-albumin protein measurement', 'leukocyte count', 'hematocrit'], ['lymphocyte count', 'serum non-albumin protein measurement', 'leukocyte count', 'monocyte count', 'blood protein measurement', 'alkaline phosphatase measurement', 'erythrocyte count', 'lymphocyte percentage of leukocytes', 'mean corpuscular hemoglobin concentration', 'hematocrit'], ['congenital disorder of glycosylation with defective fucosylation', 'pulse pressure measurement', 'diastolic blood pressure', 'Congenital disorder of glycosylation', 'mathematical ability', 'cognitive function measurement', 'systolic blood pressure', 'body mass index', 'amino acid measurement', 'melanoma'], ['breast carcinoma', 'non-small cell lung carcinoma', 'COVID-19', 'colorectal cancer', 'psoriasis', 'ovarian cancer', 'periodontal disease', 'chronic rhinosinusitis', 'Cystic fibrosis', 'cancer'], ['squalene synthase deficiency', 'Hypercholesterolemia', 'Disorder of lipid metabolism', 'C-reactive protein measurement', 'reticulocyte count', 'reticulocyte measurement', 'type II diabetes mellitus', 'lymphocyte count', 'mean corpuscular volume', 'mean reticulocyte volume'], ['Disseminated superficial actinic porokeratosis', 'osteoporosis', 'postmenopausal osteoporosis', 'multiple myeloma', 'osteitis deformans', 'prostate cancer', 'breast cancer', 'prostate carcinoma', 'metastasis', 'bone disease'], ['Congenital adrenal hyperplasia', 'coronary artery disease', 'lean body mass', 'fat body mass', 'body height', 'head and neck squamous cell carcinoma', 'FEV/FEC ratio', 'Nuclear cataract', 'Total congenital cataract', 'Zonular cataract'], ['Mitochondrial myopathy', 'Congenital adrenal hyperplasia', 'genetic disorder', 'response to statin', 'FTH1-related iron overload', "Crohn's disease", 'sclerosing cholangitis', 'inflammatory bowel disease', 'ulcerative colitis', 'Pentosuria'], ['red blood cell distribution width', 'mean corpuscular volume', 'mean corpuscular hemoglobin', 'triglyceride measurement', 'cardiovascular disease', 'mean platelet volume', 'waist-hip ratio', 'mean reticulocyte volume', 'systolic blood pressure', 'PR interval'], ['auditory neuropathy-optic atrophy syndrome', 'genetic disorder', 'Congenital adrenal hyperplasia', 'pancreatic carcinoma', 'low grade glioma', 'glioblastoma multiforme', 'squamous cell lung carcinoma', 'head and neck squamous cell carcinoma', 'urinary bladder cancer', 'optic atrophy'], ['Erythropoietic protoporphyria', 'protoporphyria, erythropoietic, 2', 'red blood cell distribution width', 'mean corpuscular hemoglobin concentration', 'mean corpuscular volume', 'mean corpuscular hemoglobin', 'lobe attachment', 'gastric carcinoma', "Alzheimer's disease", 'Dementia'], ['body height', 'appendicular lean mass', 'colitis', 'Hereditary mixed polyposis syndrome', 'MUTYH-related attenuated familial adenomatous polyposis', 'Autosomal recessive early-onset inflammatory bowel disease', 'AXIN2-related attenuated familial adenomatous polyposis', 'Hirschsprung disease', 'Intestinal epithelial dysplasia', 'desmoid tumor'], ['hematocrit', 'hemoglobin measurement', 'mean corpuscular hemoglobin concentration', 'platelet crit', 'platelet count', 'acute lymphoblastic leukemia', 'Partial pancreatic agenesis', 'insulin like growth factor measurement', 'systolic blood pressure', 'Transient neonatal diabetes mellitus'], ['neutrophil count', 'myeloid white cell count', 'leukocyte count', 'forced expiratory volume', 'body height', 'reticulocyte count', 'vital capacity', 'diastolic blood pressure', 'alcohol drinking', 'smoking status measurement'], ['HIV infection', 'multiple myeloma', 'breast adenocarcinoma', 'hepatocellular carcinoma', 'B-cell non-Hodgkins lymphoma', 'acute myeloid leukemia', 'esophageal squamous cell carcinoma', 'lung adenocarcinoma', 'endometrial cancer', 'thyroid carcinoma'], ['body height', 'intraocular pressure measurement', 'vital capacity', 'forced expiratory volume', 'BMI-adjusted waist circumference', 'systolic blood pressure', 'pulse pressure measurement', 'leukocyte count', 'fibroblastic disorder', 'coronary artery disease'], ['appendicular lean mass', 'diastolic blood pressure', 'alcohol drinking', 'smoking status measurement', 'eosinophil percentage of granulocytes', 'body height', 'lymphocyte percentage of leukocytes', 'bipolar disorder', 'eosinophil percentage of leukocytes', 'unipolar depression'], ['reticulocyte count', "Barrett's esophagus", 'esophageal adenocarcinoma', 'colorectal cancer', 'colorectal adenocarcinoma', 'myxosarcoma', 'adenocarcinoma', 'Pachyonychia congenita', 'ulcerative colitis', 'prostate cancer'], ['body height', 'sitting height measurement', 'heel bone mineral density', 'allergic rhinitis', 'Abnormality of refraction', 'large artery stroke', 'coronary artery disease', 'pulse pressure measurement', 'systolic blood pressure', 'stroke'], ['Kindler syndrome', 'Hereditary acrokeratotic poikiloderma of Kindler-Weary', 'protein measurement', 'inflammatory bowel disease', 'body height', 'appendicular lean mass', 'colorectal carcinoma', 'response to drug', 'hypertension', 'Agents acting on the renin-angiotensin system use measurement'], ['intraocular pressure measurement', 'prostate carcinoma', 'systolic blood pressure', 'PR interval', 'primary angle closure glaucoma', 'glaucoma', 'diastolic blood pressure', 'open-angle glaucoma', "Alzheimer's disease", 'family history of Alzheimer’s disease'], ['Leukocyte adhesion deficiency type III', 'Leukocyte adhesion deficiency', 'Leukocyte adhesion deficiency type I', 'reticulocyte count', 'Immature Reticulocyte Fraction Measurement', 'breast adenocarcinoma', 'mean platelet volume', 'esophageal cancer', 'platelet count', 'diet measurement'], ['head and neck squamous cell carcinoma', 'colorectal adenocarcinoma', 'Endometrial Endometrioid Adenocarcinoma', 'ovarian carcinoma', 'hemangioblastoma', 'gastric carcinoma', 'basal cell carcinoma', 'gastric adenocarcinoma', 'hypertension', 'systolic blood pressure'], ['persulfide dioxygenase ETHE1, mitochondrial measurement', 'fetuin-B measurement', 'blood protein measurement', 'acute myeloid leukemia', 'electrocardiography', 'insulin resistance', 'Non-syndromic male infertility due to sperm motility disorder', 'spermatogenic failure 1', 'Female infertility due to zona pellucida defect', 'atherosclerosis'], ['sudden infant death syndrome', 'malignant soft tissue neoplasm', 'Extraskeletal Ewing Sarcoma/Peripheral Primitive Neuroectodermal Tumor', 'Olfactory Neuroblastoma', 'colorectal adenocarcinoma', 'prostate adenocarcinoma', 'gastric adenocarcinoma', 'bile duct carcinoma', 'breast ductal adenocarcinoma', 'thyroid carcinoma'], ['platelet count', 'drug use measurement', 'infect', 'infection', 'serum alanine aminotransferase measurement', 'schizophrenia', 'HIV-1 infection', 'mean corpuscular hemoglobin concentration', 'vital capacity', 'Primary dystonia, DYT13 type'], ['lean body mass', 'body height', 'body fat percentage', 'body mass index', 'hemorrhoid', 'body weight', 'fat body mass', 'forced expiratory volume', 'vital capacity', 'apolipoprotein A 1 measurement'], ['Kallmann syndrome', 'diet measurement', 'chronotype measurement', 'age at first sexual intercourse measurement', 'hypogonadotropic hypogonadism', 'lean body mass', 'fat body mass', 'body mass index', 'body height', 'facial morphology measurement'], ['self reported educational attainment', 'fat body mass', 'body mass index', 'body fat percentage', 'lean body mass', 'mathematical ability', 'risk-taking behaviour', 'brain volume measurement', 'autism spectrum disorder', 'grip strength measurement'], ['type II diabetes mellitus', 'diabetes mellitus', 'Abnormal blood glucose concentration', 'endometrial cancer', 'glioblastoma multiforme', 'breast adenocarcinoma', 'cutaneous melanoma', 'low grade glioma', 'acute myeloid leukemia', 'Exercise-induced hyperinsulinism'], ['monocyte percentage of leukocytes', 'neutrophil count', 'hemoglobin measurement', 'mean corpuscular hemoglobin concentration', 'mean corpuscular hemoglobin', 'red blood cell distribution width', 'glioblastoma multiforme', 'transferrin saturation measurement', 'serum iron measurement', 'graft versus host disease'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'low grade glioma', 'Hyperinsulinism due to INSR deficiency', 'hyperproinsulinemia', 'Hyperinsulinism due to glucokinase deficiency', 'Hyperinsulinism due to short chain 3-hydroxylacyl-CoA dehydrogenase deficiency', 'Exercise-induced hyperinsulinism', 'Transient neonatal diabetes mellitus', 'Diazoxide-resistant focal hyperinsulinism due to Kir6.2 deficiency'], ['BMI-adjusted waist-hip ratio', 'waist-hip ratio', 'insulin like growth factor measurement', 'triglyceride measurement', 'endometrial cancer', 'leukocyte count', 'glioblastoma multiforme', 'reticulocyte count', 'vitamin A measurement', 'Obesity due to melanocortin 4 receptor deficiency'], ['Familial hypodysfibrinogenemia', 'Congenital fibrinogen deficiency', 'Familial renal amyloidosis', 'Familial dysfibrinogenemia', 'Familial afibrinogenemia', 'cancer', 'Familial hypofibrinogenemia', 'Noonan syndrome', 'Costello syndrome', 'hypertrophic cardiomyopathy'], ['Congenital fibrinogen deficiency', 'Familial afibrinogenemia', 'Familial hypofibrinogenemia', 'Familial hypodysfibrinogenemia', 'cancer', 'Familial dysfibrinogenemia', 'Noonan syndrome', 'hypertrophic cardiomyopathy', 'Costello syndrome', 'Recurrent thrombophlebitis'], ['Aarskog-Scott syndrome', 'Aarskog-Scott syndrome, X-linked', 'genetic disorder', 'X-linked intellectual disability, Siderius type', 'thyroid gland adenocarcinoma', 'breast adenocarcinoma', 'clear cell renal carcinoma', 'cutaneous melanoma', 'mental retardation', 'Intellectual disability'], ['monocyte count', 'hemoglobin measurement', 'hypothyroidism', 'Thyroid preparation use measurement', 'hematocrit', 'erythrocyte count', 'red blood cell density measurement', 'low density lipoprotein cholesterol measurement', 'monocyte percentage of leukocytes', 'mean corpuscular hemoglobin concentration'], ['mean platelet volume', 'platelet component distribution width', 'leukocyte count', 'thyroid gland adenocarcinoma', 'neutrophil count', 'breast adenocarcinoma', 'myeloid white cell count', 'medulloblastoma', 'glioblastoma multiforme', 'heel bone mineral density'], ['Charcot-Marie-Tooth disease type 4H', 'Charcot-Marie-Tooth disease type 4', 'Charcot-Marie-Tooth disease', 'mean platelet volume', 'monocyte count', 'hypertension', 'type II diabetes mellitus', 'systolic blood pressure', 'lymphocyte count', 'brain volume measurement'], ['hypertension', 'coronary artery disease', 'systolic blood pressure', 'diastolic blood pressure', 'mean arterial pressure', 'delivery measurement', 'pulse pressure measurement', 'protein measurement', 'drug use measurement', 'hemoglobin measurement'], ['FEV/FEC ratio', 'coronary artery disease', 'platelet count', 'coronary atherosclerosis', 'brain aneurysm', 'chronic obstructive pulmonary disease', 'parental longevity', 'healthspan', 'longevity', 'endometriosis'], ['cancer', 'bone development disease', 'female reproductive endometrioid cancer', 'ovarian carcinoma', 'waist-hip ratio', 'BMI-adjusted waist-hip ratio', 'Acrocephalosyndactyly', 'Craniosynostosis', 'heel bone mineral density', 'HIV-1 infection'], ['cancer', 'bone development disease', 'female reproductive endometrioid cancer', 'ovarian carcinoma', 'Acrocephalosyndactyly', 'Craniosynostosis', 'lean body mass', 'hepatocellular carcinoma', 'fat body mass', 'body height'], ['Deafness with labyrinthine aplasia, microtia, and microdontia', 'cancer', 'bone development disease', 'female reproductive endometrioid cancer', 'ovarian carcinoma', 'hearing loss', 'breast adenocarcinoma', 'lung adenocarcinoma', 'cutaneous melanoma', 'synophrys measurement'], ['cancer', 'bone development disease', 'Craniosynostosis', 'ovarian carcinoma', 'female reproductive endometrioid cancer', 'Acrocephalosyndactyly', 'lung adenocarcinoma', 'cutaneous melanoma', 'birth weight', 'protein measurement'], ['trichomegaly', 'cancer', 'bone development disease', 'systolic blood pressure', 'diastolic blood pressure', 'hair shape measurement', 'hair color', 'female reproductive endometrioid cancer', 'Craniosynostosis', 'ovarian carcinoma'], ['cancer', 'bone development disease', 'female reproductive endometrioid cancer', 'ovarian carcinoma', 'endometrial cancer', 'breast adenocarcinoma', 'cutaneous melanoma', 'low grade glioma', 'head and neck squamous cell carcinoma', 'phosphorus measurement'], ['cancer', 'bone development disease', 'female reproductive endometrioid cancer', 'ovarian carcinoma', 'Thyrotoxicosis', 'nontoxic goiter', 'thyroid gland adenocarcinoma', 'breast adenocarcinoma', 'glioblastoma multiforme', 'head and neck squamous cell carcinoma'], ['hypogonadotropic hypogonadism 6 with or without anosmia', 'Kallmann syndrome', 'Normosmic congenital hypogonadotropic hypogonadism', 'Holoprosencephaly', 'cancer', 'bone development disease', 'Septopreoptic holoprosencephaly', 'female reproductive endometrioid cancer', 'Alobar holoprosencephaly', 'Semilobar holoprosencephaly'], ['Multiple synostoses syndrome', 'cancer', 'bone development disease', 'female reproductive endometrioid cancer', 'ovarian carcinoma', 'Craniosynostosis', 'Acrocephalosyndactyly', 'refractive error measurement', 'testosterone measurement', 'hypertension'], ['Lacrimoauriculodentodigital syndrome', 'Aplasia of lacrimal and salivary glands', 'cancer', 'bone development disease', 'female reproductive endometrioid cancer', 'ovarian carcinoma', 'forced expiratory volume', 'vital capacity', 'polydactyly (disease)', 'prostate carcinoma'], ['breast adenocarcinoma', 'gastric adenocarcinoma', 'balding measurement', 'puberty onset measurement', 'red blood cell density measurement', 'Microcytic anemia with liver iron overload', 'Adult-onset autosomal recessive sideroblastic anemia', 'mean platelet volume', 'Overhydrated hereditary stomatocytosis', 'self reported educational attainment'], ['Early infantile epileptic encephalopathy', 'Epileptic encephalopathy', 'undetermined early-onset epileptic encephalopathy', 'protein measurement', 'PR interval', 'acute myeloid leukemia', 'mathematical ability', 'reticulocyte count', 'influenza A (H1N1)', 'gastric adenocarcinoma'], ['breast adenocarcinoma', 'endometrial cancer', 'glioblastoma multiforme', 'gastric adenocarcinoma', 'neoplasm', 'breast cancer', 'cervical cancer', 'glioma', 'colorectal carcinoma', 'ovarian carcinoma'], ['Spinocerebellar ataxia type 27', 'PR interval', 'neuroimaging measurement', 'alcohol consumption measurement', 'Nystagmus', 'Guillain-Barre syndrome', 'brain measurement', 'cortical thickness', 'Cognitive impairment', 'Midface retrusion'], ['Syndactyly type 8', 'cancer', 'bone development disease', 'female reproductive endometrioid cancer', 'Acrocephalosyndactyly', 'Craniosynostosis', 'ovarian carcinoma', 'cutaneous melanoma', 'hepatocellular carcinoma', 'Left ventricular noncompaction'], ['Kallmann syndrome', 'Normosmic congenital hypogonadotropic hypogonadism', 'cancer', 'bone development disease', 'ovarian carcinoma', 'Craniosynostosis', 'Acrocephalosyndactyly', 'female reproductive endometrioid cancer', 'Isolated congenital hypogonadotropic hypogonadism', 'endometrial cancer'], ['cancer', 'bone development disease', 'lean body mass', 'body height', 'appendicular lean mass', 'fat body mass', 'body weight', 'vital capacity', 'forced expiratory volume', 'PR interval'], ['cancer', 'cutaneous melanoma', 'head and neck squamous cell carcinoma', 'lung adenocarcinoma', 'hair colour measurement', 'breast cancer', 'breast carcinoma', 'hair color', 'Her2-receptor negative breast cancer', 'melanoma'], ['cancer', 'bone development disease', 'Craniosynostosis', 'Acrocephalosyndactyly', 'ovarian carcinoma', 'Bilateral renal agenesis', 'female reproductive endometrioid cancer', 'renal agenesis (disease)', "Parkinson's disease", 'thyroid gland adenocarcinoma'], ['diet measurement', 'energy intake', 'triglyceride measurement', 'Abnormal urine sodium concentration', 'protein intake measurement', 'sodium measurement', 'cutaneous melanoma', 'sugar consumption measurement', 'lung adenocarcinoma', 'breakfast skipping measurement'], ['cancer', 'bone development disease', 'female reproductive endometrioid cancer', 'ovarian carcinoma', 'head and neck squamous cell carcinoma', 'gastric adenocarcinoma', 'Homozygous familial hypercholesterolemia', 'Cholesterol-ester transfer protein deficiency', 'Sitosterolemia', 'hypercholesterolemia, autosomal dominant, 3'], ['Autosomal dominant hypophosphatemic rickets', 'tumoral calcinosis, hyperphosphatemic, familial, 2', 'familial hyperphosphatemic tumoral calcinosis/hyperphosphatemic hyperostosis syndrome', 'cancer', 'bone development disease', 'X-linked hypophosphatemia', 'tumoral calcinosis, hyperphosphatemic, familial, 3', 'Hypophosphatemia', 'Epidermal nevus syndrome', 'bone disease'], ['vitamin D measurement', 'lung adenocarcinoma', 'glioblastoma multiforme', 'breast adenocarcinoma', 'cutaneous melanoma', 'neoplasm', 'colon carcinoma', 'breast carcinoma', "Kaposi's sarcoma", 'electrocardiography'], ['lung adenocarcinoma', 'breast adenocarcinoma', 'cutaneous melanoma', 'head and neck squamous cell carcinoma', 'vitamin D measurement', 'electrocardiography', 'pulse pressure measurement', 'asthma', 'protein measurement', 'non-small cell lung carcinoma'], ['blood protein measurement', 'neutrophil count', 'alkaline phosphatase measurement', 'head and neck squamous cell carcinoma', 'leukocyte count', 'obsessive-compulsive disorder', 'myeloid white cell count', 'testosterone measurement', 'Hereditary geniospasm', 'lean body mass'], ['Hartsfield-Bixler-Demyer syndrome', 'hypogonadotropic hypogonadism 2 with or without anosmia', 'Kallmann syndrome', 'Osteoglophonic dwarfism', 'Pfeiffer syndrome', 'Encephalocraniocutaneous lipomatosis', 'Normosmic congenital hypogonadotropic hypogonadism', 'Isolated trigonocephaly', 'cancer', 'Jackson-Weiss syndrome'], ['cancer', 'leukemia', 'precursor lymphoblastic lymphoma/leukemia', 'myelodysplastic syndrome', 'chronic myelogenous leukemia', 'basophil count', 'endometrial cancer', 'breast adenocarcinoma', 'basophil percentage of leukocytes', 'mean corpuscular volume'], ['Crouzon disease', 'Pfeiffer syndrome', 'Cutis gyrata - acanthosis nigricans - craniosynostosis', 'Apert syndrome', 'Jackson-Weiss syndrome', 'Lacrimoauriculodentodigital syndrome', 'Antley-Bixler syndrome', 'Familial scaphocephaly syndrome, McGillivray type', 'FGFR2-related bent bone dysplasia', 'cancer'], ['Achondroplasia', 'Thanatophoric dysplasia type 1', 'Muenke syndrome', 'Severe achondroplasia - developmental delay - acanthosis nigricans', 'Crouzon syndrome - acanthosis nigricans', 'Hypochondroplasia', 'Camptodactyly - tall stature - scoliosis - hearing loss', 'urinary bladder cancer', 'Thanatophoric dysplasia type 2', 'seborrheic keratosis'], ['rhabdomyosarcoma', 'breast carcinoma', 'cancer', 'bone development disease', 'idiopathic pulmonary fibrosis', 'Familial prostate cancer', 'neoplasm', 'urothelial carcinoma', 'prostate adenocarcinoma', 'prostate carcinoma'], ['bone quantitative ultrasound measurement', 'heel bone mineral density', 'bone density', 'reticulocyte count', 'sex hormone-binding globulin measurement', 'appendicular lean mass', 'cortical surface area measurement', 'body height', 'femoral neck bone mineral density', 'alkaline phosphatase measurement'], ['Familial hypodysfibrinogenemia', 'Congenital fibrinogen deficiency', 'Familial dysfibrinogenemia', 'Familial afibrinogenemia', 'cancer', 'Familial hypofibrinogenemia', 'Noonan syndrome', 'Costello syndrome', 'hypertrophic cardiomyopathy', 'Hypofibrinogenemia'], ['amyotrophic lateral sclerosis', 'sporadic amyotrophic lateral sclerosis', 'familial amyotrophic lateral sclerosis', 'pulse pressure measurement', 'systolic blood pressure', 'white matter microstructure measurement', 'hematocrit', 'electrocardiography', 'diastolic blood pressure', 'acute myeloid leukemia'], ['blood protein measurement', 'FEV/FEC ratio', 'self reported educational attainment', 'rheumatoid arthritis', 'refractory anemia', 'Pallister-Hall syndrome', 'gonorrhea', 'body height', 'non-small cell lung carcinoma', 'triglyceride measurement'], ['waist-hip ratio', 'drug use measurement', 'acute myeloid leukemia', 'neoplasm', 'colitis', 'colorectal carcinoma', 'lung adenocarcinoma', 'infection', 'infect', 'body mass index'], ['chronic myelogenous leukemia', 'acute lymphoblastic leukemia', 'cutaneous Leishmaniasis', 'neoplasm', 'lymphoid neoplasm', 'systolic blood pressure', 'leukemia', 'acute myeloid leukemia', 'prostate adenocarcinoma', 'endometrial cancer'], ['Hereditary leiomyomatosis and renal cell cancer', 'Fumaric aciduria', 'Multiple cutaneous leiomyomas', 'Inherited cancer-predisposing syndrome', 'Endometrial Endometrioid Adenocarcinoma', 'Hepatobiliary Neoplasm', 'carcinoma of liver and intrahepatic biliary tract', 'Hereditary pheochromocytoma-paraganglioma', 'breast carcinoma', 'kidney cancer'], ['migraine disorder', 'Headache', 'systemic lupus erythematosus', 'esophageal adenocarcinoma', 'benign neoplasm of spinal cord', 'DNA methylation', 'diabetes mellitus', 'urinary albumin to creatinine ratio', 'squamous cell carcinoma', 'osteosarcoma'], ['reticulocyte count', 'Immature Reticulocyte Fraction Measurement', 'reticulocyte measurement', 'FEV/FEC ratio', 'mean platelet volume', 'mean corpuscular hemoglobin concentration', 'red blood cell distribution width', 'psychotic symptoms', 'Red cell distribution width', 'osteosarcoma'], ['platelet count', 'appendicular lean mass', 'lean body mass', 'leukocyte count', 'body height', 'platelet crit', 'myeloid white cell count', 'mathematical ability', 'fat body mass', 'neutrophil count'], ['body mass index', 'prostate carcinoma', 'duodenal ulcer', 'Monoamine oxidase A deficiency', 'Rapid-onset dystonia-parkinsonism', 'Myoclonus-dystonia syndrome', 'mathematical ability', 'gastroesophageal reflux disease', 'peptic ulcer disease', 'Wolfram-like syndrome'], ['alkaline phosphatase measurement', 'bipolar disorder', 'attention deficit hyperactivity disorder', 'Autosomal recessive non-syndromic intellectual disability', 'Abnormality of refraction', 'Leber hereditary optic neuropathy', 'Wolfram-like syndrome', 'Wolfram syndrome', 'Isolated optic nerve hypoplasia', 'Leber plus disease'], ['body fat percentage', 'hip circumference', 'fat body mass', 'cutaneous melanoma', 'low density lipoprotein cholesterol measurement', 'waist-hip ratio', 'nevus count', 'total cholesterol measurement', 'apolipoprotein B measurement', 'portal hypertension'], ['body mass index', 'physical activity measurement', 'body fat percentage', 'unipolar depression', 'smoking behavior', 'lean body mass', 'esophageal adenocarcinoma', 'acute myeloid leukemia', 'prostate carcinoma', 'pancreatic carcinoma'], ['Reducing body myopathy', 'X-linked myopathy with postural muscle atrophy', 'Emery-Dreifuss muscular dystrophy', 'X-linked Emery-Dreifuss muscular dystrophy', 'Uruguay Faciocardiomusculoskeletal syndrome', 'myopathy', 'hypertrophic cardiomyopathy', 'Myocardial Ischemia', 'scapuloperoneal myopathy', 'Rare familial disorder with hypertrophic cardiomyopathy'], ['Familial isolated dilated cardiomyopathy', 'protein measurement', 'pulse pressure measurement', 'left ventricular structural measurement', 'PR interval', 'acute myeloid leukemia', 'body height', 'platelet count', 'sitting height measurement', 'red blood cell density measurement'], ['erythrocyte count', 'pulse pressure measurement', 'heel bone mineral density', 'body height', 'balding measurement', 'appendicular lean mass', 'lean body mass', 'coronary artery disease', 'colorectal cancer', 'sitting height measurement'], ['migraine disorder', 'Headache', 'systolic blood pressure', 'pulse pressure measurement', 'migraine without aura', 'diastolic blood pressure', 'migraine with aura', 'brain aneurysm', 'hypertension', 'blood pressure'], ['non-small cell lung carcinoma', 'glioblastoma multiforme', 'vital capacity', 'cancer', 'sitting height measurement', 'bone quantitative ultrasound measurement', 'heel bone mineral density', 'bone density', 'hematocrit', 'reticulocyte measurement'], ['hypertrophic cardiomyopathy', 'electrocardiography', 'adolescent idiopathic scoliosis', 'diet measurement', 'bone quantitative ultrasound measurement', 'heel bone mineral density', 'heart rate', 'dilated cardiomyopathy', 'QRS complex', 'QRS duration'], ['body mass index', 'body fat percentage', 'insomnia measurement', 'body height', 'type II diabetes mellitus', 'fat body mass', 'lean body mass', 'waist circumference', 'body weight', 'balding measurement'], ['body height', 'testosterone measurement', 'bone density', 'heel bone mineral density', 'body fat percentage', 'lung carcinoma', 'non-small cell lung carcinoma', 'Ataxia-telangiectasia', "Crohn's disease", 'colorectal adenocarcinoma'], ['Macrocephaly', 'Abnormal facial shape', 'Overgrowth', 'X-linked epilepsy - learning disabilities - behavior disorders', 'Large hands', 'Ocular coloboma', "Crohn's disease", 'Blackfan-Diamond anemia', 'Delta-beta-thalassemia', 'monocyte count'], ['smoking status measurement', 'Blount disease', 'Gollop-Wolfgang complex', 'Metaphyseal anadysplasia', 'Absent tibia - polydactyly', 'Fibular aplasia - ectrodactyly', 'Hypoplastic tibiae - postaxial polydactyly', 'Brachydactyly-syndactyly, Zhao type', 'Acromesomelic dysplasia, Grebe type', 'Syndactyly type 4'], ['Charcot-Marie-Tooth disease type 4J', 'Yunis-Varon syndrome', 'amyotrophic lateral sclerosis', 'Bilateral parasagittal parieto-occipital polymicrogyria', 'Bilateral polymicrogyria', 'Charcot-Marie-Tooth disease type 4', 'Charcot-Marie-Tooth disease', 'genetic disorder', 'familial amyotrophic lateral sclerosis', 'Epileptic encephalopathy with global cerebral demyelination'], ['primary ovarian insufficiency', 'premature menopause', 'Osteosclerosis - ichthyosis - premature ovarian failure', 'Non-syndromic male infertility due to sperm motility disorder', 'Isolated follicle stimulating hormone deficiency', '46,XX testicular disorder of sex development', 'genetic non-acquired premature ovarian failure', 'Familial hyperprolactinemia', 'Partial chromosome Y deletion', 'Aromatase deficiency'], ['mathematical ability', 'body mass index', 'mean arterial pressure', 'self reported educational attainment', 'cognitive function measurement', 'body height', 'diastolic blood pressure', 'risk-taking behaviour', 'daytime rest measurement', 'primary hypertension'], ['type II diabetes mellitus', 'psoriasis', 'fat body mass', 'body weight', 'ulcerative colitis', "Crohn's disease", 'body mass index', 'lymphocyte count', 'lean body mass', 'Aromatic L-amino acid decarboxylase deficiency'], ['urate measurement', 'energy intake measurement', 'ulcerative colitis'], ['body height', 'lean body mass', 'osteoarthritis, hip', 'fat body mass', 'protein measurement', 'vital capacity', 'BMI-adjusted waist circumference', 'whole body water mass', 'osteoarthritis', 'BMI-adjusted hip circumference'], ['pulse pressure measurement', 'vital capacity', 'breast carcinoma', 'optic disc size measurement', 'heel bone mineral density', 'axial length measurement', 'waist-hip ratio', 'appendicular lean mass', 'cleft lip', 'Cleft palate'], ['cancer', 'acute myeloid leukemia', 'colorectal adenocarcinoma', 'pancreatic carcinoma', 'lung adenocarcinoma', 'hepatocellular carcinoma', 'lymphoid neoplasm', 'nasopharyngeal squamous cell carcinoma', 'cervical squamous cell carcinoma', 'colon adenocarcinoma'], ['lymphocyte count', 'hemoglobin measurement', "Alzheimer's disease", 'acute myeloid leukemia', 'Crigler-Najjar syndrome type 2', 'appendicular lean mass', 'Sarcosinemia', 'amyotrophic lateral sclerosis', 'Transient familial neonatal hyperbilirubinemia', 'Hemolytic anemia due to glutathione reductase deficiency'], ['hepatocellular carcinoma', 'neoplasm', 'cardiomyopathy', 'Hand-foot syndrome', 'hydrops fetalis (disease)', 'cancer', 'Muscular dystrophy', 'metabolic disease', 'Facioscapulohumeral dystrophy', 'Bartsocas-Papas syndrome'], ['Siddiqi syndrome', 'diet measurement', 'mean corpuscular volume', 'esophageal cancer', 'type II diabetes mellitus', 'Nuclear cataract', 'Total congenital cataract', 'Zonular cataract', 'Cerulean cataract', 'Coralliform cataract'], ['red blood cell density measurement', 'triglyceride measurement', 'retinopathy'], ['Niemann-Pick disease type C', 'cancer', 'autosomal dominant compelling helio-ophthalmic outburst syndrome', 'colorectal carcinoma', 'Testicular regression syndrome', 'Double uterus - hemivagina - renal agenesis', 'Breslow thickness', 'cutaneous melanoma', 'bone density', '46,XX ovotesticular disorder of sex development'], ['atopic eczema', 'chronic kidney disease', 'acute graft vs. host disease', 'kidney disease', 'renal cell carcinoma', 'kidney failure', 'cytomegalovirus infection', 'rheumatoid arthritis', 'coronary artery disease', 'lupus nephritis'], ['Familial atrial fibrillation', 'lifestyle measurement', 'Brugada syndrome', 'Familial short QT syndrome', 'Pallister-Hall syndrome', 'Romano-Ward syndrome', 'Familial progressive cardiac conduction defect', 'Idiopathic ventricular fibrillation, not Brugada type', 'Atrial tachyarrhythmia with short PR interval', 'Partial pancreatic agenesis'], ['cognitive function measurement', 'diet measurement', 'eosinophil count', 'lean body mass', 'body mass index', 'platelet count', 'platelet crit', 'intelligence', 'body fat percentage', 'reticulocyte count'], ['non-small cell lung carcinoma', 'colorectal carcinoma', 'breast cancer', "small bowel Crohn's disease", 'Cholesterol-ester transfer protein deficiency', 'nicotine dependence symptom count', 'self reported educational attainment', 'DNA methylation', 'colorectal cancer', 'neoplasm'], ['testosterone measurement', 'sex hormone-binding globulin measurement', 'BMI-adjusted waist-hip ratio', 'vital capacity', 'hematocrit', 'forced expiratory volume', 'apolipoprotein A 1 measurement', 'waist-hip ratio', 'estradiol measurement', 'breast adenocarcinoma'], ['lymphocyte count', 'eosinophil count', 'birth weight', 'eosinophil percentage of leukocytes', 'vital capacity', 'forced expiratory volume', 'leukocyte count', 'brain volume measurement', 'monocyte count', 'monocyte percentage of leukocytes'], ['body height', 'head and neck squamous cell carcinoma', 'breast adenocarcinoma', 'cutaneous melanoma', 'triglyceride measurement', 'squamous cell lung carcinoma', 'gastric adenocarcinoma', 'Partial chromosome Y deletion', 'Non-syndromic male infertility due to sperm motility disorder', 'Isochromosomy Yp'], ['atrioventricular block (disease)', 'aspartate aminotransferase measurement', 'atrial fibrillation', 'atrial flutter', 'serum alanine aminotransferase measurement', 'appendicular lean mass', 'body height', 'blood protein measurement', 'electrocardiography', 'forced expiratory volume'], ['glioblastoma multiforme', 'ovarian cancer', 'breast adenocarcinoma', 'potassium measurement', 'esophageal cancer', 'hematocrit', 'Microcephaly - cervical spine fusion anomalies', 'Autosomal recessive spondylocostal dysostosis', 'Autosomal recessive spastic paraplegia type 25', 'Posterior meningocele'], ['reticulocyte measurement', 'reticulocyte count', 'head and neck squamous cell carcinoma', 'lung adenocarcinoma', 'gastric adenocarcinoma', 'erythrocyte count', 'chronotype measurement', 'glioblastoma multiforme', 'hepatocellular carcinoma', 'insulin like growth factor measurement'], ['Bruck syndrome', 'Osteogenesis imperfecta', 'Osteogenesis imperfecta type 3', 'Skeletal dysplasia', 'Arthrogryposis-like syndrome', 'Osteogenesis imperfecta type 4', 'Osteogenesis imperfecta type 5', 'genetic disorder', 'Recurrent fractures', 'Kyphosis'], ['systemic lupus erythematosus', 'Mendelian susceptibility to mycobacterial diseases due to partial IRF8 deficiency', 'Gnathodiaphyseal dysplasia', 'Melorheostosis', 'Isolated osteopoikilosis', 'Dacryocystitis - osteopoikilosis', 'Mendelian susceptibility to mycobacterial diseases due to complete ISG15 deficiency', 'Hepatic veno-occlusive disease - immunodeficiency', 'Pyknoachondrogenesis', 'Hyper-IgM syndrome type 5'], ['Ehlers-Danlos syndrome, kyphoscoliotic and deafness type', 'Ehlers-Danlos syndrome, kyphoscoliotic type', 'genetic disorder', 'Hypotonia', 'Thoracolumbar scoliosis', 'Pes valgus', 'Brittle cornea syndrome', 'Congenital muscular dystrophy', 'Joint hypermobility', 'neoplasm'], ['Wiskott-Aldrich syndrome', 'neutrophil count', 'sum of neutrophil and eosinophil counts', 'sum of basophil and neutrophil counts', 'granulocyte count', 'monocyte percentage of leukocytes', 'appendicular lean mass', 'mean corpuscular hemoglobin', 'mean corpuscular volume', 'ulcerative colitis'], ['breast adenocarcinoma', 'glioblastoma multiforme', 'head and neck squamous cell carcinoma', 'esophageal cancer', 'squamous cell lung carcinoma', 'lung adenocarcinoma', 'hepatocellular carcinoma', 'age-related macular degeneration', 'atopic eczema', 'systolic blood pressure'], ['Congenital muscular dystrophy without intellectual disability', 'muscular dystrophy-dystroglycanopathy type B5', 'Autosomal recessive limb-girdle muscular dystrophy type 2I', 'Congenital muscular dystrophy with cerebellar involvement', 'Congenital muscular dystrophy with intellectual disability', 'Walker-Warburg syndrome', 'Congenital muscular dystrophy, Fukuyama type', 'Muscle-eye-brain disease', 'Limb-girdle muscular dystrophy', 'Congenital muscular dystrophy due to dystroglycanopathy'], ['Congenital muscular dystrophy, Fukuyama type', 'Walker-Warburg syndrome', 'Congenital muscular dystrophy without intellectual disability', 'Autosomal recessive limb-girdle muscular dystrophy type 2M', 'Muscle-eye-brain disease', 'Familial isolated dilated cardiomyopathy', 'Autosomal recessive limb-girdle muscular dystrophy type 2T', 'muscular dystrophy-dystroglycanopathy type B5', 'Familial dilated cardiomyopathy', 'Congenital muscular alpha-dystroglycanopathy with brain and eye anomalies'], ['basal cell carcinoma', 'lymphocyte count', 'platelet count', 'skin cancer', 'platelet crit', 'neutrophil percentage of leukocytes', 'rheumatoid arthritis', 'lymphocyte percentage of leukocytes', 'melanoma', 'non-melanoma skin carcinoma'], ['Multiple acyl-CoA dehydrogenase deficiency', 'Multiple acyl-CoA dehydrogenation deficiency, mild type', 'Multiple acyl-CoA dehydrogenation deficiency, severe neonatal type', 'myopathy with abnormal lipid metabolism', 'Glutaric aciduria', 'erythrocyte count', 'hematocrit', 'mean corpuscular hemoglobin concentration', 'hemoglobin measurement', 'red blood cell density measurement'], ['Familial spontaneous pneumothorax', 'Birt-Hogg-Dubé syndrome', 'colorectal adenocarcinoma', 'colonic neoplasm', 'Fibrofolliculoma', 'nonpapillary renal cell carcinoma', 'Inherited cancer-predisposing syndrome', '17p11.2 microduplication syndrome', 'Mosaic trisomy 17', 'Endometrial Endometrioid Adenocarcinoma'], ['atopic eczema', 'Inherited ichthyosis', 'Eczema', 'ichthyosis vulgaris', 'genetic disorder', 'asthma', 'ichthyosis (disease)', 'Lamellar ichthyosis', 'vitamin D measurement', 'Dry skin'], ['peeling skin syndrome 6', 'Peeling skin syndrome type A', 'atopic eczema', 'breast carcinoma', 'psoriasis', 'childhood onset asthma', 'contact dermatitis', 'vitamin D measurement', 'atopic march', 'Peeling skin syndrome'], ['Ewing sarcoma', 'Rare hemorrhagic disorder due to a constitutional platelet anomaly', 'osteosarcoma', 'body height', 'platelet count', 'basophil count', 'platelet crit', 'eosinophil count', 'monocyte count', 'eosinophil percentage of leukocytes'], ['response to statin', 'hair colour measurement', 'Flexion contracture', 'pathological myopia', 'neoplasm', 'colitis', 'Ewing sarcoma', 'appendicular lean mass', 'low density lipoprotein cholesterol measurement', 'obesity'], ['Periventricular nodular heterotopia', 'Osteodysplasty, Melnick-Needles type', 'Frontometaphyseal dysplasia', 'Otopalatodigital syndrome type 2', 'Otopalatodigital syndrome type 1', 'Terminal osseous dysplasia - pigmentary defects', 'Otopalatodigital syndrome', 'Congenital valvular dysplasia', 'Congenital short bowel syndrome', 'Nodular neuronal heterotopia'], ['Autosomal dominant Larsen syndrome', 'Spondylocarpotarsal synostosis', 'Atelosteogenesis type III', 'Atelosteogenesis type I', 'Boomerang dysplasia', 'Joubert syndrome and related disorders', 'arthrogryposis', 'neutrophil count', 'reticulocyte count', 'myeloid white cell count'], ['Muscle filaminopathy', 'hypertrophic cardiomyopathy', 'Familial isolated restrictive cardiomyopathy', 'Distal myopathy with posterior leg and anterior hand involvement', 'dilated cardiomyopathy', 'myopathy', 'autosomal dominant dilated cardiomyopathy', 'restrictive cardiomyopathy', 'Myofibrillar myopathy', 'Familial restrictive cardiomyopathy'], ['thyroid gland adenocarcinoma', 'chronic obstructive pulmonary disease', 'smoking status measurement', 'neoplasm', 'hepatocellular carcinoma', 'age-related macular degeneration', 'breast cancer', 'colorectal carcinoma', 'Niemann-Pick disease type C', 'metastatic prostate cancer'], ['lung adenocarcinoma', 'mathematical ability', 'cognitive function measurement', 'Niemann-Pick disease type C', 'neoplasm', 'gestational diabetes', 'gonorrhea', 'mean reticulocyte volume', 'erythrocyte count', 'colorectal carcinoma'], ['Spastic paraplegia-optic atrophy-neuropathy syndrome', 'glioblastoma multiforme', 'breast adenocarcinoma', 'cutaneous melanoma', 'endometrial cancer', 'peripheral neuropathy', 'acute myeloid leukemia', 'BMI-adjusted waist-hip ratio', 'triglyceride measurement', 'Hereditary spastic paraplegia'], ['PR interval', 'cup-to-disc ratio measurement', 'blood protein measurement', 'electrocardiography', 'vital capacity', 'erythrocyte count', 'hemoglobin measurement', 'heart rate', 'resting heart rate', 'triglyceride measurement'], ['Kallmann syndrome', 'Isolated congenital hypogonadotropic hypogonadism', 'lung adenocarcinoma', 'breast adenocarcinoma', 'cutaneous melanoma', 'femoral neck bone mineral density', 'serum alanine aminotransferase measurement', 'acute myeloid leukemia', 'glomerular filtration rate', 'head and neck squamous cell carcinoma'], ['renal cell carcinoma', 'neoplasm', 'hepatocellular carcinoma', 'Gastrointestinal stromal tumor', 'idiopathic pulmonary fibrosis', 'thyroid carcinoma', 'sarcoma', 'neuroendocrine neoplasm', 'metastatic colorectal cancer', 'thyroid cancer'], ['acute myeloid leukemia', 'hepatocellular carcinoma', 'acute lymphoblastic leukemia', 'renal cell carcinoma', 'cancer', 'neoplasm', 'Gastrointestinal stromal tumor', 'pancreatic neuroendocrine tumor', 'primary myelofibrosis', 'neuroendocrine neoplasm'], ['cancer', 'monocyte count', 'monocyte percentage of leukocytes', 'thyroid gland adenocarcinoma', 'neoplasm', 'rheumatoid arthritis', 'refractory anemia', 'arthritis', 'Hyper-IgE syndrome', 'Harlequin ichthyosis'], ['Milroy disease', 'hepatocellular carcinoma', 'non-small cell lung carcinoma', 'Gastrointestinal stromal tumor', 'thyroid carcinoma', 'soft tissue sarcoma', 'neoplasm', 'small cell lung carcinoma', 'renal cell carcinoma', 'congenital heart defects, multiple types, 7'], ['Posterior column ataxia - retinitis pigmentosa', 'Retinitis pigmentosa', 'genetic disorder', 'Retinal dystrophy', 'Hereditary sensory and autonomic neuropathy', 'Short rib-polydactyly syndrome', 'Stargardt disease', 'Jeune syndrome', 'hematocrit', 'hemoglobin measurement'], ['Fowler syndrome', 'Posterior column ataxia - retinitis pigmentosa', 'multiple sclerosis', 'marginal zone B-cell lymphoma', 'small cell lung carcinoma', 'cancer', 'lean body mass', 'cardiovascular disease', 'sitting height measurement', 'fat body mass'], ['Dowling-Degos disease', 'acute myeloid leukemia', 'Linear atrophoderma of Moulin', 'Familial generalized lentiginosis', 'Neurofibromatosis type 6', 'Dyschromatosis universalis', 'Dyschromatosis symmetrica hereditaria', 'Familial glucocorticoid deficiency', 'UV-sensitive syndrome', 'Blackfan-Diamond anemia'], [], ['BMI-adjusted waist circumference', 'X-linked retinal dysplasia', 'Atypical dentin dysplasia due to SMOC2 deficiency', 'Familial retinal arterial macroaneurysm', 'amelogenesis imperfecta, type ij', 'Familial primary pulmonary hypoplasia', 'Familial exudative vitreoretinopathy', 'Otodental syndrome', 'Jalili syndrome', 'Dentinogenesis imperfecta type 3'], ['hemoglobin measurement', 'hematocrit', 'erythrocyte count', 'hair shape measurement', 'red blood cell density measurement', 'urate measurement', 'protein measurement', 'balding measurement', 'mean corpuscular hemoglobin concentration', 'skin aging'], ['Autosomal recessive non-syndromic intellectual disability', 'body height', 'hair colour measurement', 'hair color', 'heel bone mineral density', 'acute myeloid leukemia', 'bone density', 'bone quantitative ultrasound measurement', 'platelet count', 'Drugs affecting bone structure and mineralization use measurement'], ['tiredness measurement', 'hypothyroidism', 'inflammatory bowel disease', 'asthma', 'respiratory system disease', 'drug use measurement', 'white matter microstructure measurement', 'atopic asthma', 'glioblastoma multiforme', 'childhood onset asthma'], ['intraocular pressure measurement', 'leukocyte count', 'open-angle glaucoma', 'myeloid white cell count', 'heel bone mineral density', 'lymphocyte count', 'glaucoma', 'refractive error measurement', 'neutrophil count', 'bone quantitative ultrasound measurement'], ['vital capacity', 'pulse pressure measurement', 'systolic blood pressure', 'smoking status measurement', 'forced expiratory volume', 'intelligence', 'smoking behaviour measurement', 'protein measurement', 'self reported educational attainment', 'diet measurement'], ['neutrophil count', 'myeloid white cell count', 'cutaneous melanoma', 'urinary metabolite measurement', 'chronic kidney disease', 'smoking behavior', '3-hydroxy-1-methylpropylmercapturic acid measurement', 'Kidney Oncocytoma', 'chromophobe renal cell carcinoma', 'ependymoma'], ['cutaneous melanoma', 'urinary bladder cancer', 'esophageal squamous cell carcinoma', 'lentiform nucleus measurement', 'lung carcinoma', "Barrett's esophagus", 'esophageal adenocarcinoma', 'non-small cell lung carcinoma', 'adrenal cortex carcinoma', 'lung adenocarcinoma'], ['trimethylaminuria (disease)', 'Trimethylaminuria', 'Inborn errors of metabolism', 'sex hormone-binding globulin measurement', 'blood metabolite measurement', 'testosterone measurement', 'lentiform nucleus measurement', 'genetic disorder', 'cutaneous melanoma', 'low density lipoprotein cholesterol measurement'], ['urinary metabolite measurement', 'chronic kidney disease', 'brain aneurysm', '3-hydroxy-1-methylpropylmercapturic acid measurement', 'smoking behavior', 'myeloid white cell count', 'neutrophil count', 'clear cell renal carcinoma', 'ulcerative colitis', 'lung carcinoma'], ['body height', 'appendicular lean mass', 'adolescent idiopathic scoliosis', "Barrett's esophagus", 'Obesity due to melanocortin 4 receptor deficiency', 'Berardinelli-Seip congenital lipodystrophy', 'Familial partial lipodystrophy, Dunnigan type', 'Glycogen storage disease due to liver glycogen phosphorylase deficiency', 'clear cell renal carcinoma', 'lung carcinoma'], ['blood protein measurement', 'cutaneous melanoma', 'lung adenocarcinoma', 'mean corpuscular volume', 'ovarian cancer', 'gastric adenocarcinoma', 'Pelvic dysplasia - arthrogryposis of lower limbs', 'Arthrogryposis - hyperkeratosis, lethal form', 'neoplasm', 'Balkan nephropathy'], ['Fragile X syndrome', 'Fragile X-associated tremor/ataxia syndrome', 'primary ovarian insufficiency', 'Intellectual disability', 'mental retardation', 'Osteosclerosis - ichthyosis - premature ovarian failure', 'Tremor', 'autism spectrum disorder', 'genetic disorder', 'Neurodevelopmental abnormality'], ['basophil count', 'basophil percentage of leukocytes', 'adolescent idiopathic scoliosis', 'monocyte percentage of leukocytes', 'Fragile X syndrome', 'anti-neutrophil antibody associated vasculitis', 'cancer', 'hepatocellular carcinoma', 'lung adenocarcinoma', 'Pachyonychia congenita'], ["Spondylometaphyseal dysplasia, 'corner fracture' type", 'Fibronectin glomerulopathy', 'cancer', 'Noonan syndrome', 'hypertrophic cardiomyopathy', 'Costello syndrome', 'blood protein measurement', 'systolic blood pressure', 'pulse pressure measurement', 'coronary artery disease'], ['HbA1c measurement', 'red blood cell distribution width', 'mean platelet volume', 'platelet component distribution width', 'platelet count', 'body height', 'Pentosuria', 'Nail-patella-like renal disease', 'proteinuria, chronic benign', 'Primary hyperoxaluria type 3'], ['HbA1c measurement', 'mean platelet volume', 'platelet count', 'body height', 'red blood cell distribution width', 'systemic juvenile idiopathic arthritis', 'psoriasis', 'Hyperglycemia', 'restless legs syndrome', 'triple-negative breast cancer'], ['optic cup area measurement', 'breast ductal adenocarcinoma', 'breast carcinoma', 'acute myeloid leukemia', 'pancreatic neuroendocrine tumor', 'pancreatic carcinoma', 'skin carcinoma', 'Merkel cell skin cancer', 'bile duct adenocarcinoma', 'clear cell renal carcinoma'], ['glomerular filtration rate', 'myeloid white cell count', 'systolic blood pressure', 'self reported educational attainment', 'schizophrenia', 'intelligence', 'neutrophil count', 'refractive error measurement', 'serum alanine aminotransferase measurement', 'triglyceride measurement'], ['eosinophil count', 'alcohol consumption measurement', 'alcohol use disorder measurement', 'type II diabetes mellitus', 'bitter alcoholic beverage consumption measurement', 'body mass index', 'myeloid white cell count', 'Drugs used in diabetes use measurement', 'grip strength measurement', 'brain volume measurement'], ['pulse pressure measurement', 'systolic blood pressure', 'lipoprotein A measurement', 'joint hypermobility measurement', 'eosinophil count', 'cholelithiasis', 'QRS complex', 'QRS duration', 'electrocardiography', 'gonorrhea'], ['leukocyte count', 'neutrophil count', 'electrocardiography', 'age at menarche', 'apolipoprotein A 1 measurement', 'high density lipoprotein cholesterol measurement', 'myeloid white cell count', 'body height', 'insulin like growth factor measurement', 'Male infertility due to large-headed multiflagellar polyploid spermatozoa'], ['intraocular pressure measurement', 'appendicular lean mass', 'red blood cell distribution width', 'body height', 'lean body mass', 'mean corpuscular hemoglobin concentration', 'waist-hip ratio', 'BMI-adjusted waist-hip ratio', 'BMI-adjusted waist circumference', 'physical activity measurement'], ['triglyceride measurement', 'total cholesterol measurement', 'C-reactive protein measurement', 'serum albumin measurement', 'eosinophil count', 'hair color', 'lymphocyte count', 'red blood cell distribution width', 'reticulocyte count', 'myeloid white cell count'], ['obesity', 'hypothyroidism', 'osteosarcoma', 'chronic obstructive pulmonary disease', 'diabetes mellitus', 'chronic kidney disease', 'Apparent mineralocorticoid excess', 'alkaline phosphatase measurement', 'autoimmune polyendocrinopathy', 'cardiac hypertrophy'], ['mean platelet volume', 'Homozygous familial hypercholesterolemia', 'FTH1-related iron overload', 'Cholesterol-ester transfer protein deficiency', 'Pentosuria', 'Sitosterolemia', 'Glycogen storage disease due to muscle beta-enolase deficiency', 'Myopathy due to calsequestrin and SERCA1 protein overload', 'trypsinogen deficiency', 'hypercholesterolemia, autosomal dominant, 3'], ['Hereditary breast and ovarian cancer syndrome', 'electrocardiography', 'juvenile dermatomyositis'], ['body height', 'age at menarche', 'puberty onset measurement', 'appendicular lean mass', 'sitting height measurement', 'sex hormone-binding globulin measurement', 'serum alanine aminotransferase measurement', 'serum gamma-glutamyl transferase measurement', 'C-reactive protein measurement', 'asthma'], ['fat body mass', 'visceral adipose tissue measurement', 'body fat percentage', 'body weight', 'lean body mass', 'sex hormone-binding globulin measurement', 'waist circumference', 'hip circumference', 'Tetralogy of Fallot', 'myxosarcoma'], ['cancer', 'smoking cessation', 'sum of basophil and neutrophil counts', 'neutrophil count', 'daytime rest measurement', 'red blood cell distribution width', 'lean body mass', 'neutrophil percentage of leukocytes', 'granulocyte count', 'protein measurement'], ['Agammaglobulinemia', 'hypertrophic cardiomyopathy', 'primary immunodeficiency disease', 'neutropenia', 'triglyceride measurement', 'Wolff-Parkinson-White Syndrome', 'Combined immunodeficiency with skin granulomas', 'Pancytopenia due to IKZF1 mutations', 'Okt4 epitope deficiency', 'Autosomal dominant severe congenital neutropenia'], ['sex hormone-binding globulin measurement', 'self reported educational attainment', 'intelligence', 'health study participation', 'schizophrenia', 'testosterone measurement', 'diet measurement', 'lifestyle measurement', 'African iron overload', 'mathematical ability'], ['myelodysplastic syndrome', 'hepatitis D virus infection', 'acute myeloid leukemia', 'non-small cell lung carcinoma', 'chronic myelomonocytic leukemia', 'metastasis', 'acute myeloblastic leukemia with maturation', 'adult acute megakaryoblastic leukemia', 'acute erythroleukemia', 'acute myeloblastic leukemia without maturation'], ['myelodysplastic syndrome', 'mean corpuscular hemoglobin concentration', 'mean corpuscular volume', 'mean corpuscular hemoglobin', 'hepatitis D virus infection', 'erythrocyte count', 'acute myeloid leukemia', 'non-small cell lung carcinoma', 'chronic myelomonocytic leukemia', 'hemoglobin measurement'], ['smoking status measurement', 'type II diabetes mellitus', 'mean corpuscular volume', 'mean corpuscular hemoglobin concentration', 'red blood cell density measurement', 'mean corpuscular hemoglobin', 'erythrocyte count', 'lymphocyte count', 'daytime rest measurement', 'neoplasm'], ['triglyceride measurement', 'intraocular pressure measurement', 'blood protein measurement', 'chronic kidney disease', 'urinary metabolite measurement', 'acute myeloid leukemia', 'mean platelet volume', 'apolipoprotein A 1 measurement', 'cardiomyopathy', 'reticulocyte count'], ['Neurodegenerative syndrome due to cerebral folate transport deficiency', 'Seizure', 'ovarian cancer', 'neoplasm', 'Epileptic encephalopathy', 'lung adenocarcinoma', 'cutaneous melanoma', 'genetic disorder', 'Gastrointestinal stromal tumor', 'breast adenocarcinoma'], ['body height', "Barrett's esophagus", 'neoplasm', 'rheumatoid arthritis', 'Acute hepatic failure', 'non-small cell lung carcinoma', 'protein measurement', 'cancer', 'refractory anemia', 'acute myeloid leukemia'], ['urinary albumin to creatinine ratio', 'influenza A (H1N1)', 'Sepsis', 'body height', 'mean platelet volume', "Crohn's disease", 'non-small cell lung carcinoma', 'neoplasm', 'cancer', 'clear cell renal carcinoma'], ['Berardinelli-Seip congenital lipodystrophy', "Crohn's disease", 'heel bone mineral density', 'inflammatory bowel disease', 'glioblastoma multiforme', 'basophil count', 'ovarian cancer', 'basophil percentage of leukocytes', 'head and neck squamous cell carcinoma', 'endometrial cancer'], ['heel bone mineral density', 'neoplasm', 'Benign familial neonatal seizures', 'breast cancer', 'anaplastic large cell lymphoma', 'cervical cancer', 'glioblastoma multiforme', 'Juvenile myoclonic epilepsy', 'autism', 'amyotrophic lateral sclerosis'], ['eosinophil count', 'eosinophil percentage of leukocytes', 'colorectal carcinoma', 'pancreatic carcinoma', 'esophageal squamous cell carcinoma', 'eosinophil percentage of granulocytes', 'sum of eosinophil and basophil counts', 'esophageal adenocarcinoma', 'neoplasm', 'colorectal adenocarcinoma'], ['inflammatory bowel disease', "Crohn's disease", 'hair color', 'body height', 'systolic blood pressure', 'leukocyte count', 'Eczema', 'allergic rhinitis', 'psoriasis', 'triglyceride measurement'], ['prostate adenocarcinoma', 'breast adenocarcinoma', 'prostate carcinoma', 'Mixed Lobular and Ductal Breast Carcinoma', 'lobular breast carcinoma', 'breast ductal adenocarcinoma', 'Prostate Small Cell Carcinoma', 'Breast Carcinoma by Gene Expression Profile', 'breast carcinoma', 'hepatocellular carcinoma'], ['endometrial cancer', 'hypopituitarism', 'hyperinsulinism (disease)', 'Congenital isolated hyperinsulinism', 'Combined pituitary hormone deficiencies, genetic forms', 'Non-acquired combined pituitary hormone deficiency', 'fasting blood glucose measurement', 'hypothyroidism', "Hashimoto's thyroiditis", 'nodular goiter'], ['head and neck squamous cell carcinoma', 'BMI-adjusted waist-hip ratio', 'cutaneous melanoma', 'breast adenocarcinoma', 'low grade glioma', 'waist-hip ratio', 'mean corpuscular volume', 'Biliary atresia', 'cholangiocarcinoma', 'eosinophil count'], ['protein measurement', 'polymyalgia rheumatica', 'taxonomic microbiome measurement', 'gut microbiome measurement', 'Early-onset autosomal dominant Alzheimer disease', 'pontine autosomal dominant microangiopathy with leukoencephalopathy', 'COASY protein-associated neurodegeneration', 'annexin A2 measurement', 'central nervous system cancer', 'Ataxia-telangiectasia'], ['Diarrhea', 'response to darapladib', 'acute coronary syndrome', 'coronary artery disease', 'prostate cancer', 'neoplasm', 'cancer', 'hemoglobin measurement', 'mean corpuscular hemoglobin concentration', 'red blood cell distribution width'], ['Axenfeld-Rieger syndrome', 'Rieger anomaly', 'Axenfeld anomaly', 'Peters anomaly', 'Familial ocular anterior segment mesenchymal dysgenesis', 'Aniridia', 'Isolated aniridia', 'heel bone mineral density', 'systolic blood pressure', 'open-angle glaucoma'], ['Lymphedema - distichiasis', 'cleft palate', 'lymphedema, hereditary, iii', 'heel bone mineral density', 'bone density', 'bone quantitative ultrasound measurement', 'appendicular lean mass', 'colorectal cancer', 'femoral neck bone mineral density', 'sitting height measurement'], ['heel bone mineral density', 'body height', 'bone quantitative ultrasound measurement', 'pulse pressure measurement', 'macula measurement', 'sitting height measurement', 'lung cancer', 'synophrys measurement', "Crohn's disease", 'osteoarthritis'], ['body height', 'brain volume measurement', 'neuroimaging measurement', 'brain measurement', 'hematocrit', 'erythrocyte count', 'urinary albumin to creatinine ratio', 'hemoglobin measurement', 'cortical surface area measurement', 'cortical thickness'], ['Familial ocular anterior segment mesenchymal dysgenesis', 'Aniridia', 'Peters anomaly', 'appendicular lean mass', 'cutaneous melanoma', 'skin pigmentation', 'body height', 'sitting height measurement', 'protein measurement', 'diabetes mellitus'], ['mean platelet volume', 'monocyte percentage of leukocytes', 'hair color', 'neutrophil count', 'leukocyte count', 'Microphthalmia - ankyloblepharon - intellectual disability', 'platelet count', 'myeloid white cell count', 'colorectal carcinoma', 'systolic blood pressure'], ['multiple myeloma', 'insomnia measurement', 'sleep duration', 'diastolic blood pressure', 'depressive symptom measurement', 'insomnia', 'excessive daytime sleepiness measurement', 'circadian rhythm', 'Microphthalmia - ankyloblepharon - intellectual disability', 'polycythemia vera'], ['Microphthalmia - ankyloblepharon - intellectual disability', 'polycythemia vera', 'X-linked dyserythropoetic anemia with abnormal platelets and neutropenia', 'Familial isolated congenital asplenia'], ['Arthropathy', 'Microphthalmia - ankyloblepharon - intellectual disability', 'polycythemia vera', 'X-linked dyserythropoetic anemia with abnormal platelets and neutropenia', 'Familial isolated congenital asplenia', 'leukemia', 'Harlequin ichthyosis', 'Hyper-IgE syndrome', 'nonpapillary renal cell carcinoma', 'colorectal carcinoma'], ['Arthropathy', 'Microphthalmia - ankyloblepharon - intellectual disability', 'polycythemia vera', 'X-linked dyserythropoetic anemia with abnormal platelets and neutropenia', 'Familial isolated congenital asplenia'], ['visual perception measurement', 'Microphthalmia - ankyloblepharon - intellectual disability', 'polycythemia vera', 'X-linked dyserythropoetic anemia with abnormal platelets and neutropenia', 'Familial isolated congenital asplenia', 'Harlequin ichthyosis', 'Hyper-IgE syndrome', 'cancer', 'Cholestasis-lymphedema syndrome', 'Langerhans cell sarcoma'], ['Bamforth-Lazarus syndrome', 'Familial papillary or follicular thyroid carcinoma', 'follicular thyroid carcinoma', 'Congenital hypothyroidism', 'choanal atresia', 'cleft palate', 'hypothyroidism', 'thyroid carcinoma', 'hair color', 'atrophy of thyroid'], ['Congenital primary aphakia', 'Familial ocular anterior segment mesenchymal dysgenesis', 'Non-syndromic congenital cataract', 'Familial thoracic aortic aneurysm and aortic dissection', 'Peters anomaly', 'Posterior polar cataract', 'marfan syndrome/loeys-dietz syndrome/familial thoracic aortic aneurysms and dissections', 'Aortic dissection', 'low density lipoprotein cholesterol measurement', 'sleep duration'], ['Congenital alveolar capillary dysplasia', 'persistent fetal circulation syndrome', 'appendicular lean mass', 'lean body mass', 'cup-to-disc ratio measurement', 'FEV/FEC ratio', 'fat body mass', 'Anorectal malformation', 'systolic blood pressure', "Barrett's esophagus"], ['balding measurement', 'protein measurement', 'stroke', 'Ischemic stroke', 'testosterone measurement', 'puerperal disorder', 'psoriasis', 'atopic eczema', 'neoplasm', 'breast cancer'], ['Rett syndrome', 'Atypical Rett syndrome', 'genetic disorder', 'FOXG1 disorder', 'mathematical ability', 'Global developmental delay', 'Strabismus', 'Stereotypy', 'Muscular hypotonia of the trunk', 'Intellectual disability, severe'], ['Septopreoptic holoprosencephaly', 'Microform holoprosencephaly', 'Alobar holoprosencephaly', 'Semilobar holoprosencephaly', 'Midline interhemispheric variant of holoprosencephaly', 'Lobar holoprosencephaly', 'Holoprosencephaly', 'self reported educational attainment', 'mathematical ability', 'cognitive function measurement'], ['Pendred syndrome', 'Autosomal recessive distal renal tubular acidosis', 'Autosomal recessive non-syndromic sensorineural deafness type DFNB', 'autosomal recessive nonsyndromic deafness 4', 'hearing loss', 'enlarged vestibular aqueduct syndrome', 'renal tubular acidosis', 'Non-syndromic male infertility due to sperm motility disorder', 'breast carcinoma', 'spermatogenic failure 42'], ['bladder disease', 'Hyperinsulinism due to INSR deficiency', 'Down syndrome', 'Hyperinsulinism due to glucokinase deficiency', 'Hyperinsulinism-hyperammonemia syndrome', 'Exercise-induced hyperinsulinism', 'Hyperinsulinism due to short chain 3-hydroxylacyl-CoA dehydrogenase deficiency', 'Familial glucocorticoid deficiency', 'hyperproinsulinemia', 'Transient neonatal diabetes mellitus'], ['tooth agenesis', 'optic disc size measurement', 'hair color', 'visceral adipose tissue measurement', 'cancer', 'non-small cell lung carcinoma', 'Branchio-otic syndrome', 'Branchiogenic deafness syndrome', 'Thickened earlobes - conductive deafness', 'Non-eruption of teeth - maxillary hypoplasia - genu valgum'], ['Primary ciliary dyskinesia', 'allergic rhinitis', 'body height', 'Hydrocephalus', 'neoplasm', 'colorectal carcinoma', 'colorectal cancer', 'asthma', 'Familial isolated restrictive cardiomyopathy', 'Cerebellum agenesis - hydrocephaly'], ['platelet count', 'mean platelet volume', 'mean reticulocyte volume', 'mean corpuscular volume', 'neutrophil count', 'neutrophil percentage of leukocytes', 'visual perception measurement', 'monocyte percentage of leukocytes', 'granulocyte percentage of myeloid white cells', 'myeloid white cell count'], ['age at menarche', 'Romano-Ward syndrome', 'Brugada syndrome', 'body fat percentage', 'Familial short QT syndrome', 'Familial atrial fibrillation', 'Familial progressive cardiac conduction defect', 'long QT syndrome 11', 'Jervell and Lange-Nielsen syndrome', 'Idiopathic ventricular fibrillation, not Brugada type'], ['type II diabetes mellitus', 'urate measurement', 'grip strength measurement', 'liver disease', 'heel bone mineral density', 'mosquito bite reaction size measurement', 'apolipoprotein A 1 measurement', 'hypothyroidism', 'diastolic blood pressure', 'protein measurement'], ['red blood cell distribution width', 'mean corpuscular volume', 'mean corpuscular hemoglobin', 'appendicular lean mass', 'mean corpuscular hemoglobin concentration', 'heel bone mineral density', 'reticulocyte count', 'mean platelet volume', 'lean body mass', 'mean reticulocyte volume'], ['heel bone mineral density', 'bone density', 'bone quantitative ultrasound measurement', 'appendicular lean mass', 'colorectal cancer', 'femoral neck bone mineral density', 'sitting height measurement', 'coronary artery disease', 'hip bone mineral density', 'spine bone mineral density'], ['Blepharophimosis - epicanthus inversus - ptosis', 'primary ovarian insufficiency', 'Ovarian Granulosa Cell Tumor', 'ovarian cancer', 'Ovarian Sertoli-Leydig Cell Tumor', 'colorectal adenocarcinoma', 'Ovarian Gynandroblastoma', 'cervical squamous cell carcinoma', 'basal cell carcinoma', 'lung carcinoma'], ['facial hair thickness measurement', 'brain measurement', 'neuroimaging measurement', 'platelet count', '3-hydroxypropylmercapturic acid measurement', 'smoking behavior', 'hair shape measurement', 'non-small cell lung carcinoma', 'insulin like growth factor measurement', 'endometrial carcinoma'], ['protein measurement', 'visual perception measurement', 'heel bone mineral density', 'hematocrit', 'hemoglobin measurement'], ['breast adenocarcinoma', 'glioblastoma multiforme', 'low grade glioma', 'mean corpuscular hemoglobin concentration', 'chronic lymphocytic leukemia', 'red blood cell distribution width', 'mean corpuscular hemoglobin', 'hepatocellular carcinoma', 'non-small cell lung carcinoma', 'neoplasm'], ['Alymphoid cystic thymic dysgenesis', 'T-cell lymphopenia, infantile, with or without nail dystrophy, autosomal dominant', 'Thymic aplasia', 'Severe combined immunodeficiency', 'Aplasia of the thymus', 'Relative macrocephaly', 'autism', 'tonsillectomy risk measurement', 'Ringed hair disease', 'Hypotrichosis simplex'], ['sex hormone-binding globulin measurement', 'appendicular lean mass', 'mathematical ability', 'vital capacity', 'body height', 'body fat percentage', 'smoking behavior', 'grip strength measurement', 'alkaline phosphatase measurement', 'forced expiratory volume'], ['testosterone measurement', 'electrocardiography', 'sex hormone-binding globulin measurement', 'self reported educational attainment', 'heel bone mineral density', 'intelligence', 'smoking initiation', 'smoking status measurement', 'acute myeloid leukemia', 'peak insulin response measurement'], ['triglyceride measurement', 'X-linked retinal dysplasia', 'Familial exudative vitreoretinopathy', 'Severe early-childhood-onset retinal dystrophy', 'X-linked retinoschisis', 'Familial retinal arterial macroaneurysm', 'Autosomal dominant rhegmatogenous retinal detachment', 'Congenital primary aphakia', 'Colobomatous microphthalmia', 'Senior-Loken syndrome'], ['monocyte count', 'monocyte percentage of leukocytes', 'embryonal rhabdomyosarcoma', 'mean corpuscular volume', 'alveolar rhabdomyosarcoma', 'rhabdomyosarcoma', 'heel bone mineral density', 'desmoplastic small round cell tumor', 'kidney neoplasm', 'Chordoma'], ["Alzheimer's disease", 'lymphoid neoplasm', 'prostate adenocarcinoma', 'kidney neoplasm', 'carcinoma of liver and intrahepatic biliary tract', 'breast ductal adenocarcinoma', 'esophageal adenocarcinoma', 'Endometrial Endometrioid Adenocarcinoma', 'bile duct carcinoma', 'Hepatobiliary Neoplasm'], ['Osteosclerosis - ichthyosis - premature ovarian failure', 'Activated PIK3-delta syndrome', 'Blackfan-Diamond anemia', 'Combined immunodeficiency due to ZAP70 deficiency', 'Autoimmune lymphoproliferative syndrome with recurrent viral infections', 'T-B+ severe combined immunodeficiency due to JAK3 deficiency', '46,XX testicular disorder of sex development', 'Severe combined immunodeficiency due to DCLRE1C deficiency', 'genetic non-acquired premature ovarian failure', 'Alpha-thalassemia - myelodysplastic syndrome'], ['Endometrial Endometrioid Adenocarcinoma', 'lung adenocarcinoma', 'acute myeloid leukemia', 'colorectal adenocarcinoma', 'gastric carcinoma', 'hepatocellular carcinoma', 'clear cell renal carcinoma', 'head and neck squamous cell carcinoma', 'melanoma', 'cutaneous melanoma'], ['self reported educational attainment', 'intelligence', 'mathematical ability', 'vitamin D measurement', 'cognitive function measurement', 'schizophrenia', 'systolic blood pressure', 'smoking status measurement', 'smoking behavior', 'hair colour measurement'], ['mental retardation with language impairment', 'Intellectual disability-severe speech delay-mild dysmorphism syndrome', 'basal cell carcinoma', 'skin carcinoma', 'esophageal adenocarcinoma', 'Intellectual disability', 'genetic disorder', 'mental retardation', 'prostate adenocarcinoma', 'self reported educational attainment'], ['Childhood apraxia of speech', 'risk-taking behaviour', 'self reported educational attainment', 'genetic disorder', 'specific language disorder', 'sleep duration', 'chronotype measurement', 'diet measurement', 'mathematical ability', 'insomnia measurement'], ['Immune dysregulation-polyendocrinopathy-enteropathy-X-linked syndrome', 'type I diabetes mellitus', 'hydrops fetalis (disease)', 'azoospermia', 'Fanconi anemia', 'Pancytopenia', 'autoimmune enteropathy', 'monogenic diabetes', 'genetic disorder', 'neoplasm'], ['prostate carcinoma', 'chronotype measurement', 'non-small cell lung carcinoma', 'lung adenocarcinoma', 'self reported educational attainment', 'gut microbiome measurement', 'Global developmental delay', 'Feeding difficulties', 'Ventricular septal defect', 'cancer'], ['hair colour measurement', 'Ischemic stroke', 'stroke', 'puerperal disorder', 'breast carcinoma', 'cutaneous melanoma', 'nevus count', 'protein measurement', 'acute myeloid leukemia', 'neoplasm'], ['bladder transitional cell carcinoma', 'esophageal squamous cell carcinoma', 'prostate adenocarcinoma', 'lymphoid neoplasm', 'gastric adenocarcinoma', 'colorectal adenocarcinoma', 'pancreatic ductal adenocarcinoma', 'basal cell carcinoma', 'clear cell renal carcinoma', 'colon adenocarcinoma'], ['glioma', 'colorectal carcinoma', 'urinary bladder cancer', 'malignant peripheral nerve sheath tumor', 'neoplasm', 'prostate cancer', 'non-small cell lung carcinoma', 'hepatocellular carcinoma', 'breast cancer', 'colorectal cancer'], ['Isolated NADH-CoQ reductase deficiency', 'Leigh syndrome', 'Mitochondrial disease', 'mitochondrial complex I deficiency', 'Maternally-inherited Leigh syndrome', 'Leigh syndrome with leukodystrophy', 'Mitochondrial encephalopathy', 'genetic disorder', 'developmental disability', 'Seizure'], ['protein measurement', 'reticulocyte count', 'heel bone mineral density', 'body height', 'neoplasm', 'ulcerative colitis', 'vulvar intraepithelial neoplasia', 'cancer', 'myelodysplastic syndrome', 'non-small cell lung carcinoma'], ['mean corpuscular volume', 'mean corpuscular hemoglobin', 'mean corpuscular hemoglobin concentration', 'gastric cancer', 'neoplasm', 'hepatocellular carcinoma', 'body height', 'red blood cell density measurement', 'gonorrhea', 'sitting height measurement'], ['ovarian carcinoma', 'lung adenocarcinoma', 'acute lymphoblastic leukemia', 'gastric cancer', 'Hyper-IgM syndrome type 3', 'bone quantitative ultrasound measurement', 'Hyper-IgM syndrome type 5', 'Primary systemic amyloidosis', 'Severe combined immunodeficiency due to CARD11 deficiency', 'Recurrent infections associated with rare immunoglobulin isotypes deficiency'], ['self reported educational attainment', 'mathematical ability', 'health study participation', 'cognitive function measurement', 'lymphocyte count', 'DNA methylation', 'lean body mass', 'infect', 'infection', 'bacteriemia'], ['periodontitis', 'ovarian cancer', 'pancreatic adenocarcinoma', 'eosinophil count', 'lymphocyte count', 'FEV/FEC ratio', 'eosinophil percentage of leukocytes', 'head and neck squamous cell carcinoma', 'breast adenocarcinoma', 'high density lipoprotein cholesterol measurement'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'cutaneous melanoma', 'prostate adenocarcinoma', 'neoplasm', 'infection', 'infect', 'ovarian cancer', 'Townes-Brocks syndrome'], ['eosinophil count', 'eosinophil percentage of leukocytes', 'high density lipoprotein cholesterol measurement', 'neutrophil percentage of leukocytes', 'neutrophil percentage of granulocytes', 'glioblastoma multiforme', 'ovarian cancer', 'alcohol drinking', 'electrocardiography', 'FEV/FEC ratio'], ['DNA methylation', 'appendicular lean mass', 'soluble triggering receptor expressed on myeloid cells 2 measurement', 'myelodysplastic syndrome', 'Menkes disease', 'Miller-Dieker syndrome', 'type I diabetes mellitus'], ['Fraser syndrome', 'Unilateral renal agenesis', 'polydactyly (disease)', 'hair color', 'ulcerative colitis', 'hair colour measurement', 'hair shape measurement', 'congenital diaphragmatic hernia', 'FEV/FEC ratio', 'hemoglobin measurement'], ['neutrophil count', 'myeloid white cell count', 'sum of basophil and neutrophil counts', 'granulocyte count', 'monocyte count', 'sum of neutrophil and eosinophil counts', 'lymphocyte count', 'eosinophil count', 'type II diabetes mellitus', 'leukocyte count'], ['neutrophil count', 'grip strength measurement', 'monocyte count', 'monocyte percentage of leukocytes', 'leukocyte count', 'refractive error measurement', 'lymphocyte percentage of leukocytes', 'body mass index', 'granulocyte percentage of myeloid white cells', 'risk-taking behaviour'], ['Oculotrichoanal syndrome', 'BNAR syndrome', 'Isolated trigonocephaly', 'Unilateral renal agenesis', 'genetic disorder', 'body mass index', 'lip morphology measurement', 'self reported educational attainment', 'body fat percentage', 'protein measurement'], ['Fraser syndrome', 'Isolated cryptophthalmia', 'Unilateral renal agenesis', 'hair color', 'polydactyly (disease)', 'Complete cryptophthalmia', 'Cryptophthalmia', 'Ambiguous genitalia', 'Toe syndactyly', 'Abnormality of the anus'], ['reticulocyte count', 'HbA1c measurement', 'red blood cell distribution width', 'Red cell distribution width', 'leukocyte count', 'platelet crit', 'chronic obstructive pulmonary disease', 'neutrophil count', 'smoking status measurement', 'platelet count'], ['albuminuria', 'urinary albumin to creatinine ratio', 'Facioscapulohumeral dystrophy', 'neoplasm', 'cancer', 'prostate cancer', 'lung adenocarcinoma', 'gastric cancer', 'colon carcinoma', 'colorectal adenocarcinoma'], ['albuminuria', 'Facioscapulohumeral dystrophy', 'urinary albumin to creatinine ratio', "Crohn's disease"], [], ['visual perception measurement'], ['chronic myelogenous leukemia', 'acute lymphoblastic leukemia', 'neoplasm', 'metastatic colorectal cancer', 'low density lipoprotein cholesterol measurement', 'total cholesterol measurement', 'C-reactive protein measurement', 'lymphoid neoplasm', 'sex hormone-binding globulin measurement', 'alcohol drinking'], ['erythrocyte count', 'bone quantitative ultrasound measurement', 'myopathy', 'protein measurement', 'heel bone mineral density', 'gastric carcinoma', 'breast carcinoma', 'ulcerative colitis', "Crohn's disease", 'uterine leiomyosarcoma'], ['hypertension', 'body height', 'diastolic blood pressure', 'pulse pressure measurement', 'systolic blood pressure', 'neuroblastoma', 'Nephropathy', 'liver dysplastic nodule', 'autosomal recessive nonsyndromic deafness 9', 'Autosomal recessive non-syndromic sensorineural deafness type DFNB'], ["Alzheimer's disease", 'refractive error measurement', 'FEV/FEC ratio', 'body height', 'acute myeloid leukemia', 'peak expiratory flow', 'daytime rest measurement', 'forced expiratory volume', 'red blood cell distribution width', 'hemoglobin measurement'], ['eye measurement', 'PR interval', 'atrial fibrillation', 'resting heart rate', 'diet measurement', 'celiac disease', 'Myopia', 'Malabsorption', 'Neurofibromatosis type 6', 'acute lymphoblastic leukemia'], ['triglyceride measurement', 'acute coronary syndrome', 'coronary artery disease', 'lipoprotein-associated phospholipase A(2) measurement', 'hemoglobin measurement', 'high density lipoprotein cholesterol measurement', 'serum albumin measurement', 'alkaline phosphatase measurement', 'sleep duration', 'schizophrenia'], ['arterial stiffness measurement', 'protein measurement', 'cleft lip', 'Cleft palate', 'glioblastoma multiforme', 'drug use measurement', 'cervical cancer', 'Colobomatous microphthalmia', 'Delta-beta-thalassemia', 'asthma'], ['congenital nystagmus', 'Nystagmus', 'genetic disorder', 'Retinal dystrophy', 'uterine fibroid', 'Uterine leiomyoma', 'postmenopausal osteoporosis', 'Melorheostosis', 'Normosmic congenital hypogonadotropic hypogonadism', 'Pyknoachondrogenesis'], ['serum alanine aminotransferase measurement', 'urate measurement', 'acute myeloid leukemia', 'fat body mass', 'lean body mass', 'whole body water mass', 'myeloid white cell count', 'open-angle glaucoma', 'platelet crit', 'BMI-adjusted hip circumference'], ['HIV-1 infection', 'Susceptibility to viral and mycobacterial infections', 'lung cancer', 'reticulocyte count', 'neoplasm', 'squamous cell lung carcinoma', 'lung adenocarcinoma', 'cancer', 'non-small cell lung carcinoma', 'breast cancer'], ['refractive error measurement', 'Myopia', 'age at onset', 'Abnormality of refraction', 'refractive error', 'Blackfan-Diamond anemia', 'Autosomal dominant secondary polycythemia', 'Heinz body anemia', 'Delta-beta-thalassemia', 'Alpha-thalassemia - myelodysplastic syndrome'], ['acute myeloid leukemia', 'Ataxia-telangiectasia', 'Friedreich ataxia', 'brucellosis', "Crohn's disease", 'Abnormality of neuronal migration'], ['X-linked non-syndromic intellectual disability', 'Intellectual disability', 'mental retardation', 'Global developmental delay', 'genetic disorder', 'FTH1-related iron overload', 'Pentosuria', 'Gollop-Wolfgang complex', 'attention deficit hyperactivity disorder', 'Myopathy due to calsequestrin and SERCA1 protein overload'], ['infection', 'infect', 'nephritis', 'hepatocellular carcinoma', 'aortic stenosis', 'gestational diabetes', 'Abnormal heart morphology', 'aortic valve calcification', 'neoplasm', 'heart valve disease'], ['Early infantile epileptic encephalopathy', 'Seizure', 'Progressive encephalopathy', 'chorea', 'Autosomal recessive non-syndromic intellectual disability', 'Young adult-onset Parkinsonism', 'body mass index', 'self reported educational attainment', 'body fat percentage', 'fat body mass'], ['cancer', 'bone development disease', 'forced expiratory volume', 'body height', 'lean body mass', 'appendicular lean mass', 'heel bone mineral density', 'Moderate albuminuria', 'urinary albumin to creatinine ratio', 'anthropometric measurement'], ['thyroid gland adenocarcinoma', 'breast adenocarcinoma', 'cutaneous melanoma', 'endometrial cancer', 'head and neck squamous cell carcinoma', 'mean corpuscular hemoglobin', 'mean corpuscular hemoglobin concentration', 'gastric adenocarcinoma', 'body mass index', 'mean corpuscular volume'], ['mean platelet volume', 'Abnormality of refraction', 'DNA methylation', 'body height', 'platelet count', 'peak expiratory flow', 'schizophrenia', 'Autosomal recessive non-syndromic intellectual disability', 'pancreatitis', 'testosterone measurement'], ['systolic blood pressure', 'platelet count', 'body height', 'age at menopause', 'menopause', 'sitting height measurement', 'mean platelet volume', 'hemoglobin measurement', 'mean corpuscular hemoglobin concentration', 'hematocrit'], ['osteoarthritis', 'body height', 'heel bone mineral density', 'blood protein measurement', 'Secreted frizzled-related protein 3 measurement', 'self reported educational attainment', 'appendicular lean mass', 'sitting height measurement', 'hepatocellular carcinoma', 'amyotrophic lateral sclerosis'], ['self reported educational attainment', 'Cleft palate', 'cleft lip', 'waist circumference', 'hip circumference', 'lean body mass', 'body fat percentage', 'fat body mass', 'body weight', 'cancer'], ['infertility', 'osteosarcoma', 'infectious disease', 'cerebral palsy'], ['leukocyte count', 'head and neck squamous cell carcinoma', 'endometrial cancer', 'breast adenocarcinoma', 'gastric adenocarcinoma', 'esophageal cancer', 'hemoglobin measurement', 'neoplasm', 'breast cancer', 'esophageal squamous cell carcinoma'], ['Retinitis pigmentosa', 'age-related macular degeneration', 'Leber congenital amaurosis', 'hearing loss', 'intellectual developmental disorder and retinitis pigmentosa; IDDRP', 'X-linked retinal dysplasia', 'Severe early-childhood-onset retinal dystrophy', 'Autosomal dominant non-syndromic sensorineural deafness type DFNA', 'Reticular dystrophy of the retinal pigment epithelium', 'Familial exudative vitreoretinopathy'], ['type II diabetes mellitus', 'intelligence', 'cognitive function measurement', 'Abnormality of refraction', 'self reported educational attainment', 'Cleft palate', 'low density lipoprotein cholesterol measurement', 'Hypotonia - cystinuria syndrome'], ['vitamin D measurement', 'neoplasm', 'uterine leiomyosarcoma', "Alzheimer's disease", 'fungal infectious disease', 'neuroblastoma', 'infect', 'infection', 'Friedreich ataxia', 'ileal neuroendocrine tumor, well differentiated, low or intermediate grade'], ['age at menarche', 'esophageal adenocarcinoma', 'alcohol consumption measurement', 'apolipoprotein A 1 measurement', 'BMI-adjusted waist-hip ratio', 'mathematical ability', 'adolescent idiopathic scoliosis', 'BMI-adjusted waist circumference', 'waist-hip ratio', 'colorectal cancer'], ['high density lipoprotein cholesterol measurement', 'acute myeloid leukemia', 'type II diabetes mellitus', 'vital capacity', 'forced expiratory volume', 'Apolipoprotein A-I deficiency', 'FEV/FEC ratio', 'Fish-eye disease', 'Hepatic veno-occlusive disease - immunodeficiency', 'platelet count'], ['Isolated follicle stimulating hormone deficiency', 'Kallmann syndrome', 'menstrual cycle measurement', 'lung adenocarcinoma', 'ovarian cancer', 'cutaneous melanoma', 'age at menopause', 'heel bone mineral density', 'luteinizing hormone measurement', 'hormone measurement'], ['Ovarian hyperstimulation syndrome', '46,XX gonadal dysgenesis', 'polycystic ovary syndrome', 'infertility', 'female infertility', 'hypogonadotropic hypogonadism', 'male infertility', 'Kallmann syndrome', 'obesity', 'follicle stimulating hormone measurement'], ['urate measurement', 'serum gamma-glutamyl transferase measurement', 'cortical surface area measurement', 'benign neoplasm of skin', 'vital capacity', 'breast cancer', 'urinary bladder cancer', 'neoplasm', 'non-small cell lung carcinoma', 'colon carcinoma'], ['Non-syndromic male infertility due to sperm motility disorder', 'azoospermia', 'self reported educational attainment', 'cognitive function measurement', 'chronotype measurement', 'intelligence', 'protein measurement', 'nonpapillary renal cell carcinoma', "Barrett's esophagus", 'Retinal dystrophy'], ['self reported educational attainment', 'appendicular lean mass', 'body height', 'glomerular filtration rate', 'follistatin measurement', 'C-reactive protein measurement', 'urate measurement', 'fat body mass', 'erythrocyte count', 'lean body mass'], ['body height', 'blood protein measurement', 'appendicular lean mass', 'refractive error measurement', 'neoplasm', 'rheumatoid arthritis', 'refractory anemia', 'colorectal carcinoma', 'breast cancer', 'cirrhosis of liver'], ['sex hormone-binding globulin measurement', 'lung carcinoma', 'breast ductal adenocarcinoma', 'lung adenocarcinoma', 'hepatocellular carcinoma', 'head and neck squamous cell carcinoma', 'brain glioblastoma', 'carcinoma of liver and intrahepatic biliary tract', 'pancreatic ductal adenocarcinoma', 'colon adenocarcinoma'], ['gut microbiome measurement', 'rheumatoid arthritis', 'joint disease', 'drug use measurement', 'eosinophil count', 'central nervous system cancer', 'electrocardiography', 'cup-to-disc ratio measurement', 'hepatocellular carcinoma', 'Sepsis'], ['body mass index', 'bipolar disorder', 'Nail dystrophy', 'alcohol dependence', 'risky sexual behaviour measurement', 'acute myeloid leukemia', 'inflammatory biomarker measurement', 'attention deficit hyperactivity disorder', 'unipolar depression', 'autism spectrum disorder'], ['Formiminoglutamic aciduria', 'Mastocytosis', 'BMI-adjusted waist-hip ratio', 'Red cell distribution width', 'geographic atrophy lesion growth', 'age-related macular degeneration', 'mental retardation', 'Intellectual disability', 'cutaneous melanoma', 'body height'], ['osteoporosis', 'schizophrenia', 'insomnia', 'red blood cell distribution width', 'autism spectrum disorder', 'anorexia nervosa', 'attention deficit hyperactivity disorder', 'bipolar disorder', 'self reported educational attainment', 'Red cell distribution width'], ['FTH1-related iron overload', 'Autosomal recessive bestrophinopathy', 'Hemochromatosis type 3', 'squamous cell lung carcinoma', 'red blood cell distribution width', 'mean corpuscular hemoglobin', 'Retinitis pigmentosa', 'Best vitelliform macular dystrophy', 'MRCS syndrome', 'neoplasm'], ['lung cancer', 'cancer', 'chronic kidney disease', 'colorectal adenoma', 'synovial sarcoma', 'breast carcinoma', 'neoplasm', 'head and neck squamous cell carcinoma', "Kaposi's sarcoma", 'Kindler syndrome'], ['Hereditary hyperferritinemia with congenital cataracts', 'Neuroferritinopathy', 'L-ferritin deficiency', 'movement disorder', 'Genetic hyperferritinemia without iron overload', 'lung adenocarcinoma', 'head and neck squamous cell carcinoma', 'glioblastoma multiforme', 'glioma', 'Atypical dentin dysplasia due to SMOC2 deficiency'], ['beta-amyloid 1-42 measurement', 'cerebrospinal fluid biomarker measurement', 'left inferior lateral ventricle volume measurement', 'systolic blood pressure', 'Young adult-onset Parkinsonism', 'FEV/FEC ratio', "Alzheimer's disease", 'cognitive decline measurement', 'Anti-Toxoplasma gondii IgG measurement', 'Cognitive impairment'], ['Lethal polymalformative syndrome, Boissel type', 'obesity', 'Obesity due to melanocortin 4 receptor deficiency', 'Genetic obesity', 'type II diabetes mellitus', 'body mass index', 'lean body mass', 'body weight', 'body fat percentage', 'fat body mass'], ['X-linked non-syndromic intellectual disability', 'mental retardation', 'genetic disorder', 'Intellectual disability', 'non-small cell lung carcinoma', 'neoplasm', 'X-linked retinal dysplasia', 'Familial exudative vitreoretinopathy', 'Severe early-childhood-onset retinal dystrophy', 'Pentosuria'], ['hematocrit', 'hemoglobin measurement', 'erythrocyte count', 'mean corpuscular hemoglobin concentration', 'myeloid white cell count', 'breast cancer', 'mean platelet volume', 'body height', 'mathematical ability', 'neuroimaging measurement'], ['chronic lymphocytic leukemia', 'glioma', 'anaplastic oligodendroglioma', 'gastric carcinoma', 'hemangioblastoma', 'carcinoma of liver and intrahepatic biliary tract', 'bile duct carcinoma', 'Transitional Meningioma', 'colon adenocarcinoma', 'basal cell carcinoma'], ['body height', 'body weights and measures', 'heel bone mineral density', 'bone quantitative ultrasound measurement', 'femoral neck bone mineral density', 'lean body mass', 'sitting height measurement', 'bone density', 'appendicular lean mass', 'fat body mass'], ['Fucosidosis', 'Intellectual disability', 'Seizure', 'Immature Reticulocyte Fraction Measurement', 'eosinophil percentage of leukocytes', 'mean corpuscular hemoglobin', 'reticulocyte count', 'eosinophil count', 'mean corpuscular volume', 'erythrocyte count'], ['pathological myopia', 'cardiac conduction defect', 'Schnyder corneal dystrophy', 'graft versus host disease', 'Sickle cell anemia', 'renal cell adenocarcinoma', 'renal cell carcinoma', 'Tuberculosis', 'neoplasm', 'gastric cancer'], ['cancer', 'neoplasm', 'Hand-foot syndrome', 'hydrops fetalis (disease)', 'Familial primary hypomagnesemia with normocalciuria and normocalcemia', 'Primary hypomagnesemia with secondary hypocalcemia', 'acute kidney failure', 'African iron overload', 'Melorheostosis', 'obesity'], ['Hemophilia A', 'Pseudo-von Willebrand disease', 'Beta-thalassemia - X-linked thrombocytopenia', 'Gray platelet syndrome', 'Congenital amegakaryocytic thrombocytopenia', 'X-linked dyserythropoetic anemia with abnormal platelets and neutropenia', 'platelet storage pool deficiency', 'Familial platelet syndrome with predisposition to acute myelogenous leukemia', 'Bernard-Soulier syndrome', 'Von Willebrand disease type 3'], ['Asperger syndrome', 'longevity', 'urinary metabolite measurement', 'chronic kidney disease', 'blood metabolite measurement', 'Sepsis', "Crohn's disease", 'pancreatic ductal adenocarcinoma', 'neoplasm'], ['anthrax infection', 'COVID-19', 'HIV infection', 'ovarian cancer', 'lung adenocarcinoma', 'breast adenocarcinoma', 'endometrial cancer', 'cutaneous melanoma', 'head and neck squamous cell carcinoma', 'coronary artery disease'], ['Frontotemporal dementia with motor neuron disease', 'sporadic amyotrophic lateral sclerosis', 'amyotrophic lateral sclerosis', 'essential tremor', 'Juvenile amyotrophic lateral sclerosis', 'liposarcoma', 'acute myeloid leukemia', 'undifferentiated pleomorphic sarcoma', 'Dermatofibrosarcoma protuberans', 'synovial sarcoma'], ['urinary albumin to creatinine ratio', 'albuminuria', 'alkaline phosphatase measurement', 'breakfast skipping measurement', 'diet measurement', 'atrial fibrillation', 'diabetes mellitus', 'stroke', 'triglyceride measurement', 'coronary artery disease'], ['low density lipoprotein cholesterol measurement', "Crohn's disease", 'vitamin B12 measurement', 'type I diabetes mellitus', 'total cholesterol measurement', 'inflammatory bowel disease', 'hypertension', 'psoriasis', 'blood metabolite measurement', 'serum carcinoembryonic antigen measurement'], ['galactoside 34-L-fucosyltransferase measurement', 'e-selectin measurement', 'red blood cell distribution width', 'galectin-3 measurement', 'cancer antigen 19.9 measurement', 'cholelithiasis', 'cancer biomarker measurement', 'serum carcinoembryonic antigen measurement', 'alkaline phosphatase measurement', 'vitamin B12 measurement'], ['neoplasm', 'colorectal carcinoma', 'lung cancer', 'colon carcinoma', 'colonic neoplasm', 'malignant colon neoplasm', 'colorectal adenocarcinoma', 'breast cancer', 'Autosomal dominant severe congenital neutropenia', 'Blackfan-Diamond anemia'], ['heart rate response to recovery post exercise', 'heart rate variability measurement', 'colorectal carcinoma', 'heel bone mineral density', 'eosinophil count', 'vital capacity', 'eosinophil percentage of leukocytes', 'forced expiratory volume', 'neoplasm', 'monocyte percentage of leukocytes'], ['basophil count', 'cancer biomarker measurement', 'mosquito bite reaction itch intensity measurement', 'blood protein measurement', 'basophil percentage of leukocytes', 'mosquito bite reaction size measurement', 'protein measurement', 'N-glycan measurement', 'vitamin B12 measurement', 'basophil percentage of granulocytes'], ['lymphocyte count', 'leukocyte count', 'monocyte count', 'e-selectin measurement', 'lymphocyte percentage of leukocytes', 'monocyte percentage of leukocytes', "Meckel's diverticulum", 'Neutrophil immunodeficiency syndrome', 'Recurrent infection due to specific granule deficiency', 'Hyper-IgM syndrome type 3'], ['congenital disorder of glycosylation with defective fucosylation', 'Intellectual disability', 'Seizure', 'COVID-19', 'genetic disorder', 'serum IgG glycosylation measurement', 'N-glycan measurement', 'platelet count', 'blood protein measurement', 'platelet crit'], ['self reported educational attainment', 'eye adnexa disease', 'HIV-1 infection', 'viral load', 'schizophrenia', 'intelligence', 'colorectal cancer', 'neoplasm', 'colorectal adenocarcinoma', 'colonic neoplasm'], ['protein measurement', 'sleep duration', 'low density lipoprotein cholesterol measurement', 'vital capacity', 'pulse pressure measurement', 'blood protein measurement', 'chronotype measurement', 'renal tubule disease', 'interstitial nephritis', 'risk-taking behaviour'], ['vital capacity', 'forced expiratory volume', 'self reported educational attainment', 'asthma', 'atrial fibrillation', 'systolic blood pressure', 'pulse pressure measurement', 'atrial flutter', 'visceral adipose tissue measurement', 'chronic obstructive pulmonary disease'], ['Caudal regression sequence', 'Thoracolumbosacral spina bifida cystica', 'Lumbosacral spina bifida aperta', 'Thoracolumbosacral spina bifida aperta', 'Upper thoracic spina bifida cystica', 'Cervical spina bifida cystica', 'Upper thoracic spina bifida aperta', 'Cervicothoracic spina bifida aperta', 'Arnold-Chiari malformation type II', 'Lumbosacral spina bifida cystica'], ['Friedreich ataxia', 'hereditary ataxia', 'Abnormality of the cardiovascular system', 'Headache', 'hypertrophic cardiomyopathy', 'Agents acting on the renin-angiotensin system use measurement', 'neoplasm', 'cancer', 'systemic lupus erythematosus', 'breast cancer'], ['myopathy, congenital, with respiratory insufficiency and bone fractures', 'myopathy, congenital proximal, with minicore lesions', 'cancer', 'schizophrenia', 'cholelithiasis', 'anorexia nervosa', 'mental retardation', 'Intellectual disability', 'acute myeloid leukemia', 'thyroid gland adenocarcinoma'], ['red blood cell distribution width', 'monocyte count', 'Calcium channel blocker use measurement', 'sex hormone-binding globulin measurement', 'testosterone measurement', 'cancer', 'Spinocerebellar ataxia type 14', 'Huntington disease-like 2', 'Huntington disease-like syndrome due to C9ORF72 expansions', 'Huntington disease-like 1'], ['hematocrit', 'hemoglobin measurement', 'Left ventricular noncompaction', 'Familial isolated restrictive cardiomyopathy', 'hypertrophic cardiomyopathy 20', 'Rett syndrome', 'hypertrophic cardiomyopathy 4', 'reticulocyte count', 'red blood cell distribution width', 'hepatocellular carcinoma'], ['Autosomal dominant primary hypomagnesemia with hypocalciuria', 'type II diabetes mellitus', 'Myopia', 'urate measurement', 'refractive error measurement', 'Abnormality of refraction', 'age at onset', 'neoplasm', 'ovarian cancer', 'high density lipoprotein cholesterol measurement'], ['breast adenocarcinoma', 'endometrial cancer', 'Blackfan-Diamond anemia', 'lung cancer', 'cholangiocarcinoma', 'esophageal squamous cell carcinoma', 'calf circumference measurement', 'breast cancer', 'neoplasm', 'Delta-beta-thalassemia'], ['blood urea nitrogen measurement', 'breast adenocarcinoma', 'eosinophil count', 'Hirschsprung disease', 'clear cell renal carcinoma', 'Senior-Loken syndrome', 'papillary renal cell carcinoma', 'Renal glucosuria', 'Hereditary central diabetes insipidus', 'Juvenile nephronophthisis'], ['mean platelet volume', 'platelet component distribution width', 'platelet count', 'monocyte count', 'age-related hearing impairment', 'red blood cell distribution width', 'reticulocyte count', 'reticulocyte measurement', 'neoplasm', 'Immature Reticulocyte Fraction Measurement'], ['neoplasm', 'hepatocellular carcinoma', 'DNA methylation', 'Monoamine oxidase A deficiency', 'high density lipoprotein cholesterol measurement', 'schizophrenia', 'very low density lipoprotein cholesterol measurement', 'phospholipid measurement', 'colorectal cancer', 'triglyceride measurement'], ['endometrial cancer', 'mean platelet volume', 'Transient neonatal diabetes mellitus', 'reticulocyte count', 'age-related hearing impairment', 'red blood cell distribution width', 'platelet component distribution width', 'Partial pancreatic agenesis', 'Hyperinsulinism due to glucokinase deficiency', 'Wolfram-like syndrome'], ['Hereditary thrombocytopenia with normal platelets', 'thrombocytopenia 3', 'Autosomal thrombocytopenia with normal platelets', 'mean corpuscular volume', 'blood platelet disease', 'platelet component distribution width', 'mean corpuscular hemoglobin concentration', 'drug use measurement', 'acute myeloid leukemia', 'blood protein measurement'], ['adolescent idiopathic scoliosis', 'Visual loss', 'breast carcinoma', 'diet measurement', 'white matter microstructure measurement', 'platelet count', 'neurexophilin-1 measurement', 'C-C motif chemokine 1 measurement', 'protein measurement', 'Acute hepatic failure'], ['Non-syndromic congenital cataract', 'Nuclear cataract', 'Zonular cataract', 'cataract', 'Partial congenital cataract', 'Total congenital cataract', 'Autosomal recessive childhood-onset cortical cataract', 'eosinophil count', 'eosinophil percentage of leukocytes', 'sum of eosinophil and basophil counts'], ['cancer', 'chronic myelogenous leukemia', 'acute lymphoblastic leukemia', 'cutaneous Leishmaniasis', 'neoplasm', 'HIV infection', 'lymphoid neoplasm', 'schizophrenia', "Parkinson's disease", 'systolic blood pressure'], ['osteosarcoma', 'neoplasm', 'chronic kidney disease', 'cancer', 'head and neck squamous cell carcinoma', 'atrial fibrillation', "Alzheimer's disease", 'Alopecia universalis', 'pancreatic ductal adenocarcinoma'], ['hair color', 'heel bone mineral density', 'hair colour measurement', 'endometrial cancer', 'protein measurement', 'glioblastoma multiforme', 'balding measurement', 'lung adenocarcinoma', 'Osteosclerosis - ichthyosis - premature ovarian failure', '46,XX testicular disorder of sex development'], ['Autosomal dominant omodysplasia', 'Omodysplasia', 'Autosomal dominant Robinow syndrome', 'Robinow syndrome', 'Short stature due to GHSR deficiency', 'endometrial cancer', 'hip bone size', 'white matter microstructure measurement', 'Keratoconus', 'lung adenocarcinoma'], ['Agenesis of corpus callosum', 'Cerebellar hypoplasia', 'Arthrogryposis syndrome', 'Hydrocephalus', 'Multicystic dysplastic kidney', 'endometrial cancer', 'ovarian cancer', 'glioblastoma multiforme', 'thyroid gland adenocarcinoma', 'neoplasm'], ['Familial exudative vitreoretinopathy', 'Coats disease', 'Persistent hyperplastic primary vitreous', 'Retinal dystrophy', 'Norrie disease', 'retinopathy of prematurity', 'endometrial cancer', 'age at menarche', 'lung adenocarcinoma', 'medulloblastoma'], ['Ocular coloboma', 'Coloboma of iris', 'Coloboma of macula', 'Coloboma of optic papilla', 'Coloboma of eye lens', 'Coloboma of choroid and retina', 'Coloboma of eyelid', 'hemoglobin measurement', 'hematocrit', 'red blood cell density measurement'], ['Autosomal dominant nail dysplasia', 'Autosomal recessive nail dysplasia', 'Nail-patella-like renal disease', 'mathematical ability', 'Non-immune hydrops fetalis', 'endometrial cancer', 'breast adenocarcinoma', 'Nephroblastoma', 'neoplasm', 'colorectal carcinoma'], ['self reported educational attainment', 'heel bone mineral density', 'bone quantitative ultrasound measurement', 'hair color', 'alcohol consumption measurement', 'smoking behaviour measurement', 'bone density', 'age at first sexual intercourse measurement', 'smoking status measurement', 'hair colour measurement'], ['drug use measurement', 'optic cup area measurement', 'ulcerative colitis', 'thyroid gland adenocarcinoma', 'breast adenocarcinoma', 'medulloblastoma', 'BMI-adjusted waist circumference', 'waist-hip ratio', 'pulmonary neuroendocrine tumor', 'smoking status measurement'], ['protein C measurement', 'glioblastoma multiforme', 'protein measurement', 'neoplasm', 'Normosmic congenital hypogonadotropic hypogonadism', 'platelet count', 'birth weight', 'high density lipoprotein cholesterol measurement', 'Osteomesopyknosis', 'pontine autosomal dominant microangiopathy with leukoencephalopathy'], ['balding measurement', 'androgenetic alopecia', 'hair shape measurement', 'Myopia', 'Alopecia', 'mathematical ability', 'glioblastoma multiforme', 'refractive error measurement', 'low grade glioma', 'thyroid gland adenocarcinoma'], ['low grade glioma', 'clear cell renal carcinoma', 'glioblastoma multiforme', 'ovarian carcinoma', 'red blood cell distribution width', 'endometrial cancer', 'head and neck squamous cell carcinoma', 'urinary bladder cancer', 'Miyoshi myopathy', 'Hyperinsulinism due to INSR deficiency'], ['self reported educational attainment', 'mean corpuscular volume', 'mathematical ability', 'body height', 'obesity', 'cancer', 'Eiken syndrome', 'colorectal cancer', 'colon carcinoma', 'axial length measurement'], ['red blood cell distribution width', 'platelet component distribution width', 'neoplasm', 'creatinine measurement', 'renal cell carcinoma', 'breast cancer', 'renal cell adenocarcinoma', 'glomerular filtration rate', 'body mass index', 'colon carcinoma'], ['high density lipoprotein cholesterol measurement', 'red blood cell distribution width', 'Testicular Germ Cell Tumor', 'neoplasm', 'Familial primary hypomagnesemia with normocalciuria and normocalcemia', 'type II diabetes mellitus', 'Primary hypomagnesemia with secondary hypocalcemia', 'African iron overload', 'Familial isolated hyperparathyroidism', 'Familial parathyroid adenoma'], ['Glycogen storage disease due to glucose-6-phosphatase deficiency type a', 'Glycogen storage disease due to glucose-6-phosphatase deficiency', 'Glycogen storage disease', 'genetic disorder', 'Short stature due to GHSR deficiency', 'leucine-induced hypoglycemia', 'refractive error measurement', 'lymphocyte percentage of leukocytes', 'neutrophil percentage of leukocytes', 'neutrophil count'], ['fasting blood glucose measurement', 'HbA1c measurement', 'glucose measurement', 'birth weight', 'HOMA-B', 'BMI-adjusted fasting blood glucose measurement', 'body mass index', 'metabolite measurement', 'coronary artery calcification', 'parental genotype effect measurement'], ['Autosomal recessive severe congenital neutropenia due to G6PC3 deficiency', 'Severe congenital neutropenia', 'Constitutional neutropenia', 'congenital heart disease', 'heel bone mineral density', 'pulse pressure measurement', 'bone density', 'Autosomal agammaglobulinemia', 'bone quantitative ultrasound measurement', 'Immunodeficiency due to interleukin-1 receptor-associated kinase-4 deficiency'], ['anemia, nonspherocytic hemolytic, due to G6PD deficiency', 'anemia (phenotype)', 'class I glucose-6-phosphate dehydrogenase deficiency', 'congenital nonspherocytic hemolytic anemia', 'glucosephosphate dehydrogenase deficiency', 'genetic disorder', 'susceptibility to malaria', 'Congenital hemolytic anemia', 'Chronic granulomatous disease', 'congenital heart disease'], ['Glycogen storage disease due to acid maltase deficiency', 'Glycogen storage disease due to acid maltase deficiency, adult onset', 'Glycogen storage disease due to acid maltase deficiency, infantile onset', 'type II diabetes mellitus', 'Glycogen storage disease', 'diabetes mellitus', 'genetic disorder', 'abnormal glucose tolerance', 'myopathy', 'Elevated circulating creatine kinase concentration'], ['cancer', 'bone development disease', 'Autosomal recessive non-syndromic sensorineural deafness type DFNB', 'protein measurement', 'body mass index', 'acute myeloid leukemia', 'diet measurement', 'platelet crit', 'lung adenocarcinoma', 'breast adenocarcinoma'], ['cancer', 'chronic myelogenous leukemia', 'hair color', 'glomerular filtration rate', 'age at menarche', 'hair colour measurement', 'triglyceride measurement', 'sex hormone-binding globulin measurement', 'low density lipoprotein cholesterol measurement', 'FEV/FEC ratio'], ['inflammatory bowel disease', 'Reticular dysgenesis', 'Combined immunodeficiency with skin granulomas', 'T-B+ severe combined immunodeficiency due to JAK3 deficiency', 'Wiskott-Aldrich syndrome', 'Familial hemophagocytic lymphohistiocytosis', 'mean corpuscular volume', 'erythrocyte count', 'Monocytopenia with susceptibility to infections', 'Short-limb skeletal dysplasia with severe combined immunodeficiency'], ['monocyte count', 'drug use measurement', 'leukocyte count', 'neutrophil count', 'breast carcinoma', 'myeloid white cell count', 'hepatitis B virus infection', 'infect', 'infection'], ['breast adenocarcinoma', 'urokinase plasminogen activator surface receptor measurement', 'birth weight', 'lymphocyte count', 'body height at birth', 'infect', 'infection', 'lymphocyte percentage of leukocytes', 'platelet count', 'monocyte count'], ['pancreatic adenocarcinoma', 'neoplasm', 'Cryopyrin-associated periodic syndrome', 'cancer', 'breast cancer', 'genetic disorder', 'metastatic melanoma', 'prostate cancer', 'gastric cancer', 'triple-negative breast cancer'], ['refractory anemia', 'rheumatoid arthritis', 'infect', 'infection', 'Autosomal dominant severe congenital neutropenia', 'blood protein measurement', 'X-linked severe congenital neutropenia', 'Blackfan-Diamond anemia', 'Neutropenia - monocytopenia - deafness', 'Recurrent infection due to specific granule deficiency'], ['Spasticity', 'alcohol dependence', 'narcolepsy with cataplexy', 'chronic fatigue syndrome', 'cerebral palsy', 'alcohol withdrawal', 'multiple sclerosis', 'obstructive sleep apnea', "Alzheimer's disease", 'pain'], ['Early infantile epileptic encephalopathy', 'Atypical Rett syndrome', 'Epileptic encephalopathy', 'Rett syndrome', 'Spasticity', 'alcohol dependence', 'narcolepsy with cataplexy', 'chronic fatigue syndrome', 'cerebral palsy', 'alcohol withdrawal'], ['neoplasm', 'cancer', 'hepatocellular carcinoma', 'non-small cell lung carcinoma', 'chronic kidney disease', "Alzheimer's disease", 'breast cancer', 'chronic obstructive pulmonary disease', 'infection', 'infect'], ['platelet count', 'testosterone measurement', 'platelet crit', 'adolescent idiopathic scoliosis', 'non-small cell lung carcinoma', 'X-linked intellectual disability - spastic quadriparesis', 'Spastic paraplegia - glaucoma - intellectual disability', 'tropical spastic paraparesis', 'X-linked spastic paraplegia type 34', 'Spastic paraplegia - epilepsy - intellectual disability'], ['body mass index', 'body height', 'appendicular lean mass', 'monocyte percentage of leukocytes', 'osteoporosis', 'age at menarche', 'skin carcinoma', 'schizophrenia', 'attention deficit hyperactivity disorder', 'waist circumference'], ['Juvenile myoclonic epilepsy', 'Dravet syndrome', 'Childhood absence epilepsy', 'Seizure', 'anxiety', 'epilepsy', 'unipolar depression', 'insomnia', 'Agitation', 'alcohol dependence'], ['Early infantile epileptic encephalopathy', 'epilepsy', 'alcohol dependence', 'unipolar depression', 'insomnia', 'Agitation', 'anxiety', 'panic disorder', 'anxiety disorder', 'pain'], ['unipolar depression', 'epilepsy', 'insomnia', 'Agitation', 'panic disorder', 'anxiety disorder', 'migraine disorder', 'anxiety', 'pain', 'alcohol dependence'], ['epilepsy', 'migraine disorder', 'alcohol dependence', 'pain', 'delirium', 'coronary artery disease', 'obesity', 'depressive disorder', 'post-operative sign or symptom', 'ischemia reperfusion injury'], ['Early infantile epileptic encephalopathy', 'Seizure', 'epilepsy', 'panic disorder', 'unipolar depression', 'insomnia', 'Agitation', 'anxiety', 'anxiety disorder', 'alcohol dependence'], ['epilepsy', 'alcohol dependence', 'pain', 'migraine disorder', 'depressive disorder', 'delirium', 'coronary artery disease', 'obesity', 'post-operative sign or symptom', 'ischemia reperfusion injury'], ['Early infantile epileptic encephalopathy', 'insomnia', 'epilepsy', 'migraine disorder', 'alcohol dependence', 'pain', 'delirium', 'coronary artery disease', 'Seizure', 'depressive disorder'], ['Early infantile epileptic encephalopathy', 'epilepsy', 'migraine disorder', 'alcohol dependence', 'pain', 'delirium', 'coronary artery disease', 'obesity', 'depressive disorder', 'post-operative sign or symptom'], ['Early infantile epileptic encephalopathy', 'Childhood absence epilepsy', 'Lennox-Gastaut syndrome', 'Epileptic encephalopathy', 'epilepsy', 'migraine disorder', 'alcohol dependence', 'pain', 'Seizure', 'coronary artery disease'], ['epilepsy', 'pain', 'Juvenile myoclonic epilepsy', 'alcohol dependence', 'migraine disorder', 'Generalized epilepsy with febrile seizures-plus', 'delirium', 'coronary artery disease', 'Seizure', 'obesity'], ['epilepsy', 'alcohol dependence', 'migraine disorder', 'pain', 'delirium', 'coronary artery disease', 'obesity', 'post-operative sign or symptom', 'depressive disorder', 'ischemia reperfusion injury'], ['alcohol dependence', 'epilepsy', 'insomnia', 'unipolar depression', 'Agitation', 'panic disorder', 'anxiety disorder', 'pain', 'anxiety', 'delirium'], ['Generalized epilepsy with febrile seizures-plus', 'Childhood absence epilepsy', 'Early infantile epileptic encephalopathy', 'epilepsy', 'Seizure', 'anxiety', 'Rolandic epilepsy', 'alcohol dependence', 'insomnia', 'unipolar depression'], ['epilepsy', 'unipolar depression', 'insomnia', 'Agitation', 'panic disorder', 'alcohol dependence', 'anxiety disorder', 'pain', 'anxiety', 'delirium'], ['epilepsy', 'alcohol dependence', 'pain', 'migraine disorder', 'delirium', 'coronary artery disease', 'obesity', 'post-operative sign or symptom', 'depressive disorder', 'neoplasm'], ['epilepsy', 'migraine disorder', 'pain', 'alcohol dependence', 'delirium', 'coronary artery disease', 'obesity', 'post-operative sign or symptom', 'depressive disorder', 'ischemia reperfusion injury'], ['self reported educational attainment', 'refractive error measurement', 'type II diabetes mellitus', 'diabetic nephropathy', 'cutaneous melanoma', 'esophageal cancer', 'lung adenocarcinoma', 'squamous cell lung carcinoma', 'breast adenocarcinoma', 'blood metabolite measurement'], ['thyroid gland adenocarcinoma', 'cutaneous melanoma', 'glioblastoma multiforme', 'esophageal cancer', 'lung adenocarcinoma', 'squamous cell lung carcinoma', 'Progressive sensorineural hearing impairment', 'breast adenocarcinoma', '46,XY partial gonadal dysgenesis', 'Endomyocardial fibroelastosis'], ['protein measurement', 'anxiety disorder', 'Retinitis pigmentosa', 'retinopathy', 'essential thrombocythemia', 'diabetes mellitus', 'hepatocellular carcinoma', 'migraine disorder', 'Rett syndrome', 'Myopia'], ['Inherited congenital spastic tetraplegia', 'developmental and epileptic encephalopathy', 'sex hormone-binding globulin measurement', 'cognitive function measurement', 'mathematical ability', 'intelligence', 'self reported educational attainment', 'adolescent idiopathic scoliosis', 'cutaneous melanoma', 'Hereditary spastic paraplegia'], ['arterial stiffness measurement', 'adolescent idiopathic scoliosis', 'thyroid gland adenocarcinoma', 'cutaneous melanoma', 'obesity', 'diabetes mellitus', 'schizophrenia', 'Autosomal dominant nocturnal frontal lobe epilepsy', 'Female restricted epilepsy with intellectual disability', 'Myoclonus-dystonia syndrome'], ['mean corpuscular volume', 'platelet count', 'red blood cell density measurement', 'monocyte count', 'facial morphology measurement', 'platelet crit', 'erythrocyte count', 'C-reactive protein measurement', 'parental genotype effect measurement', 'birth weight'], ['hepatocellular carcinoma', 'non-small cell lung carcinoma', 'rheumatoid arthritis', 'colorectal carcinoma', 'melanoma', 'refractory anemia', 'systemic lupus erythematosus', 'ovarian cancer', 'heel bone mineral density', 'bone quantitative ultrasound measurement'], ['body mass index', 'body fat percentage', 'vital capacity', 'cleft lip', 'forced expiratory volume', 'fat body mass', 'lean body mass', 'age at menarche', 'self reported educational attainment', 'hematocrit'], ['leukemia', "Alzheimer's disease", 'acute myeloid leukemia', 'prostate cancer', 'ovarian cancer', 'hepatocellular carcinoma', 'insulin resistance', 'mean corpuscular hemoglobin', 'mean corpuscular volume', 'neoplasm'], ['glomerular filtration rate', 'blood metabolite measurement', 'appendicular lean mass', 'body height', 'acute myeloid leukemia', 'fat body mass', 'vital capacity', 'systolic blood pressure', 'migraine disorder', 'schizophrenia'], ['hepatocellular carcinoma', 'neoplasm', 'head and neck squamous cell carcinoma', 'non-small cell lung carcinoma', 'melanoma', 'Epstein-Barr virus infection', 'cancer', 'mesothelioma', 'synovial sarcoma', 'liver neoplasm'], ['melanoma', 'non-small cell lung carcinoma', 'sarcoma', 'bladder tumor', 'head and neck malignant neoplasia', 'head and neck neoplasia', 'Pachyonychia congenita', 'ovarian cancer'], [], [], [], ['Miyoshi myopathy'], [], [], [], [], ['Miyoshi myopathy'], ['Vitiligo', 'cancer', 'trigeminal ganglion'], ['neoplasm', 'infertility'], ['Craniosynostosis'], [], [], [], [], ['cancer', 'Vitiligo', 'neoplasm'], [], ['neoplasm', 'gastric cancer', 'cancer'], [], ["Parkinson's disease", 'triglyceride measurement', 'glioblastoma multiforme', 'high density lipoprotein cholesterol measurement', 'Ehlers-Danlos syndrome, progeroid type', 'fat body mass', 'body fat percentage', 'lean body mass', 'hip circumference', 'prostate cancer'], ['temporal lobe epilepsy', 'body height', 'head and neck squamous cell carcinoma', 'breast adenocarcinoma', 'Seizure', 'neoplasm', 'arterial stiffness measurement', 'hepatocellular carcinoma', 'breast cancer', 'infection'], ['white matter microstructure measurement', 'neuroimaging measurement', 'brain measurement', 'brain volume measurement', 'sporadic Creutzfeld Jacob disease', 'hemoglobin measurement', 'inflammatory bowel disease', 'irritable bowel syndrome', "Barrett's esophagus", 'axial length measurement'], ['cancer antigen 125 measurement', 'adult onset asthma', "Crohn's disease", 'allergic asthma', 'blood protein measurement', 'kallikrein-11 measurement', 'asthma', 'ulcerative colitis', 'Chronic Obstructive Asthma', 'psoriasis'], ['red blood cell distribution width', 'childhood onset asthma', 'otosclerosis', 'obesity', 'bipolar I disorder', 'pituitary cancer', 'Tetralogy of Fallot', 'infect', 'infection', 'leprosy'], ['age at menarche', 'leprosy', 'diet measurement', 'reticulocyte measurement', 'mean corpuscular volume', 'age at menopause', 'mean corpuscular hemoglobin concentration', 'diastolic blood pressure', 'neuroimaging measurement', 'cortical thickness'], ['Krabbe disease', 'Seizure', 'body mass index', 'genetic disorder', 'Adult Krabbe disease', 'Late-infantile/juvenile Krabbe disease', 'Infantile Krabbe disease', 'basophil count', 'inflammatory bowel disease', 'basophil percentage of leukocytes'], ['Galactose epimerase deficiency', 'Galactosemia', 'Intellectual disability', 'Generalized galactose epimerase deficiency', 'Erythrocyte galactose epimerase deficiency', 'Immature Reticulocyte Fraction Measurement', 'reticulocyte count', 'psychotic symptoms', 'mean corpuscular hemoglobin', 'erythrocyte count'], ['Galactokinase deficiency', 'Galactosemia', 'leukocyte count', 'Junctional epidermolysis bullosa - pyloric atresia', 'low density lipoprotein cholesterol measurement', 'sex hormone-binding globulin measurement', 'apolipoprotein B measurement', 'red blood cell distribution width', 'insulin like growth factor measurement', 'sum of eosinophil and basophil counts'], ['mean platelet volume', 'FEV/FEC ratio', 'thyroid stimulating hormone measurement', 'hormone measurement', 'low density lipoprotein cholesterol measurement', 'blood protein measurement', 'peak expiratory flow', 'lung adenocarcinoma', 'nontoxic goiter', 'thyroid disease'], ['galactosemia 4', 'protein measurement', 'monocyte count', 'body height', 'red blood cell distribution width', "Barrett's esophagus", 'hip bone size', 'hip bone mineral density', 'lung carcinoma', 'Galactosemia'], ['Mucopolysaccharidosis type 4A', 'Mucopolysaccharidosis type 4', 'genetic disorder', 'hair colour measurement', 'sunburn', 'sitting height measurement', 'Adenine phosphoribosyltransferase deficiency', 'infection', 'infect', 'obstructive sleep apnea'], ['COVID-19', 'severe acute respiratory syndrome', 'sclerostin measurement', 'heel bone mineral density', 'appendicular lean mass', 'glioblastoma multiforme', 'monocyte count', 'neoplasm', 'hepatocellular carcinoma', 'X-linked lymphoproliferative disease'], ['congenital disorder of glycosylation, type iit', 'high density lipoprotein cholesterol measurement', 'triglyceride measurement', 'serum metabolite measurement', 'red blood cell distribution width', 'platelet count', 'mean platelet volume', 'hemoglobin measurement', 'sleep duration', 'neuroticism measurement'], ['familial hyperphosphatemic tumoral calcinosis/hyperphosphatemic hyperostosis syndrome', 'Tumoral calcinosis', 'tumoral calcinosis, hyperphosphatemic, familial, 3', 'bone density', 'Hyperphosphatemia', 'cognitive function measurement', 'type II diabetes mellitus', 'intelligence', 'lean body mass', 'spine bone mineral density'], ['Rare pervasive developmental disorder', 'monocyte count', 'eosinophil count', 'CMRF35-like molecule 6 measurement', 'lymphocyte count', 'C-reactive protein measurement', 'B-type natriuretic peptide to N-terminal pro B-type natriuretic peptide ratio', 'low density lipoprotein cholesterol measurement', 'insulin like growth factor measurement', 'monocyte percentage of leukocytes'], ['body height', 'glioblastoma multiforme', 'central nervous system cancer', 'lung carcinoma', 'adolescent idiopathic scoliosis', 'gastric cancer', 'lung adenocarcinoma', 'sessile serrated polyp', 'Leishmaniasis', 'pancreatic carcinoma'], ['low density lipoprotein cholesterol measurement', 'refractive error measurement', 'red blood cell distribution width', 'Keratoconus', 'lymphocyte count', 'glioblastoma multiforme', 'alcohol consumption measurement', 'alcohol use disorder measurement', 'diet measurement', 'self reported educational attainment'], ['alcohol consumption measurement', 'neuritic plaque measurement', 'self reported educational attainment', 'alcohol use disorder measurement', 'glioblastoma multiforme', 'glioma', 'body mass index', 'cervical cancer', 'heel bone mineral density', 'parental genotype effect measurement'], ['electrocardiography', 'physical activity measurement', 'Abnormality of refraction', 'taxonomic microbiome measurement', 'gut microbiome measurement', 'glioblastoma multiforme', 'genetic disorder', "Barrett's esophagus", 'Ataxia-telangiectasia', 'atypical teratoid rhabdoid tumor'], ['body mass index', 'glioblastoma multiforme', 'neoplasm', 'central nervous system cancer', 'refractory anemia', 'rheumatoid arthritis', 'breast cancer', 'neuroblastoma', 'Bartsocas-Papas syndrome', 'ulcerative colitis'], ['schizophrenia', 'autism spectrum disorder', 'self reported educational attainment', 'intelligence', 'feeling tense measurement', 'neuroticism measurement', 'body mass index', 'body fat percentage', 'paranasal sinus disease', 'lean body mass'], ['adolescent idiopathic scoliosis', 'glioblastoma multiforme', 'Pentosuria', 'Karyomegalic interstitial nephritis', 'Dent disease type 1', 'Hereditary renal hypouricemia', 'Hyperprolinemia type 1', 'Fibronectin glomerulopathy', 'Autosomal dominant progressive nephropathy with hypertension', 'Hypoxanthine guanine phosphoribosyltransferase partial deficiency'], ['colorectal cancer', 'colorectal adenocarcinoma', 'brain volume measurement', 'neuroimaging measurement', 'brain measurement', 'serum galactose-deficient IgA1 measurement', 'IGA glomerulonephritis', 'cortical surface area measurement', 'Inherited cancer-predisposing syndrome', 'forced expiratory volume'], ['mathematical ability', 'body fat percentage', 'body mass index', 'waist circumference', 'lean body mass', 'schizophrenia', 'schizoaffective disorder', 'visual perception measurement', 'pancreatic adenocarcinoma', 'depressive disorder'], ['acute myeloid leukemia', 'Non-immune hydrops fetalis', 'protein measurement', 'eosinophilic esophagitis', 'glioblastoma multiforme', 'Nijmegen breakage syndrome', 'Charcot-Marie-Tooth disease axonal type 2V', 'breast cancer', 'Benign familial chorea', 'cholangiocarcinoma'], ['protein measurement', 'acute myeloid leukemia', 'refractive error measurement', 'Myopia', 'age at onset', 'Abnormality of refraction', 'hepatocellular carcinoma', "Crohn's disease", 'esophageal squamous cell carcinoma', 'myxosarcoma'], ['self reported educational attainment', 'body mass index', 'body fat percentage', 'body height', 'phosphatidylethanolamine ether measurement', 'blood protein measurement', 'mathematical ability', 'smoking status measurement', 'adolescent idiopathic scoliosis', 'erythrocyte count'], ['self reported educational attainment', 'smoking behavior', 'stomach neoplasm', 'protein measurement', 'fibromyalgia', 'motion sickness', 'lymphocyte count', 'lean body mass', 'body height', 'appendicular lean mass'], ['self reported educational attainment', 'body mass index', 'glioblastoma multiforme', 'diastolic blood pressure', 'body fat percentage', 'alcohol drinking', 'systolic blood pressure', 'cerebrospinal fluid biomarker measurement', 'diabetes mellitus, permanent neonatal 4', 'Permanent neonatal diabetes mellitus'], ['male infertility', 'Non-syndromic male infertility due to sperm motility disorder', 'Dowling-Degos disease', 'spermatogenic failure 43', 'Partial chromosome Y deletion', 'Male infertility due to globozoospermia', 'spermatogenic failure 42', 'Familial generalized lentiginosis', 'Neurofibromatosis type 6', 'Linear atrophoderma of Moulin'], ['drug use measurement', 'smoking behavior', 'reticulocyte count', 'protein measurement', 'smoking status measurement', 'self reported educational attainment', 'body mass index', 'mathematical ability', 'body weight', 'attention deficit hyperactivity disorder'], ['Down syndrome', 'Ring chromosome Y', 'Primary hypergonadotropic hypogonadism - partial alopecia', 'obesity', 'diabetes mellitus', 'type I diabetes mellitus', 'Marchiafava-Bignami Disease', "Alzheimer's disease", 'ganglioneuroma', 'breast cancer'], ['lean body mass', 'appendicular lean mass', 'fat body mass', 'body height', 'insulin like growth factor measurement', 'smoking status measurement', 'whole body water mass', 'base metabolic rate measurement', 'DNA methylation', 'protein measurement'], ['head and neck squamous cell carcinoma', 'breast adenocarcinoma', 'mononeuropathy', 'colitis', 'Severe combined immunodeficiency due to CORO1A deficiency', 'Autosomal dominant severe congenital neutropenia', 'Combined immunodeficiency with skin granulomas', 'inflammatory bowel disease', 'irritable bowel syndrome', 'aplasia cutis congenita (disease)'], ['heel bone mineral density', 'colitis', 'Homozygous familial hypercholesterolemia', 'sitting height measurement', 'bone quantitative ultrasound measurement', 'Myoclonus-dystonia syndrome', 'obsessive-compulsive disorder', 'Rapid-onset dystonia-parkinsonism', 'Asperger syndrome', 'Benign familial chorea'], ['Classic galactosemia', 'Galactosemia', 'cataract', 'Intellectual disability', 'primary ovarian insufficiency', 'blood protein measurement', 'chemokine (C-C motif) ligand 27 measurement', 'testosterone measurement', 'adolescent idiopathic scoliosis', 'appendicular lean mass'], ['Guanidinoacetate methyltransferase deficiency', 'Disorder of creatine biosynthesis', 'Seizure', 'Intellectual disability', 'mental retardation', 'cervical cancer', 'Ichthyosis - intellectual disability - dwarfism - renal impairment', 'Ring chromosome Y', 'Classic galactosemia', 'lung cancer'], ['Giant axonal neuropathy', 'genetic disorder', 'Hypotonia', 'Intellectual disability', 'mental retardation', 'clear cell renal carcinoma', 'Charcot-Marie-Tooth disease', 'Myopia', 'age at onset', 'Hereditary myoclonus - progressive distal muscular atrophy'], ['Autosomal dominant polycystic kidney disease', 'polycystic kidney disease 3 with or without polycystic liver disease', 'kidney disease', 'Isolated polycystic liver disease', 'severe acute respiratory syndrome', 'COVID-19', 'Polycystic Kidney Disease', 'self reported educational attainment', 'cognitive function measurement', 'mathematical ability'], ['fibrinogen measurement', 'basophil count', 'basophil percentage of leukocytes', 'hemoglobin measurement', 'monocyte percentage of leukocytes', 'hematocrit', 'monocyte count', 'Ehlers-Danlos syndrome, progeroid type', 'Tuberculosis', 'IGA glomerulonephritis'], ['mathematical ability', 'diet measurement', 'DNA methylation', 'smoking status measurement', 'brain measurement', 'neuroimaging measurement', 'pulse pressure measurement', 'systolic blood pressure', 'breast adenocarcinoma', 'type II diabetes mellitus'], ['endometrial cancer', 'Glycogen storage disease due to lactate dehydrogenase H-subunit deficiency', 'neoplasm', 'non-small cell lung carcinoma', 'amyotrophic lateral sclerosis', 'glioblastoma multiforme', 'axial length measurement', 'Huntington disease', 'hepatocellular carcinoma', 'experimental autoimmune encephalomyelitis'], ['glioblastoma multiforme', 'head and neck squamous cell carcinoma', 'lung adenocarcinoma', 'gastric adenocarcinoma', 'breast adenocarcinoma', 'platelet count', 'squamous cell lung carcinoma', 'platelet crit', 'cholelithiasis', 'cholestasis, intrahepatic, of pregnancy 3'], ['self reported educational attainment', 'mathematical ability', 'intelligence', 'lymphocyte count', 'adolescent idiopathic scoliosis', 'schizophrenia', 'myeloid white cell count', 'reticulocyte count', 'neutrophil count', 'leukocyte count'], ['Familial idiopathic steroid-resistant nephrotic syndrome', 'smoking behavior', 'sleep apnea', 'smoking status measurement', 'lean body mass', 'platelet component distribution width', 'smoking initiation', 'appendicular lean mass', 'fat body mass', 'glioblastoma multiforme'], ['endometrial cancer', 'lean body mass', 'body weight', 'fat body mass', 'whole body water mass', 'base metabolic rate measurement', 'diabetes mellitus, permanent neonatal 4', 'Transient neonatal diabetes mellitus', 'DNA methylation', 'Permanent neonatal diabetes mellitus'], ['PR interval', 'resting heart rate', 'diet measurement', 'breast carcinoma', 'left ventricular systolic function measurement', 'left ventricular diastolic function measurement', 'sitting height measurement', 'adolescent idiopathic scoliosis', 'body height', 'TNF-related apoptosis-inducing ligand measurement'], ['red blood cell distribution width', 'mean corpuscular hemoglobin concentration', 'Myoclonus-dystonia syndrome', 'Rapid-onset dystonia-parkinsonism', 'Childhood disintegrative disorder', 'obsessive-compulsive disorder', 'Huntington disease-like 2', 'Benign familial chorea', 'Huntington disease-like syndrome due to C9ORF72 expansions', 'Huntington disease-like 1'], ['Intellectual disability', 'mental retardation', 'alcohol consumption measurement', 'alcohol use disorder measurement', 'body height', 'platelet crit', 'appendicular lean mass', 'body fat distribution', 'intraocular pressure measurement', 'body fat percentage'], ['heel bone mineral density', 'pulse pressure measurement', 'mean corpuscular volume', 'red blood cell distribution width', 'electrocardiography', 'attention deficit hyperactivity disorder', 'intellectual disability, autosomal dominant 45', 'X-linked dominant intellectual disability - epilepsy syndrome', 'mean corpuscular hemoglobin', 'Severe combined immunodeficiency due to CORO1A deficiency'], ['Autosomal dominant Charcot-Marie-Tooth disease type 2D', 'Distal hereditary motor neuropathy type 5', 'Charcot-Marie-Tooth disease', 'Distal hereditary motor neuropathy type 2', 'spinal muscular atrophy, infantile, James type', 'genetic disorder', 'Distal spinal muscular atrophy', 'Giant axonal neuropathy', 'Hereditary motor and sensory neuropathy type 5', 'motor neuron disease'], ['non-small cell lung carcinoma', 'lung adenocarcinoma', 'neoplasm', 'malignant pleural mesothelioma', 'lung cancer', 'mesothelioma', 'cancer', 'lung neoplasm', 'squamous cell lung carcinoma', 'metastasis'], ['Midline interhemispheric variant of holoprosencephaly', 'Semilobar holoprosencephaly', 'Alobar holoprosencephaly', 'Microform holoprosencephaly', 'Lobar holoprosencephaly', 'Septopreoptic holoprosencephaly', 'heel bone mineral density', 'reticulocyte count', 'reticulocyte measurement', 'bone quantitative ultrasound measurement'], ['platelet count', 'Beta blocking agent use measurement', 'diastolic blood pressure', 'systolic blood pressure', 'breast adenocarcinoma', 'endometrial cancer', 'low grade glioma', 'ovarian cancer', 'smoking status measurement', 'Fanconi anemia complementation group F'], ['mean platelet volume', 'platelet component distribution width', 'heel bone mineral density', 'reticulocyte measurement', 'platelet count', 'type II diabetes mellitus', 'hemorrhoid', 'benign prostatic hyperplasia', 'Testicular regression syndrome', 'Normosmic congenital hypogonadotropic hypogonadism'], ['Primary ciliary dyskinesia', 'sinusitis', 'reticulocyte count', 'Lung agenesis - heart defect - thumb anomalies', 'Wolfram-like syndrome', 'Wolfram syndrome', 'endometrial carcinoma', 'papillary thyroid carcinoma', 'neoplasm'], ['low density lipoprotein cholesterol measurement', 'Left ventricular noncompaction', 'Familial isolated restrictive cardiomyopathy', 'Sensorineural deafness with dilated cardiomyopathy', 'Glycogen storage disease due to LAMP-2 deficiency', 'Fatal infantile cytochrome C oxidase deficiency', 'His bundle tachycardia', 'Autosomal recessive limb-girdle muscular dystrophy type 2E', 'Distal myopathy with posterior leg and anterior hand involvement', 'Glycogen Storage Disease Type 2b'], ['triglyceride measurement', 'red blood cell distribution width', 'low density lipoprotein cholesterol measurement', 'erythrocyte count', 'hemoglobin measurement', 'hematocrit', 'lymphocyte count', 'red blood cell density measurement', 'alcohol drinking', 'alcohol consumption measurement'], ['intraocular pressure measurement', 'open-angle glaucoma', 'glaucoma', 'heel bone mineral density', 'visual perception measurement', 'schizophrenia', 'intelligence', 'self reported educational attainment', 'body height', 'sitting height measurement'], ['Primary ciliary dyskinesia', 'hair color', 'cancer', 'sunburn', 'mathematical ability', 'suntan', 'cognitive function measurement', 'skin cancer', 'non-melanoma skin carcinoma', 'obesity'], ['body height', 'sitting height measurement', 'BMI-adjusted waist circumference', 'insomnia measurement', 'Knee pain', 'CCL11 measurement', 'monocyte count', 'acute myeloid leukemia', 'Acute hepatic failure', 'colorectal cancer'], ['self reported educational attainment', 'ovarian cancer', 'chronotype measurement', 'in situ carcinoma', 'Secondary polycythemia', 'lung adenocarcinoma', 'neoplasm', 'clear cell renal carcinoma', "Alzheimer's disease", 'juvenile dermatomyositis'], ['breast adenocarcinoma', 'peptic ulcer disease', 'infect', 'infection', 'obstructive sleep apnea', 'pancreatic neoplasm', 'Malignant Pancreatic Neoplasm', 'pancreatic carcinoma', 'neoplasm', 'irritable bowel syndrome'], ['Thrombocytopenia with congenital dyserythropoietic anemia', 'X-linked dyserythropoetic anemia with abnormal platelets and neutropenia', 'Blackfan-Diamond anemia', 'thrombocytopenia, X-linked, with or without dyserythropoietic anemia', 'Beta-thalassemia - X-linked thrombocytopenia', 'Thrombocytopenia', 'Congenital erythropoietic porphyria', 'neutropenia', 'acute myeloid leukemia', 'myeloproliferative disorder'], ['Monocytopenia with susceptibility to infections', 'Deafness - lymphedema - leukemia', 'acute myeloid leukemia', 'myelodysplastic syndrome', 'GATA2 deficiency with susceptibility to MDS/AML', 'Constitutional neutropenia', 'prostate carcinoma', 'neutrophil percentage of leukocytes', 'pulse pressure measurement', 'ulcerative colitis'], ['Hypoparathyroidism - deafness - renal disease', 'breast adenocarcinoma', 'breast carcinoma', 'asthma', 'eosinophil count', 'acute lymphoblastic leukemia', 'B-cell acute lymphoblastic leukemia', 'eosinophil percentage of leukocytes', 'Invasive Breast Carcinoma', 'rheumatoid arthritis'], ['Interauricular communication', 'Ventricular septal defect', 'Single ventricular septal defect', 'Tetralogy of Fallot', 'Genetic cardiac anomaly', '46,XY partial gonadal dysgenesis', 'atrioventricular septal defect 4', 'congenital heart disease', 'Familial atrial fibrillation', 'Atrial septal defect, ostium secundum type'], ['congenital heart defects, multiple types, 5', 'Familial bicuspid aortic valve', 'Familial atrial fibrillation', 'Ventricular septal defect', 'vital capacity', 'forced expiratory volume', 'FEV/FEC ratio', 'benign prostatic hyperplasia', 'lower urinary tract symptom', 'platelet count'], ['Pancreatic hypoplasia - diabetes - congenital heart disease', 'Tetralogy of Fallot', 'Interauricular communication', 'congenital heart disease', 'congenital diaphragmatic hernia', 'Abnormality of cardiovascular system morphology', 'Conotruncal heart malformations', 'persistent truncus arteriosus (disease)', 'Neonatal diabetes mellitus', 'Atrial septal defect, ostium secundum type'], ['Familial isolated dilated cardiomyopathy', 'dilated cardiomyopathy', 'Familial dilated cardiomyopathy', 'Abnormality of the cardiovascular system', 'heel bone mineral density', 'neoplasm', 'hepatocellular carcinoma', 'eosinophil percentage of leukocytes', 'ulcer disease', 'bone quantitative ultrasound measurement'], ['endometrial cancer', 'breast adenocarcinoma', 'ovarian cancer', 'gastric adenocarcinoma', 'head and neck squamous cell carcinoma', 'lung adenocarcinoma', 'urinary bladder cancer', 'breast carcinoma', 'red blood cell distribution width', 'appendicular lean mass'], ['Severe intellectual disability-poor language-strabismus-grimacing face-long fingers syndrome', 'mental retardation', 'genetic disorder', 'Intellectual disability', 'endometrial cancer', 'breast adenocarcinoma', 'acute myeloid leukemia', 'lung adenocarcinoma', 'head and neck squamous cell carcinoma', 'gastric adenocarcinoma'], ['combined oxidative phosphorylation deficiency 41', 'mathematical ability', 'intelligence', 'cognitive function measurement', 'self reported educational attainment', 'nasal cavity and paranasal sinus carcinoma', 'nasal cavity and paranasal sinus neoplasm', 'neoplasm of middle ear', 'lean body mass', 'body mass index'], ['combined oxidative phosphorylation deficiency 42', 'red blood cell density measurement', 'reticulocyte count', 'mean corpuscular volume', 'reticulocyte measurement', 'hematocrit', 'mathematical ability', 'sex hormone-binding globulin measurement', 'mean corpuscular hemoglobin', 'Immature Reticulocyte Fraction Measurement'], ['low density lipoprotein cholesterol measurement', 'corneal endothelial cell measurement', 'intraocular pressure measurement', 'Transaldolase deficiency', 'attention deficit hyperactivity disorder', 'Ehlers-Danlos syndrome, progeroid type', 'squamous cell carcinoma', 'glioblastoma multiforme'], ["Crohn's disease", 'ulcerative colitis', 'inflammatory bowel disease', 'ankylosing spondylitis', 'psoriasis', 'sclerosing cholangitis', 'diet measurement', 'trichinosis', 'Dravet syndrome', 'lung carcinoma'], ['relapsing-remitting multiple sclerosis'], ['Arginine:glycine amidinotransferase deficiency', 'Primary Fanconi syndrome', 'glomerular filtration rate', 'creatinine measurement', 'kidney failure', 'cerebral creatine deficiency syndrome', 'chronic kidney disease', 'urinary albumin to creatinine ratio', 'appendicular lean mass', 'albuminuria'], ['Gaucher disease', 'Gaucher disease type 1', 'Fetal Gaucher disease', 'Gaucher disease type 2', 'Gaucher disease type 3', 'Gaucher disease - ophthalmoplegia - cardiovascular calcification', "Parkinson's disease", 'Lewy body dementia', 'late-onset Parkinson disease', 'Thrombocytopenia'], ['Autosomal recessive spastic paraplegia type 46', 'Autosomal recessive cerebellar ataxia with late-onset spasticity', 'Spastic paraplegia', 'Hereditary spastic paraplegia', 'neuroimaging measurement', 'brain measurement', 'brain volume measurement', 'red blood cell distribution width', 'type II diabetes mellitus', 'mental retardation'], ['neoplasm', 'infection', 'infect', 'hepatocellular carcinoma', 'glioma', 'viral disease', 'stomach neoplasm', 'gastric cancer', 'metastatic neoplasm', 'metabolic syndrome'], ['Glycogen storage disease due to glycogen branching enzyme deficiency', 'Glycogen storage disease due to glycogen branching enzyme deficiency, adult neuromuscular form', 'Adult polyglucosan body disease', 'Glycogen storage disease due to glycogen branching enzyme deficiency, congenital neuromuscular form', 'Glycogen storage disease due to glycogen branching enzyme deficiency, fatal perinatal neuromuscular form', 'Glycogen storage disease due to glycogen branching enzyme deficiency, childhood neuromuscular form', 'Glycogen storage disease due to glycogen branching enzyme deficiency, childhood combined hepatic and myopathic form', 'Glycogen storage disease due to glycogen branching enzyme deficiency, non progressive hepatic form', 'Glycogen storage disease due to glycogen branching enzyme deficiency, progressive hepatic form', 'Glycogen storage disease'], ['mean corpuscular volume', 'cognitive function measurement', 'self reported educational attainment', 'mathematical ability', 'pancreatic adenocarcinoma', 'smoking status measurement', 'mean corpuscular hemoglobin concentration', "Parkinson's disease", 'sleep duration', 'monocyte count'], ['platelet count', 'platelet component distribution width', 'sum of eosinophil and basophil counts', 'ovarian cancer', 'Testicular regression syndrome', 'Normosmic congenital hypogonadotropic hypogonadism', '46,XY disorder of sex development due to 5-alpha-reductase 2 deficiency', 'Familial adrenal hypoplasia with absent pituitary luteinizing hormone', 'Persistent Müllerian duct syndrome', '46,XX testicular disorder of sex development'], ['acute myeloid leukemia', 'inflammatory bowel disease', 'neoplasm', 'glioblastoma multiforme', 'cancer', 'glioma', 'prostate cancer', 'lung cancer', 'cervical cancer', 'colorectal carcinoma'], ['thyroid gland adenocarcinoma', 'inflammatory bowel disease', 'breast cancer', 'neoplasm', 'Autosomal agammaglobulinemia', 'Mendelian susceptibility to mycobacterial diseases due to complete IL12RB1 deficiency', 'X-linked mendelian susceptibility to mycobacterial diseases due to CYBB deficiency', 'Mendelian susceptibility to mycobacterial diseases due to partial IRF8 deficiency', 'Autosomal recessive hyper-IgE syndrome due to TYK2 deficiency', 'Mendelian susceptibility to mycobacterial diseases due to complete IFNgammaR2 deficiency'], ['Autosomal agammaglobulinemia', 'Recurrent infections associated with rare immunoglobulin isotypes deficiency', 'Autosomal recessive early-onset inflammatory bowel disease', 'Mendelian susceptibility to mycobacterial diseases due to partial IRF8 deficiency', 'Autosomal recessive hyper-IgE syndrome due to TYK2 deficiency', 'Hyper-IgM syndrome type 2', 'Immunodeficiency by defective expression of HLA class 2', 'Immunodeficiency due to interleukin-1 receptor-associated kinase-4 deficiency', 'Immunodeficiency due to ficolin3 deficiency', 'T-B+ severe combined immunodeficiency due to gamma chain deficiency'], ['acute myeloid leukemia', 'inflammatory bowel disease', 'infection', 'infect', 'neoplasm', 'colorectal carcinoma', 'Homozygous familial hypercholesterolemia', 'Cholesterol-ester transfer protein deficiency', 'Sitosterolemia', 'Fish-eye disease'], ['thyroid gland adenocarcinoma', 'inflammatory bowel disease', 'Neurodevelopmental disorder', 'Autosomal agammaglobulinemia', 'Hereditary renal hypouricemia', 'Autosomal recessive hyper-IgE syndrome due to TYK2 deficiency', 'Immunodeficiency due to interleukin-1 receptor-associated kinase-4 deficiency', 'Hyper-IgM syndrome type 2', 'breast carcinoma', 'Recurrent infection due to specific granule deficiency'], ['FTH1-related iron overload', 'Pentosuria', 'Myopathy due to calsequestrin and SERCA1 protein overload', 'Metabolic myopathy due to lactate transporter defect', 'Hereditary persistence of alpha-fetoprotein', 'Congenital deficiency in alpha-fetoprotein', 'non-small cell lung carcinoma', 'psoriasis', 'head and neck squamous cell carcinoma', 'infection'], ['pancreatic adenocarcinoma', 'attention deficit hyperactivity disorder', 'intellectual disability, autosomal dominant 45', 'X-linked dominant intellectual disability - epilepsy syndrome', 'Severe combined immunodeficiency due to CORO1A deficiency', 'Rolandic epilepsy - speech dyspraxia', 'Rolandic epilepsy', 'Continuous spikes and waves during sleep', 'Early infantile epileptic encephalopathy', 'viral disease'], ['Benign familial chorea', 'Spinocerebellar ataxia type 4', 'Autosomal recessive spondylocostal dysostosis', 'Syndactyly type 5', 'Metaphyseal chondrodysplasia, Schmid type', 'Spondylometaphyseal dysplasia, A4 type', 'Huntington disease-like 2', 'Early infantile epileptic encephalopathy', 'Autosomal dominant Charcot-Marie-Tooth disease type 2O', 'Huntington disease-like 1'], ['self reported educational attainment', 'mathematical ability', 'schizophrenia', 'intelligence', 'drug use measurement', 'adolescent idiopathic scoliosis', 'Hemifacial microsomia', 'lean body mass', 'body mass index', 'fat body mass'], ['vitamin D measurement', 'lymphocyte count', 'vitamin D-binding protein measurement', 'platelet count', 'leukocyte count', 'monocyte count', 'neutrophil count', 'blood protein measurement', 'platelet crit', 'myeloid white cell count'], ['type II diabetes mellitus', 'hypothyroidism', 'psoriasis', 'lymphocyte count', 'leukocyte count', 'adolescent idiopathic scoliosis', 'psoriasis vulgaris', 'neutrophil count', 'ulcerative colitis', 'cancer'], ['erythrocyte count', 'hip bone size', 'hip bone mineral density', 'mean corpuscular volume', 'sex hormone-binding globulin measurement', 'intraocular pressure measurement', 'open-angle glaucoma', 'red blood cell density measurement', 'cognitive function measurement', 'mathematical ability'], ['intelligence', 'cognitive function measurement', 'type II diabetes mellitus', 'Abnormality of refraction', 'self reported educational attainment', 'Cleft palate', 'diabetes mellitus'], ['low density lipoprotein cholesterol measurement', 'mean platelet volume', 'platelet count', 'body height', 'helix rolling', 'Progressive sensorineural hearing impairment', 'Hirsutism', 'lobe attachment', 'androgenetic alopecia', 'apolipoprotein B measurement'], ['Glutaryl-CoA dehydrogenase deficiency', 'dystonic disorder', 'mean corpuscular hemoglobin concentration', 'mean corpuscular volume', 'erythrocyte count', 'mean corpuscular hemoglobin', 'hematocrit', 'blood metabolite measurement', 'red blood cell distribution width', 'mean reticulocyte volume'], ['hippocampal atrophy', 'acute myeloid leukemia', 'chronotype measurement', 'reaction time measurement', 'Ehlers-Danlos syndrome, progeroid type', 'gonorrhea', 'mucous membrane pemphigoid', 'gingivitis', 'dyslexia', 'diabetes mellitus'], ['glioblastoma multiforme', 'breast adenocarcinoma', 'medulloblastoma', 'myeloid white cell count', 'hepatocellular carcinoma', 'mathematical ability', 'cognitive function measurement', 'type I diabetes mellitus', 'Hypoglycemia', 'type II diabetes mellitus'], ['type I diabetes mellitus', 'infertility', 'diabetes mellitus', 'Hypoglycemia', 'obesity', 'hemoglobin measurement', 'hematocrit', 'breast adenocarcinoma', 'erythrocyte count', 'red blood cell density measurement'], ['GTP cyclohydrolase I deficiency', 'GTP cyclohydrolase I deficiency with hyperphenylalaninemia', 'Autosomal dominant dopa-responsive dystonia', 'Hyperphenylalaninemia', 'Dopa-responsive dystonia', 'Dystonia', 'Hereditary spastic paraplegia', 'dystonic disorder', 'Autosomal recessive dopa-responsive dystonia', 'genetic disorder'], ['triglyceride measurement', 'neoplasm', 'nasopharyngeal squamous cell carcinoma', 'nasopharyngeal neoplasm', 'lung cancer', 'Pituitary Gland Adenoma', 'experimental autoimmune encephalomyelitis', 'mean corpuscular volume', 'small cell lung carcinoma', "Alzheimer's disease"], ['MODY', 'Permanent neonatal diabetes mellitus', 'Hyperinsulinism due to glucokinase deficiency', 'type II diabetes mellitus', 'DEND syndrome', 'gestational diabetes', 'monogenic diabetes', 'permanent neonatal diabetes mellitus', 'Maternal diabetes', 'Transient neonatal diabetes mellitus'], ['type II diabetes mellitus', 'triglyceride measurement', 'lean body mass', 'total cholesterol measurement', 'platelet count', 'urate measurement', 'C-reactive protein measurement', 'low density lipoprotein cholesterol measurement', 'neutrophil count', 'appendicular lean mass'], ['Gamma-glutamylcysteine synthetase deficiency', 'hemolytic anemia', 'monocyte count', 'monocyte percentage of leukocytes', 'low density lipoprotein cholesterol measurement', 'sleep duration', 'birth weight', 'vascular endothelial growth factor A measurement', 'total cholesterol measurement', 'platelet component distribution width'], ['hemolytic anemia', 'Abnormality of refraction', 'neoplasm', 'triglyceride measurement', 'balding measurement', 'high density lipoprotein cholesterol measurement', 'urate measurement', 'polyunsaturated fatty acid measurement', 'Pentosuria', 'refractive error measurement'], ['fetal growth restriction', 'placenta disease', 'seminoma', 'hypertension', 'ischemia reperfusion injury', 'neoplasm', 'choriocarcinoma', 'iris', 'infection', 'infect'], ['Familial isolated hyperparathyroidism', 'hypoparathyroidism, familial isolated, 2', 'Familial isolated hypoparathyroidism', 'Familial isolated hypoparathyroidism due to agenesis of parathyroid gland', 'hyperparathyroidism', 'benign neoplasm of parathyroid gland', 'parathyroid disease', 'genetic disorder', 'age at menopause', 'Pseudohypoparathyroidism type 2'], ['bipolar disorder', 'platelet count', 'Fetal akinesia deformation sequence', 'Arthrogryposis syndrome', 'drug use measurement', 'adolescent idiopathic scoliosis', 'erythrocyte count', 'bacteriemia', 'neoplasm', 'renal cell adenocarcinoma'], ['acute myeloid leukemia', 'teratoma', 'endocarditis', 'cancer', 'germ cell tumor', 'infertility', 'prostate carcinoma', 'Ewing sarcoma', 'angiosarcoma', 'adrenal cortex carcinoma'], ['monocyte count', 'gut microbiome measurement', 'P-selectin glycoprotein ligand 1 measurement', 'glioblastoma multiforme', 'Posterior cortical atrophy', 'primary cutaneous amyloidosis', 'prostate cancer', 'choriocarcinoma', 'neoplasm', 'refractive error measurement'], ['cataract 13 with adult I phenotype', 'Non-syndromic congenital cataract', 'Total congenital cataract', 'monocyte count', 'lymphocyte count', 'monocyte percentage of leukocytes', 'myeloid white cell count', 'protein measurement', 'leukocyte count', 'hair colour measurement'], ['glioblastoma multiforme', 'colorectal carcinoma', 'gut microbiome measurement', 'taxonomic microbiome measurement', 'polymyalgia rheumatica', 'sex hormone-binding globulin measurement', 'Autosomal recessive early-onset inflammatory bowel disease', 'Autosomal agammaglobulinemia', 'macula measurement', 'Hyper-IgM syndrome type 2'], ['body mass index', 'C-reactive protein measurement', 'aspartate aminotransferase measurement', 'total cholesterol measurement', 'low density lipoprotein cholesterol measurement', 'glioblastoma multiforme', 'protein measurement', 'mean platelet volume', 'blood metabolite measurement', 'lipoprotein measurement'], ['smoking status measurement', 'systolic blood pressure', 'Abnormality of refraction', 'hair color', 'hair shape measurement', 'balding measurement', 'protein measurement', 'hair colour measurement', 'Alopecia', 'snoring measurement'], ['eosinophil count', 'neutrophil percentage of granulocytes', 'eosinophil percentage of leukocytes', 'eosinophil percentage of granulocytes', 'sum of eosinophil and basophil counts', 'lymphoma', 'diffuse large B-cell lymphoma', 'follicular lymphoma', 'classic Hodgkin lymphoma', "Alzheimer's disease"], ['platelet count', 'mean platelet volume', 'platelet component distribution width', 'platelet crit', 'liver fibrosis measurement', 'alcohol abuse', 'reticulocyte count', 'mean corpuscular volume', 'basophil count', 'monocyte count'], ['Neonatal glycine encephalopathy', 'Atypical glycine encephalopathy', 'Infantile glycine encephalopathy', 'Glycine encephalopathy', 'glycine measurement', 'serum metabolite measurement', 'age at onset', 'Myopia', 'amino acid measurement', 'mosaic loss of chromosome Y measurement'], ['cutaneous melanoma', 'adolescent idiopathic scoliosis', "Barrett's esophagus", 'psoriasis', 'dry eye syndrome', 'esophageal adenocarcinoma', 'central nervous system cancer', 'glioblastoma multiforme', 'Hemolytic anemia due to red cell pyruvate kinase deficiency', 'medulloblastoma'], ['Autosomal recessive Charcot-Marie-Tooth disease with hoarseness', 'Charcot-Marie-Tooth disease type 4A', 'Autosomal dominant Charcot-Marie-Tooth disease type 2K', 'Autosomal recessive intermediate Charcot-Marie-Tooth disease type A', 'Charcot-Marie-Tooth disease', 'Sensory neuropathy', 'Progressive demyelinating neuropathy with bilateral striatal necrosis', 'Elevated circulating creatine kinase concentration', 'Giant axonal neuropathy', 'Isolated asymptomatic elevation of creatine phosphokinase'], ['type II diabetes mellitus', 'platelet count', 'vasculitis', 'erythrocyte count', 'platelet component distribution width', 'lymphocyte percentage of leukocytes', 'red blood cell density measurement', 'Diuretic use measurement', 'high density lipoprotein cholesterol measurement', 'mean corpuscular volume'], ['spinocerebellar ataxia, autosomal recessive, 27', 'Intellectual disability-coarse face-macrocephaly-cerebellar hypotrophy syndrome', 'myeloid white cell count', 'vital capacity', 'body height', 'BMI-adjusted waist circumference', 'physical activity measurement', 'lean body mass', 'mean corpuscular volume', 'smoking behavior'], ['Glycogen storage disease due to lactate dehydrogenase H-subunit deficiency', 'Glutathione synthetase deficiency without 5-oxoprolinuria', 'acatalasia', 'Acatalasemia', 'Cerulean cataract', 'Familial exudative vitreoretinopathy', 'Zonular cataract', 'Pulverulent cataract', 'Total congenital cataract', 'Posterior polar cataract'], ['Ivemark syndrome', 'Congenitally uncorrected transposition of the great arteries', 'Tetralogy of Fallot', 'Double outlet right ventricle', 'Heterotaxia', 'Conotruncal heart malformations', 'genetic disorder', 'Progressive myoclonic epilepsy', 'Spinocerebellar ataxia type 41', 'Dysequilibrium syndrome'], ['Hereditary hemorrhagic telangiectasia', 'pulmonary arterial hypertension', 'Heritable pulmonary arterial hypertension', 'colorectal neoplasm', 'erythrocyte count', 'hemoglobin measurement', 'hematocrit', 'systolic blood pressure', 'red blood cell density measurement', 'hepatocellular carcinoma'], ['Colobomatous microphthalmia', 'Isolated anophthalmia - microphthalmia', 'Isolated Klippel-Feil syndrome', 'microphthalmia', 'neoplasm', "Alzheimer's disease", 'cancer', 'melanoma', 'retinal vein occlusion', 'breast cancer'], ['Fibular aplasia - complex brachydactyly', 'Proximal symphalangism', 'Multiple synostoses syndrome', 'Brachydactyly type A2', 'Brachydactyly type C', 'Acromesomelic dysplasia, Grebe type', 'Brachydactyly type A1', 'osteoarthritis', 'Acromesomelic dysplasia, Hunter-Thomson type', 'Brachydactyly'], ['Isolated Klippel-Feil syndrome', 'Leber congenital amaurosis', 'Multiple synostoses syndrome', 'Isolated anophthalmia - microphthalmia', 'microphthalmia', 'Congenital hydronephrosis', 'Colobomatous microphthalmia', 'Klippel-Feil syndrome', "Parkinson's disease", 'type II diabetes mellitus'], ['prostate carcinoma', 'hemorrhoid', 'pulse pressure measurement', 'esophageal adenocarcinoma', "Barrett's esophagus", 'pelvic organ prolapse', 'Abdominal Aortic Aneurysm', 'systolic blood pressure', 'type II diabetes mellitus', 'low density lipoprotein cholesterol measurement'], ['46,XX gonadal dysgenesis', 'primary ovarian insufficiency', 'Isolated CoQ-cytochrome C reductase deficiency', 'adult onset asthma', 'genetic non-acquired premature ovarian failure', '46,XX testicular disorder of sex development', 'Testicular regression syndrome', 'Osteosclerosis - ichthyosis - premature ovarian failure', 'polycystic ovary syndrome', 'Normosmic congenital hypogonadotropic hypogonadism'], ['systolic blood pressure', 'mean arterial pressure', 'prostate carcinoma', 'sitting height measurement', 'body height', 'rheumatoid arthritis', 'stroke', 'refractory anemia', 'lung cancer', 'neoplasm'], ['Orofacial clefting syndrome', 'orofacial cleft', 'chronic obstructive pulmonary disease', 'stroke', 'Hepatic fibrosis', 'cirrhosis of liver', 'schizophrenia', 'Uveal Melanoma', 'Uveal melanoma', 'non-alcoholic steatohepatitis'], ['growth differentiation factor 15 measurement', 'coffee consumption measurement', 'hyperemesis gravidarum', 'nausea and vomiting of pregnancy severity measurement', 'obesity', 'ulcer disease', 'neoplasm', 'glioblastoma multiforme', 'blood protein measurement', 'cancer'], ['X-linked non-syndromic intellectual disability', 'X-linked intellectual disability, Siderius type', 'Intellectual disability', 'mental retardation', 'genetic disorder', 'pontine autosomal dominant microangiopathy with leukoencephalopathy', 'infectious disease', 'Adrenocorticotropic hormone deficiency', 'nervous system disease', 'X-linked intellectual disability'], ['blood protein measurement', 'nevus count', 'cutaneous melanoma', 'monocyte count', 'drug use measurement', 'age at menopause', 'systolic blood pressure', 'body height', 'neoplasm', 'grip strength measurement'], ['Hirschsprung disease', 'Ondine syndrome', 'Haddad syndrome', 'prostate carcinoma', 'blood protein measurement', 'vital capacity', 'glioblastoma multiforme', 'breast adenocarcinoma', 'adrenal gland pheochromocytoma', 'Hereditary pheochromocytoma-paraganglioma'], ['leukocyte count', 'breast adenocarcinoma', 'injury', 'myeloid white cell count', 'head and neck squamous cell carcinoma', 'systolic blood pressure', 'protein measurement', 'platelet count', 'drug use measurement', 'invasive breast ductal carcinoma'], ['hyperproinsulinemia', 'Transient neonatal diabetes mellitus', "Barrett's esophagus", 'Glycogen storage disease due to hepatic glycogen synthase deficiency', 'breast carcinoma', 'diabetes mellitus, permanent neonatal 4', 'colorectal adenocarcinoma', 'ulcerative colitis', 'colorectal cancer', "Crohn's disease"], ['glioblastoma multiforme', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'mean platelet volume', 'mean corpuscular hemoglobin concentration', 'systolic blood pressure', 'eosinophil count', 'red blood cell density measurement', 'eosinophil percentage of leukocytes', 'FEV/FEC ratio'], ['body height', 'ulcerative colitis', 'fat body mass', 'protein measurement', 'body fat percentage', 'Cutaneous T-cell lymphoma', 'Chédiak-Higashi syndrome', 'Corpuscular Hemoglobin Content', 'Cyclic neutropenia', 'autosomal dominant nonsyndromic deafness'], ['acute myeloid leukemia', 'coronary artery disease', 'body height', 'glioblastoma multiforme', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'attention deficit hyperactivity disorder', 'Severe combined immunodeficiency due to CORO1A deficiency', 'Macular corneal dystrophy', 'Isolated congenital megalocornea'], ['alkaline phosphatase measurement', 'hair color', 'tauopathy'], ['systolic blood pressure', 'coronary artery disease', 'pulse pressure measurement', 'alcohol drinking', 'systemic lupus erythematosus', 'non-small cell lung carcinoma', 'Hodgkins lymphoma', 'hepatocellular carcinoma', 'Familial Mediterranean fever', 'acute lymphoblastic leukemia'], ['vitamin D measurement', 'seasonality measurement', 'self reported educational attainment', 'hepatocellular carcinoma', 'gut microbiome measurement', 'Blackfan-Diamond anemia', 'intrahepatic cholangiocarcinoma', "Behcet's syndrome", 'body mass index', 'colorectal carcinoma'], ['neurodevelopmental disorder with microcephaly, cataracts, and renal abnormalities', 'Global developmental delay', 'Microcephaly', 'Non-syndromic congenital cataract', 'osteosarcoma', 'Ewing sarcoma', 'protein measurement', 'drug use measurement', 'cancer', 'neoplasm'], ['mean corpuscular hemoglobin', 'adolescent idiopathic scoliosis', 'age at menopause', 'mean corpuscular hemoglobin concentration', 'mean corpuscular volume', 'Autosomal dominant severe congenital neutropenia', 'X-linked severe congenital neutropenia', 'erythrocyte count', 'red blood cell density measurement', 'Neutropenia - monocytopenia - deafness'], ['osteosarcoma', 'Ewing sarcoma', 'body height', 'colorectal cancer', 'severe malarial anemia', 'Proximal spinal muscular atrophy', 'infect', 'infection', 'breast carcinoma', 'motor neuron disease'], ['adolescent idiopathic scoliosis', 'monocyte count', 'systemic juvenile idiopathic arthritis', "Alzheimer's disease", 'cervical cancer'], ['esophageal adenocarcinoma', 'esophageal squamous cell carcinoma', 'acute myeloid leukemia', 'B-cell non-Hodgkins lymphoma', 'multiple myeloma', 'low grade glioma', 'glioblastoma multiforme', 'axial length measurement', 'amyotrophic lateral sclerosis', 'psoriasis'], ['breast adenocarcinoma', 'total cholesterol measurement', 'ovarian cancer', 'Inherited cancer-predisposing syndrome', 'glioblastoma multiforme', 'insulin like growth factor measurement', 'clear cell renal carcinoma', 'protein measurement', 'Ochoa syndrome', 'Familial vesicoureteral reflux'], ['platelet count', 'mean platelet volume', 'platelet crit', 'Non-accomodative esotropia', 'Esotropia', 'sitting height measurement', 'FEV/FEC ratio', 'pulse pressure measurement', 'body height', 'smoking status measurement'], ['lung adenocarcinoma', 'ovarian carcinoma', 'Ewing sarcoma', 'osteosarcoma', 'prostate carcinoma', 'infect', 'infection', 'platelet component distribution width', 'viral disease', 'ovarian cancer'], ['red blood cell distribution width', 'atrial fibrillation', 'forced expiratory volume', 'insulin like growth factor measurement', 'body mass index', 'PR interval', 'vital capacity', 'mean platelet volume', 'monocyte count', 'cyst'], ['Alexander disease', 'Autosomal dominant cerebellar ataxia', 'Seizure', 'Alexander disease type I', 'Alexander disease type II', 'genetic disorder', 'scoliosis', 'lung adenocarcinoma', 'Primary ciliary dyskinesia', 'cutaneous melanoma'], ['Congenital cataract - progressive muscular hypotonia - hearing loss - developmental delay', 'genetic disorder', 'Mitochondrial disease', 'Intellectual disability', 'red blood cell distribution width', 'hepatocellular carcinoma', 'non-alcoholic steatohepatitis', 'atherosclerosis', 'obesity', 'Barth syndrome'], ['Autosomal dominant severe congenital neutropenia', 'Severe congenital neutropenia', 'Constitutional neutropenia', 'monocyte count', 'neutropenia', 'lymphopenia (disease)', 'lymphocyte percentage of leukocytes', 'red blood cell distribution width', 'body height', 'monocyte percentage of leukocytes'], ['platelet-type bleeding disorder 17', 'Autosomal dominant macrothrombocytopenia', 'Alpha delta granule deficiency', 'Rare hemorrhagic disorder due to a constitutional platelet anomaly', 'Gray platelet syndrome', 'Thrombocytopenia', 'reticulocyte count', 'mean platelet volume', 'mean corpuscular volume', 'eosinophil count'], ['Hepatoencephalopathy due to combined oxidative phosphorylation deficiency type 1', 'combined oxidative phosphorylation deficiency', 'systolic blood pressure', 'blood protein measurement', 'PR interval', 'Leigh syndrome', 'Relative macrocephaly', 'Severe muscular hypotonia', 'Developmental regression', 'diastolic blood pressure'], ['combined oxidative phosphorylation deficiency', 'Mitochondrial disease', 'Leigh syndrome', 'Sandhoff disease', 'adrenocortical carcinoma, hereditary', 'Cytomegalic congenital adrenal hypoplasia', 'Primary pigmented nodular adrenocortical disease', 'heel bone mineral density', 'Congenital adrenal hyperplasia due to cytochrome P450 oxidoreductase deficiency', 'protein measurement'], ['lymphocyte count', 'hemoglobin measurement', 'lymphocyte percentage of leukocytes', 'aspartate aminotransferase measurement', 'red blood cell density measurement', 'monocyte count', 'neutrophil percentage of leukocytes', 'hematocrit', 'infect', 'infection'], ['infection', 'infect', 'red blood cell density measurement', 'nephritis', 'hepatocellular carcinoma', 'balding measurement', 'gestational diabetes', 'neoplasm', 'breast cancer', 'mean corpuscular hemoglobin concentration'], ['Congenital myasthenic syndromes', 'Congenital myasthenic syndromes with glycosylation defect', 'congenital disorder of glycosylation type I', 'vertebral disease', 'musculoskeletal system disease', 'spinal stenosis', 'vertebral joint disease', 'spondyloarthropathy', 'fat body mass', 'body fat percentage'], ['airway imaging measurement', 'BMI-adjusted waist circumference', 'body height', 'appendicular lean mass', 'lean body mass', 'fat body mass', 'forced expiratory volume', 'whole body water mass', 'base metabolic rate measurement', 'non-small cell lung carcinoma'], ['blood protein measurement', 'insulin like growth factor measurement', 'diet measurement', 'appendicular lean mass', 'glioblastoma multiforme', 'Hirschsprung disease', 'lung adenocarcinoma', 'neoplasm', 'breast cancer', 'colorectal carcinoma'], ['response to pazopanib', 'GDNF family receptor alpha-2 measurement', 'blood protein measurement', 'DNA methylation', 'Frontotemporal dementia', 'self reported educational attainment', 'breast adenocarcinoma', 'testosterone measurement', 'Hyperkalemia', 'fat body mass'], ['lean body mass', 'fat body mass', 'thyroid gland adenocarcinoma', 'breast adenocarcinoma', 'whole body water mass', 'base metabolic rate measurement', 'appendicular lean mass', 'prostate carcinoma', 'body height', 'Congenital ptosis'], ['glioblastoma multiforme', 'reticulocyte count', 'Arthrogryposis syndrome', 'Fetal akinesia deformation sequence', 'monocyte count', 'reticulocyte measurement', 'Familial thyroid dyshormonogenesis', 'Genetic transient congenital hypothyroidism', 'Selective pituitary resistance to thyroid hormone', 'Melorheostosis'], ['heel bone mineral density', 'body mass index', 'chronotype measurement', 'blood protein measurement', 'arterial stiffness measurement', 'daytime rest measurement', 'nausea and vomiting of pregnancy severity measurement', 'circadian rhythm', 'excessive daytime sleepiness measurement', 'Obesity due to melanocortin 4 receptor deficiency'], ['mean corpuscular hemoglobin concentration', 'esophageal squamous cell carcinoma', 'glioblastoma multiforme', 'protein measurement', 'Red cell distribution width', 'neoplasm', 'red blood cell distribution width', 'astrocytoma', 'Microvillus inclusion disease', 'Congenital enterocyte heparan sulfate deficiency'], ['Normosmic congenital hypogonadotropic hypogonadism', 'Absent thumb - short stature - immunodeficiency', 'Brachytelephalangy - dysmorphism - Kallmann syndrome', 'hypogonadotropic hypogonadism 5 with or without anosmia', 'coronary artery disease', 'pulmonary arterial hypertension', 'cardiovascular disease', 'diabetes mellitus', 'diabetic maculopathy', 'dermatomyositis'], ['Normosmic congenital hypogonadotropic hypogonadism', 'Testicular regression syndrome', 'myeloid white cell count', 'Familial adrenal hypoplasia with absent pituitary luteinizing hormone', 'diabetes mellitus, permanent neonatal 4', '46,XX testicular disorder of sex development', 'Permanent neonatal diabetes mellitus', 'Hypoinsulinemic hypoglycemia and body hemihypertrophy', '46,XY partial gonadal dysgenesis', 'cryptorchidism'], ['hepatocellular carcinoma', 'Glycogen storage disease due to hepatic glycogen synthase deficiency', 'nephrolithiasis', 'Exercise-induced hyperinsulinism', 'Hypoinsulinemic hypoglycemia and body hemihypertrophy', 'neoplasm', 'colorectal carcinoma', "Alzheimer's disease", 'head and neck squamous cell carcinoma', 'Tuberculosis'], ['lung adenocarcinoma', 'breast adenocarcinoma', 'head and neck squamous cell carcinoma', 'testosterone measurement', "Alzheimer's disease", 'waist-hip ratio', 'systemic scleroderma', 'diabetes mellitus, permanent neonatal 4', 'Permanent neonatal diabetes mellitus', 'Hypoinsulinemic hypoglycemia and body hemihypertrophy'], ['smoking status measurement', 'body height', 'Myopia', 'age at onset', 'refractive error measurement', 'smoking behavior', 'smoking initiation', 'smoking cessation', 'Propionic acidemia', 'self reported educational attainment'], ['obesity', 'adolescent idiopathic scoliosis', 'cancer', 'lung cancer', 'gastric cancer', 'reticulocyte count', 'myeloid white cell count', 'Abnormality of refraction', 'breast cancer', 'neoplasm'], ['Body skin hyperlaxity due to vitamin K-dependent coagulation factor deficiency', 'Hereditary combined deficiency of vitamin K-dependent clotting factors', 'cholesterol embolism', 'eosinophil count', 'coronary artery disease', 'cutaneous melanoma', 'prostate carcinoma', 'sum of eosinophil and basophil counts', 'Thrombocytopenia', 'Abnormal bleeding'], ['hemoglobin measurement', 'blood protein measurement', 'pes', 'neoplasm', 'Pituitary Gland Adenoma', 'adenoma', 'gestational diabetes', 'melanoma', 'infection', 'infect'], ['basophil count', "Crohn's disease", 'basophil percentage of leukocytes', 'platelet crit', 'colorectal carcinoma', 'urinary bladder cancer', 'colorectal cancer', 'Complete androgen insensitivity syndrome', 'Globozoospermia', 'neoplasm'], ['macrophage inflammatory protein 1b measurement', 'Fuchs endothelial corneal dystrophy', 'body fat percentage', 'primary cutaneous amyloidosis', 'Posterior cortical atrophy', 'fat body mass', 'lean body mass', 'body weight', 'body mass index', 'self reported educational attainment'], ['lean body mass', 'base metabolic rate measurement', 'body height', 'fat body mass', 'forced expiratory volume', 'whole body water mass', 'gastric carcinoma', 'neuroblastoma', 'sensorineural hearing loss', 'myopathy'], ['Gamma-glutamyl transpeptidase deficiency', 'Inborn errors of metabolism', 'chronic kidney disease', 'urinary metabolite measurement', 'serum gamma-glutamyl transferase measurement', 'alcohol abuse', 'blood metabolite measurement', 'liver fibrosis measurement', 'serum metabolite measurement', 'alcohol consumption measurement'], ['Inguinal hernia', 'Metaphyseal anadysplasia', 'Multiple epiphyseal dysplasia, with severe proximal femoral dysplasia', "Spondylometaphyseal dysplasia, 'corner fracture' type", 'Spondyloepimetaphyseal dysplasia - hypotrichosis', 'Smith-McCort dysplasia', 'spondyloepimetaphyseal dysplasia, Isidor-Toutain type', 'Acromesomelic dysplasia, Grebe type', 'Metaphyseal chondrodysplasia, Schmid type', 'Gollop-Wolfgang complex'], ['monocyte count', 'eosinophil count', 'myeloid white cell count', 'leukocyte count', 'monocyte percentage of leukocytes', 'eosinophil percentage of leukocytes', 'mean corpuscular hemoglobin', 'mean corpuscular volume', 'lymphocyte count', 'granulocyte percentage of myeloid white cells'], ['leukocyte count', 'clear cell renal carcinoma', 'neoplasm', 'hepatocellular carcinoma', 'papillary renal cell carcinoma', 'prostate carcinoma', 'esophageal squamous cell carcinoma', 'oral squamous cell carcinoma', 'chronic interstitial cystitis', 'colorectal cancer'], ['age at menarche', 'glomerular filtration rate', 'urate measurement', 'lean body mass', 'fat body mass', 'reticulocyte count', 'coronary artery disease', 'hair color', 'peak expiratory flow', 'reticulocyte measurement'], ['breast carcinoma', 'Acute hepatic failure', 'non-small cell lung carcinoma', 'clear cell renal carcinoma', 'oral squamous cell carcinoma', 'juvenile idiopathic arthritis', 'meningococcal infection', 'lung adenocarcinoma', 'invasive breast ductal carcinoma', 'multiple sclerosis'], ['ulcerative colitis', 'Metaphyseal anadysplasia', 'Multiple epiphyseal dysplasia, with severe proximal femoral dysplasia', "Spondylometaphyseal dysplasia, 'corner fracture' type", 'Spondyloepimetaphyseal dysplasia - hypotrichosis', 'Smith-McCort dysplasia', 'spondyloepimetaphyseal dysplasia, Isidor-Toutain type', 'Acromesomelic dysplasia, Grebe type', 'Metaphyseal chondrodysplasia, Schmid type', 'Gollop-Wolfgang complex'], ['Thyrotoxicosis', 'visual perception measurement', 'goiter', 'Metaphyseal anadysplasia', 'Multiple epiphyseal dysplasia, with severe proximal femoral dysplasia', "Spondylometaphyseal dysplasia, 'corner fracture' type", 'Spondyloepimetaphyseal dysplasia - hypotrichosis', 'Smith-McCort dysplasia', 'spondyloepimetaphyseal dysplasia, Isidor-Toutain type', 'Acromesomelic dysplasia, Grebe type'], ['Metaphyseal anadysplasia', 'Multiple epiphyseal dysplasia, with severe proximal femoral dysplasia', "Spondylometaphyseal dysplasia, 'corner fracture' type", 'Spondyloepimetaphyseal dysplasia - hypotrichosis', 'Smith-McCort dysplasia', 'spondyloepimetaphyseal dysplasia, Isidor-Toutain type', 'Acromesomelic dysplasia, Grebe type', 'Metaphyseal chondrodysplasia, Schmid type', 'Gollop-Wolfgang complex', 'Czech dysplasia, metatarsal type'], ['Non-acquired isolated growth hormone deficiency', 'Short stature due to growth hormone qualitative anomaly', 'Isolated growth hormone deficiency type IB', 'Isolated growth hormone deficiency type II', 'Isolated growth hormone deficiency type IA', 'medulloblastoma', 'isolated congenital growth hormone deficiency', 'lean body mass', 'body height', 'pes'], ['body height', 'PR interval', 'diet measurement', 'Normosmic congenital hypogonadotropic hypogonadism', 'Short stature-delayed bone age due to thyroid hormone metabolism deficiency', 'Prolactin-Producing Pituitary Gland Adenoma', 'growth hormone-secreting pituitary adenoma', 'congenital disorder of glycosylation, type IIq', 'Familial hyperprolactinemia', 'Short stature due to GHSR deficiency'], ['eosinophil count', 'mean corpuscular hemoglobin', 'whole body water mass', 'lean body mass', 'diabetic nephropathy', 'Hypoglycemia', 'diabetes mellitus', 'cancer'], ['influenza A (H1N1)', 'allergic rhinitis', 'asthma', 'serum metabolite measurement', 'allergy', 'atopic eczema', 'cancer', 'allergic asthma', 'rhinitis', 'non-small cell lung carcinoma'], ['Laron syndrome', 'Short stature due to GHSR deficiency', 'pituitary dwarfism', 'Short stature due to partial GHR deficiency', 'acromegaly', 'Turner syndrome', 'fetal growth restriction', 'Short stature', 'chronic kidney disease', 'Prader-Willi syndrome'], ['ovarian cancer', 'cancer', 'neoplasm', 'endometriosis', 'lean body mass', 'Growth Hormone-Producing Pituitary Gland Adenoma', 'prostate cancer', 'body weight', 'adrenal gland pheochromocytoma', "Alzheimer's disease"], ['isolated congenital growth hormone deficiency', 'Isolated growth hormone deficiency type IB', 'HIV-Associated Lipodystrophy Syndrome', 'Non-acquired isolated growth hormone deficiency', 'insulin like growth factor measurement', 'adolescent idiopathic scoliosis', 'ovarian cancer', 'glioblastoma multiforme', 'body height', 'genetic disorder'], ['obesity', 'ovarian cancer', 'breast adenocarcinoma', 'self reported educational attainment', 'head and neck squamous cell carcinoma', 'breast cancer', 'neoplasm', 'cancer', 'Sepsis', 'chronic obstructive pulmonary disease'], ['Short stature due to GHSR deficiency', 'lean body mass', 'body height', 'refractive error measurement', 'appendicular lean mass', 'forced expiratory volume', 'corneal topography', 'fat body mass', 'whole body water mass', 'vital capacity'], ['schizophrenia', 'autism spectrum disorder', 'obsessive-compulsive disorder', 'unipolar depression', 'attention deficit hyperactivity disorder', 'Tourette syndrome', 'bipolar disorder', 'anorexia nervosa', 'testosterone measurement', 'sex hormone-binding globulin measurement'], ['body weight', 'lean body mass', 'mean corpuscular volume', 'hip circumference', 'fat body mass', 'base metabolic rate measurement', 'whole body water mass', 'colorectal carcinoma', 'appendicular lean mass', 'monocyte count'], ['type II diabetes mellitus', 'mean corpuscular hemoglobin concentration', 'myxedema', 'pursuit maintenance gain measurement', 'schizophrenia', 'schizoaffective disorder', 'bipolar disorder', 'eosinophil count', 'low density lipoprotein cholesterol measurement', 'autoimmune thyroid disease'], ['Hereditary late-onset Parkinson disease', 'Young adult-onset Parkinsonism', "Parkinson's disease", 'lean body mass', 'schizophrenia', 'mathematical ability', 'appendicular lean mass', 'coronary artery disease', 'self reported educational attainment', 'autism spectrum disorder'], ['Combined immunodeficiency with skin granulomas', 'Activated PIK3-delta syndrome', 'Pancytopenia due to IKZF1 mutations', 'Hyper-IgM syndrome type 3', 'X-Linked Combined Immunodeficiency Diseases', 'Short-limb skeletal dysplasia with severe combined immunodeficiency', 'Autosomal agammaglobulinemia', 'Wiskott-Aldrich syndrome', 'Reticular dysgenesis', 'Autosomal dominant severe congenital neutropenia'], ['hematocrit', 'high density lipoprotein cholesterol measurement', 'fibrinogen measurement', 'eosinophil count', 'lymphocyte count', 'C-reactive protein measurement', 'lymphocyte percentage of leukocytes', 'Oral ulcer', 'neutrophil percentage of leukocytes', 'hemoglobin measurement'], ['eosinophil count', 'eosinophil percentage of leukocytes', 'lymphocyte count', 'neutrophil percentage of leukocytes', 'von Willebrand factor measurement', 'lymphocyte percentage of leukocytes', 'monocyte count', 'triglyceride measurement', 'mean platelet volume', 'reticulocyte count'], ['Fuchs endothelial corneal dystrophy', 'Posterior polymorphous corneal dystrophy', 'Lisch epithelial corneal dystrophy', 'Congenital cornea plana', 'Okt4 epitope deficiency', 'X-linked endothelial corneal dystrophy', 'Nuclear cataract', 'X-linked corneal dermoid', 'Congenital trigeminal anesthesia', 'Microcystic corneal dystrophy'], ['high density lipoprotein cholesterol measurement', 'lymphocyte count', 'C-reactive protein measurement', 'lymphocyte percentage of leukocytes', 'fibrinogen measurement', 'Oral ulcer', 'neutrophil percentage of leukocytes', 'von Willebrand factor measurement', 'triglyceride measurement', 'reticulocyte count'], ['red blood cell density measurement', 'neutrophil percentage of leukocytes', 'lymphocyte percentage of leukocytes', 'mean platelet volume', 'lymphocyte count', 'reticulocyte count', 'monocyte count', 'systolic blood pressure', 'adiponectin measurement', 'galectin-3 measurement'], ['electroencephalogram measurement', 'lymphocyte percentage of leukocytes', 'neutrophil percentage of leukocytes', 'lymphocyte count', 'monocyte count', 'reticulocyte count', 'systolic blood pressure', 'adiponectin measurement', 'galectin-3 measurement', 'hemoglobin measurement'], ['balding measurement', 'hair shape measurement', 'Alopecia', 'FEV/FEC ratio'], ['BMI-adjusted waist-hip ratio', 'sleep duration', 'type II diabetes mellitus', 'body height', 'mathematical ability', "Parkinson's disease", 'insomnia measurement', 'daytime rest measurement', 'balding measurement', 'insulin like growth factor measurement']]#list of list for conditions
uwu = ['EOLA1', 'EOLA2', 'EOMES', 'EP300', 'EP400', 'EPAS1', 'EPB41', 'EPB41L1', 'EPB41L2', 'EPB41L3', 'EPB41L4A', 'EPB41L4B', 'EPB41L5', 'EPB42', 'EPC1', 'EPC2', 'EPCAM', 'EPDR1', 'EPG5', 'EPGN', 'EPHA1', 'EPHA2', 'EPHA3', 'EPHA4', 'EPHA5', 'EPHA6', 'EPHA7', 'EPHA8', 'EPHA10', 'EPHB1', 'EPHB2', 'EPHB3', 'EPHB4', 'EPHB6', 'EPHX1', 'EPHX2', 'EPHX3', 'EPHX4', 'EPM2A', 'EPM2AIP1', 'EPN1', 'EPN2', 'EPN3', 'EPO', 'EPOP', 'EPOR', 'EPPIN', 'EPPK1', 'EPRS1', 'EPS8', 'EPS8L1', 'EPS8L2', 'EPS8L3', 'EPS15', 'EPS15L1', 'EPSTI1', 'EPX', 'EPYC', 'EQTN', 'ERAL1', 'ERAP1', 'ERAP2', 'ERAS', 'ERBB2', 'ERBB3', 'ERBB4', 'ERBIN', 'ERC1', 'ERC2', 'ERCC1', 'ERCC2', 'ERCC3', 'ERCC4', 'ERCC5', 'ERCC6', 'ERCC6L', 'ERCC6L2', 'ERCC8', 'EREG', 'ERF', 'ERFE', 'ERFL', 'ERG', 'ERG28', 'ERGIC1', 'ERGIC2', 'ERGIC3', 'ERH', 'ERI1', 'ERI2', 'ERI3', 'ERICH1', 'ERICH2', 'ERICH3', 'ERICH4', 'ERICH5', 'ERICH6', 'ERICH6B', 'ERLEC1', 'ERLIN1', 'ERLIN2', 'ERMAP', 'ERMARD', 'ERMN', 'ERMP1', 'ERN1', 'ERN2', 'ERO1A', 'ERO1B', 'ERP27', 'ERP29', 'ERP44', 'ERRFI1', 'ERV3-1', 'ERVFRD-1', 'ERVMER34-1', 'ERVV-1', 'ERVV-2', 'ERVW-1', 'ESAM', 'ESCO1', 'ESCO2', 'ESD', 'ESF1', 'ESM1', 'ESPL1', 'ESPN', 'ESPNL', 'ESR1', 'ESR2', 'ESRP1', 'ESRP2', 'ESRRA', 'ESRRB', 'ESRRG', 'ESS2', 'ESX1', 'ESYT1', 'ESYT2', 'ESYT3', 'ETAA1', 'ETDA', 'ETDB', 'ETDC', 'ETF1', 'ETFA', 'ETFB', 'ETFBKMT', 'ETFDH', 'ETFRF1', 'ETHE1', 'ETNK1', 'ETNK2', 'ETNPPL', 'ETS1', 'ETS2', 'ETV1', 'ETV2', 'ETV3', 'ETV3L', 'ETV4', 'ETV5', 'ETV6', 'ETV7', 'EVA1A', 'EVA1B', 'EVA1C', 'EVC', 'EVC2', 'EVI2A', 'EVI2B', 'EVI5', 'EVI5L', 'EVL', 'EVPL', 'EVPLL', 'EVX1', 'EVX2', 'EWSR1', 'EXD1', 'EXD2', 'EXD3', 'EXO1', 'EXO5', 'EXOC1', 'EXOC1L', 'EXOC2', 'EXOC3', 'EXOC3L1', 'EXOC3L2', 'EXOC3L4', 'EXOC4', 'EXOC5', 'EXOC6', 'EXOC6B', 'EXOC7', 'EXOC8', 'EXOG', 'EXOSC1', 'EXOSC2', 'EXOSC3', 'EXOSC4', 'EXOSC5', 'EXOSC6', 'EXOSC7', 'EXOSC8', 'EXOSC9', 'EXOSC10', 'EXPH5', 'EXT1', 'EXT2', 'EXTL1', 'EXTL2', 'EXTL3', 'EYA1', 'EYA2', 'EYA3', 'EYA4', 'EYS', 'EZH1', 'EZH2', 'EZHIP', 'EZR', 'F2', 'F2R', 'F2RL1', 'F2RL2', 'F2RL3', 'F3', 'F5', 'F7', 'F8', 'F8A1', 'F8A2', 'F8A3', 'F9', 'F10', 'F11', 'F11R', 'F12', 'F13A1', 'F13B', 'FA2H', 'FAAH', 'FAAH2', 'FAAP20', 'FAAP24', 'FAAP100', 'FABP1', 'FABP2', 'FABP3', 'FABP4', 'FABP5', 'FABP6', 'FABP7', 'FABP9', 'FABP12', 'FADD', 'FADS1', 'FADS2', 'FADS3', 'FADS6', 'FAF1', 'FAF2', 'FAH', 'FAHD1', 'FAHD2A', 'FAHD2B', 'FAIM', 'FAIM2', 'FAM3A', 'FAM3B', 'FAM3C', 'FAM3D', 'FAM8A1', 'FAM9A', 'FAM9B', 'FAM9C', 'FAM13A', 'FAM13B', 'FAM13C', 'FAM20A', 'FAM20B', 'FAM20C', 'FAM24A', 'FAM24B', 'FAM25A', 'FAM25C', 'FAM25G', 'FAM32A', 'FAM43A', 'FAM43B', 'FAM47A', 'FAM47B', 'FAM47C', 'FAM47E', 'FAM50A', 'FAM50B', 'FAM53A', 'FAM53B', 'FAM53C', 'FAM71A', 'FAM71B', 'FAM71C', 'FAM71D', 'FAM71E1', 'FAM71E2', 'FAM71F1', 'FAM71F2', 'FAM72A', 'FAM72B', 'FAM72C', 'FAM72D', 'FAM76A', 'FAM76B', 'FAM78A', 'FAM78B', 'FAM81A', 'FAM81B', 'FAM83A', 'FAM83B', 'FAM83C', 'FAM83D', 'FAM83E', 'FAM83F', 'FAM83G', 'FAM83H', 'FAM86B1', 'FAM86B2', 'FAM89A', 'FAM89B', 'FAM90A1', 'FAM90A26', 'FAM91A1', 'FAM98A', 'FAM98B', 'FAM98C', 'FAM102A', 'FAM102B', 'FAM104A', 'FAM104B', 'FAM107A', 'FAM107B', 'FAM110A', 'FAM110B', 'FAM110C', 'FAM110D', 'FAM111A', 'FAM111B', 'FAM114A1', 'FAM114A2', 'FAM117A', 'FAM117B', 'FAM118A', 'FAM118B', 'FAM120A', 'FAM120AOS', 'FAM120B', 'FAM120C', 'FAM124A', 'FAM124B', 'FAM126A', 'FAM126B', 'FAM131A', 'FAM131B', 'FAM131C', 'FAM133A', 'FAM133B', 'FAM135A', 'FAM135B', 'FAM136A', 'FAM149A', 'FAM149B1', 'FAM151A', 'FAM151B', 'FAM153A', 'FAM153B', 'FAM156A', 'FAM156B', 'FAM161A', 'FAM161B', 'FAM162A', 'FAM162B', 'FAM163A', 'FAM163B', 'FAM166A', 'FAM166B', 'FAM166C', 'FAM167A', 'FAM167B', 'FAM168A', 'FAM168B', 'FAM169A', 'FAM169B', 'FAM170A', 'FAM170B', 'FAM171A1', 'FAM171A2', 'FAM171B', 'FAM172A', 'FAM174A', 'FAM174B', 'FAM174C', 'FAM177A1', 'FAM177B', 'FAM178B', 'FAM180A', 'FAM180B', 'FAM181A', 'FAM181B', 'FAM183A', 'FAM184A', 'FAM184B', 'FAM185A', 'FAM186A', 'FAM186B', 'FAM187A', 'FAM187B', 'FAM189A1', 'FAM189A2', 'FAM189B', 'FAM193A', 'FAM193B', 'FAM199X', 'FAM200A', 'FAM200B', 'FAM204A', 'FAM205A', 'FAM205C', 'FAM209A', 'FAM209B', 'FAM210A', 'FAM210B', 'FAM214A', 'FAM214B', 'FAM216A', 'FAM216B', 'FAM217A', 'FAM217B', 'FAM219A', 'FAM219B', 'FAM220A', 'FAM221A', 'FAM221B', 'FAM222A', 'FAM222B', 'FAM227A', 'FAM227B', 'FAM228A', 'FAM228B', 'FAM229A', 'FAM229B', 'FAM234A', 'FAM234B', 'FAM236A', 'FAM236B', 'FAM236C', 'FAM236D', 'FAM237A', 'FAM237B', 'FAM240A', 'FAM240B', 'FAM240C', 'FAM241A', 'FAM241B', 'FAM243A', 'FAM243B', 'FAM246A', 'FAM246B', 'FAM246C', 'FAN1', 'FANCA', 'FANCB', 'FANCC', 'FANCD2', 'FANCD2OS', 'FANCE', 'FANCF', 'FANCG', 'FANCI', 'FANCL', 'FANCM', 'FANK1', 'FAP', 'FAR1', 'FAR2', 'FARP1', 'FARP2', 'FARS2', 'FARSA', 'FARSB', 'FAS', 'FASLG', 'FASN', 'FASTK', 'FASTKD1', 'FASTKD2', 'FASTKD3', 'FASTKD5', 'FAT1', 'FAT2', 'FAT3', 'FAT4', 'FATE1', 'FAU', 'FAXC', 'FAXDC2', 'FBF1', 'FBH1', 'FBL', 'FBLIM1', 'FBLL1', 'FBLN1', 'FBLN2', 'FBLN5', 'FBLN7', 'FBN1', 'FBN2', 'FBN3', 'FBP1', 'FBP2', 'FBRS', 'FBRSL1', 'FBXL2', 'FBXL3', 'FBXL4', 'FBXL5', 'FBXL6', 'FBXL7', 'FBXL8', 'FBXL12', 'FBXL13', 'FBXL14', 'FBXL15', 'FBXL16', 'FBXL17', 'FBXL18', 'FBXL19', 'FBXL20', 'FBXL22', 'FBXO2', 'FBXO3', 'FBXO4', 'FBXO5', 'FBXO6', 'FBXO7', 'FBXO8', 'FBXO9', 'FBXO10', 'FBXO11', 'FBXO15', 'FBXO16', 'FBXO17', 'FBXO21', 'FBXO22', 'FBXO24', 'FBXO25', 'FBXO27', 'FBXO28', 'FBXO30', 'FBXO31', 'FBXO32', 'FBXO33', 'FBXO34', 'FBXO36', 'FBXO38', 'FBXO39', 'FBXO40', 'FBXO41', 'FBXO42', 'FBXO43', 'FBXO44', 'FBXO45', 'FBXO46', 'FBXO47', 'FBXO48', 'FBXW2', 'FBXW4', 'FBXW5', 'FBXW7', 'FBXW8', 'FBXW9', 'FBXW10', 'FBXW11', 'FBXW12', 'FCAMR', 'FCAR', 'FCER1A', 'FCER1G', 'FCER2', 'FCF1', 'FCGBP', 'FCGR1A', 'FCGR1B', 'FCGR2A', 'FCGR2B', 'FCGR2C', 'FCGR3A', 'FCGR3B', 'FCGRT', 'FCHO1', 'FCHO2', 'FCHSD1', 'FCHSD2', 'FCMR', 'FCN1', 'FCN2', 'FCN3', 'FCRL1', 'FCRL2', 'FCRL3', 'FCRL4', 'FCRL5', 'FCRL6', 'FCRLA', 'FCRLB', 'FCSK', 'FDCSP', 'FDFT1', 'FDPS', 'FDX1', 'FDX2', 'FDXACB1', 'FDXR', 'FECH', 'FEM1A', 'FEM1B', 'FEM1C', 'FEN1', 'FER', 'FER1L5', 'FER1L6', 'FERD3L', 'FERMT1', 'FERMT2', 'FERMT3', 'FES', 'FETUB', 'FEV', 'FEZ1', 'FEZ2', 'FEZF1', 'FEZF2', 'FFAR1', 'FFAR2', 'FFAR3', 'FFAR4', 'FGA', 'FGB', 'FGD1', 'FGD2', 'FGD3', 'FGD4', 'FGD5', 'FGD6', 'FGF1', 'FGF2', 'FGF3', 'FGF4', 'FGF5', 'FGF6', 'FGF7', 'FGF8', 'FGF9', 'FGF10', 'FGF11', 'FGF12', 'FGF13', 'FGF14', 'FGF16', 'FGF17', 'FGF18', 'FGF19', 'FGF20', 'FGF21', 'FGF22', 'FGF23', 'FGFBP1', 'FGFBP2', 'FGFBP3', 'FGFR1', 'FGFR1OP2', 'FGFR2', 'FGFR3', 'FGFR4', 'FGFRL1', 'FGG', 'FGGY', 'FGL1', 'FGL2', 'FGR', 'FH', 'FHAD1', 'FHDC1', 'FHIP1A', 'FHIP1B', 'FHIP2A', 'FHIP2B', 'FHIT', 'FHL1', 'FHL2', 'FHL3', 'FHL5', 'FHOD1', 'FHOD3', 'FIBCD1', 'FIBIN', 'FIBP', 'FICD', 'FIG4', 'FIGLA', 'FIGN', 'FIGNL1', 'FIGNL2', 'FILIP1', 'FILIP1L', 'FIP1L1', 'FIS1', 'FITM1', 'FITM2', 'FIZ1', 'FJX1', 'FKBP1A', 'FKBP1B', 'FKBP2', 'FKBP3', 'FKBP4', 'FKBP5', 'FKBP6', 'FKBP7', 'FKBP8', 'FKBP9', 'FKBP10', 'FKBP11', 'FKBP14', 'FKBP15', 'FKBPL', 'FKRP', 'FKTN', 'FLACC1', 'FLAD1', 'FLCN', 'FLG', 'FLG2', 'FLI1', 'FLII', 'FLNA', 'FLNB', 'FLNC', 'FLOT1', 'FLOT2', 'FLRT1', 'FLRT2', 'FLRT3', 'FLT1', 'FLT3', 'FLT3LG', 'FLT4', 'FLVCR1', 'FLVCR2', 'FLYWCH1', 'FLYWCH2', 'FMC1', 'FMN1', 'FMN2', 'FMNL1', 'FMNL2', 'FMNL3', 'FMO1', 'FMO2', 'FMO3', 'FMO4', 'FMO5', 'FMOD', 'FMR1', 'FMR1NB', 'FN1', 'FN3K', 'FN3KRP', 'FNBP1', 'FNBP1L', 'FNBP4', 'FNDC1', 'FNDC3A', 'FNDC3B', 'FNDC4', 'FNDC5', 'FNDC7', 'FNDC8', 'FNDC9', 'FNDC10', 'FNDC11', 'FNIP1', 'FNIP2', 'FNTA', 'FNTB', 'FOCAD', 'FOLH1', 'FOLR1', 'FOLR2', 'FOLR3', 'FOS', 'FOSB', 'FOSL1', 'FOSL2', 'FOXA1', 'FOXA2', 'FOXA3', 'FOXB1', 'FOXB2', 'FOXC1', 'FOXC2', 'FOXD1', 'FOXD2', 'FOXD3', 'FOXD4', 'FOXD4L1', 'FOXD4L3', 'FOXD4L4', 'FOXD4L5', 'FOXD4L6', 'FOXE1', 'FOXE3', 'FOXF1', 'FOXF2', 'FOXG1', 'FOXH1', 'FOXI1', 'FOXI2', 'FOXI3', 'FOXJ1', 'FOXJ2', 'FOXJ3', 'FOXK1', 'FOXK2', 'FOXL1', 'FOXL2', 'FOXL2NB', 'FOXL3', 'FOXM1', 'FOXN1', 'FOXN2', 'FOXN3', 'FOXN4', 'FOXO1', 'FOXO3', 'FOXO3B', 'FOXO4', 'FOXO6', 'FOXP1', 'FOXP2', 'FOXP3', 'FOXP4', 'FOXQ1', 'FOXR1', 'FOXR2', 'FOXRED1', 'FOXRED2', 'FOXS1', 'FPGS', 'FPGT', 'FPR1', 'FPR2', 'FPR3', 'FRA10AC1', 'FRAS1', 'FRAT1', 'FRAT2', 'FREM1', 'FREM2', 'FREM3', 'FRG1', 'FRG2', 'FRG2B', 'FRG2C', 'FRK', 'FRMD1', 'FRMD3', 'FRMD4A', 'FRMD4B', 'FRMD5', 'FRMD6', 'FRMD7', 'FRMD8', 'FRMPD1', 'FRMPD2', 'FRMPD3', 'FRMPD4', 'FRRS1', 'FRRS1L', 'FRS2', 'FRS3', 'FRY', 'FRYL', 'FRZB', 'FSBP', 'FSCB', 'FSCN1', 'FSCN2', 'FSCN3', 'FSD1', 'FSD1L', 'FSD2', 'FSHB', 'FSHR', 'FSIP1', 'FSIP2', 'FST', 'FSTL1', 'FSTL3', 'FSTL4', 'FSTL5', 'FTCD', 'FTCDNL1', 'FTH1', 'FTHL17', 'FTL', 'FTMT', 'FTO', 'FTSJ1', 'FTSJ3', 'FUBP1', 'FUBP3', 'FUCA1', 'FUCA2', 'FUNDC1', 'FUNDC2', 'FUOM', 'FURIN', 'FUS', 'FUT1', 'FUT2', 'FUT3', 'FUT4', 'FUT5', 'FUT6', 'FUT7', 'FUT8', 'FUT9', 'FUT10', 'FUT11', 'FUZ', 'FXN', 'FXR1', 'FXR2', 'FXYD1', 'FXYD2', 'FXYD3', 'FXYD4', 'FXYD5', 'FXYD6', 'FXYD7', 'FYB1', 'FYB2', 'FYCO1', 'FYN', 'FYTTD1', 'FZD1', 'FZD2', 'FZD3', 'FZD4', 'FZD5', 'FZD6', 'FZD7', 'FZD8', 'FZD9', 'FZD10', 'FZR1', 'G2E3', 'G3BP1', 'G3BP2', 'G6PC1', 'G6PC2', 'G6PC3', 'G6PD', 'GAA', 'GAB1', 'GAB2', 'GAB3', 'GAB4', 'GABARAP', 'GABARAPL1', 'GABARAPL2', 'GABBR1', 'GABBR2', 'GABPA', 'GABPB1', 'GABPB2', 'GABRA1', 'GABRA2', 'GABRA3', 'GABRA4', 'GABRA5', 'GABRA6', 'GABRB1', 'GABRB2', 'GABRB3', 'GABRD', 'GABRE', 'GABRG1', 'GABRG2', 'GABRG3', 'GABRP', 'GABRQ', 'GABRR1', 'GABRR2', 'GABRR3', 'GAD1', 'GAD2', 'GADD45A', 'GADD45B', 'GADD45G', 'GADD45GIP1', 'GADL1', 'GAGE1', 'GAGE2A', 'GAGE2B', 'GAGE2C', 'GAGE2D', 'GAGE2E', 'GAGE4', 'GAGE5', 'GAGE6', 'GAGE7', 'GAGE8', 'GAGE10', 'GAGE12B', 'GAGE12C', 'GAGE12D', 'GAGE12E', 'GAGE12F', 'GAGE12G', 'GAGE12H', 'GAGE12I', 'GAGE12J', 'GAGE13', 'GAK', 'GAL', 'GAL3ST1', 'GAL3ST2', 'GAL3ST3', 'GAL3ST4', 'GALC', 'GALE', 'GALK1', 'GALK2', 'GALM', 'GALNS', 'GALNT1', 'GALNT2', 'GALNT3', 'GALNT4', 'GALNT5', 'GALNT6', 'GALNT7', 'GALNT8', 'GALNT9', 'GALNT10', 'GALNT11', 'GALNT12', 'GALNT13', 'GALNT14', 'GALNT15', 'GALNT16', 'GALNT17', 'GALNT18', 'GALNTL5', 'GALNTL6', 'GALP', 'GALR1', 'GALR2', 'GALR3', 'GALT', 'GAMT', 'GAN', 'GANAB', 'GANC', 'GAP43', 'GAPDH', 'GAPDHS', 'GAPT', 'GAPVD1', 'GAR1', 'GAREM1', 'GAREM2', 'GARNL3', 'GARRE1', 'GARS1', 'GART', 'GAS1', 'GAS2', 'GAS2L1', 'GAS2L2', 'GAS2L3', 'GAS6', 'GAS7', 'GAS8', 'GASK1A', 'GASK1B', 'GAST', 'GATA1', 'GATA2', 'GATA3', 'GATA4', 'GATA5', 'GATA6', 'GATAD1', 'GATAD2A', 'GATAD2B', 'GATB', 'GATC', 'GATD1', 'GATD3A', 'GATD3B', 'GATM', 'GBA', 'GBA2', 'GBA3', 'GBE1', 'GBF1', 'GBGT1', 'GBP1', 'GBP2', 'GBP3', 'GBP4', 'GBP5', 'GBP6', 'GBP7', 'GBX1', 'GBX2', 'GC', 'GCA', 'GCAT', 'GCC1', 'GCC2', 'GCDH', 'GCFC2', 'GCG', 'GCGR', 'GCH1', 'GCHFR', 'GCK', 'GCKR', 'GCLC', 'GCLM', 'GCM1', 'GCM2', 'GCN1', 'GCNA', 'GCNT1', 'GCNT2', 'GCNT3', 'GCNT4', 'GCNT7', 'GCSAM', 'GCSAML', 'GCSH', 'GDA', 'GDAP1', 'GDAP1L1', 'GDAP2', 'GDE1', 'GDF1', 'GDF2', 'GDF3', 'GDF5', 'GDF6', 'GDF7', 'GDF9', 'GDF10', 'GDF11', 'GDF15', 'GDI1', 'GDI2', 'GDNF', 'GDPD1', 'GDPD2', 'GDPD3', 'GDPD4', 'GDPD5', 'GDPGP1', 'GEM', 'GEMIN2', 'GEMIN4', 'GEMIN5', 'GEMIN6', 'GEMIN7', 'GEMIN8', 'GEN1', 'GET1', 'GET3', 'GET4', 'GFAP', 'GFER', 'GFI1', 'GFI1B', 'GFM1', 'GFM2', 'GFOD1', 'GFOD2', 'GFPT1', 'GFPT2', 'GFRA1', 'GFRA2', 'GFRA3', 'GFRA4', 'GFRAL', 'GFUS', 'GFY', 'GGA1', 'GGA2', 'GGA3', 'GGACT', 'GGCT', 'GGCX', 'GGH', 'GGN', 'GGNBP2', 'GGPS1', 'GGT1', 'GGT2', 'GGT5', 'GGT6', 'GGT7', 'GGTA1', 'GGTLC1', 'GGTLC2', 'GGTLC3', 'GH1', 'GH2', 'GHDC', 'GHITM', 'GHR', 'GHRH', 'GHRHR', 'GHRL', 'GHSR', 'GID4', 'GID8', 'GIGYF1', 'GIGYF2', 'GIMAP1', 'GIMAP2', 'GIMAP4', 'GIMAP5', 'GIMAP6', 'GIMAP7', 'GIMAP8', 'GIMD1', 'GIN1', 'GINM1', 'GINS1', 'GINS2', 'GINS3', 'GINS4', 'GIP', 'GIPC1', 'GIPC2', 'GIPC3', 'GIPR', 'GIT1', 'GIT2', 'GJA1', 'GJA3', 'GJA4', 'GJA5', 'GJA8', 'GJA9', 'GJA10', 'GJB1', 'GJB2', 'GJB3', 'GJB4', 'GJB5', 'GJB6', 'GJB7', 'GJC1', 'GJC2', 'GJC3', 'GJD2', 'GJD3', 'GJD4', 'GJE1', 'GK', 'GK2', 'GK5', 'GKAP1', 'GKN1', 'GKN2', 'GLA', 'GLB1', 'GLB1L', 'GLB1L2', 'GLB1L3', 'GLCCI1', 'GLCE', 'GLDC', 'GLDN', 'GLE1', 'GLG1', 'GLI1', 'GLI2', 'GLI3', 'GLI4', 'GLIPR1', 'GLIPR1L1', 'GLIPR1L2', 'GLIPR2', 'GLIS1', 'GLIS2', 'GLIS3', 'GLMN', 'GLMP', 'GLO1', 'GLOD4', 'GLOD5', 'GLP1R', 'GLP2R', 'GLRA1', 'GLRA2', 'GLRA3', 'GLRB', 'GLRX', 'GLRX2', 'GLRX3', 'GLRX5', 'GLS', 'GLS2', 'GLT1D1', 'GLT6D1', 'GLT8D1', 'GLT8D2', 'GLTP', 'GLTPD2', 'GLUD1', 'GLUD2', 'GLUL', 'GLYAT', 'GLYATL1', 'GLYATL1B', 'GLYATL2', 'GLYATL3', 'GLYCTK', 'GLYR1', 'GM2A', 'GMCL1', 'GMCL2', 'GMDS', 'GMEB1', 'GMEB2', 'GMFB', 'GMFG', 'GMIP', 'GML', 'GMNC', 'GMNN', 'GMPPA', 'GMPPB', 'GMPR', 'GMPR2', 'GMPS', 'GNA11', 'GNA12', 'GNA13', 'GNA14', 'GNA15', 'GNAI1', 'GNAI2', 'GNAI3', 'GNAL', 'GNAO1', 'GNAQ', 'GNAS', 'GNAT1', 'GNAT2', 'GNAT3', 'GNAZ', 'GNB1', 'GNB1L', 'GNB2', 'GNB3', 'GNB4', 'GNB5', 'GNE', 'GNG2', 'GNG3', 'GNG4', 'GNG5', 'GNG7', 'GNG8', 'GNG10', 'GNG11', 'GNG12', 'GNG13', 'GNG14', 'GNGT1', 'GNGT2', 'GNL1', 'GNL2', 'GNL3', 'GNL3L', 'GNLY', 'GNMT', 'GNPAT', 'GNPDA1', 'GNPDA2', 'GNPNAT1', 'GNPTAB', 'GNPTG', 'GNRH1', 'GNRH2', 'GNRHR', 'GNS', 'GOLGA1', 'GOLGA2', 'GOLGA3', 'GOLGA4', 'GOLGA5', 'GOLGA6A', 'GOLGA6B', 'GOLGA6C', 'GOLGA6D', 'GOLGA6L1', 'GOLGA6L2', 'GOLGA6L4', 'GOLGA6L6', 'GOLGA6L7', 'GOLGA6L9', 'GOLGA6L10', 'GOLGA6L22', 'GOLGA7', 'GOLGA7B', 'GOLGA8A', 'GOLGA8B', 'GOLGA8F', 'GOLGA8G', 'GOLGA8H', 'GOLGA8J', 'GOLGA8K', 'GOLGA8M', 'GOLGA8N', 'GOLGA8O', 'GOLGA8Q', 'GOLGA8R', 'GOLGA8S', 'GOLGA8T', 'GOLGB1', 'GOLIM4', 'GOLM1', 'GOLM2', 'GOLPH3', 'GOLPH3L', 'GOLT1A', 'GOLT1B', 'GON4L', 'GON7', 'GOPC', 'GORAB', 'GORASP1', 'GORASP2', 'GOSR1', 'GOSR2', 'GOT1', 'GOT1L1', 'GOT2', 'GP1BA', 'GP1BB', 'GP2', 'GP5', 'GP6', 'GP9', 'GPA33', 'GPAA1', 'GPALPP1', 'GPAM', 'GPANK1', 'GPAT2', 'GPAT3', 'GPAT4', 'GPATCH1', 'GPATCH2', 'GPATCH2L', 'GPATCH3', 'GPATCH4', 'GPATCH8', 'GPATCH11', 'GPBAR1', 'GPBP1', 'GPBP1L1', 'GPC1', 'GPC2', 'GPC3', 'GPC4', 'GPC5', 'GPC6', 'GPCPD1', 'GPD1', 'GPD1L', 'GPD2', 'GPER1', 'GPHA2', 'GPHB5', 'GPHN', 'GPI', 'GPIHBP1', 'GPKOW', 'GPLD1', 'GPM6A', 'GPM6B', 'GPN1', 'GPN2', 'GPN3', 'GPNMB', 'GPR3', 'GPR4', 'GPR6', 'GPR12', 'GPR15', 'GPR17', 'GPR18', 'GPR19', 'GPR20', 'GPR21', 'GPR22', 'GPR25', 'GPR26', 'GPR27', 'GPR31', 'GPR32', 'GPR33', 'GPR34', 'GPR35', 'GPR37', 'GPR37L1', 'GPR39', 'GPR42', 'GPR45', 'GPR50', 'GPR52', 'GPR55', 'GPR61', 'GPR62', 'GPR63', 'GPR65', 'GPR68', 'GPR75', 'GPR78', 'GPR82', 'GPR83', 'GPR84', 'GPR85', 'GPR87', 'GPR88', 'GPR89A', 'GPR89B', 'GPR101', 'GPR107', 'GPR108', 'GPR119', 'GPR132', 'GPR135', 'GPR137', 'GPR137B', 'GPR137C', 'GPR139', 'GPR141', 'GPR142', 'GPR143', 'GPR146', 'GPR148', 'GPR149', 'GPR150', 'GPR151', 'GPR152', 'GPR153', 'GPR155', 'GPR156', 'GPR157', 'GPR158', 'GPR160', 'GPR161', 'GPR162', 'GPR171', 'GPR173', 'GPR174', 'GPR176', 'GPR179', 'GPR180', 'GPR182', 'GPR183', 'GPRASP1', 'GPRASP2', 'GPRC5A', 'GPRC5B', 'GPRC5C', 'GPRC5D', 'GPRC6A', 'GPRIN1', 'GPRIN2', 'GPRIN3', 'GPS1', 'GPS2', 'GPSM1', 'GPSM2', 'GPSM3', 'GPT', 'GPT2', 'GPX1', 'GPX2', 'GPX3', 'GPX4', 'GPX5', 'GPX6', 'GPX7', 'GPX8', 'GRAMD1A', 'GRAMD1B', 'GRAMD1C', 'GRAMD2A', 'GRAMD2B', 'GRAMD4', 'GRAP', 'GRAP2', 'GRAPL', 'GRB2', 'GRB7', 'GRB10', 'GRB14', 'GREB1', 'GREB1L', 'GREM1', 'GREM2', 'GREP1', 'GRHL1', 'GRHL2', 'GRHL3', 'GRHPR', 'GRIA1', 'GRIA2', 'GRIA3', 'GRIA4', 'GRID1', 'GRID2', 'GRID2IP', 'GRIFIN', 'GRIK1', 'GRIK2', 'GRIK3', 'GRIK4', 'GRIK5', 'GRIN1', 'GRIN2A', 'GRIN2B', 'GRIN2C', 'GRIN2D', 'GRIN3A', 'GRIN3B', 'GRINA', 'GRIP1', 'GRIP2', 'GRIPAP1', 'GRK1', 'GRK2', 'GRK3', 'GRK4', 'GRK5', 'GRK6', 'GRK7', 'GRM1', 'GRM2', 'GRM3', 'GRM4', 'GRM5', 'GRM6', 'GRM7', 'GRM8', 'GRN', 'GRP', 'GRPEL1', 'GRPEL2', 'GRPR', 'GRSF1', 'GRTP1', 'GRWD1', 'GRXCR1', 'GRXCR2', 'G0S2', 'GSAP', 'GSC', 'GSC2', 'GSDMA', 'GSDMB', 'GSDMC', 'GSDMD', 'GSDME', 'GSE1', 'GSG1', 'GSG1L', 'GSG1L2', 'GSK3A', 'GSK3B', 'GSKIP', 'GSN', 'GSPT1', 'GSPT2', 'GSR', 'GSS', 'GSTA1', 'GSTA2', 'GSTA3', 'GSTA4', 'GSTA5', 'GSTCD', 'GSTK1', 'GSTM1', 'GSTM2', 'GSTM3', 'GSTM4', 'GSTM5', 'GSTO1', 'GSTO2', 'GSTP1', 'GSTT2', 'GSTT2B', 'GSTT4', 'GSTZ1', 'GSX1', 'GSX2', 'GTDC1', 'GTF2A1', 'GTF2A1L', 'GTF2A2', 'GTF2B', 'GTF2E1', 'GTF2E2', 'GTF2F1', 'GTF2F2', 'GTF2H1', 'GTF2H2', 'GTF2H2C', 'GTF2H3', 'GTF2H4', 'GTF2H5', 'GTF2I', 'GTF2IRD1', 'GTF2IRD2', 'GTF2IRD2B', 'GTF3A', 'GTF3C1', 'GTF3C2', 'GTF3C3', 'GTF3C4', 'GTF3C5', 'GTF3C6', 'GTPBP1', 'GTPBP2', 'GTPBP3', 'GTPBP4', 'GTPBP6', 'GTPBP8', 'GTPBP10', 'GTSE1', 'GTSF1', 'GTSF1L', 'GUCA1A', 'GUCA1ANB', 'GUCA1B', 'GUCA1C', 'GUCA2A', 'GUCA2B', 'GUCD1', 'GUCY1A1', 'GUCY1A2', 'GUCY1B1', 'GUCY2C', 'GUCY2D', 'GUCY2F', 'GUF1', 'GUK1', 'GULP1', 'GUSB', 'GVQW3', 'GXYLT1', 'GXYLT2', 'GYG1', 'GYG2', 'GYPA', 'GYPB', 'GYPC', 'GYPE', 'GYS1', 'GYS2', 'GZF1', 'GZMA', 'GZMB', 'GZMH', 'GZMK', 'GZMM', 'H1-0', 'H1-1', 'H1-2', 'H1-3', 'H1-4', 'H1-5', 'H1-6', 'H1-7', 'H1-8', 'H1-10', 'H2AB1', 'H2AB2', 'H2AB3', 'H2AC1', 'H2AC4', 'H2AC6', 'H2AC7', 'H2AC8', 'H2AC11', 'H2AC12', 'H2AC13', 'H2AC14', 'H2AC15', 'H2AC16', 'H2AC17', 'H2AC18', 'H2AC19', 'H2AC20', 'H2AC21', 'H2AJ', 'H2AP', 'H2AW', 'H2AX', 'H2AZ1', 'H2AZ2', 'H2BC1', 'H2BC3', 'H2BC4', 'H2BC5', 'H2BC6', 'H2BC7', 'H2BC8', 'H2BC9', 'H2BC10', 'H2BC11', 'H2BC12', 'H2BC13', 'H2BC14', 'H2BC15', 'H2BC17', 'H2BC18', 'H2BC21', 'H2BE1', 'H2BS1', 'H2BU1', 'H2BW1', 'H2BW2', 'H3-2', 'H3-3A', 'H3-3B', 'H3-4', 'H3-5', 'H3C1', 'H3C2', 'H3C3', 'H3C4', 'H3C6', 'H3C7', 'H3C8', 'H3C10', 'H3C11', 'H3C12', 'H3C13', 'H3C14', 'H3C15', 'H3Y1', 'H3Y2', 'H4-16', 'H4C1', 'H4C2', 'H4C3', 'H4C4', 'H4C5', 'H4C6', 'H4C7', 'H4C8', 'H4C9', 'H4C11', 'H4C12', 'H4C13', 'H4C14', 'H4C15', 'H6PD', 'HAAO', 'HABP2', 'HABP4', 'HACD1', 'HACD2', 'HACD3', 'HACD4', 'HACE1', 'HACL1', 'HADH', 'HADHA', 'HADHB', 'HAGH', 'HAGHL', 'HAL', 'HAMP', 'HAND1', 'HAND2', 'HAO1', 'HAO2', 'HAP1', 'HAPLN1', 'HAPLN2', 'HAPLN3', 'HAPLN4', 'HARBI1', 'HARS1', 'HARS2', 'HAS1', 'HAS2', 'HAS3', 'HASPIN', 'HAT1', 'HAUS1', 'HAUS2', 'HAUS3', 'HAUS4', 'HAUS5', 'HAUS6', 'HAUS7', 'HAUS8', 'HAVCR1', 'HAVCR2', 'HAX1', 'HBA1', 'HBA2', 'HBB', 'HBD', 'HBE1', 'HBEGF', 'HBG1', 'HBG2', 'HBM', 'HBP1', 'HBQ1', 'HBS1L', 'HBZ', 'HCAR1', 'HCAR2', 'HCAR3', 'HCCS', 'HCFC1', 'HCFC1R1', 'HCFC2', 'HCG22', 'HCK', 'HCLS1', 'HCN1', 'HCN2', 'HCN3', 'HCN4', 'HCRT', 'HCRTR1', 'HCRTR2', 'HCST', 'HDAC1', 'HDAC2', 'HDAC3', 'HDAC4', 'HDAC5', 'HDAC6', 'HDAC7', 'HDAC8', 'HDAC9', 'HDAC10', 'HDAC11', 'HDC', 'HDDC2', 'HDDC3', 'HDGF', 'HDGFL1', 'HDGFL2', 'HDGFL3', 'HDHD2', 'HDHD3', 'HDHD5', 'HDLBP', 'HDX', 'HEATR1', 'HEATR3', 'HEATR4', 'HEATR5A', 'HEATR5B', 'HEATR6', 'HEATR9', 'HEBP1', 'HEBP2', 'HECA', 'HECTD1', 'HECTD2', 'HECTD3', 'HECTD4', 'HECW1', 'HECW2', 'HEG1', 'HELB', 'HELLS', 'HELQ', 'HELT', 'HELZ', 'HELZ2', 'HEMGN', 'HEMK1', 'HENMT1', 'HEPACAM', 'HEPACAM2', 'HEPH', 'HEPHL1', 'HEPN1', 'HERC1', 'HERC2', 'HERC3', 'HERC4', 'HERC5', 'HERC6', 'HERPUD1', 'HERPUD2', 'HES1', 'HES2', 'HES3', 'HES4', 'HES5', 'HES6', 'HES7', 'HESX1', 'HEXA', 'HEXB', 'HEXD', 'HEXIM1', 'HEXIM2', 'HEY1', 'HEY2', 'HEYL', 'HFE', 'HFM1', 'HGD', 'HGF', 'HGFAC', 'HGH1', 'HGS', 'HGSNAT', 'HHAT', 'HHATL', 'HHEX', 'HHIP', 'HHIPL1', 'HHIPL2', 'HHLA1', 'HHLA2', 'HIBADH', 'HIBCH', 'HIC1', 'HIC2', 'HID1', 'HIF1A', 'HIF1AN', 'HIF3A', 'HIGD1A', 'HIGD1B', 'HIGD1C', 'HIGD2A', 'HIGD2B', 'HIKESHI', 'HILPDA', 'HINFP', 'HINT1', 'HINT2', 'HINT3', 'HIP1', 'HIP1R', 'HIPK1', 'HIPK2', 'HIPK3', 'HIPK4', 'HIRA', 'HIRIP3', 'HIVEP1', 'HIVEP2', 'HIVEP3', 'HJURP', 'HJV', 'HK1', 'HK2', 'HK3', 'HKDC1', 'HLA-A', 'HLA-B', 'HLA-C', 'HLA-DMA', 'HLA-DMB', 'HLA-DOA', 'HLA-DOB', 'HLA-DPA1', 'HLA-DPB1', 'HLA-DQA1', 'HLA-DQA2', 'HLA-DQB1', 'HLA-DQB2', 'HLA-DRA', 'HLA-DRB1', 'HLA-DRB5', 'HLA-E', 'HLA-F', 'HLA-G', 'HLCS', 'HLF', 'HLTF', 'HLX', 'HM13', 'HMBOX1', 'HMBS', 'HMCES', 'HMCN1', 'HMCN2', 'HMG20A', 'HMG20B', 'HMGA1', 'HMGA2', 'HMGB1', 'HMGB2', 'HMGB3', 'HMGB4', 'HMGCL', 'HMGCLL1', 'HMGCR', 'HMGCS1', 'HMGCS2', 'HMGN1', 'HMGN2', 'HMGN3', 'HMGN4', 'HMGN5', 'HMGXB3', 'HMGXB4', 'HMHB1', 'HMMR', 'HMOX1', 'HMOX2', 'HMSD', 'HMX1', 'HMX2', 'HMX3', 'HNF1A', 'HNF1B', 'HNF4A', 'HNF4G', 'HNMT', 'HNRNPA0', 'HNRNPA1', 'HNRNPA1L2', 'HNRNPA2B1', 'HNRNPA3', 'HNRNPAB', 'HNRNPC', 'HNRNPCL1', 'HNRNPCL2', 'HNRNPCL3', 'HNRNPCL4', 'HNRNPD', 'HNRNPDL', 'HNRNPF', 'HNRNPH1', 'HNRNPH2', 'HNRNPH3', 'HNRNPK', 'HNRNPL', 'HNRNPLL', 'HNRNPM', 'HNRNPR', 'HNRNPU', 'HNRNPUL1', 'HNRNPUL2', 'HOATZ', 'HOGA1', 'HOMER1', 'HOMER2', 'HOMER3', 'HOMEZ', 'HOOK1', 'HOOK2', 'HOOK3', 'HOPX', 'HORMAD1', 'HORMAD2', 'HOXA1', 'HOXA2', 'HOXA3', 'HOXA4', 'HOXA5', 'HOXA6', 'HOXA7', 'HOXA9', 'HOXA10', 'HOXA11', 'HOXA13', 'HOXB1', 'HOXB2', 'HOXB3', 'HOXB4', 'HOXB5', 'HOXB6', 'HOXB7', 'HOXB8', 'HOXB9', 'HOXB13', 'HOXC4', 'HOXC5', 'HOXC6', 'HOXC8', 'HOXC9', 'HOXC10', 'HOXC11', 'HOXC12', 'HOXC13', 'HOXD1', 'HOXD3', 'HOXD4', 'HOXD8', 'HOXD9', 'HOXD10', 'HOXD11', 'HOXD12', 'HOXD13', 'HP', 'HP1BP3', 'HPCA', 'HPCAL1', 'HPCAL4', 'HPD', 'HPDL', 'HPF1', 'HPGD', 'HPGDS', 'HPN', 'HPR', 'HPRT1', 'HPS1', 'HPS3', 'HPS4', 'HPS5', 'HPS6', 'HPSE', 'HPSE2', 'HPX', 'HR', 'HRAS', 'HRC', 'HRCT1', 'HRG', 'HRH1', 'HRH2', 'HRH3', 'HRH4', 'HRK', 'HRNR', 'HROB', 'HRURF', 'HS1BP3', 'HS2ST1', 'HS3ST1', 'HS3ST2', 'HS3ST3A1', 'HS3ST3B1', 'HS3ST4', 'HS3ST5', 'HS3ST6', 'HS6ST1', 'HS6ST2', 'HS6ST3', 'HSBP1', 'HSBP1L1', 'HSCB', 'HSD3B1', 'HSD3B2', 'HSD3B7', 'HSD11B1', 'HSD11B1L', 'HSD11B2', 'HSD17B1', 'HSD17B2', 'HSD17B3', 'HSD17B4', 'HSD17B6', 'HSD17B7', 'HSD17B8', 'HSD17B10', 'HSD17B11', 'HSD17B12', 'HSD17B13', 'HSD17B14', 'HSDL1', 'HSDL2', 'HSF1', 'HSF2', 'HSF2BP', 'HSF4', 'HSF5', 'HSFX1', 'HSFX2', 'HSFX3', 'HSFX4', 'HSFY1', 'HSFY2', 'HSH2D', 'HSP90AA1', 'HSP90AB1', 'HSP90B1', 'HSPA1A', 'HSPA1B', 'HSPA1L', 'HSPA2', 'HSPA4', 'HSPA4L', 'HSPA5', 'HSPA6', 'HSPA8', 'HSPA9', 'HSPA12A', 'HSPA12B', 'HSPA13', 'HSPA14', 'HSPB1', 'HSPB2', 'HSPB3', 'HSPB6', 'HSPB7', 'HSPB8', 'HSPB9', 'HSPB11', 'HSPBAP1', 'HSPBP1', 'HSPD1', 'HSPE1', 'HSPG2', 'HSPH1', 'HTATIP2', 'HTATSF1', 'HTD2', 'HTN1', 'HTN3', 'HTR1A', 'HTR1B', 'HTR1D', 'HTR1E', 'HTR1F', 'HTR2A', 'HTR2B', 'HTR2C', 'HTR3A', 'HTR3B', 'HTR3C', 'HTR3D', 'HTR3E', 'HTR4', 'HTR5A', 'HTR6', 'HTR7', 'HTRA1', 'HTRA2', 'HTRA3', 'HTRA4', 'HTT', 'HUNK', 'HUS1', 'HUS1B', 'HUWE1', 'HVCN1', 'HYAL1', 'HYAL2', 'HYAL3', 'HYAL4', 'HYDIN', 'HYI', 'HYKK', 'HYLS1', 'HYOU1', 'HYPK', 'IAH1', 'IAPP', 'IARS1', 'IARS2', 'IBA57', 'IBSP', 'IBTK', 'ICA1', 'ICA1L', 'ICAM1', 'ICAM2', 'ICAM3', 'ICAM4', 'ICAM5', 'ICE1', 'ICE2', 'ICMT', 'ICOS', 'ICOSLG', 'ID1', 'ID2', 'ID3', 'ID4', 'IDE', 'IDH1', 'IDH2', 'IDH3A', 'IDH3B', 'IDH3G', 'IDI1', 'IDI2', 'IDNK', 'IDO1', 'IDO2', 'IDS', 'IDUA', 'IER2', 'IER3', 'IER3IP1', 'IER5', 'IER5L', 'IFFO1', 'IFFO2', 'IFI6', 'IFI16', 'IFI27', 'IFI27L1', 'IFI27L2', 'IFI30', 'IFI35', 'IFI44', 'IFI44L', 'IFIH1', 'IFIT1', 'IFIT1B', 'IFIT2', 'IFIT3', 'IFIT5', 'IFITM1', 'IFITM2', 'IFITM3', 'IFITM5', 'IFITM10', 'IFNA1', 'IFNA2', 'IFNA4', 'IFNA5', 'IFNA6', 'IFNA7', 'IFNA8', 'IFNA10', 'IFNA13', 'IFNA14', 'IFNA16', 'IFNA17', 'IFNA21', 'IFNAR1', 'IFNAR2', 'IFNB1', 'IFNE', 'IFNG', 'IFNGR1', 'IFNGR2', 'IFNK', 'IFNL1', 'IFNL2', 'IFNL3', 'IFNL4', 'IFNLR1', 'IFNW1', 'IFRD1', 'IFRD2', 'IFT20', 'IFT22', 'IFT27', 'IFT43', 'IFT46', 'IFT52', 'IFT57', 'IFT74', 'IFT80', 'IFT81', 'IFT88', 'IFT122', 'IFT140', 'IFT172', 'IFTAP', 'IGBP1', 'IGDCC3', 'IGDCC4', 'IGF1', 'IGF1R', 'IGF2', 'IGF2BP1', 'IGF2BP2', 'IGF2BP3', 'IGF2R', 'IGFALS', 'IGFBP1', 'IGFBP2', 'IGFBP3', 'IGFBP4', 'IGFBP5', 'IGFBP6', 'IGFBP7', 'IGFBPL1', 'IGFL1', 'IGFL2', 'IGFL3', 'IGFL4', 'IGFLR1', 'IGFN1', 'IGH', 'IGHMBP2', 'IGIP', 'IGK', 'IGL', 'IGLL1', 'IGLL5', 'IGLON5', 'IGSF1', 'IGSF3', 'IGSF5', 'IGSF6', 'IGSF8', 'IGSF9', 'IGSF9B', 'IGSF10', 'IGSF11', 'IGSF21', 'IGSF22', 'IGSF23', 'IHH', 'IHO1', 'IK', 'IKBIP', 'IKBKB', 'IKBKE', 'IKBKG', 'IKZF1', 'IKZF2', 'IKZF3', 'IKZF4', 'IKZF5', 'IL1A', 'IL1B', 'IL1F10', 'IL1R1', 'IL1R2', 'IL1RAP', 'IL1RAPL1', 'IL1RAPL2', 'IL1RL1', 'IL1RL2', 'IL1RN', 'IL2', 'IL2RA', 'IL2RB', 'IL2RG', 'IL3', 'IL3RA', 'IL4', 'IL4I1', 'IL4R', 'IL5', 'IL5RA', 'IL6', 'IL6R', 'IL6ST', 'IL7', 'IL7R', 'IL9', 'IL9R', 'IL10', 'IL10RA', 'IL10RB', 'IL11', 'IL11RA', 'IL12A', 'IL12B', 'IL12RB1', 'IL12RB2', 'IL13', 'IL13RA1', 'IL13RA2', 'IL15', 'IL15RA', 'IL16', 'IL17A', 'IL17B', 'IL17C', 'IL17D', 'IL17F', 'IL17RA', 'IL17RB', 'IL17RC', 'IL17RD', 'IL17RE', 'IL17REL', 'IL18', 'IL18BP', 'IL18R1', 'IL18RAP', 'IL19', 'IL20', 'IL20RA', 'IL20RB', 'IL21', 'IL21R', 'IL22', 'IL22RA1', 'IL22RA2', 'IL23A', 'IL23R', 'IL24', 'IL25', 'IL26', 'IL27', 'IL27RA', 'IL31', 'IL31RA', 'IL32', 'IL33', 'IL34', 'IL36A', 'IL36B', 'IL36G', 'IL36RN', 'IL37', 'ILDR1', 'ILDR2', 'ILF2', 'ILF3', 'ILK', 'ILKAP', 'ILRUN', 'ILVBL', 'IMMP1L', 'IMMP2L', 'IMMT', 'IMP3', 'IMP4', 'IMPA1', 'IMPA2', 'IMPACT', 'IMPDH1', 'IMPDH2', 'IMPG1', 'IMPG2', 'INA', 'INAFM1', 'INAFM2', 'INAVA', 'INCA1', 'INCENP', 'INF2', 'ING1', 'ING2', 'ING3', 'ING4', 'ING5', 'INHA', 'INHBA', 'INHBB', 'INHBC', 'INHBE', 'INIP', 'INKA1', 'INKA2', 'INMT', 'INO80', 'INO80B', 'INO80C', 'INO80D', 'INO80E', 'INPP1', 'INPP4A', 'INPP4B', 'INPP5A', 'INPP5B', 'INPP5D', 'INPP5E', 'INPP5F', 'INPP5J', 'INPP5K', 'INPPL1', 'INS', 'INSC', 'INSIG1', 'INSIG2', 'INSL3', 'INSL4', 'INSL5', 'INSL6', 'INSM1', 'INSM2', 'INSR', 'INSRR', 'INSYN1', 'INSYN2A', 'INSYN2B', 'INTS1', 'INTS2', 'INTS3', 'INTS4', 'INTS5', 'INTS6', 'INTS6L', 'INTS7', 'INTS8', 'INTS9', 'INTS10', 'INTS11', 'INTS12', 'INTS13', 'INTS14', 'INTU', 'INVS', 'IP6K1', 'IP6K2', 'IP6K3', 'IPCEF1', 'IPMK', 'IPO4', 'IPO5', 'IPO7', 'IPO8', 'IPO9', 'IPO11', 'IPO13', 'IPP', 'IPPK', 'IQANK1', 'IQCA1', 'IQCA1L', 'IQCB1', 'IQCC', 'IQCD', 'IQCE', 'IQCF1', 'IQCF2', 'IQCF3', 'IQCF5', 'IQCF6', 'IQCG', 'IQCH', 'IQCJ', 'IQCK', 'IQCM', 'IQCN', 'IQGAP1', 'IQGAP2', 'IQGAP3', 'IQSEC1', 'IQSEC2', 'IQSEC3', 'IQUB', 'IRAG1', 'IRAG2', 'IRAK1', 'IRAK1BP1', 'IRAK2', 'IRAK3', 'IRAK4', 'IREB2', 'IRF1', 'IRF2', 'IRF2BP1', 'IRF2BP2', 'IRF2BPL', 'IRF3', 'IRF4', 'IRF5', 'IRF6', 'IRF7', 'IRF8', 'IRF9', 'IRGC', 'IRGM', 'IRGQ', 'IRS1', 'IRS2', 'IRS4', 'IRX1', 'IRX2', 'IRX3', 'IRX4', 'IRX5', 'IRX6', 'ISCA1', 'ISCA2', 'ISCU', 'ISG15', 'ISG20', 'ISG20L2', 'ISL1', 'ISL2', 'ISLR', 'ISLR2', 'ISM1', 'ISM2', 'ISOC1', 'ISOC2', 'IST1', 'ISX', 'ISY1', 'ISYNA1', 'ITCH', 'ITFG1', 'ITFG2', 'ITGA1', 'ITGA2', 'ITGA2B', 'ITGA3', 'ITGA4', 'ITGA5', 'ITGA6', 'ITGA7', 'ITGA8', 'ITGA9', 'ITGA10', 'ITGA11', 'ITGAD', 'ITGAE', 'ITGAL', 'ITGAM', 'ITGAV', 'ITGAX', 'ITGB1', 'ITGB1BP1', 'ITGB1BP2', 'ITGB2', 'ITGB3', 'ITGB3BP', 'ITGB4', 'ITGB5', 'ITGB6', 'ITGB7', 'ITGB8', 'ITGBL1', 'ITIH1', 'ITIH2', 'ITIH3', 'ITIH4', 'ITIH5', 'ITIH6', 'ITK', 'ITLN1', 'ITLN2', 'ITM2A', 'ITM2B', 'ITM2C', 'ITPA', 'ITPK1', 'ITPKA', 'ITPKB', 'ITPKC', 'ITPR1', 'ITPR2', 'ITPR3', 'ITPRID1', 'ITPRID2', 'ITPRIP', 'ITPRIPL1', 'ITPRIPL2', 'ITSN1', 'ITSN2', 'IVD', 'IVL', 'IVNS1ABP', 'IWS1', 'IYD', 'IZUMO1', 'IZUMO1R', 'IZUMO2', 'IZUMO3', 'IZUMO4', 'JADE1', 'JADE2', 'JADE3', 'JAG1', 'JAG2', 'JAGN1', 'JAK1', 'JAK2', 'JAK3', 'JAKMIP1', 'JAKMIP2', 'JAKMIP3', 'JAM2', 'JAM3', 'JAML', 'JARID2', 'JAZF1', 'JCAD', 'JCHAIN', 'JDP2', 'JHY', 'JKAMP', 'JMJD1C', 'JMJD4', 'JMJD6', 'JMJD7', 'JMJD8', 'JMY', 'JOSD1', 'JOSD2', 'JPH1', 'JPH2', 'JPH3', 'JPH4', 'JPT1', 'JPT2', 'JRK', 'JRKL', 'JSRP1', 'JTB', 'JUN', 'JUNB', 'JUND', 'JUP', 'KAAG1', 'KALRN', 'KANK1', 'KANK2', 'KANK3', 'KANK4', 'KANSL1', 'KANSL1L', 'KANSL2', 'KANSL3', 'KARS1', 'KASH5', 'KAT2A', 'KAT2B', 'KAT5', 'KAT6A', 'KAT6B', 'KAT7', 'KAT8', 'KAT14', 'KATNA1', 'KATNAL1', 'KATNAL2', 'KATNB1', 'KATNBL1', 'KATNIP', 'KAZALD1', 'KAZN', 'KBTBD2', 'KBTBD3', 'KBTBD4', 'KBTBD6', 'KBTBD7', 'KBTBD8', 'KBTBD11', 'KBTBD12', 'KBTBD13', 'KCMF1', 'KCNA1', 'KCNA2', 'KCNA3', 'KCNA4', 'KCNA5', 'KCNA6', 'KCNA7', 'KCNA10', 'KCNAB1', 'KCNAB2', 'KCNAB3', 'KCNB1', 'KCNB2', 'KCNC1', 'KCNC2', 'KCNC3', 'KCNC4', 'KCND1', 'KCND2', 'KCND3', 'KCNE1', 'KCNE1B', 'KCNE2', 'KCNE3', 'KCNE4', 'KCNE5', 'KCNF1', 'KCNG1', 'KCNG2', 'KCNG3', 'KCNG4', 'KCNH1', 'KCNH2', 'KCNH3', 'KCNH4', 'KCNH5', 'KCNH6', 'KCNH7', 'KCNH8', 'KCNIP1', 'KCNIP2', 'KCNIP3', 'KCNIP4', 'KCNJ1', 'KCNJ2', 'KCNJ3', 'KCNJ4', 'KCNJ5', 'KCNJ6', 'KCNJ8', 'KCNJ9', 'KCNJ10', 'KCNJ11', 'KCNJ12', 'KCNJ13', 'KCNJ14', 'KCNJ15', 'KCNJ16', 'KCNJ18', 'KCNK1', 'KCNK2', 'KCNK3', 'KCNK4', 'KCNK5', 'KCNK6', 'KCNK7', 'KCNK9', 'KCNK10', 'KCNK12', 'KCNK13', 'KCNK15', 'KCNK16', 'KCNK17', 'KCNK18', 'KCNMA1', 'KCNMB1', 'KCNMB2', 'KCNMB3', 'KCNMB4', 'KCNN1', 'KCNN2', 'KCNN3', 'KCNN4', 'KCNQ1', 'KCNQ2', 'KCNQ3', 'KCNQ4', 'KCNQ5', 'KCNRG', 'KCNS1', 'KCNS2', 'KCNS3', 'KCNT1', 'KCNT2', 'KCNU1', 'KCNV1', 'KCNV2', 'KCP', 'KCTD1', 'KCTD2', 'KCTD3', 'KCTD4', 'KCTD5', 'KCTD6', 'KCTD7', 'KCTD8', 'KCTD9', 'KCTD10', 'KCTD11', 'KCTD12', 'KCTD13', 'KCTD14', 'KCTD15', 'KCTD16', 'KCTD17', 'KCTD18', 'KCTD19', 'KCTD20', 'KCTD21', 'KDELR1', 'KDELR2', 'KDELR3', 'KDF1', 'KDM1A', 'KDM1B', 'KDM2A', 'KDM2B', 'KDM3A', 'KDM3B', 'KDM4A', 'KDM4B', 'KDM4C', 'KDM4D', 'KDM4E', 'KDM5A', 'KDM5B', 'KDM5C', 'KDM5D', 'KDM6A', 'KDM6B', 'KDM7A', 'KDM8', 'KDR', 'KDSR', 'KEAP1', 'KEL', 'KERA', 'KHDC1', 'KHDC1L', 'KHDC3L', 'KHDC4', 'KHDRBS1', 'KHDRBS2', 'KHDRBS3', 'KHK', 'KHNYN', 'KHSRP', 'KIAA0040', 'KIAA0100', 'KIAA0232', 'KIAA0319', 'KIAA0319L', 'KIAA0408', 'KIAA0513', 'KIAA0586', 'KIAA0753', 'KIAA0754', 'KIAA0825', 'KIAA0895', 'KIAA0895L', 'KIAA0930', 'KIAA1107', 'KIAA1109', 'KIAA1143', 'KIAA1191', 'KIAA1210', 'KIAA1217', 'KIAA1328', 'KIAA1522', 'KIAA1549', 'KIAA1549L', 'KIAA1586', 'KIAA1614', 'KIAA1671', 'KIAA1755', 'KIAA1958', 'KIAA2012', 'KIAA2013', 'KIAA2026', 'KICS2', 'KIDINS220', 'KIF1A', 'KIF1B', 'KIF1C', 'KIF2A', 'KIF2B', 'KIF2C', 'KIF3A', 'KIF3B', 'KIF3C', 'KIF4A', 'KIF4B', 'KIF5A', 'KIF5B', 'KIF5C', 'KIF6', 'KIF7', 'KIF9', 'KIF11', 'KIF12', 'KIF13A', 'KIF13B', 'KIF14', 'KIF15', 'KIF16B', 'KIF17', 'KIF18A', 'KIF18B', 'KIF19', 'KIF20A', 'KIF20B', 'KIF21A', 'KIF21B', 'KIF22', 'KIF23', 'KIF24', 'KIF25', 'KIF26A', 'KIF26B', 'KIF27', 'KIFAP3', 'KIFBP', 'KIFC1', 'KIFC2', 'KIFC3', 'KIN', 'KIR2DL1', 'KIR2DL3', 'KIR2DL4', 'KIR2DS4', 'KIR3DL1', 'KIR3DL2', 'KIR3DL3', 'KIRREL1', 'KIRREL2', 'KIRREL3', 'KISS1', 'KISS1R', 'KIT', 'KITLG', 'KIZ', 'KL', 'KLB', 'KLC1', 'KLC2', 'KLC3', 'KLC4', 'KLF1', 'KLF2', 'KLF3', 'KLF4', 'KLF5', 'KLF6', 'KLF7', 'KLF8', 'KLF9', 'KLF10', 'KLF11', 'KLF12', 'KLF13', 'KLF14', 'KLF15', 'KLF16', 'KLF17', 'KLF18', 'KLHDC1', 'KLHDC2', 'KLHDC3', 'KLHDC4', 'KLHDC7A', 'KLHDC7B', 'KLHDC8A', 'KLHDC8B', 'KLHDC9', 'KLHDC10', 'KLHL1', 'KLHL2', 'KLHL3', 'KLHL4', 'KLHL5', 'KLHL6', 'KLHL7', 'KLHL8', 'KLHL9', 'KLHL10', 'KLHL11', 'KLHL12', 'KLHL13', 'KLHL14', 'KLHL15', 'KLHL17', 'KLHL18', 'KLHL20', 'KLHL21', 'KLHL22', 'KLHL23', 'KLHL24', 'KLHL25', 'KLHL26', 'KLHL28', 'KLHL29', 'KLHL30', 'KLHL31', 'KLHL32', 'KLHL33', 'KLHL34', 'KLHL35', 'KLHL36', 'KLHL38', 'KLHL40', 'KLHL41', 'KLHL42', 'KLK1', 'KLK2', 'KLK3', 'KLK4', 'KLK5', 'KLK6', 'KLK7', 'KLK8', 'KLK9', 'KLK10', 'KLK11', 'KLK12', 'KLK13', 'KLK14', 'KLK15', 'KLKB1', 'KLLN', 'KLRB1', 'KLRC1', 'KLRC2', 'KLRC3', 'KLRC4', 'KLRD1', 'KLRF1', 'KLRF2', 'KLRG1', 'KLRG2', 'KLRK1', 'KMO', 'KMT2A', 'KMT2B', 'KMT2C', 'KMT2D', 'KMT2E', 'KMT5A', 'KMT5B', 'KMT5C', 'KNCN', 'KNDC1', 'KNG1', 'KNL1', 'KNOP1', 'KNSTRN', 'KNTC1', 'KPNA1', 'KPNA2', 'KPNA3', 'KPNA4', 'KPNA5', 'KPNA6', 'KPNA7', 'KPNB1', 'KPRP', 'KPTN', 'KRAS', 'KRBA1', 'KRBA2', 'KRBOX1', 'KRBOX4', 'KRCC1', 'KREMEN1', 'KREMEN2', 'KRI1', 'KRIT1', 'KRR1', 'KRT1', 'KRT2', 'KRT3', 'KRT4', 'KRT5', 'KRT6A', 'KRT6B', 'KRT6C', 'KRT7', 'KRT8', 'KRT9', 'KRT10', 'KRT12', 'KRT13', 'KRT14', 'KRT15', 'KRT16', 'KRT17', 'KRT18', 'KRT19', 'KRT20', 'KRT23', 'KRT24', 'KRT25', 'KRT26', 'KRT27', 'KRT28', 'KRT31', 'KRT32', 'KRT33A', 'KRT33B', 'KRT34', 'KRT35', 'KRT36', 'KRT37', 'KRT38', 'KRT39', 'KRT40', 'KRT71', 'KRT72', 'KRT73', 'KRT74', 'KRT75', 'KRT76', 'KRT77', 'KRT78', 'KRT79', 'KRT80', 'KRT81', 'KRT82', 'KRT83', 'KRT84', 'KRT85', 'KRT86', 'KRT222', 'KRTAP1-1', 'KRTAP1-3', 'KRTAP1-4', 'KRTAP1-5', 'KRTAP2-1', 'KRTAP2-2', 'KRTAP2-3', 'KRTAP2-4', 'KRTAP3-1', 'KRTAP3-2', 'KRTAP3-3', 'KRTAP4-1', 'KRTAP4-2', 'KRTAP4-3', 'KRTAP4-4', 'KRTAP4-5', 'KRTAP4-6', 'KRTAP4-7', 'KRTAP4-8', 'KRTAP4-9', 'KRTAP4-11', 'KRTAP4-12', 'KRTAP5-1', 'KRTAP5-2', 'KRTAP5-3', 'KRTAP5-4', 'KRTAP5-5', 'KRTAP5-6', 'KRTAP5-7', 'KRTAP5-8', 'KRTAP5-9', 'KRTAP5-10', 'KRTAP5-11', 'KRTAP6-1', 'KRTAP6-2', 'KRTAP6-3', 'KRTAP7-1', 'KRTAP8-1', 'KRTAP9-1', 'KRTAP9-2', 'KRTAP9-3', 'KRTAP9-4', 'KRTAP9-6', 'KRTAP9-7', 'KRTAP9-8', 'KRTAP9-9', 'KRTAP10-1', 'KRTAP10-2', 'KRTAP10-3', 'KRTAP10-4', 'KRTAP10-5', 'KRTAP10-6', 'KRTAP10-7', 'KRTAP10-8', 'KRTAP10-9', 'KRTAP10-10', 'KRTAP10-11', 'KRTAP10-12', 'KRTAP11-1', 'KRTAP12-1', 'KRTAP12-2', 'KRTAP12-3', 'KRTAP12-4', 'KRTAP13-1', 'KRTAP13-2', 'KRTAP13-3', 'KRTAP13-4', 'KRTAP15-1', 'KRTAP16-1', 'KRTAP17-1', 'KRTAP19-1', 'KRTAP19-2', 'KRTAP19-3', 'KRTAP19-4', 'KRTAP19-5', 'KRTAP19-6', 'KRTAP19-7', 'KRTAP19-8', 'KRTAP20-1', 'KRTAP20-2', 'KRTAP20-3', 'KRTAP20-4', 'KRTAP21-1', 'KRTAP21-2', 'KRTAP21-3', 'KRTAP22-1', 'KRTAP22-2', 'KRTAP23-1', 'KRTAP24-1', 'KRTAP25-1', 'KRTAP26-1', 'KRTAP27-1', 'KRTAP29-1', 'KRTCAP2', 'KRTCAP3', 'KRTDAP', 'KSR1', 'KSR2', 'KTI12', 'KTN1', 'KXD1', 'KY', 'KYAT1', 'KYAT3', 'KYNU', 'L1CAM', 'L1TD1', 'L2HGDH', 'L3HYPDH', 'L3MBTL1', 'L3MBTL2', 'L3MBTL3', 'L3MBTL4', 'LACC1', 'LACRT', 'LACTB', 'LACTB2', 'LACTBL1', 'LAD1', 'LAG3', 'LAGE3', 'LAIR1', 'LAIR2', 'LALBA', 'LAMA1', 'LAMA2', 'LAMA3', 'LAMA4', 'LAMA5', 'LAMB1', 'LAMB2', 'LAMB3', 'LAMB4', 'LAMC1', 'LAMC2', 'LAMC3', 'LAMP1', 'LAMP2', 'LAMP3', 'LAMP5', 'LAMTOR1', 'LAMTOR2', 'LAMTOR3', 'LAMTOR4', 'LAMTOR5', 'LANCL1', 'LANCL2', 'LANCL3', 'LAP3', 'LAPTM4A', 'LAPTM4B', 'LAPTM5', 'LARGE1', 'LARGE2', 'LARP1', 'LARP1B', 'LARP4', 'LARP4B', 'LARP6', 'LARP7', 'LARS1', 'LARS2', 'LAS1L', 'LASP1', 'LAT', 'LAT2', 'LATS1', 'LATS2', 'LAX1', 'LAYN', 'LBH', 'LBHD1', 'LBHD2', 'LBP', 'LBR', 'LBX1', 'LBX2', 'LCA5', 'LCA5L', 'LCAT', 'LCE1A', 'LCE1B', 'LCE1C', 'LCE1D', 'LCE1E', 'LCE1F', 'LCE2A', 'LCE2B', 'LCE2C', 'LCE2D', 'LCE3A', 'LCE3B', 'LCE3C', 'LCE3D', 'LCE3E', 'LCE4A', 'LCE5A', 'LCE6A', 'LCK', 'LCLAT1', 'LCMT1', 'LCMT2', 'LCN1', 'LCN2', 'LCN6', 'LCN8', 'LCN9', 'LCN10', 'LCN12', 'LCN15', 'LCNL1', 'LCOR', 'LCORL', 'LCP1', 'LCP2', 'LCT', 'LCTL', 'LDAF1', 'LDAH', 'LDB1', 'LDB2', 'LDB3', 'LDHA', 'LDHAL6A', 'LDHAL6B', 'LDHB', 'LDHC', 'LDHD', 'LDLR', 'LDLRAD1', 'LDLRAD2', 'LDLRAD3', 'LDLRAD4', 'LDLRAP1', 'LDOC1', 'LEAP2', 'LECT2', 'LEF1', 'LEFTY1', 'LEFTY2', 'LEKR1', 'LELP1', 'LEMD1', 'LEMD2', 'LEMD3', 'LENEP', 'LENG1', 'LENG8', 'LENG9', 'LEO1', 'LEP', 'LEPR', 'LEPROT', 'LEPROTL1', 'LETM1', 'LETM2', 'LETMD1', 'LEUTX', 'LEXM', 'LFNG', 'LGALS1', 'LGALS2', 'LGALS3', 'LGALS3BP', 'LGALS4', 'LGALS7', 'LGALS7B', 'LGALS8', 'LGALS9', 'LGALS9B', 'LGALS9C', 'LGALS12', 'LGALS13', 'LGALS14', 'LGALS16', 'LGALSL', 'LGI1', 'LGI2', 'LGI3', 'LGI4', 'LGMN', 'LGR4', 'LGR5', 'LGR6', 'LGSN', 'LHB', 'LHCGR', 'LHFPL1', 'LHFPL2', 'LHFPL3', 'LHFPL4', 'LHFPL5', 'LHFPL6', 'LHPP', 'LHX1', 'LHX2', 'LHX3', 'LHX4', 'LHX5', 'LHX6', 'LHX8', 'LHX9', 'LIAS', 'LIF', 'LIFR', 'LIG1', 'LIG3', 'LIG4', 'LILRA1', 'LILRA2', 'LILRA4', 'LILRA5', 'LILRA6', 'LILRB1', 'LILRB2', 'LILRB3', 'LILRB4', 'LILRB5', 'LIM2', 'LIMA1', 'LIMCH1', 'LIMD1', 'LIMD2', 'LIME1', 'LIMK1', 'LIMK2', 'LIMS1', 'LIMS2', 'LIMS3', 'LIMS4', 'LIN7A', 'LIN7B', 'LIN7C', 'LIN9', 'LIN28A', 'LIN28B', 'LIN37', 'LIN52', 'LIN54', 'LINGO1', 'LINGO2', 'LINGO3', 'LINGO4', 'LINS1', 'LIPA', 'LIPC', 'LIPE', 'LIPF', 'LIPG', 'LIPH', 'LIPI', 'LIPJ', 'LIPK', 'LIPM', 'LIPN', 'LIPT1', 'LIPT2', 'LITAF', 'LITAFD', 'LIX1', 'LIX1L', 'LKAAEAR1', 'LLCFC1', 'LLGL1', 'LLGL2', 'LLPH', 'LMAN1', 'LMAN1L', 'LMAN2', 'LMAN2L', 'LMBR1', 'LMBR1L', 'LMBRD1', 'LMBRD2', 'LMCD1', 'LMF1', 'LMF2', 'LMLN', 'LMLN2', 'LMNA', 'LMNB1', 'LMNB2', 'LMNTD1', 'LMNTD2', 'LMO1', 'LMO2', 'LMO3', 'LMO4', 'LMO7', 'LMOD1', 'LMOD2', 'LMOD3', 'LMTK2', 'LMTK3', 'LMX1A', 'LMX1B', 'LNP1', 'LNPEP', 'LNPK', 'LNX1', 'LNX2', 'LONP1', 'LONP2', 'LONRF1', 'LONRF2', 'LONRF3', 'LORICRIN', 'LOX', 'LOXHD1', 'LOXL1', 'LOXL2', 'LOXL3', 'LOXL4', 'LPA', 'LPAR1', 'LPAR2', 'LPAR3', 'LPAR4', 'LPAR5', 'LPAR6', 'LPCAT1', 'LPCAT2', 'LPCAT3', 'LPCAT4', 'LPGAT1', 'LPIN1', 'LPIN2', 'LPIN3', 'LPL', 'LPO', 'LPP', 'LPXN', 'LRAT', 'LRATD1', 'LRATD2', 'LRBA', 'LRCH1', 'LRCH2', 'LRCH3', 'LRCH4', 'LRCOL1', 'LRFN1', 'LRFN2', 'LRFN3', 'LRFN4', 'LRFN5', 'LRG1', 'LRGUK', 'LRIF1', 'LRIG1', 'LRIG2', 'LRIG3', 'LRIT1', 'LRIT2', 'LRIT3', 'LRMDA', 'LRP1', 'LRP1B', 'LRP2', 'LRP2BP', 'LRP3', 'LRP4', 'LRP5', 'LRP5L', 'LRP6', 'LRP8', 'LRP10', 'LRP11', 'LRP12', 'LRPAP1', 'LRPPRC', 'LRR1', 'LRRC1', 'LRRC2', 'LRRC3', 'LRRC3B', 'LRRC3C', 'LRRC4', 'LRRC4B', 'LRRC4C', 'LRRC7', 'LRRC8A', 'LRRC8B', 'LRRC8C', 'LRRC8D', 'LRRC8E', 'LRRC9', 'LRRC10', 'LRRC10B', 'LRRC14', 'LRRC14B', 'LRRC15', 'LRRC17', 'LRRC18', 'LRRC19', 'LRRC20', 'LRRC23', 'LRRC24', 'LRRC25', 'LRRC26', 'LRRC27', 'LRRC28', 'LRRC29', 'LRRC30', 'LRRC31', 'LRRC32', 'LRRC34', 'LRRC36', 'LRRC37A', 'LRRC37A2', 'LRRC37A3', 'LRRC37B', 'LRRC38', 'LRRC39', 'LRRC40', 'LRRC41', 'LRRC42', 'LRRC43', 'LRRC45', 'LRRC46', 'LRRC47', 'LRRC49', 'LRRC51', 'LRRC52', 'LRRC53', 'LRRC55', 'LRRC56', 'LRRC57', 'LRRC58', 'LRRC59', 'LRRC61', 'LRRC63', 'LRRC66', 'LRRC69', 'LRRC70', 'LRRC71', 'LRRC72', 'LRRC73', 'LRRC74A', 'LRRC74B', 'LRRC75A', 'LRRC75B', 'LRRCC1', 'LRRD1', 'LRRFIP1', 'LRRFIP2', 'LRRIQ1', 'LRRIQ3', 'LRRIQ4', 'LRRK1', 'LRRK2', 'LRRN1', 'LRRN2', 'LRRN3', 'LRRN4', 'LRRN4CL', 'LRRTM1', 'LRRTM2', 'LRRTM3', 'LRRTM4', 'LRSAM1', 'LRTM1', 'LRTM2', 'LRWD1', 'LSAMP', 'LSG1', 'LSM1', 'LSM2', 'LSM3', 'LSM4', 'LSM5', 'LSM6', 'LSM7', 'LSM8', 'LSM10', 'LSM11', 'LSM12', 'LSM14A', 'LSM14B', 'LSMEM1', 'LSMEM2', 'LSP1', 'LSR', 'LSS', 'LST1', 'LTA', 'LTA4H', 'LTB', 'LTB4R', 'LTB4R2', 'LTBP1', 'LTBP2', 'LTBP3', 'LTBP4', 'LTBR', 'LTC4S', 'LTF', 'LTK', 'LTN1', 'LTO1', 'LTV1', 'LUC7L', 'LUC7L2', 'LUC7L3', 'LUM', 'LURAP1', 'LURAP1L', 'LUZP1', 'LUZP2', 'LUZP4', 'LUZP6', 'LVRN', 'LXN', 'LY6D', 'LY6E', 'LY6G5B', 'LY6G5C', 'LY6G6C', 'LY6G6D', 'LY6G6F', 'LY6H', 'LY6K', 'LY6L', 'LY9', 'LY75', 'LY86', 'LY96', 'LYAR', 'LYG1', 'LYG2', 'LYL1', 'LYN', 'LYNX1', 'LYPD1', 'LYPD2', 'LYPD3', 'LYPD4', 'LYPD5', 'LYPD6', 'LYPD6B', 'LYPD8', 'LYPLA1', 'LYPLA2', 'LYPLAL1', 'LYRM1', 'LYRM2', 'LYRM4', 'LYRM7', 'LYRM9', 'LYSMD1', 'LYSMD2', 'LYSMD3', 'LYSMD4', 'LYST', 'LYVE1', 'LYZ', 'LYZL1', 'LYZL2', 'LYZL4', 'LYZL6', 'LZIC', 'LZTFL1', 'LZTR1', 'LZTS1', 'LZTS2', 'LZTS3', 'M1AP', 'M6PR', 'MAB21L1', 'MAB21L2', 'MAB21L3', 'MAB21L4', 'MACC1', 'MACF1', 'MACIR', 'MACO1', 'MACROD1', 'MACROD2', 'MACROH2A1', 'MACROH2A2', 'MAD1L1', 'MAD2L1', 'MAD2L1BP', 'MAD2L2', 'MADCAM1', 'MADD', 'MAEA', 'MAEL', 'MAF', 'MAF1', 'MAFA', 'MAFB', 'MAFF', 'MAFG', 'MAFK', 'MAG', 'MAGEA1', 'MAGEA2', 'MAGEA2B', 'MAGEA3', 'MAGEA4', 'MAGEA6', 'MAGEA8', 'MAGEA9', 'MAGEA9B', 'MAGEA10', 'MAGEA11', 'MAGEA12', 'MAGEB1', 'MAGEB2', 'MAGEB3', 'MAGEB4', 'MAGEB5', 'MAGEB6', 'MAGEB10', 'MAGEB16', 'MAGEB17', 'MAGEB18', 'MAGEC1', 'MAGEC2', 'MAGEC3', 'MAGED1', 'MAGED2', 'MAGED4', 'MAGED4B', 'MAGEE1', 'MAGEE2', 'MAGEF1', 'MAGEH1', 'MAGEL2', 'MAGI1', 'MAGI2', 'MAGI3', 'MAGIX', 'MAGOH', 'MAGOHB', 'MAGT1', 'MAIP1', 'MAJIN', 'MAK', 'MAK16', 'MAL', 'MAL2', 'MALL', 'MALRD1', 'MALSU1', 'MALT1', 'MAMDC2', 'MAMDC4', 'MAML1', 'MAML2', 'MAML3', 'MAMLD1', 'MAMSTR', 'MAN1A1', 'MAN1A2', 'MAN1B1', 'MAN1C1', 'MAN2A1', 'MAN2A2', 'MAN2B1', 'MAN2B2', 'MAN2C1', 'MANBA', 'MANBAL', 'MANEA', 'MANEAL', 'MANF', 'MANSC1', 'MANSC4', 'MAOA', 'MAOB', 'MAP1A', 'MAP1B', 'MAP1LC3A', 'MAP1LC3B', 'MAP1LC3B2', 'MAP1LC3C', 'MAP1S', 'MAP2', 'MAP2K1', 'MAP2K2', 'MAP2K3', 'MAP2K4', 'MAP2K5', 'MAP2K6', 'MAP2K7', 'MAP3K1', 'MAP3K2', 'MAP3K3', 'MAP3K4', 'MAP3K5', 'MAP3K6', 'MAP3K7', 'MAP3K7CL', 'MAP3K8', 'MAP3K9', 'MAP3K10', 'MAP3K11', 'MAP3K12', 'MAP3K13', 'MAP3K14', 'MAP3K15', 'MAP3K19', 'MAP3K20', 'MAP3K21', 'MAP4', 'MAP4K1', 'MAP4K2', 'MAP4K3', 'MAP4K4', 'MAP4K5', 'MAP6', 'MAP6D1', 'MAP7', 'MAP7D1', 'MAP7D2', 'MAP7D3', 'MAP9', 'MAP10', 'MAPK1', 'MAPK1IP1L', 'MAPK3', 'MAPK4', 'MAPK6', 'MAPK7', 'MAPK8', 'MAPK8IP1', 'MAPK8IP2', 'MAPK8IP3', 'MAPK9', 'MAPK10', 'MAPK11', 'MAPK12', 'MAPK13', 'MAPK14', 'MAPK15', 'MAPKAP1', 'MAPKAPK2', 'MAPKAPK3', 'MAPKAPK5', 'MAPKBP1', 'MAPRE1', 'MAPRE2', 'MAPRE3', 'MAPT', 'MARCHF1', 'MARCHF2', 'MARCHF3', 'MARCHF4', 'MARCHF5', 'MARCHF6', 'MARCHF7', 'MARCHF8', 'MARCHF9', 'MARCHF10', 'MARCHF11', 'MARCKS', 'MARCKSL1', 'MARCO', 'MARCOL', 'MARF1', 'MARK1', 'MARK2', 'MARK3', 'MARK4', 'MARS1', 'MARS2', 'MARVELD1', 'MARVELD2', 'MARVELD3', 'MAS1', 'MAS1L', 'MASP1', 'MASP2', 'MAST1', 'MAST2', 'MAST3', 'MAST4', 'MASTL', 'MAT1A', 'MAT2A', 'MAT2B', 'MATK', 'MATN1', 'MATN2', 'MATN3', 'MATN4', 'MATR3', 'MAU2', 'MAVS', 'MAX', 'MAZ', 'MB', 'MB21D2', 'MBD1', 'MBD2', 'MBD3', 'MBD3L1', 'MBD3L2', 'MBD3L2B', 'MBD3L3', 'MBD3L4', 'MBD3L5', 'MBD4', 'MBD5', 'MBD6', 'MBIP', 'MBL2', 'MBLAC1', 'MBLAC2', 'MBNL1', 'MBNL2', 'MBNL3', 'MBOAT1', 'MBOAT2', 'MBOAT4', 'MBOAT7', 'MBP', 'MBTD1', 'MBTPS1', 'MBTPS2', 'MC1R', 'MC2R', 'MC3R', 'MC4R', 'MC5R', 'MCAM', 'MCAT', 'MCC', 'MCCC1', 'MCCC2', 'MCCD1', 'MCEE', 'MCEMP1', 'MCF2', 'MCF2L', 'MCF2L2', 'MCFD2', 'MCHR1', 'MCHR2', 'MCIDAS', 'MCL1', 'MCM2', 'MCM3', 'MCM3AP', 'MCM4', 'MCM5', 'MCM6', 'MCM7', 'MCM8', 'MCM9', 'MCM10', 'MCMBP', 'MCMDC2', 'MCOLN1', 'MCOLN2', 'MCOLN3', 'MCPH1', 'MCRIP1', 'MCRIP2', 'MCRS1', 'MCTP1', 'MCTP2', 'MCTS1', 'MCU', 'MCUB', 'MCUR1', 'MDC1', 'MDFI', 'MDFIC', 'MDFIC2', 'MDGA1', 'MDGA2', 'MDH1', 'MDH1B', 'MDH2', 'MDK', 'MDM1', 'MDM2', 'MDM4', 'MDN1', 'MDP1', 'ME1', 'ME2', 'ME3', 'MEA1', 'MEAF6', 'MEAK7', 'MECOM', 'MECP2', 'MECR', 'MED1', 'MED4', 'MED6', 'MED7', 'MED8', 'MED9', 'MED10', 'MED11', 'MED12', 'MED12L', 'MED13', 'MED13L', 'MED14', 'MED14OS', 'MED15', 'MED16', 'MED17', 'MED18', 'MED19', 'MED20', 'MED21', 'MED22', 'MED23', 'MED24', 'MED25', 'MED26', 'MED27', 'MED28', 'MED29', 'MED30', 'MED31', 'MEDAG', 'MEF2A', 'MEF2B', 'MEF2C', 'MEF2D', 'MEFV', 'MEGF6', 'MEGF8', 'MEGF9', 'MEGF10', 'MEGF11', 'MEI1', 'MEI4', 'MEIG1', 'MEIKIN', 'MEIOB', 'MEIOC', 'MEIOSIN', 'MEIS1', 'MEIS2', 'MEIS3', 'MELK', 'MELTF', 'MEMO1', 'MEN1', 'MEOX1', 'MEOX2', 'MEP1A', 'MEP1B', 'MEPCE', 'MEPE', 'MERTK', 'MESD', 'MESP1', 'MESP2', 'MEST', 'MET', 'METAP1', 'METAP1D', 'METAP2', 'METRN', 'METRNL', 'METTL1', 'METTL2A', 'METTL2B', 'METTL3', 'METTL4', 'METTL5', 'METTL6', 'METTL7A', 'METTL7B', 'METTL8', 'METTL9', 'METTL13', 'METTL14', 'METTL15', 'METTL16', 'METTL17', 'METTL18', 'METTL21A', 'METTL21C', 'METTL22', 'METTL23', 'METTL24', 'METTL25', 'METTL25B', 'METTL26', 'METTL27', 'MEX3A', 'MEX3B', 'MEX3C', 'MEX3D', 'MFAP1', 'MFAP2', 'MFAP3', 'MFAP3L', 'MFAP4', 'MFAP5', 'MFF', 'MFGE8', 'MFHAS1', 'MFN1', 'MFN2', 'MFNG', 'MFRP', 'MFSD1', 'MFSD2A', 'MFSD2B', 'MFSD3', 'MFSD4A', 'MFSD4B', 'MFSD5', 'MFSD6', 'MFSD6L', 'MFSD8', 'MFSD9', 'MFSD10', 'MFSD11', 'MFSD12', 'MFSD13A', 'MFSD14A', 'MFSD14B', 'MFSD14C', 'MGA', 'MGAM', 'MGAM2', 'MGARP', 'MGAT1', 'MGAT2', 'MGAT3', 'MGAT4A', 'MGAT4B', 'MGAT4C', 'MGAT4D', 'MGAT5', 'MGAT5B', 'MGLL', 'MGME1', 'MGMT', 'MGP', 'MGRN1', 'MGST1', 'MGST2', 'MGST3', 'MIA', 'MIA2', 'MIA3', 'MIB1', 'MIB2', 'MICA', 'MICAL1', 'MICAL2', 'MICAL3', 'MICALCL', 'MICALL1', 'MICALL2', 'MICB', 'MICOS10', 'MICOS13', 'MICU1', 'MICU2', 'MICU3', 'MID1', 'MID1IP1', 'MID2', 'MIDEAS', 'MIDN', 'MIEF1', 'MIEF2', 'MIEN1', 'MIER1', 'MIER2', 'MIER3', 'MIF', 'MIF4GD', 'MIGA1', 'MIGA2', 'MIIP', 'MILR1', 'MINAR1', 'MINAR2', 'MINDY1', 'MINDY2', 'MINDY3', 'MINDY4', 'MINDY4B', 'MINK1', 'MINPP1', 'MIOS', 'MIOX', 'MIP', 'MIPEP', 'MIPOL1', 'MIS12', 'MIS18A', 'MIS18BP1', 'MISP', 'MISP3', 'MITD1', 'MITF', 'MIX23', 'MIXL1', 'MKI67', 'MKKS', 'MKLN1', 'MKNK1', 'MKNK2', 'MKRN1', 'MKRN2', 'MKRN2OS', 'MKRN3', 'MKS1', 'MKX', 'MLANA', 'MLC1', 'MLEC', 'MLF1', 'MLF2', 'MLH1', 'MLH3', 'MLIP', 'MLKL', 'MLLT1', 'MLLT3', 'MLLT6', 'MLLT10', 'MLLT11', 'MLN', 'MLNR', 'MLPH', 'MLST8', 'MLX', 'MLXIP', 'MLXIPL', 'MLYCD', 'MMAA', 'MMAB', 'MMACHC', 'MMADHC', 'MMD', 'MMD2', 'MME', 'MMEL1', 'MMGT1', 'MMP1', 'MMP2', 'MMP3', 'MMP7', 'MMP8', 'MMP9', 'MMP10', 'MMP11', 'MMP12', 'MMP13', 'MMP14', 'MMP15', 'MMP16', 'MMP17', 'MMP19', 'MMP20', 'MMP21', 'MMP23B', 'MMP24', 'MMP24OS', 'MMP25', 'MMP26', 'MMP27', 'MMP28', 'MMRN1', 'MMRN2', 'MMS19', 'MMS22L', 'MMUT', 'MN1', 'MNAT1', 'MND1', 'MNDA', 'MNS1', 'MNT', 'MNX1', 'MOAP1', 'MOB1A', 'MOB1B', 'MOB2', 'MOB3A', 'MOB3B', 'MOB3C', 'MOB4', 'MOBP', 'MOCOS', 'MOCS1', 'MOCS2', 'MOCS3', 'MOG', 'MOGAT1', 'MOGAT2', 'MOGAT3', 'MOGS', 'MOK', 'MON1A', 'MON1B', 'MON2', 'MORC1', 'MORC2', 'MORC3', 'MORC4', 'MORF4L1', 'MORF4L2', 'MORN1', 'MORN2', 'MORN3', 'MORN4', 'MORN5', 'MOS', 'MOSMO', 'MOSPD1', 'MOSPD2', 'MOSPD3', 'MOV10', 'MOV10L1', 'MOXD1', 'MPC1', 'MPC1L', 'MPC2', 'MPDU1', 'MPDZ', 'MPEG1', 'MPG', 'MPHOSPH6', 'MPHOSPH8', 'MPHOSPH9', 'MPHOSPH10', 'MPI', 'MPIG6B', 'MPL', 'MPLKIP', 'MPND', 'MPO', 'MPP1', 'MPP2', 'MPP3', 'MPP4', 'MPP7', 'MPPE1', 'MPPED1', 'MPPED2', 'MPRIP', 'MPST', 'MPV17', 'MPV17L', 'MPV17L2', 'MPZ', 'MPZL1', 'MPZL2', 'MPZL3', 'MR1', 'MRAP', 'MRAP2', 'MRAS', 'MRC1', 'MRC2', 'MRE11', 'MREG', 'MRFAP1', 'MRFAP1L1', 'MRGBP', 'MRGPRD', 'MRGPRE', 'MRGPRF', 'MRGPRG', 'MRGPRX1', 'MRGPRX2', 'MRGPRX3', 'MRGPRX4', 'MRI1', 'MRLN', 'MRM1', 'MRM2', 'MRM3', 'MRNIP', 'MRO', 'MROH1', 'MROH2A', 'MROH2B', 'MROH5', 'MROH6', 'MROH7', 'MROH8', 'MROH9', 'MRPL1', 'MRPL2', 'MRPL3', 'MRPL4', 'MRPL9', 'MRPL10', 'MRPL11', 'MRPL12', 'MRPL13', 'MRPL14', 'MRPL15', 'MRPL16', 'MRPL17', 'MRPL18', 'MRPL19', 'MRPL20', 'MRPL21', 'MRPL22', 'MRPL23', 'MRPL24', 'MRPL27', 'MRPL28', 'MRPL30', 'MRPL32', 'MRPL33', 'MRPL34', 'MRPL35', 'MRPL36', 'MRPL37', 'MRPL38', 'MRPL39', 'MRPL40', 'MRPL41', 'MRPL42', 'MRPL43', 'MRPL44', 'MRPL45', 'MRPL46', 'MRPL47', 'MRPL48', 'MRPL49', 'MRPL50', 'MRPL51', 'MRPL52', 'MRPL53', 'MRPL54', 'MRPL55', 'MRPL57', 'MRPL58', 'MRPS2', 'MRPS5', 'MRPS6', 'MRPS7', 'MRPS9', 'MRPS10', 'MRPS11', 'MRPS12', 'MRPS14', 'MRPS15', 'MRPS16', 'MRPS17', 'MRPS18A', 'MRPS18B', 'MRPS18C', 'MRPS21', 'MRPS22', 'MRPS23', 'MRPS24', 'MRPS25', 'MRPS26', 'MRPS27', 'MRPS28', 'MRPS30', 'MRPS31', 'MRPS33', 'MRPS34', 'MRPS35', 'MRPS36', 'MRRF', 'MRS2', 'MRTFA', 'MRTFB', 'MRTO4', 'MS4A1', 'MS4A2', 'MS4A3', 'MS4A4A', 'MS4A4E', 'MS4A5', 'MS4A6A', 'MS4A6E', 'MS4A7', 'MS4A8', 'MS4A10', 'MS4A12', 'MS4A13', 'MS4A14', 'MS4A15', 'MS4A18', 'MSANTD1', 'MSANTD2', 'MSANTD3', 'MSANTD4', 'MSANTD5', 'MSC', 'MSGN1', 'MSH2', 'MSH3', 'MSH4', 'MSH5', 'MSH6', 'MSI1', 'MSI2', 'MSL1', 'MSL2', 'MSL3', 'MSLN', 'MSMB', 'MSMO1', 'MSMP', 'MSN', 'MSR1', 'MSRA', 'MSRB1', 'MSRB2', 'MSRB3', 'MSS51', 'MST1', 'MST1R', 'MSTN', 'MSTO1', 'MSX1', 'MSX2', 'MT-ATP6', 'MT-ATP8', 'MT-CO1', 'MT-CO2', 'MT-CO3', 'MT-CYB', 'MT-ND1', 'MT-ND2', 'MT-ND3', 'MT-ND4', 'MT-ND4L', 'MT-ND5', 'MT-ND6', 'MT1A', 'MT1B', 'MT1E', 'MT1F', 'MT1G', 'MT1H', 'MT1HL1', 'MT1M', 'MT1X', 'MT2A', 'MT3', 'MT4', 'MTA1', 'MTA2', 'MTA3', 'MTAP', 'MTARC1', 'MTARC2', 'MTBP', 'MTCH1', 'MTCH2', 'MTCL1', 'MTCP1', 'MTDH', 'MTERF1', 'MTERF2', 'MTERF3', 'MTERF4', 'MTF1', 'MTF2', 'MTFMT', 'MTFP1', 'MTFR1', 'MTFR1L', 'MTFR2', 'MTG1', 'MTG2', 'MTHFD1', 'MTHFD1L', 'MTHFD2', 'MTHFD2L', 'MTHFR', 'MTHFS', 'MTHFSD', 'MTIF2', 'MTIF3', 'MTLN', 'MTM1', 'MTMR1', 'MTMR2', 'MTMR3', 'MTMR4']

owo = owo + joesalisbury
uwu = uwu + rajeevram

df = pd.DataFrame(index = uwu, data = owo)
writer = pd.ExcelWriter('Gene_Aliases_5k_to_10k.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='Aliases', index=True)
writer.save()


df = pd.DataFrame(index = uwu, data = umu)
writer = pd.ExcelWriter('Gene_Conditions_5k_to_10k.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='Conditions', index=True)
writer.save()
